In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression

def process_spiral_arms_polar_intermediate(
    id_halo,
    quartile_threshold=0.55,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    eps_polar=0.7,
    min_samples_polar=18,
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,  # Para detectar un gap y dividir el grupo en 2, si es muy grande
    do_subdivide=True    # Activar o desactivar la subdivisión
):
    """
    Versión "intermedia" que:
      1. Aplica DBSCAN en (r, theta).
      2. Fusiona clusters cercanos.
      3. Ajuste lineal iterativo (bondadoso), pero controlando la variación de la pendiente (20%).
      4. Reasigna descartados.
      5. (Opcional) Subdivide un grupo en 2 si detecta un gap grande en theta.
      6. Filtra por PA al final.

    Parámetros:
      - slope_variation_threshold: variación relativa de pendiente máxima permitida (ej: 0.20 para 20%).
      - max_theta_gap: umbral para dividir un grupo en 2 segmentos si hay un salto grande en theta.
      - do_subdivide: si es True, se subdivide al final si se detecta un gap grande.

    Retorna:
      - valid_groups: Lista de grupos (diccionarios) con sus parámetros ajustados.
      - df: DataFrame original con las columnas añadidas (r, θ, cluster).
    """
    # 1. Cargar datos
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')

    # 2. Convertir a coordenadas polares
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])

    # 3. Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)

    # 4. DBSCAN en el plano (r, θ)
    X_polar = df_filtered[['r', 'theta']].values
    dbscan = DBSCAN(eps=eps_polar, min_samples=min_samples_polar)
    df_filtered['cluster'] = dbscan.fit_predict(X_polar)

    # 5. Fusión de clusters cercanos
    clusters_tolerance = 0.70
    def merge_close_clusters(clusters, clusters_tolerance):
        merged_clusters = {}
        used_clusters = set()
        for id1, points1 in clusters.items():
            if id1 in used_clusters:
                continue
            merged_points = points1.copy()
            for id2, points2 in clusters.items():
                if id1 >= id2 or id2 in used_clusters:
                    continue
                min_distance = np.min(np.sqrt(
                    (points1['r'].values[:, None] - points2['r'].values)**2 +
                    (points1['theta'].values[:, None] - points2['theta'].values)**2
                ))
                if min_distance < clusters_tolerance:
                    merged_points = pd.concat([merged_points, points2], ignore_index=True)
                    used_clusters.add(id2)
            merged_clusters[id1] = merged_points
            used_clusters.add(id1)
        return merged_clusters

    # Crear diccionario de clusters
    clusters_polar = {}
    for cluster_id in np.unique(df_filtered['cluster']):
        if cluster_id == -1:
            continue
        cluster_points = df_filtered[df_filtered['cluster'] == cluster_id].copy()
        clusters_polar[cluster_id] = cluster_points[['r', 'theta']]

    clusters_polar = merge_close_clusters(clusters_polar, clusters_tolerance)

    # 6. Inicializar grupos
    model = LinearRegression()

    def calculate_distance(slope, intercept, theta, r):
        pred_r = slope * theta + intercept
        return abs(pred_r - r)

    def calculate_pa(slope, intercept):
        if intercept != 0:
            return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
        return np.nan

    def is_valid_pa(pa, valid_range):
        return valid_range[0] <= abs(pa) <= valid_range[1]

    groups = []
    for cid, pts in clusters_polar.items():
        if len(pts) >= 3:
            X_ = pts[['theta']].values
            y_ = pts['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None

        groups.append({
            'cluster_id': cid,
            'points': pts.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })

    # 7. Asignación iterativa de puntos (incluye control de pendiente)
    df_local = df_filtered.copy()
    discarded_points = []

    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        assigned = False

        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue

            dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
            if dist <= radius_tolerance:
                # Verificar distancia a los puntos
                distances = g['points'].apply(
                    lambda row: calculate_distance(g['slope'], g['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if all(d <= point_distance_tolerance for d in distances):
                    # Recalcular ajuste
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']

                    # Controlar variación relativa de la pendiente
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= slope_variation_threshold:
                        g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(g['slope'], g['intercept'])
                        assigned = True
                        break

        if not assigned:
            discarded_points.append(point.to_dict())

    # 8. Reasignar descartados
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            theta_pt = p['theta']
            r_pt = p['r']
            assigned = False

            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
                if dist <= radius_tolerance:
                    # Verificar PA
                    if g['pa'] is not None and is_valid_pa(g['pa'], flexible_pa_range):
                        X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                        y_new = np.hstack([g['points']['r'].values, r_pt])
                        model.fit(X_new, y_new)
                        new_slope = model.coef_[0]
                        old_slope = g['slope']
                        if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= slope_variation_threshold:
                            g['points'] = pd.concat([g['points'], p.to_frame().T], ignore_index=True)
                            g['slope'] = new_slope
                            g['intercept'] = model.intercept_
                            g['pa'] = calculate_pa(new_slope, g['intercept'])
                            assigned = True
                            break
            if not assigned:
                # Crear grupo nuevo
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new, i_new)
                if is_valid_pa(pa_new, flexible_pa_range):
                    groups.append({
                        'cluster_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })

    # 9. Subdivisión en 2 segmentos si hay un gap muy grande (opcional)
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_

    def subdivide_if_gap(group, max_gap=max_theta_gap):
        """
        Subdivide en 2 si encuentra un gap > max_gap en theta.
        """
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]

        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            # No hay gap grande
            return [group]
        # Tomar el primer gap para dividir
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]

        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)

        seg1 = {
            'cluster_id': group['cluster_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'cluster_id': group['cluster_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]

    final_groups = []
    if do_subdivide:
        # Subdividir cada grupo en 2 si hay gap
        for g in groups:
            segs = subdivide_if_gap(g, max_gap=max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups

    # 10. Filtrar por PA
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and is_valid_pa(pa_val, flexible_pa_range):
                valid_groups.append(grp)

    return valid_groups, df

###############################################################################
# EJEMPLO DE EJECUCIÓN Y GRAFICACIÓN
###############################################################################

if __name__ == "__main__":
    # Parámetros
    id_halo = "17"
    quartile_threshold = 0.155
    radius_tolerance = 0.45
    point_distance_tolerance = 0.59
    flexible_pa_range = (4.0, 30.0)
    eps_polar = 0.7
    min_samples_polar = 18
    slope_variation_threshold = 0.20  # 20%
    max_theta_gap = 15.0
    do_subdivide = True

    valid_groups, df = process_spiral_arms_polar_intermediate(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        radius_tolerance=radius_tolerance,
        point_distance_tolerance=point_distance_tolerance,
        flexible_pa_range=flexible_pa_range,
        eps_polar=eps_polar,
        min_samples_polar=min_samples_polar,
        slope_variation_threshold=slope_variation_threshold,
        max_theta_gap=max_theta_gap,
        do_subdivide=do_subdivide
    )

    print("Total de grupos finales:", len(valid_groups))

    # Gráfica en (θ vs r)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['theta'], df['r'], s=1, color='silver', alpha=0.5, label='Datos Originales')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']

    for idx, grp in enumerate(valid_groups):
        pts = grp['points']
        theta_vals = pts['theta']
        r_vals = pts['r']
        pa_val = grp['pa'] if grp['pa'] is not None else 0.0
        color = colors[idx % len(colors)]
        plt.scatter(theta_vals, r_vals, s=10, alpha=0.8, color=color,
                    label=f"Grupo {idx+1}, PA={pa_val:.2f}")
        if grp['slope'] is not None:
            theta_line = np.linspace(theta_vals.min(), theta_vals.max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)

    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Ajustes intermedios en (r, θ) - Halo {id_halo}")
    #plt.legend(loc='best')
    plt.grid(True)
    plt.show()

    # Gráfica polar
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 8))
    theta_all_rad = np.radians(df['theta'])
    ax.scatter(theta_all_rad, df['r'], s=1, color='gray', alpha=0.5, label='Datos Originales')

    for idx, grp in enumerate(valid_groups):
        pts = grp['points']
        theta_rad = np.radians(pts['theta'])
        r_vals = pts['r']
        pa_val = grp['pa'] if grp['pa'] is not None else 0.0
        color = colors[idx % len(colors)]
        ax.scatter(theta_rad, r_vals, s=10, alpha=0.8, color=color,
                   label=f"Grupo {idx+1} PA: {pa_val:.2f}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)

    ax.set_title(f"Ajustes intermedios en proyección polar - Halo {id_halo}", fontsize=14, pad=20)
    #ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0), fontsize=9)
    plt.show()



In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression
import copy
from ipywidgets import interact, IntSlider

def process_spiral_arms_interactive(
    id_halo,
    quartile_threshold=0.55,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    eps_polar=0.7,
    min_samples_polar=18,
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10  # cada cuántas iteraciones se guarda un estado
):
    """
    Procesa los datos y retorna una lista de estados intermedios para visualización interactiva.
    Solo se consideran los puntos con theta entre 50 y 250 grados.
    
    Retorna:
       - states: lista de estados intermedios (cada estado es un diccionario con grupos, descartados, y puntos restantes)
       - final_groups: grupos finales después de todo el proceso.
       - df: DataFrame original filtrado.
    """
    # 1. Cargar datos
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    
    # 2. Convertir a coordenadas polares
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    
    # Filtrar theta entre 50 y 250
    df = df[(df['theta'] >= 50) & (df['theta'] <= 250)].copy()
    
    # 3. Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 4. DBSCAN en el plano (r, θ)
    X_polar = df_filtered[['r', 'theta']].values
    dbscan = DBSCAN(eps=eps_polar, min_samples=min_samples_polar)
    df_filtered['cluster'] = dbscan.fit_predict(X_polar)
    
    # 5. Fusión de clusters cercanos
    clusters_tolerance = 0.70
    def merge_close_clusters(clusters, clusters_tolerance):
        merged_clusters = {}
        used_clusters = set()
        for id1, points1 in clusters.items():
            if id1 in used_clusters:
                continue
            merged_points = points1.copy()
            for id2, points2 in clusters.items():
                if id1 >= id2 or id2 in used_clusters:
                    continue
                min_distance = np.min(np.sqrt(
                    (points1['r'].values[:, None] - points2['r'].values)**2 +
                    (points1['theta'].values[:, None] - points2['theta'].values)**2
                ))
                if min_distance < clusters_tolerance:
                    merged_points = pd.concat([merged_points, points2], ignore_index=True)
                    used_clusters.add(id2)
            merged_clusters[id1] = merged_points
            used_clusters.add(id1)
        return merged_clusters
    
    clusters_polar = {}
    for cluster_id in np.unique(df_filtered['cluster']):
        if cluster_id == -1:
            continue
        cluster_points = df_filtered[df_filtered['cluster'] == cluster_id].copy()
        clusters_polar[cluster_id] = cluster_points[['r', 'theta']]
    
    clusters_polar = merge_close_clusters(clusters_polar, clusters_tolerance)
    
    # 6. Inicializar grupos con ajuste lineal (por cada cluster)
    model = LinearRegression()
    
    def calculate_distance(slope, intercept, theta, r):
        pred_r = slope * theta + intercept
        return abs(pred_r - r)
    
    def calculate_pa(slope, intercept):
        if intercept != 0:
            return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
        return np.nan
    
    def is_valid_pa(pa, valid_range):
        return valid_range[0] <= abs(pa) <= valid_range[1]
    
    groups = []
    for cid, pts in clusters_polar.items():
        if len(pts) >= 3:
            X_ = pts[['theta']].values
            y_ = pts['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'cluster_id': cid,
            'points': pts.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # 7. Asignación iterativa de puntos, guardando estados intermedios
    df_local = df_filtered.copy()
    discarded_points = []
    states = []  # se guardarán estados intermedios
    iteration = 0
    
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        assigned = False
        
        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue
            dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
            if dist <= radius_tolerance:
                distances = g['points'].apply(
                    lambda row: calculate_distance(g['slope'], g['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if all(d <= point_distance_tolerance for d in distances):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= slope_variation_threshold:
                        g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(g['slope'], g['intercept'])
                        assigned = True
                        break
        if not assigned:
            discarded_points.append(point.to_dict())
        
        iteration += 1
        # Guardar el estado cada 'snapshot_interval' iteraciones o al finalizar
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            groups_copy = copy.deepcopy(groups)
            discarded_copy = copy.deepcopy(discarded_points)
            states.append({
                'iteration': iteration,
                'groups': groups_copy,
                'discarded': discarded_copy,
                'remaining': df_local.copy()
            })
    
    # 8. Reasignar descartados (etapa final, no interactiva)
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            theta_pt = p['theta']
            r_pt = p['r']
            assigned = False
            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
                if dist <= radius_tolerance:
                    if g['pa'] is not None and is_valid_pa(g['pa'], flexible_pa_range):
                        X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                        y_new = np.hstack([g['points']['r'].values, r_pt])
                        model.fit(X_new, y_new)
                        new_slope = model.coef_[0]
                        old_slope = g['slope']
                        if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= slope_variation_threshold:
                            g['points'] = pd.concat([g['points'], p.to_frame().T], ignore_index=True)
                            g['slope'] = new_slope
                            g['intercept'] = model.intercept_
                            g['pa'] = calculate_pa(new_slope, g['intercept'])
                            assigned = True
                            break
            if not assigned:
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new, i_new)
                if is_valid_pa(pa_new, flexible_pa_range):
                    groups.append({
                        'cluster_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })
    
    # 9. (Opcional) Subdividir grupos si hay un gap grande en theta
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    def subdivide_if_gap(group, max_gap=max_theta_gap):
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)
        seg1 = {
            'cluster_id': group['cluster_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'cluster_id': group['cluster_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]
    
    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap=max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups
    
    # 10. Filtrar grupos por PA válido
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and is_valid_pa(pa_val, flexible_pa_range):
                valid_groups.append(grp)
    
    # Agregar estado final
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'discarded': copy.deepcopy(discarded_points),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered

def plot_intermediate_state(state, df_all):
    """
    Grafica el estado intermedio en el plano polar:
      - Fondo: todos los datos en gris pálido.
      - Grupos asignados: cada grupo con un color y su línea de ajuste (si se ha calculado).
      - Puntos descartados: marcados con una 'x' negra.
    """
    fig, ax = plt.subplots(subplot_kw={}, figsize=(10, 8))
    
    # Fondo: todos los datos
    theta_all = np.radians(df_all['theta'])
    r_all = df_all['r']
    ax.scatter(theta_all, r_all, s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    
    # Graficar cada grupo
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        theta_vals = np.radians(pts['theta'])
        r_vals = pts['r']
        color = colors[idx % len(colors)]
        ax.scatter(theta_vals, r_vals, s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)
    
    # Graficar puntos descartados
    if len(state['discarded']) > 0:
        df_desc = pd.DataFrame(state['discarded'])
        theta_desc = np.radians(df_desc['theta'])
        r_desc = df_desc['r']
        ax.scatter(theta_desc, r_desc, s=10, color='black', marker='x', label='Descartados')
    
    ax.set_title(f"Estado en iteración {state['iteration']}")
    ax.legend(loc='upper right')
    plt.show()



In [0]:


# Ejemplo de ejecución interactiva:
states, final_groups, df_filtered = process_spiral_arms_interactive(
    id_halo="17",
    quartile_threshold=0.155,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    eps_polar=0.7,
    min_samples_polar=18,
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10
)

print("Total de estados guardados:", len(states))
print("Total de grupos finales:", len(final_groups))

# Widget interactivo para ver el proceso iterativo
def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0));


### edición código - JUANK

In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression
import copy
from ipywidgets import interact, IntSlider

# Función auxiliar para calcular distancia respecto a una línea (ajuste lineal)
def calculate_distance(slope, intercept, theta, r):
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

# Función auxiliar para calcular PA
def calculate_pa(slope, intercept):
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan

# Función para agregar un punto a un grupo temporal (si coincide en θ y en r)
def add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window):
    assigned = False
    for tg in temp_groups:
        tg_theta_min = tg['points']['theta'].min()
        tg_theta_max = tg['points']['theta'].max()
        tg_r_min = tg['points']['r'].min()
        tg_r_max = tg['points']['r'].max()
        if ((point['theta'] >= tg_theta_min - theta_fit_window/2) and 
            (point['theta'] <= tg_theta_max + theta_fit_window/2)) and \
           ((point['r'] >= tg_r_min - r_fit_window/2) and 
            (point['r'] <= tg_r_max + r_fit_window/2)):
            tg['points'] = pd.concat([tg['points'], point.to_frame().T], ignore_index=True)
            assigned = True
            break
    if not assigned:
        new_tg = {'points': point.to_frame().T.copy()}
        temp_groups.append(new_tg)

# Función para revisar y fusionar (o promover) los grupos temporales
def review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range):
    new_temp_groups = []
    for tg in temp_groups:
        if len(tg['points']) < 3:
            new_temp_groups.append(tg)
            continue
        X_tg = tg['points'][['theta']]
        y_tg = tg['points']['r']
        model.fit(X_tg, y_tg)
        tg_slope = model.coef_[0]
        tg_intercept = model.intercept_
        tg_mean = tg['points']['theta'].mean()
        merged = False
        for mg in groups:
            mg_min = mg['points']['theta'].min()
            mg_max = mg['points']['theta'].max()
            if mg_min <= tg_mean <= mg_max:
                dist_values = tg['points'].apply(lambda row: calculate_distance(mg['slope'], mg['intercept'], row['theta'], row['r']), axis=1)
                if dist_values.mean() < radius_tolerance:
                    mg['points'] = pd.concat([mg['points'], tg['points']], ignore_index=True)
                    model.fit(mg['points'][['theta']], mg['points']['r'])
                    new_slope = model.coef_[0]
                    if abs(new_slope - mg['slope'])/(abs(mg['slope'])+1e-12) < slope_variation_threshold:
                        mg['slope'] = new_slope
                        mg['intercept'] = model.intercept_
                        mg['pa'] = calculate_pa(mg['slope'], mg['intercept'])
                    merged = True
                    break
        if not merged:
            new_group = {
                'cluster_id': None,
                'points': tg['points'],
                'slope': tg_slope,
                'intercept': tg_intercept,
                'pa': calculate_pa(tg_slope, tg_intercept)
            }
            groups.append(new_group)
    temp_groups.clear()
    for tg in new_temp_groups:
        temp_groups.append(tg)

def process_spiral_arms_interactive(
    id_halo,
    quartile_threshold=0.55,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    eps_polar=0.7,
    min_samples_polar=18,
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10,     # Cada cuántas iteraciones se guarda un estado
    theta_fit_window=30.0,    # Ventana en grados para ajustar la recta localmente
    r_fit_window=10.0,        # Ventana fija para r
    fusion_theta_threshold=7.0,  # Diferencia máxima fija en θ para fusionar
    fusion_r_threshold=7.0       # Diferencia máxima fija en r para fusionar
):
    """
    Procesa los datos y retorna una lista de estados intermedios para visualización interactiva.
    Se filtran los puntos con θ entre 50 y 250 grados.
    
    La fusión de grupos se realiza utilizando condiciones fijas:
      - Se fusionan si la diferencia entre el límite inferior de un grupo y el límite superior de otro 
        (y viceversa) es menor que fusion_theta_threshold, y
      - la diferencia en los límites de r es menor que fusion_r_threshold.
    """
    # 1. Cargar datos
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    
    # 2. Convertir a coordenadas polares
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= 50) & (df['theta'] <= 250)].copy()
    
    # 3. Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 4. DBSCAN en el plano (r, θ)
    X_polar = df_filtered[['r', 'theta']].values
    dbscan = DBSCAN(eps=eps_polar, min_samples=min_samples_polar)
    df_filtered['cluster'] = dbscan.fit_predict(X_polar)
    
    # 5. Fusión de clusters cercanos
    clusters_tolerance = 0.70
    def merge_close_clusters(clusters, clusters_tolerance):
        merged_clusters = {}
        used_clusters = set()
        for id1, points1 in clusters.items():
            if id1 in used_clusters:
                continue
            merged_points = points1.copy()
            for id2, points2 in clusters.items():
                if id1 >= id2 or id2 in used_clusters:
                    continue
                min_distance = np.min(np.sqrt(
                    (points1['r'].values[:, None] - points2['r'].values)**2 +
                    (points1['theta'].values[:, None] - points2['theta'].values)**2
                ))
                if min_distance < clusters_tolerance:
                    merged_points = pd.concat([merged_points, points2], ignore_index=True)
                    used_clusters.add(id2)
            merged_clusters[id1] = merged_points
            used_clusters.add(id1)
        return merged_clusters
    
    clusters_polar = {}
    for cluster_id in np.unique(df_filtered['cluster']):
        if cluster_id == -1:
            continue
        cluster_points = df_filtered[df_filtered['cluster'] == cluster_id].copy()
        clusters_polar[cluster_id] = cluster_points[['r', 'theta']]
    
    clusters_polar = merge_close_clusters(clusters_polar, clusters_tolerance)
    
    # 6. Inicializar grupos principales a partir de los clusters obtenidos
    model = LinearRegression()
    groups = []
    for cid, pts in clusters_polar.items():
        if len(pts) >= 3:
            X_ = pts[['theta']].values
            y_ = pts['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'cluster_id': cid,
            'points': pts.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # Preparar estados y grupos temporales
    df_local = df_filtered.copy()
    df_local.sort_values(by='theta', inplace=True)
    states = []
    temp_groups = []
    iteration = 0
    temp_review_interval = snapshot_interval * 15
    
    # 7. Asignación iterativa de puntos
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        best_group = None
        best_distance = None
        
        # Buscar grupo candidato que contenga el punto
        for g in groups:
            g_theta_min = g['points']['theta'].min()
            g_theta_max = g['points']['theta'].max()
            g_r_min = g['points']['r'].min()
            g_r_max = g['points']['r'].max()
            if not (g_theta_min <= theta_pt <= g_theta_max and g_r_min <= r_pt <= g_r_max):
                continue
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = g['points'][(g['points']['theta'] >= window_min) & (g['points']['theta'] <= window_max)]
            if len(subset) == 0:
                continue
            r_min = subset['r'].min()
            r_max = subset['r'].max()
            if not (r_min <= r_pt <= r_max):
                continue
            subset = pd.concat([subset, point.to_frame().T], ignore_index=True)
            if len(subset) < 2:
                continue
            model.fit(subset[['theta']], subset['r'])
            new_slope = model.coef_[0]
            new_intercept = model.intercept_
            old_slope = g['slope']
            if old_slope is None:
                continue
            if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) < 0.10:
                continue
            dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
            if dist > radius_tolerance:
                continue
            if best_distance is None or dist < best_distance:
                best_distance = dist
                best_group = g
        
        if best_group is not None:
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = best_group['points'][(best_group['points']['theta'] >= window_min) & (best_group['points']['theta'] <= window_max)]
            if len(subset) < 2:
                subset = best_group['points']
            model.fit(subset[['theta']], subset['r'])
            best_group['slope'] = model.coef_[0]
            best_group['intercept'] = model.intercept_
            best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
            
            # Fusión fija con grupos anteriores: se fusionan solo si las diferencias en ambos límites son menores que los umbrales fijos.
            merged_flag = True
            while merged_flag:
                merged_flag = False
                for other_group in groups:
                    if other_group is best_group:
                        continue
                    best_theta_min = best_group['points']['theta'].min()
                    best_theta_max = best_group['points']['theta'].max()
                    best_r_min = best_group['points']['r'].min()
                    best_r_max = best_group['points']['r'].max()
                    
                    other_theta_min = other_group['points']['theta'].min()
                    other_theta_max = other_group['points']['theta'].max()
                    other_r_min = other_group['points']['r'].min()
                    other_r_max = other_group['points']['r'].max()
                    
                    if (abs(best_theta_min - other_theta_max) < fusion_theta_threshold and
                        abs(best_theta_max - other_theta_min) < fusion_theta_threshold and
                        abs(best_r_min - other_r_max) < fusion_r_threshold and
                        abs(best_r_max - other_r_min) < fusion_r_threshold):
                        merged_points = pd.concat([best_group['points'], other_group['points']], ignore_index=True)
                        model.fit(merged_points[['theta']], merged_points['r'])
                        best_group['points'] = merged_points
                        best_group['slope'] = model.coef_[0]
                        best_group['intercept'] = model.intercept_
                        best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
                        # Eliminar other_group por identidad
                        for idx, grp in enumerate(groups):
                            if grp is other_group:
                                groups.pop(idx)
                                break
                        merged_flag = True
                        break
                        
        else:
            # Si no hay grupo candidato, intentar fusionar con el último grupo principal sin filtro de pendiente
            merged_with_last = False
            if len(groups) > 0:
                last_g = groups[-1]
                last_theta_min = last_g['points']['theta'].min()
                last_theta_max = last_g['points']['theta'].max()
                last_r_min = last_g['points']['r'].min()
                last_r_max = last_g['points']['r'].max()
                if (last_theta_min <= theta_pt <= last_theta_max and last_r_min <= r_pt <= last_r_max):
                    window_min = theta_pt - theta_fit_window/2
                    window_max = theta_pt + theta_fit_window/2
                    subset_last = last_g['points'][(last_g['points']['theta'] >= window_min) & (last_g['points']['theta'] <= window_max)]
                    if len(subset_last) > 0:
                        r_min_l = subset_last['r'].min()
                        r_max_l = subset_last['r'].max()
                        if r_min_l <= r_pt <= r_max_l:
                            subset_last = pd.concat([subset_last, point.to_frame().T], ignore_index=True)
                            model.fit(subset_last[['theta']], subset_last['r'])
                            new_slope_l = model.coef_[0]
                            new_intercept_l = model.intercept_
                            dist_l = calculate_distance(new_slope_l, new_intercept_l, theta_pt, r_pt)
                            if dist_l < radius_tolerance:
                                last_g['points'] = pd.concat([last_g['points'], point.to_frame().T], ignore_index=True)
                                model.fit(last_g['points'][['theta']], last_g['points']['r'])
                                last_g['slope'] = model.coef_[0]
                                last_g['intercept'] = model.intercept_
                                last_g['pa'] = calculate_pa(last_g['slope'], last_g['intercept'])
                                merged_with_last = True
            if not merged_with_last:
                add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window)
        
        iteration += 1
        
        if iteration % temp_review_interval == 0:
            review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range)
        
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_groups': copy.deepcopy(temp_groups),
                'remaining': df_local.copy()
            })
    
    if len(temp_groups) > 0:
        review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range)
    
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    def subdivide_if_gap(group, max_gap=max_theta_gap):
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)
        seg1 = {
            'cluster_id': group['cluster_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'cluster_id': group['cluster_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]
    
    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap=max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups
    
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'temp_groups': copy.deepcopy(temp_groups),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered

def plot_intermediate_state_cartesian(state, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    if state['temp_groups']:
        for idx, tg in enumerate(state['temp_groups']):
            pts = tg['points']
            plt.scatter(pts['theta'], pts['r'], s=10, color='black', marker='x', label=f"Temp {idx+1}")
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()



In [0]:
# Ejecución interactiva:
states, final_groups, df_filtered = process_spiral_arms_interactive(
    id_halo="17",
    quartile_threshold=0.155,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    eps_polar=0.7,
    min_samples_polar=18,
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10,
    theta_fit_window=25.10,
    r_fit_window=2.50 
)

print("Total de estados guardados:", len(states))
print("Total de grupos finales:", len(final_groups))

def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state_cartesian(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0));

In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression
import copy
from ipywidgets import interact, IntSlider

# ---------------------------
# Funciones auxiliares
# ---------------------------
def calculate_distance(slope, intercept, theta, r):
    """Calcula la diferencia absoluta entre r observado y r predicho por la línea."""
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    """Calcula el ángulo de inclinación (PA) en grados, a partir de la pendiente e intercepto."""
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan

def update_group_fit(group, model):
    """Recalcula el ajuste lineal (pendiente, intercepto y PA) para los puntos de un grupo."""
    X = group['points'][['theta']]
    y = group['points']['r']
    if len(group['points']) < 2:
        return
    model.fit(X, y)
    group['slope'] = model.coef_[0]
    group['intercept'] = model.intercept_
    group['pa'] = calculate_pa(group['slope'], group['intercept'])

def merge_groups_if_close(groups, fusion_theta_threshold, fusion_r_threshold, model):
    """
    Fusiona grupos si la diferencia entre los límites de sus cajas (bounding boxes)
    en θ y en r es menor a los umbrales fijos.
    """
    merged = True
    while merged:
        merged = False
        for i in range(len(groups)):
            for j in range(i + 1, len(groups)):
                g1 = groups[i]
                g2 = groups[j]
                # Obtener límites de g1
                g1_theta_min = g1['points']['theta'].min()
                g1_theta_max = g1['points']['theta'].max()
                g1_r_min = g1['points']['r'].min()
                g1_r_max = g1['points']['r'].max()
                # Límites de g2
                g2_theta_min = g2['points']['theta'].min()
                g2_theta_max = g2['points']['theta'].max()
                g2_r_min = g2['points']['r'].min()
                g2_r_max = g2['points']['r'].max()
                # Verificar diferencias fijas en los límites:
                theta_diff_lower = abs(g1_theta_min - g2_theta_max)
                theta_diff_upper = abs(g1_theta_max - g2_theta_min)
                r_diff_lower = abs(g1_r_min - g2_r_max)
                r_diff_upper = abs(g1_r_max - g2_r_min)
                if (theta_diff_lower < fusion_theta_threshold and theta_diff_upper < fusion_theta_threshold and
                    r_diff_lower < fusion_r_threshold and r_diff_upper < fusion_r_threshold):
                    # Fusionar g2 en g1 y actualizar
                    merged_points = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    g1['points'] = merged_points
                    update_group_fit(g1, model)
                    groups.pop(j)
                    merged = True
                    break
            if merged:
                break

def add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window):
    """
    Agrega un punto a un grupo temporal si este punto se encuentra dentro de
    ventanas fijas (theta_fit_window y r_fit_window) de un grupo temporal existente.
    """
    assigned = False
    for tg in temp_groups:
        tg_theta_min = tg['points']['theta'].min()
        tg_theta_max = tg['points']['theta'].max()
        tg_r_min = tg['points']['r'].min()
        tg_r_max = tg['points']['r'].max()
        if ((point['theta'] >= tg_theta_min - theta_fit_window/2) and 
            (point['theta'] <= tg_theta_max + theta_fit_window/2)) and \
           ((point['r'] >= tg_r_min - r_fit_window/2) and 
            (point['r'] <= tg_r_max + r_fit_window/2)):
            tg['points'] = pd.concat([tg['points'], point.to_frame().T], ignore_index=True)
            assigned = True
            break
    if not assigned:
        new_tg = {'points': point.to_frame().T.copy()}
        temp_groups.append(new_tg)

def review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold):
    """
    Revisa los grupos temporales; si tienen al menos 3 puntos se ajusta la recta y se
    intenta fusionarlos con algún grupo principal si la distancia promedio es menor a radius_tolerance.
    """
    new_temp_groups = []
    for tg in temp_groups:
        if len(tg['points']) < 3:
            new_temp_groups.append(tg)
            continue
        X_tg = tg['points'][['theta']]
        y_tg = tg['points']['r']
        model.fit(X_tg, y_tg)
        tg_slope = model.coef_[0]
        tg_intercept = model.intercept_
        tg_mean = tg['points']['theta'].mean()
        merged = False
        for mg in groups:
            mg_theta_min = mg['points']['theta'].min()
            mg_theta_max = mg['points']['theta'].max()
            if mg_theta_min <= tg_mean <= mg_theta_max:
                dist_values = tg['points'].apply(lambda row: calculate_distance(mg['slope'], mg['intercept'], row['theta'], row['r']), axis=1)
                if dist_values.mean() < radius_tolerance:
                    mg['points'] = pd.concat([mg['points'], tg['points']], ignore_index=True)
                    update_group_fit(mg, model)
                    merged = True
                    break
        if not merged:
            new_group = {
                'cluster_id': None,
                'points': tg['points'],
                'slope': tg_slope,
                'intercept': tg_intercept,
                'pa': calculate_pa(tg_slope, tg_intercept)
            }
            groups.append(new_group)
    temp_groups.clear()
    for tg in new_temp_groups:
        temp_groups.append(tg)

# ---------------------------
# Función principal optimizada
# ---------------------------
def process_spiral_arms_interactive(
    id_halo,
    quartile_threshold=0.55,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    eps_polar=0.7,
    min_samples_polar=18,
    slope_variation_threshold=0.20,  # Variación máxima de pendiente del 20%
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10,       # Cada cuántas iteraciones se guarda un estado
    theta_fit_window=30.0,      # Ventana en grados para ajuste local
    r_fit_window=10.0,          # Ventana fija para r
    fusion_theta_threshold=7.0, # Diferencia máxima fija en θ para fusión
    fusion_r_threshold=7.0      # Diferencia máxima fija en r para fusión
):
    """
    Procesa los datos, agrupa puntos y retorna estados intermedios para visualización.
    
    Se agrupan puntos en grupos principales manteniendo que la variación en la pendiente al actualizar
    el ajuste local no sea mayor al 20%. Cada grupo calcula su PA de forma independiente, y al final
    se filtran los grupos cuyo PA (valor absoluto) esté dentro del rango flexible_pa_range.
    
    Además, se fusionan grupos contiguos (con umbrales fijos) si sus límites en θ y en r están muy próximos.
    """
    # Cargar y convertir datos
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= 50) & (df['theta'] <= 250)].copy()
    
    # Filtrado por percentil
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # DBSCAN en (r, theta)
    X_polar = df_filtered[['r', 'theta']].values
    dbscan = DBSCAN(eps=eps_polar, min_samples=min_samples_polar)
    df_filtered['cluster'] = dbscan.fit_predict(X_polar)
    
    # Fusionar clusters cercanos
    clusters_tolerance = 0.70
    def merge_close_clusters(clusters, clusters_tolerance):
        merged_clusters = {}
        used_clusters = set()
        for id1, points1 in clusters.items():
            if id1 in used_clusters:
                continue
            merged_points = points1.copy()
            for id2, points2 in clusters.items():
                if id1 >= id2 or id2 in used_clusters:
                    continue
                min_distance = np.min(np.sqrt(
                    (points1['r'].values[:, None] - points2['r'].values)**2 +
                    (points1['theta'].values[:, None] - points2['theta'].values)**2
                ))
                if min_distance < clusters_tolerance:
                    merged_points = pd.concat([merged_points, points2], ignore_index=True)
                    used_clusters.add(id2)
            merged_clusters[id1] = merged_points
            used_clusters.add(id1)
        return merged_clusters
    
    clusters_polar = {}
    for cluster_id in np.unique(df_filtered['cluster']):
        if cluster_id == -1:
            continue
        cluster_points = df_filtered[df_filtered['cluster'] == cluster_id].copy()
        clusters_polar[cluster_id] = cluster_points[['r', 'theta']]
    clusters_polar = merge_close_clusters(clusters_polar, clusters_tolerance)
    
    # Inicializar grupos principales a partir de los clusters
    model = LinearRegression()
    groups = []
    for cid, pts in clusters_polar.items():
        if len(pts) >= 3:
            X_ = pts[['theta']]
            y_ = pts['r']
            model.fit(X_, y_)
            groups.append({
                'cluster_id': cid,
                'points': pts.copy(),
                'slope': model.coef_[0],
                'intercept': model.intercept_,
                'pa': calculate_pa(model.coef_[0], model.intercept_)
            })
        else:
            groups.append({
                'cluster_id': cid,
                'points': pts.copy(),
                'slope': None,
                'intercept': None,
                'pa': None
            })
    
    # Preparar para la asignación iterativa
    df_local = df_filtered.copy()
    df_local.sort_values(by='theta', inplace=True)
    states = []
    temp_groups = []  # Puntos que no se asignaron aún a ningún grupo
    iteration = 0
    temp_review_interval = snapshot_interval * 15
    
    # Asignación iterativa de puntos
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        best_group = None
        best_distance = None
        
        # Buscar grupo candidato: el punto debe estar dentro de los límites (theta y r) del grupo
        for g in groups:
            g_theta_min = g['points']['theta'].min()
            g_theta_max = g['points']['theta'].max()
            g_r_min = g['points']['r'].min()
            g_r_max = g['points']['r'].max()
            if not (g_theta_min <= theta_pt <= g_theta_max and g_r_min <= r_pt <= g_r_max):
                continue
            # Tomar una ventana centrada en el punto
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = g['points'][(g['points']['theta'] >= window_min) & (g['points']['theta'] <= window_max)]
            if len(subset) == 0:
                continue
            # Verificar que el valor de r del punto esté dentro de la ventana
            if not (subset['r'].min() <= r_pt <= subset['r'].max()):
                continue
            # Actualizar ajuste local con el nuevo punto
            subset = pd.concat([subset, point.to_frame().T], ignore_index=True)
            if len(subset) < 2:
                continue
            model.fit(subset[['theta']], subset['r'])
            new_slope = model.coef_[0]
            new_intercept = model.intercept_
            if g['slope'] is None:
                continue
            # Solo se acepta si la variación en la pendiente es menor o igual al 20%
            if abs(new_slope - g['slope'])/(abs(g['slope'])+1e-12) > slope_variation_threshold:
                continue
            dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
            if dist > radius_tolerance:
                continue
            if best_distance is None or dist < best_distance:
                best_distance = dist
                best_group = g
        
        if best_group is not None:
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            update_group_fit(best_group, model)
            # Intentar fusionar con otros grupos usando umbrales fijos
            merge_groups_if_close(groups, fusion_theta_threshold, fusion_r_threshold, model)
        else:
            # Si no se encontró grupo candidato, se intenta fusionar con el último grupo (sin filtro de pendiente)
            merged_with_last = False
            if len(groups) > 0:
                last_g = groups[-1]
                if (last_g['points']['theta'].min() <= theta_pt <= last_g['points']['theta'].max() and
                    last_g['points']['r'].min() <= r_pt <= last_g['points']['r'].max()):
                    last_g['points'] = pd.concat([last_g['points'], point.to_frame().T], ignore_index=True)
                    update_group_fit(last_g, model)
                    merged_with_last = True
            if not merged_with_last:
                add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window)
        
        iteration += 1
        if iteration % temp_review_interval == 0:
            review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold)
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_groups': copy.deepcopy(temp_groups),
                'remaining': df_local.copy()
            })
    
    if len(temp_groups) > 0:
        review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold)
    
    # Opcional: subdividir grupos por gaps en theta (como en versiones anteriores)
    def subdivide_if_gap(group, max_gap=max_theta_gap):
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = None, None, None
        s2, i2, pa2 = None, None, None
        if len(pts1) >= 2:
            model.fit(pts1[['theta']], pts1['r'])
            s1 = model.coef_[0]
            i1 = model.intercept_
            pa1 = calculate_pa(s1, i1)
        if len(pts2) >= 2:
            model.fit(pts2[['theta']], pts2['r'])
            s2 = model.coef_[0]
            i2 = model.intercept_
            pa2 = calculate_pa(s2, i2)
        seg1 = {
            'cluster_id': group['cluster_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'cluster_id': group['cluster_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]
    
    final_groups = []
    if do_subdivide:
        for g in groups:
            final_groups.extend(subdivide_if_gap(g, max_theta_gap))
    else:
        final_groups = groups
    
    # Filtrar grupos según su PA de forma independiente
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'temp_groups': copy.deepcopy(temp_groups),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered

def plot_intermediate_state_cartesian(state, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    if state['temp_groups']:
        for idx, tg in enumerate(state['temp_groups']):
            pts = tg['points']
            plt.scatter(pts['theta'], pts['r'], s=10, color='black', marker='x', label=f"Temp {idx+1}")
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()


In [0]:
# Ejecución interactiva:
states, final_groups, df_filtered = process_spiral_arms_interactive(
    id_halo="17",
    quartile_threshold=0.155,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    eps_polar=0.7,
    min_samples_polar=18,
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10,
    theta_fit_window=25.10,
    r_fit_window=2.50 
)

print("Total de estados guardados:", len(states))
print("Total de grupos finales:", len(final_groups))

def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state_cartesian(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0));

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

def plot_intermediate_state_cartesian(state, df_all):
    """
    Grafica un estado en el plano (theta, r).
    - state['groups'] es una lista de diccionarios con 'points' (DataFrame).
    - df_all es el DataFrame original filtrado, para mostrar todos los puntos en gris.
    """
    plt.figure(figsize=(10, 6))
    # Puntos completos (en gris)
    plt.scatter(df_all['theta'], df_all['r'], s=5, color='lightgrey', alpha=0.5, label='Datos completos')
    
    # Colores para cada grupo
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan', 'gold', 'darkblue']
    
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=15, color=color, label=f"Grupo {idx+1}")
    
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def add_point_to_groups(point, groups, theta_fit_window, r_fit_window):
    """
    Intenta agregar 'point' a uno de los 'groups' ya existentes.
    La condición es que el punto esté dentro de la 'caja delimitadora' 
    del grupo extendida por las ventanas fijas (theta_fit_window, r_fit_window).
    
    Retorna True si se agregó a algún grupo, False en caso contrario.
    """
    theta_pt = point['theta']
    r_pt = point['r']
    
    for g in groups:
        # Caja delimitadora actual del grupo
        theta_min = g['theta_min']
        theta_max = g['theta_max']
        r_min = g['r_min']
        r_max = g['r_max']
        
        # Ventana extendida
        if ((theta_pt >= theta_min - theta_fit_window/2) and 
            (theta_pt <= theta_max + theta_fit_window/2) and
            (r_pt >= r_min - r_fit_window/2) and 
            (r_pt <= r_max + r_fit_window/2)):
            # Se agrega el punto al grupo
            g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
            # Actualizar bounding box del grupo
            g['theta_min'] = g['points']['theta'].min()
            g['theta_max'] = g['points']['theta'].max()
            g['r_min'] = g['points']['r'].min()
            g['r_max'] = g['points']['r'].max()
            return True
    return False

def merge_all_groups(groups, theta_merge_threshold, r_merge_threshold):
    """
    Fusiona grupos cuyos rangos en theta y r estén muy próximos.
    Se hace en un bucle while hasta que no haya más fusiones posibles.
    """
    merged_flag = True
    while merged_flag:
        merged_flag = False
        i = 0
        while i < len(groups):
            j = i + 1
            while j < len(groups):
                g1 = groups[i]
                g2 = groups[j]
                # Diferencias de límites
                # g1
                g1_th_min = g1['theta_min']
                g1_th_max = g1['theta_max']
                g1_r_min  = g1['r_min']
                g1_r_max  = g1['r_max']
                # g2
                g2_th_min = g2['theta_min']
                g2_th_max = g2['theta_max']
                g2_r_min  = g2['r_min']
                g2_r_max  = g2['r_max']
                
                # Verificar si están dentro de thresholds fijos en los límites
                # Queremos que ambos grupos se "toquen" en theta y en r
                cond_theta = (abs(g1_th_min - g2_th_max) < theta_merge_threshold and
                              abs(g1_th_max - g2_th_min) < theta_merge_threshold)
                cond_r = (abs(g1_r_min - g2_r_max) < r_merge_threshold and
                          abs(g1_r_max - g2_r_min) < r_merge_threshold)
                
                if cond_theta and cond_r:
                    # Fusionar g2 en g1
                    g1['points'] = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    g1['theta_min'] = g1['points']['theta'].min()
                    g1['theta_max'] = g1['points']['theta'].max()
                    g1['r_min'] = g1['points']['r'].min()
                    g1['r_max'] = g1['points']['r'].max()
                    
                    # Eliminar g2
                    groups.pop(j)
                    merged_flag = True
                else:
                    j += 1
            i += 1

def process_spiral_arms_by_proximity(
    id_halo,
    quartile_threshold=0.55,
    # Ventanas para añadir puntos a un grupo
    theta_fit_window=30.0,
    r_fit_window=10.0,
    # Umbrales para fusionar grupos
    theta_merge_threshold=7.0,
    r_merge_threshold=7.0,
    # Cada cuántas iteraciones guardamos un "estado"
    snapshot_interval=10
):
    """
    Procesa datos y agrupa puntos solo por cercanía en (theta, r):
      1) Se filtran por un percentil (quartile_threshold).
      2) Se recorren en orden creciente de theta.
      3) Se intenta agregar cada punto a un grupo si cae dentro de la 
         bounding box del grupo extendida por (theta_fit_window, r_fit_window).
      4) Si no cabe, se crea un grupo nuevo.
      5) Al final se fusionan grupos cercanos si sus bounding boxes 
         difieren menos de (theta_merge_threshold, r_merge_threshold).
    
    Retorna:
      - states: lista de estados intermedios para graficar
      - groups: lista final de grupos
      - df_filtered: DataFrame filtrado
    """
    # 1. Cargar datos
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    
    # 2. Convertir a coordenadas polares
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    
    # Filtrar theta entre 50 y 250
    df = df[(df['theta'] >= 50) & (df['theta'] <= 250)].copy()
    
    # 3. Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    
    # Ordenar por theta ascendente
    df_filtered.sort_values(by='theta', inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 4. Agrupación por cercanía
    groups = []  # cada elemento tendrá: { 'points': df, 'theta_min':..., 'theta_max':..., 'r_min':..., 'r_max':... }
    states = []
    iteration = 0
    df_local = df_filtered.copy()
    
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        
        # Intentar añadir el punto a un grupo existente
        assigned = add_point_to_groups(point, groups, theta_fit_window, r_fit_window)
        
        if not assigned:
            # Crear un grupo nuevo
            new_grp = {
                'points': point.to_frame().T.copy(),
                'theta_min': point['theta'],
                'theta_max': point['theta'],
                'r_min': point['r'],
                'r_max': point['r']
            }
            groups.append(new_grp)
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            # Guardar un estado intermedio
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'remaining': df_local.copy()
            })
    
    # 5. Fusión final de grupos cercanos
    merge_all_groups(groups, theta_merge_threshold, r_merge_threshold)
    
    # Guardar estado final
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(groups),
        'remaining': df_local.copy()
    })
    
    return states, groups, df_filtered


In [0]:
states, final_groups, df_filtered = process_spiral_arms_by_proximity(
    id_halo="17",
    quartile_threshold=0.155,
    theta_fit_window=30.0,
    r_fit_window=2.50,
    theta_merge_threshold=7.0,
    r_merge_threshold=7.0,
    snapshot_interval=10
)

print("Total de estados guardados:", len(states))
print("Total de grupos finales:", len(final_groups))

def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state_cartesian(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0));

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

def plot_groups(groups, df_all, title="Grupos en (theta, r)"):
    """
    Grafica los grupos resultantes en el plano (theta, r),
    junto con los puntos originales en gris.
    """
    plt.figure(figsize=(10, 6))
    # Graficar todos los puntos en gris
    plt.scatter(df_all['theta'], df_all['r'], s=5, color='lightgrey', alpha=0.5, label='Datos completos')
    
    # Colores de ejemplo
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    
    for idx, grp in enumerate(groups):
        pts = grp['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=15, color=color, label=f"Grupo {idx+1}")
    
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def build_graph(points, max_theta_diff, max_r_diff, r_gap_exclusion=2.0):
    """
    Construye un grafo (lista de adyacencia) donde cada nodo es un punto,
    y hay arista entre i y j si:
      - |theta_i - theta_j| <= max_theta_diff
      - |r_i - r_j| <= max_r_diff
      - |r_i - r_j| <= r_gap_exclusion (para evitar saltos > 2 en r)
    Retorna:
      - graph: lista de listas de índices adyacentes
      - n: número total de puntos
    """
    n = len(points)
    graph = [[] for _ in range(n)]
    
    # Convertimos a arrays para vectorizar
    theta_arr = points['theta'].values
    r_arr = points['r'].values
    
    # Para optimizar, se puede hacer un barrido lineal o 2D-tree, 
    # pero aquí haremos un doble bucle simple (O(n^2)) para mayor claridad.
    for i in range(n):
        for j in range(i+1, n):
            dtheta = abs(theta_arr[i] - theta_arr[j])
            dr = abs(r_arr[i] - r_arr[j])
            if dtheta <= max_theta_diff and dr <= max_r_diff and dr <= r_gap_exclusion:
                # Son vecinos
                graph[i].append(j)
                graph[j].append(i)
    
    return graph, n

def bfs_connected_components(graph, n, points_df):
    """
    Realiza BFS para encontrar componentes conexas en el grafo.
    graph: lista de adyacencia
    n: número de nodos
    points_df: DataFrame original de puntos
    Retorna una lista de grupos, cada grupo es un dict con:
      { 'points': DataFrame con filas correspondientes a la componente }
    """
    visited = [False]*n
    groups = []
    
    for start in range(n):
        if not visited[start]:
            # Iniciar BFS desde start
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            
            # Crear un grupo con los puntos de comp_indices
            grp_points = points_df.iloc[comp_indices].copy()
            grp_points.reset_index(drop=True, inplace=True)
            groups.append({'points': grp_points})
    
    return groups

def process_spiral_arms_by_advanced_proximity(
    id_halo,
    # Filtrado
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    # Parámetros para cercanía
    max_theta_diff=5.0,   # en grados
    max_r_diff=2.0,       # en r
    r_gap_exclusion=2.0,  # si hay un salto > 2 en r, no se conectan
):
    """
    Carga y filtra puntos, luego construye un grafo de vecindad
    en (theta, r) donde dos puntos se conectan si:
      - |theta_i - theta_j| <= max_theta_diff
      - |r_i - r_j| <= max_r_diff
      - |r_i - r_j| <= r_gap_exclusion  (evita saltos > 2 en r)
    Finalmente extrae los grupos como componentes conexas (BFS).
    Retorna:
      - groups: lista de dicts con 'points' (DataFrame de cada grupo)
      - df_filtered: DataFrame de los puntos usados
    """
    # 1. Cargar datos
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    
    # 2. Convertir a polares
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    
    # 3. Filtrar theta
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    # 4. Filtrar por percentil
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 5. Construir grafo
    graph, n = build_graph(df_filtered, max_theta_diff, max_r_diff, r_gap_exclusion=r_gap_exclusion)
    
    # 6. Extraer componentes conexas
    groups = bfs_connected_components(graph, n, df_filtered)
    
    return groups, df_filtered


# Ejemplo de uso:
if __name__ == "__main__":
    # Llamada de ejemplo
    groups, df_filtered = process_spiral_arms_by_advanced_proximity(
        id_halo="17",
        quartile_threshold=0.155,
        theta_min=50.0,
        theta_max=250.0,
        max_theta_diff=5.0,
        max_r_diff=2.0,
        r_gap_exclusion=2.0
    )
    
    print(f"Se formaron {len(groups)} grupos.")
    # Graficar
    plot_groups(groups, df_filtered, title="Grupos por BFS y salto en r<=2")


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree, connected_components

def process_by_MST_convex(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    edge_threshold=5.0,   # Umbral máximo para conservar una arista en el MST
    min_cluster_size=5    # Tamaño mínimo de un grupo
):
    """
    Procesa los datos y agrupa los puntos en (theta, r) utilizando el MST:
      1. Carga y filtra los datos (por rango de theta y percentil en 'rho_resta_final_exp').
      2. Calcula la matriz de distancias entre puntos en (theta, r) (sin escalado adicional).
      3. Obtiene el MST y "poda" las aristas con longitud > edge_threshold.
      4. Obtiene los componentes conexos (grupos) del grafo resultante.
      5. Se conservan únicamente los grupos con al menos min_cluster_size puntos.
    
    Retorna:
      - clusters: lista de diccionarios, cada uno con 'points' (DataFrame) para ese grupo.
      - df_filtered: DataFrame filtrado.
      - labels: array de etiquetas para cada punto (útil para ver a qué grupo pertenece cada uno).
    """
    # 1. Cargar datos
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    
    # 2. Convertir a coordenadas polares
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    
    # Filtrar por theta
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    # Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 3. Construir la matriz de distancias en el espacio (theta, r)
    # Si las unidades de theta y r son muy diferentes, se podrían escalar.
    X = df_filtered[['theta', 'r']].values
    D = distance_matrix(X, X)
    
    # 4. Calcular el MST
    mst = minimum_spanning_tree(D)
    mst = mst.toarray()
    
    # 5. Podar (prune) el MST: ponemos a cero las aristas con longitud mayor a edge_threshold
    pruned_mst = np.where(mst > edge_threshold, 0, mst)
    # Para construir el grafo, hacemos simétrico (ya que MST es dirigido)
    graph = pruned_mst + pruned_mst.T
    
    # 6. Extraer componentes conexas
    n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
    
    clusters = []
    for i in range(n_components):
        indices = np.where(labels == i)[0]
        if len(indices) >= min_cluster_size:
            cluster_df = df_filtered.iloc[indices].copy()
            cluster_df.reset_index(drop=True, inplace=True)
            clusters.append({'points': cluster_df})
    
    return clusters, df_filtered, labels

def plot_MST_clusters(clusters, df_filtered, labels, title="Clustering usando MST y Convex Hull"):
    """
    Grafica los grupos obtenidos y, para cada grupo, dibuja su envolvente convexa.
    """
    plt.figure(figsize=(10, 6))
    # Graficar todos los puntos en gris
    plt.scatter(df_filtered['theta'], df_filtered['r'], s=5, color='lightgrey', alpha=0.5, label='Datos')
    
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    
    for idx, grp in enumerate(clusters):
        pts = grp['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=15, color=color, label=f"Grupo {idx+1}")
        if len(pts) >= 3:
            points = pts[['theta', 'r']].values
            hull = ConvexHull(points)
            for simplex in hull.simplices:
                plt.plot(points[simplex, 0], points[simplex, 1], color=color, lw=2)
    
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()


# Ejemplo de ejecución:
if __name__ == "__main__":
    # Ajusta los parámetros según necesites
    clusters, df_filtered, labels = process_by_MST_convex(
        id_halo="17",
        quartile_threshold=0.155,
        theta_min=50.0,
        theta_max=70.0,
        edge_threshold=2.0,
        min_cluster_size=5
    )
    
    print(f"Se han formado {len(clusters)} grupos (componentes conexas).")
    plot_MST_clusters(clusters, df_filtered, labels, title="Clustering MST + Convex Hull")


In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression
import copy

def process_spiral_arms_polar_intermediate(
    id_halo,
    quartile_threshold=0.55,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    eps_polar=0.7,
    min_samples_polar=18,
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,  # Para detectar un gap y dividir el grupo en 2, si es muy grande
    do_subdivide=True    # Activar o desactivar la subdivisión
):
    """
    Versión "intermedia" que:
      1. Aplica DBSCAN en (r, theta).
      2. Fusiona clusters cercanos.
      3. Ajuste lineal iterativo (bondadoso), controlando la variación de la pendiente (20%).
      4. Reasigna descartados.
      5. (Opcional) Subdivide un grupo en 2 si se detecta un gap grande en theta.
      6. Filtra por PA al final.
    
    Se ha añadido una condición extra en la asignación iterativa para que el nuevo punto
    deba estar también dentro del rango horizontal (θ) actual del grupo, extendido en ±theta_margin.
    
    Retorna:
      - valid_groups: Lista de grupos (diccionarios) con sus parámetros ajustados.
      - df: DataFrame original con las columnas añadidas (r, θ, cluster).
    """
    # Parámetro extra: margen horizontal para la asignación (por ejemplo, la mitad de theta_fit_window)
    theta_margin = 15.0  # Puedes ajustar este valor (por ejemplo, theta_fit_window/2)
    
    # 1. Cargar datos
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')

    # 2. Convertir a coordenadas polares
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])

    # 3. Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)

    # 4. DBSCAN en el plano (r, theta)
    X_polar = df_filtered[['r', 'theta']].values
    dbscan = DBSCAN(eps=eps_polar, min_samples=min_samples_polar)
    df_filtered['cluster'] = dbscan.fit_predict(X_polar)

    # 5. Fusión de clusters cercanos
    clusters_tolerance = 0.70
    def merge_close_clusters(clusters, clusters_tolerance):
        merged_clusters = {}
        used_clusters = set()
        for id1, points1 in clusters.items():
            if id1 in used_clusters:
                continue
            merged_points = points1.copy()
            for id2, points2 in clusters.items():
                if id1 >= id2 or id2 in used_clusters:
                    continue
                min_distance = np.min(np.sqrt(
                    (points1['r'].values[:, None] - points2['r'].values)**2 +
                    (points1['theta'].values[:, None] - points2['theta'].values)**2
                ))
                if min_distance < clusters_tolerance:
                    merged_points = pd.concat([merged_points, points2], ignore_index=True)
                    used_clusters.add(id2)
            merged_clusters[id1] = merged_points
            used_clusters.add(id1)
        return merged_clusters

    # Crear diccionario de clusters
    clusters_polar = {}
    for cluster_id in np.unique(df_filtered['cluster']):
        if cluster_id == -1:
            continue
        cluster_points = df_filtered[df_filtered['cluster'] == cluster_id].copy()
        clusters_polar[cluster_id] = cluster_points[['r', 'theta']]

    clusters_polar = merge_close_clusters(clusters_polar, clusters_tolerance)

    # 6. Inicializar grupos
    model = LinearRegression()

    def calculate_distance(slope, intercept, theta, r):
        pred_r = slope * theta + intercept
        return abs(pred_r - r)

    def calculate_pa(slope, intercept):
        if intercept != 0:
            return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
        return np.nan

    def is_valid_pa(pa, valid_range):
        return valid_range[0] <= abs(pa) <= valid_range[1]

    groups = []
    for cid, pts in clusters_polar.items():
        if len(pts) >= 3:
            X_ = pts[['theta']].values
            y_ = pts['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None

        groups.append({
            'cluster_id': cid,
            'points': pts.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })

    # 7. Asignación iterativa de puntos (incluye control de pendiente y vecindad horizontal)
    df_local = df_filtered.copy()
    discarded_points = []

    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        assigned = False

        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue

            # Control extra: el nuevo punto debe estar dentro del rango horizontal del grupo extendido en ±theta_margin
            group_theta_min = g['points']['theta'].min()
            group_theta_max = g['points']['theta'].max()
            if not (group_theta_min - theta_margin <= theta_pt <= group_theta_max + theta_margin):
                continue

            dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
            if dist <= radius_tolerance:
                # Verificar que la distancia a TODOS los puntos del grupo sea menor a point_distance_tolerance
                distances = g['points'].apply(
                    lambda row: calculate_distance(g['slope'], g['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if all(d <= point_distance_tolerance for d in distances):
                    # Actualizar ajuste lineal con el nuevo punto
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= slope_variation_threshold:
                        g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(g['slope'], g['intercept'])
                        assigned = True
                        break

        if not assigned:
            discarded_points.append(point.to_dict())

    # 8. Reasignar descartados
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            theta_pt = p['theta']
            r_pt = p['r']
            assigned = False
            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                # Exigir que el punto esté dentro del rango horizontal del grupo
                group_theta_min = g['points']['theta'].min()
                group_theta_max = g['points']['theta'].max()
                if not (group_theta_min - theta_margin <= theta_pt <= group_theta_max + theta_margin):
                    continue
                dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
                if dist <= radius_tolerance and g['pa'] is not None and is_valid_pa(g['pa'], flexible_pa_range):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= slope_variation_threshold:
                        g['points'] = pd.concat([g['points'], p.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(new_slope, g['intercept'])
                        assigned = True
                        break
            if not assigned:
                # Crear grupo nuevo para este punto
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new, i_new)
                if is_valid_pa(pa_new, flexible_pa_range):
                    groups.append({
                        'cluster_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })

    # 9. Subdivisión (opcional) en caso de gaps grandes en theta
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_

    def subdivide_if_gap(group, max_gap=max_theta_gap):
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)
        seg1 = {
            'cluster_id': group['cluster_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'cluster_id': group['cluster_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]

    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap=max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups

    # 10. Filtrar por PA
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and is_valid_pa(pa_val, flexible_pa_range):
                valid_groups.append(grp)

    return valid_groups, df

###############################################################################
# EJEMPLO DE EJECUCIÓN Y GRAFICACIÓN
###############################################################################
if __name__ == "__main__":
    id_halo = "17"
    quartile_threshold = 0.155
    radius_tolerance = 0.45
    point_distance_tolerance = 0.59
    flexible_pa_range = (4.0, 30.0)
    eps_polar = 0.7
    min_samples_polar = 18
    slope_variation_threshold = 0.20  # 20%
    max_theta_gap = 15.0
    do_subdivide = True

    valid_groups, df = process_spiral_arms_polar_intermediate(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        radius_tolerance=radius_tolerance,
        point_distance_tolerance=point_distance_tolerance,
        flexible_pa_range=flexible_pa_range,
        eps_polar=eps_polar,
        min_samples_polar=min_samples_polar,
        slope_variation_threshold=slope_variation_threshold,
        max_theta_gap=max_theta_gap,
        do_subdivide=do_subdivide
    )

    print("Total de grupos finales:", len(valid_groups))

    # Gráfica en (theta vs r)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['theta'], df['r'], s=1, color='silver', alpha=0.5, label='Datos Originales')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']

    for idx, grp in enumerate(valid_groups):
        pts = grp['points']
        theta_vals = pts['theta']
        r_vals = pts['r']
        pa_val = grp['pa'] if grp['pa'] is not None else 0.0
        color = colors[idx % len(colors)]
        plt.scatter(theta_vals, r_vals, s=10, alpha=0.8, color=color,
                    label=f"Grupo {idx+1}, PA={pa_val:.2f}")
        if grp['slope'] is not None:
            theta_line = np.linspace(theta_vals.min(), theta_vals.max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)

    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Ajustes intermedios en (r, θ) - Halo {id_halo}")
    plt.grid(True)
    plt.show()

    # Gráfica polar
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 8))
    theta_all_rad = np.radians(df['theta'])
    ax.scatter(theta_all_rad, df['r'], s=1, color='gray', alpha=0.5, label='Datos Originales')

    for idx, grp in enumerate(valid_groups):
        pts = grp['points']
        theta_rad = np.radians(pts['theta'])
        r_vals = pts['r']
        pa_val = grp['pa'] if grp['pa'] is not None else 0.0
        color = colors[idx % len(colors)]
        ax.scatter(theta_rad, r_vals, s=10, alpha=0.8, color=color,
                   label=f"Grupo {idx+1} PA: {pa_val:.2f}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)

    ax.set_title(f"Ajustes intermedios en proyección polar - Halo {id_halo}", fontsize=14, pad=20)
    plt.show()


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.spatial import ConvexHull
from matplotlib.path import Path

def plot_intermediate_state_cartesian(state, df_all):
    """
    Grafica un estado en el plano (theta, r).
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=5, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan', 'gold', 'darkblue']
    
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=15, color=color, label=f"Grupo {idx+1}")
        # Dibujar el convex hull
        if len(pts) >= 3:
            pts_arr = pts[['theta','r']].values
            try:
                hull = ConvexHull(pts_arr)
                hull_pts = pts_arr[hull.vertices]
                hull_pts = np.concatenate([hull_pts, hull_pts[:1]], axis=0)  # cerrar polígono
                plt.plot(hull_pts[:,0], hull_pts[:,1], '-', color=color, lw=2)
            except Exception:
                pass
    
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def merge_all_groups(groups, theta_merge_threshold, r_merge_threshold):
    """
    Fusiona grupos si sus límites están muy próximos. (Opcional)
    """
    merged_flag = True
    while merged_flag:
        merged_flag = False
        i = 0
        while i < len(groups):
            j = i + 1
            while j < len(groups):
                g1 = groups[i]
                g2 = groups[j]
                # Límites de g1
                g1_th_min = g1['theta_min']
                g1_th_max = g1['theta_max']
                g1_r_min  = g1['r_min']
                g1_r_max  = g1['r_max']
                # Límites de g2
                g2_th_min = g2['theta_min']
                g2_th_max = g2['theta_max']
                g2_r_min  = g2['r_min']
                g2_r_max  = g2['r_max']
                
                cond_theta = (abs(g1_th_min - g2_th_max) < theta_merge_threshold and
                              abs(g1_th_max - g2_th_min) < theta_merge_threshold)
                cond_r = (abs(g1_r_min - g2_r_max) < r_merge_threshold and
                          abs(g1_r_max - g2_r_min) < r_merge_threshold)
                
                if cond_theta and cond_r:
                    g1['points'] = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    # Recalcular límites con bounding box (o hull)
                    update_group_limits(g1)
                    groups.pop(j)
                    merged_flag = True
                else:
                    j += 1
            i += 1

def update_group_limits(group):
    """
    Recalcula los límites (theta_min, etc.) usando el convex hull si hay 3 o más puntos.
    De lo contrario, usa bounding box.
    """
    pts = group['points']
    if len(pts) >= 3:
        arr = pts[['theta','r']].values
        try:
            hull = ConvexHull(arr)
            hull_pts = arr[hull.vertices]
            group['theta_min'] = hull_pts[:,0].min()
            group['theta_max'] = hull_pts[:,0].max()
            group['r_min'] = hull_pts[:,1].min()
            group['r_max'] = hull_pts[:,1].max()
            return
        except Exception:
            pass
    # Si falla o hay menos de 3 puntos
    group['theta_min'] = pts['theta'].min()
    group['theta_max'] = pts['theta'].max()
    group['r_min'] = pts['r'].min()
    group['r_max'] = pts['r'].max()

def point_in_hull(point, hull_points):
    """
    Verifica si un punto (theta_pt, r_pt) está estrictamente dentro de un polígono (hull_points).
    hull_points es un array Nx2 (theta, r).
    """
    poly = Path(hull_points, closed=True)
    return poly.contains_point(point, radius=-1e-9)  # radio negativo => más estricto

def add_point_strict_convex(point, groups, theta_strict_window, r_strict_window, r_gap=2.0):
    """
    Agrega 'point' a un grupo existente de forma estricta:
      - Si el grupo tiene >=3 puntos, se calcula su convex hull y se verifica si el punto está DENTRO.
        (sin margen, más estricto)
      - No se añade si está fuera del hull.
      - Si el grupo tiene <3 puntos, se usa bounding box reducida (la mitad de theta_strict_window y r_strict_window).
      - Se verifica que no haya un salto > r_gap en la coordenada r.
    """
    theta_pt = point['theta']
    r_pt = point['r']
    
    for g in groups:
        # Comprobar gap en r
        if r_pt < g['r_min'] - r_gap or r_pt > g['r_max'] + r_gap:
            continue
        
        pts = g['points']
        if len(pts) >= 3:
            arr = pts[['theta','r']].values
            try:
                hull = ConvexHull(arr)
                hull_pts = arr[hull.vertices]
                # Verificar si el punto está ESTRICTAMENTE dentro
                inside = point_in_hull([theta_pt, r_pt], hull_pts)
                if inside:
                    # Agregar y recalcular límites
                    g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                    update_group_limits(g)
                    return True
            except Exception:
                pass
            # Si no está dentro, no se agrega.
        else:
            # Para grupos con <3 puntos, bounding box muy estricta (la mitad)
            half_theta = theta_strict_window / 2.0
            half_r = r_strict_window / 2.0
            t_min = pts['theta'].min()
            t_max = pts['theta'].max()
            rr_min = pts['r'].min()
            rr_max = pts['r'].max()
            
            if (theta_pt >= t_min - half_theta/2 and theta_pt <= t_max + half_theta/2 and
                r_pt >= rr_min - half_r/2 and r_pt <= rr_max + half_r/2):
                g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                update_group_limits(g)
                return True
    
    return False

def process_spiral_arms_by_proximity(
    id_halo,
    quartile_threshold=0.55,
    # Ventanas "estrictas" para agregar puntos
    theta_fit_window=30.0,
    r_fit_window=10.0,
    # Umbrales para fusionar grupos
    theta_merge_threshold=7.0,
    r_merge_threshold=7.0,
    # Salto máximo en r para no fusionar
    r_gap=2.0,
    snapshot_interval=10
):
    """
    Procesa datos y agrupa puntos de forma más estricta con convex hull:
      1) Se filtran por percentil y por theta (50-250).
      2) Se recorre de izquierda a derecha (orden ascending de theta).
      3) Para cada punto, se intenta agregar a un grupo:
         - Si el grupo tiene >=3 puntos, se exige que el punto esté ESTRICTAMENTE dentro del hull (sin margen).
         - Si el grupo tiene <3 puntos, se usa bounding box reducida (la mitad de la ventana).
         - También se verifica que no haya un salto en r mayor a r_gap.
      4) Si no se agrega, se crea un grupo nuevo.
      5) Al final se fusionan grupos con bounding boxes cercanas (opcional).
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= 50) & (df['theta'] <= 250)].copy()
    
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='theta', inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    groups = []
    states = []
    iteration = 0
    df_local = df_filtered.copy()
    
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        added = add_point_strict_convex(point, groups, theta_fit_window, r_fit_window, r_gap=r_gap)
        if not added:
            new_group = {
                'points': point.to_frame().T.copy()
            }
            # Inicializar límites
            new_group['theta_min'] = point['theta']
            new_group['theta_max'] = point['theta']
            new_group['r_min'] = point['r']
            new_group['r_max'] = point['r']
            groups.append(new_group)
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'remaining': df_local.copy()
            })
    
    # Fusión final (opcional). Coméntalo si no deseas fusión.
    merge_all_groups(groups, theta_merge_threshold, r_merge_threshold)
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(groups),
        'remaining': df_local.copy()
    })
    
    return states, groups, df_filtered

# Ejemplo de ejecución (coméntalo si pegas en un notebook):
if __name__ == "__main__":
    states, final_groups, df_filtered = process_spiral_arms_by_proximity(
        id_halo="17",
        quartile_threshold=0.155,
        theta_fit_window=30.0,
        r_fit_window=3.0,
        theta_merge_threshold=10.0,
        r_merge_threshold=2.0,
        r_gap=2.0,
        snapshot_interval=10
    )
    print(f"Se formaron {len(final_groups)} grupos.")
    plot_intermediate_state_cartesian(states[-1], df_filtered)




In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
from sklearn.neighbors import KDTree

# ==============================
# Unión–Encuentra (Union–Find)
# ==============================
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    def find(self, a):
        while self.parent[a] != a:
            self.parent[a] = self.parent[self.parent[a]]
            a = self.parent[a]
        return a
    def union(self, a, b):
        ra = self.find(a)
        rb = self.find(b)
        if ra != rb:
            self.parent[rb] = ra

# ==============================
# Función de graficación
# ==============================
def plot_groups(groups, df_all, title="Grupos finales"):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=5, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    for idx, g in enumerate(groups):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=15, color=color, label=f"Grupo {idx+1}")
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

# ==============================
# Función de fusión final de grupos
# ==============================
def merge_groups(groups, theta_merge_threshold, r_merge_threshold):
    merged_flag = True
    while merged_flag:
        merged_flag = False
        i = 0
        while i < len(groups):
            j = i + 1
            while j < len(groups):
                g1 = groups[i]
                g2 = groups[j]
                tmin1, tmax1, rmin1, rmax1 = g1['theta_min'], g1['theta_max'], g1['r_min'], g1['r_max']
                tmin2, tmax2, rmin2, rmax2 = g2['theta_min'], g2['theta_max'], g2['r_min'], g2['r_max']
                cond_theta = (abs(tmin1 - tmax2) < theta_merge_threshold and abs(tmax1 - tmin2) < theta_merge_threshold)
                cond_r = (abs(rmin1 - rmax2) < r_merge_threshold and abs(rmax1 - rmin2) < r_merge_threshold)
                if cond_theta and cond_r:
                    g1['points'] = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    g1['theta_min'] = g1['points']['theta'].min()
                    g1['theta_max'] = g1['points']['theta'].max()
                    g1['r_min'] = g1['points']['r'].min()
                    g1['r_max'] = g1['points']['r'].max()
                    groups.pop(j)
                    merged_flag = True
                else:
                    j += 1
            i += 1

# ==============================
# Función principal optimizada
# ==============================
def process_spiral_arms_by_proximity_optimized(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    # Parámetros para la vecindad: dos puntos se consideran vecinos si:
    # |theta_i - theta_j| <= theta_diff y |r_i - r_j| <= r_gap.
    theta_diff=10.0,
    r_gap=2.0,
    # Umbrales para fusión final de grupos (bounding boxes)
    theta_merge_threshold=7.0,
    r_merge_threshold=7.0
):
    """
    Agrupa puntos en (theta, r) usando KDTree y Union-Find.
    1) Se filtran los datos (theta en [theta_min, theta_max] y percentil en 'rho_resta_final_exp').
    2) Se ordenan por theta (de izquierda a derecha).
    3) Se construye un KDTree en un espacio escalado:
         X_scaled = [theta/theta_diff, r/r_gap]
       de modo que dos puntos se consideren vecinos si su Chebyshev distance <= 1.
    4) Se utiliza Union-Find para agrupar los puntos conectados.
    5) Para cada grupo se calcula la bounding box.
    6) (Opcional) Se fusionan grupos cuyas bounding boxes estén muy próximas.
    Retorna:
      - groups: lista de grupos (cada grupo es un diccionario con 'points', 'theta_min', 'theta_max', 'r_min', 'r_max').
      - df_filtered: DataFrame filtrado.
    """
    # 1. Cargar y filtrar datos
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    thresh = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > thresh].copy()
    df_filtered.sort_values(by='theta', inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    n = len(df_filtered)
    if n == 0:
        return [], df_filtered
    
    # 2. Construir KDTree en espacio escalado
    # Escalamos: en la primera dimensión dividimos por theta_diff y en la segunda por r_gap.
    X = df_filtered[['theta', 'r']].values
    X_scaled = np.column_stack([X[:,0] / theta_diff, X[:,1] / r_gap])
    
    tree = KDTree(X_scaled, metric='chebyshev')
    # Query: vecinos con Chebyshev distance <= 1 => |theta_i - theta_j| <= theta_diff and |r_i - r_j| <= r_gap
    indices_list = tree.query_radius(X_scaled, r=1.0)
    
    # 3. Agrupar con Union-Find
    uf = UnionFind(n)
    for i, neighbors in enumerate(indices_list):
        for j in neighbors:
            if i != j:
                uf.union(i, j)
    
    # 4. Extraer componentes conexas y calcular bounding box
    groups_dict = {}
    for i in range(n):
        root = uf.find(i)
        if root not in groups_dict:
            groups_dict[root] = []
        groups_dict[root].append(i)
    
    groups = []
    for comp in groups_dict.values():
        comp_df = df_filtered.iloc[comp].copy()
        comp_df.reset_index(drop=True, inplace=True)
        group = {
            'points': comp_df,
            'theta_min': comp_df['theta'].min(),
            'theta_max': comp_df['theta'].max(),
            'r_min': comp_df['r'].min(),
            'r_max': comp_df['r'].max()
        }
        groups.append(group)
    
    # 5. Fusión final de grupos (opcional)
    merge_groups(groups, theta_merge_threshold, r_merge_threshold)
    
    return groups, df_filtered

# ==============================
# Ejecución y visualización
# ==============================
if __name__ == "__main__":
    # Ajusta los parámetros según tus necesidades.
    groups, df_filtered = process_spiral_arms_by_proximity_optimized(
        id_halo="17",
        quartile_threshold=0.155,
        theta_min=50.0,
        theta_max=250.0,
        theta_diff=10.0,      # Vecindad en theta: 10 grados
        r_gap=2.50,            # Vecindad en r: 2 unidades (separación vertical)
        theta_merge_threshold=5.0,
        r_merge_threshold=4.0
    )
    print(f"Se formaron {len(groups)} grupos.")
    plot_groups(groups, df_filtered, title="Grupos finales (Optimizado)")


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
from collections import deque

def plot_groups(groups, df_all, title="Grupos finales"):
    """
    Grafica los grupos resultantes en (theta, r), con df_all en gris de fondo.
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=5, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    
    for idx, g in enumerate(groups):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=15, color=color, label=f"Grupo {idx+1}")
    
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def build_graph_rectangular(df_points, theta_diff, r_diff, r_gap=None):
    """
    Construye un grafo (lista de adyacencia) con vecindad rectangular:
      - |theta_i - theta_j| <= theta_diff
      - |r_i - r_j| <= r_diff
      - (opcional) |r_i - r_j| <= r_gap para excluir saltos grandes en r
    
    df_points debe estar ordenado por theta ascendente (para optimizar).
    """
    n = len(df_points)
    graph = [[] for _ in range(n)]
    
    # Extraemos arrays
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    
    # Para optimizar, buscaremos vecinos solo en un rango de theta
    # (por ejemplo, usando índices incrementales)
    j_start = 0
    for i in range(n):
        # Avanzamos j_start si theta_j < theta_i - theta_diff
        while j_start < n and theta_arr[j_start] < theta_arr[i] - theta_diff:
            j_start += 1
        
        # Explorar desde j_start hacia adelante mientras theta_j <= theta_i + theta_diff
        j = j_start
        while j < n and theta_arr[j] <= theta_arr[i] + theta_diff:
            if i != j:
                dtheta = abs(theta_arr[i] - theta_arr[j])
                dr = abs(r_arr[i] - r_arr[j])
                if dtheta <= theta_diff and dr <= r_diff:
                    if r_gap is None or dr <= r_gap:
                        graph[i].append(j)
                        graph[j].append(i)
            j += 1
    
    return graph

def bfs_components(graph, df_points):
    """
    Extrae componentes conexas mediante BFS en el grafo dado.
    Retorna lista de grupos, donde cada grupo = {'points': DataFrame, ... bounding box ...}.
    """
    n = len(graph)
    visited = [False]*n
    groups = []
    
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            
            comp_df = df_points.iloc[comp_indices].copy()
            comp_df.reset_index(drop=True, inplace=True)
            
            g = {
                'points': comp_df,
                'theta_min': comp_df['theta'].min(),
                'theta_max': comp_df['theta'].max(),
                'r_min': comp_df['r'].min(),
                'r_max': comp_df['r'].max()
            }
            groups.append(g)
    return groups

def merge_groups_bounding_box(groups, theta_merge_threshold, r_merge_threshold):
    """
    Fusiona grupos cuyas bounding boxes se superponen dentro de thresholds fijos en theta y r.
    """
    merged_flag = True
    while merged_flag:
        merged_flag = False
        i = 0
        while i < len(groups):
            j = i + 1
            while j < len(groups):
                g1 = groups[i]
                g2 = groups[j]
                # Verificar superposición
                cond_theta = (abs(g1['theta_min'] - g2['theta_max']) < theta_merge_threshold and
                              abs(g1['theta_max'] - g2['theta_min']) < theta_merge_threshold)
                cond_r = (abs(g1['r_min'] - g2['r_max']) < r_merge_threshold and
                          abs(g1['r_max'] - g2['r_min']) < r_merge_threshold)
                
                if cond_theta and cond_r:
                    # Fusionar
                    g1['points'] = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    # Recalcular bounding box
                    g1['theta_min'] = g1['points']['theta'].min()
                    g1['theta_max'] = g1['points']['theta'].max()
                    g1['r_min'] = g1['points']['r'].min()
                    g1['r_max'] = g1['points']['r'].max()
                    groups.pop(j)
                    merged_flag = True
                else:
                    j += 1
            i += 1

def process_spiral_arms_unified(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    # Parámetros para la vecindad rectangular
    theta_diff=10.0,
    r_diff=2.0,
    r_gap=2.0,
    # Umbrales para fusionar grupos
    theta_merge_threshold=7.0,
    r_merge_threshold=7.0
):
    """
    Enfoque unificado:
      1) Se cargan y filtran los datos (theta en [theta_min, theta_max], percentil).
      2) Se ordenan por theta (izquierda a derecha).
      3) Se construye un grafo con vecindad rectangular: 
         |theta_i - theta_j| <= theta_diff, |r_i - r_j| <= r_diff, y (opcional) dr <= r_gap.
      4) Se extraen los grupos como componentes conexas (BFS).
      5) Fusión final de bounding boxes con thresholds fijos (theta_merge_threshold, r_merge_threshold).
    Retorna:
      - groups: lista de grupos (cada uno con 'points', 'theta_min', etc.)
      - df_filtered: DataFrame filtrado
    """
    # 1) Carga y filtrado
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    # 2) Ordenar por theta
    df_filtered.sort_values(by='theta', inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 3) Construir grafo
    graph = build_graph_rectangular(df_filtered, theta_diff, r_diff, r_gap=r_gap)
    
    # 4) BFS para componentes conexas
    groups = bfs_components(graph, df_filtered)
    
    # 5) Fusión final de bounding boxes
    merge_groups_bounding_box(groups, theta_merge_threshold, r_merge_threshold)
    
    return groups, df_filtered

# Ejemplo de uso en un script:
if __name__ == "__main__":
    groups, df_filtered = process_spiral_arms_unified(
        id_halo="17",
        quartile_threshold=0.155,
        theta_min=50,
        theta_max=250,
        theta_diff=10.0,       # Cambia para ser más estricto o más laxo en theta
        r_diff=2.0,            # Cambia para ser más estricto o más laxo en r
        r_gap=2.0,             # Evita que se unan saltos en r > 2
        theta_merge_threshold=7.0,
        r_merge_threshold=7.0
    )
    print(f"Se formaron {len(groups)} grupos.")
    plot_groups(groups, df_filtered, title="Resultado final con BFS + bounding box merge")


## version combinada

In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression
import copy
from scipy.spatial import ConvexHull
from matplotlib.path import Path

def plot_intermediate_state_cartesian(state, df_all):
    """
    Grafica un estado en el plano (theta, r).
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=5, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan', 'gold', 'darkblue']
    
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=15, color=color, label=f"Grupo {idx+1}")
        # Dibujar el convex hull si hay suficientes puntos
        if len(pts) >= 3:
            pts_arr = pts[['theta','r']].values
            try:
                hull = ConvexHull(pts_arr)
                hull_pts = pts_arr[hull.vertices]
                hull_pts = np.concatenate([hull_pts, hull_pts[:1]], axis=0)  # Cerrar polígono
                plt.plot(hull_pts[:,0], hull_pts[:,1], '-', color=color, lw=2)
            except Exception:
                pass
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def update_group_limits(group):
    """
    Recalcula los límites (theta_min, theta_max, r_min, r_max) usando el convex hull si hay 3 o más puntos;
    de lo contrario, usa la bounding box.
    """
    pts = group['points']
    if len(pts) >= 3:
        arr = pts[['theta','r']].values
        try:
            hull = ConvexHull(arr)
            hull_pts = arr[hull.vertices]
            group['theta_min'] = hull_pts[:,0].min()
            group['theta_max'] = hull_pts[:,0].max()
            group['r_min'] = hull_pts[:,1].min()
            group['r_max'] = hull_pts[:,1].max()
            return
        except Exception:
            pass
    group['theta_min'] = pts['theta'].min()
    group['theta_max'] = pts['theta'].max()
    group['r_min'] = pts['r'].min()
    group['r_max'] = pts['r'].max()

def calculate_pa(slope, intercept):
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.nan

def merge_all_groups(groups, theta_merge_threshold, r_merge_threshold):
    """
    Fusiona grupos si, al extender sus bounding boxes en un 10%, se superponen y la variación
    en la pendiente (del ajuste lineal de cada grupo) es ≤ 15%.
    La fusión se realiza actualizando el ajuste lineal sobre el grupo combinado.
    """
    merged_flag = True
    while merged_flag:
        merged_flag = False
        i = 0
        while i < len(groups):
            j = i + 1
            while j < len(groups):
                g1 = groups[i]
                g2 = groups[j]
                # Límites originales de g1 y g2
                th_min1, th_max1, r_min1, r_max1 = g1['theta_min'], g1['theta_max'], g1['r_min'], g1['r_max']
                th_min2, th_max2, r_min2, r_max2 = g2['theta_min'], g2['theta_max'], g2['r_min'], g2['r_max']
                
                # Calcular rangos y extender en un 10%
                range_th1 = th_max1 - th_min1 if th_max1 - th_min1 != 0 else 1e-3
                range_r1 = r_max1 - r_min1 if r_max1 - r_min1 != 0 else 1e-3
                ext_th_min1 = th_min1 - 0.1 * range_th1
                ext_th_max1 = th_max1 + 0.1 * range_th1
                ext_r_min1  = r_min1 - 0.1 * range_r1
                ext_r_max1  = r_max1 + 0.1 * range_r1
                
                range_th2 = th_max2 - th_min2 if th_max2 - th_min2 != 0 else 1e-3
                range_r2 = r_max2 - r_min2 if r_max2 - r_min2 != 0 else 1e-3
                ext_th_min2 = th_min2 - 0.1 * range_th2
                ext_th_max2 = th_max2 + 0.1 * range_th2
                ext_r_min2  = r_min2 - 0.1 * range_r2
                ext_r_max2  = r_max2 + 0.1 * range_r2
                
                # Comprobar si las cajas extendidas se superponen:
                overlap_theta = not (ext_th_max1 < ext_th_min2 or ext_th_min1 > ext_th_max2)
                overlap_r = not (ext_r_max1 < ext_r_min2 or ext_r_min1 > ext_r_max2)
                
                # Comprobar variación relativa de la pendiente:
                slope1 = g1['slope']
                slope2 = g2['slope']
                if slope1 is None or slope2 is None:
                    rel_variation = 0  # Si alguno no tiene ajuste, se permite fusionar
                else:
                    rel_variation = abs(slope1 - slope2) / (abs(slope1) + 1e-12)
                
                if overlap_theta and overlap_r and rel_variation <= 0.15:
                    # Fusionar g2 en g1:
                    merged_points = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    # Recalcular el ajuste lineal para el grupo combinado:
                    model = LinearRegression()
                    X_new = merged_points[['theta']].values
                    y_new = merged_points['r'].values
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    new_intercept = model.intercept_
                    g1['points'] = merged_points
                    g1['slope'] = new_slope
                    g1['intercept'] = new_intercept
                    g1['pa'] = calculate_pa(new_slope, new_intercept)
                    update_group_limits(g1)
                    groups.pop(j)
                    merged_flag = True
                else:
                    j += 1
            i += 1

def add_point_strict_convex(point, groups, theta_strict_window, r_strict_window, r_gap=2.0):
    """
    Agrega 'point' a un grupo existente de forma estricta:
      - Si el grupo tiene ≥3 puntos, se calcula su convex hull y se verifica si el punto está DENTRO
        (sin margen).
      - Para grupos con <3 puntos, se usa una bounding box reducida (la mitad de la ventana).
      - Se verifica que el nuevo punto no exceda r_gap en la coordenada r.
    """
    theta_pt = point['theta']
    r_pt = point['r']
    
    for g in groups:
        if r_pt < g['r_min'] - r_gap or r_pt > g['r_max'] + r_gap:
            continue
        
        pts = g['points']
        if len(pts) >= 3:
            arr = pts[['theta','r']].values
            try:
                hull = ConvexHull(arr)
                hull_pts = arr[hull.vertices]
                poly = Path(hull_pts, closed=True)
                inside = poly.contains_point([theta_pt, r_pt], radius=-1e-9)
                if inside:
                    g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                    update_group_limits(g)
                    return True
            except Exception:
                pass
            # Si falla, no se agrega.
        else:
            half_theta = theta_strict_window / 2.0
            half_r = r_strict_window / 2.0
            t_min = pts['theta'].min()
            t_max = pts['theta'].max()
            rr_min = pts['r'].min()
            rr_max = pts['r'].max()
            if (theta_pt >= t_min - half_theta and theta_pt <= t_max + half_theta and
                r_pt >= rr_min - half_r and r_pt <= rr_max + half_r):
                g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                update_group_limits(g)
                return True
    return False

def process_spiral_arms_by_proximity(
    id_halo,
    quartile_threshold=0.55,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    theta_merge_threshold=7.0,
    r_merge_threshold=7.0,
    r_gap=2.0,
    snapshot_interval=10
):
    """
    Procesa datos y agrupa puntos de forma estricta con convex hull:
      1) Filtra por percentil y por theta (50-250).
      2) Recorre de izquierda a derecha (orden ascendente por theta).
      3) Para cada punto, se intenta agregar a un grupo:
         - Si el grupo tiene ≥3 puntos, se exige que el punto esté estrictamente dentro del hull.
         - Para grupos con <3 puntos, se usa una bounding box reducida (la mitad de la ventana).
         - Se verifica que no haya un salto en r mayor a r_gap.
      4) Si no se agrega, se crea un grupo nuevo.
      5) Al final se fusionan grupos: se extienden sus bounding boxes en 10% y se fusionan si
         hay superposición y la variación en la pendiente es ≤15%.
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= 50) & (df['theta'] <= 250)].copy()
    
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='theta', inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # Inicializar grupos a partir de clusters de DBSCAN
    clusters_tolerance = 0.70
    def merge_close_clusters(clusters, clusters_tolerance):
        merged_clusters = {}
        used_clusters = set()
        for id1, points1 in clusters.items():
            if id1 in used_clusters:
                continue
            merged_points = points1.copy()
            for id2, points2 in clusters.items():
                if id1 >= id2 or id2 in used_clusters:
                    continue
                min_distance = np.min(np.sqrt(
                    (points1['r'].values[:, None] - points2['r'].values)**2 +
                    (points1['theta'].values[:, None] - points2['theta'].values)**2
                ))
                if min_distance < clusters_tolerance:
                    merged_points = pd.concat([merged_points, points2], ignore_index=True)
                    used_clusters.add(id2)
            merged_clusters[id1] = merged_points
            used_clusters.add(id1)
        return merged_clusters

    clusters_polar = {}
    for cluster_id in np.unique(df_filtered['cluster']):
        if cluster_id == -1:
            continue
        cluster_points = df_filtered[df_filtered['cluster'] == cluster_id].copy()
        clusters_polar[cluster_id] = cluster_points[['r', 'theta']]
    clusters_polar = merge_close_clusters(clusters_polar, clusters_tolerance)
    
    model = LinearRegression()
    def calculate_distance(slope, intercept, theta, r):
        pred_r = slope * theta + intercept
        return abs(pred_r - r)
    
    def is_valid_pa(pa, valid_range):
        return valid_range[0] <= abs(pa) <= valid_range[1]
    
    groups = []
    for cid, pts in clusters_polar.items():
        if len(pts) >= 3:
            X_ = pts[['theta']].values
            y_ = pts['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'cluster_id': cid,
            'points': pts.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # Asignación iterativa de puntos
    df_local = df_filtered.copy()
    discarded_points = []
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        assigned = False
        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue
            # Condición extra: el punto debe estar dentro del rango horizontal del grupo extendido ±theta_margin
            group_theta_min = g['points']['theta'].min()
            group_theta_max = g['points']['theta'].max()
            theta_margin = 0.10 * (group_theta_max - group_theta_min)  # 10% de extensión
            if not (group_theta_min - theta_margin <= theta_pt <= group_theta_max + theta_margin):
                continue
            dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
            if dist <= radius_tolerance:
                # Verificar que la distancia de todos los puntos al ajuste sea menor a point_distance_tolerance
                distances = g['points'].apply(
                    lambda row: calculate_distance(g['slope'], g['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if all(d <= point_distance_tolerance for d in distances):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    # Fusionar solo si la variación en pendiente es ≤15%
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= 0.15:
                        g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(g['slope'], g['intercept'])
                        assigned = True
                        break
        if not assigned:
            discarded_points.append(point.to_dict())
    
    # Reasignar descartados
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            theta_pt = p['theta']
            r_pt = p['r']
            assigned = False
            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                group_theta_min = g['points']['theta'].min()
                group_theta_max = g['points']['theta'].max()
                theta_margin = 0.10 * (group_theta_max - group_theta_min)
                if not (group_theta_min - theta_margin <= theta_pt <= group_theta_max + theta_margin):
                    continue
                dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
                if dist <= radius_tolerance and g['pa'] is not None and is_valid_pa(g['pa'], flexible_pa_range):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= 0.15:
                        g['points'] = pd.concat([g['points'], p.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(new_slope, g['intercept'])
                        assigned = True
                        break
            if not assigned:
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new, i_new)
                if is_valid_pa(pa_new, flexible_pa_range):
                    groups.append({
                        'cluster_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })

    # Subdivisión opcional si hay un gap grande en theta
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_

    def subdivide_if_gap(group, max_gap=max_theta_gap):
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)
        seg1 = {
            'cluster_id': group['cluster_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'cluster_id': group['cluster_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]

    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap=max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups

    # Filtrar grupos según PA
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and is_valid_pa(pa_val, flexible_pa_range):
                valid_groups.append(grp)

    return valid_groups, df

###############################################################################
# EJEMPLO DE EJECUCIÓN Y GRAFICACIÓN
###############################################################################
if __name__ == "__main__":
    id_halo = "17"
    quartile_threshold = 0.155
    radius_tolerance = 0.45
    point_distance_tolerance = 0.59
    flexible_pa_range = (4.0, 30.0)
    eps_polar = 0.7
    min_samples_polar = 18
    slope_variation_threshold = 0.20  # 20%
    max_theta_gap = 15.0
    do_subdivide = True

    valid_groups, df = process_spiral_arms_polar_intermediate(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        radius_tolerance=radius_tolerance,
        point_distance_tolerance=point_distance_tolerance,
        flexible_pa_range=flexible_pa_range,
        eps_polar=eps_polar,
        min_samples_polar=min_samples_polar,
        slope_variation_threshold=slope_variation_threshold,
        max_theta_gap=max_theta_gap,
        do_subdivide=do_subdivide
    )

    print("Total de grupos finales:", len(valid_groups))

    # Gráfica en (theta vs r)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['theta'], df['r'], s=1, color='silver', alpha=0.5, label='Datos Originales')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']

    for idx, grp in enumerate(valid_groups):
        pts = grp['points']
        theta_vals = pts['theta']
        r_vals = pts['r']
        pa_val = grp['pa'] if grp['pa'] is not None else 0.0
        color = colors[idx % len(colors)]
        plt.scatter(theta_vals, r_vals, s=10, alpha=0.8, color=color,
                    label=f"Grupo {idx+1}, PA={pa_val:.2f}")
        if grp['slope'] is not None:
            theta_line = np.linspace(theta_vals.min(), theta_vals.max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)

    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Ajustes intermedios en (r, θ) - Halo {id_halo}")
    plt.grid(True)
    plt.show()

    # Gráfica polar
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 8))
    theta_all_rad = np.radians(df['theta'])
    ax.scatter(theta_all_rad, df['r'], s=1, color='gray', alpha=0.5, label='Datos Originales')

    for idx, grp in enumerate(valid_groups):
        pts = grp['points']
        theta_rad = np.radians(pts['theta'])
        r_vals = pts['r']
        pa_val = grp['pa'] if grp['pa'] is not None else 0.0
        color = colors[idx % len(colors)]
        ax.scatter(theta_rad, r_vals, s=10, alpha=0.8, color=color,
                   label=f"Grupo {idx+1} PA: {pa_val:.2f}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)

    ax.set_title(f"Ajustes intermedios en proyección polar - Halo {id_halo}", fontsize=14, pad=20)
    plt.show()


In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.spatial import ConvexHull
from matplotlib.path import Path
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression

def plot_intermediate_state_cartesian(state, df_all):
    """
    Grafica un estado en el plano (θ, r).
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=5, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan', 'gold', 'darkblue']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=15, color=color, label=f"Grupo {idx+1}")
        if len(pts) >= 3:
            pts_arr = pts[['theta','r']].values
            try:
                hull = ConvexHull(pts_arr)
                hull_pts = pts_arr[hull.vertices]
                hull_pts = np.concatenate([hull_pts, hull_pts[:1]], axis=0)  # Cerrar el polígono
                plt.plot(hull_pts[:,0], hull_pts[:,1], '-', color=color, lw=2)
            except Exception:
                pass
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def update_group_limits(group):
    """
    Recalcula los límites (θ_min, θ_max, r_min, r_max) usando el convex hull si hay ≥3 puntos;
    en caso contrario, usa la bounding box simple.
    """
    pts = group['points']
    if len(pts) >= 3:
        arr = pts[['theta','r']].values
        try:
            hull = ConvexHull(arr)
            hull_pts = arr[hull.vertices]
            group['theta_min'] = hull_pts[:,0].min()
            group['theta_max'] = hull_pts[:,0].max()
            group['r_min'] = hull_pts[:,1].min()
            group['r_max'] = hull_pts[:,1].max()
            return
        except Exception:
            pass
    group['theta_min'] = pts['theta'].min()
    group['theta_max'] = pts['theta'].max()
    group['r_min'] = pts['r'].min()
    group['r_max'] = pts['r'].max()

def calculate_pa(slope, intercept):
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.nan

def point_in_hull(point, hull_points):
    """
    Verifica si un punto [θ, r] está estrictamente dentro del polígono definido por hull_points.
    """
    poly = Path(hull_points, closed=True)
    return poly.contains_point(point, radius=-1e-9)  # Usamos un radio negativo para mayor estrictidad

def merge_all_groups(groups, theta_merge_threshold, r_merge_threshold):
    """
    Fusiona grupos si, al extender sus límites en un 10%, se superponen y la variación
    relativa en la pendiente es ≤15%.
    
    Para cada grupo se calculan los límites extendidos:
      ext_θ_min = θ_min - 0.1*(θ_max-θ_min)
      ext_θ_max = θ_max + 0.1*(θ_max-θ_min)
      (similar para r)
    
    Se fusionan dos grupos si sus cajas extendidas se superponen en ambas dimensiones
    y la variación en pendiente es ≤ 0.15.
    """
    merged_flag = True
    while merged_flag:
        merged_flag = False
        i = 0
        while i < len(groups):
            j = i + 1
            while j < len(groups):
                g1 = groups[i]
                g2 = groups[j]
                # Límites originales
                th_min1, th_max1, r_min1, r_max1 = g1['theta_min'], g1['theta_max'], g1['r_min'], g1['r_max']
                th_min2, th_max2, r_min2, r_max2 = g2['theta_min'], g2['theta_max'], g2['r_min'], g2['r_max']
                
                # Calcular rangos y extender en 10%
                range_th1 = th_max1 - th_min1 if (th_max1 - th_min1) != 0 else 1e-3
                range_r1  = r_max1 - r_min1 if (r_max1 - r_min1) != 0 else 1e-3
                ext_th_min1 = th_min1 - 0.1 * range_th1
                ext_th_max1 = th_max1 + 0.1 * range_th1
                ext_r_min1  = r_min1 - 0.1 * range_r1
                ext_r_max1  = r_max1 + 0.1 * range_r1
                
                range_th2 = th_max2 - th_min2 if (th_max2 - th_min2) != 0 else 1e-3
                range_r2  = r_max2 - r_min2 if (r_max2 - r_min2) != 0 else 1e-3
                ext_th_min2 = th_min2 - 0.1 * range_th2
                ext_th_max2 = th_max2 + 0.1 * range_th2
                ext_r_min2  = r_min2 - 0.1 * range_r2
                ext_r_max2  = r_max2 + 0.1 * range_r2
                
                # Superposición de las cajas extendidas
                overlap_theta = not (ext_th_max1 < ext_th_min2 or ext_th_min1 > ext_th_max2)
                overlap_r = not (ext_r_max1 < ext_r_min2 or ext_r_min1 > ext_r_max2)
                
                # Variación relativa en pendiente
                if g1['slope'] is None or g2['slope'] is None:
                    rel_variation = 0
                else:
                    rel_variation = abs(g1['slope'] - g2['slope']) / (abs(g1['slope']) + 1e-12)
                
                if overlap_theta and overlap_r and rel_variation <= 0.15:
                    # Fusionar: unir puntos y recalcular ajuste
                    merged_points = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    model = LinearRegression()
                    X_new = merged_points[['theta']].values
                    y_new = merged_points['r'].values
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    new_intercept = model.intercept_
                    g1['points'] = merged_points
                    g1['slope'] = new_slope
                    g1['intercept'] = new_intercept
                    g1['pa'] = calculate_pa(new_slope, new_intercept)
                    update_group_limits(g1)
                    groups.pop(j)
                    merged_flag = True
                else:
                    j += 1
            i += 1

def add_point_strict_convex(point, groups, theta_strict_window, r_strict_window, r_gap=2.0):
    """
    Intenta agregar 'point' a un grupo existente de forma estricta:
      - Para grupos con ≥3 puntos se calcula el convex hull y se exige que el punto esté estrictamente dentro.
      - Para grupos con <3 puntos se usa una bounding box reducida (usando la mitad de theta_strict_window y r_strict_window).
      - Se verifica que el nuevo punto no exceda r_gap en r.
    """
    theta_pt = point['theta']
    r_pt = point['r']
    
    for g in groups:
        if r_pt < g['r_min'] - r_gap or r_pt > g['r_max'] + r_gap:
            continue
        pts = g['points']
        if len(pts) >= 3:
            arr = pts[['theta','r']].values
            try:
                hull = ConvexHull(arr)
                hull_pts = arr[hull.vertices]
                poly = Path(hull_pts, closed=True)
                inside = poly.contains_point([theta_pt, r_pt], radius=-1e-9)
                if inside:
                    g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                    update_group_limits(g)
                    return True
            except Exception:
                pass
        else:
            half_theta = theta_strict_window / 2.0
            half_r = r_strict_window / 2.0
            t_min = pts['theta'].min()
            t_max = pts['theta'].max()
            rr_min = pts['r'].min()
            rr_max = pts['r'].max()
            if (theta_pt >= t_min - half_theta and theta_pt <= t_max + half_theta and
                r_pt >= rr_min - half_r and r_pt <= rr_max + half_r):
                g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                update_group_limits(g)
                return True
    return False

def process_spiral_arms_by_proximity(
    id_halo,
    quartile_threshold=0.55,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    theta_merge_threshold=7.0,
    r_merge_threshold=7.0,
    r_gap=2.0,
    snapshot_interval=10
):
    """
    Procesa datos y agrupa puntos de forma estricta usando convex hull, recorriendo de izquierda a derecha.
    1) Se filtra por percentil y por θ (50-250).
    2) Se ordena el DataFrame por θ (izquierda a derecha).
    3) Para cada punto se intenta agregar a un grupo:
         - Para grupos con ≥3 puntos se exige que el punto esté estrictamente dentro del convex hull.
         - Para grupos con <3 puntos se usa una bounding box reducida (la mitad de la ventana).
         - Se verifica que no haya un salto en r mayor a r_gap.
         - Además, el punto debe estar dentro del rango horizontal actual extendido en ±10% del rango.
    4) Si no se agrega, se crea un grupo nuevo.
    5) Finalmente, se fusionan grupos extendiendo sus límites en un 10% y fusionando solo si la variación en pendiente es ≤15%.
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= 50) & (df['theta'] <= 250)].copy()
    
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='theta', inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)

    X_polar = df_filtered[['r', 'theta']].values
    dbscan = DBSCAN(eps=eps_polar, min_samples=min_samples_polar)
    df_filtered['cluster'] = dbscan.fit_predict(X_polar)

    
    # Inicializar grupos a partir de clusters de DBSCAN fusionados
    clusters_tolerance = 0.70
    def merge_close_clusters(clusters, clusters_tolerance):
        merged_clusters = {}
        used_clusters = set()
        for id1, points1 in clusters.items():
            if id1 in used_clusters:
                continue
            merged_points = points1.copy()
            for id2, points2 in clusters.items():
                if id1 >= id2 or id2 in used_clusters:
                    continue
                min_distance = np.min(np.sqrt(
                    (points1['r'].values[:, None] - points2['r'].values)**2 +
                    (points1['theta'].values[:, None] - points2['theta'].values)**2
                ))
                if min_distance < clusters_tolerance:
                    merged_points = pd.concat([merged_points, points2], ignore_index=True)
                    used_clusters.add(id2)
            merged_clusters[id1] = merged_points
            used_clusters.add(id1)
        return merged_clusters

    clusters_polar = {}
    for cluster_id in np.unique(df_filtered['cluster']):
        if cluster_id == -1:
            continue
        cluster_points = df_filtered[df_filtered['cluster'] == cluster_id].copy()
        clusters_polar[cluster_id] = cluster_points[['r', 'theta']]
    clusters_polar = merge_close_clusters(clusters_polar, clusters_tolerance)
    
    model = LinearRegression()
    def calculate_distance(slope, intercept, theta, r):
        pred_r = slope * theta + intercept
        return abs(pred_r - r)
    
    def is_valid_pa(pa, valid_range):
        return valid_range[0] <= abs(pa) <= valid_range[1]
    
    groups = []
    for cid, pts in clusters_polar.items():
        if len(pts) >= 3:
            X_ = pts[['theta']].values
            y_ = pts['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'cluster_id': cid,
            'points': pts.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # Recorrido de izquierda a derecha: se ordena por θ
    df_local = df_filtered.copy()
    df_local.sort_values(by='theta', inplace=True)
    df_local.reset_index(drop=True, inplace=True)
    
    discarded_points = []
    iteration = 0
    while len(df_local) > 0:
        # Siempre se toma el primer punto (el de menor θ)
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        assigned = False
        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue
            group_theta_min = g['points']['theta'].min()
            group_theta_max = g['points']['theta'].max()
            theta_range = group_theta_max - group_theta_min if group_theta_max - group_theta_min > 0 else 1e-3
            theta_margin = 0.10 * theta_range  # extensión del 10%
            if not (group_theta_min - theta_margin <= theta_pt <= group_theta_max + theta_margin):
                continue
            dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
            if dist <= radius_tolerance:
                distances = g['points'].apply(
                    lambda row: calculate_distance(g['slope'], g['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if all(d <= point_distance_tolerance for d in distances):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= 0.15:
                        g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(g['slope'], g['intercept'])
                        assigned = True
                        break
        if not assigned:
            discarded_points.append(point.to_dict())
        iteration += 1

    # Reasignar descartados
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            theta_pt = p['theta']
            r_pt = p['r']
            assigned = False
            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                group_theta_min = g['points']['theta'].min()
                group_theta_max = g['points']['theta'].max()
                theta_range = group_theta_max - group_theta_min if group_theta_max - group_theta_min > 0 else 1e-3
                theta_margin = 0.10 * theta_range
                if not (group_theta_min - theta_margin <= theta_pt <= group_theta_max + theta_margin):
                    continue
                dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
                if dist <= radius_tolerance and g['pa'] is not None and is_valid_pa(g['pa'], flexible_pa_range):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= 0.15:
                        g['points'] = pd.concat([g['points'], p.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(new_slope, g['intercept'])
                        assigned = True
                        break
            if not assigned:
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new, i_new)
                if is_valid_pa(pa_new, flexible_pa_range):
                    groups.append({
                        'cluster_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })

    # Subdivisión opcional en caso de gap grande en θ
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    def subdivide_if_gap(group, max_gap=max_theta_gap):
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)
        seg1 = {
            'cluster_id': group['cluster_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'cluster_id': group['cluster_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]
    
    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap=max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups

    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and is_valid_pa(pa_val, flexible_pa_range):
                valid_groups.append(grp)

    return valid_groups, df

###############################################################################
# EJEMPLO DE EJECUCIÓN Y GRAFICACIÓN
###############################################################################
if __name__ == "__main__":
    id_halo = "17"
    quartile_threshold = 0.155
    radius_tolerance = 0.45
    point_distance_tolerance = 0.59
    flexible_pa_range = (4.0, 30.0)
    eps_polar = 0.7
    min_samples_polar = 18
    slope_variation_threshold = 0.20  # 20%
    max_theta_gap = 15.0
    do_subdivide = True

    valid_groups, df = process_spiral_arms_by_proximity(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_fit_window=30.0,
        r_fit_window=3.0,
        theta_merge_threshold=10.0,
        r_merge_threshold=2.0,
        r_gap=2.0,
        snapshot_interval=10
    )
    print("Total de grupos finales:", len(valid_groups))
    
    # Gráfica en (θ vs r)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['theta'], df['r'], s=1, color='silver', alpha=0.5, label='Datos Originales')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, grp in enumerate(valid_groups):
        pts = grp['points']
        theta_vals = pts['theta']
        r_vals = pts['r']
        pa_val = grp['pa'] if grp['pa'] is not None else 0.0
        color = colors[idx % len(colors)]
        plt.scatter(theta_vals, r_vals, s=10, alpha=0.8, color=color,
                    label=f"Grupo {idx+1}, PA={pa_val:.2f}")
        if grp['slope'] is not None:
            theta_line = np.linspace(theta_vals.min(), theta_vals.max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Ajustes intermedios en (r, θ) - Halo {id_halo}")
    plt.grid(True)
    plt.show()

    # Gráfica en proyección polar
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 8))
    theta_all_rad = np.radians(df['theta'])
    ax.scatter(theta_all_rad, df['r'], s=1, color='gray', alpha=0.5, label='Datos Originales')
    for idx, grp in enumerate(valid_groups):
        pts = grp['points']
        theta_rad = np.radians(pts['theta'])
        r_vals = pts['r']
        pa_val = grp['pa'] if grp['pa'] is not None else 0.0
        color = colors[idx % len(colors)]
        ax.scatter(theta_rad, r_vals, s=10, alpha=0.8, color=color,
                   label=f"Grupo {idx+1} PA: {pa_val:.2f}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)
    ax.set_title(f"Ajustes intermedios en proyección polar - Halo {id_halo}", fontsize=14, pad=20)
    plt.show()


In [0]:
from sklearn.cluster import Birch

# Preparar la matriz de características en el espacio polar:
X_polar = df_filtered[['r', 'theta']].values

# Configurar Birch: 
# - 'threshold' controla la distancia máxima entre puntos en un subcluster.
# - 'n_clusters=None' hace que el modelo forme tantos subclusters como sea necesario.
# - 'branching_factor' se puede usar para ajustar la estructura del árbol (opcional).
birch = Birch(threshold=eps_polar, n_clusters=None, branching_factor=min_samples_polar)
df_filtered['cluster'] = birch.fit_predict(X_polar)


In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.spatial import ConvexHull
from matplotlib.path import Path
from sklearn.cluster import Birch
from sklearn.linear_model import LinearRegression

def plot_intermediate_state_cartesian(state, df_all):
    """
    Grafica un estado en el plano (θ, r).
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=5, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan', 'gold', 'darkblue']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=15, color=color, label=f"Grupo {idx+1}")
        if len(pts) >= 3:
            pts_arr = pts[['theta','r']].values
            try:
                hull = ConvexHull(pts_arr)
                hull_pts = pts_arr[hull.vertices]
                hull_pts = np.concatenate([hull_pts, hull_pts[:1]], axis=0)
                plt.plot(hull_pts[:,0], hull_pts[:,1], '-', color=color, lw=2)
            except Exception:
                pass
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def update_group_limits(group):
    pts = group['points']
    if len(pts) >= 3:
        arr = pts[['theta','r']].values
        try:
            hull = ConvexHull(arr)
            hull_pts = arr[hull.vertices]
            group['theta_min'] = hull_pts[:,0].min()
            group['theta_max'] = hull_pts[:,0].max()
            group['r_min'] = hull_pts[:,1].min()
            group['r_max'] = hull_pts[:,1].max()
            return
        except Exception:
            pass
    group['theta_min'] = pts['theta'].min()
    group['theta_max'] = pts['theta'].max()
    group['r_min'] = pts['r'].min()
    group['r_max'] = pts['r'].max()

def calculate_pa(slope, intercept):
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.nan

def point_in_hull(point, hull_points):
    poly = Path(hull_points, closed=True)
    return poly.contains_point(point, radius=-1e-9)

def merge_all_groups(groups, theta_merge_threshold, r_merge_threshold):
    merged_flag = True
    while merged_flag:
        merged_flag = False
        i = 0
        while i < len(groups):
            j = i + 1
            while j < len(groups):
                g1 = groups[i]
                g2 = groups[j]
                th_min1, th_max1, r_min1, r_max1 = g1['theta_min'], g1['theta_max'], g1['r_min'], g1['r_max']
                th_min2, th_max2, r_min2, r_max2 = g2['theta_min'], g2['theta_max'], g2['r_min'], g2['r_max']
                range_th1 = th_max1 - th_min1 if (th_max1 - th_min1) != 0 else 1e-3
                range_r1  = r_max1 - r_min1 if (r_max1 - r_min1) != 0 else 1e-3
                ext_th_min1 = th_min1 - 0.1 * range_th1
                ext_th_max1 = th_max1 + 0.1 * range_th1
                ext_r_min1  = r_min1 - 0.1 * range_r1
                ext_r_max1  = r_max1 + 0.1 * range_r1
                range_th2 = th_max2 - th_min2 if (th_max2 - th_min2) != 0 else 1e-3
                range_r2  = r_max2 - r_min2 if (r_max2 - r_min2) != 0 else 1e-3
                ext_th_min2 = th_min2 - 0.1 * range_th2
                ext_th_max2 = th_max2 + 0.1 * range_th2
                ext_r_min2  = r_min2 - 0.1 * range_r2
                ext_r_max2  = r_max2 + 0.1 * range_r2
                overlap_theta = not (ext_th_max1 < ext_th_min2 or ext_th_min1 > ext_th_max2)
                overlap_r = not (ext_r_max1 < ext_r_min2 or ext_r_min1 > ext_r_max2)
                if g1['slope'] is None or g2['slope'] is None:
                    rel_variation = 0
                else:
                    rel_variation = abs(g1['slope'] - g2['slope']) / (abs(g1['slope']) + 1e-12)
                if overlap_theta and overlap_r and rel_variation <= 0.15:
                    merged_points = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    model = LinearRegression()
                    X_new = merged_points[['theta']].values
                    y_new = merged_points['r'].values
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    new_intercept = model.intercept_
                    g1['points'] = merged_points
                    g1['slope'] = new_slope
                    g1['intercept'] = new_intercept
                    g1['pa'] = calculate_pa(new_slope, new_intercept)
                    update_group_limits(g1)
                    groups.pop(j)
                    merged_flag = True
                else:
                    j += 1
            i += 1

def add_point_strict_convex(point, groups, theta_strict_window, r_strict_window, r_gap=2.0):
    theta_pt = point['theta']
    r_pt = point['r']
    for g in groups:
        if r_pt < g['r_min'] - r_gap or r_pt > g['r_max'] + r_gap:
            continue
        pts = g['points']
        if len(pts) >= 3:
            arr = pts[['theta','r']].values
            try:
                hull = ConvexHull(arr)
                hull_pts = arr[hull.vertices]
                poly = Path(hull_pts, closed=True)
                inside = poly.contains_point([theta_pt, r_pt], radius=-1e-9)
                if inside:
                    g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                    update_group_limits(g)
                    return True
            except Exception:
                pass
        else:
            half_theta = theta_strict_window / 2.0
            half_r = r_strict_window / 2.0
            t_min = pts['theta'].min()
            t_max = pts['theta'].max()
            rr_min = pts['r'].min()
            rr_max = pts['r'].max()
            if (theta_pt >= t_min - half_theta and theta_pt <= t_max + half_theta and
                r_pt >= rr_min - half_r and r_pt <= rr_max + half_r):
                g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                update_group_limits(g)
                return True
    return False

def process_spiral_arms_by_proximity(
    id_halo,
    quartile_threshold=0.55,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    theta_merge_threshold=7.0,
    r_merge_threshold=7.0,
    r_gap=2.0,
    snapshot_interval=10
):
    """
    Procesa datos y agrupa puntos de forma estricta con convex hull, recorriendo de izquierda a derecha.
    1) Se filtra por percentil y por θ (50-250).
    2) Se ordena el DataFrame por θ (recorrido de izquierda a derecha).
    3) Para cada punto se intenta agregar a un grupo:
         - Para grupos con ≥3 puntos se exige que el punto esté estrictamente dentro del convex hull.
         - Para grupos con <3 puntos se usa una bounding box reducida (la mitad de la ventana).
         - Se verifica que no haya un salto en r mayor a r_gap.
         - Además, el punto debe estar dentro del rango horizontal actual extendido en ±10% del rango.
    4) Si no se agrega, se crea un grupo nuevo.
    5) Finalmente, se fusionan grupos extendiendo sus límites en un 10% y fusionando solo si la variación en pendiente es ≤15%.
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= 50) & (df['theta'] <= 250)].copy()
    
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='theta', inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # Inicializar grupos a partir de clusters de BIRCH o DBSCAN (aquí usamos DBSCAN como base)
    # En este ejemplo se asume que df_filtered tiene la columna 'cluster' ya asignada.
    # Si no existe, puedes asignarla usando:
    # from sklearn.cluster import Birch
    # birch = Birch(threshold=eps_polar, n_clusters=None, branching_factor=min_samples_polar)
    # df_filtered['cluster'] = birch.fit_predict(df_filtered[['r','theta']].values)
    if 'cluster' not in df_filtered.columns:
        from sklearn.cluster import Birch
        birch = Birch(threshold=0.7, n_clusters=None, branching_factor=min_samples_polar)
        df_filtered['cluster'] = birch.fit_predict(df_filtered[['r','theta']].values)
    
    clusters_tolerance = 0.70
    def merge_close_clusters(clusters, clusters_tolerance):
        merged_clusters = {}
        used_clusters = set()
        for id1, points1 in clusters.items():
            if id1 in used_clusters:
                continue
            merged_points = points1.copy()
            for id2, points2 in clusters.items():
                if id1 >= id2 or id2 in used_clusters:
                    continue
                min_distance = np.min(np.sqrt(
                    (points1['r'].values[:, None] - points2['r'].values)**2 +
                    (points1['theta'].values[:, None] - points2['theta'].values)**2
                ))
                if min_distance < clusters_tolerance:
                    merged_points = pd.concat([merged_points, points2], ignore_index=True)
                    used_clusters.add(id2)
            merged_clusters[id1] = merged_points
            used_clusters.add(id1)
        return merged_clusters

    clusters_polar = {}
    for cluster_id in np.unique(df_filtered['cluster']):
        if cluster_id == -1:
            continue
        cluster_points = df_filtered[df_filtered['cluster'] == cluster_id].copy()
        clusters_polar[cluster_id] = cluster_points[['r', 'theta']]
    clusters_polar = merge_close_clusters(clusters_polar, clusters_tolerance)
    
    model = LinearRegression()
    def calculate_distance(slope, intercept, theta, r):
        pred_r = slope * theta + intercept
        return abs(pred_r - r)
    
    def is_valid_pa(pa, valid_range):
        return valid_range[0] <= abs(pa) <= valid_range[1]
    
    groups = []
    for cid, pts in clusters_polar.items():
        if len(pts) >= 3:
            X_ = pts[['theta']].values
            y_ = pts['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'cluster_id': cid,
            'points': pts.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # Recorrido de izquierda a derecha
    df_local = df_filtered.copy()
    df_local.sort_values(by='theta', inplace=True)
    df_local.reset_index(drop=True, inplace=True)
    
    discarded_points = []
    iteration = 0
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        assigned = False
        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue
            group_theta_min = g['points']['theta'].min()
            group_theta_max = g['points']['theta'].max()
            theta_range = group_theta_max - group_theta_min if group_theta_max - group_theta_min > 0 else 1e-3
            theta_margin = 0.10 * theta_range
            if not (group_theta_min - theta_margin <= theta_pt <= group_theta_max + theta_margin):
                continue
            dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
            if dist <= radius_tolerance:
                distances = g['points'].apply(
                    lambda row: calculate_distance(g['slope'], g['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if all(d <= point_distance_tolerance for d in distances):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= 0.15:
                        g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(g['slope'], g['intercept'])
                        assigned = True
                        break
        if not assigned:
            discarded_points.append(point.to_dict())
        iteration += 1

    # Reasignar descartados
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            theta_pt = p['theta']
            r_pt = p['r']
            assigned = False
            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                group_theta_min = g['points']['theta'].min()
                group_theta_max = g['points']['theta'].max()
                theta_range = group_theta_max - group_theta_min if group_theta_max - group_theta_min > 0 else 1e-3
                theta_margin = 0.10 * theta_range
                if not (group_theta_min - theta_margin <= theta_pt <= group_theta_max + theta_margin):
                    continue
                dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
                if dist <= radius_tolerance and g['pa'] is not None and is_valid_pa(g['pa'], flexible_pa_range):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= 0.15:
                        g['points'] = pd.concat([g['points'], p.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(new_slope, g['intercept'])
                        assigned = True
                        break
            if not assigned:
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new, i_new)
                if is_valid_pa(pa_new, flexible_pa_range):
                    groups.append({
                        'cluster_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })

    # Subdivisión opcional en caso de gap grande en θ
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_

    def subdivide_if_gap(group, max_gap=max_theta_gap):
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)
        seg1 = {
            'cluster_id': group['cluster_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'cluster_id': group['cluster_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]

    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap=max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups

    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and is_valid_pa(pa_val, flexible_pa_range):
                valid_groups.append(grp)

    return valid_groups, df

###############################################################################
# EJEMPLO DE EJECUCIÓN Y GRAFICACIÓN
###############################################################################
if __name__ == "__main__":
    id_halo = "17"
    quartile_threshold = 0.155
    radius_tolerance = 0.45
    point_distance_tolerance = 0.59
    flexible_pa_range = (4.0, 30.0)
    eps_polar = 0.7
    min_samples_polar = 18
    slope_variation_threshold = 0.20  # 20%
    max_theta_gap = 15.0
    do_subdivide = True

    valid_groups, df = process_spiral_arms_by_proximity(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_fit_window=30.0,
        r_fit_window=3.0,
        theta_merge_threshold=10.0,
        r_merge_threshold=2.0,
        r_gap=2.0,
        snapshot_interval=10
    )
    print("Total de grupos finales:", len(valid_groups))

    # Gráfica en (θ vs r)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['theta'], df['r'], s=1, color='silver', alpha=0.5, label='Datos Originales')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, grp in enumerate(valid_groups):
        pts = grp['points']
        theta_vals = pts['theta']
        r_vals = pts['r']
        pa_val = grp['pa'] if grp['pa'] is not None else 0.0
        color = colors[idx % len(colors)]
        plt.scatter(theta_vals, r_vals, s=10, alpha=0.8, color=color,
                    label=f"Grupo {idx+1}, PA={pa_val:.2f}")
        if grp['slope'] is not None:
            theta_line = np.linspace(theta_vals.min(), theta_vals.max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Ajustes intermedios en (r, θ) - Halo {id_halo}")
    plt.grid(True)
    plt.show()

    # Gráfica en proyección polar
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 8))
    theta_all_rad = np.radians(df['theta'])
    ax.scatter(theta_all_rad, df['r'], s=1, color='gray', alpha=0.5, label='Datos Originales')
    for idx, grp in enumerate(valid_groups):
        pts = grp['points']
        theta_rad = np.radians(pts['theta'])
        r_vals = pts['r']
        pa_val = grp['pa'] if grp['pa'] is not None else 0.0
        color = colors[idx % len(colors)]
        ax.scatter(theta_rad, r_vals, s=10, alpha=0.8, color=color,
                   label=f"Grupo {idx+1} PA: {pa_val:.2f}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)
    ax.set_title(f"Ajustes intermedios en proyección polar - Halo {id_halo}", fontsize=14, pad=20)
    plt.show()


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.spatial import ConvexHull
from matplotlib.path import Path
from sklearn.cluster import Birch
from sklearn.linear_model import LinearRegression

##############################
# Funciones de Visualización
##############################
def plot_intermediate_state_cartesian(state, df_all):
    """
    Grafica los grupos en el plano (θ, r) junto con los datos completos.
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=5, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan', 'gold', 'darkblue']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=15, color=color, label=f"Grupo {idx+1}")
        if len(pts) >= 3:
            arr = pts[['theta','r']].values
            try:
                hull = ConvexHull(arr)
                hull_pts = arr[hull.vertices]
                hull_pts = np.concatenate([hull_pts, hull_pts[:1]], axis=0)
                plt.plot(hull_pts[:,0], hull_pts[:,1], '-', color=color, lw=2)
            except Exception:
                pass
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

##############################
# Funciones de Actualización y Fusión
##############################
def update_group_limits(group):
    """
    Recalcula los límites del grupo (θ_min, θ_max, r_min, r_max) usando el convex hull
    si el grupo tiene al menos 3 puntos; si no, usa la bounding box.
    """
    pts = group['points']
    if len(pts) >= 3:
        arr = pts[['theta','r']].values
        try:
            hull = ConvexHull(arr)
            hull_pts = arr[hull.vertices]
            group['theta_min'] = hull_pts[:,0].min()
            group['theta_max'] = hull_pts[:,0].max()
            group['r_min'] = hull_pts[:,1].min()
            group['r_max'] = hull_pts[:,1].max()
            return
        except Exception:
            pass
    group['theta_min'] = pts['theta'].min()
    group['theta_max'] = pts['theta'].max()
    group['r_min'] = pts['r'].min()
    group['r_max'] = pts['r'].max()

def calculate_pa(slope, intercept):
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.nan

def point_in_hull(point, hull_points):
    poly = Path(hull_points, closed=True)
    return poly.contains_point(point, radius=-1e-9)

def merge_all_groups(groups, theta_merge_threshold, r_merge_threshold):
    """
    Fusiona grupos extendiendo sus límites en un 10% y fusiona
    solo si las cajas extendidas se superponen y la variación relativa en pendiente es ≤15%.
    """
    merged_flag = True
    while merged_flag:
        merged_flag = False
        i = 0
        while i < len(groups):
            j = i + 1
            while j < len(groups):
                g1 = groups[i]
                g2 = groups[j]
                # Límites originales
                th_min1, th_max1, r_min1, r_max1 = g1['theta_min'], g1['theta_max'], g1['r_min'], g1['r_max']
                th_min2, th_max2, r_min2, r_max2 = g2['theta_min'], g2['theta_max'], g2['r_min'], g2['r_max']
                # Extender límites en 10%
                range_th1 = th_max1 - th_min1 if (th_max1 - th_min1)!=0 else 1e-3
                range_r1 = r_max1 - r_min1 if (r_max1 - r_min1)!=0 else 1e-3
                ext_th_min1 = th_min1 - 0.1 * range_th1
                ext_th_max1 = th_max1 + 0.1 * range_th1
                ext_r_min1  = r_min1 - 0.1 * range_r1
                ext_r_max1  = r_max1 + 0.1 * range_r1
                range_th2 = th_max2 - th_min2 if (th_max2 - th_min2)!=0 else 1e-3
                range_r2 = r_max2 - r_min2 if (r_max2 - r_min2)!=0 else 1e-3
                ext_th_min2 = th_min2 - 0.1 * range_th2
                ext_th_max2 = th_max2 + 0.1 * range_th2
                ext_r_min2  = r_min2 - 0.1 * range_r2
                ext_r_max2  = r_max2 + 0.1 * range_r2
                # Superposición
                overlap_theta = not (ext_th_max1 < ext_th_min2 or ext_th_min1 > ext_th_max2)
                overlap_r = not (ext_r_max1 < ext_r_min2 or ext_r_min1 > ext_r_max2)
                # Variación en pendiente
                if g1['slope'] is None or g2['slope'] is None:
                    rel_variation = 0
                else:
                    rel_variation = abs(g1['slope'] - g2['slope']) / (abs(g1['slope'])+1e-12)
                if overlap_theta and overlap_r and rel_variation <= 0.15:
                    # Fusionar g2 en g1
                    merged_points = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    model = LinearRegression()
                    X_new = merged_points[['theta']].values
                    y_new = merged_points['r'].values
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    new_intercept = model.intercept_
                    g1['points'] = merged_points
                    g1['slope'] = new_slope
                    g1['intercept'] = new_intercept
                    g1['pa'] = calculate_pa(new_slope, new_intercept)
                    update_group_limits(g1)
                    groups.pop(j)
                    merged_flag = True
                else:
                    j += 1
            i += 1

##############################
# Asignación de puntos con Convex Hull y recorrido de izquierda a derecha
##############################
def add_point_strict_convex(point, groups, theta_strict_window, r_strict_window, r_gap=2.0):
    """
    Intenta agregar 'point' a un grupo existente de forma estricta:
      - Si el grupo tiene ≥3 puntos, se calcula su convex hull y se exige que el punto esté estrictamente dentro.
      - Para grupos con <3 puntos se usa una bounding box reducida (usando la mitad de theta_strict_window y r_strict_window).
      - Se verifica que el nuevo punto no exceda r_gap en r.
    """
    theta_pt = point['theta']
    r_pt = point['r']
    for g in groups:
        if r_pt < g['r_min'] - r_gap or r_pt > g['r_max'] + r_gap:
            continue
        pts = g['points']
        if len(pts) >= 3:
            arr = pts[['theta','r']].values
            try:
                hull = ConvexHull(arr)
                hull_pts = arr[hull.vertices]
                poly = Path(hull_pts, closed=True)
                inside = poly.contains_point([theta_pt, r_pt], radius=-1e-9)
                if inside:
                    g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                    update_group_limits(g)
                    return True
            except Exception:
                pass
        else:
            half_theta = theta_strict_window / 2.0
            half_r = r_strict_window / 2.0
            t_min = pts['theta'].min()
            t_max = pts['theta'].max()
            rr_min = pts['r'].min()
            rr_max = pts['r'].max()
            if (theta_pt >= t_min - half_theta and theta_pt <= t_max + half_theta and
                r_pt >= rr_min - half_r and r_pt <= rr_max + half_r):
                g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                update_group_limits(g)
                return True
    return False

##############################
# Función principal final
##############################
def process_spiral_arms_by_proximity(
    id_halo,
    quartile_threshold=0.55,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    theta_merge_threshold=7.0,
    r_merge_threshold=7.0,
    r_gap=2.0,
    snapshot_interval=10
):
    """
    Procesa datos y agrupa puntos de forma estricta con convex hull, recorriendo de izquierda a derecha.
    1) Filtra por percentil y por θ (50-250).
    2) Ordena el DataFrame por θ.
    3) Para cada punto se intenta agregar a un grupo:
         - Para grupos con ≥3 puntos se exige que el punto esté estrictamente dentro del convex hull.
         - Para grupos con <3 puntos se usa una bounding box reducida (la mitad de la ventana).
         - Se verifica que el punto no exceda r_gap en r.
         - Además, el punto debe estar dentro del rango horizontal actual extendido en ±10% del rango.
    4) Si no se agrega, se crea un grupo nuevo.
    5) Finalmente, se fusionan grupos extendiendo sus límites en un 10% y fusionando solo si la variación en pendiente es ≤15%.
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= 50) & (df['theta'] <= 250)].copy()
    
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='theta', inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # Inicializar grupos a partir de clusters de BIRCH (más robusto y eficiente)
    from sklearn.cluster import Birch
    birch = Birch(threshold=0.7, n_clusters=None, branching_factor=20)
    X_polar = df_filtered[['r','theta']].values
    df_filtered['cluster'] = birch.fit_predict(X_polar)
    
    clusters_tolerance = 0.70
    def merge_close_clusters(clusters, clusters_tolerance):
        merged_clusters = {}
        used_clusters = set()
        for id1, points1 in clusters.items():
            if id1 in used_clusters:
                continue
            merged_points = points1.copy()
            for id2, points2 in clusters.items():
                if id1 >= id2 or id2 in used_clusters:
                    continue
                min_distance = np.min(np.sqrt(
                    (points1['r'].values[:, None] - points2['r'].values)**2 +
                    (points1['theta'].values[:, None] - points2['theta'].values)**2
                ))
                if min_distance < clusters_tolerance:
                    merged_points = pd.concat([merged_points, points2], ignore_index=True)
                    used_clusters.add(id2)
            merged_clusters[id1] = merged_points
            used_clusters.add(id1)
        return merged_clusters

    clusters_polar = {}
    for cluster_id in np.unique(df_filtered['cluster']):
        if cluster_id == -1:
            continue
        cluster_points = df_filtered[df_filtered['cluster'] == cluster_id].copy()
        clusters_polar[cluster_id] = cluster_points[['r', 'theta']]
    clusters_polar = merge_close_clusters(clusters_polar, clusters_tolerance)
    
    model = LinearRegression()
    def calculate_distance(slope, intercept, theta, r):
        pred_r = slope * theta + intercept
        return abs(pred_r - r)
    
    def is_valid_pa(pa, valid_range):
        return valid_range[0] <= abs(pa) <= valid_range[1]
    
    groups = []
    for cid, pts in clusters_polar.items():
        if len(pts) >= 3:
            X_ = pts[['theta']].values
            y_ = pts['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'cluster_id': cid,
            'points': pts.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # Recorrido de izquierda a derecha (orden creciente de θ)
    df_local = df_filtered.copy()
    df_local.sort_values(by='theta', inplace=True)
    df_local.reset_index(drop=True, inplace=True)
    
    discarded_points = []
    iteration = 0
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        assigned = False
        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue
            group_theta_min = g['points']['theta'].min()
            group_theta_max = g['points']['theta'].max()
            theta_range = group_theta_max - group_theta_min if group_theta_max - group_theta_min > 0 else 1e-3
            theta_margin = 0.10 * theta_range
            if not (group_theta_min - theta_margin <= theta_pt <= group_theta_max + theta_margin):
                continue
            dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
            if dist <= radius_tolerance:
                # Verificar que la distancia de cada punto en el grupo al ajuste sea ≤ point_distance_tolerance
                distances = g['points'].apply(
                    lambda row: calculate_distance(g['slope'], g['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if all(d <= point_distance_tolerance for d in distances):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= 0.15:
                        g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(new_slope, g['intercept'])
                        assigned = True
                        break
        if not assigned:
            discarded_points.append(point.to_dict())
        iteration += 1

    # Reasignar puntos descartados
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            theta_pt = p['theta']
            r_pt = p['r']
            assigned = False
            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                group_theta_min = g['points']['theta'].min()
                group_theta_max = g['points']['theta'].max()
                theta_range = group_theta_max - group_theta_min if group_theta_max - group_theta_min > 0 else 1e-3
                theta_margin = 0.10 * theta_range
                if not (group_theta_min - theta_margin <= theta_pt <= group_theta_max + theta_margin):
                    continue
                dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
                if dist <= radius_tolerance and g['pa'] is not None and is_valid_pa(g['pa'], flexible_pa_range):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) <= 0.15:
                        g['points'] = pd.concat([g['points'], p.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(new_slope, g['intercept'])
                        assigned = True
                        break
            if not assigned:
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new, i_new)
                if is_valid_pa(pa_new, flexible_pa_range):
                    groups.append({
                        'cluster_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })

    # Subdivisión opcional en caso de gap grande en θ
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    def subdivide_if_gap(group, max_gap=max_theta_gap):
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)
        seg1 = {
            'cluster_id': group['cluster_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'cluster_id': group['cluster_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]
    
    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap=max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups

    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and is_valid_pa(pa_val, flexible_pa_range):
                valid_groups.append(grp)

    return valid_groups, df

##############################
# Ejecución y Visualización
##############################
if __name__ == "__main__":
    # Parámetros de ejemplo (ajústalos según tus necesidades)
    id_halo = "17"
    quartile_threshold = 0.155
    radius_tolerance = 0.45
    point_distance_tolerance = 0.59
    flexible_pa_range = (4.0, 30.0)
    eps_polar = 0.7
    min_samples_polar = 18
    slope_variation_threshold = 0.20  # 20%
    max_theta_gap = 15.0
    do_subdivide = True

    valid_groups, df = process_spiral_arms_by_proximity(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_fit_window=30.0,
        r_fit_window=3.0,
        theta_merge_threshold=10.0,
        r_merge_threshold=2.0,
        r_gap=2.0,
        snapshot_interval=10
    )
    print("Total de grupos finales:", len(valid_groups))
    
    # Gráfica en (θ vs r)
    plt.figure(figsize=(10, 6))
    plt.scatter(df['theta'], df['r'], s=1, color='silver', alpha=0.5, label='Datos Originales')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, grp in enumerate(valid_groups):
        pts = grp['points']
        theta_vals = pts['theta']
        r_vals = pts['r']
        pa_val = grp['pa'] if grp['pa'] is not None else 0.0
        color = colors[idx % len(colors)]
        plt.scatter(theta_vals, r_vals, s=10, alpha=0.8, color=color,
                    label=f"Grupo {idx+1}, PA={pa_val:.2f}")
        if grp['slope'] is not None:
            theta_line = np.linspace(theta_vals.min(), theta_vals.max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Ajustes intermedios en (r, θ) - Halo {id_halo}")
    plt.grid(True)
    plt.show()

    # Gráfica en proyección polar
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 8))
    theta_all_rad = np.radians(df['theta'])
    ax.scatter(theta_all_rad, df['r'], s=1, color='gray', alpha=0.5, label='Datos Originales')
    for idx, grp in enumerate(valid_groups):
        pts = grp['points']
        theta_rad = np.radians(pts['theta'])
        r_vals = pts['r']
        pa_val = grp['pa'] if grp['pa'] is not None else 0.0
        color = colors[idx % len(colors)]
        ax.scatter(theta_rad, r_vals, s=10, alpha=0.8, color=color,
                   label=f"Grupo {idx+1} PA: {pa_val:.2f}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)
    ax.set_title(f"Ajustes intermedios en proyección polar - Halo {id_halo}", fontsize=14, pad=20)
    plt.show()


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import Birch

# Cargar datos
df = pd.read_csv("data_rho_17_filtered.csv")

# Calcular coordenadas polares
df['r'] = np.sqrt(df['x']**2 + df['y']**2)
df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])

# Filtrar puntos en el rango de theta (por ejemplo, 50 a 250 grados)
df = df[(df['theta'] >= 50) & (df['theta'] <= 250)].copy()

# Filtrar por percentil en 'rho_resta_final_exp' (ajusta el valor según necesites)
quartile_threshold = 0.55
threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()

# Crear matriz de características en el espacio polar
X = df_filtered[['r', 'theta']].values

# Aplicar clustering con BIRCH
birch = Birch(threshold=0.7, n_clusters=10, branching_factor=50)
df_filtered['cluster'] = birch.fit_predict(X)

# Visualizar los clusters en un gráfico de θ vs r
plt.figure(figsize=(10, 6))
colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan', 'gold', 'darkblue']
for cluster in np.unique(df_filtered['cluster']):
    # Puedes filtrar el ruido (si se usa -1) o mostrarlos todos
    subset = df_filtered[df_filtered['cluster'] == cluster]
    plt.scatter(subset['theta'], subset['r'], s=10, color=colors[int(cluster) % len(colors)], label=f"Cluster {cluster}")
plt.xlabel("θ (grados)")
plt.ylabel("r")
plt.title("Clustering con BIRCH en coordenadas polares")
plt.legend(loc="upper right")
plt.grid(True)
plt.show()


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import Birch

# 1. Cargar datos
df = pd.read_csv("data_rho_17_filtered.csv")

# 2. Calcular coordenadas polares
df['r'] = np.sqrt(df['x']**2 + df['y']**2)
df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])

# 3. Filtrar por rango de θ
df = df[(df['theta'] >= 50) & (df['theta'] <= 250)].copy()

# 4. Filtrar por percentil
quartile_threshold = 0.55
threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()

# 5. Escalar variables para equilibrar r y theta
#    Ajusta los divisores según tus datos (prueba y error).
df_filtered['r_scaled'] = df_filtered['r'] / 5.0      # por ejemplo
df_filtered['theta_scaled'] = df_filtered['theta'] / 50.0

X = df_filtered[['r_scaled', 'theta_scaled']].values

# 6. Ajustar BIRCH con parámetros ajustados
#    threshold más bajo => más subclusters
birch = Birch(threshold=0.5, n_clusters=None, branching_factor=20)
df_filtered['cluster'] = birch.fit_predict(X)

# 7. Visualizar
plt.figure(figsize=(10,6))
colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
for cluster_id in np.unique(df_filtered['cluster']):
    subset = df_filtered[df_filtered['cluster'] == cluster_id]
    c = colors[int(cluster_id) % len(colors)]
    plt.scatter(subset['theta'], subset['r'], s=10, color=c, label=f"Cluster {cluster_id}")
plt.xlabel("θ (grados)")
plt.ylabel("r")
plt.title("Clustering con BIRCH (escalado) en (r, θ)")
plt.legend(loc='upper right')
plt.grid(True)
plt.show()


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import Birch

# ============================
# Funciones para subdividir clusters por gaps
# ============================
def subdivide_by_theta_gap(df_cluster, gap_threshold=2.5):
    """
    Recibe un DataFrame con un cluster y lo ordena por θ.
    Si existe un gap (diferencia entre puntos consecutivos) mayor a gap_threshold en θ,
    subdivide el cluster en segmentos.
    Retorna una lista de DataFrames.
    """
    df_cluster = df_cluster.sort_values(by='theta').reset_index(drop=True)
    theta_vals = df_cluster['theta'].values
    dtheta = np.diff(theta_vals)
    if np.all(dtheta <= gap_threshold):
        return [df_cluster]
    groups = []
    start = 0
    for i, diff in enumerate(dtheta):
        if diff > gap_threshold:
            groups.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    groups.append(df_cluster.iloc[start:].copy())
    return groups

def subdivide_by_r_gap(df_cluster, gap_threshold=1.5):
    """
    Recibe un DataFrame con un cluster y lo ordena por r.
    Si existe un gap mayor a gap_threshold en r, subdivide el cluster.
    Retorna una lista de DataFrames.
    """
    df_cluster = df_cluster.sort_values(by='r').reset_index(drop=True)
    r_vals = df_cluster['r'].values
    dr = np.diff(r_vals)
    if np.all(dr <= gap_threshold):
        return [df_cluster]
    groups = []
    start = 0
    for i, diff in enumerate(dr):
        if diff > gap_threshold:
            groups.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    groups.append(df_cluster.iloc[start:].copy())
    return groups

# ============================
# Función para graficar clusters
# ============================
def plot_clusters(clusters, title="Clusters segmentados"):
    plt.figure(figsize=(10, 6))
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    for idx, df_cluster in enumerate(clusters):
        c = colors[idx % len(colors)]
        plt.scatter(df_cluster['theta'], df_cluster['r'], s=10, color=c, label=f"Grupo {idx+1}")
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()

# ============================
# Ejecución final: BIRCH + subdivisión por gaps
# ============================
# 1. Cargar datos
df = pd.read_csv("data_rho_17_filtered.csv")

# 2. Calcular coordenadas polares
df['r'] = np.sqrt(df['x']**2 + df['y']**2)
df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])

# 3. Filtrar por rango de θ (por ejemplo, 50 a 250 grados)
df = df[(df['theta'] >= 20) & (df['theta'] <= 270)].copy()

# 4. Filtrar por percentil en 'rho_resta_final_exp'
quartile_threshold = 0.55
threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()

# 5. Escalar variables para equilibrar (opcional, ajusta según tus datos)
# Aquí podemos escalar θ y r para que tengan una contribución similar.
# Por ejemplo, si r suele estar en [0,20] y θ en [50,250], se puede dividir θ entre 50.
df_filtered['r_scaled'] = df_filtered['r'] /5.0 # O puedes dividir por algún factor
df_filtered['theta_scaled'] = df_filtered['theta'] / 50.0

# 6. Aplicar BIRCH para clustering inicial
X = df_filtered[['r_scaled', 'theta_scaled']].values
birch = Birch(threshold=0.5, n_clusters=None, branching_factor=20)
df_filtered['cluster'] = birch.fit_predict(X)

# 7. Extraer clusters de BIRCH
clusters = []
for cluster_id in np.unique(df_filtered['cluster']):
    subset = df_filtered[df_filtered['cluster'] == cluster_id].copy()
    subset.reset_index(drop=True, inplace=True)
    clusters.append(subset)

# 8. Subdividir cada cluster si existe un gap mayor a 2.5 en θ o en r
final_clusters = []
for cluster in clusters:
    # Primero subdividimos por θ
    theta_subclusters = subdivide_by_theta_gap(cluster, gap_threshold=2.5)
    for subc in theta_subclusters:
        # Luego, subdividimos cada resultado por r
        r_subclusters = subdivide_by_r_gap(subc, gap_threshold=2.5)
        final_clusters.extend(r_subclusters)

# 9. Visualizar el resultado final
plot_clusters(final_clusters, title="Segmentación final con BIRCH + subdivisión por gap (umbral 2.5)")


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

def build_graph_rectangular(df_points, theta_diff, r_diff):
    n = len(df_points)
    graph = [[] for _ in range(n)]
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    
    for i in range(n):
        for j in range(i+1, n):
            dtheta = abs(theta_arr[i] - theta_arr[j])
            dr = abs(r_arr[i] - r_arr[j])
            # Vecindad más estricta
            if dtheta <= theta_diff and dr <= r_diff:
                graph[i].append(j)
                graph[j].append(i)
    return graph, n

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False]*n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            cluster_df.reset_index(drop=True, inplace=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    if mode=='theta':
        df_cluster = df_cluster.sort_values(by='theta').reset_index(drop=True)
        arr = df_cluster['theta'].values
    else:
        df_cluster = df_cluster.sort_values(by='r').reset_index(drop=True)
        arr = df_cluster['r'].values
    
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff>gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    # Parámetros de cercanía
    theta_diff=5.0,   # Ajusta para ser más o menos estricto
    r_diff=1.0,       # Ajusta para ser más o menos estricto
    # Subdivisión
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta']<0, df['theta']+360, df['theta'])
    df = df[(df['theta']>=theta_min)&(df['theta']<=theta_max)].copy()
    
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp']>threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    # Contruir grafo rectangular
    graph, n = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    # BFS
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        # Subdividir por θ
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            # Subdividir por r
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    return final_clusters

def plot_clusters_rectangular_gaps(clusters, title="Clusters con BFS + Gaps"):
    plt.figure(figsize=(10,6))
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    for idx, dfc in enumerate(clusters):
        c = colors[idx % len(colors)]
        plt.scatter(dfc['theta'], dfc['r'], s=10, color=c, label=f"Grupo {idx+1}")
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()

if __name__=="__main__":
    # Ajusta los parámetros para forzar mayor segmentación
    final_clusters = process_clusters_rectangular_gaps(
        id_halo="17",
        quartile_threshold=0.55,
        theta_min=50,
        theta_max=250,
        theta_diff=3.0,     # Vecindad horizontal
        r_diff=0.68,         # Vecindad radial
        gap_threshold_theta=1.9,
        gap_threshold_r=1.23
    )
    print(f"Se formaron {len(final_clusters)} grupos finales.")
    plot_clusters_rectangular_gaps(final_clusters, "Clusters rectangulares + subdivisión por gap")



In [0]:
import warnings
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

# ========================
# Parte 1: Segmentación inicial con vecinos rectangulares
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    n = len(df_points)
    graph = [[] for _ in range(n)]
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    
    for i in range(n):
        for j in range(i+1, n):
            dtheta = abs(theta_arr[i] - theta_arr[j])
            dr = abs(r_arr[i] - r_arr[j])
            # Vecindad más estricta
            if dtheta <= theta_diff and dr <= r_diff:
                graph[i].append(j)
                graph[j].append(i)
    return graph, n

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False]*n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            cluster_df.reset_index(drop=True, inplace=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    if mode=='theta':
        df_cluster = df_cluster.sort_values(by='theta').reset_index(drop=True)
        arr = df_cluster['theta'].values
    else:
        df_cluster = df_cluster.sort_values(by='r').reset_index(drop=True)
        arr = df_cluster['r'].values
    
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    # Parámetros de cercanía
    theta_diff=5.0,   # Ajusta para ser más o menos estricto
    r_diff=1.0,       # Ajusta para ser más o menos estricto
    # Subdivisión
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    # Construir grafo rectangular y obtener clusters (grupos semilla)
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        # Subdividir por theta
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            # Subdividir por r
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    return final_clusters, df_filtered

def plot_clusters_rectangular_gaps(clusters, title="Clusters con BFS + Gaps"):
    plt.figure(figsize=(10,6))
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan', 'gold', 'darkblue']
    for idx, dfc in enumerate(clusters):
        c = colors[idx % len(colors)]
        plt.scatter(dfc['theta'], dfc['r'], s=10, color=c, label=f"Grupo {idx+1}")
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()


# ========================
# Parte 2: Proceso iterativo (similar a process_spiral_arms_interactive)
# utilizando los grupos semilla de la segmentación rectangular
# ========================

def process_spiral_arms_with_seeds(
    id_halo,
    quartile_threshold=0.55,
    # Parámetros para la segmentación rectangular
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,     # Vecindad horizontal para seeds
    r_diff=0.68,        # Vecindad radial para seeds
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    # Parámetros para la asignación iterativa
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10
):
    # 1. Cargar datos y calcular coordenadas polares
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    # 2. Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 3. Obtener grupos semilla mediante segmentación rectangular
    seeds_all, df_seed_source = process_clusters_rectangular_gaps(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    
    # Filtrar solo los grupos semilla con más de 60 puntos
    seed_clusters = [seed for seed in seeds_all if len(seed) > 60]
    print(f"Grupos semilla con más de 60 puntos: {len(seed_clusters)}")
    
    # Para asegurarnos de que usamos los mismos puntos que en df_filtered,
    # marcamos un identificador en cada punto que pertenezca a un seed
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        # Se asume que el índice de df_filtered es el mismo que el de df_seed_source
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 4. Inicializar grupos usando los seeds (regresión lineal)
    model = LinearRegression()
    
    def calculate_distance(slope, intercept, theta, r):
        pred_r = slope * theta + intercept
        return abs(pred_r - r)
    
    def calculate_pa(slope, intercept):
        if intercept != 0:
            return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
        return np.nan
    
    def is_valid_pa(pa, valid_range):
        return valid_range[0] <= abs(pa) <= valid_range[1]
    
    groups = []
    for idx, seed in enumerate(seed_clusters):
        # Solo se aplica regresión si hay suficientes puntos
        if len(seed) >= 3:
            X_ = seed[['theta']].values
            y_ = seed['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'seed_id': idx,
            'points': seed.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # 5. Preparar puntos restantes para asignación iterativa:
    # Se consideran los puntos de df_filtered que NO fueron asignados a seeds.
    df_local = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    discarded_points = []
    states = []  # estados intermedios para visualización
    iteration = 0
    
    # 6. Asignación iterativa de puntos
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        assigned = False
        
        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue
            dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
            if dist <= radius_tolerance:
                # Se verifica que todos los puntos del grupo estén cercanos
                distances = g['points'].apply(
                    lambda row: calculate_distance(g['slope'], g['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if all(d <= point_distance_tolerance for d in distances):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) <= slope_variation_threshold:
                        # Actualizar grupo con el nuevo punto
                        g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(g['slope'], g['intercept'])
                        assigned = True
                        break
        if not assigned:
            discarded_points.append(point.to_dict())
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            groups_copy = copy.deepcopy(groups)
            discarded_copy = copy.deepcopy(discarded_points)
            states.append({
                'iteration': iteration,
                'groups': groups_copy,
                'discarded': discarded_copy,
                'remaining': df_local.copy()
            })
    
    # 7. Reasignar puntos descartados (etapa final)
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            theta_pt = p['theta']
            r_pt = p['r']
            assigned = False
            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
                if dist <= radius_tolerance:
                    if g['pa'] is not None and is_valid_pa(g['pa'], flexible_pa_range):
                        X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                        y_new = np.hstack([g['points']['r'].values, r_pt])
                        model.fit(X_new, y_new)
                        new_slope = model.coef_[0]
                        old_slope = g['slope']
                        if abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) <= slope_variation_threshold:
                            g['points'] = pd.concat([g['points'], p.to_frame().T], ignore_index=True)
                            g['slope'] = new_slope
                            g['intercept'] = model.intercept_
                            g['pa'] = calculate_pa(g['slope'], g['intercept'])
                            assigned = True
                            break
            if not assigned:
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new, i_new)
                if is_valid_pa(pa_new, flexible_pa_range):
                    groups.append({
                        'seed_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })
    
    # 8. (Opcional) Subdividir grupos si existe un gran gap en theta
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    def subdivide_if_gap(group, max_gap=max_theta_gap):
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)
        seg1 = {
            'seed_id': group['seed_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'seed_id': group['seed_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]
    
    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap=max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups
    
    # 9. Filtrar grupos por PA válido
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and is_valid_pa(pa_val, flexible_pa_range):
                valid_groups.append(grp)
    
    # Agregar estado final
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'discarded': copy.deepcopy(discarded_points),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered

def plot_intermediate_state(state, df_all):
    """
    Grafica el estado intermedio en el plano polar:
      - Fondo: todos los datos en gris pálido.
      - Grupos asignados: cada grupo con un color y su línea de ajuste (si se ha calculado).
      - Puntos descartados: marcados con una 'x' negra.
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Fondo: todos los datos
    theta_all = np.radians(df_all['theta'])
    r_all = df_all['r']
    ax.scatter(theta_all, r_all, s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        theta_vals = np.radians(pts['theta'])
        r_vals = pts['r']
        color = colors[idx % len(colors)]
        ax.scatter(theta_vals, r_vals, s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)
    
    if len(state['discarded']) > 0:
        df_desc = pd.DataFrame(state['discarded'])
        theta_desc = np.radians(df_desc['theta'])
        r_desc = df_desc['r']
        ax.scatter(theta_desc, r_desc, s=10, color='black', marker='x', label='Descartados')
    
    ax.set_title(f"Estado en iteración {state['iteration']}")
    ax.legend(loc='upper right')
    plt.show()


# ========================
# Ejecución interactiva e impresión de resultados
# ========================

states, final_groups, df_filtered = process_spiral_arms_with_seeds(
    id_halo="17",
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,
    r_diff=0.68,
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10
)

print("Total de estados guardados:", len(states))
print("Total de grupos finales:", len(final_groups))

# Widget interactivo para ver el proceso iterativo
def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))


In [0]:
import warnings
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

# ========================
# Parte 1: Segmentación inicial con vecinos rectangulares
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    n = len(df_points)
    graph = [[] for _ in range(n)]
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    
    for i in range(n):
        for j in range(i+1, n):
            dtheta = abs(theta_arr[i] - theta_arr[j])
            dr = abs(r_arr[i] - r_arr[j])
            # Vecindad más estricta
            if dtheta <= theta_diff and dr <= r_diff:
                graph[i].append(j)
                graph[j].append(i)
    return graph, n

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False]*n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            cluster_df.reset_index(drop=True, inplace=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    if mode=='theta':
        df_cluster = df_cluster.sort_values(by='theta').reset_index(drop=True)
        arr = df_cluster['theta'].values
    else:
        df_cluster = df_cluster.sort_values(by='r').reset_index(drop=True)
        arr = df_cluster['r'].values
    
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    # Parámetros de cercanía
    theta_diff=5.0,   # Ajusta para ser más o menos estricto
    r_diff=1.0,       # Ajusta para ser más o menos estricto
    # Subdivisión
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    # Construir grafo rectangular y obtener clusters (grupos semilla)
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        # Subdividir por theta
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            # Subdividir por r
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    return final_clusters, df_filtered

def plot_clusters_rectangular_gaps(clusters, title="Clusters con BFS + Gaps"):
    plt.figure(figsize=(10,6))
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan', 'gold', 'darkblue']
    for idx, dfc in enumerate(clusters):
        c = colors[idx % len(colors)]
        plt.scatter(dfc['theta'], dfc['r'], s=10, color=c, label=f"Grupo {idx+1}")
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()


# ========================
# Parte 2: Proceso iterativo (similar a process_spiral_arms_interactive)
# utilizando los grupos semilla de la segmentación rectangular
# ========================

def process_spiral_arms_with_seeds(
    id_halo,
    quartile_threshold=0.55,
    # Parámetros para la segmentación rectangular
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,     # Vecindad horizontal para seeds
    r_diff=0.68,        # Vecindad radial para seeds
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    # Parámetros para la asignación iterativa
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10
):
    # 1. Cargar datos y calcular coordenadas polares
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    # 2. Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 3. Obtener grupos semilla mediante segmentación rectangular
    seeds_all, df_seed_source = process_clusters_rectangular_gaps(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    
    # Filtrar solo los grupos semilla con más de 60 puntos
    seed_clusters = [seed for seed in seeds_all if len(seed) > 60]
    print(f"Grupos semilla con más de 60 puntos: {len(seed_clusters)}")
    
    # Para asegurarnos de que usamos los mismos puntos que en df_filtered,
    # marcamos un identificador en cada punto que pertenezca a un seed
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        # Se asume que el índice de df_filtered es el mismo que el de df_seed_source
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 4. Inicializar grupos usando los seeds (regresión lineal)
    model = LinearRegression()
    
    def calculate_distance(slope, intercept, theta, r):
        pred_r = slope * theta + intercept
        return abs(pred_r - r)
    
    def calculate_pa(slope, intercept):
        if intercept != 0:
            return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
        return np.nan
    
    def is_valid_pa(pa, valid_range):
        return valid_range[0] <= abs(pa) <= valid_range[1]
    
    groups = []
    for idx, seed in enumerate(seed_clusters):
        # Solo se aplica regresión si hay suficientes puntos
        if len(seed) >= 3:
            X_ = seed[['theta']].values
            y_ = seed['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'seed_id': idx,
            'points': seed.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # 5. Preparar puntos restantes para asignación iterativa:
    # Se consideran los puntos de df_filtered que NO fueron asignados a seeds.
    df_local = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    discarded_points = []
    states = []  # estados intermedios para visualización
    iteration = 0
    
    # 6. Asignación iterativa de puntos
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        assigned = False
        
        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue
            dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
            if dist <= radius_tolerance:
                # Se verifica que todos los puntos del grupo estén cercanos
                distances = g['points'].apply(
                    lambda row: calculate_distance(g['slope'], g['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if all(d <= point_distance_tolerance for d in distances):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) <= slope_variation_threshold:
                        # Actualizar grupo con el nuevo punto
                        g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(g['slope'], g['intercept'])
                        assigned = True
                        break
        if not assigned:
            discarded_points.append(point.to_dict())
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            groups_copy = copy.deepcopy(groups)
            discarded_copy = copy.deepcopy(discarded_points)
            states.append({
                'iteration': iteration,
                'groups': groups_copy,
                'discarded': discarded_copy,
                'remaining': df_local.copy()
            })
    
    # 7. Reasignar puntos descartados (etapa final)
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            theta_pt = p['theta']
            r_pt = p['r']
            assigned = False
            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
                if dist <= radius_tolerance:
                    if g['pa'] is not None and is_valid_pa(g['pa'], flexible_pa_range):
                        X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                        y_new = np.hstack([g['points']['r'].values, r_pt])
                        model.fit(X_new, y_new)
                        new_slope = model.coef_[0]
                        old_slope = g['slope']
                        if abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) <= slope_variation_threshold:
                            g['points'] = pd.concat([g['points'], p.to_frame().T], ignore_index=True)
                            g['slope'] = new_slope
                            g['intercept'] = model.intercept_
                            g['pa'] = calculate_pa(g['slope'], g['intercept'])
                            assigned = True
                            break
            if not assigned:
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new, i_new)
                if is_valid_pa(pa_new, flexible_pa_range):
                    groups.append({
                        'seed_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })
    
    # 8. (Opcional) Subdividir grupos si existe un gran gap en theta
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    def subdivide_if_gap(group, max_gap=max_theta_gap):
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)
        seg1 = {
            'seed_id': group['seed_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'seed_id': group['seed_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]
    
    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap=max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups
    
    # 9. Filtrar grupos por PA válido
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and is_valid_pa(pa_val, flexible_pa_range):
                valid_groups.append(grp)
    
    # Agregar estado final
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'discarded': copy.deepcopy(discarded_points),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered

def plot_intermediate_state(state, df_all):
    """
    Grafica el estado intermedio en el plano polar:
      - Fondo: todos los datos en gris pálido.
      - Grupos asignados: cada grupo con un color y su línea de ajuste (si se ha calculado).
      - Puntos descartados: marcados con una 'x' negra.
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Fondo: todos los datos
    theta_all = np.radians(df_all['theta'])
    r_all = df_all['r']
    ax.scatter(theta_all, r_all, s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        theta_vals = np.radians(pts['theta'])
        r_vals = pts['r']
        color = colors[idx % len(colors)]
        ax.scatter(theta_vals, r_vals, s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)
    
    if len(state['discarded']) > 0:
        df_desc = pd.DataFrame(state['discarded'])
        theta_desc = np.radians(df_desc['theta'])
        r_desc = df_desc['r']
        ax.scatter(theta_desc, r_desc, s=10, color='black', marker='x', label='Descartados')
    
    ax.set_title(f"Estado en iteración {state['iteration']}")
    ax.legend(loc='upper right')
    plt.show()


# ========================
# Ejecución interactiva e impresión de resultados
# ========================

states, final_groups, df_filtered = process_spiral_arms_with_seeds(
    id_halo="17",
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,
    r_diff=0.68,
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 40.0),
    slope_variation_threshold=0.18,
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10
)

print("Total de estados guardados:", len(states))
print("Total de grupos finales:", len(final_groups))

# Widget interactivo para ver el proceso iterativo
def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))

In [0]:
import warnings
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

# ========================
# Parte 1: Segmentación inicial con vecinos rectangulares
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    n = len(df_points)
    graph = [[] for _ in range(n)]
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    for i in range(n):
        for j in range(i+1, n):
            dtheta = abs(theta_arr[i] - theta_arr[j])
            dr = abs(r_arr[i] - r_arr[j])
            # Se conecta si las diferencias son menores o iguales a los umbrales
            if dtheta <= theta_diff and dr <= r_diff:
                graph[i].append(j)
                graph[j].append(i)
    return graph, n

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False]*n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            cluster_df.reset_index(drop=True, inplace=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    # Ordena según el valor de theta o r y subdivide si hay un salto mayor al gap_threshold
    if mode == 'theta':
        df_cluster = df_cluster.sort_values(by='theta').reset_index(drop=True)
        arr = df_cluster['theta'].values
    else:
        df_cluster = df_cluster.sort_values(by='r').reset_index(drop=True)
        arr = df_cluster['r'].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    # Construcción del grafo y obtención de clusters (grupos semilla)
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        # Subdividir primero por theta y luego por r
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    return final_clusters, df_filtered

def plot_clusters_rectangular_gaps(clusters, title="Clusters con BFS + Gaps"):
    plt.figure(figsize=(10,6))
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    for idx, dfc in enumerate(clusters):
        c = colors[idx % len(colors)]
        plt.scatter(dfc['theta'], dfc['r'], s=10, color=c, label=f"Grupo {idx+1}")
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()


# ========================
# Parte 2: Proceso iterativo utilizando los grupos semilla
# ========================

def process_spiral_arms_with_seeds(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,     # Vecindad horizontal para seeds
    r_diff=0.68,        # Vecindad radial para seeds
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10
):
    # 1. Cargar datos y convertir a coordenadas polares
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    # 2. Filtrar según el percentil de 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 3. Obtener grupos semilla con segmentación rectangular
    seeds_all, df_seed_source = process_clusters_rectangular_gaps(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    # Se filtran aquellos grupos con más de 60 puntos
    seed_clusters = [seed for seed in seeds_all if len(seed) > 60]
    print(f"Grupos semilla con más de 60 puntos: {len(seed_clusters)}")
    
    # Marcar en df_filtered los puntos que pertenecen a un seed
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 4. Inicialización de grupos mediante regresión lineal
    model = LinearRegression()
    
    def calculate_distance(slope, intercept, theta, r):
        pred_r = slope * theta + intercept
        return abs(pred_r - r)
    
    def calculate_pa(slope, intercept):
        if intercept != 0:
            return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
        return np.nan
    
    def is_valid_pa(pa, valid_range):
        return valid_range[0] <= abs(pa) <= valid_range[1]
    
    groups = []
    for idx, seed in enumerate(seed_clusters):
        if len(seed) >= 3:
            X_ = seed[['theta']].values
            y_ = seed['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'seed_id': idx,
            'points': seed.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # 5. Preparar puntos restantes (los que no están en los seeds) para asignación iterativa
    df_local = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    discarded_points = []
    states = []  # Para guardar estados intermedios
    iteration = 0
    
    # 6. Asignación iterativa de puntos
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        assigned = False
        
        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue
            dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
            if dist <= radius_tolerance:
                distances = g['points'].apply(
                    lambda row: calculate_distance(g['slope'], g['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if all(d <= point_distance_tolerance for d in distances):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) <= slope_variation_threshold:
                        # Actualiza el grupo con el nuevo punto
                        g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(g['slope'], g['intercept'])
                        assigned = True
                        break
        if not assigned:
            discarded_points.append(point.to_dict())
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            groups_copy = copy.deepcopy(groups)
            discarded_copy = copy.deepcopy(discarded_points)
            states.append({
                'iteration': iteration,
                'groups': groups_copy,
                'discarded': discarded_copy,
                'remaining': df_local.copy()
            })
    
    # 7. Reasignar los puntos descartados en una etapa final
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            theta_pt = p['theta']
            r_pt = p['r']
            assigned = False
            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
                if dist <= radius_tolerance:
                    if g['pa'] is not None and is_valid_pa(g['pa'], flexible_pa_range):
                        X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                        y_new = np.hstack([g['points']['r'].values, r_pt])
                        model.fit(X_new, y_new)
                        new_slope = model.coef_[0]
                        old_slope = g['slope']
                        if abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) <= slope_variation_threshold:
                            g['points'] = pd.concat([g['points'], p.to_frame().T], ignore_index=True)
                            g['slope'] = new_slope
                            g['intercept'] = model.intercept_
                            g['pa'] = calculate_pa(g['slope'], g['intercept'])
                            assigned = True
                            break
            if not assigned:
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new, i_new)
                if is_valid_pa(pa_new, flexible_pa_range):
                    groups.append({
                        'seed_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })
    
    # 8. (Opcional) Subdividir grupos si existe un gap grande en theta
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    def subdivide_if_gap(group, max_gap=max_theta_gap):
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)
        seg1 = {
            'seed_id': group['seed_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'seed_id': group['seed_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]
    
    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap=max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups
    
    # 9. Filtrar grupos que cumplan la condición de PA válida
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and is_valid_pa(pa_val, flexible_pa_range):
                valid_groups.append(grp)
    
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'discarded': copy.deepcopy(discarded_points),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered

def plot_intermediate_state(state, df_all):
    """
    Grafica el estado intermedio en el plano polar:
      - Fondo con todos los datos en gris pálido.
      - Los grupos asignados se muestran en colores y, si es posible, se traza la línea de ajuste.
      - Los puntos descartados se marcan con una 'x' negra.
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    theta_all = np.radians(df_all['theta'])
    r_all = df_all['r']
    ax.scatter(theta_all, r_all, s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        theta_vals = np.radians(pts['theta'])
        r_vals = pts['r']
        color = colors[idx % len(colors)]
        ax.scatter(theta_vals, r_vals, s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)
    
    if len(state['discarded']) > 0:
        df_desc = pd.DataFrame(state['discarded'])
        theta_desc = np.radians(df_desc['theta'])
        r_desc = df_desc['r']
        ax.scatter(theta_desc, r_desc, s=10, color='black', marker='x', label='Descartados')
    
    ax.set_title(f"Estado en iteración {state['iteration']}")
    ax.legend(loc='upper right')
    plt.show()


# ========================
# Ejecución interactiva e impresión de resultados
# ========================

states, final_groups, df_filtered = process_spiral_arms_with_seeds(
    id_halo="17",
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,
    r_diff=0.68,
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 40.0),
    slope_variation_threshold=0.18,
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10
)

print("Total de estados guardados:", len(states))
print("Total de grupos finales:", len(final_groups))

# Widget interactivo para ver el proceso iterativo
def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1))


In [0]:
import warnings
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

# ========================
# Parte 1: Segmentación inicial con vecinos rectangulares
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    n = len(df_points)
    graph = [[] for _ in range(n)]
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    for i in range(n):
        for j in range(i+1, n):
            dtheta = abs(theta_arr[i] - theta_arr[j])
            dr = abs(r_arr[i] - r_arr[j])
            # Conecta si las diferencias son menores o iguales a los umbrales
            if dtheta <= theta_diff and dr <= r_diff:
                graph[i].append(j)
                graph[j].append(i)
    return graph, n

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False]*n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            cluster_df.reset_index(drop=True, inplace=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    # Ordena según el valor de theta o r y subdivide si hay un gap mayor al umbral
    if mode == 'theta':
        df_cluster = df_cluster.sort_values(by='theta').reset_index(drop=True)
        arr = df_cluster['theta'].values
    else:
        df_cluster = df_cluster.sort_values(by='r').reset_index(drop=True)
        arr = df_cluster['r'].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    # Construir el grafo y obtener clusters (grupos semilla)
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        # Subdividir por theta y luego por r
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    return final_clusters, df_filtered

def plot_clusters_rectangular_gaps(clusters, title="Clusters con BFS + Gaps"):
    plt.figure(figsize=(10,6))
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    for idx, dfc in enumerate(clusters):
        c = colors[idx % len(colors)]
        plt.scatter(dfc['theta'], dfc['r'], s=10, color=c, label=f"Grupo {idx+1}")
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()

# ========================
# Función para subdividir un grupo si hay gap en θ o en r
# ========================

def subdivide_if_gap(group, max_gap_theta, max_gap_r):
    """
    Subdivide el grupo si existe un gap en θ o en r.
    Primero se ordena por θ y se subdivide si hay un gap mayor a max_gap_theta.
    Luego, para cada subgrupo se ordena por r y se subdivide si hay un gap mayor a max_gap_r.
    Se reestima la línea en cada subgrupo (si hay al menos 2 puntos).
    """
    # Subdivisión por theta
    pts_theta = group['points'].sort_values(by='theta').reset_index(drop=True)
    dtheta = np.diff(pts_theta['theta'].values)
    if not np.any(dtheta > max_gap_theta):
        subgroups_theta = [pts_theta]
    else:
        subgroups_theta = subdivide_by_gap(pts_theta, gap_threshold=max_gap_theta, mode='theta')
    
    final_subgroups = []
    # Para cada subgrupo obtenido, subdividir por r
    for sub in subgroups_theta:
        sub_sorted_r = sub.sort_values(by='r').reset_index(drop=True)
        dr = np.diff(sub_sorted_r['r'].values)
        if not np.any(dr > max_gap_r):
            subgroups_r = [sub_sorted_r]
        else:
            subgroups_r = subdivide_by_gap(sub_sorted_r, gap_threshold=max_gap_r, mode='r')
        # Recalcular el ajuste lineal para cada subdivisión
        for sg in subgroups_r:
            if len(sg) >= 2:
                X_ = sg[['theta']].values
                y_ = sg['r'].values
                model = LinearRegression()
                model.fit(X_, y_)
                s_ = model.coef_[0]
                i_ = model.intercept_
                pa_ = np.degrees(np.arctan((s_ * (180/np.pi)) / i_)) if i_ != 0 else np.nan
            else:
                s_, i_, pa_ = None, None, None
            new_group = {
                'seed_id': group.get('seed_id', None),
                'points': sg,
                'slope': s_,
                'intercept': i_,
                'pa': pa_
            }
            final_subgroups.append(new_group)
    return final_subgroups

# ========================
# Parte 2: Proceso iterativo utilizando los grupos semilla
# ========================

def process_spiral_arms_with_seeds(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,     # Vecindad horizontal para seeds
    r_diff=0.68,        # Vecindad radial para seeds
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    max_r_gap=5.0,    # Umbral para gap en r en la subdivisión
    do_subdivide=True,
    snapshot_interval=10
):
    # 1. Cargar datos y convertir a coordenadas polares
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    # 2. Filtrar según percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 3. Obtener grupos semilla mediante segmentación rectangular
    seeds_all, df_seed_source = process_clusters_rectangular_gaps(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    # Filtrar solo grupos con más de 60 puntos
    seed_clusters = [seed for seed in seeds_all if len(seed) > 60]
    print(f"Grupos semilla con más de 60 puntos: {len(seed_clusters)}")
    
    # Marcar en df_filtered los puntos que pertenecen a un seed
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 4. Inicialización de grupos mediante regresión lineal
    model = LinearRegression()
    
    def calculate_distance(slope, intercept, theta, r):
        pred_r = slope * theta + intercept
        return abs(pred_r - r)
    
    def calculate_pa(slope, intercept):
        if intercept != 0:
            return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
        return np.nan
    
    def is_valid_pa(pa, valid_range):
        return valid_range[0] <= abs(pa) <= valid_range[1]
    
    groups = []
    for idx, seed in enumerate(seed_clusters):
        if len(seed) >= 3:
            X_ = seed[['theta']].values
            y_ = seed['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'seed_id': idx,
            'points': seed.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # 5. Preparar puntos restantes (no asignados a seeds)
    df_local = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    discarded_points = []
    states = []  # Para estados intermedios
    iteration = 0
    
    # 6. Asignación iterativa de puntos
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        assigned = False
        
        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue
            dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
            if dist <= radius_tolerance:
                distances = g['points'].apply(
                    lambda row: calculate_distance(g['slope'], g['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if all(d <= point_distance_tolerance for d in distances):
                    X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                    y_new = np.hstack([g['points']['r'].values, r_pt])
                    model.fit(X_new, y_new)
                    new_slope = model.coef_[0]
                    old_slope = g['slope']
                    if abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) <= slope_variation_threshold:
                        # Actualizar grupo con el nuevo punto
                        g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                        g['slope'] = new_slope
                        g['intercept'] = model.intercept_
                        g['pa'] = calculate_pa(g['slope'], g['intercept'])
                        assigned = True
                        break
        if not assigned:
            discarded_points.append(point.to_dict())
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            groups_copy = copy.deepcopy(groups)
            discarded_copy = copy.deepcopy(discarded_points)
            states.append({
                'iteration': iteration,
                'groups': groups_copy,
                'discarded': discarded_copy,
                'remaining': df_local.copy()
            })
    
    # 7. Reasignar puntos descartados (etapa final)
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            theta_pt = p['theta']
            r_pt = p['r']
            assigned = False
            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                dist = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
                if dist <= radius_tolerance:
                    if g['pa'] is not None and is_valid_pa(g['pa'], flexible_pa_range):
                        X_new = np.vstack([g['points'][['theta']].values, [[theta_pt]]])
                        y_new = np.hstack([g['points']['r'].values, r_pt])
                        model.fit(X_new, y_new)
                        new_slope = model.coef_[0]
                        old_slope = g['slope']
                        if abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) <= slope_variation_threshold:
                            g['points'] = pd.concat([g['points'], p.to_frame().T], ignore_index=True)
                            g['slope'] = new_slope
                            g['intercept'] = model.intercept_
                            g['pa'] = calculate_pa(g['slope'], g['intercept'])
                            assigned = True
                            break
            if not assigned:
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new, i_new)
                if is_valid_pa(pa_new, flexible_pa_range):
                    groups.append({
                        'seed_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })
    
    # 8. Subdividir grupos si existe un gap grande en θ o en r
    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap_theta=max_theta_gap, max_gap_r=max_r_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups
    
    # 9. Filtrar grupos con PA válida
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'discarded': copy.deepcopy(discarded_points),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered
def plot_intermediate_state(state, df_all):
    """
    Grafica el estado intermedio en el plano polar:
      - Fondo: todos los datos en gris pálido.
      - Grupos asignados: cada grupo con un color y su línea de ajuste (si se ha calculado).
      - Puntos descartados: marcados con una 'x' negra.
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    theta_all = np.radians(df_all['theta'].values)
    r_all = df_all['r']
    ax.scatter(theta_all, r_all, s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        theta_vals = np.radians(pts['theta'].values)
        r_vals = pts['r']
        color = colors[idx % len(colors)]
        ax.scatter(theta_vals, r_vals, s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)
    
    if len(state['discarded']) > 0:
        df_desc = pd.DataFrame(state['discarded'])
        theta_desc = np.radians(df_desc['theta'].values)
        r_desc = df_desc['r']
        ax.scatter(theta_desc, r_desc, s=10, color='black', marker='x', label='Descartados')
    
    ax.set_title(f"Estado en iteración {state['iteration']}")
    ax.legend(loc='upper right')
    plt.show()



# ========================
# Ejecución interactiva e impresión de resultados
# ========================

states, final_groups, df_filtered = process_spiral_arms_with_seeds(
    id_halo="17",
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,
    r_diff=0.68,
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 40.0),
    slope_variation_threshold=0.18,
    max_theta_gap=15.0,
    max_r_gap=5.0,
    do_subdivide=True,
    snapshot_interval=10
)

print("Total de estados guardados:", len(states))
print("Total de grupos finales:", len(final_groups))

# Widget interactivo para ver el proceso iterativo
def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))




**Resumen del Proceso y Código Integrado:**

1. **Segmentación Inicial con Vecinos Rectangulares:**  
   - Se construyó un grafo en el que cada punto se conecta a otros si la diferencia en θ y r es menor o igual a unos umbrales predefinidos.  
   - Se aplicó una búsqueda en anchura (BFS) para extraer clusters de puntos (los grupos semilla).  
   - Se subdividieron estos clusters usando una función `subdivide_by_gap` que detecta “gaps” en los valores de θ o r.  
   - Finalmente, se filtran los grupos semilla para conservar solo aquellos con más de 60 puntos.

2. **Proceso Iterativo de Asignación y Validación:**  
   - Se cargaron los datos y se calcularon las coordenadas polares, filtrando los puntos por percentil en la columna `rho_resta_final_exp`.  
   - Se inicializaron los grupos semilla aplicando una regresión lineal para obtener los parámetros de ajuste (pendiente, intersección y un parámetro adicional denominado *pa*).  
   - Se asignaron de forma iterativa los puntos restantes a los grupos existentes, validando que la actualización de la pendiente no varíe demasiado (usando un umbral para la variación).  
   - Los puntos que no cumplían los criterios se almacenaron como "descartados" y se reevalúan al final para asignarlos (o crear nuevos grupos).

3. **Subdivisión Adicional en Caso de Gaps en θ o r:**  
   - Se implementó una función `subdivide_if_gap` que primero ordena los puntos por θ y subdivide si se detecta un gap mayor a un umbral (`max_theta_gap`).
   - Luego, cada subgrupo se ordena por r y se subdivide si hay un gap mayor a otro umbral (`max_r_gap`).
   - En cada subgrupo resultante se recalcula el ajuste lineal para actualizar los parámetros.

4. **Visualización con Widgets Interactivos:**  
   - Se utiliza la biblioteca `ipywidgets` y la función `interact` para crear un slider que permite recorrer los diferentes estados intermedios del proceso de asignación y visualizarlos.  
   - Se corrigió el error en la función de graficación forzando la conversión de las columnas de θ a arrays de NumPy (usando `.values`) para evitar que se intente aplicar `np.radians` sobre un valor escalar.

---

**Funcionamiento General:**

- El código integra el proceso de segmentación inicial para generar grupos semilla y luego utiliza esos grupos para iniciar un proceso iterativo en el que se asignan y validan puntos.  
- Durante la asignación se verifica que los puntos se encuentren dentro de una tolerancia respecto a la línea de ajuste, manteniendo la consistencia de la pendiente.  
- Finalmente, se permite una subdivisión adicional de los grupos en caso de que se detecten gaps grandes en las variables θ o r, y se visualizan los resultados interactivamente con un widget.

Este resumen puede servirte como referencia para estructurar futuros prompts o para recordar el flujo general y las modificaciones realizadas en el código.

In [0]:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

# ========================
# Funciones de Preprocesamiento
# ========================

def load_and_filter_data(id_halo, theta_min, theta_max):
    """
    Carga los datos desde CSV, calcula coordenadas polares y filtra puntos según el rango de θ.
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    return df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()

def calculate_pa(slope):
    """
    Calcula el ángulo (en grados) correspondiente a la pendiente usando la función arctan.
    Si la pendiente es None, retorna None.
    """
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

# ========================
# Parte 1: Segmentación inicial con vecinos rectangulares
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo donde cada punto se conecta a otros si las diferencias en θ y r
    son menores o iguales a los umbrales especificados. Se utiliza una aproximación
    vectorizada para cada punto.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Excluir el mismo índice
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    """
    Realiza una búsqueda en anchura (BFS) sobre el grafo para extraer componentes conexas.
    Cada componente se retorna como un DataFrame de puntos.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Ordena el DataFrame según 'theta' o 'r' y subdivide en subclusters si se detecta un gap mayor
    que el umbral especificado.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i + 1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    """
    Carga y filtra los datos, construye el grafo, extrae componentes (clusters) y los subdivide
    en función de gaps en θ y r.
    """
    df = load_and_filter_data(id_halo, theta_min, theta_max)
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    return final_clusters, df_filtered

def plot_clusters_rectangular_gaps(clusters, title="Clusters con BFS + Gaps"):
    """
    Genera una visualización de los clusters obtenidos.
    """
    plt.figure(figsize=(10,6))
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    for idx, dfc in enumerate(clusters):
        color = colors[idx % len(colors)]
        plt.scatter(dfc['theta'], dfc['r'], s=10, color=color, label=f"Grupo {idx+1}")
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()

# ========================
# Función para subdividir un grupo si hay gap en θ o en r
# ========================

def subdivide_if_gap(group, max_gap_theta, max_gap_r):
    """
    Subdivide el grupo si existe un gap en θ o en r.
    Primero se ordena por θ y se subdivide si hay un gap mayor a max_gap_theta.
    Luego, para cada subgrupo se ordena por r y se subdivide si hay un gap mayor a max_gap_r.
    Se reestima el ajuste lineal para cada subdivisión (si hay al menos 2 puntos).
    """
    # Verificar que 'points' sea un DataFrame
    if not isinstance(group.get('points', None), pd.DataFrame):
        print("Error en subdivide_if_gap: group['points'] no es un DataFrame.")
        print("Valor recibido:", group.get('points'))
        raise ValueError(f"Se esperaba un DataFrame en 'points', pero se obtuvo {type(group.get('points', None))}")
    
    pts_theta = group['points'].sort_values(by='theta').reset_index(drop=True)
    dtheta = np.diff(pts_theta['theta'].values)
    if not np.any(dtheta > max_gap_theta):
        subgroups_theta = [pts_theta]
    else:
        subgroups_theta = subdivide_by_gap(pts_theta, gap_threshold=max_gap_theta, mode='theta')
    
    final_subgroups = []
    for sub in subgroups_theta:
        sub_sorted_r = sub.sort_values(by='r').reset_index(drop=True)
        dr = np.diff(sub_sorted_r['r'].values)
        if not np.any(dr > max_gap_r):
            subgroups_r = [sub_sorted_r]
        else:
            subgroups_r = subdivide_by_gap(sub_sorted_r, gap_threshold=max_gap_r, mode='r')
        for sg in subgroups_r:
            if len(sg) >= 2:
                X_ = sg[['theta']].values
                y_ = sg['r'].values
                model = LinearRegression()
                model.fit(X_, y_)
                s_ = model.coef_[0]
                i_ = model.intercept_
                pa_ = calculate_pa(s_)
            else:
                s_, i_, pa_ = None, None, None
            new_group = {
                'seed_id': group.get('seed_id', None),
                'points': sg,
                'slope': s_,
                'intercept': i_,
                'pa': pa_
            }
            final_subgroups.append(new_group)
    return final_subgroups

# ========================
# Función auxiliar para asignar un punto a un grupo (utilizada en el proceso iterativo)
# ========================

def try_assign_point(point, group, model, radius_tolerance, point_distance_tolerance, slope_variation_threshold):
    """
    Intenta asignar 'point' al 'group' si cumple con la tolerancia de distancia al ajuste actual.
    Actualiza la línea de ajuste y retorna True si la asignación es exitosa.
    """
    theta_pt, r_pt = point['theta'], point['r']
    pred_r = group['slope'] * theta_pt + group['intercept']
    dist = abs(pred_r - r_pt)
    if dist > radius_tolerance:
        return False
    
    distances = group['points'].apply(
        lambda row: abs(group['slope'] * row['theta'] + group['intercept'] - row['r']),
        axis=1
    )
    if not all(d <= point_distance_tolerance for d in distances):
        return False
    
    X_new = np.vstack([group['points'][['theta']].values, [[theta_pt]]])
    y_new = np.hstack([group['points']['r'].values, r_pt])
    model.fit(X_new, y_new)
    new_slope = model.coef_[0]
    if abs(new_slope - group['slope']) / (abs(group['slope']) + 1e-12) > slope_variation_threshold:
        return False
    
    group['points'] = pd.concat([group['points'], point.to_frame().T], ignore_index=True)
    group['slope'] = new_slope
    group['intercept'] = model.intercept_
    group['pa'] = calculate_pa(new_slope)
    return True

# ========================
# Parte 2: Proceso iterativo utilizando los grupos semilla
# ========================

def process_spiral_arms_with_seeds(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,     # Vecindad horizontal para seeds
    r_diff=0.68,        # Vecindad radial para seeds
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    max_r_gap=5.0,    # Umbral para gap en r en la subdivisión
    do_subdivide=True,
    snapshot_interval=10
):
    """
    Ejecuta el proceso iterativo de asignación de puntos a grupos (espirales) usando seeds iniciales.
    Incluye reasignación de puntos descartados y subdivisión adicional si se detectan gaps.
    Retorna los estados intermedios, los grupos finales válidos y el DataFrame filtrado.
    """
    # 1. Cargar y filtrar datos
    df = load_and_filter_data(id_halo, theta_min, theta_max)
    
    # 2. Filtrar según percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 3. Obtener grupos semilla mediante segmentación rectangular
    seeds_all, _ = process_clusters_rectangular_gaps(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    # Filtrar solo seeds con más de 60 puntos
    seed_clusters = [seed for seed in seeds_all if len(seed) > 60]
    print(f"Grupos semilla con más de 60 puntos: {len(seed_clusters)}")
    
    # Marcar en df_filtered los puntos que pertenecen a un seed
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 4. Inicialización de grupos mediante regresión lineal
    model = LinearRegression()
    groups = []
    for idx, seed in enumerate(seed_clusters):
        if len(seed) >= 3:
            X_ = seed[['theta']].values
            y_ = seed['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'seed_id': idx,
            'points': seed.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # 5. Preparar puntos restantes (no asignados a seeds)
    df_local = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    discarded_points = []
    states = []  # Para guardar estados intermedios
    iteration = 0
    
    # 6. Asignación iterativa de puntos
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        assigned = False
        
        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue
            if try_assign_point(point, g, model, radius_tolerance, point_distance_tolerance, slope_variation_threshold):
                assigned = True
                break
        if not assigned:
            discarded_points.append(point.to_dict())
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'discarded': copy.deepcopy(discarded_points),
                'remaining': df_local.copy()
            })
    
    # 7. Reasignar puntos descartados (etapa final)
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            assigned = False
            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                if try_assign_point(p, g, model, radius_tolerance, point_distance_tolerance, slope_variation_threshold):
                    assigned = True
                    break
            if not assigned:
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new)
                if flexible_pa_range[0] <= abs(pa_new) <= flexible_pa_range[1]:
                    groups.append({
                        'seed_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })
    
    # 8. Subdividir grupos si existe un gap grande en θ o en r
    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap_theta=max_theta_gap, max_gap_r=max_r_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups
    
    # 9. Filtrar grupos con PA válida
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'discarded': copy.deepcopy(discarded_points),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered
    
def plot_intermediate_state(state, df_all):
    fig, ax = plt.subplots(figsize=(10, 8))

    # Fondo de todos los datos
    df_all['theta'] = pd.to_numeric(df_all['theta'], errors='coerce')
    theta_all = np.radians(df_all['theta'].values)
    r_all = df_all['r'].values
    ax.scatter(theta_all, r_all, s=1, color='lightgrey', alpha=0.5, label='Datos completos')

    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    
    for idx, g in enumerate(state['groups']):
        pts = g['points'].copy()

        # Verificar que la columna 'theta' sea numérica
        pts['theta'] = pd.to_numeric(pts['theta'], errors='coerce')
        # Si hay nulos, imprimirlos
        if pts['theta'].isna().any():
            print(f"Hay valores NaN en 'theta' para el grupo {idx}. Filas:")
            print(pts[pts['theta'].isna()])

        theta_vals = np.radians(pts['theta'].values)
        r_vals = pts['r'].values
        color = colors[idx % len(colors)]
        ax.scatter(theta_vals, r_vals, s=10, color=color, label=f"Grupo {idx+1}")

        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)

    # Puntos descartados
    if len(state['discarded']) > 0:
        df_desc = pd.DataFrame(state['discarded'])
        df_desc['theta'] = pd.to_numeric(df_desc['theta'], errors='coerce')
        theta_desc = np.radians(df_desc['theta'].values)
        r_desc = df_desc['r'].values
        ax.scatter(theta_desc, r_desc, s=10, color='black', marker='x', label='Descartados')
    
    ax.set_title(f"Estado en iteración {state['iteration']}")
    ax.legend(loc='upper right')
    plt.show()


# ========================
# Ejecución interactiva e impresión de resultados
# ========================

states, final_groups, df_filtered = process_spiral_arms_with_seeds(
    id_halo="17",
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,
    r_diff=0.68,
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 40.0),
    slope_variation_threshold=0.18,
    max_theta_gap=15.0,
    max_r_gap=5.0,
    do_subdivide=True,
    snapshot_interval=10
)

print(f"Total de estados guardados: {len(states)}")
print(f"Total de grupos finales: {len(final_groups)}")

def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))


In [0]:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

# ========================
# Funciones de Preprocesamiento
# ========================

def load_and_filter_data(id_halo, theta_min, theta_max):
    """
    Carga los datos desde CSV, calcula coordenadas polares y filtra puntos según el rango de θ.
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    return df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()

def calculate_pa(slope):
    """
    Calcula el ángulo (en grados) correspondiente a la pendiente usando la función arctan.
    Si la pendiente es None, retorna None.
    """
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

# ========================
# Parte 1: Segmentación inicial con vecinos rectangulares
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo donde cada punto se conecta a otros si las diferencias en θ y r
    son menores o iguales a los umbrales especificados. Se utiliza una aproximación
    vectorizada para cada punto.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Excluir el mismo índice
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    """
    Realiza una búsqueda en anchura (BFS) sobre el grafo para extraer componentes conexas.
    Cada componente se retorna como un DataFrame de puntos.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Ordena el DataFrame según 'theta' o 'r' y subdivide en subclusters si se detecta un gap mayor
    que el umbral especificado.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i + 1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    """
    Carga y filtra los datos, construye el grafo, extrae componentes (clusters) y los subdivide
    en función de gaps en θ y r.
    """
    df = load_and_filter_data(id_halo, theta_min, theta_max)
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    return final_clusters, df_filtered

def plot_clusters_rectangular_gaps(clusters, title="Clusters con BFS + Gaps"):
    """
    Genera una visualización de los clusters obtenidos.
    """
    plt.figure(figsize=(10,6))
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    for idx, dfc in enumerate(clusters):
        color = colors[idx % len(colors)]
        plt.scatter(dfc['theta'], dfc['r'], s=10, color=color, label=f"Grupo {idx+1}")
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()

# ========================
# Función para subdividir un grupo si hay gap en θ o en r
# ========================

def subdivide_if_gap(group, max_gap_theta, max_gap_r):
    """
    Subdivide el grupo si existe un gap en θ o en r.
    Primero se ordena por θ y se subdivide si hay un gap mayor a max_gap_theta.
    Luego, para cada subgrupo se ordena por r y se subdivide si hay un gap mayor a max_gap_r.
    Se reestima el ajuste lineal para cada subdivisión (si hay al menos 2 puntos).
    """
    # Verificar que 'points' sea un DataFrame
    if not isinstance(group.get('points', None), pd.DataFrame):
        print("Error en subdivide_if_gap: group['points'] no es un DataFrame.")
        print("Valor recibido:", group.get('points'))
        raise ValueError(f"Se esperaba un DataFrame en 'points', pero se obtuvo {type(group.get('points', None))}")
    
    pts_theta = group['points'].sort_values(by='theta').reset_index(drop=True)
    dtheta = np.diff(pts_theta['theta'].values)
    if not np.any(dtheta > max_gap_theta):
        subgroups_theta = [pts_theta]
    else:
        subgroups_theta = subdivide_by_gap(pts_theta, gap_threshold=max_gap_theta, mode='theta')
    
    final_subgroups = []
    for sub in subgroups_theta:
        sub_sorted_r = sub.sort_values(by='r').reset_index(drop=True)
        dr = np.diff(sub_sorted_r['r'].values)
        if not np.any(dr > max_gap_r):
            subgroups_r = [sub_sorted_r]
        else:
            subgroups_r = subdivide_by_gap(sub_sorted_r, gap_threshold=max_gap_r, mode='r')
        for sg in subgroups_r:
            if len(sg) >= 2:
                X_ = sg[['theta']].values
                y_ = sg['r'].values
                model = LinearRegression()
                model.fit(X_, y_)
                s_ = model.coef_[0]
                i_ = model.intercept_
                pa_ = calculate_pa(s_)
            else:
                s_, i_, pa_ = None, None, None
            new_group = {
                'seed_id': group.get('seed_id', None),
                'points': sg,
                'slope': s_,
                'intercept': i_,
                'pa': pa_
            }
            final_subgroups.append(new_group)
    return final_subgroups

# ========================
# Función auxiliar para asignar un punto a un grupo (utilizada en el proceso iterativo)
# ========================

def try_assign_point(point, group, model, radius_tolerance, point_distance_tolerance, slope_variation_threshold):
    """
    Intenta asignar 'point' al 'group' si cumple con la tolerancia de distancia al ajuste actual.
    Actualiza la línea de ajuste y retorna True si la asignación es exitosa.
    """
    theta_pt, r_pt = point['theta'], point['r']
    pred_r = group['slope'] * theta_pt + group['intercept']
    dist = abs(pred_r - r_pt)
    if dist > radius_tolerance:
        return False
    
    distances = group['points'].apply(
        lambda row: abs(group['slope'] * row['theta'] + group['intercept'] - row['r']),
        axis=1
    )
    if not all(d <= point_distance_tolerance for d in distances):
        return False
    
    X_new = np.vstack([group['points'][['theta']].values, [[theta_pt]]])
    y_new = np.hstack([group['points']['r'].values, r_pt])
    model.fit(X_new, y_new)
    new_slope = model.coef_[0]
    if abs(new_slope - group['slope']) / (abs(group['slope']) + 1e-12) > slope_variation_threshold:
        return False
    
    group['points'] = pd.concat([group['points'], point.to_frame().T], ignore_index=True)
    group['slope'] = new_slope
    group['intercept'] = model.intercept_
    group['pa'] = calculate_pa(new_slope)
    return True

# ========================
# Parte 2: Proceso iterativo utilizando los grupos semilla
# ========================

def process_spiral_arms_with_seeds(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,     # Vecindad horizontal para seeds
    r_diff=0.68,        # Vecindad radial para seeds
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    max_r_gap=5.0,    # Umbral para gap en r en la subdivisión
    do_subdivide=True,
    snapshot_interval=10
):
    """
    Ejecuta el proceso iterativo de asignación de puntos a grupos (espirales) usando seeds iniciales.
    Incluye reasignación de puntos descartados y subdivisión adicional si se detectan gaps.
    Retorna los estados intermedios, los grupos finales válidos y el DataFrame filtrado.
    """
    # 1. Cargar y filtrar datos
    df = load_and_filter_data(id_halo, theta_min, theta_max)
    
    # 2. Filtrar según percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 3. Obtener grupos semilla mediante segmentación rectangular
    seeds_all, _ = process_clusters_rectangular_gaps(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    # Filtrar solo seeds con más de 60 puntos
    seed_clusters = [seed for seed in seeds_all if len(seed) > 60]
    print(f"Grupos semilla con más de 60 puntos: {len(seed_clusters)}")
    
    # Marcar en df_filtered los puntos que pertenecen a un seed
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 4. Inicialización de grupos mediante regresión lineal
    model = LinearRegression()
    groups = []
    for idx, seed in enumerate(seed_clusters):
        if len(seed) >= 3:
            X_ = seed[['theta']].values
            y_ = seed['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'seed_id': idx,
            'points': seed.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # 5. Preparar puntos restantes (no asignados a seeds)
    df_local = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    # Importante: recorrer de izquierda a derecha (ordenar por θ)
    df_local = df_local.sort_values(by='theta').reset_index(drop=True)
    
    discarded_points = []
    states = []  # Para guardar estados intermedios
    iteration = 0
    
    # 6. Asignación iterativa de puntos
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        assigned = False
        
        for g in groups:
            if g['slope'] is None or g['intercept'] is None:
                continue
            if try_assign_point(point, g, model, radius_tolerance, point_distance_tolerance, slope_variation_threshold):
                assigned = True
                break
        if not assigned:
            discarded_points.append(point.to_dict())
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'discarded': copy.deepcopy(discarded_points),
                'remaining': df_local.copy()
            })
    
    # 7. Reasignar puntos descartados (etapa final)
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            assigned = False
            for g in groups:
                if g['slope'] is None or g['intercept'] is None:
                    continue
                if try_assign_point(p, g, model, radius_tolerance, point_distance_tolerance, slope_variation_threshold):
                    assigned = True
                    break
            if not assigned:
                new_grp = pd.DataFrame([p])
                model.fit(new_grp[['theta']], new_grp['r'])
                s_new = model.coef_[0]
                i_new = model.intercept_
                pa_new = calculate_pa(s_new)
                if flexible_pa_range[0] <= abs(pa_new) <= flexible_pa_range[1]:
                    groups.append({
                        'seed_id': None,
                        'points': new_grp,
                        'slope': s_new,
                        'intercept': i_new,
                        'pa': pa_new
                    })
    
    # 8. Subdividir grupos si existe un gap grande en θ o en r
    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap_theta=max_theta_gap, max_gap_r=max_r_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups
    
    # 9. Filtrar grupos con PA válida
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'discarded': copy.deepcopy(discarded_points),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered
    
def plot_intermediate_state(state, df_all):
    fig, ax = plt.subplots(figsize=(10, 8))

    # Fondo de todos los datos (asegurando conversión a numérico)
    df_all['theta'] = pd.to_numeric(df_all['theta'], errors='coerce')
    theta_all = np.radians(df_all['theta'].values)
    r_all = df_all['r'].values
    ax.scatter(theta_all, r_all, s=1, color='lightgrey', alpha=0.5, label='Datos completos')

    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    
    for idx, g in enumerate(state['groups']):
        pts = g['points'].copy()
        pts['theta'] = pd.to_numeric(pts['theta'], errors='coerce')
        if pts['theta'].isna().any():
            print(f"Hay valores NaN en 'theta' para el grupo {idx}. Filas:")
            print(pts[pts['theta'].isna()])

        theta_vals = np.radians(pts['theta'].values)
        r_vals = pts['r'].values
        color = colors[idx % len(colors)]
        ax.scatter(theta_vals, r_vals, s=10, color=color, label=f"Grupo {idx+1}")

        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)

    if len(state['discarded']) > 0:
        df_desc = pd.DataFrame(state['discarded'])
        df_desc['theta'] = pd.to_numeric(df_desc['theta'], errors='coerce')
        theta_desc = np.radians(df_desc['theta'].values)
        r_desc = df_desc['r'].values
        ax.scatter(theta_desc, r_desc, s=10, color='black', marker='x', label='Descartados')
    
    ax.set_title(f"Estado en iteración {state['iteration']}")
    ax.legend(loc='upper right')
    plt.show()

# ========================
# Ejecución interactiva e impresión de resultados
# ========================

states, final_groups, df_filtered = process_spiral_arms_with_seeds(
    id_halo="17",
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,
    r_diff=0.68,
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 40.0),
    slope_variation_threshold=0.18,
    max_theta_gap=15.0,
    max_r_gap=5.0,
    do_subdivide=True,
    snapshot_interval=10
)

print(f"Total de estados guardados: {len(states)}")
print(f"Total de grupos finales: {len(final_groups)}")

def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))


In [0]:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

# ========================
# Funciones de Preprocesamiento
# ========================

def load_and_filter_data(id_halo, theta_min, theta_max):
    """
    Carga los datos desde CSV, calcula coordenadas polares y filtra puntos según el rango de θ.
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    return df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()

def calculate_pa(slope):
    """
    Calcula el ángulo (en grados) correspondiente a la pendiente usando la función arctan.
    Si la pendiente es None, retorna None.
    """
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

# ========================
# Parte 1: Segmentación inicial con vecinos rectangulares y subdivisión por gaps
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo donde cada punto se conecta a otros si las diferencias en θ y r
    son menores o iguales a los umbrales especificados.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Excluir el mismo índice
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    """
    Realiza una búsqueda en anchura (BFS) sobre el grafo para extraer componentes conexas.
    Cada componente se retorna como un DataFrame de puntos.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Ordena el DataFrame según 'theta' o 'r' y subdivide en subclusters si se detecta un gap mayor
    que el umbral especificado.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i + 1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    """
    Carga y filtra los datos, construye el grafo, extrae componentes (clusters) y los subdivide
    en función de gaps en θ y r.
    """
    df = load_and_filter_data(id_halo, theta_min, theta_max)
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    return final_clusters, df_filtered

def plot_clusters_rectangular_gaps(clusters, title="Clusters con BFS + Gaps"):
    """
    Genera una visualización de los clusters obtenidos.
    """
    plt.figure(figsize=(10,6))
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    for idx, dfc in enumerate(clusters):
        color = colors[idx % len(colors)]
        plt.scatter(dfc['theta'], dfc['r'], s=10, color=color, label=f"Grupo {idx+1}")
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()

# ========================
# Asignación de puntos a grupos sin ajuste (solo agrupación por rango)
# ========================

def try_assign_point_to_group(point, group, theta_tol=2.0, r_tol=0.5):
    """
    Asigna 'point' al 'group' si su valor de θ y r está dentro de un rango definido
    por el mínimo y máximo del grupo, con una tolerancia.
    """
    # Aseguramos que el punto sea una Series
    if isinstance(point, dict):
        point = pd.DataFrame([point]).iloc[0]
    theta_pt, r_pt = point['theta'], point['r']
    group_theta_min = group['points']['theta'].min()
    group_theta_max = group['points']['theta'].max()
    group_r_min = group['points']['r'].min()
    group_r_max = group['points']['r'].max()
    if (theta_pt >= group_theta_min - theta_tol) and (theta_pt <= group_theta_max + theta_tol) and \
       (r_pt >= group_r_min - r_tol) and (r_pt <= group_r_max + r_tol):
        new_row = point.to_frame().T
        group['points'] = pd.concat([group['points'], new_row], ignore_index=True)
        return True
    return False

# ========================
# Finalización recursiva de segmentos: aplicar ajuste solo si el grupo tiene >=80 puntos y sin huecos.
# Si hay huecos, subdividir y repetir.
# ========================

def check_no_gap(df_group, gap_threshold_theta, gap_threshold_r):
    """
    Verifica que en el grupo no existan diferencias mayores que los umbrales en θ ni en r.
    """
    sorted_theta = df_group.sort_values(by='theta')['theta'].values
    sorted_r = df_group.sort_values(by='r')['r'].values
    if len(sorted_theta) > 1 and np.max(np.diff(sorted_theta)) > gap_threshold_theta:
        return False
    if len(sorted_r) > 1 and np.max(np.diff(sorted_r)) > gap_threshold_r:
        return False
    return True

def process_group(pts, min_points, gap_threshold_theta, gap_threshold_r, seed_id=None):
    """
    Procesa recursivamente un grupo de puntos:
      - Si el grupo tiene menos de min_points, se retorna sin ajuste.
      - Si no hay huecos y tiene >= min_points, se aplica la regresión.
      - Si hay huecos, se subdivide primero por θ y luego por r, y se procesa cada subgrupo.
    """
    pts = pts.copy().reset_index(drop=True)
    if len(pts) < min_points:
        return [{'seed_id': seed_id, 'points': pts, 'slope': None, 'intercept': None, 'pa': None}]
    
    if check_no_gap(pts, gap_threshold_theta, gap_threshold_r):
        model = LinearRegression()
        X = pts[['theta']].values
        y = pts['r'].values
        model.fit(X, y)
        slope = model.coef_[0]
        intercept = model.intercept_
        pa = calculate_pa(slope)
        return [{'seed_id': seed_id, 'points': pts, 'slope': slope, 'intercept': intercept, 'pa': pa}]
    else:
        segments = []
        sub_theta = subdivide_by_gap(pts, gap_threshold=gap_threshold_theta, mode='theta')
        for sub in sub_theta:
            sub_r = subdivide_by_gap(sub, gap_threshold=gap_threshold_r, mode='r')
            for sg in sub_r:
                segments.extend(process_group(sg, min_points, gap_threshold_theta, gap_threshold_r, seed_id=seed_id))
        return segments

def finalize_segments(groups, min_points=80, gap_threshold_theta=2.5, gap_threshold_r=2.5):
    """
    Para cada grupo se aplica el proceso recursivo:
      - Si el grupo es semilla (tiene seed_id) y cumple la condición (>= min_points y sin huecos),
        se mantiene como grupo base actualizando su ajuste.
      - En caso contrario, se subdivide recursivamente.
    Retorna la lista de segmentos finales.
    """
    final_segments = []
    for g in groups:
        pts = g['points']
        seed_id = g.get('seed_id')
        # Si es un grupo semilla y cumple la condición, se actualiza y se deja como está
        if seed_id is not None and len(pts) >= min_points and check_no_gap(pts, gap_threshold_theta, gap_threshold_r):
            model = LinearRegression()
            X = pts[['theta']].values
            y = pts['r'].values
            model.fit(X, y)
            g['slope'] = model.coef_[0]
            g['intercept'] = model.intercept_
            g['pa'] = calculate_pa(g['slope'])
            final_segments.append(g)
        else:
            # Para grupos que no cumplen la condición (o no son semilla), se procesa recursivamente
            segments = process_group(pts, min_points, gap_threshold_theta, gap_threshold_r, seed_id=seed_id)
            final_segments.extend(segments)
    return final_segments

# ========================
# Parte 2: Proceso iterativo utilizando los grupos semilla (sin ajuste inicial)
# ========================

def process_spiral_arms_with_seeds(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,     # Vecindad horizontal para seeds
    r_diff=0.68,        # Vecindad radial para seeds
    gap_threshold_theta=1.9,
    gap_threshold_r=1.23,
    snapshot_interval=10,
    theta_assignment_tol=2.0,
    r_assignment_tol=0.5,
    min_points_final=80  # Mínimo de puntos para aplicar el ajuste final
):
    """
    Ejecuta el proceso iterativo de asignación de puntos a grupos usando los grupos semilla como base.
    Primero se agrupan los puntos (sin ajuste), luego se asignan los puntos restantes según su rango.
    Finalmente, se finalizan los segmentos: si un grupo (preferiblemente una semilla) tiene al menos
    min_points_final y no presenta huecos, se realiza la regresión; de lo contrario, se subdivide recursivamente.
    Retorna los estados intermedios, los segmentos finales y el DataFrame filtrado.
    """
    # 1. Cargar y filtrar datos
    df = load_and_filter_data(id_halo, theta_min, theta_max)
    
    # 2. Filtrar según percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 3. Obtener grupos semilla mediante segmentación rectangular
    seeds_all, _ = process_clusters_rectangular_gaps(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    # Filtrar solo seeds con más de 60 puntos
    seed_clusters = [seed for seed in seeds_all if len(seed) > 60]
    print(f"Grupos semilla con más de 60 puntos: {len(seed_clusters)}")
    
    # Marcar en df_filtered los puntos que pertenecen a un seed
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 4. Inicialización de grupos a partir de seeds SIN aplicar ajuste (sólo agrupación)
    groups = []
    for idx, seed in enumerate(seed_clusters):
        groups.append({
            'seed_id': idx,
            'points': seed.copy(),
            'slope': None,
            'intercept': None,
            'pa': None
        })
    
    # 5. Preparar puntos restantes (los que no pertenecen a ningún seed)
    df_local = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    df_local = df_local.sort_values(by='theta').reset_index(drop=True)
    
    discarded_points = []  # Puntos que no se pudieron asignar inicialmente
    states = []            # Para guardar estados intermedios
    iteration = 0
    
    # 6. Asignación iterativa de puntos a grupos usando criterio basado en rango
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        assigned = False
        for g in groups:
            if try_assign_point_to_group(point, g, theta_tol=theta_assignment_tol, r_tol=r_assignment_tol):
                assigned = True
                break
        if not assigned:
            discarded_points.append(point.to_dict())
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'discarded': copy.deepcopy(discarded_points),
                'remaining': df_local.copy()
            })
    
    # 7. Reasignar puntos descartados
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            for g in groups:
                if try_assign_point_to_group(p, g, theta_tol=theta_assignment_tol, r_tol=r_assignment_tol):
                    break
    
    # 8. Finalización de segmentos: se aplica el proceso recursivo para evaluar gaps y, si corresponde, aplicar ajuste
    final_segments = finalize_segments(groups, min_points=min_points_final, gap_threshold_theta=gap_threshold_theta, gap_threshold_r=gap_threshold_r)
    
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(final_segments),
        'discarded': copy.deepcopy(discarded_points),
        'remaining': df_local.copy()
    })
    
    return states, final_segments, df_filtered

def plot_intermediate_state(state, df_all):
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Fondo de todos los datos (asegurando conversión a numérico)
    df_all['theta'] = pd.to_numeric(df_all['theta'], errors='coerce')
    theta_all = np.radians(df_all['theta'].values)
    r_all = df_all['r'].values
    ax.scatter(theta_all, r_all, s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    
    for idx, g in enumerate(state['groups']):
        pts = g['points'].copy()
        pts['theta'] = pd.to_numeric(pts['theta'], errors='coerce')
        theta_vals = np.radians(pts['theta'].values)
        r_vals = pts['r'].values
        color = colors[idx % len(colors)]
        ax.scatter(theta_vals, r_vals, s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)
    
    if len(state['discarded']) > 0:
        df_desc = pd.DataFrame(state['discarded'])
        df_desc['theta'] = pd.to_numeric(df_desc['theta'], errors='coerce')
        theta_desc = np.radians(df_desc['theta'].values)
        r_desc = df_desc['r'].values
        ax.scatter(theta_desc, r_desc, s=10, color='black', marker='x', label='Descartados')
    
    ax.set_title(f"Estado en iteración {state['iteration']}")
    ax.legend(loc='upper right')
    plt.show()

# ========================
# Ejecución interactiva e impresión de resultados
# ========================

states, final_segments, df_filtered = process_spiral_arms_with_seeds(
    id_halo="17",
    quartile_threshold=0.155,
    theta_min=50,
    theta_max=250,
    theta_diff=2.0,
    r_diff=0.468,
    gap_threshold_theta=1.89,
    gap_threshold_r=0.523,
    snapshot_interval=10,
    theta_assignment_tol=2.0,
    r_assignment_tol=0.5,
    min_points_final=80
)

print(f"Total de estados guardados: {len(states)}")
print(f"Total de segmentos finales: {len(final_segments)}")

def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))


In [0]:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

# ========================
# Funciones de Preprocesamiento
# ========================

def load_and_filter_data(id_halo, theta_min, theta_max):
    """
    Carga los datos desde CSV, calcula coordenadas polares y filtra puntos según el rango de θ.
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    return df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()

def calculate_pa(slope):
    """
    Calcula el ángulo (en grados) correspondiente a la pendiente usando la función arctan.
    Si la pendiente es None, retorna None.
    """
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

# ========================
# Parte 1: Segmentación inicial con vecinos rectangulares y subdivisión por gaps
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo donde cada punto se conecta a otros si las diferencias en θ y r
    son menores o iguales a los umbrales especificados.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Excluir el mismo índice
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    """
    Realiza una búsqueda en anchura (BFS) sobre el grafo para extraer componentes conexas.
    Cada componente se retorna como un DataFrame de puntos.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Ordena el DataFrame según 'theta' o 'r' y subdivide en subclusters si se detecta un gap mayor
    que el umbral especificado.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i + 1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    """
    Carga y filtra los datos, construye el grafo, extrae componentes (clusters) y los subdivide
    en función de gaps en θ y r.
    """
    df = load_and_filter_data(id_halo, theta_min, theta_max)
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    return final_clusters, df_filtered

def plot_clusters_rectangular_gaps(clusters, title="Clusters con BFS + Gaps"):
    """
    Genera una visualización de los clusters obtenidos.
    """
    plt.figure(figsize=(10,6))
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    for idx, dfc in enumerate(clusters):
        color = colors[idx % len(colors)]
        plt.scatter(dfc['theta'], dfc['r'], s=10, color=color, label=f"Grupo {idx+1}")
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()

# ========================
# Asignación de puntos a grupos (sin ajuste inicial) basada en rango
# ========================

def try_assign_point_to_group(point, group, theta_tol=2.0, r_tol=0.5):
    """
    Asigna 'point' al 'group' si su valor de θ y r está dentro de un rango definido
    por el mínimo y máximo del grupo, con una tolerancia.
    """
    if isinstance(point, dict):
        point = pd.DataFrame([point]).iloc[0]
    theta_pt, r_pt = point['theta'], point['r']
    group_theta_min = group['points']['theta'].min()
    group_theta_max = group['points']['theta'].max()
    group_r_min = group['points']['r'].min()
    group_r_max = group['points']['r'].max()
    if (theta_pt >= group_theta_min - theta_tol) and (theta_pt <= group_theta_max + theta_tol) and \
       (r_pt >= group_r_min - r_tol) and (r_pt <= group_r_max + r_tol):
        new_row = point.to_frame().T
        group['points'] = pd.concat([group['points'], new_row], ignore_index=True)
        return True
    return False

# ========================
# Función para ajustar hasta dos rectas a cada grupo
# ========================

def adjust_two_lines(pts, min_segment_points=10, improvement_ratio_threshold=0.7):
    """
    Dado un DataFrame de puntos (ordenado por 'theta'), intenta ajustar dos rectas.
    
    - Si el número de puntos es insuficiente para dividir (menor que 2*min_segment_points),
      se ajusta solo una recta.
    - Se ajusta una recta única a todos los puntos y se calcula su error (suma de residuos al cuadrado).
    - Se prueban posibles puntos de quiebre (breakpoints) para dividir en dos segmentos y se
      calcula el error total (suma de errores de ambos segmentos).
    - Si la mejora es significativa (error_total < improvement_ratio_threshold * error_unico),
      se retornan dos segmentos; en caso contrario, se retorna un único segmento.
    """
    pts = pts.copy().reset_index(drop=True)
    n = len(pts)
    if n < 2 * min_segment_points:
        # No es posible dividir, se ajusta una recta
        model = LinearRegression()
        X = pts[['theta']].values
        y = pts['r'].values
        model.fit(X, y)
        seg = {'points': pts, 'slope': model.coef_[0],
               'intercept': model.intercept_, 'pa': calculate_pa(model.coef_[0])}
        return [seg]
    
    # Ajuste con una recta única
    model_all = LinearRegression()
    X_all = pts[['theta']].values
    y_all = pts['r'].values
    model_all.fit(X_all, y_all)
    error_all = np.sum((model_all.predict(X_all) - y_all)**2)
    
    best_error = np.inf
    best_split = None
    # Se prueban puntos de quiebre, asegurando que cada segmento tenga al menos min_segment_points
    for i in range(min_segment_points, n - min_segment_points + 1):
        pts_left = pts.iloc[:i]
        pts_right = pts.iloc[i:]
        model_left = LinearRegression()
        model_right = LinearRegression()
        X_left = pts_left[['theta']].values
        y_left = pts_left['r'].values
        X_right = pts_right[['theta']].values
        y_right = pts_right['r'].values
        model_left.fit(X_left, y_left)
        model_right.fit(X_right, y_right)
        error_left = np.sum((model_left.predict(X_left) - y_left)**2)
        error_right = np.sum((model_right.predict(X_right) - y_right)**2)
        total_error = error_left + error_right
        if total_error < best_error:
            best_error = total_error
            best_split = i

    if best_split is None:
        seg = {'points': pts, 'slope': model_all.coef_[0],
               'intercept': model_all.intercept_, 'pa': calculate_pa(model_all.coef_[0])}
        return [seg]

    if best_error < improvement_ratio_threshold * error_all:
        # Se opta por dos segmentos
        pts_left = pts.iloc[:best_split]
        pts_right = pts.iloc[best_split:]
        model_left = LinearRegression()
        model_right = LinearRegression()
        X_left = pts_left[['theta']].values
        y_left = pts_left['r'].values
        X_right = pts_right[['theta']].values
        y_right = pts_right['r'].values
        model_left.fit(X_left, y_left)
        model_right.fit(X_right, y_right)
        seg_left = {'points': pts_left, 'slope': model_left.coef_[0],
                    'intercept': model_left.intercept_, 'pa': calculate_pa(model_left.coef_[0])}
        seg_right = {'points': pts_right, 'slope': model_right.coef_[0],
                     'intercept': model_right.intercept_, 'pa': calculate_pa(model_right.coef_[0])}
        return [seg_left, seg_right]
    else:
        # No se obtiene una mejora significativa; se opta por un único segmento.
        seg = {'points': pts, 'slope': model_all.coef_[0],
               'intercept': model_all.intercept_, 'pa': calculate_pa(model_all.coef_[0])}
        return [seg]

# ========================
# Finalización recursiva de segmentos usando ajuste de hasta dos rectas
# ========================

def finalize_segments(groups, min_points=80, gap_threshold_theta=2.05, gap_threshold_r=2.05,
                      min_segment_points=10, improvement_ratio_threshold=0.7):
    """
    Para cada grupo (base, preferentemente semilla), se intenta ajustar hasta dos rectas.
    Si el grupo es muy pequeño o uniforme se ajusta una recta; en caso contrario, se
    busca dividir el grupo en dos segmentos. Si no se logra una división significativa,
    se retorna un único segmento.
    Retorna la lista de segmentos finales.
    """
    final_segments = []
    for g in groups:
        pts = g['points']
        if len(pts) < min_points:
            seg = {'seed_id': g.get('seed_id'), 'points': pts,
                   'slope': None, 'intercept': None, 'pa': None}
            final_segments.append(seg)
        else:
            segs = adjust_two_lines(pts, min_segment_points, improvement_ratio_threshold)
            # Aquí se podrían aplicar criterios adicionales; por ejemplo, si se obtuvo
            # un único segmento pero el error es alto, se podría desagregar y re-procesar.
            final_segments.extend(segs)
    return final_segments

# ========================
# Parte 2: Proceso iterativo utilizando los grupos semilla (sin ajuste inicial)
# ========================

def process_spiral_arms_with_seeds(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,     # Vecindad horizontal para seeds
    r_diff=0.468,        # Vecindad radial para seeds
    gap_threshold_theta=1.59,
    gap_threshold_r=1.023,
    snapshot_interval=10,
    theta_assignment_tol=2.0,
    r_assignment_tol=0.5,
    min_points_final=80  # Mínimo de puntos para aplicar el ajuste final
):
    """
    Ejecuta el proceso iterativo de asignación de puntos a grupos usando los grupos semilla como base.
    Primero se agrupan los puntos (sin ajuste), luego se asignan los puntos restantes según su rango.
    Finalmente, se finalizan los segmentos aplicando el ajuste de hasta dos rectas para cada grupo.
    Retorna los estados intermedios, los segmentos finales y el DataFrame filtrado.
    """
    # 1. Cargar y filtrar datos
    df = load_and_filter_data(id_halo, theta_min, theta_max)
    
    # 2. Filtrar según percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 3. Obtener grupos semilla mediante segmentación rectangular
    seeds_all, _ = process_clusters_rectangular_gaps(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    # Filtrar solo seeds con más de 60 puntos
    seed_clusters = [seed for seed in seeds_all if len(seed) > 60]
    print(f"Grupos semilla con más de 60 puntos: {len(seed_clusters)}")
    
    # Marcar en df_filtered los puntos que pertenecen a un seed
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 4. Inicialización de grupos a partir de seeds SIN aplicar ajuste (solo agrupación)
    groups = []
    for idx, seed in enumerate(seed_clusters):
        groups.append({
            'seed_id': idx,
            'points': seed.copy(),
            'slope': None,
            'intercept': None,
            'pa': None
        })
    
    # 5. Preparar puntos restantes (los que no pertenecen a ningún seed)
    df_local = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    df_local = df_local.sort_values(by='theta').reset_index(drop=True)
    
    discarded_points = []  # Puntos que no se pudieron asignar inicialmente
    states = []            # Para guardar estados intermedios
    iteration = 0
    
    # 6. Asignación iterativa de puntos a grupos usando criterio basado en rango
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        assigned = False
        for g in groups:
            if try_assign_point_to_group(point, g, theta_tol=theta_assignment_tol, r_tol=r_assignment_tol):
                assigned = True
                break
        if not assigned:
            discarded_points.append(point.to_dict())
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'discarded': copy.deepcopy(discarded_points),
                'remaining': df_local.copy()
            })
    
    # 7. Reasignar puntos descartados
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            for g in groups:
                if try_assign_point_to_group(p, g, theta_tol=theta_assignment_tol, r_tol=r_assignment_tol):
                    break
    
    # 8. Finalización de segmentos: aplicar el ajuste de hasta dos rectas a cada grupo
    final_segments = finalize_segments(groups, min_points=min_points_final,
                                       gap_threshold_theta=gap_threshold_theta,
                                       gap_threshold_r=gap_threshold_r,
                                       min_segment_points=10,
                                       improvement_ratio_threshold=0.7)
    
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(final_segments),
        'discarded': copy.deepcopy(discarded_points),
        'remaining': df_local.copy()
    })
    
    return states, final_segments, df_filtered

def plot_intermediate_state(state, df_all):
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Fondo de todos los datos (asegurando conversión a numérico)
    df_all['theta'] = pd.to_numeric(df_all['theta'], errors='coerce')
    theta_all = np.radians(df_all['theta'].values)
    r_all = df_all['r'].values
    ax.scatter(theta_all, r_all, s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    
    for idx, g in enumerate(state['groups']):
        pts = g['points'].copy()
        pts['theta'] = pd.to_numeric(pts['theta'], errors='coerce')
        theta_vals = np.radians(pts['theta'].values)
        r_vals = pts['r'].values
        color = colors[idx % len(colors)]
        ax.scatter(theta_vals, r_vals, s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)
    
    if len(state['discarded']) > 0:
        df_desc = pd.DataFrame(state['discarded'])
        df_desc['theta'] = pd.to_numeric(df_desc['theta'], errors='coerce')
        theta_desc = np.radians(df_desc['theta'].values)
        r_desc = df_desc['r'].values
        ax.scatter(theta_desc, r_desc, s=10, color='black', marker='x', label='Descartados')
    
    ax.set_title(f"Estado en iteración {state['iteration']}")
    ax.legend(loc='upper right')
    plt.show()

# ========================
# Ejecución interactiva e impresión de resultados
# ========================

states, final_segments, df_filtered = process_spiral_arms_with_seeds(
    id_halo="17",
    quartile_threshold=0.155,
    theta_min=50,
    theta_max=250,
    theta_diff=2.0,
    r_diff=0.468,
    gap_threshold_theta=1.89,
    gap_threshold_r=0.523,
    snapshot_interval=10,
    theta_assignment_tol=2.0,
    r_assignment_tol=0.5,
    min_points_final=80
)

print(f"Total de estados guardados: {len(states)}")
print(f"Total de segmentos finales: {len(final_segments)}")

def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))


necesito que actúes como un experto en astrofísica computacional con experiencia en ingeniería de software y en algoritmos avanzados bueno qué va a pasar te voy a compartir dos códigos el primer código eh te voy a llamar lo voy a llamar código uno es el código que está tomando segmentos de recta y los requerimientos físicos para evaluar si un punto de de un conjunto dado de un submarino pertenece o no a a un grupo esto en términos de la pendiente en términos de teta y r de una ventana esa ventana digamos que permite que no haya una variación mayor de el 20% de esa pendiente digamos que con este con el código 1 hemos tenido muchos problemas porque no se ajusta de manera adecuacada uno de los puntos entonces tenemos una versión 2 te comparto la versión 2 el código 2 entonces el código 2 lo que te está tratando de hacer es de coger una un enfoque alternativo no utilizar de scan sino utilizar una técnica distinta para entonces eh ir evaluando qué puntos hacen parte de pues para coger esa semilla pero es una semilla que tiene muchos puntos entonces ese código 2 está iterando de manera reiterativa y no está teniendo en cuenta esa semilla de manera adecuada porque se está no se está teniendo pues no sé está preguntando si ese punto siguiente en realidad realmente hace parte de la semilla o es un punto distinto nuevo entonces la idea esexcluir los puntos que no son parte de la semilla antes de empezar a iterar para qué para que entonces evalúe si analice muy bien esos dos códigos y primero eh mira cómo se podrían integrar en uno solo de tal manera que yo tenga las bondades iniciales de la nueva técnica de las semillas pero que adicional también tenga los los las formas de validar que los puntos estén segmentados dentro de cada una de esas semillas dentro de cada uno de esos grupos de la manera en que el código uno lo hace me hago entender?
código uno:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression
import copy
from ipywidgets import interact, IntSlider

# Función auxiliar para calcular distancia respecto a una línea (ajuste lineal)
def calculate_distance(slope, intercept, theta, r):
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

# Función auxiliar para calcular PA
def calculate_pa(slope, intercept):
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan

# Función para agregar un punto a un grupo temporal (si coincide en θ y en r)
def add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window):
    assigned = False
    for tg in temp_groups:
        tg_theta_min = tg['points']['theta'].min()
        tg_theta_max = tg['points']['theta'].max()
        tg_r_min = tg['points']['r'].min()
        tg_r_max = tg['points']['r'].max()
        if ((point['theta'] >= tg_theta_min - theta_fit_window/2) and 
            (point['theta'] <= tg_theta_max + theta_fit_window/2)) and \
           ((point['r'] >= tg_r_min - r_fit_window/2) and 
            (point['r'] <= tg_r_max + r_fit_window/2)):
            tg['points'] = pd.concat([tg['points'], point.to_frame().T], ignore_index=True)
            assigned = True
            break
    if not assigned:
        new_tg = {'points': point.to_frame().T.copy()}
        temp_groups.append(new_tg)

# Función para revisar y fusionar (o promover) los grupos temporales
def review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range):
    new_temp_groups = []
    for tg in temp_groups:
        if len(tg['points']) < 3:
            new_temp_groups.append(tg)
            continue
        X_tg = tg['points'][['theta']]
        y_tg = tg['points']['r']
        model.fit(X_tg, y_tg)
        tg_slope = model.coef_[0]
        tg_intercept = model.intercept_
        tg_mean = tg['points']['theta'].mean()
        merged = False
        for mg in groups:
            mg_min = mg['points']['theta'].min()
            mg_max = mg['points']['theta'].max()
            if mg_min <= tg_mean <= mg_max:
                dist_values = tg['points'].apply(lambda row: calculate_distance(mg['slope'], mg['intercept'], row['theta'], row['r']), axis=1)
                if dist_values.mean() < radius_tolerance:
                    mg['points'] = pd.concat([mg['points'], tg['points']], ignore_index=True)
                    model.fit(mg['points'][['theta']], mg['points']['r'])
                    new_slope = model.coef_[0]
                    if abs(new_slope - mg['slope'])/(abs(mg['slope'])+1e-12) < slope_variation_threshold:
                        mg['slope'] = new_slope
                        mg['intercept'] = model.intercept_
                        mg['pa'] = calculate_pa(mg['slope'], mg['intercept'])
                    merged = True
                    break
        if not merged:
            new_group = {
                'cluster_id': None,
                'points': tg['points'],
                'slope': tg_slope,
                'intercept': tg_intercept,
                'pa': calculate_pa(tg_slope, tg_intercept)
            }
            groups.append(new_group)
    temp_groups.clear()
    for tg in new_temp_groups:
        temp_groups.append(tg)

def process_spiral_arms_interactive(
    id_halo,
    quartile_threshold=0.55,
    radius_tolerance=0.45,
    point_distance_tolerance=0.59,
    flexible_pa_range=(4.0, 30.0),
    eps_polar=0.7,
    min_samples_polar=18,
    slope_variation_threshold=0.20,
    max_theta_gap=15.0,
    do_subdivide=True,
    snapshot_interval=10,     # Cada cuántas iteraciones se guarda un estado
    theta_fit_window=30.0,    # Ventana en grados para ajustar la recta localmente
    r_fit_window=10.0,        # Ventana fija para r
    fusion_theta_threshold=7.0,  # Diferencia máxima fija en θ para fusionar
    fusion_r_threshold=7.0       # Diferencia máxima fija en r para fusionar
):
    """
    Procesa los datos y retorna una lista de estados intermedios para visualización interactiva.
    Se filtran los puntos con θ entre 50 y 250 grados.
    
    La fusión de grupos se realiza utilizando condiciones fijas:
      - Se fusionan si la diferencia entre el límite inferior de un grupo y el límite superior de otro 
        (y viceversa) es menor que fusion_theta_threshold, y
      - la diferencia en los límites de r es menor que fusion_r_threshold.
    """
    # 1. Cargar datos
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    
    # 2. Convertir a coordenadas polares
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= 50) & (df['theta'] <= 250)].copy()
    
    # 3. Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 4. DBSCAN en el plano (r, θ)
    X_polar = df_filtered[['r', 'theta']].values
    dbscan = DBSCAN(eps=eps_polar, min_samples=min_samples_polar)
    df_filtered['cluster'] = dbscan.fit_predict(X_polar)
    
    # 5. Fusión de clusters cercanos
    clusters_tolerance = 0.70
    def merge_close_clusters(clusters, clusters_tolerance):
        merged_clusters = {}
        used_clusters = set()
        for id1, points1 in clusters.items():
            if id1 in used_clusters:
                continue
            merged_points = points1.copy()
            for id2, points2 in clusters.items():
                if id1 >= id2 or id2 in used_clusters:
                    continue
                min_distance = np.min(np.sqrt(
                    (points1['r'].values[:, None] - points2['r'].values)**2 +
                    (points1['theta'].values[:, None] - points2['theta'].values)**2
                ))
                if min_distance < clusters_tolerance:
                    merged_points = pd.concat([merged_points, points2], ignore_index=True)
                    used_clusters.add(id2)
            merged_clusters[id1] = merged_points
            used_clusters.add(id1)
        return merged_clusters
    
    clusters_polar = {}
    for cluster_id in np.unique(df_filtered['cluster']):
        if cluster_id == -1:
            continue
        cluster_points = df_filtered[df_filtered['cluster'] == cluster_id].copy()
        clusters_polar[cluster_id] = cluster_points[['r', 'theta']]
    
    clusters_polar = merge_close_clusters(clusters_polar, clusters_tolerance)
    
    # 6. Inicializar grupos principales a partir de los clusters obtenidos
    model = LinearRegression()
    groups = []
    for cid, pts in clusters_polar.items():
        if len(pts) >= 3:
            X_ = pts[['theta']].values
            y_ = pts['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'cluster_id': cid,
            'points': pts.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # Preparar estados y grupos temporales
    df_local = df_filtered.copy()
    df_local.sort_values(by='theta', inplace=True)
    states = []
    temp_groups = []
    iteration = 0
    temp_review_interval = snapshot_interval * 15
    
    # 7. Asignación iterativa de puntos
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        best_group = None
        best_distance = None
        
        # Buscar grupo candidato que contenga el punto
        for g in groups:
            g_theta_min = g['points']['theta'].min()
            g_theta_max = g['points']['theta'].max()
            g_r_min = g['points']['r'].min()
            g_r_max = g['points']['r'].max()
            if not (g_theta_min <= theta_pt <= g_theta_max and g_r_min <= r_pt <= g_r_max):
                continue
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = g['points'][(g['points']['theta'] >= window_min) & (g['points']['theta'] <= window_max)]
            if len(subset) == 0:
                continue
            r_min = subset['r'].min()
            r_max = subset['r'].max()
            if not (r_min <= r_pt <= r_max):
                continue
            subset = pd.concat([subset, point.to_frame().T], ignore_index=True)
            if len(subset) < 2:
                continue
            model.fit(subset[['theta']], subset['r'])
            new_slope = model.coef_[0]
            new_intercept = model.intercept_
            old_slope = g['slope']
            if old_slope is None:
                continue
            if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) < 0.10:
                continue
            dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
            if dist > radius_tolerance:
                continue
            if best_distance is None or dist < best_distance:
                best_distance = dist
                best_group = g
        
        if best_group is not None:
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = best_group['points'][(best_group['points']['theta'] >= window_min) & (best_group['points']['theta'] <= window_max)]
            if len(subset) < 2:
                subset = best_group['points']
            model.fit(subset[['theta']], subset['r'])
            best_group['slope'] = model.coef_[0]
            best_group['intercept'] = model.intercept_
            best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
            
            # Fusión fija con grupos anteriores: se fusionan solo si las diferencias en ambos límites son menores que los umbrales fijos.
            merged_flag = True
            while merged_flag:
                merged_flag = False
                for other_group in groups:
                    if other_group is best_group:
                        continue
                    best_theta_min = best_group['points']['theta'].min()
                    best_theta_max = best_group['points']['theta'].max()
                    best_r_min = best_group['points']['r'].min()
                    best_r_max = best_group['points']['r'].max()
                    
                    other_theta_min = other_group['points']['theta'].min()
                    other_theta_max = other_group['points']['theta'].max()
                    other_r_min = other_group['points']['r'].min()
                    other_r_max = other_group['points']['r'].max()
                    
                    if (abs(best_theta_min - other_theta_max) < fusion_theta_threshold and
                        abs(best_theta_max - other_theta_min) < fusion_theta_threshold and
                        abs(best_r_min - other_r_max) < fusion_r_threshold and
                        abs(best_r_max - other_r_min) < fusion_r_threshold):
                        merged_points = pd.concat([best_group['points'], other_group['points']], ignore_index=True)
                        model.fit(merged_points[['theta']], merged_points['r'])
                        best_group['points'] = merged_points
                        best_group['slope'] = model.coef_[0]
                        best_group['intercept'] = model.intercept_
                        best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
                        # Eliminar other_group por identidad
                        for idx, grp in enumerate(groups):
                            if grp is other_group:
                                groups.pop(idx)
                                break
                        merged_flag = True
                        break
                        
        else:
            # Si no hay grupo candidato, intentar fusionar con el último grupo principal sin filtro de pendiente
            merged_with_last = False
            if len(groups) > 0:
                last_g = groups[-1]
                last_theta_min = last_g['points']['theta'].min()
                last_theta_max = last_g['points']['theta'].max()
                last_r_min = last_g['points']['r'].min()
                last_r_max = last_g['points']['r'].max()
                if (last_theta_min <= theta_pt <= last_theta_max and last_r_min <= r_pt <= last_r_max):
                    window_min = theta_pt - theta_fit_window/2
                    window_max = theta_pt + theta_fit_window/2
                    subset_last = last_g['points'][(last_g['points']['theta'] >= window_min) & (last_g['points']['theta'] <= window_max)]
                    if len(subset_last) > 0:
                        r_min_l = subset_last['r'].min()
                        r_max_l = subset_last['r'].max()
                        if r_min_l <= r_pt <= r_max_l:
                            subset_last = pd.concat([subset_last, point.to_frame().T], ignore_index=True)
                            model.fit(subset_last[['theta']], subset_last['r'])
                            new_slope_l = model.coef_[0]
                            new_intercept_l = model.intercept_
                            dist_l = calculate_distance(new_slope_l, new_intercept_l, theta_pt, r_pt)
                            if dist_l < radius_tolerance:
                                last_g['points'] = pd.concat([last_g['points'], point.to_frame().T], ignore_index=True)
                                model.fit(last_g['points'][['theta']], last_g['points']['r'])
                                last_g['slope'] = model.coef_[0]
                                last_g['intercept'] = model.intercept_
                                last_g['pa'] = calculate_pa(last_g['slope'], last_g['intercept'])
                                merged_with_last = True
            if not merged_with_last:
                add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window)
        
        iteration += 1
        
        if iteration % temp_review_interval == 0:
            review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range)
        
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_groups': copy.deepcopy(temp_groups),
                'remaining': df_local.copy()
            })
    
    if len(temp_groups) > 0:
        review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range)
    
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    def subdivide_if_gap(group, max_gap=max_theta_gap):
        pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)
        seg1 = {
            'cluster_id': group['cluster_id'],
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'cluster_id': group['cluster_id'],
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]
    
    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_gap=max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups
    
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'temp_groups': copy.deepcopy(temp_groups),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered

def plot_intermediate_state_cartesian(state, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    if state['temp_groups']:
        for idx, tg in enumerate(state['temp_groups']):
            pts = tg['points']
            plt.scatter(pts['theta'], pts['r'], s=10, color='black', marker='x', label=f"Temp {idx+1}")
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()
fin de código uno.

código dos:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

# ========================
# Funciones de Preprocesamiento
# ========================

def load_and_filter_data(id_halo, theta_min, theta_max):
    """
    Carga los datos desde CSV, calcula coordenadas polares y filtra puntos según el rango de θ.
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    return df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()

def calculate_pa(slope):
    """
    Calcula el ángulo (en grados) correspondiente a la pendiente usando la función arctan.
    Si la pendiente es None, retorna None.
    """
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

# ========================
# Parte 1: Segmentación inicial con vecinos rectangulares y subdivisión por gaps
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo donde cada punto se conecta a otros si las diferencias en θ y r
    son menores o iguales a los umbrales especificados.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Excluir el mismo índice
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    """
    Realiza una búsqueda en anchura (BFS) sobre el grafo para extraer componentes conexas.
    Cada componente se retorna como un DataFrame de puntos.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Ordena el DataFrame según 'theta' o 'r' y subdivide en subclusters si se detecta un gap mayor
    que el umbral especificado.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i + 1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    """
    Carga y filtra los datos, construye el grafo, extrae componentes (clusters) y los subdivide
    en función de gaps en θ y r.
    """
    df = load_and_filter_data(id_halo, theta_min, theta_max)
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    return final_clusters, df_filtered

def plot_clusters_rectangular_gaps(clusters, title="Clusters con BFS + Gaps"):
    """
    Genera una visualización de los clusters obtenidos.
    """
    plt.figure(figsize=(10,6))
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','darkblue']
    for idx, dfc in enumerate(clusters):
        color = colors[idx % len(colors)]
        plt.scatter(dfc['theta'], dfc['r'], s=10, color=color, label=f"Grupo {idx+1}")
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(title)
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()

# ========================
# Asignación de puntos a grupos (sin ajuste inicial) basada en rango
# ========================

def try_assign_point_to_group(point, group, theta_tol=2.0, r_tol=0.5):
    """
    Asigna 'point' al 'group' si su valor de θ y r está dentro de un rango definido
    por el mínimo y máximo del grupo, con una tolerancia.
    """
    if isinstance(point, dict):
        point = pd.DataFrame([point]).iloc[0]
    theta_pt, r_pt = point['theta'], point['r']
    group_theta_min = group['points']['theta'].min()
    group_theta_max = group['points']['theta'].max()
    group_r_min = group['points']['r'].min()
    group_r_max = group['points']['r'].max()
    if (theta_pt >= group_theta_min - theta_tol) and (theta_pt <= group_theta_max + theta_tol) and \
       (r_pt >= group_r_min - r_tol) and (r_pt <= group_r_max + r_tol):
        new_row = point.to_frame().T
        group['points'] = pd.concat([group['points'], new_row], ignore_index=True)
        return True
    return False

# ========================
# Función para ajustar hasta dos rectas a cada grupo
# ========================

def adjust_two_lines(pts, min_segment_points=10, improvement_ratio_threshold=0.7):
    """
    Dado un DataFrame de puntos (ordenado por 'theta'), intenta ajustar dos rectas.
    
    - Si el número de puntos es insuficiente para dividir (menor que 2*min_segment_points),
      se ajusta solo una recta.
    - Se ajusta una recta única a todos los puntos y se calcula su error (suma de residuos al cuadrado).
    - Se prueban posibles puntos de quiebre (breakpoints) para dividir en dos segmentos y se
      calcula el error total (suma de errores de ambos segmentos).
    - Si la mejora es significativa (error_total < improvement_ratio_threshold * error_unico),
      se retornan dos segmentos; en caso contrario, se retorna un único segmento.
    """
    pts = pts.copy().reset_index(drop=True)
    n = len(pts)
    if n < 2 * min_segment_points:
        # No es posible dividir, se ajusta una recta
        model = LinearRegression()
        X = pts[['theta']].values
        y = pts['r'].values
        model.fit(X, y)
        seg = {'points': pts, 'slope': model.coef_[0],
               'intercept': model.intercept_, 'pa': calculate_pa(model.coef_[0])}
        return [seg]
    
    # Ajuste con una recta única
    model_all = LinearRegression()
    X_all = pts[['theta']].values
    y_all = pts['r'].values
    model_all.fit(X_all, y_all)
    error_all = np.sum((model_all.predict(X_all) - y_all)**2)
    
    best_error = np.inf
    best_split = None
    # Se prueban puntos de quiebre, asegurando que cada segmento tenga al menos min_segment_points
    for i in range(min_segment_points, n - min_segment_points + 1):
        pts_left = pts.iloc[:i]
        pts_right = pts.iloc[i:]
        model_left = LinearRegression()
        model_right = LinearRegression()
        X_left = pts_left[['theta']].values
        y_left = pts_left['r'].values
        X_right = pts_right[['theta']].values
        y_right = pts_right['r'].values
        model_left.fit(X_left, y_left)
        model_right.fit(X_right, y_right)
        error_left = np.sum((model_left.predict(X_left) - y_left)**2)
        error_right = np.sum((model_right.predict(X_right) - y_right)**2)
        total_error = error_left + error_right
        if total_error < best_error:
            best_error = total_error
            best_split = i

    if best_split is None:
        seg = {'points': pts, 'slope': model_all.coef_[0],
               'intercept': model_all.intercept_, 'pa': calculate_pa(model_all.coef_[0])}
        return [seg]

    if best_error < improvement_ratio_threshold * error_all:
        # Se opta por dos segmentos
        pts_left = pts.iloc[:best_split]
        pts_right = pts.iloc[best_split:]
        model_left = LinearRegression()
        model_right = LinearRegression()
        X_left = pts_left[['theta']].values
        y_left = pts_left['r'].values
        X_right = pts_right[['theta']].values
        y_right = pts_right['r'].values
        model_left.fit(X_left, y_left)
        model_right.fit(X_right, y_right)
        seg_left = {'points': pts_left, 'slope': model_left.coef_[0],
                    'intercept': model_left.intercept_, 'pa': calculate_pa(model_left.coef_[0])}
        seg_right = {'points': pts_right, 'slope': model_right.coef_[0],
                     'intercept': model_right.intercept_, 'pa': calculate_pa(model_right.coef_[0])}
        return [seg_left, seg_right]
    else:
        # No se obtiene una mejora significativa; se opta por un único segmento.
        seg = {'points': pts, 'slope': model_all.coef_[0],
               'intercept': model_all.intercept_, 'pa': calculate_pa(model_all.coef_[0])}
        return [seg]

# ========================
# Finalización recursiva de segmentos usando ajuste de hasta dos rectas
# ========================

def finalize_segments(groups, min_points=80, gap_threshold_theta=2.05, gap_threshold_r=2.05,
                      min_segment_points=10, improvement_ratio_threshold=0.7):
    """
    Para cada grupo (base, preferentemente semilla), se intenta ajustar hasta dos rectas.
    Si el grupo es muy pequeño o uniforme se ajusta una recta; en caso contrario, se
    busca dividir el grupo en dos segmentos. Si no se logra una división significativa,
    se retorna un único segmento.
    Retorna la lista de segmentos finales.
    """
    final_segments = []
    for g in groups:
        pts = g['points']
        if len(pts) < min_points:
            seg = {'seed_id': g.get('seed_id'), 'points': pts,
                   'slope': None, 'intercept': None, 'pa': None}
            final_segments.append(seg)
        else:
            segs = adjust_two_lines(pts, min_segment_points, improvement_ratio_threshold)
            # Aquí se podrían aplicar criterios adicionales; por ejemplo, si se obtuvo
            # un único segmento pero el error es alto, se podría desagregar y re-procesar.
            final_segments.extend(segs)
    return final_segments

# ========================
# Parte 2: Proceso iterativo utilizando los grupos semilla (sin ajuste inicial)
# ========================

def process_spiral_arms_with_seeds(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50,
    theta_max=250,
    theta_diff=3.0,     # Vecindad horizontal para seeds
    r_diff=0.468,        # Vecindad radial para seeds
    gap_threshold_theta=1.59,
    gap_threshold_r=1.023,
    snapshot_interval=10,
    theta_assignment_tol=2.0,
    r_assignment_tol=0.5,
    min_points_final=80  # Mínimo de puntos para aplicar el ajuste final
):
    """
    Ejecuta el proceso iterativo de asignación de puntos a grupos usando los grupos semilla como base.
    Primero se agrupan los puntos (sin ajuste), luego se asignan los puntos restantes según su rango.
    Finalmente, se finalizan los segmentos aplicando el ajuste de hasta dos rectas para cada grupo.
    Retorna los estados intermedios, los segmentos finales y el DataFrame filtrado.
    """
    # 1. Cargar y filtrar datos
    df = load_and_filter_data(id_halo, theta_min, theta_max)
    
    # 2. Filtrar según percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 3. Obtener grupos semilla mediante segmentación rectangular
    seeds_all, _ = process_clusters_rectangular_gaps(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    # Filtrar solo seeds con más de 60 puntos
    seed_clusters = [seed for seed in seeds_all if len(seed) > 60]
    print(f"Grupos semilla con más de 60 puntos: {len(seed_clusters)}")
    
    # Marcar en df_filtered los puntos que pertenecen a un seed
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 4. Inicialización de grupos a partir de seeds SIN aplicar ajuste (solo agrupación)
    groups = []
    for idx, seed in enumerate(seed_clusters):
        groups.append({
            'seed_id': idx,
            'points': seed.copy(),
            'slope': None,
            'intercept': None,
            'pa': None
        })
    
    # 5. Preparar puntos restantes (los que no pertenecen a ningún seed)
    df_local = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    df_local = df_local.sort_values(by='theta').reset_index(drop=True)
    
    discarded_points = []  # Puntos que no se pudieron asignar inicialmente
    states = []            # Para guardar estados intermedios
    iteration = 0
    
    # 6. Asignación iterativa de puntos a grupos usando criterio basado en rango
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        assigned = False
        for g in groups:
            if try_assign_point_to_group(point, g, theta_tol=theta_assignment_tol, r_tol=r_assignment_tol):
                assigned = True
                break
        if not assigned:
            discarded_points.append(point.to_dict())
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'discarded': copy.deepcopy(discarded_points),
                'remaining': df_local.copy()
            })
    
    # 7. Reasignar puntos descartados
    discarded_df = pd.DataFrame(discarded_points)
    if not discarded_df.empty:
        for _, p in discarded_df.iterrows():
            for g in groups:
                if try_assign_point_to_group(p, g, theta_tol=theta_assignment_tol, r_tol=r_assignment_tol):
                    break
    
    # 8. Finalización de segmentos: aplicar el ajuste de hasta dos rectas a cada grupo
    final_segments = finalize_segments(groups, min_points=min_points_final,
                                       gap_threshold_theta=gap_threshold_theta,
                                       gap_threshold_r=gap_threshold_r,
                                       min_segment_points=10,
                                       improvement_ratio_threshold=0.7)
    
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(final_segments),
        'discarded': copy.deepcopy(discarded_points),
        'remaining': df_local.copy()
    })
    
    return states, final_segments, df_filtered

def plot_intermediate_state(state, df_all):
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Fondo de todos los datos (asegurando conversión a numérico)
    df_all['theta'] = pd.to_numeric(df_all['theta'], errors='coerce')
    theta_all = np.radians(df_all['theta'].values)
    r_all = df_all['r'].values
    ax.scatter(theta_all, r_all, s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    
    for idx, g in enumerate(state['groups']):
        pts = g['points'].copy()
        pts['theta'] = pd.to_numeric(pts['theta'], errors='coerce')
        theta_vals = np.radians(pts['theta'].values)
        r_vals = pts['r'].values
        color = colors[idx % len(colors)]
        ax.scatter(theta_vals, r_vals, s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            ax.plot(np.radians(theta_line), r_line, '--', color=color, linewidth=1.5)
    
    if len(state['discarded']) > 0:
        df_desc = pd.DataFrame(state['discarded'])
        df_desc['theta'] = pd.to_numeric(df_desc['theta'], errors='coerce')
        theta_desc = np.radians(df_desc['theta'].values)
        r_desc = df_desc['r'].values
        ax.scatter(theta_desc, r_desc, s=10, color='black', marker='x', label='Descartados')
    
    ax.set_title(f"Estado en iteración {state['iteration']}")
    ax.legend(loc='upper right')
    plt.show()

# ========================
# Ejecución interactiva e impresión de resultados
# ========================

states, final_segments, df_filtered = process_spiral_arms_with_seeds(
    id_halo="17",
    quartile_threshold=0.155,
    theta_min=50,
    theta_max=250,
    theta_diff=2.0,
    r_diff=0.468,
    gap_threshold_theta=1.89,
    gap_threshold_r=0.523,
    snapshot_interval=10,
    theta_assignment_tol=2.0,
    r_assignment_tol=0.5,
    min_points_final=80
)

print(f"Total de estados guardados: {len(states)}")
print(f"Total de segmentos finales: {len(final_segments)}")

def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0)).
fin de codigo dos.----------
**algunos ajustes para entender que hacer y la razón de cada uno**:

1. **Semillas construidas con BFS pero sin ajustar pendiente**  
   - **Motivo**: Iniciar con grupos (semillas) “en bruto” sin slope/intercept, lo cual evita un ajuste prematuro y permite que, en la iteración posterior, se calcule la pendiente únicamente cuando el grupo tenga suficientes puntos (≥ 2).

2. **Exclusión de los puntos ya usados en las semillas en el bucle iterativo**  
   - **Motivo**: Evitar redundancia y sobreasignación. Ya que las semillas están definidas, no volvemos a procesar esos puntos en la misma rutina; únicamente trabajamos con los que no fueron capturados por las semillas.

3. **Asignación de cada nuevo punto a un grupo mediante un criterio rápido de “radio de tolerancia” (radius_tolerance)**  
   - **Motivo**: Reducir la complejidad de la comparación y agilizar la asignación, chequeando si el punto está cerca (en el espacio θ–r) del grupo (sea euclídeamente o respecto a la recta si el grupo ya tiene slope).

4. **Cálculo del slope cuando el grupo no lo tenía**  
   - **Motivo**: Una vez que el grupo tiene al menos 2 puntos, se hace su primer ajuste lineal. Hasta ese momento, no se asigna pendiente.

5. **Fusión “temprana” de grupos contiguos o cercanos**  
   - **Motivo**: En cuanto un punto actualiza un grupo, se chequea si hay grupos vecinos superpuestos o muy próximos (por bounding box **o** por una distancia mínima en θ–r) para **fusionarlos** inmediatamente. Así se evitan grupos duplicados o separados artificialmente.

6. **Extrapolación (20% extra) y variación de la pendiente hasta 30%** (solo en la versión donde se pedía “extrapolar” un 20% y permitir un 30% de variación)  
   - **Motivo**: Permitir cierta flexibilidad en la asignación de puntos que están en el borde (extrapolación) y mayor tolerancia en la pendiente en esas zonas “extendidas”.

7. **Criterio de cercanía más robusto** (bounding box + distancia mínima)  
   - **Motivo**: En ciertos casos, grupos que lucen “muy próximos” no se fusionan porque sus bounding boxes no se superponen lo suficiente. Añadir la verificación de distancia mínima entre puntos de ambos grupos garantiza que sí se fusionen cuando existen subzonas verdaderamente cercanas.

8. **Eliminar grupos por identidad y no por igualdad** (reemplazar `groups.remove(...)` con un `for ... pop(...)`)  
   - **Motivo**: Evitar el `ValueError: Can only compare identically-labeled DataFrame objects`. Python intenta comparar DataFrames cuando se usa `.remove()`, lo que genera un conflicto. Con `if grp is other_group: pop(...)` se elimina el grupo exacto sin comparar los contenidos.

9. **Forzar `astype(float)` en las columnas `theta` y `r`** para evitar errores de `ufunc`  
   - **Motivo**: Asegurarse de que, al calcular distancias con NumPy, los valores sean numéricos válidos. Si alguna columna contiene objetos no convertibles a float, se suscita el error “loop of ufunc does not support argument 0 of type...”.

**En conjunto**, todos estos ajustes:
- **Optimizan** la construcción de grupos iniciales (semillas).  
- **Agilizan** la asignación iterativa de puntos.  
- **Permiten** un ajuste lineal más coherente (solo cuando un grupo lo necesita).  
- **Garantizan** que grupos realmente cercanos se fusionen adecuadamente.  
- **Evitan** errores de comparación de DataFrames y de tipos no numéricos.

## combinación c+ódigos y engfoques.

In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from collections import deque
import copy
from ipywidgets import interact, IntSlider

# ========================
# Funciones Auxiliares
# ========================

def calculate_distance(slope, intercept, theta, r):
    """Calcula la distancia vertical (en r) de un punto a la recta definida por slope e intercept."""
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    """
    Calcula el ángulo (en grados) de la recta a partir de su pendiente.
    Si se dispone de intercepto, se usa para un cálculo ajustado.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.degrees(np.arctan(slope))

# ========================
# Carga y Preprocesamiento de Datos
# ========================

def load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold):
    """
    Carga el CSV, calcula coordenadas polares y filtra puntos:
      - Por rango de θ (en grados).
      - Por percentil en 'rho_resta_final_exp'.
    """
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    # Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    return df_filtered

# ========================
# Detección de Semillas utilizando Grafos y BFS
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo conectando puntos si la diferencia en θ y en r es menor o igual a los umbrales.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Se excluye el mismo punto
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph

def bfs_components(graph, df_points):
    """
    Extrae las componentes conexas (clusters) del grafo usando búsqueda en anchura (BFS).
    Cada componente se retorna como un DataFrame.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold, mode='theta'):
    """
    Ordena el DataFrame según 'theta' o 'r' y subdivide el cluster en subclusters 
    si existe un gap mayor que gap_threshold.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def detect_seeds(id_halo, quartile_threshold=0.55, theta_min=50.0, theta_max=250.0, 
                 theta_diff=5.0, r_diff=1.0, gap_threshold_theta=2.5, gap_threshold_r=2.5, min_seed_points=60):
    """
    Detecta semillas robustas usando la construcción de grafo, BFS y subdivisión por gaps.
    Retorna una lista de DataFrames (cada uno es una semilla) y el DataFrame filtrado.
    """
    df_filtered = load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold)
    
    # Construir grafo y extraer componentes conexas
    graph = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    # Subdividir cada cluster según gaps en θ y luego en r
    seeds_all = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = []
            for group in subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r'):
                sub_r.append(group)
            seeds_all.extend(sub_r)
    
    # Filtrar semillas que tengan al menos min_seed_points
    seed_clusters = [seed for seed in seeds_all if len(seed) >= min_seed_points]
    print(f"Total de semillas robustas (>= {min_seed_points} puntos): {len(seed_clusters)}")
    
    # Marcar en df_filtered los puntos que pertenecen a alguna semilla
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    return seed_clusters, df_filtered

# ========================
# Asignación Iterativa de Puntos a Grupos con Validación Local
# ========================

def iterative_assignment(seed_clusters, df_filtered, theta_fit_window=30.0, r_fit_window=10.0, 
                          radius_tolerance=0.45, slope_variation_threshold=0.20, snapshot_interval=10):
    """
    Usa las semillas como grupos iniciales (con ajuste lineal para obtener pendiente, intercepto y ángulo)
    y asigna iterativamente los puntos restantes (no pertenecientes a semillas) a estos grupos, validando
    que la incorporación de cada punto mantenga la coherencia geométrica.
    Retorna estados intermedios y los grupos finales.
    """
    model = LinearRegression()
    groups = []
    # Inicializar grupos a partir de semillas
    for idx, seed in enumerate(seed_clusters):
        if len(seed) >= 2:
            X_seed = seed[['theta']].values
            y_seed = seed['r'].values
            model.fit(X_seed, y_seed)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'group_id': idx,
            'points': seed.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # Puntos restantes que no están en semillas
    df_remaining = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    df_remaining = df_remaining.sort_values(by='theta').reset_index(drop=True)
    
    temp_groups = []  # Para almacenar puntos que inicialmente no se asignan
    states = []
    iteration = 0
    
    # Bucle iterativo de asignación
    while len(df_remaining) > 0:
        point = df_remaining.iloc[0]
        df_remaining = df_remaining.iloc[1:].reset_index(drop=True)
        theta_pt = point['theta']
        r_pt = point['r']
        best_group = None
        best_distance = None
        
        # Buscar grupo candidato en el que el punto caiga dentro de una ventana local
        for g in groups:
            g_theta_min = g['points']['theta'].min()
            g_theta_max = g['points']['theta'].max()
            g_r_min = g['points']['r'].min()
            g_r_max = g['points']['r'].max()
            if not (g_theta_min <= theta_pt <= g_theta_max and g_r_min <= r_pt <= g_r_max):
                continue
            # Definir ventana local para ajuste
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = g['points'][(g['points']['theta'] >= window_min) & (g['points']['theta'] <= window_max)]
            if subset.empty:
                continue
            # Agregar el punto y reajustar la recta
            subset = pd.concat([subset, point.to_frame().T], ignore_index=True)
            if len(subset) < 2:
                continue
            model.fit(subset[['theta']], subset['r'])
            new_slope = model.coef_[0]
            new_intercept = model.intercept_
            old_slope = g['slope']
            # Validar que la variación en la pendiente no exceda el umbral
            if old_slope is not None and abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) > slope_variation_threshold:
                dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
                if dist <= radius_tolerance:
                    if best_distance is None or dist < best_distance:
                        best_distance = dist
                        best_group = g
        
        if best_group is not None:
            # Asigna el punto al grupo y actualiza sus parámetros
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = best_group['points'][(best_group['points']['theta'] >= window_min) & (best_group['points']['theta'] <= window_max)]
            if len(subset) < 2:
                subset = best_group['points']
            model.fit(subset[['theta']], subset['r'])
            best_group['slope'] = model.coef_[0]
            best_group['intercept'] = model.intercept_
            best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
        else:
            # Si no se pudo asignar, se agrega a un grupo temporal
            temp_groups.append(point.to_frame().T)
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_remaining) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_points': pd.concat(temp_groups, ignore_index=True) if temp_groups else pd.DataFrame(),
                'remaining': df_remaining.copy()
            })
    
    # Reintentar asignar puntos temporales a los grupos existentes
    if temp_groups:
        temp_df = pd.concat(temp_groups, ignore_index=True)
        for idx, point in temp_df.iterrows():
            assigned = False
            for g in groups:
                if (g['points']['theta'].min() - theta_fit_window/2 <= point['theta'] <= g['points']['theta'].max() + theta_fit_window/2 and
                    g['points']['r'].min() - r_fit_window/2 <= point['r'] <= g['points']['r'].max() + r_fit_window/2):
                    g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                    model.fit(g['points'][['theta']], g['points']['r'])
                    g['slope'] = model.coef_[0]
                    g['intercept'] = model.intercept_
                    g['pa'] = calculate_pa(g['slope'], g['intercept'])
                    assigned = True
                    break
            if not assigned:
                # Si aun no se asigna, se puede dejar descartado o manejarlo aparte
                pass
    
    return states, groups, df_filtered

# ========================
# Fusión y Subdivisión Final de Grupos
# ========================

def subdivide_if_gap(group, max_theta_gap=15.0):
    """
    Divide el grupo en dos segmentos si se detecta un gap significativo en la variable θ.
    """
    pts = group['points'].sort_values(by='theta').reset_index(drop=True)
    if len(pts) < 4:
        return [group]
    theta_vals = pts['theta'].values
    dtheta = np.diff(theta_vals)
    gap_idx = np.where(dtheta > max_theta_gap)[0]
    if len(gap_idx) == 0:
        return [group]
    split_idx = gap_idx[0]
    pts1 = pts.iloc[:split_idx+1]
    pts2 = pts.iloc[split_idx+1:]
    model = LinearRegression()
    if len(pts1) >= 2:
        model.fit(pts1[['theta']], pts1['r'])
        s1 = model.coef_[0]
        i1 = model.intercept_
        pa1 = calculate_pa(s1, i1)
    else:
        s1, i1, pa1 = None, None, None
    if len(pts2) >= 2:
        model.fit(pts2[['theta']], pts2['r'])
        s2 = model.coef_[0]
        i2 = model.intercept_
        pa2 = calculate_pa(s2, i2)
    else:
        s2, i2, pa2 = None, None, None
    seg1 = {
        'group_id': group.get('group_id'),
        'points': pts1,
        'slope': s1,
        'intercept': i1,
        'pa': pa1
    }
    seg2 = {
        'group_id': group.get('group_id'),
        'points': pts2,
        'slope': s2,
        'intercept': i2,
        'pa': pa2
    }
    return [seg1, seg2]

def finalize_groups(groups, max_theta_gap=15.0, flexible_pa_range=(4.0, 30.0)):
    """
    Aplica la subdivisión por gaps y filtra los grupos que cumplan con un rango flexible para pa.
    """
    final_groups = []
    for g in groups:
        segs = subdivide_if_gap(g, max_theta_gap)
        final_groups.extend(segs)
    
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    return valid_groups

# ========================
# Funciones de Visualización
# ========================

def plot_intermediate_state_cartesian(state, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    if 'temp_points' in state and not state['temp_points'].empty:
        plt.scatter(state['temp_points']['theta'], state['temp_points']['r'], s=10, color='black', marker='x', label='Temp')
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def plot_final_groups(final_groups, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, grp in enumerate(final_groups):
        pts = grp['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Segmento {idx+1}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title("Segmentos Finales (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

# ========================
# Proceso Integrado
# ========================

def process_spiral_arms_integrated(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5,
    min_seed_points=60,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    radius_tolerance=0.45,
    slope_variation_threshold=0.20,
    snapshot_interval=10,
    max_theta_gap=15.0,
    flexible_pa_range=(4.0, 30.0)
):
    """
    Ejecuta el proceso integrado:
      1. Detecta semillas robustas.
      2. Asigna iterativamente los puntos restantes a grupos existentes.
      3. Realiza una fusión/subdivisión final para obtener los segmentos finales.
    """
    # 1. Detección de semillas
    seed_clusters, df_filtered = detect_seeds(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points
    )
    
    # 2. Asignación iterativa de puntos restantes
    states, groups, df_filtered = iterative_assignment(
        seed_clusters, df_filtered,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval
    )
    
    # 3. Fusión y subdivisión final de grupos
    final_groups = finalize_groups(groups, max_theta_gap=max_theta_gap, flexible_pa_range=flexible_pa_range)
    
    return states, final_groups, df_filtered

# ========================
# Ejecución Interactiva
# ========================

if __name__ == '__main__':
    # Parámetros de ejemplo (ajusta según tus necesidades)
    id_halo = "17"
    quartile_threshold = 0.55
    theta_min = 20.0
    theta_max = 270.0
    theta_diff = 4.0
    r_diff = 1.0
    gap_threshold_theta = 1.5
    gap_threshold_r = 1.5
    min_seed_points = 50
    theta_fit_window = 20.0
    r_fit_window = 2.0
    radius_tolerance = 0.345
    slope_variation_threshold = 0.18
    snapshot_interval = 10
    max_theta_gap = 8.0
    flexible_pa_range = (4.0, 30.0)
    
    # Ejecutar el proceso integrado
    states, final_groups, df_filtered = process_spiral_arms_integrated(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval,
        max_theta_gap=max_theta_gap,
        flexible_pa_range=flexible_pa_range
    )
    
    print(f"Total de estados intermedios: {len(states)}")
    print(f"Total de segmentos finales: {len(final_groups)}")
    
    # Función interactiva para visualizar estados intermedios
    def view_state(state_index):
        state = states[state_index]
        plot_intermediate_state_cartesian(state, df_filtered)
    
    interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))
    
    # Visualizar los segmentos finales
    plot_final_groups(final_groups, df_filtered)


In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from collections import deque
import copy
from ipywidgets import interact, IntSlider

# ========================
# Funciones Auxiliares
# ========================

def calculate_distance(slope, intercept, theta, r):
    """Calcula la distancia vertical (en r) de un punto a la recta definida por slope e intercept."""
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    """
    Calcula el ángulo (en grados) de la recta a partir de su pendiente.
    Si se dispone de intercepto, se usa para un cálculo ajustado.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.degrees(np.arctan(slope))

# ========================
# Carga y Preprocesamiento de Datos
# ========================

def load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold):
    """
    Carga el CSV, calcula coordenadas polares y filtra puntos:
      - Por rango de θ (en grados).
      - Por percentil en 'rho_resta_final_exp'.
    """
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    # Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    return df_filtered

# ========================
# Detección de Semillas utilizando Grafos y BFS
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo conectando puntos si la diferencia en θ y en r es menor o igual a los umbrales.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Se excluye el mismo punto
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph

def bfs_components(graph, df_points):
    """
    Extrae las componentes conexas (clusters) del grafo usando búsqueda en anchura (BFS).
    Cada componente se retorna como un DataFrame.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold, mode='theta'):
    """
    Ordena el DataFrame según 'theta' o 'r' y subdivide el cluster en subclusters 
    si existe un gap mayor que gap_threshold.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def detect_seeds(id_halo, quartile_threshold=0.55, theta_min=50.0, theta_max=250.0, 
                 theta_diff=5.0, r_diff=1.0, gap_threshold_theta=2.5, gap_threshold_r=2.5, min_seed_points=60):
    """
    Detecta semillas robustas usando la construcción de grafo, BFS y subdivisión por gaps.
    Retorna una lista de DataFrames (cada uno es una semilla) y el DataFrame filtrado.
    """
    df_filtered = load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold)
    
    # Construir grafo y extraer componentes conexas
    graph = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    # Subdividir cada cluster según gaps en θ y luego en r
    seeds_all = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = []
            for group in subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r'):
                sub_r.append(group)
            seeds_all.extend(sub_r)
    
    # Filtrar semillas que tengan al menos min_seed_points
    seed_clusters = [seed for seed in seeds_all if len(seed) >= min_seed_points]
    print(f"Total de semillas robustas (>= {min_seed_points} puntos): {len(seed_clusters)}")
    
    # Marcar en df_filtered los puntos que pertenecen a alguna semilla
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    return seed_clusters, df_filtered

# ========================
# Asignación Iterativa de Puntos a Grupos con Validación Local
# ========================

def iterative_assignment(seed_clusters, df_filtered, theta_fit_window=30.0, r_fit_window=10.0, 
                          radius_tolerance=0.45, slope_variation_threshold=0.20, snapshot_interval=10):
    """
    Usa las semillas como grupos iniciales (con ajuste lineal para obtener pendiente, intercepto y ángulo)
    y asigna iterativamente los puntos restantes (no pertenecientes a semillas) a estos grupos, validando
    que la incorporación de cada punto mantenga la coherencia geométrica.
    Retorna estados intermedios y los grupos finales.
    """
    model = LinearRegression()
    groups = []
    # Inicializar grupos a partir de semillas
    for idx, seed in enumerate(seed_clusters):
        if len(seed) >= 2:
            X_seed = seed[['theta']].values
            y_seed = seed['r'].values
            model.fit(X_seed, y_seed)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'group_id': idx,
            'points': seed.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # Puntos restantes que no están en semillas
    df_remaining = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    df_remaining = df_remaining.sort_values(by='theta').reset_index(drop=True)
    
    temp_groups = []  # Para almacenar puntos que inicialmente no se asignan
    states = []
    iteration = 0
    
    # Bucle iterativo de asignación
    while len(df_remaining) > 0:
        point = df_remaining.iloc[0]
        df_remaining = df_remaining.iloc[1:].reset_index(drop=True)
        theta_pt = point['theta']
        r_pt = point['r']
        best_group = None
        best_distance = None
        
        # Buscar grupo candidato en el que el punto caiga dentro de una ventana local
        for g in groups:
            g_theta_min = g['points']['theta'].min()
            g_theta_max = g['points']['theta'].max()
            g_r_min = g['points']['r'].min()
            g_r_max = g['points']['r'].max()
            if not (g_theta_min <= theta_pt <= g_theta_max and g_r_min <= r_pt <= g_r_max):
                continue
            # Definir ventana local para ajuste
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = g['points'][(g['points']['theta'] >= window_min) & (g['points']['theta'] <= window_max)]
            if subset.empty:
                continue
            # Agregar el punto y reajustar la recta
            subset = pd.concat([subset, point.to_frame().T], ignore_index=True)
            if len(subset) < 2:
                continue
            model.fit(subset[['theta']], subset['r'])
            new_slope = model.coef_[0]
            new_intercept = model.intercept_
            old_slope = g['slope']
            # Validar que la variación en la pendiente no exceda el umbral
            if old_slope is not None and abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) > slope_variation_threshold:
                dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
                if dist <= radius_tolerance:
                    if best_distance is None or dist < best_distance:
                        best_distance = dist
                        best_group = g
        
        if best_group is not None:
            # Asigna el punto al grupo y actualiza sus parámetros
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = best_group['points'][(best_group['points']['theta'] >= window_min) & (best_group['points']['theta'] <= window_max)]
            if len(subset) < 2:
                subset = best_group['points']
            model.fit(subset[['theta']], subset['r'])
            best_group['slope'] = model.coef_[0]
            best_group['intercept'] = model.intercept_
            best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
        else:
            # Si no se pudo asignar, se agrega a un grupo temporal
            temp_groups.append(point.to_frame().T)
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_remaining) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_points': pd.concat(temp_groups, ignore_index=True) if temp_groups else pd.DataFrame(),
                'remaining': df_remaining.copy()
            })
    
    # Reintentar asignar puntos temporales a los grupos existentes
    if temp_groups:
        temp_df = pd.concat(temp_groups, ignore_index=True)
        for idx, point in temp_df.iterrows():
            assigned = False
            for g in groups:
                if (g['points']['theta'].min() - theta_fit_window/2 <= point['theta'] <= g['points']['theta'].max() + theta_fit_window/2 and
                    g['points']['r'].min() - r_fit_window/2 <= point['r'] <= g['points']['r'].max() + r_fit_window/2):
                    g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                    model.fit(g['points'][['theta']], g['points']['r'])
                    g['slope'] = model.coef_[0]
                    g['intercept'] = model.intercept_
                    g['pa'] = calculate_pa(g['slope'], g['intercept'])
                    assigned = True
                    break
            if not assigned:
                # Si aun no se asigna, se puede dejar descartado o manejarlo aparte
                pass
    
    return states, groups, df_filtered

# ========================
# Validación de Gaps en θ y en r
# ========================

def validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r):
    """
    Valida que el grupo no tenga gaps en θ ni en r.
    Primero se subdivide por gaps en θ y luego, para cada subgrupo, se subdivide por gaps en r.
    Retorna una lista de grupos validados.
    """
    subgroups_theta = subdivide_by_gap(group['points'], gap_threshold_theta, mode='theta')
    refined_groups = []
    for sg in subgroups_theta:
        subgroups_r = subdivide_by_gap(sg, gap_threshold_r, mode='r')
        for sgr in subgroups_r:
            if len(sgr) >= 2:
                model = LinearRegression()
                model.fit(sgr[['theta']], sgr['r'])
                slope = model.coef_[0]
                intercept = model.intercept_
                pa = calculate_pa(slope, intercept)
            else:
                slope, intercept, pa = None, None, None
            refined_groups.append({
                'group_id': group.get('group_id'),
                'points': sgr,
                'slope': slope,
                'intercept': intercept,
                'pa': pa
            })
    return refined_groups

# ========================
# Cálculo de la Dispersión
# ========================

def compute_dispersion(group):
    """
    Calcula la dispersión del grupo como la desviación estándar de las distancias de cada punto
    a la recta ajustada (usando calculate_distance).
    """
    if group['slope'] is None or group['points'].empty:
        return 0
    distances = group['points'].apply(lambda row: calculate_distance(group['slope'], group['intercept'], row['theta'], row['r']), axis=1)
    return distances.std()

# ========================
# Refinamiento Recursivo de Grupos basándose en la Dispersión
# ========================

def refine_group(group, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    """
    Refina recursivamente un grupo:
      - Primero valida que no tenga gaps en θ ni en r.
      - Si la dispersión (std de las distancias) es mayor que dispersion_threshold, se divide el grupo en dos
        (por la mediana de θ) y se refina cada subgrupo.
    Retorna una lista de grupos refinados.
    """
    # Validar gaps en el grupo
    valid_groups = validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r)
    refined = []
    for vg in valid_groups:
        if len(vg['points']) < min_points:
            refined.append(vg)
            continue
        disp = compute_dispersion(vg)
        if disp <= dispersion_threshold:
            refined.append(vg)
        else:
            # Dividir el grupo usando la mediana de θ
            pts = vg['points'].sort_values(by='theta').reset_index(drop=True)
            median_index = len(pts) // 2
            group1_pts = pts.iloc[:median_index].copy()
            group2_pts = pts.iloc[median_index:].copy()
            model = LinearRegression()
            if len(group1_pts) >= 2:
                model.fit(group1_pts[['theta']], group1_pts['r'])
                slope1 = model.coef_[0]
                intercept1 = model.intercept_
                pa1 = calculate_pa(slope1, intercept1)
            else:
                slope1, intercept1, pa1 = None, None, None
            if len(group2_pts) >= 2:
                model.fit(group2_pts[['theta']], group2_pts['r'])
                slope2 = model.coef_[0]
                intercept2 = model.intercept_
                pa2 = calculate_pa(slope2, intercept2)
            else:
                slope2, intercept2, pa2 = None, None, None
            sub_group1 = {
                'group_id': vg.get('group_id'),
                'points': group1_pts,
                'slope': slope1,
                'intercept': intercept1,
                'pa': pa1
            }
            sub_group2 = {
                'group_id': vg.get('group_id'),
                'points': group2_pts,
                'slope': slope2,
                'intercept': intercept2,
                'pa': pa2
            }
            # Refinar recursivamente cada subgrupo
            refined.extend(refine_group(sub_group1, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
            refined.extend(refine_group(sub_group2, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined

def refine_all_groups(groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    """
    Aplica el refinamiento recursivo a todos los grupos de la lista.
    """
    refined_groups = []
    for g in groups:
        refined_groups.extend(refine_group(g, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined_groups

# ========================
# Fusión y Subdivisión Final de Grupos (por gap en θ, como paso final)
# ========================

def subdivide_if_gap_final(group, max_theta_gap=15.0):
    """
    Divide el grupo en dos segmentos si se detecta un gap significativo en la variable θ.
    """
    pts = group['points'].sort_values(by='theta').reset_index(drop=True)
    if len(pts) < 4:
        return [group]
    theta_vals = pts['theta'].values
    dtheta = np.diff(theta_vals)
    gap_idx = np.where(dtheta > max_theta_gap)[0]
    if len(gap_idx) == 0:
        return [group]
    split_idx = gap_idx[0]
    pts1 = pts.iloc[:split_idx+1]
    pts2 = pts.iloc[split_idx+1:]
    model = LinearRegression()
    if len(pts1) >= 2:
        model.fit(pts1[['theta']], pts1['r'])
        s1 = model.coef_[0]
        i1 = model.intercept_
        pa1 = calculate_pa(s1, i1)
    else:
        s1, i1, pa1 = None, None, None
    if len(pts2) >= 2:
        model.fit(pts2[['theta']], pts2['r'])
        s2 = model.coef_[0]
        i2 = model.intercept_
        pa2 = calculate_pa(s2, i2)
    else:
        s2, i2, pa2 = None, None, None
    seg1 = {
        'group_id': group.get('group_id'),
        'points': pts1,
        'slope': s1,
        'intercept': i1,
        'pa': pa1
    }
    seg2 = {
        'group_id': group.get('group_id'),
        'points': pts2,
        'slope': s2,
        'intercept': i2,
        'pa': pa2
    }
    return [seg1, seg2]

def finalize_groups(groups, max_theta_gap=15.0, flexible_pa_range=(4.0, 30.0)):
    """
    Aplica una subdivisión final por gaps en θ y filtra los grupos que cumplan con un rango flexible para pa.
    """
    final_groups = []
    for g in groups:
        segs = subdivide_if_gap_final(g, max_theta_gap)
        final_groups.extend(segs)
    
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    return valid_groups

# ========================
# Funciones de Visualización
# ========================

def plot_intermediate_state_cartesian(state, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    if 'temp_points' in state and not state['temp_points'].empty:
        plt.scatter(state['temp_points']['theta'], state['temp_points']['r'], s=10, color='black', marker='x', label='Temp')
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def plot_final_groups(final_groups, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, grp in enumerate(final_groups):
        pts = grp['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Segmento {idx+1}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title("Segmentos Finales (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

# ========================
# Proceso Integrado
# ========================

def process_spiral_arms_integrated(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5,
    min_seed_points=60,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    radius_tolerance=0.45,
    slope_variation_threshold=0.20,
    snapshot_interval=10,
    max_theta_gap=15.0,
    flexible_pa_range=(4.0, 30.0),
    dispersion_threshold=0.35  # Parámetro para decidir si un grupo es muy disperso
):
    """
    Ejecuta el proceso integrado:
      1. Detecta semillas robustas.
      2. Asigna iterativamente los puntos restantes a grupos existentes.
      3. Aplica una fusión/subdivisión final.
      4. Refina recursivamente los grupos que tengan alta dispersión, validando que no tengan gaps en r ni en θ.
    """
    # 1. Detección de semillas
    seed_clusters, df_filtered = detect_seeds(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points
    )
    
    # 2. Asignación iterativa de puntos restantes
    states, groups, df_filtered = iterative_assignment(
        seed_clusters, df_filtered,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval
    )
    
    # 3. Fusión y subdivisión final de grupos (por gaps en θ)
    preliminary_groups = finalize_groups(groups, max_theta_gap=max_theta_gap, flexible_pa_range=flexible_pa_range)
    
    # 4. Refinamiento recursivo basado en la dispersión y validación de gaps en θ y r
    refined_groups = refine_all_groups(preliminary_groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r)
    
    return states, refined_groups, df_filtered

# ========================
# Ejecución Interactiva
# ========================

if __name__ == '__main__':
    # Parámetros de ejemplo (ajusta según tus necesidades)
    id_halo = "17"
    quartile_threshold = 0.55
    theta_min = 20.0
    theta_max = 270.0
    theta_diff = 4.0
    r_diff = 1.0
    gap_threshold_theta = 1.5
    gap_threshold_r = 1.5
    min_seed_points = 50
    theta_fit_window = 20.0
    r_fit_window = 2.0
    radius_tolerance = 0.345
    slope_variation_threshold = 0.18
    snapshot_interval = 10
    max_theta_gap = 8.0
    flexible_pa_range = (4.0, 30.0)
    dispersion_threshold = 0.35  # Ajusta según la escala de las distancias
    
    # Ejecutar el proceso integrado
    states, final_groups, df_filtered = process_spiral_arms_integrated(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval,
        max_theta_gap=max_theta_gap,
        flexible_pa_range=flexible_pa_range,
        dispersion_threshold=dispersion_threshold
    )
    
    print(f"Total de estados intermedios: {len(states)}")
    print(f"Total de segmentos finales refinados: {len(final_groups)}")
    
    # Función interactiva para visualizar estados intermedios
    def view_state(state_index):
        state = states[state_index]
        plot_intermediate_state_cartesian(state, df_filtered)
    
    interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))
    
    # Visualizar los segmentos finales refinados
    plot_final_groups(final_groups, df_filtered)


In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from collections import deque
import copy
from ipywidgets import interact, IntSlider

# ========================
# Funciones Auxiliares
# ========================

def calculate_distance(slope, intercept, theta, r):
    """Calcula la distancia vertical (en r) de un punto a la recta definida por slope e intercept."""
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    """
    Calcula el ángulo (en grados) de la recta a partir de su pendiente.
    Si se dispone de intercepto, se usa para un cálculo ajustado.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.degrees(np.arctan(slope))

# ========================
# Carga y Preprocesamiento de Datos
# ========================

def load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold):
    """
    Carga el CSV, calcula coordenadas polares y filtra puntos:
      - Por rango de θ (en grados).
      - Por percentil en 'rho_resta_final_exp'.
    """
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    # Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    return df_filtered

# ========================
# Detección de Semillas utilizando Grafos y BFS
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo conectando puntos si la diferencia en θ y en r es menor o igual a los umbrales.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Se excluye el mismo punto
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph

def bfs_components(graph, df_points):
    """
    Extrae las componentes conexas (clusters) del grafo usando búsqueda en anchura (BFS).
    Cada componente se retorna como un DataFrame.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold, mode='theta'):
    """
    Ordena el DataFrame según 'theta' o 'r' y subdivide el cluster en subclusters 
    si existe un gap mayor que gap_threshold.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def detect_seeds(id_halo, quartile_threshold=0.55, theta_min=50.0, theta_max=250.0, 
                 theta_diff=5.0, r_diff=1.0, gap_threshold_theta=2.5, gap_threshold_r=2.5, min_seed_points=60):
    """
    Detecta semillas robustas usando la construcción de grafo, BFS y subdivisión por gaps.
    Retorna una lista de DataFrames (cada uno es una semilla) y el DataFrame filtrado.
    """
    df_filtered = load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold)
    
    # Construir grafo y extraer componentes conexas
    graph = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    # Subdividir cada cluster según gaps en θ y luego en r
    seeds_all = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = []
            for group in subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r'):
                sub_r.append(group)
            seeds_all.extend(sub_r)
    
    # Filtrar semillas que tengan al menos min_seed_points
    seed_clusters = [seed for seed in seeds_all if len(seed) >= min_seed_points]
    print(f"Total de semillas robustas (>= {min_seed_points} puntos): {len(seed_clusters)}")
    
    # Marcar en df_filtered los puntos que pertenecen a alguna semilla
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    return seed_clusters, df_filtered

# ========================
# Asignación Iterativa de Puntos a Grupos con Validación Local
# ========================

def iterative_assignment(seed_clusters, df_filtered, theta_fit_window=30.0, r_fit_window=10.0, 
                          radius_tolerance=0.45, slope_variation_threshold=0.20, snapshot_interval=10):
    """
    Usa las semillas como grupos iniciales (con ajuste lineal para obtener pendiente, intercepto y ángulo)
    y asigna iterativamente los puntos restantes (no pertenecientes a semillas) a estos grupos, validando
    que la incorporación de cada punto mantenga la coherencia geométrica.
    Retorna estados intermedios y los grupos finales.
    """
    model = LinearRegression()
    groups = []
    # Inicializar grupos a partir de semillas
    for idx, seed in enumerate(seed_clusters):
        if len(seed) >= 2:
            X_seed = seed[['theta']].values
            y_seed = seed['r'].values
            model.fit(X_seed, y_seed)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'group_id': idx,
            'points': seed.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # Puntos restantes que no están en semillas
    df_remaining = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    df_remaining = df_remaining.sort_values(by='theta').reset_index(drop=True)
    
    temp_groups = []  # Para almacenar puntos que inicialmente no se asignan
    states = []
    iteration = 0
    
    # Bucle iterativo de asignación
    while len(df_remaining) > 0:
        point = df_remaining.iloc[0]
        df_remaining = df_remaining.iloc[1:].reset_index(drop=True)
        theta_pt = point['theta']
        r_pt = point['r']
        best_group = None
        best_distance = None
        
        # Buscar grupo candidato en el que el punto caiga dentro de una ventana local
        for g in groups:
            g_theta_min = g['points']['theta'].min()
            g_theta_max = g['points']['theta'].max()
            g_r_min = g['points']['r'].min()
            g_r_max = g['points']['r'].max()
            if not (g_theta_min <= theta_pt <= g_theta_max and g_r_min <= r_pt <= g_r_max):
                continue
            # Definir ventana local para ajuste
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = g['points'][(g['points']['theta'] >= window_min) & (g['points']['theta'] <= window_max)]
            if subset.empty:
                continue
            # Agregar el punto y reajustar la recta
            subset = pd.concat([subset, point.to_frame().T], ignore_index=True)
            if len(subset) < 2:
                continue
            model.fit(subset[['theta']], subset['r'])
            new_slope = model.coef_[0]
            new_intercept = model.intercept_
            old_slope = g['slope']
            # Validar que la variación en la pendiente no exceda el umbral
            if old_slope is not None and abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) > slope_variation_threshold:
                dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
                if dist <= radius_tolerance:
                    if best_distance is None or dist < best_distance:
                        best_distance = dist
                        best_group = g
        
        if best_group is not None:
            # Asigna el punto al grupo y actualiza sus parámetros
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = best_group['points'][(best_group['points']['theta'] >= window_min) & (best_group['points']['theta'] <= window_max)]
            if len(subset) < 2:
                subset = best_group['points']
            model.fit(subset[['theta']], subset['r'])
            best_group['slope'] = model.coef_[0]
            best_group['intercept'] = model.intercept_
            best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
        else:
            # Si no se pudo asignar, se agrega a un grupo temporal
            temp_groups.append(point.to_frame().T)
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_remaining) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_points': pd.concat(temp_groups, ignore_index=True) if temp_groups else pd.DataFrame(),
                'remaining': df_remaining.copy()
            })
    
    # Reintentar asignar puntos temporales a los grupos existentes
    if temp_groups:
        temp_df = pd.concat(temp_groups, ignore_index=True)
        for idx, point in temp_df.iterrows():
            assigned = False
            for g in groups:
                if (g['points']['theta'].min() - theta_fit_window/2 <= point['theta'] <= g['points']['theta'].max() + theta_fit_window/2 and
                    g['points']['r'].min() - r_fit_window/2 <= point['r'] <= g['points']['r'].max() + r_fit_window/2):
                    g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                    model.fit(g['points'][['theta']], g['points']['r'])
                    g['slope'] = model.coef_[0]
                    g['intercept'] = model.intercept_
                    g['pa'] = calculate_pa(g['slope'], g['intercept'])
                    assigned = True
                    break
            if not assigned:
                pass
    
    return states, groups, df_filtered

# ========================
# Validación de Gaps en θ y en r
# ========================

def validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r):
    """
    Valida que el grupo no tenga gaps en θ ni en r.
    Primero se subdivide por gaps en θ y luego, para cada subgrupo, se subdivide por gaps en r.
    Retorna una lista de grupos validados.
    """
    subgroups_theta = subdivide_by_gap(group['points'], gap_threshold_theta, mode='theta')
    refined_groups = []
    for sg in subgroups_theta:
        subgroups_r = subdivide_by_gap(sg, gap_threshold_r, mode='r')
        for sgr in subgroups_r:
            if len(sgr) >= 2:
                model = LinearRegression()
                model.fit(sgr[['theta']], sgr['r'])
                slope = model.coef_[0]
                intercept = model.intercept_
                pa = calculate_pa(slope, intercept)
            else:
                slope, intercept, pa = None, None, None
            refined_groups.append({
                'group_id': group.get('group_id'),
                'points': sgr,
                'slope': slope,
                'intercept': intercept,
                'pa': pa
            })
    return refined_groups

# ========================
# Cálculo de la Dispersión
# ========================

def compute_dispersion(group):
    """
    Calcula la dispersión del grupo como la desviación estándar de las distancias de cada punto
    a la recta ajustada (usando calculate_distance).
    """
    if group['slope'] is None or group['points'].empty:
        return 0
    distances = group['points'].apply(lambda row: calculate_distance(group['slope'], group['intercept'], row['theta'], row['r']), axis=1)
    return distances.std()

# ========================
# Refinamiento Recursivo de Grupos basándose en la Dispersión
# ========================

def refine_group(group, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    """
    Refina recursivamente un grupo:
      - Primero valida que no tenga gaps en θ ni en r.
      - Si la dispersión (std de las distancias) es mayor que dispersion_threshold, se divide el grupo en dos
        (por la mediana de θ) y se refina cada subgrupo.
    Retorna una lista de grupos refinados.
    """
    # Validar gaps en el grupo
    valid_groups = validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r)
    refined = []
    for vg in valid_groups:
        if len(vg['points']) < min_points:
            refined.append(vg)
            continue
        disp = compute_dispersion(vg)
        if disp <= dispersion_threshold:
            refined.append(vg)
        else:
            # Dividir el grupo usando la mediana de θ
            pts = vg['points'].sort_values(by='theta').reset_index(drop=True)
            median_index = len(pts) // 2
            group1_pts = pts.iloc[:median_index].copy()
            group2_pts = pts.iloc[median_index:].copy()
            model = LinearRegression()
            if len(group1_pts) >= 2:
                model.fit(group1_pts[['theta']], group1_pts['r'])
                slope1 = model.coef_[0]
                intercept1 = model.intercept_
                pa1 = calculate_pa(slope1, intercept1)
            else:
                slope1, intercept1, pa1 = None, None, None
            if len(group2_pts) >= 2:
                model.fit(group2_pts[['theta']], group2_pts['r'])
                slope2 = model.coef_[0]
                intercept2 = model.intercept_
                pa2 = calculate_pa(slope2, intercept2)
            else:
                slope2, intercept2, pa2 = None, None, None
            sub_group1 = {
                'group_id': vg.get('group_id'),
                'points': group1_pts,
                'slope': slope1,
                'intercept': intercept1,
                'pa': pa1
            }
            sub_group2 = {
                'group_id': vg.get('group_id'),
                'points': group2_pts,
                'slope': slope2,
                'intercept': intercept2,
                'pa': pa2
            }
            # Refinar recursivamente cada subgrupo
            refined.extend(refine_group(sub_group1, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
            refined.extend(refine_group(sub_group2, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined

def refine_all_groups(groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    """
    Aplica el refinamiento recursivo a todos los grupos de la lista.
    """
    refined_groups = []
    for g in groups:
        refined_groups.extend(refine_group(g, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined_groups

# ========================
# Fusión y Subdivisión Final de Grupos (por gap en θ, como paso final)
# ========================

def subdivide_if_gap_final(group, max_theta_gap=15.0):
    """
    Divide el grupo en dos segmentos si se detecta un gap significativo en la variable θ.
    """
    pts = group['points'].sort_values(by='theta').reset_index(drop=True)
    if len(pts) < 4:
        return [group]
    theta_vals = pts['theta'].values
    dtheta = np.diff(theta_vals)
    gap_idx = np.where(dtheta > max_theta_gap)[0]
    if len(gap_idx) == 0:
        return [group]
    split_idx = gap_idx[0]
    pts1 = pts.iloc[:split_idx+1]
    pts2 = pts.iloc[split_idx+1:]
    model = LinearRegression()
    if len(pts1) >= 2:
        model.fit(pts1[['theta']], pts1['r'])
        s1 = model.coef_[0]
        i1 = model.intercept_
        pa1 = calculate_pa(s1, i1)
    else:
        s1, i1, pa1 = None, None, None
    if len(pts2) >= 2:
        model.fit(pts2[['theta']], pts2['r'])
        s2 = model.coef_[0]
        i2 = model.intercept_
        pa2 = calculate_pa(s2, i2)
    else:
        s2, i2, pa2 = None, None, None
    seg1 = {
        'group_id': group.get('group_id'),
        'points': pts1,
        'slope': s1,
        'intercept': i1,
        'pa': pa1
    }
    seg2 = {
        'group_id': group.get('group_id'),
        'points': pts2,
        'slope': s2,
        'intercept': i2,
        'pa': pa2
    }
    return [seg1, seg2]

def finalize_groups(groups, max_theta_gap=15.0, flexible_pa_range=(4.0, 30.0)):
    """
    Aplica una subdivisión final por gaps en θ y filtra los grupos que cumplan con un rango flexible para pa.
    """
    final_groups = []
    for g in groups:
        segs = subdivide_if_gap_final(g, max_theta_gap)
        final_groups.extend(segs)
    
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    return valid_groups

# ========================
# Funciones de Visualización
# ========================

def plot_intermediate_state_cartesian(state, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    if 'temp_points' in state and not state['temp_points'].empty:
        plt.scatter(state['temp_points']['theta'], state['temp_points']['r'], s=10, color='black', marker='x', label='Temp')
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def plot_final_groups(final_groups, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, grp in enumerate(final_groups):
        pts = grp['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Segmento {idx+1}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title("Segmentos Finales (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

# ========================
# Proceso Integrado
# ========================

def process_spiral_arms_integrated(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5,
    min_seed_points=60,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    radius_tolerance=0.45,
    slope_variation_threshold=0.20,
    snapshot_interval=10,
    max_theta_gap=15.0,
    flexible_pa_range=(4.0, 30.0),
    dispersion_threshold=0.35  # Parámetro para decidir si un grupo es muy disperso
):
    """
    Ejecuta el proceso integrado:
      1. Detecta semillas robustas.
      2. Asigna iterativamente los puntos restantes a grupos existentes.
      3. Aplica una fusión/subdivisión final.
      4. Refina recursivamente los grupos que tengan alta dispersión, validando que no tengan gaps en r ni en θ.
      5. Al final, almacena en un diccionario aquellos grupos que tienen buena dispersión y cumplen los criterios.
    """
    # 1. Detección de semillas
    seed_clusters, df_filtered = detect_seeds(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points
    )
    
    # 2. Asignación iterativa de puntos restantes
    states, groups, df_filtered = iterative_assignment(
        seed_clusters, df_filtered,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval
    )
    
    # 3. Fusión y subdivisión final de grupos (por gaps en θ)
    preliminary_groups = finalize_groups(groups, max_theta_gap=max_theta_gap, flexible_pa_range=flexible_pa_range)
    
    # 4. Refinamiento recursivo basado en la dispersión y validación de gaps en θ y r
    refined_groups = refine_all_groups(preliminary_groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r)
    
    # 5. Almacenar en un diccionario los grupos que tienen buena dispersión y cumplen criterios
    good_groups = {}
    good_counter = 0
    for g in refined_groups:
        disp = compute_dispersion(g)
        # Se consideran "buenos" aquellos grupos cuya dispersión sea menor o igual al umbral
        # y cuyo ángulo (pa) esté en el rango flexible
        if disp <= dispersion_threshold and g['slope'] is not None and g['intercept'] is not None and \
           g['pa'] is not None and flexible_pa_range[0] <= abs(g['pa']) <= flexible_pa_range[1]:
            good_groups[good_counter] = {'group': g, 'dispersion': disp}
            good_counter += 1
    
    return states, refined_groups, good_groups, df_filtered

# ========================
# Ejecución Interactiva
# ========================

if __name__ == '__main__':
    # Parámetros de ejemplo (ajusta según tus necesidades)
    id_halo = "17"
    quartile_threshold = 0.55
    theta_min = 20.0
    theta_max = 270.0
    theta_diff = 4.0
    r_diff = 1.0
    gap_threshold_theta = 1.5
    gap_threshold_r = 1.5
    min_seed_points = 50
    theta_fit_window = 20.0
    r_fit_window = 2.0
    radius_tolerance = 0.345
    slope_variation_threshold = 0.18
    snapshot_interval = 10
    max_theta_gap = 8.0
    flexible_pa_range = (4.0, 30.0)
    dispersion_threshold = 0.35  # Ajusta según la escala de las distancias
    
    # Ejecutar el proceso integrado
    states, final_groups, good_groups, df_filtered = process_spiral_arms_integrated(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval,
        max_theta_gap=max_theta_gap,
        flexible_pa_range=flexible_pa_range,
        dispersion_threshold=dispersion_threshold
    )
    
    print(f"Total de estados intermedios: {len(states)}")
    print(f"Total de segmentos finales refinados: {len(final_groups)}")
    print(f"Total de grupos con buena dispersión: {len(good_groups)}")
    
    # Función interactiva para visualizar estados intermedios
    def view_state(state_index):
        state = states[state_index]
        plot_intermediate_state_cartesian(state, df_filtered)
    
    interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))
    
    # Visualizar los segmentos finales refinados
    plot_final_groups(final_groups, df_filtered)
    
    # Visualizar los grupos con buena dispersión (convertidos a lista)
    good_groups_list = [entry['group'] for entry in good_groups.values()]
    if good_groups_list:
        print("Mostrando grupos con buena dispersión:")
        plot_final_groups(good_groups_list, df_filtered)
    else:
        print("No se encontraron grupos que cumplan con los criterios de buena dispersión.")


In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from collections import deque
import copy
from ipywidgets import interact, IntSlider

# ========================
# Funciones Auxiliares
# ========================

def calculate_distance(slope, intercept, theta, r):
    """Calcula la distancia vertical (en r) de un punto a la recta definida por slope e intercept."""
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    """
    Calcula el ángulo (en grados) de la recta a partir de su pendiente.
    Si se dispone de intercepto, se usa para un cálculo ajustado.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.degrees(np.arctan(slope))

# ========================
# Carga y Preprocesamiento de Datos
# ========================

def load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold):
    """
    Carga el CSV, calcula coordenadas polares y filtra puntos:
      - Por rango de θ (en grados).
      - Por percentil en 'rho_resta_final_exp'.
    """
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    # Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    return df_filtered

# ========================
# Detección de Semillas utilizando Grafos y BFS
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo conectando puntos si la diferencia en θ y en r es menor o igual a los umbrales.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Se excluye el mismo punto
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph

def bfs_components(graph, df_points):
    """
    Extrae las componentes conexas (clusters) del grafo usando búsqueda en anchura (BFS).
    Cada componente se retorna como un DataFrame.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold, mode='theta'):
    """
    Ordena el DataFrame según 'theta' o 'r' y subdivide el cluster en subclusters 
    si existe un gap mayor que gap_threshold.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def detect_seeds(id_halo, quartile_threshold=0.55, theta_min=50.0, theta_max=250.0, 
                 theta_diff=5.0, r_diff=1.0, gap_threshold_theta=2.5, gap_threshold_r=2.5, min_seed_points=60):
    """
    Detecta semillas robustas usando la construcción de grafo, BFS y subdivisión por gaps.
    Retorna una lista de DataFrames (cada uno es una semilla) y el DataFrame filtrado.
    """
    df_filtered = load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold)
    
    # Construir grafo y extraer componentes conexas
    graph = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    # Subdividir cada cluster según gaps en θ y luego en r
    seeds_all = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = []
            for group in subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r'):
                sub_r.append(group)
            seeds_all.extend(sub_r)
    
    # Filtrar semillas que tengan al menos min_seed_points
    seed_clusters = [seed for seed in seeds_all if len(seed) >= min_seed_points]
    print(f"Total de semillas robustas (>= {min_seed_points} puntos): {len(seed_clusters)}")
    
    # Marcar en df_filtered los puntos que pertenecen a alguna semilla
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    return seed_clusters, df_filtered

# ========================
# Asignación Iterativa de Puntos a Grupos con Validación Local
# ========================

def iterative_assignment(seed_clusters, df_filtered, theta_fit_window=30.0, r_fit_window=10.0, 
                          radius_tolerance=0.45, slope_variation_threshold=0.20, snapshot_interval=10):
    """
    Usa las semillas como grupos iniciales (con ajuste lineal para obtener pendiente, intercepto y ángulo)
    y asigna iterativamente los puntos restantes (no pertenecientes a semillas) a estos grupos, validando
    que la incorporación de cada punto mantenga la coherencia geométrica.
    Retorna estados intermedios y los grupos finales.
    """
    model = LinearRegression()
    groups = []
    # Inicializar grupos a partir de semillas
    for idx, seed in enumerate(seed_clusters):
        if len(seed) >= 2:
            X_seed = seed[['theta']].values
            y_seed = seed['r'].values
            model.fit(X_seed, y_seed)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'group_id': idx,
            'points': seed.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # Puntos restantes que no están en semillas
    df_remaining = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    df_remaining = df_remaining.sort_values(by='theta').reset_index(drop=True)
    
    temp_groups = []  # Para almacenar puntos que inicialmente no se asignan
    states = []
    iteration = 0
    
    # Bucle iterativo de asignación
    while len(df_remaining) > 0:
        point = df_remaining.iloc[0]
        df_remaining = df_remaining.iloc[1:].reset_index(drop=True)
        theta_pt = point['theta']
        r_pt = point['r']
        best_group = None
        best_distance = None
        
        # Buscar grupo candidato en el que el punto caiga dentro de una ventana local
        for g in groups:
            g_theta_min = g['points']['theta'].min()
            g_theta_max = g['points']['theta'].max()
            g_r_min = g['points']['r'].min()
            g_r_max = g['points']['r'].max()
            if not (g_theta_min <= theta_pt <= g_theta_max and g_r_min <= r_pt <= g_r_max):
                continue
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = g['points'][(g['points']['theta'] >= window_min) & (g['points']['theta'] <= window_max)]
            if subset.empty:
                continue
            subset = pd.concat([subset, point.to_frame().T], ignore_index=True)
            if len(subset) < 2:
                continue
            model.fit(subset[['theta']], subset['r'])
            new_slope = model.coef_[0]
            new_intercept = model.intercept_
            old_slope = g['slope']
            if old_slope is not None and abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) > slope_variation_threshold:
                dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
                if dist <= radius_tolerance:
                    if best_distance is None or dist < best_distance:
                        best_distance = dist
                        best_group = g
        
        if best_group is not None:
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = best_group['points'][(best_group['points']['theta'] >= window_min) & (best_group['points']['theta'] <= window_max)]
            if len(subset) < 2:
                subset = best_group['points']
            model.fit(subset[['theta']], subset['r'])
            best_group['slope'] = model.coef_[0]
            best_group['intercept'] = model.intercept_
            best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
        else:
            temp_groups.append(point.to_frame().T)
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_remaining) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_points': pd.concat(temp_groups, ignore_index=True) if temp_groups else pd.DataFrame(),
                'remaining': df_remaining.copy()
            })
    
    if temp_groups:
        temp_df = pd.concat(temp_groups, ignore_index=True)
        for idx, point in temp_df.iterrows():
            assigned = False
            for g in groups:
                if (g['points']['theta'].min() - theta_fit_window/2 <= point['theta'] <= g['points']['theta'].max() + theta_fit_window/2 and
                    g['points']['r'].min() - r_fit_window/2 <= point['r'] <= g['points']['r'].max() + r_fit_window/2):
                    g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                    model.fit(g['points'][['theta']], g['points']['r'])
                    g['slope'] = model.coef_[0]
                    g['intercept'] = model.intercept_
                    g['pa'] = calculate_pa(g['slope'], g['intercept'])
                    assigned = True
                    break
            if not assigned:
                pass
    
    return states, groups, df_filtered

# ========================
# Validación de Gaps en θ y en r
# ========================

def validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r):
    """
    Valida que el grupo no tenga gaps en θ ni en r.
    Primero se subdivide por gaps en θ y luego, para cada subgrupo, se subdivide por gaps en r.
    Retorna una lista de grupos validados.
    """
    subgroups_theta = subdivide_by_gap(group['points'], gap_threshold_theta, mode='theta')
    refined_groups = []
    for sg in subgroups_theta:
        subgroups_r = subdivide_by_gap(sg, gap_threshold_r, mode='r')
        for sgr in subgroups_r:
            if len(sgr) >= 2:
                model = LinearRegression()
                model.fit(sgr[['theta']], sgr['r'])
                slope = model.coef_[0]
                intercept = model.intercept_
                pa = calculate_pa(slope, intercept)
            else:
                slope, intercept, pa = None, None, None
            refined_groups.append({
                'group_id': group.get('group_id'),
                'points': sgr,
                'slope': slope,
                'intercept': intercept,
                'pa': pa
            })
    return refined_groups

# ========================
# Cálculo de la Dispersión
# ========================

def compute_dispersion(group):
    """
    Calcula la dispersión del grupo como la desviación estándar de las distancias de cada punto
    a la recta ajustada (usando calculate_distance).
    """
    if group['slope'] is None or group['points'].empty:
        return 0
    distances = group['points'].apply(lambda row: calculate_distance(group['slope'], group['intercept'], row['theta'], row['r']), axis=1)
    return distances.std()

# ========================
# Refinamiento Recursivo de Grupos basándose en la Dispersión
# ========================

def refine_group(group, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    """
    Refina recursivamente un grupo:
      - Primero valida que no tenga gaps en θ ni en r.
      - Si la dispersión es mayor que dispersion_threshold, se divide el grupo en dos (por la mediana de θ)
        y se refina cada subgrupo.
    Retorna una lista de grupos refinados.
    """
    valid_groups = validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r)
    refined = []
    for vg in valid_groups:
        if len(vg['points']) < min_points:
            refined.append(vg)
            continue
        disp = compute_dispersion(vg)
        if disp <= dispersion_threshold:
            refined.append(vg)
        else:
            pts = vg['points'].sort_values(by='theta').reset_index(drop=True)
            median_index = len(pts) // 2
            group1_pts = pts.iloc[:median_index].copy()
            group2_pts = pts.iloc[median_index:].copy()
            model = LinearRegression()
            if len(group1_pts) >= 2:
                model.fit(group1_pts[['theta']], group1_pts['r'])
                slope1 = model.coef_[0]
                intercept1 = model.intercept_
                pa1 = calculate_pa(slope1, intercept1)
            else:
                slope1, intercept1, pa1 = None, None, None
            if len(group2_pts) >= 2:
                model.fit(group2_pts[['theta']], group2_pts['r'])
                slope2 = model.coef_[0]
                intercept2 = model.intercept_
                pa2 = calculate_pa(slope2, intercept2)
            else:
                slope2, intercept2, pa2 = None, None, None
            sub_group1 = {
                'group_id': vg.get('group_id'),
                'points': group1_pts,
                'slope': slope1,
                'intercept': intercept1,
                'pa': pa1
            }
            sub_group2 = {
                'group_id': vg.get('group_id'),
                'points': group2_pts,
                'slope': slope2,
                'intercept': intercept2,
                'pa': pa2
            }
            refined.extend(refine_group(sub_group1, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
            refined.extend(refine_group(sub_group2, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined

def refine_all_groups(groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    """
    Aplica el refinamiento recursivo a todos los grupos de la lista.
    """
    refined_groups = []
    for g in groups:
        refined_groups.extend(refine_group(g, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined_groups

# ========================
# Fusión y Subdivisión Final de Grupos (por gap en θ)
# ========================

def subdivide_if_gap_final(group, max_theta_gap=15.0):
    """
    Divide el grupo en dos segmentos si se detecta un gap significativo en la variable θ.
    """
    pts = group['points'].sort_values(by='theta').reset_index(drop=True)
    if len(pts) < 4:
        return [group]
    theta_vals = pts['theta'].values
    dtheta = np.diff(theta_vals)
    gap_idx = np.where(dtheta > max_theta_gap)[0]
    if len(gap_idx) == 0:
        return [group]
    split_idx = gap_idx[0]
    pts1 = pts.iloc[:split_idx+1]
    pts2 = pts.iloc[split_idx+1:]
    model = LinearRegression()
    if len(pts1) >= 2:
        model.fit(pts1[['theta']], pts1['r'])
        s1 = model.coef_[0]
        i1 = model.intercept_
        pa1 = calculate_pa(s1, i1)
    else:
        s1, i1, pa1 = None, None, None
    if len(pts2) >= 2:
        model.fit(pts2[['theta']], pts2['r'])
        s2 = model.coef_[0]
        i2 = model.intercept_
        pa2 = calculate_pa(s2, i2)
    else:
        s2, i2, pa2 = None, None, None
    seg1 = {
        'group_id': group.get('group_id'),
        'points': pts1,
        'slope': s1,
        'intercept': i1,
        'pa': pa1
    }
    seg2 = {
        'group_id': group.get('group_id'),
        'points': pts2,
        'slope': s2,
        'intercept': i2,
        'pa': pa2
    }
    return [seg1, seg2]

def finalize_groups(groups, max_theta_gap=15.0, flexible_pa_range=(4.0, 30.0)):
    """
    Aplica una subdivisión final por gaps en θ y filtra los grupos que cumplan con un rango flexible para pa.
    """
    final_groups = []
    for g in groups:
        segs = subdivide_if_gap_final(g, max_theta_gap)
        final_groups.extend(segs)
    
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    return valid_groups

# ========================
# Extrapolación de Good Groups
# ========================

def extrapolate_good_groups(good_groups, refined_groups, slope_variation_merge=0.28, theta_relative_margin=0.28, r_relative_margin=0.28):
    """
    Para cada good group, se revisan los grupos candidatos en refined_groups (que no sean el mismo)
    y si la diferencia relativa en pendiente es ≤ slope_variation_merge y los rangos de θ y r se solapan
    dentro de un margen relativo (theta_relative_margin y r_relative_margin), se fusionan.
    Se recalculan los parámetros y se actualiza el good group.
    """
    model = LinearRegression()
    updated_good_groups = {}
    for key, entry in good_groups.items():
        good_group = entry['group']
        good_theta_min = good_group['points']['theta'].min()
        good_theta_max = good_group['points']['theta'].max()
        theta_range = good_theta_max - good_theta_min if good_theta_max - good_theta_min != 0 else 1.0
        
        good_r_min = good_group['points']['r'].min()
        good_r_max = good_group['points']['r'].max()
        r_range = good_r_max - good_r_min if good_r_max - good_r_min != 0 else 1.0
        
        for candidate in refined_groups:
            if candidate is good_group:
                continue
            if good_group['slope'] is None or candidate['slope'] is None:
                continue
            relative_diff = abs(candidate['slope'] - good_group['slope']) / (abs(good_group['slope']) + 1e-12)
            if relative_diff <= slope_variation_merge:
                cand_theta_min = candidate['points']['theta'].min()
                cand_theta_max = candidate['points']['theta'].max()
                cand_r_min = candidate['points']['r'].min()
                cand_r_max = candidate['points']['r'].max()
                theta_condition = (cand_theta_min >= good_theta_min - theta_relative_margin * theta_range and
                                   cand_theta_max <= good_theta_max + theta_relative_margin * theta_range)
                r_condition = (cand_r_min >= good_r_min - r_relative_margin * r_range and
                               cand_r_max <= good_r_max + r_relative_margin * r_range)
                if theta_condition and r_condition:
                    merged_points = pd.concat([good_group['points'], candidate['points']], ignore_index=True)
                    if len(merged_points) >= 2:
                        model.fit(merged_points[['theta']], merged_points['r'])
                        new_slope = model.coef_[0]
                        new_intercept = model.intercept_
                        new_pa = calculate_pa(new_slope, new_intercept)
                    else:
                        new_slope, new_intercept, new_pa = good_group['slope'], good_group['intercept'], good_group['pa']
                    good_group['points'] = merged_points
                    good_group['slope'] = new_slope
                    good_group['intercept'] = new_intercept
                    good_group['pa'] = new_pa
                    good_theta_min = good_group['points']['theta'].min()
                    good_theta_max = good_group['points']['theta'].max()
                    theta_range = good_theta_max - good_theta_min if good_theta_max - good_theta_min != 0 else 1.0
                    good_r_min = good_group['points']['r'].min()
                    good_r_max = good_group['points']['r'].max()
                    r_range = good_r_max - good_r_min if good_r_max - good_r_min != 0 else 1.0
        updated_good_groups[key] = {'group': good_group, 'dispersion': compute_dispersion(good_group)}
    return updated_good_groups

# ========================
# Funciones de Visualización
# ========================

def plot_intermediate_state_cartesian(state, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    if 'temp_points' in state and not state['temp_points'].empty:
        plt.scatter(state['temp_points']['theta'], state['temp_points']['r'], s=10, color='black', marker='x', label='Temp')
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    #plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def plot_final_groups(final_groups, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, grp in enumerate(final_groups):
        pts = grp['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Segmento {idx+1}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title("Segmentos Finales (Plano r, θ)")
    #plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

# ========================
# Proceso Integrado
# ========================

def process_spiral_arms_integrated(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5,
    min_seed_points=60,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    radius_tolerance=0.45,
    slope_variation_threshold=0.20,
    snapshot_interval=10,
    max_theta_gap=15.0,
    flexible_pa_range=(4.0, 30.0),
    dispersion_threshold=0.35  # Umbral para dispersión
):
    """
    Ejecuta el proceso integrado:
      1. Detecta semillas robustas.
      2. Asigna iterativamente los puntos restantes a grupos.
      3. Aplica fusión/subdivisión final.
      4. Refina recursivamente los grupos con alta dispersión (sin gaps).
      5. Almacena en un diccionario los grupos (good groups) que cumplen los criterios
         y extrapola su ajuste incluyendo subgrupos con variación en pendiente ≤25%
         y solapamiento en θ y r dentro del 18%.
    """
    seed_clusters, df_filtered = detect_seeds(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points
    )
    
    states, groups, df_filtered = iterative_assignment(
        seed_clusters, df_filtered,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval
    )
    
    preliminary_groups = finalize_groups(groups, max_theta_gap=max_theta_gap, flexible_pa_range=flexible_pa_range)
    
    refined_groups = refine_all_groups(preliminary_groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r)
    
    good_groups = {}
    good_counter = 0
    for g in refined_groups:
        disp = compute_dispersion(g)
        if disp <= dispersion_threshold and g['slope'] is not None and g['intercept'] is not None and \
           g['pa'] is not None and flexible_pa_range[0] <= abs(g['pa']) <= flexible_pa_range[1]:
            good_groups[good_counter] = {'group': g, 'dispersion': disp}
            good_counter += 1

    updated_good_groups = extrapolate_good_groups(good_groups, refined_groups, slope_variation_merge=0.25, theta_relative_margin=0.18, r_relative_margin=0.18)
    
    return states, refined_groups, updated_good_groups, df_filtered

# ========================
# Ejecución Interactiva
# ========================

if __name__ == '__main__':
    # Parámetros de ejemplo (ajusta según tus necesidades)
    id_halo = "17"
    quartile_threshold = 0.55
    theta_min = 20.0
    theta_max = 270.0
    theta_diff = 4.0
    r_diff = 1.0
    gap_threshold_theta = 1.5
    gap_threshold_r = 1.5
    min_seed_points = 50
    theta_fit_window = 20.0
    r_fit_window = 2.0
    radius_tolerance = 0.345
    slope_variation_threshold = 0.18
    snapshot_interval = 10
    max_theta_gap = 8.0
    flexible_pa_range = (4.0, 30.0)
    dispersion_threshold = 0.35  # Ajusta según la escala de las distancias
    
    states, final_groups, good_groups, df_filtered = process_spiral_arms_integrated(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval,
        max_theta_gap=max_theta_gap,
        flexible_pa_range=flexible_pa_range,
        dispersion_threshold=dispersion_threshold
    )
    
    print(f"Total de estados intermedios: {len(states)}")
    print(f"Total de segmentos finales refinados: {len(final_groups)}")
    print(f"Total de Good Groups extrapolados: {len(good_groups)}")
    
    def view_state(state_index):
        state = states[state_index]
        plot_intermediate_state_cartesian(state, df_filtered)
    
    interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))
    
    plot_final_groups(final_groups, df_filtered)
    
    good_groups_list = [entry['group'] for entry in good_groups.values()]
    if good_groups_list:
        print("Mostrando Good Groups extrapolados:")
        plot_final_groups(good_groups_list, df_filtered)
    else:
        print("No se encontraron grupos que cumplan con los criterios de buena dispersión.")


In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from collections import deque
import copy
from ipywidgets import interact, IntSlider

# ========================
# Funciones Auxiliares
# ========================

def calculate_distance(slope, intercept, theta, r):
    """Calcula la distancia vertical (en r) de un punto a la recta definida por slope e intercept."""
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    """
    Calcula el ángulo (en grados) de la recta a partir de su pendiente.
    Si se dispone de intercepto, se usa para un cálculo ajustado.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.degrees(np.arctan(slope))

# ========================
# Carga y Preprocesamiento de Datos
# ========================

def load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold):
    """
    Carga el CSV, calcula coordenadas polares y filtra puntos:
      - Por rango de θ (en grados).
      - Por percentil en 'rho_resta_final_exp'.
    """
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    
    # Filtrar por percentil en 'rho_resta_final_exp'
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    
    return df_filtered

# ========================
# Detección de Semillas utilizando Grafos y BFS
# ========================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo conectando puntos si la diferencia en θ y en r es menor o igual a los umbrales.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Se excluye el mismo punto
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph

def bfs_components(graph, df_points):
    """
    Extrae las componentes conexas (clusters) del grafo usando búsqueda en anchura (BFS).
    Cada componente se retorna como un DataFrame.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold, mode='theta'):
    """
    Ordena el DataFrame según 'theta' o 'r' y subdivide el cluster en subclusters 
    si existe un gap mayor que gap_threshold.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def detect_seeds(id_halo, quartile_threshold=0.55, theta_min=50.0, theta_max=250.0, 
                 theta_diff=5.0, r_diff=1.0, gap_threshold_theta=2.5, gap_threshold_r=2.5, min_seed_points=60):
    """
    Detecta semillas robustas usando la construcción de grafo, BFS y subdivisión por gaps.
    Retorna una lista de DataFrames (cada uno es una semilla) y el DataFrame filtrado.
    """
    df_filtered = load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold)
    
    # Construir grafo y extraer componentes conexas
    graph = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    # Subdividir cada cluster según gaps en θ y luego en r
    seeds_all = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = []
            for group in subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r'):
                sub_r.append(group)
            seeds_all.extend(sub_r)
    
    # Filtrar semillas que tengan al menos min_seed_points
    seed_clusters = [seed for seed in seeds_all if len(seed) >= min_seed_points]
    print(f"Total de semillas robustas (>= {min_seed_points} puntos): {len(seed_clusters)}")
    
    # Marcar en df_filtered los puntos que pertenecen a alguna semilla
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    return seed_clusters, df_filtered

# ========================
# Asignación Iterativa de Puntos a Grupos con Validación Local
# ========================

def iterative_assignment(seed_clusters, df_filtered, theta_fit_window=30.0, r_fit_window=10.0, 
                          radius_tolerance=0.45, slope_variation_threshold=0.20, snapshot_interval=10):
    """
    Usa las semillas como grupos iniciales (con ajuste lineal para obtener pendiente, intercepto y ángulo)
    y asigna iterativamente los puntos restantes (no pertenecientes a semillas) a estos grupos, validando
    que la incorporación de cada punto mantenga la coherencia geométrica.
    Retorna estados intermedios y los grupos finales.
    """
    model = LinearRegression()
    groups = []
    # Inicializar grupos a partir de semillas
    for idx, seed in enumerate(seed_clusters):
        if len(seed) >= 2:
            X_seed = seed[['theta']].values
            y_seed = seed['r'].values
            model.fit(X_seed, y_seed)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        groups.append({
            'group_id': idx,
            'points': seed.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # Puntos restantes que no están en semillas
    df_remaining = df_filtered[~df_filtered['in_seed']].copy().reset_index(drop=True)
    df_remaining = df_remaining.sort_values(by='theta').reset_index(drop=True)
    
    temp_groups = []  # Para almacenar puntos que inicialmente no se asignan
    states = []
    iteration = 0
    
    # Bucle iterativo de asignación
    while len(df_remaining) > 0:
        point = df_remaining.iloc[0]
        df_remaining = df_remaining.iloc[1:].reset_index(drop=True)
        theta_pt = point['theta']
        r_pt = point['r']
        best_group = None
        best_distance = None
        
        # Buscar grupo candidato en el que el punto caiga dentro de una ventana local
        for g in groups:
            g_theta_min = g['points']['theta'].min()
            g_theta_max = g['points']['theta'].max()
            g_r_min = g['points']['r'].min()
            g_r_max = g['points']['r'].max()
            if not (g_theta_min <= theta_pt <= g_theta_max and g_r_min <= r_pt <= g_r_max):
                continue
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = g['points'][(g['points']['theta'] >= window_min) & (g['points']['theta'] <= window_max)]
            if subset.empty:
                continue
            subset = pd.concat([subset, point.to_frame().T], ignore_index=True)
            if len(subset) < 2:
                continue
            model.fit(subset[['theta']], subset['r'])
            new_slope = model.coef_[0]
            new_intercept = model.intercept_
            old_slope = g['slope']
            if old_slope is not None and abs(new_slope - old_slope) / (abs(old_slope) + 1e-12) > slope_variation_threshold:
                dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
                if dist <= radius_tolerance:
                    if best_distance is None or dist < best_distance:
                        best_distance = dist
                        best_group = g
        
        if best_group is not None:
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = best_group['points'][(best_group['points']['theta'] >= window_min) & (best_group['points']['theta'] <= window_max)]
            if len(subset) < 2:
                subset = best_group['points']
            model.fit(subset[['theta']], subset['r'])
            best_group['slope'] = model.coef_[0]
            best_group['intercept'] = model.intercept_
            best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
        else:
            temp_groups.append(point.to_frame().T)
        
        iteration += 1
        if iteration % snapshot_interval == 0 or len(df_remaining) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_points': pd.concat(temp_groups, ignore_index=True) if temp_groups else pd.DataFrame(),
                'remaining': df_remaining.copy()
            })
    
    if temp_groups:
        temp_df = pd.concat(temp_groups, ignore_index=True)
        for idx, point in temp_df.iterrows():
            assigned = False
            for g in groups:
                if (g['points']['theta'].min() - theta_fit_window/2 <= point['theta'] <= g['points']['theta'].max() + theta_fit_window/2 and
                    g['points']['r'].min() - r_fit_window/2 <= point['r'] <= g['points']['r'].max() + r_fit_window/2):
                    g['points'] = pd.concat([g['points'], point.to_frame().T], ignore_index=True)
                    model.fit(g['points'][['theta']], g['points']['r'])
                    g['slope'] = model.coef_[0]
                    g['intercept'] = model.intercept_
                    g['pa'] = calculate_pa(g['slope'], g['intercept'])
                    assigned = True
                    break
            if not assigned:
                pass
    
    return states, groups, df_filtered

# ========================
# Validación de Gaps en θ y en r
# ========================

def validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r):
    """
    Valida que el grupo no tenga gaps en θ ni en r.
    Primero se subdivide por gaps en θ y luego, para cada subgrupo, se subdivide por gaps en r.
    Retorna una lista de grupos validados.
    """
    subgroups_theta = subdivide_by_gap(group['points'], gap_threshold_theta, mode='theta')
    refined_groups = []
    for sg in subgroups_theta:
        subgroups_r = subdivide_by_gap(sg, gap_threshold_r, mode='r')
        for sgr in subgroups_r:
            if len(sgr) >= 2:
                model = LinearRegression()
                model.fit(sgr[['theta']], sgr['r'])
                slope = model.coef_[0]
                intercept = model.intercept_
                pa = calculate_pa(slope, intercept)
            else:
                slope, intercept, pa = None, None, None
            refined_groups.append({
                'group_id': group.get('group_id'),
                'points': sgr,
                'slope': slope,
                'intercept': intercept,
                'pa': pa
            })
    return refined_groups

# ========================
# Cálculo de la Dispersión
# ========================

def compute_dispersion(group):
    """
    Calcula la dispersión del grupo como la desviación estándar de las distancias de cada punto
    a la recta ajustada (usando calculate_distance).
    """
    if group['slope'] is None or group['points'].empty:
        return 0
    distances = group['points'].apply(lambda row: calculate_distance(group['slope'], group['intercept'], row['theta'], row['r']), axis=1)
    return distances.std()

# ========================
# Refinamiento Recursivo de Grupos basándose en la Dispersión
# ========================

def refine_group(group, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    """
    Refina recursivamente un grupo:
      - Primero valida que no tenga gaps en θ ni en r.
      - Si la dispersión es mayor que dispersion_threshold, se divide el grupo en dos (por la mediana de θ)
        y se refina cada subgrupo.
    Retorna una lista de grupos refinados.
    """
    valid_groups = validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r)
    refined = []
    for vg in valid_groups:
        if len(vg['points']) < min_points:
            refined.append(vg)
            continue
        disp = compute_dispersion(vg)
        if disp <= dispersion_threshold:
            refined.append(vg)
        else:
            pts = vg['points'].sort_values(by='theta').reset_index(drop=True)
            median_index = len(pts) // 2
            group1_pts = pts.iloc[:median_index].copy()
            group2_pts = pts.iloc[median_index:].copy()
            model = LinearRegression()
            if len(group1_pts) >= 2:
                model.fit(group1_pts[['theta']], group1_pts['r'])
                slope1 = model.coef_[0]
                intercept1 = model.intercept_
                pa1 = calculate_pa(slope1, intercept1)
            else:
                slope1, intercept1, pa1 = None, None, None
            if len(group2_pts) >= 2:
                model.fit(group2_pts[['theta']], group2_pts['r'])
                slope2 = model.coef_[0]
                intercept2 = model.intercept_
                pa2 = calculate_pa(slope2, intercept2)
            else:
                slope2, intercept2, pa2 = None, None, None
            sub_group1 = {
                'group_id': vg.get('group_id'),
                'points': group1_pts,
                'slope': slope1,
                'intercept': intercept1,
                'pa': pa1
            }
            sub_group2 = {
                'group_id': vg.get('group_id'),
                'points': group2_pts,
                'slope': slope2,
                'intercept': intercept2,
                'pa': pa2
            }
            refined.extend(refine_group(sub_group1, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
            refined.extend(refine_group(sub_group2, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined

def refine_all_groups(groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    """
    Aplica el refinamiento recursivo a todos los grupos de la lista.
    """
    refined_groups = []
    for g in groups:
        refined_groups.extend(refine_group(g, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined_groups

# ========================
# Fusión y Subdivisión Final de Grupos (por gap en θ)
# ========================

def subdivide_if_gap_final(group, max_theta_gap=15.0):
    """
    Divide el grupo en dos segmentos si se detecta un gap significativo en la variable θ.
    """
    pts = group['points'].sort_values(by='theta').reset_index(drop=True)
    if len(pts) < 4:
        return [group]
    theta_vals = pts['theta'].values
    dtheta = np.diff(theta_vals)
    gap_idx = np.where(dtheta > max_theta_gap)[0]
    if len(gap_idx) == 0:
        return [group]
    split_idx = gap_idx[0]
    pts1 = pts.iloc[:split_idx+1]
    pts2 = pts.iloc[split_idx+1:]
    model = LinearRegression()
    if len(pts1) >= 2:
        model.fit(pts1[['theta']], pts1['r'])
        s1 = model.coef_[0]
        i1 = model.intercept_
        pa1 = calculate_pa(s1, i1)
    else:
        s1, i1, pa1 = None, None, None
    if len(pts2) >= 2:
        model.fit(pts2[['theta']], pts2['r'])
        s2 = model.coef_[0]
        i2 = model.intercept_
        pa2 = calculate_pa(s2, i2)
    else:
        s2, i2, pa2 = None, None, None
    seg1 = {
        'group_id': group.get('group_id'),
        'points': pts1,
        'slope': s1,
        'intercept': i1,
        'pa': pa1
    }
    seg2 = {
        'group_id': group.get('group_id'),
        'points': pts2,
        'slope': s2,
        'intercept': i2,
        'pa': pa2
    }
    return [seg1, seg2]

def finalize_groups(groups, max_theta_gap=15.0, flexible_pa_range=(4.0, 30.0)):
    """
    Aplica una subdivisión final por gaps en θ y filtra los grupos que cumplan con un rango flexible para pa.
    """
    final_groups = []
    for g in groups:
        segs = subdivide_if_gap_final(g, max_theta_gap)
        final_groups.extend(segs)
    
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    return valid_groups

# ========================
# Extrapolación de Good Groups
# ========================

def extrapolate_good_groups(good_groups, refined_groups, slope_variation_merge=0.28, theta_relative_margin=0.18, r_relative_margin=0.18):
    """
    Para cada good group, se revisan los grupos candidatos en refined_groups (que no sean el mismo)
    y si la diferencia relativa en pendiente es ≤ slope_variation_merge (28% en este caso)
    y los rangos de θ y r se solapan dentro de un margen relativo (theta_relative_margin y r_relative_margin),
    se fusionan los puntos, se recalculan los parámetros y se actualiza el good group.
    """
    model = LinearRegression()
    updated_good_groups = {}
    for key, entry in good_groups.items():
        good_group = entry['group']
        good_theta_min = good_group['points']['theta'].min()
        good_theta_max = good_group['points']['theta'].max()
        theta_range = good_theta_max - good_theta_min if good_theta_max - good_theta_min != 0 else 1.0
        
        good_r_min = good_group['points']['r'].min()
        good_r_max = good_group['points']['r'].max()
        r_range = good_r_max - good_r_min if good_r_max - good_r_min != 0 else 1.0
        
        for candidate in refined_groups:
            if candidate is good_group:
                continue
            if good_group['slope'] is None or candidate['slope'] is None:
                continue
            relative_diff = abs(candidate['slope'] - good_group['slope']) / (abs(good_group['slope']) + 1e-12)
            if relative_diff <= slope_variation_merge:
                cand_theta_min = candidate['points']['theta'].min()
                cand_theta_max = candidate['points']['theta'].max()
                cand_r_min = candidate['points']['r'].min()
                cand_r_max = candidate['points']['r'].max()
                theta_condition = (cand_theta_min >= good_theta_min - theta_relative_margin * theta_range and
                                   cand_theta_max <= good_theta_max + theta_relative_margin * theta_range)
                r_condition = (cand_r_min >= good_r_min - r_relative_margin * r_range and
                               cand_r_max <= good_r_max + r_relative_margin * r_range)
                if theta_condition and r_condition:
                    merged_points = pd.concat([good_group['points'], candidate['points']], ignore_index=True)
                    if len(merged_points) >= 2:
                        model.fit(merged_points[['theta']], merged_points['r'])
                        new_slope = model.coef_[0]
                        new_intercept = model.intercept_
                        new_pa = calculate_pa(new_slope, new_intercept)
                    else:
                        new_slope, new_intercept, new_pa = good_group['slope'], good_group['intercept'], good_group['pa']
                    good_group['points'] = merged_points
                    good_group['slope'] = new_slope
                    good_group['intercept'] = new_intercept
                    good_group['pa'] = new_pa
                    good_theta_min = good_group['points']['theta'].min()
                    good_theta_max = good_group['points']['theta'].max()
                    theta_range = good_theta_max - good_theta_min if good_theta_max - good_theta_min != 0 else 1.0
                    good_r_min = good_group['points']['r'].min()
                    good_r_max = good_group['points']['r'].max()
                    r_range = good_r_max - good_r_min if good_r_max - good_r_min != 0 else 1.0
        updated_good_groups[key] = {'group': good_group, 'dispersion': compute_dispersion(good_group)}
    return updated_good_groups

# ========================
# Funciones de Visualización
# ========================

def plot_intermediate_state_cartesian(state, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Grupo {idx+1}")
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    if 'temp_points' in state and not state['temp_points'].empty:
        plt.scatter(state['temp_points']['theta'], state['temp_points']['r'], s=10, color='black', marker='x', label='Temp')
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    #plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def plot_final_groups(final_groups, df_all):
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, grp in enumerate(final_groups):
        pts = grp['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Segmento {idx+1}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title("Segmentos Finales (Plano r, θ)")
    #plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

# ========================
# Proceso Integrado
# ========================

def process_spiral_arms_integrated(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5,
    min_seed_points=60,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    radius_tolerance=0.45,
    slope_variation_threshold=0.20,
    snapshot_interval=10,
    max_theta_gap=15.0,
    flexible_pa_range=(4.0, 30.0),
    dispersion_threshold=0.35  # Umbral para dispersión
):
    """
    Ejecuta el proceso integrado:
      1. Detecta semillas robustas.
      2. Asigna iterativamente los puntos restantes a grupos.
      3. Aplica fusión/subdivisión final.
      4. Refina recursivamente los grupos con alta dispersión (sin gaps).
      5. Almacena en un diccionario los grupos (good groups) que cumplen los criterios
         y extrapola su ajuste incluyendo subgrupos con variación en pendiente ≤28%
         y solapamiento en θ y r dentro del 18%.
    """
    seed_clusters, df_filtered = detect_seeds(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points
    )
    
    states, groups, df_filtered = iterative_assignment(
        seed_clusters, df_filtered,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval
    )
    
    preliminary_groups = finalize_groups(groups, max_theta_gap=max_theta_gap, flexible_pa_range=flexible_pa_range)
    
    refined_groups = refine_all_groups(preliminary_groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r)
    
    good_groups = {}
    good_counter = 0
    for g in refined_groups:
        disp = compute_dispersion(g)
        if disp <= dispersion_threshold and g['slope'] is not None and g['intercept'] is not None and \
           g['pa'] is not None and flexible_pa_range[0] <= abs(g['pa']) <= flexible_pa_range[1]:
            good_groups[good_counter] = {'group': g, 'dispersion': disp}
            good_counter += 1

    updated_good_groups = extrapolate_good_groups(good_groups, refined_groups, slope_variation_merge=0.28, theta_relative_margin=0.18, r_relative_margin=0.18)
    
    return states, refined_groups, updated_good_groups, df_filtered

# ========================
# Ejecución Interactiva
# ========================

if __name__ == '__main__':
    # Parámetros de ejemplo (ajusta según tus necesidades)
    id_halo = "17"
    quartile_threshold = 0.55
    theta_min = 20.0
    theta_max = 270.0
    theta_diff = 4.0
    r_diff = 1.0
    gap_threshold_theta = 1.5
    gap_threshold_r = 1.5
    min_seed_points = 50
    theta_fit_window = 20.0
    r_fit_window = 2.0
    radius_tolerance = 0.345
    slope_variation_threshold = 0.18
    snapshot_interval = 10
    max_theta_gap = 8.0
    flexible_pa_range = (4.0, 30.0)
    dispersion_threshold = 0.35  # Ajusta según la escala de las distancias
    
    states, final_groups, good_groups, df_filtered = process_spiral_arms_integrated(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval,
        max_theta_gap=max_theta_gap,
        flexible_pa_range=flexible_pa_range,
        dispersion_threshold=dispersion_threshold
    )
    
    print(f"Total de estados intermedios: {len(states)}")
    print(f"Total de segmentos finales refinados: {len(final_groups)}")
    print(f"Total de Good Groups extrapolados: {len(good_groups)}")
    
    def view_state(state_index):
        state = states[state_index]
        plot_intermediate_state_cartesian(state, df_filtered)
    
    interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))
    
    plot_final_groups(final_groups, df_filtered)
    
    good_groups_list = [entry['group'] for entry in good_groups.values()]
    if good_groups_list:
        print("Mostrando Good Groups extrapolados:")
        plot_final_groups(good_groups_list, df_filtered)
    else:
        print("No se encontraron grupos que cumplan con los criterios de buena dispersión.")


In [0]:
import os
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression
from collections import deque
import copy
from ipywidgets import interact, IntSlider

warnings.filterwarnings("ignore", category=DeprecationWarning)

# =============================================================================
# 1. Carga y Preprocesamiento de Datos
# =============================================================================

def load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold):
    """
    Carga el archivo CSV 'data_rho_{id_halo}_filtered.csv' desde el directorio actual,
    calcula las coordenadas polares y filtra los puntos según el percentil de 'rho_resta_final_exp'.
    """
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    return df_filtered

# =============================================================================
# 2. Obtención de Semillas con DBSCAN y Fusión de Clusters Cercanos
# =============================================================================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold, mode='theta'):
    order_col = 'theta' if mode=='theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def detect_seeds(id_halo, quartile_threshold=0.55, theta_min=50.0, theta_max=250.0, 
                 theta_diff=5.0, r_diff=1.0, gap_threshold_theta=2.5, gap_threshold_r=2.5, min_seed_points=60):
    df_filtered = load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold)
    graph = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    seeds_all = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            for group in subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r'):
                seeds_all.append(group)
    seed_clusters = [seed for seed in seeds_all if len(seed) >= min_seed_points]
    print(f"Total de semillas robustas (>= {min_seed_points} puntos): {len(seed_clusters)}")
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    return seed_clusters, df_filtered

# =============================================================================
# 3. Funciones Auxiliares para Ajuste y Validación
# =============================================================================

def calculate_distance(slope, intercept, theta, r):
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.degrees(np.arctan(slope))

def is_valid_pa(pa, valid_range):
    return valid_range[0] <= abs(pa) <= valid_range[1]

# =============================================================================
# 4. Asignación Iterativa y Refinamiento de Grupos
# =============================================================================

def iterative_assignment(seed_clusters, df_filtered, theta_fit_window=30.0, r_fit_window=10.0, 
                          radius_tolerance=0.45, slope_variation_threshold=0.20, snapshot_interval=10):
    model = LinearRegression()
    groups = []
    for idx, seed in enumerate(seed_clusters):
        if len(seed) >= 2:
            X_seed = seed[['theta']].values
            y_seed = seed['r'].values
            model.fit(X_seed, y_seed)
            groups.append({
                'group_id': idx,
                'points': seed.copy(),
                'slope': model.coef_[0],
                'intercept': model.intercept_,
                'pa': calculate_pa(model.coef_[0], model.intercept_)
            })
    states = []  # No se generan estados intermedios en esta versión
    return states, groups, df_filtered

def validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r):
    subgroups_theta = subdivide_by_gap(group['points'], gap_threshold_theta, mode='theta')
    refined_groups = []
    for sg in subgroups_theta:
        for sgr in subdivide_by_gap(sg, gap_threshold_r, mode='r'):
            if len(sgr) >= 2:
                model = LinearRegression()
                model.fit(sgr[['theta']], sgr['r'])
                slope = model.coef_[0]
                intercept = model.intercept_
                pa = calculate_pa(slope, intercept)
            else:
                slope, intercept, pa = None, None, None
            refined_groups.append({
                'group_id': group.get('group_id'),
                'points': sgr,
                'slope': slope,
                'intercept': intercept,
                'pa': pa
            })
    return refined_groups

def compute_dispersion(group):
    if group['slope'] is None or group['points'].empty:
        return 0
    distances = group['points'].apply(lambda row: calculate_distance(group['slope'], group['intercept'], row['theta'], row['r']), axis=1)
    return distances.std()

def refine_group(group, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    valid_groups = validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r)
    refined = []
    for vg in valid_groups:
        if len(vg['points']) < min_points:
            refined.append(vg)
            continue
        disp = compute_dispersion(vg)
        if disp <= dispersion_threshold:
            refined.append(vg)
        else:
            pts = vg['points'].sort_values(by='theta').reset_index(drop=True)
            median_index = len(pts) // 2
            group1_pts = pts.iloc[:median_index].copy()
            group2_pts = pts.iloc[median_index:].copy()
            model = LinearRegression()
            if len(group1_pts) >= 2:
                model.fit(group1_pts[['theta']], group1_pts['r'])
                slope1, intercept1 = model.coef_[0], model.intercept_
                pa1 = calculate_pa(slope1, intercept1)
            else:
                slope1, intercept1, pa1 = None, None, None
            if len(group2_pts) >= 2:
                model.fit(group2_pts[['theta']], group2_pts['r'])
                slope2, intercept2 = model.coef_[0], model.intercept_
                pa2 = calculate_pa(slope2, intercept2)
            else:
                slope2, intercept2, pa2 = None, None, None
            sub_group1 = {'group_id': vg.get('group_id'), 'points': group1_pts, 'slope': slope1, 'intercept': intercept1, 'pa': pa1}
            sub_group2 = {'group_id': vg.get('group_id'), 'points': group2_pts, 'slope': slope2, 'intercept': intercept2, 'pa': pa2}
            refined.extend(refine_group(sub_group1, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
            refined.extend(refine_group(sub_group2, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined

def refine_all_groups(groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    refined_groups = []
    for g in groups:
        refined_groups.extend(refine_group(g, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined_groups

def subdivide_if_gap_final(group, max_theta_gap=15.0):
    pts = group['points'].sort_values(by='theta').reset_index(drop=True)
    if len(pts) < 4:
        return [group]
    theta_vals = pts['theta'].values
    dtheta = np.diff(theta_vals)
    gap_idx = np.where(dtheta > max_theta_gap)[0]
    if len(gap_idx) == 0:
        return [group]
    split_idx = gap_idx[0]
    pts1 = pts.iloc[:split_idx+1]
    pts2 = pts.iloc[split_idx+1:]
    model = LinearRegression()
    if len(pts1) >= 2:
        model.fit(pts1[['theta']], pts1['r'])
        s1, i1 = model.coef_[0], model.intercept_
        pa1 = calculate_pa(s1, i1)
    else:
        s1, i1, pa1 = None, None, None
    if len(pts2) >= 2:
        model.fit(pts2[['theta']], pts2['r'])
        s2, i2 = model.coef_[0], model.intercept_
        pa2 = calculate_pa(s2, i2)
    else:
        s2, i2, pa2 = None, None, None
    seg1 = {'group_id': group.get('group_id'), 'points': pts1, 'slope': s1, 'intercept': i1, 'pa': pa1}
    seg2 = {'group_id': group.get('group_id'), 'points': pts2, 'slope': s2, 'intercept': i2, 'pa': pa2}
    return [seg1, seg2]

def finalize_groups(groups, max_theta_gap=15.0, flexible_pa_range=(4.0, 30.0)):
    final_groups = []
    for g in groups:
        segs = subdivide_if_gap_final(g, max_theta_gap)
        final_groups.extend(segs)
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    return valid_groups

# =============================================================================
# 5. Extrapolación de Good Groups (Revisada)
# =============================================================================

def extrapolate_good_groups(good_groups, refined_groups, slope_variation_merge=0.28, theta_relative_margin=0.18, r_relative_margin=0.18):
    """
    Para cada Good Group, se revisan los candidatos en refined_groups (diferentes del grupo)
    y se fusionan si:
      - La diferencia relativa en pendiente es ≤28%.
      - El overlap ratio de los intervalos de θ y de r es ≥ 0.5.
    Se recalculan los parámetros y se actualiza el Good Group.
    """
    model = LinearRegression()
    updated_good_groups = {}
    for key, entry in good_groups.items():
        good_group = entry['group']
        good_theta_min = good_group['points']['theta'].min()
        good_theta_max = good_group['points']['theta'].max()
        good_r_min = good_group['points']['r'].min()
        good_r_max = good_group['points']['r'].max()
        theta_range = good_theta_max - good_theta_min if good_theta_max - good_theta_min != 0 else 1.0
        r_range = good_r_max - good_r_min if good_r_max - good_r_min != 0 else 1.0
        for candidate in refined_groups:
            if candidate is good_group:
                continue
            if good_group['slope'] is None or candidate['slope'] is None:
                continue
            relative_diff = abs(candidate['slope'] - good_group['slope']) / (abs(good_group['slope']) + 1e-12)
            if relative_diff > slope_variation_merge:
                continue
            cand_theta_min = candidate['points']['theta'].min()
            cand_theta_max = candidate['points']['theta'].max()
            cand_r_min = candidate['points']['r'].min()
            cand_r_max = candidate['points']['r'].max()
            theta_overlap = max(0, min(good_theta_max, cand_theta_max) - max(good_theta_min, cand_theta_min))
            theta_union = max(good_theta_max, cand_theta_max) - min(good_theta_min, cand_theta_min)
            theta_overlap_ratio = theta_overlap / theta_union if theta_union > 0 else 0
            r_overlap = max(0, min(good_r_max, cand_r_max) - max(good_r_min, cand_r_min))
            r_union = max(good_r_max, cand_r_max) - min(good_r_min, cand_r_min)
            r_overlap_ratio = r_overlap / r_union if r_union > 0 else 0
            if theta_overlap_ratio >= 0.5 and r_overlap_ratio >= 0.5:
                merged_points = pd.concat([good_group['points'], candidate['points']], ignore_index=True)
                if len(merged_points) >= 2:
                    model.fit(merged_points[['theta']], merged_points['r'])
                    new_slope = model.coef_[0]
                    new_intercept = model.intercept_
                    new_pa = calculate_pa(new_slope, new_intercept)
                else:
                    new_slope, new_intercept, new_pa = good_group['slope'], good_group['intercept'], good_group['pa']
                good_group['points'] = merged_points
                good_group['slope'] = new_slope
                good_group['intercept'] = new_intercept
                good_group['pa'] = new_pa
                good_theta_min = good_group['points']['theta'].min()
                good_theta_max = good_group['points']['theta'].max()
                good_r_min = good_group['points']['r'].min()
                good_r_max = good_group['points']['r'].max()
                theta_range = good_theta_max - good_theta_min if good_theta_max - good_theta_min != 0 else 1.0
                r_range = good_r_max - good_r_min if good_r_max - good_r_min != 0 else 1.0
        updated_good_groups[key] = {'group': good_group, 'dispersion': compute_dispersion(good_group)}
    return updated_good_groups

# =============================================================================
# 6. Funciones de Visualización
# =============================================================================

def plot_intermediate_state_cartesian(state, df_all):
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f'Grupo {idx+1}')
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    if 'temp_points' in state and not state['temp_points'].empty:
        plt.scatter(state['temp_points']['theta'], state['temp_points']['r'], s=10, color='black', marker='x', label='Temp')
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    #plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def plot_final_groups(final_groups, df_all):
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan']
    for idx, grp in enumerate(final_groups):
        pts = grp['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Segmento {idx+1}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title("Segmentos Finales (Plano r, θ)")
    #plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

# =============================================================================
# 7. Proceso Integrado Mejorado
# =============================================================================

def process_spiral_arms_integrated_improved(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5,
    min_seed_points=60,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    radius_tolerance=0.45,
    slope_variation_threshold=0.20,
    snapshot_interval=10,
    max_theta_gap=15.0,
    flexible_pa_range=(4.0,30.0),
    dispersion_threshold=0.35
):
    """
    Proceso completo mejorado:
      1. Detecta semillas robustas.
      2. Genera grupos iniciales mediante ajuste lineal.
      3. Realiza una subdivisión final y refina los grupos según dispersión.
      4. Selecciona Good Groups y extrapola su ajuste incorporando candidatos con variación de pendiente ≤28%
         y overlap en θ y r ≥50%.
    """
    seed_clusters, df_filtered = detect_seeds(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points
    )
    states, groups, df_filtered = iterative_assignment(seed_clusters, df_filtered, theta_fit_window, r_fit_window, radius_tolerance, slope_variation_threshold, snapshot_interval)
    preliminary_groups = finalize_groups(groups, max_theta_gap=max_theta_gap, flexible_pa_range=flexible_pa_range)
    refined_groups = refine_all_groups(preliminary_groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r)
    good_groups = {}
    good_counter = 0
    for g in refined_groups:
        disp = compute_dispersion(g)
        if disp <= dispersion_threshold and g['slope'] is not None and g['intercept'] is not None and \
           g['pa'] is not None and flexible_pa_range[0] <= abs(g['pa']) <= flexible_pa_range[1]:
            good_groups[good_counter] = {'group': g, 'dispersion': disp}
            good_counter += 1
    updated_good_groups = extrapolate_good_groups(good_groups, refined_groups, slope_variation_merge=0.208, theta_relative_margin=0.38, r_relative_margin=0.38)
    return states, refined_groups, updated_good_groups, df_filtered

# =============================================================================
# Bloque Principal de Ejecución
# =============================================================================

if __name__ == '__main__':
    id_halo = "17"
    quartile_threshold = 0.55
    theta_min = 20.0
    theta_max = 290.0
    theta_diff = 4.0
    r_diff = 1.0
    gap_threshold_theta = 1.5
    gap_threshold_r = 1.5
    min_seed_points = 50
    theta_fit_window = 20.0
    r_fit_window = 2.0
    radius_tolerance = 0.345
    slope_variation_threshold = 0.28
    snapshot_interval = 10
    max_theta_gap = 8.0
    flexible_pa_range = (4.0, 40.0)
    dispersion_threshold = 0.35

    states, refined_groups, good_groups, df_filtered = process_spiral_arms_integrated_improved(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval,
        max_theta_gap=max_theta_gap,
        flexible_pa_range=flexible_pa_range,
        dispersion_threshold=dispersion_threshold
    )
    
    print(f"Total de grupos refinados: {len(refined_groups)}")
    print(f"Total de Good Groups extrapolados: {len(good_groups)}")
    
    good_groups_list = [entry['group'] for entry in good_groups.values()]
    if good_groups_list:
        print("Mostrando Good Groups extrapolados:")
        plot_final_groups(good_groups_list, df_filtered)
    else:
        print("No se encontraron Good Groups que cumplan con los criterios de buena dispersión.")


In [0]:
import os
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression
from collections import deque
import copy
from ipywidgets import interact, IntSlider

warnings.filterwarnings("ignore", category=DeprecationWarning)

# =============================================================================
# 1. Carga y Preprocesamiento de Datos
# =============================================================================

def load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold):
    """
    Carga el archivo CSV 'data_rho_{id_halo}_filtered.csv' desde el directorio actual,
    calcula las coordenadas polares y filtra los puntos según el percentil de 'rho_resta_final_exp'.
    """
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    return df_filtered

# =============================================================================
# 2. Obtención de Semillas con DBSCAN y Fusión de Clusters Cercanos
# =============================================================================

def build_graph_rectangular(df_points, theta_diff, r_diff):
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold, mode='theta'):
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def detect_seeds(id_halo, quartile_threshold=0.55, theta_min=50.0, theta_max=250.0, 
                 theta_diff=5.0, r_diff=1.0, gap_threshold_theta=2.5, gap_threshold_r=2.5, min_seed_points=60):
    df_filtered = load_and_filter_data(id_halo, theta_min, theta_max, quartile_threshold)
    graph = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    seeds_all = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            for group in subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r'):
                seeds_all.append(group)
    seed_clusters = [seed for seed in seeds_all if len(seed) >= min_seed_points]
    print(f"Total de semillas robustas (>= {min_seed_points} puntos): {len(seed_clusters)}")
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    return seed_clusters, df_filtered

# =============================================================================
# 3. Funciones Auxiliares para Ajuste y Validación
# =============================================================================

def calculate_distance(slope, intercept, theta, r):
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.degrees(np.arctan(slope))

def is_valid_pa(pa, valid_range):
    return valid_range[0] <= abs(pa) <= valid_range[1]

# =============================================================================
# 4. Asignación Iterativa y Refinamiento de Grupos
# =============================================================================

def iterative_assignment(seed_clusters, df_filtered, theta_fit_window=30.0, r_fit_window=10.0, 
                          radius_tolerance=0.45, slope_variation_threshold=0.20, snapshot_interval=10):
    model = LinearRegression()
    groups = []
    for idx, seed in enumerate(seed_clusters):
        if len(seed) >= 2:
            X_seed = seed[['theta']].values
            y_seed = seed['r'].values
            model.fit(X_seed, y_seed)
            groups.append({
                'group_id': idx,
                'points': seed.copy(),
                'slope': model.coef_[0],
                'intercept': model.intercept_,
                'pa': calculate_pa(model.coef_[0], model.intercept_)
            })
    states = []  # Aquí no se generan estados intermedios
    return states, groups, df_filtered

def validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r):
    subgroups_theta = subdivide_by_gap(group['points'], gap_threshold_theta, mode='theta')
    refined_groups = []
    for sg in subgroups_theta:
        for sgr in subdivide_by_gap(sg, gap_threshold_r, mode='r'):
            if len(sgr) >= 2:
                model = LinearRegression()
                model.fit(sgr[['theta']], sgr['r'])
                slope = model.coef_[0]
                intercept = model.intercept_
                pa = calculate_pa(slope, intercept)
            else:
                slope, intercept, pa = None, None, None
            refined_groups.append({
                'group_id': group.get('group_id'),
                'points': sgr,
                'slope': slope,
                'intercept': intercept,
                'pa': pa
            })
    return refined_groups

def compute_dispersion(group):
    if group['slope'] is None or group['points'].empty:
        return 0
    distances = group['points'].apply(lambda row: calculate_distance(group['slope'], group['intercept'], row['theta'], row['r']), axis=1)
    return distances.std()

def refine_group(group, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    valid_groups = validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r)
    refined = []
    for vg in valid_groups:
        if len(vg['points']) < min_points:
            refined.append(vg)
            continue
        disp = compute_dispersion(vg)
        if disp <= dispersion_threshold:
            refined.append(vg)
        else:
            pts = vg['points'].sort_values(by='theta').reset_index(drop=True)
            median_index = len(pts) // 2
            group1_pts = pts.iloc[:median_index].copy()
            group2_pts = pts.iloc[median_index:].copy()
            model = LinearRegression()
            if len(group1_pts) >= 2:
                model.fit(group1_pts[['theta']], group1_pts['r'])
                slope1, intercept1 = model.coef_[0], model.intercept_
                pa1 = calculate_pa(slope1, intercept1)
            else:
                slope1, intercept1, pa1 = None, None, None
            if len(group2_pts) >= 2:
                model.fit(group2_pts[['theta']], group2_pts['r'])
                slope2, intercept2 = model.coef_[0], model.intercept_
                pa2 = calculate_pa(slope2, intercept2)
            else:
                slope2, intercept2, pa2 = None, None, None
            sub_group1 = {'group_id': vg.get('group_id'), 'points': group1_pts, 'slope': slope1, 'intercept': intercept1, 'pa': pa1}
            sub_group2 = {'group_id': vg.get('group_id'), 'points': group2_pts, 'slope': slope2, 'intercept': intercept2, 'pa': pa2}
            refined.extend(refine_group(sub_group1, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
            refined.extend(refine_group(sub_group2, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined

def refine_all_groups(groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    refined_groups = []
    for g in groups:
        refined_groups.extend(refine_group(g, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined_groups

def subdivide_if_gap_final(group, max_theta_gap=15.0):
    pts = group['points'].sort_values(by='theta').reset_index(drop=True)
    if len(pts) < 4:
        return [group]
    theta_vals = pts['theta'].values
    dtheta = np.diff(theta_vals)
    gap_idx = np.where(dtheta > max_theta_gap)[0]
    if len(gap_idx) == 0:
        return [group]
    split_idx = gap_idx[0]
    pts1 = pts.iloc[:split_idx+1]
    pts2 = pts.iloc[split_idx+1:]
    model = LinearRegression()
    if len(pts1) >= 2:
        model.fit(pts1[['theta']], pts1['r'])
        s1, i1 = model.coef_[0], model.intercept_
        pa1 = calculate_pa(s1, i1)
    else:
        s1, i1, pa1 = None, None, None
    if len(pts2) >= 2:
        model.fit(pts2[['theta']], pts2['r'])
        s2, i2 = model.coef_[0], model.intercept_
        pa2 = calculate_pa(s2, i2)
    else:
        s2, i2, pa2 = None, None, None
    seg1 = {'group_id': group.get('group_id'), 'points': pts1, 'slope': s1, 'intercept': i1, 'pa': pa1}
    seg2 = {'group_id': group.get('group_id'), 'points': pts2, 'slope': s2, 'intercept': i2, 'pa': pa2}
    return [seg1, seg2]

def finalize_groups(groups, max_theta_gap=15.0, flexible_pa_range=(4.0, 30.0)):
    final_groups = []
    for g in groups:
        segs = subdivide_if_gap_final(g, max_theta_gap)
        final_groups.extend(segs)
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    return valid_groups

# =============================================================================
# 5. Extrapolación de Good Groups (Revisada y Mejorada)
# =============================================================================

def extrapolate_good_groups(good_groups, refined_groups, slope_variation_merge=0.28, theta_relative_margin=0.18, r_relative_margin=0.18, max_dispersion_increase=1.2):
    """
    Para cada Good Group, se revisan los candidatos en refined_groups (diferentes del grupo)
    y se fusionan si:
      - La diferencia relativa en pendiente es ≤ 28%.
      - El overlap ratio de los intervalos de θ y r es ≥ 0.5.
      - La dispersión del grupo fusionado no aumenta más que un factor max_dispersion_increase
        respecto a la dispersión original del Good Group.
    Se recalculan los parámetros y se actualiza el Good Group.
    """
    model = LinearRegression()
    updated_good_groups = {}
    for key, entry in good_groups.items():
        good_group = entry['group']
        original_disp = compute_dispersion(good_group)
        good_theta_min = good_group['points']['theta'].min()
        good_theta_max = good_group['points']['theta'].max()
        good_r_min = good_group['points']['r'].min()
        good_r_max = good_group['points']['r'].max()
        theta_range = good_theta_max - good_theta_min if good_theta_max - good_theta_min != 0 else 1.0
        r_range = good_r_max - good_r_min if good_r_max - good_r_min != 0 else 1.0
        for candidate in refined_groups:
            if candidate is good_group:
                continue
            if good_group['slope'] is None or candidate['slope'] is None:
                continue
            relative_diff = abs(candidate['slope'] - good_group['slope']) / (abs(good_group['slope']) + 1e-12)
            if relative_diff > slope_variation_merge:
                continue
            cand_theta_min = candidate['points']['theta'].min()
            cand_theta_max = candidate['points']['theta'].max()
            cand_r_min = candidate['points']['r'].min()
            cand_r_max = candidate['points']['r'].max()
            theta_overlap = max(0, min(good_theta_max, cand_theta_max) - max(good_theta_min, cand_theta_min))
            theta_union = max(good_theta_max, cand_theta_max) - min(good_theta_min, cand_theta_min)
            theta_overlap_ratio = theta_overlap / theta_union if theta_union > 0 else 0
            r_overlap = max(0, min(good_r_max, cand_r_max) - max(good_r_min, cand_r_min))
            r_union = max(good_r_max, cand_r_max) - min(good_r_min, cand_r_min)
            r_overlap_ratio = r_overlap / r_union if r_union > 0 else 0
            if theta_overlap_ratio >= 0.5 and r_overlap_ratio >= 0.5:
                merged_points = pd.concat([good_group['points'], candidate['points']], ignore_index=True)
                if len(merged_points) >= 2:
                    model.fit(merged_points[['theta']], merged_points['r'])
                    new_slope = model.coef_[0]
                    new_intercept = model.intercept_
                    new_pa = calculate_pa(new_slope, new_intercept)
                else:
                    new_slope, new_intercept, new_pa = good_group['slope'], good_group['intercept'], good_group['pa']
                # Calcular la dispersión del grupo fusionado
                temp_group = {'points': merged_points, 'slope': new_slope, 'intercept': new_intercept, 'pa': new_pa}
                merged_disp = compute_dispersion(temp_group)
                if merged_disp <= original_disp * max_dispersion_increase:
                    good_group['points'] = merged_points
                    good_group['slope'] = new_slope
                    good_group['intercept'] = new_intercept
                    good_group['pa'] = new_pa
                    good_theta_min = good_group['points']['theta'].min()
                    good_theta_max = good_group['points']['theta'].max()
                    good_r_min = good_group['points']['r'].min()
                    good_r_max = good_group['points']['r'].max()
                    theta_range = good_theta_max - good_theta_min if good_theta_max - good_theta_min != 0 else 1.0
                    r_range = good_r_max - good_r_min if good_r_max - good_r_min != 0 else 1.0
                    original_disp = merged_disp
        updated_good_groups[key] = {'group': good_group, 'dispersion': compute_dispersion(good_group)}
    return updated_good_groups

# =============================================================================
# 6. Funciones de Visualización
# =============================================================================

def plot_intermediate_state_cartesian(state, df_all):
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f'Grupo {idx+1}')
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    if 'temp_points' in state and not state['temp_points'].empty:
        plt.scatter(state['temp_points']['theta'], state['temp_points']['r'], s=10, color='black', marker='x', label='Temp')
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def plot_final_groups(final_groups, df_all):
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan']
    for idx, grp in enumerate(final_groups):
        pts = grp['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Segmento {idx+1}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title("Segmentos Finales (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

# =============================================================================
# 7. Proceso Integrado Mejorado
# =============================================================================

def process_spiral_arms_integrated_improved(
    id_halo,
    quartile_threshold=0.55,
    theta_min=50.0,
    theta_max=250.0,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5,
    min_seed_points=60,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    radius_tolerance=0.45,
    slope_variation_threshold=0.20,
    snapshot_interval=10,
    max_theta_gap=15.0,
    flexible_pa_range=(4.0,30.0),
    dispersion_threshold=0.35
):
    """
    Proceso completo mejorado:
      1. Detecta semillas robustas.
      2. Genera grupos iniciales mediante ajuste lineal.
      3. Realiza una subdivisión final y refina los grupos según dispersión.
      4. Selecciona Good Groups y extrapola su ajuste incorporando candidatos con variación en pendiente ≤28%
         y overlap en θ y r ≥50%, siempre que la fusión no incremente excesivamente la dispersión.
    """
    seed_clusters, df_filtered = detect_seeds(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points
    )
    states, groups, df_filtered = iterative_assignment(seed_clusters, df_filtered, theta_fit_window, r_fit_window, radius_tolerance, slope_variation_threshold, snapshot_interval)
    preliminary_groups = finalize_groups(groups, max_theta_gap=max_theta_gap, flexible_pa_range=flexible_pa_range)
    refined_groups = refine_all_groups(preliminary_groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r)
    good_groups = {}
    good_counter = 0
    for g in refined_groups:
        disp = compute_dispersion(g)
        if disp <= dispersion_threshold and g['slope'] is not None and g['intercept'] is not None and \
           g['pa'] is not None and flexible_pa_range[0] <= abs(g['pa']) <= flexible_pa_range[1]:
            good_groups[good_counter] = {'group': g, 'dispersion': disp}
            good_counter += 1
    updated_good_groups = extrapolate_good_groups(good_groups, refined_groups, slope_variation_merge=0.28, theta_relative_margin=0.18, r_relative_margin=0.18, max_dispersion_increase=1.2)
    return states, refined_groups, updated_good_groups, df_filtered

# =============================================================================
# Bloque Principal de Ejecución
# =============================================================================

if __name__ == '__main__':
    id_halo = "17"
    quartile_threshold = 0.55
    theta_min = 20.0
    theta_max = 270.0
    theta_diff = 4.0
    r_diff = 1.0
    gap_threshold_theta = 1.5
    gap_threshold_r = 1.5
    min_seed_points = 50
    theta_fit_window = 20.0
    r_fit_window = 2.0
    radius_tolerance = 0.345
    slope_variation_threshold = 0.28
    snapshot_interval = 10
    max_theta_gap = 8.0
    flexible_pa_range = (4.0, 40.0)
    dispersion_threshold = 0.35

    states, refined_groups, good_groups, df_filtered = process_spiral_arms_integrated_improved(
        id_halo=id_halo,
        quartile_threshold=quartile_threshold,
        theta_min=theta_min,
        theta_max=theta_max,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r,
        min_seed_points=min_seed_points,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval,
        max_theta_gap=max_theta_gap,
        flexible_pa_range=flexible_pa_range,
        dispersion_threshold=dispersion_threshold
    )
    
    print(f"Total de grupos refinados: {len(refined_groups)}")
    print(f"Total de Good Groups extrapolados: {len(good_groups)}")
    
    good_groups_list = [entry['group'] for entry in good_groups.values()]
    if good_groups_list:
        print("Mostrando Good Groups extrapolados:")
        plot_final_groups(good_groups_list, df_filtered)
    else:
        print("No se encontraron Good Groups que cumplan con los criterios de buena dispersión.")


In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression
from collections import deque
import copy
from ipywidgets import interact, IntSlider

warnings.filterwarnings("ignore", category=DeprecationWarning)

# =============================================================================
# 1. Carga y Preprocesamiento de Datos
# =============================================================================

def load_data(id_halo, quartile_threshold):
    """
    Lee el archivo CSV desde '../DATA/processed/TracingPoints/' y calcula las
    coordenadas polares, filtrando los puntos según el percentil de 'rho_resta_final_exp'.
    """
    df = pd.read_csv(f'data_rho_{id_halo}_filtered.csv')
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    df_filtered.sort_values(by='rho_resta_final_exp', ascending=False, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    return df_filtered

# =============================================================================
# 2. Obtención de Semillas con DBSCAN y Fusión de Clusters Cercanos
# =============================================================================

def merge_close_clusters(clusters, clusters_tolerance):
    """
    Fusiona clusters cuyos puntos estén lo suficientemente cerca según clusters_tolerance.
    """
    merged_clusters = {}
    used_keys = set()
    for key1, points1 in clusters.items():
        if key1 in used_keys:
            continue
        merged_points = points1.copy()
        close_keys = [key1]
        for key2, points2 in clusters.items():
            if key2 == key1 or key2 in used_keys:
                continue
            d = np.sqrt((points1['r'].values[:, None] - points2['r'].values)**2 +
                        (points1['theta'].values[:, None] - points2['theta'].values)**2)
            if np.min(d) < clusters_tolerance:
                merged_points = pd.concat([merged_points, points2], ignore_index=True)
                used_keys.add(key2)
                close_keys.append(key2)
        merged_clusters[tuple(close_keys)] = merged_points
        used_keys.add(key1)
    return merged_clusters

def dbscan_seeds(df, eps=0.198, min_samples=18, min_seed_points=60):
    """
    Aplica DBSCAN en el espacio cartesiano, transforma los clusters a polar y
    devuelve las semillas (clusters) que tengan al menos min_seed_points.
    """
    X = df[['x', 'y']].values
    db = DBSCAN(eps=eps, min_samples=min_samples)
    df['cluster'] = db.fit_predict(X)
    
    clusters_polar = {}
    for cluster_id in np.unique(df['cluster']):
        if cluster_id == -1:
            continue
        cluster_points = df[df['cluster'] == cluster_id]
        clusters_polar[cluster_id] = cluster_points[['r', 'theta']]
    
    clusters_tolerance = 0.70
    clusters_polar = merge_close_clusters(clusters_polar, clusters_tolerance)
    
    seed_clusters = [cluster for cluster in clusters_polar.values() if len(cluster) >= min_seed_points]
    print(f"Total de semillas robustas (>= {min_seed_points} puntos): {len(seed_clusters)}")
    df['in_seed'] = False
    seed_indices = set()
    for seed in seed_clusters:
        seed_indices.update(seed.index.tolist())
    df.loc[list(seed_indices), 'in_seed'] = True
    return seed_clusters

# =============================================================================
# 3. Funciones Auxiliares para Ajuste y Validación
# =============================================================================

def calculate_distance(slope, intercept, theta, r):
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.degrees(np.arctan(slope))

def is_valid_pa(pa, valid_range):
    return valid_range[0] <= abs(pa) <= valid_range[1]

# =============================================================================
# 4. Asignación Iterativa y Refinamiento de Grupos
# =============================================================================

def iterative_assignment(seed_clusters, theta_fit_window=30.0, r_fit_window=10.0, 
                          radius_tolerance=0.45, slope_variation_threshold=0.20, snapshot_interval=10):
    """
    Genera grupos iniciales a partir de las semillas aplicando una regresión lineal.
    En esta versión se asume que la mayor parte de los puntos relevantes ya se encuentran en las semillas.
    """
    model = LinearRegression()
    groups = []
    for idx, seed in enumerate(seed_clusters):
        if len(seed) >= 2:
            X = seed[['theta']].values
            y = seed['r'].values
            model.fit(X, y)
            groups.append({
                'group_id': idx,
                'points': seed.copy(),
                'slope': model.coef_[0],
                'intercept': model.intercept_,
                'pa': calculate_pa(model.coef_[0], model.intercept_)
            })
    return groups

def subdivide_by_gap(df_cluster, gap_threshold, mode='theta'):
    order_col = 'theta' if mode=='theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r):
    subgroups_theta = subdivide_by_gap(group['points'], gap_threshold_theta, mode='theta')
    refined_groups = []
    for sg in subgroups_theta:
        subgroups_r = subdivide_by_gap(sg, gap_threshold_r, mode='r')
        for sgr in subgroups_r:
            if len(sgr) >= 2:
                model = LinearRegression()
                model.fit(sgr[['theta']], sgr['r'])
                slope = model.coef_[0]
                intercept = model.intercept_
                pa = calculate_pa(slope, intercept)
            else:
                slope, intercept, pa = None, None, None
            refined_groups.append({
                'group_id': group.get('group_id'),
                'points': sgr,
                'slope': slope,
                'intercept': intercept,
                'pa': pa
            })
    return refined_groups

def compute_dispersion(group):
    if group['slope'] is None or group['points'].empty:
        return 0
    distances = group['points'].apply(lambda row: calculate_distance(group['slope'], group['intercept'], row['theta'], row['r']), axis=1)
    return distances.std()

def refine_group(group, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    valid_groups = validate_group_no_gaps(group, gap_threshold_theta, gap_threshold_r)
    refined = []
    for vg in valid_groups:
        if len(vg['points']) < min_points:
            refined.append(vg)
            continue
        disp = compute_dispersion(vg)
        if disp <= dispersion_threshold:
            refined.append(vg)
        else:
            pts = vg['points'].sort_values(by='theta').reset_index(drop=True)
            median_index = len(pts) // 2
            group1_pts = pts.iloc[:median_index].copy()
            group2_pts = pts.iloc[median_index:].copy()
            model = LinearRegression()
            if len(group1_pts) >= 2:
                model.fit(group1_pts[['theta']], group1_pts['r'])
                slope1, intercept1 = model.coef_[0], model.intercept_
                pa1 = calculate_pa(slope1, intercept1)
            else:
                slope1, intercept1, pa1 = None, None, None
            if len(group2_pts) >= 2:
                model.fit(group2_pts[['theta']], group2_pts['r'])
                slope2, intercept2 = model.coef_[0], model.intercept_
                pa2 = calculate_pa(slope2, intercept2)
            else:
                slope2, intercept2, pa2 = None, None, None
            sub_group1 = {'group_id': vg.get('group_id'), 'points': group1_pts, 'slope': slope1, 'intercept': intercept1, 'pa': pa1}
            sub_group2 = {'group_id': vg.get('group_id'), 'points': group2_pts, 'slope': slope2, 'intercept': intercept2, 'pa': pa2}
            refined.extend(refine_group(sub_group1, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
            refined.extend(refine_group(sub_group2, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined

def refine_all_groups(groups, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points=5):
    refined_groups = []
    for g in groups:
        refined_groups.extend(refine_group(g, dispersion_threshold, gap_threshold_theta, gap_threshold_r, min_points))
    return refined_groups

def subdivide_if_gap_final(group, max_theta_gap=15.0):
    pts = group['points'].sort_values(by='theta').reset_index(drop=True)
    if len(pts) < 4:
        return [group]
    theta_vals = pts['theta'].values
    dtheta = np.diff(theta_vals)
    gap_idx = np.where(dtheta > max_theta_gap)[0]
    if len(gap_idx) == 0:
        return [group]
    split_idx = gap_idx[0]
    pts1 = pts.iloc[:split_idx+1]
    pts2 = pts.iloc[split_idx+1:]
    model = LinearRegression()
    if len(pts1) >= 2:
        model.fit(pts1[['theta']], pts1['r'])
        s1, i1 = model.coef_[0], model.intercept_
        pa1 = calculate_pa(s1, i1)
    else:
        s1, i1, pa1 = None, None, None
    if len(pts2) >= 2:
        model.fit(pts2[['theta']], pts2['r'])
        s2, i2 = model.coef_[0], model.intercept_
        pa2 = calculate_pa(s2, i2)
    else:
        s2, i2, pa2 = None, None, None
    seg1 = {'group_id': group.get('group_id'), 'points': pts1, 'slope': s1, 'intercept': i1, 'pa': pa1}
    seg2 = {'group_id': group.get('group_id'), 'points': pts2, 'slope': s2, 'intercept': i2, 'pa': pa2}
    return [seg1, seg2]

def finalize_groups(groups, max_theta_gap=15.0, flexible_pa_range=(4.0, 30.0)):
    final_groups = []
    for g in groups:
        segs = subdivide_if_gap_final(g, max_theta_gap)
        final_groups.extend(segs)
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    return valid_groups

# =============================================================================
# 5. Extrapolación de Good Groups
# =============================================================================

def extrapolate_good_groups(good_groups, refined_groups, slope_variation_merge=0.28, theta_relative_margin=0.18, r_relative_margin=0.18):
    """
    Para cada Good Group, revisa candidatos en refined_groups (diferentes del grupo)
    y, si la diferencia relativa en pendiente es ≤28% y los rangos de θ y r se solapan dentro
    del 18% de su rango, fusiona los puntos y recalcula el ajuste.
    """
    model = LinearRegression()
    updated_good_groups = {}
    for key, entry in good_groups.items():
        good_group = entry['group']
        good_theta_min = good_group['points']['theta'].min()
        good_theta_max = good_group['points']['theta'].max()
        theta_range = good_theta_max - good_theta_min if good_theta_max - good_theta_min != 0 else 1.0
        
        good_r_min = good_group['points']['r'].min()
        good_r_max = good_group['points']['r'].max()
        r_range = good_r_max - good_r_min if good_r_max - good_r_min != 0 else 1.0
        
        for candidate in refined_groups:
            if candidate is good_group:
                continue
            if good_group['slope'] is None or candidate['slope'] is None:
                continue
            relative_diff = abs(candidate['slope'] - good_group['slope']) / (abs(good_group['slope']) + 1e-12)
            if relative_diff <= slope_variation_merge:
                cand_theta_min = candidate['points']['theta'].min()
                cand_theta_max = candidate['points']['theta'].max()
                cand_r_min = candidate['points']['r'].min()
                cand_r_max = candidate['points']['r'].max()
                theta_condition = (cand_theta_min >= good_theta_min - theta_relative_margin * theta_range and
                                   cand_theta_max <= good_theta_max + theta_relative_margin * theta_range)
                r_condition = (cand_r_min >= good_r_min - r_relative_margin * r_range and
                               cand_r_max <= good_r_max + r_relative_margin * r_range)
                if theta_condition and r_condition:
                    merged_points = pd.concat([good_group['points'], candidate['points']], ignore_index=True)
                    if len(merged_points) >= 2:
                        model.fit(merged_points[['theta']], merged_points['r'])
                        new_slope = model.coef_[0]
                        new_intercept = model.intercept_
                        new_pa = calculate_pa(new_slope, new_intercept)
                    else:
                        new_slope, new_intercept, new_pa = good_group['slope'], good_group['intercept'], good_group['pa']
                    good_group['points'] = merged_points
                    good_group['slope'] = new_slope
                    good_group['intercept'] = new_intercept
                    good_group['pa'] = new_pa
                    good_theta_min = good_group['points']['theta'].min()
                    good_theta_max = good_group['points']['theta'].max()
                    theta_range = good_theta_max - good_theta_min if good_theta_max - good_theta_min != 0 else 1.0
                    good_r_min = good_group['points']['r'].min()
                    good_r_max = good_group['points']['r'].max()
                    r_range = good_r_max - good_r_min if good_r_max - good_r_min != 0 else 1.0
        updated_good_groups[key] = {'group': good_group, 'dispersion': compute_dispersion(good_group)}
    return updated_good_groups

# =============================================================================
# 6. Funciones de Visualización
# =============================================================================

def plot_intermediate_state_cartesian(state, df_all):
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f'Grupo {idx+1}')
        if g['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    if 'temp_points' in state and not state['temp_points'].empty:
        plt.scatter(state['temp_points']['theta'], state['temp_points']['r'], s=10, color='black', marker='x', label='Temp')
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    #plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def plot_final_groups(final_groups, df_all):
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos completos')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan']
    for idx, grp in enumerate(final_groups):
        pts = grp['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=color, label=f"Segmento {idx+1}")
        if grp['slope'] is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = grp['slope'] * theta_line + grp['intercept']
            plt.plot(theta_line, r_line, '--', color=color, linewidth=1.5)
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title("Segmentos Finales (Plano r, θ)")
    #plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

# =============================================================================
# 7. Proceso Integrado Mejorado
# =============================================================================

def process_spiral_arms_integrated_improved(
    id_halo,
    data_folder,
    quartile_threshold=0.55,
    eps=0.198,
    min_samples=18,
    min_seed_points=60,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    radius_tolerance=0.45,
    slope_variation_threshold=0.20,
    snapshot_interval=10,
    max_theta_gap=15.0,
    flexible_pa_range=(4.0,30.0),
    dispersion_threshold=0.35
):
    """
    Proceso completo mejorado:
      1. Lee y filtra los datos para el halo.
      2. Obtiene semillas aplicando DBSCAN en el espacio cartesiano y transforma a polar (fusionando clusters cercanos).
      3. Genera grupos iniciales mediante ajuste lineal.
      4. Realiza una subdivisión final de los grupos.
      5. Refina los grupos según su dispersión.
      6. Selecciona los Good Groups (grupos con dispersión aceptable y PA válido).
      7. Extrapola los Good Groups incorporando candidatos cuyo ajuste varíe ≤28%
         y cuyos rangos en θ y r se solapen dentro del 18%.
    """
    # Paso 1: Cargar datos
    df = load_data(id_halo, quartile_threshold)
    # Paso 2: Obtener semillas
    seed_clusters = dbscan_seeds(df, eps=eps, min_samples=min_samples, min_seed_points=min_seed_points)
    # Paso 3: Grupos iniciales a partir de semillas
    groups = iterative_assignment(seed_clusters, theta_fit_window, r_fit_window, radius_tolerance, slope_variation_threshold, snapshot_interval)
    # Paso 4: Subdivisión final de grupos
    prelim_groups = finalize_groups(groups, max_theta_gap=max_theta_gap, flexible_pa_range=flexible_pa_range)
    # Paso 5: Refinar grupos según dispersión
    refined_groups = refine_all_groups(prelim_groups, dispersion_threshold, gap_threshold_theta=2.5, gap_threshold_r=2.5)
    # Paso 6: Seleccionar Good Groups
    good_groups = {}
    good_counter = 0
    for g in refined_groups:
        disp = compute_dispersion(g)
        if disp <= dispersion_threshold and g['slope'] is not None and g['intercept'] is not None and \
           g['pa'] is not None and flexible_pa_range[0] <= abs(g['pa']) <= flexible_pa_range[1]:
            good_groups[good_counter] = {'group': g, 'dispersion': disp}
            good_counter += 1
    # Paso 7: Extrapolar ajustes en los Good Groups
    updated_good_groups = extrapolate_good_groups(good_groups, refined_groups, slope_variation_merge=0.28, theta_relative_margin=0.18, r_relative_margin=0.18)
    
    return df, groups, prelim_groups, refined_groups, updated_good_groups

# =============================================================================
# Bloque Principal de Ejecución
# =============================================================================

if __name__ == '__main__':
    # Parámetros (ajusta según sea necesario)
    id_halo = "17"
    data_folder = "../DATA/processed/TracingPoints"
    quartile_threshold = 0.55
    eps = 0.198
    min_samples = 18
    min_seed_points = 60
    theta_fit_window = 30.0
    r_fit_window = 3.0
    radius_tolerance = 0.45
    slope_variation_threshold = 0.20
    snapshot_interval = 10
    max_theta_gap = 15.0
    flexible_pa_range = (4.0, 30.0)
    dispersion_threshold = 0.35

    df, groups, prelim_groups, refined_groups, good_groups = process_spiral_arms_integrated_improved(
        id_halo=id_halo,
        data_folder=data_folder,
        quartile_threshold=quartile_threshold,
        eps=eps,
        min_samples=min_samples,
        min_seed_points=min_seed_points,
        theta_fit_window=theta_fit_window,
        r_fit_window=r_fit_window,
        radius_tolerance=radius_tolerance,
        slope_variation_threshold=slope_variation_threshold,
        snapshot_interval=snapshot_interval,
        max_theta_gap=max_theta_gap,
        flexible_pa_range=flexible_pa_range,
        dispersion_threshold=dispersion_threshold
    )
    
    print(f"Total de grupos iniciales: {len(groups)}")
    print(f"Total de grupos preliminares: {len(prelim_groups)}")
    print(f"Total de grupos refinados: {len(refined_groups)}")
    print(f"Total de Good Groups extrapolados: {len(good_groups)}")
    
    good_groups_list = [entry['group'] for entry in good_groups.values()]
    if good_groups_list:
        print("Mostrando Good Groups extrapolados:")
        plot_final_groups(good_groups_list, df)
    else:
        print("No se encontraron Good Groups que cumplan con los criterios.")


### último intento

In [0]:
import copy
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

warnings.filterwarnings('ignore')

# =========================================================
# ============ FUNCIONES AUXILIARES DEL CÓDIGO 2 ==========
# =========================================================

def load_and_filter_data(id_halo, theta_min=50, theta_max=250):
    """
    Carga los datos desde CSV, calcula coordenadas polares y filtra
    según el rango angular [theta_min, theta_max].
    Ajusta la columna 'theta' para que esté en [0, 360).
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    return df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo donde cada punto se conecta a otros
    si las diferencias en θ y r son <= a los umbrales dados.
    Retorna: (graph, n) donde graph[i] es la lista de vecinos de i.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Vecinos cumplen (dtheta <= theta_diff) & (dr <= r_diff), excluyendo i mismo
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    """
    Extrae componentes conexas del grafo usando BFS.
    Cada componente es un DataFrame con los puntos de ese subgrupo.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Subdivide el DataFrame (ordenado por 'theta' o 'r') en subclusters
    si hay saltos (gaps) mayores a gap_threshold.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    
    # Si no hay gap mayor al umbral, devolvemos un solo subcluster
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i + 1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    df_base,
    quartile_threshold=0.55,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    """
    Toma un DF (ya filtrado en θ si es necesario), filtra por quantile
    en 'rho_resta_final_exp', luego construye grafo y extrae
    componentes BFS. Finalmente, subdivide cada componente
    por gaps en θ y r.
    
    Devuelve una lista de DataFrames (clusters finales) y el DataFrame filtrado.
    """
    threshold = df_base['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df_base[df_base['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        # Subdividir por gaps en θ
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            # Subdividir por gaps en r
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters, df_filtered


# =========================================================
# ============ FUNCIONES AUXILIARES DEL CÓDIGO 1 ==========
# =========================================================

def calculate_distance(slope, intercept, theta, r):
    """
    Calcula la distancia vertical de un punto (theta, r)
    a la recta r_pred = slope * theta + intercept.
    """
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    """
    Calcula el ángulo PA en grados dado slope e intercept.
    Si intercept es 0, se retorna NaN.
    Ajustar la fórmula según tu convención.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan

def add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window):
    """
    Agrega un punto a un temp_group si encaja (según min/max theta/r).
    De lo contrario, crea un nuevo temp_group.
    """
    assigned = False
    for tg in temp_groups:
        tg_theta_min = tg['points']['theta'].min()
        tg_theta_max = tg['points']['theta'].max()
        tg_r_min = tg['points']['r'].min()
        tg_r_max = tg['points']['r'].max()
        if ((point['theta'] >= tg_theta_min - theta_fit_window/2) and 
            (point['theta'] <= tg_theta_max + theta_fit_window/2)) and \
           ((point['r'] >= tg_r_min - r_fit_window/2) and 
            (point['r'] <= tg_r_max + r_fit_window/2)):
            tg['points'] = pd.concat([tg['points'], point.to_frame().T], ignore_index=True)
            assigned = True
            break
    if not assigned:
        new_tg = {'points': point.to_frame().T.copy()}
        temp_groups.append(new_tg)

def review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range):
    """
    Revisa los temp_groups. 
    - Si tienen <3 puntos, no se ajustan todavía.
    - Si >=3, se hace ajuste lineal y se trata de fusionar/promover.
    - Se validan distancias y variación de pendiente con los grupos principales.
    """
    new_temp_groups = []
    for tg in temp_groups:
        # Si muy pocos puntos, los dejamos tal cual
        if len(tg['points']) < 3:
            new_temp_groups.append(tg)
            continue
        
        # Ajuste lineal del temp_group
        X_tg = tg['points'][['theta']]
        y_tg = tg['points']['r']
        model.fit(X_tg, y_tg)
        tg_slope = model.coef_[0]
        tg_intercept = model.intercept_
        
        # Promediamos la theta para ver dónde "cae"
        tg_mean = tg['points']['theta'].mean()
        merged = False
        
        # Intentamos ver si se fusiona con algún grupo principal
        for mg in groups:
            mg_min = mg['points']['theta'].min()
            mg_max = mg['points']['theta'].max()
            if mg_min <= tg_mean <= mg_max:
                # Calculamos distancias con la pendiente del mg
                dist_values = tg['points'].apply(
                    lambda row: calculate_distance(mg['slope'], mg['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if dist_values.mean() < radius_tolerance:
                    # Fusionar
                    mg['points'] = pd.concat([mg['points'], tg['points']], ignore_index=True)
                    model.fit(mg['points'][['theta']], mg['points']['r'])
                    new_slope = model.coef_[0]
                    old_slope = mg['slope']
                    # Ver si la variación de pendiente es aceptable
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) < slope_variation_threshold:
                        mg['slope'] = new_slope
                        mg['intercept'] = model.intercept_
                        mg['pa'] = calculate_pa(mg['slope'], mg['intercept'])
                    merged = True
                    break
        if not merged:
            # No se fusionó -> crear un nuevo grupo principal
            new_group = {
                'cluster_id': None,
                'points': tg['points'],
                'slope': tg_slope,
                'intercept': tg_intercept,
                'pa': calculate_pa(tg_slope, tg_intercept)
            }
            groups.append(new_group)
    
    # Limpiar temp_groups y reinsertar los que quedaban
    temp_groups.clear()
    for tg in new_temp_groups:
        temp_groups.append(tg)

def subdivide_if_gap(group, max_gap=15.0):
    """
    Toma un grupo (dict con 'points') y, si hay un gap grande en θ, 
    lo divide en dos subgrupos. Retorna una lista (1 o 2 subgrupos).
    """
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        model = LinearRegression()
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
    if len(pts) < 4:
        # No dividimos si hay muy pocos puntos
        return [group]
    
    theta_vals = pts['theta'].values
    dtheta = np.diff(theta_vals)
    gap_idx = np.where(dtheta > max_gap)[0]
    if len(gap_idx) == 0:
        return [group]
    
    # Tomamos el primer gap
    split_idx = gap_idx[0]
    pts1 = pts.iloc[:split_idx+1]
    pts2 = pts.iloc[split_idx+1:]
    
    s1, i1, pa1 = fit_line(pts1)
    s2, i2, pa2 = fit_line(pts2)
    
    seg1 = {
        'cluster_id': group['cluster_id'],
        'points': pts1,
        'slope': s1,
        'intercept': i1,
        'pa': pa1
    }
    seg2 = {
        'cluster_id': group['cluster_id'],
        'points': pts2,
        'slope': s2,
        'intercept': i2,
        'pa': pa2
    }
    return [seg1, seg2]


# =========================================================
# ============ FUNCIÓN PRINCIPAL UNIFICADA ===============
# =========================================================

def process_spiral_arms_unificado(
    id_halo,
    # --- Parámetros BFS / seeds ---
    quartile_threshold=0.55,
    theta_min=50, theta_max=250,
    theta_diff=5.0,          # vecindad horizontal BFS
    r_diff=1.0,              # vecindad radial BFS
    gap_threshold_theta=2.5, # gap en θ
    gap_threshold_r=2.5,     # gap en r
    min_seed_size=60,        # tamaño mínimo de semilla

    # --- Parámetros del proceso iterativo (código 1) ---
    radius_tolerance=0.45,    
    slope_variation_threshold=0.20,
    flexible_pa_range=(4.0, 30.0),
    max_theta_gap=15.0,      # para subdividir
    snapshot_interval=10,    # intervalo para guardar states
    review_interval=50,      # cada cuántas iteraciones revisamos temp_groups
    theta_fit_window=30.0,
    r_fit_window=10.0,
    fusion_theta_threshold=7.0,
    fusion_r_threshold=7.0
):
    """
    Fusión de técnicas:
    1) Se obtienen "semillas" via BFS y subdivisión por gaps (código 2).
    2) Se inicializan grupos con slope/intercept calculados.
    3) Se itera sobre el resto de puntos al estilo del código 1, 
       validando distancias, pendiente, etc.
    4) Se revisan temp_groups y se promueven/fusionan cuando aplicable.
    5) Al final se subdividen los grupos si hay gaps grandes en θ.
    6) Se filtran los grupos cuya pa no cae en [flexible_pa_range].

    Retorna:
      - states: lista de diccionarios con estados intermedios
      - final_groups: lista de grupos resultantes
      - df_filtered: DataFrame filtrado por quartile
    """
    
    # 1. Cargar datos base y filtrar por theta
    df_base = load_and_filter_data(id_halo, theta_min, theta_max)
    
    # 2. Semillas mediante BFS + gaps
    seeds_all, df_filtered = process_clusters_rectangular_gaps(
        df_base,
        quartile_threshold=quartile_threshold,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    
    # 3. Quedarnos con seeds grandes
    seed_clusters = [s for s in seeds_all if len(s) >= min_seed_size]
    
    # 4. Marcar en df_filtered cuáles puntos son semilla
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed_df in seed_clusters:
        seed_indices.update(seed_df.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 5. Inicializar grupos con slope/intercept a partir de las semillas
    model = LinearRegression()
    groups = []
    for idx, seed_pts in enumerate(seed_clusters):
        if len(seed_pts) >= 2:
            X_ = seed_pts[['theta']]
            y_ = seed_pts['r']
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        
        groups.append({
            'cluster_id': idx,
            'points': seed_pts.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # 6. Puntos restantes a df_local
    df_local = df_filtered[~df_filtered['in_seed']].copy()
    df_local.sort_values(by='theta', inplace=True)
    df_local.reset_index(drop=True, inplace=True)
    
    # 7. Iteración al estilo "código 1"
    temp_groups = []
    states = []
    iteration = 0
    
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        
        best_group = None
        best_distance = None
        
        # Intentar asignar a un grupo existente
        for g in groups:
            if g['slope'] is None:
                # Si no tiene slope no podemos verificar
                continue
            
            g_theta_min = g['points']['theta'].min()
            g_theta_max = g['points']['theta'].max()
            g_r_min = g['points']['r'].min()
            g_r_max = g['points']['r'].max()
            
            # Chequeo rápido: ¿está el punto dentro de [theta_min,theta_max] y [r_min,r_max] con algo de margen?
            # (Opcional, a tu gusto: si no se cumple, skip)
            if not (g_theta_min - theta_fit_window <= theta_pt <= g_theta_max + theta_fit_window):
                continue
            if not (g_r_min - r_fit_window <= r_pt <= g_r_max + r_fit_window):
                continue
            
            # Tomar subset local alrededor del punto
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = g['points'][(g['points']['theta'] >= window_min) & (g['points']['theta'] <= window_max)]
            subset = pd.concat([subset, point.to_frame().T], ignore_index=True)
            
            if len(subset) < 2:
                continue
            
            # Ajustar recta al subset
            model.fit(subset[['theta']], subset['r'])
            new_slope = model.coef_[0]
            new_intercept = model.intercept_
            old_slope = g['slope']
            
            # Verificamos variación de pendiente
            if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) > slope_variation_threshold:
                continue
            
            # Chequeamos la distancia del punto a esta nueva recta
            dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
            if dist > radius_tolerance:
                continue
            
            # Si pasa todo, es candidato
            if (best_distance is None) or (dist < best_distance):
                best_distance = dist
                best_group = g
        
        if best_group is not None:
            # Asignamos el punto al best_group
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            
            # Reajustamos la pendiente del grupo (con subset local o todo el grupo, a tu criterio)
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset_bg = best_group['points'][
                (best_group['points']['theta'] >= window_min) & 
                (best_group['points']['theta'] <= window_max)
            ]
            if len(subset_bg) < 2:
                # Si quedamos con un subset muy chico, usamos todos los puntos
                subset_bg = best_group['points']
            
            model.fit(subset_bg[['theta']], subset_bg['r'])
            best_group['slope'] = model.coef_[0]
            best_group['intercept'] = model.intercept_
            best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
            
            # Fusión fija con grupos cercanos (opcional, usando la lógica de "código 1")
            merged_flag = True
            while merged_flag:
                merged_flag = False
                for other_group in groups:
                    if other_group is best_group:
                        continue
                    if other_group['slope'] is None:
                        continue
                    best_theta_min = best_group['points']['theta'].min()
                    best_theta_max = best_group['points']['theta'].max()
                    best_r_min = best_group['points']['r'].min()
                    best_r_max = best_group['points']['r'].max()

                    other_theta_min = other_group['points']['theta'].min()
                    other_theta_max = other_group['points']['theta'].max()
                    other_r_min = other_group['points']['r'].min()
                    other_r_max = other_group['points']['r'].max()

                    # Criterio de fusión: si los límites en θ y r están lo bastante cerca
                    if (abs(best_theta_min - other_theta_max) < fusion_theta_threshold and
                        abs(best_theta_max - other_theta_min) < fusion_theta_threshold and
                        abs(best_r_min - other_r_max) < fusion_r_threshold and
                        abs(best_r_max - other_r_min) < fusion_r_threshold):
                        
                        # Fusionar
                        merged_points = pd.concat([best_group['points'], other_group['points']], ignore_index=True)
                        model.fit(merged_points[['theta']], merged_points['r'])
                        best_group['points'] = merged_points
                        best_group['slope'] = model.coef_[0]
                        best_group['intercept'] = model.intercept_
                        best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
                        for idx, grp in enumerate(groups):
                            if grp is other_group:
                                groups.pop(idx)
                                break

                        merged_flag = True
                        break
            
        else:
            # No se pudo asignar -> va a temp_groups
            add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window)
        
        iteration += 1
        
        # Revisar temp_groups cada 'review_interval' iteraciones
        if iteration % review_interval == 0:
            review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range)
        
        # Guardar snapshot
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_groups': copy.deepcopy(temp_groups),
                'remaining': df_local.copy()
            })
    
    # Revisar temp_groups final
    if len(temp_groups) > 0:
        review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range)
    
    # Subdividir si hay gaps
    final_groups = []
    for g in groups:
        segs = subdivide_if_gap(g, max_gap=max_theta_gap)
        final_groups.extend(segs)
    
    # Filtrar grupos con pa fuera de flexible_pa_range
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    
    # Guardar estado final
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'temp_groups': copy.deepcopy(temp_groups),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered


# =========================================================
# ============ OPCIONAL: FUNCIÓN DE PLOTEO =================
# =========================================================

def plot_intermediate_state_unificado(state, df_all):
    """
    Grafica un estado en el plano (theta, r).
    df_all son todos los puntos usados (por ejemplo df_filtered).
    """
    plt.figure(figsize=(10, 6))
    # Puntos de fondo
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos totales')
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, label=f"Grupo {idx+1}")
        if g.get('slope') is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', linewidth=1.5)
    
    # Temp groups (opcional)
    if 'temp_groups' in state and state['temp_groups']:
        for idx, tg in enumerate(state['temp_groups']):
            pts = tg['points']
            plt.scatter(pts['theta'], pts['r'], s=10, color='black', marker='x', label=f"Temp {idx+1}")
    
    plt.xlabel("theta (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']}")
    #plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()


# =========================================================
# ============ EJEMPLO DE USO =============================
# =========================================================
if __name__ == "__main__":
    """
    Ejemplo rápido de cómo llamar a la función unificada.
    Ajusta parámetros y rutas a tus necesidades.
    """

    states, final_groups, df_filtered = process_spiral_arms_unificado(
        id_halo="17",
        quartile_threshold=0.155,
        theta_min=50,
        theta_max=250,
        theta_diff=2.0,
        r_diff=0.468,
        gap_threshold_theta=1.89,
        gap_threshold_r=0.523,
        min_seed_size=50,
        radius_tolerance=0.45,
        slope_variation_threshold=0.20,
        flexible_pa_range=(4.0, 30.0),
        max_theta_gap=15.0,
        snapshot_interval=10,
        review_interval=30,
        theta_fit_window=30.0,
        r_fit_window=10.0,
        fusion_theta_threshold=7.0,
        fusion_r_threshold=7.0
    )

    print(f"Total de estados guardados: {len(states)}")
    print(f"Total de grupos finales: {len(final_groups)}")
    
    # Visualizar un estado específico
    # (Requiere Jupyter o similar para interact(...))
    # from ipywidgets import interact, IntSlider
    # def view_state(i=0):
    #     plot_intermediate_state_unificado(states[i], df_filtered)
    # interact(view_state, i=IntSlider(min=0, max=len(states)-1, step=1, value=0))

    
def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state_cartesian(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0));


In [0]:
import copy
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

warnings.filterwarnings('ignore')

# =========================================================
# ============ FUNCIONES AUXILIARES DEL CÓDIGO 2 ==========
# =========================================================

def load_and_filter_data(id_halo, theta_min=50, theta_max=250):
    """
    Carga los datos desde CSV, calcula coordenadas polares y filtra
    según el rango angular [theta_min, theta_max].
    Ajusta la columna 'theta' para que esté en [0, 360).
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    return df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo donde cada punto se conecta a otros
    si las diferencias en θ y r son <= a los umbrales dados.
    Retorna: (graph, n) donde graph[i] es la lista de vecinos de i.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Vecinos cumplen (dtheta <= theta_diff) & (dr <= r_diff), excluyendo i mismo
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    """
    Extrae componentes conexas del grafo usando BFS.
    Cada componente es un DataFrame con los puntos de ese subgrupo.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Subdivide el DataFrame (ordenado por 'theta' o 'r') en subclusters
    si hay saltos (gaps) mayores a gap_threshold.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    
    # Si no hay gap mayor al umbral, devolvemos un solo subcluster
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i + 1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    df_base,
    quartile_threshold=0.55,
    theta_diff=1.5,       # Más pequeño => semillas iniciales más pequeñas
    r_diff=0.3,           # Más pequeño => semillas iniciales más pequeñas
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    """
    Toma un DF (ya filtrado en θ si es necesario), filtra por quantile
    en 'rho_resta_final_exp', luego construye grafo y extrae
    componentes BFS. Finalmente, subdivide cada componente
    por gaps en θ y r.
    
    Devuelve una lista de DataFrames (clusters finales) y el DataFrame filtrado.
    """
    threshold = df_base['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df_base[df_base['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        # Subdividir por gaps en θ
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            # Subdividir por gaps en r
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters, df_filtered


# =========================================================
# ============ FUNCIONES AUXILIARES DEL CÓDIGO 1 ==========
# =========================================================

def calculate_distance(slope, intercept, theta, r):
    """
    Calcula la distancia vertical de un punto (theta, r)
    a la recta r_pred = slope * theta + intercept.
    """
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    """
    Calcula el ángulo PA en grados dado slope e intercept.
    Si intercept es 0, se retorna NaN.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan

def add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window):
    """
    Agrega un punto a un temp_group si encaja (según min/max theta/r).
    De lo contrario, crea un nuevo temp_group.
    """
    assigned = False
    for tg in temp_groups:
        tg_theta_min = tg['points']['theta'].min()
        tg_theta_max = tg['points']['theta'].max()
        tg_r_min = tg['points']['r'].min()
        tg_r_max = tg['points']['r'].max()
        if ((point['theta'] >= tg_theta_min - theta_fit_window/2) and 
            (point['theta'] <= tg_theta_max + theta_fit_window/2)) and \
           ((point['r'] >= tg_r_min - r_fit_window/2) and 
            (point['r'] <= tg_r_max + r_fit_window/2)):
            tg['points'] = pd.concat([tg['points'], point.to_frame().T], ignore_index=True)
            assigned = True
            break
    if not assigned:
        new_tg = {'points': point.to_frame().T.copy()}
        temp_groups.append(new_tg)

def review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range):
    """
    Revisa los temp_groups. 
    - Si tienen <3 puntos, no se ajustan todavía.
    - Si >=3, se hace ajuste lineal y se trata de fusionar/promover.
    - Se validan distancias y variación de pendiente con los grupos principales.
    """
    new_temp_groups = []
    for tg in temp_groups:
        # Si muy pocos puntos, los dejamos tal cual
        if len(tg['points']) < 3:
            new_temp_groups.append(tg)
            continue
        
        # Ajuste lineal del temp_group
        X_tg = tg['points'][['theta']]
        y_tg = tg['points']['r']
        model.fit(X_tg, y_tg)
        tg_slope = model.coef_[0]
        tg_intercept = model.intercept_
        
        # Promediamos la theta para ver dónde "cae"
        tg_mean = tg['points']['theta'].mean()
        merged = False
        
        # Intentamos ver si se fusiona con algún grupo principal
        for mg in groups:
            mg_min = mg['points']['theta'].min()
            mg_max = mg['points']['theta'].max()
            if mg_min <= tg_mean <= mg_max:
                # Calculamos distancias con la pendiente del mg
                dist_values = tg['points'].apply(
                    lambda row: calculate_distance(mg['slope'], mg['intercept'], row['theta'], row['r']),
                    axis=1
                )
                if dist_values.mean() < radius_tolerance:
                    # Fusionar
                    mg['points'] = pd.concat([mg['points'], tg['points']], ignore_index=True)
                    model.fit(mg['points'][['theta']], mg['points']['r'])
                    new_slope = model.coef_[0]
                    old_slope = mg['slope']
                    # Ver si la variación de pendiente es aceptable
                    if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) < slope_variation_threshold:
                        mg['slope'] = new_slope
                        mg['intercept'] = model.intercept_
                        mg['pa'] = calculate_pa(mg['slope'], mg['intercept'])
                    merged = True
                    break
        if not merged:
            # No se fusionó -> crear un nuevo grupo principal
            new_group = {
                'cluster_id': None,
                'points': tg['points'],
                'slope': tg_slope,
                'intercept': tg_intercept,
                'pa': calculate_pa(tg_slope, tg_intercept)
            }
            groups.append(new_group)
    
    # Limpiar temp_groups y reinsertar los que quedaban
    temp_groups.clear()
    for tg in new_temp_groups:
        temp_groups.append(tg)

def subdivide_if_gap(group, max_gap=15.0):
    """
    Toma un grupo (dict con 'points') y, si hay un gap grande en θ, 
    lo divide en dos subgrupos. Retorna una lista (1 o 2 subgrupos).
    """
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        model = LinearRegression()
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
    if len(pts) < 4:
        # No dividimos si hay muy pocos puntos
        return [group]
    
    theta_vals = pts['theta'].values
    dtheta = np.diff(theta_vals)
    gap_idx = np.where(dtheta > max_gap)[0]
    if len(gap_idx) == 0:
        return [group]
    
    # Tomamos el primer gap
    split_idx = gap_idx[0]
    pts1 = pts.iloc[:split_idx+1]
    pts2 = pts.iloc[split_idx+1:]
    
    s1, i1, pa1 = fit_line(pts1)
    s2, i2, pa2 = fit_line(pts2)
    
    seg1 = {
        'cluster_id': group['cluster_id'],
        'points': pts1,
        'slope': s1,
        'intercept': i1,
        'pa': pa1
    }
    seg2 = {
        'cluster_id': group['cluster_id'],
        'points': pts2,
        'slope': s2,
        'intercept': i2,
        'pa': pa2
    }
    return [seg1, seg2]


# =========================================================
# ============ FUNCIÓN PRINCIPAL UNIFICADA ===============
# =========================================================

def process_spiral_arms_unificado(
    id_halo,
    # --- Parámetros BFS / seeds ---
    quartile_threshold=0.55,
    theta_min=50, theta_max=250,
    theta_diff=1.5,          # Ventanas pequeñas para BFS
    r_diff=0.3,              # Ventanas pequeñas para BFS
    gap_threshold_theta=2.5, # gap en θ
    gap_threshold_r=2.5,     # gap en r
    min_seed_size=60,        # tamaño mínimo de semilla

    # --- Parámetros del proceso iterativo (código 1) ---
    radius_tolerance=0.45,    
    slope_variation_threshold=0.20,   # Umbral normal de variación
    flexible_pa_range=(4.0, 30.0),
    max_theta_gap=15.0,      # para subdividir
    snapshot_interval=10,    # intervalo para guardar states
    review_interval=50,      # cada cuántas iteraciones revisamos temp_groups
    theta_fit_window=30.0,
    r_fit_window=10.0,
    fusion_theta_threshold=7.0,
    fusion_r_threshold=7.0
):
    """
    Fusión de técnicas con modificaciones:
      - Ventanas BFS más pequeñas (theta_diff, r_diff)
      - Extrapolación 20% en θ y r
      - Variación de pendiente hasta 30% en la zona extrapolada

    Retorna:
      - states: lista de diccionarios con estados intermedios
      - final_groups: lista de grupos resultantes
      - df_filtered: DataFrame filtrado por quartile
    """
    
    # 1. Cargar datos base y filtrar por theta
    df_base = load_and_filter_data(id_halo, theta_min, theta_max)
    
    # 2. Semillas mediante BFS + gaps (ventanas más pequeñas)
    seeds_all, df_filtered = process_clusters_rectangular_gaps(
        df_base,
        quartile_threshold=quartile_threshold,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    
    # 3. Quedarnos con seeds grandes
    seed_clusters = [s for s in seeds_all if len(s) >= min_seed_size]
    
    # 4. Marcar en df_filtered cuáles puntos son semilla
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed_df in seed_clusters:
        seed_indices.update(seed_df.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 5. Inicializar grupos con slope/intercept a partir de las semillas
    model = LinearRegression()
    groups = []
    for idx, seed_pts in enumerate(seed_clusters):
        if len(seed_pts) >= 2:
            X_ = seed_pts[['theta']]
            y_ = seed_pts['r']
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        else:
            slope_init, intercept_init, pa_init = None, None, None
        
        groups.append({
            'cluster_id': idx,
            'points': seed_pts.copy(),
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # 6. Puntos restantes a df_local
    df_local = df_filtered[~df_filtered['in_seed']].copy()
    df_local.sort_values(by='theta', inplace=True)
    df_local.reset_index(drop=True, inplace=True)
    
    # 7. Iteración
    temp_groups = []
    states = []
    iteration = 0
    
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        
        best_group = None
        best_distance = None
        
        for g in groups:
            if g['slope'] is None:
                continue
            
            # Hallar rangos en θ, r
            g_theta_min = g['points']['theta'].min()
            g_theta_max = g['points']['theta'].max()
            g_r_min = g['points']['r'].min()
            g_r_max = g['points']['r'].max()
            
            # Calculamos la extensión normal
            # y la región extendida (20%)
            theta_range = g_theta_max - g_theta_min
            r_range = g_r_max - g_r_min
            
            # Zona normal
            normal_theta_min = g_theta_min
            normal_theta_max = g_theta_max
            normal_r_min = g_r_min
            normal_r_max = g_r_max
            
            # Extensión: 20% adicional a cada lado
            ext_theta = 0.2 * theta_range
            ext_r = 0.2 * r_range
            
            extended_theta_min = normal_theta_min - ext_theta
            extended_theta_max = normal_theta_max + ext_theta
            
            extended_r_min = normal_r_min - ext_r
            extended_r_max = normal_r_max + ext_r
            
            # Ver si el punto cae al menos en la región extendida
            if not (extended_theta_min <= theta_pt <= extended_theta_max) or not (extended_r_min <= r_pt <= extended_r_max):
                # Ni siquiera está en la zona extrapolada => skip
                continue
            
            # Determinar si está en la zona "normal" o en la "extendida"
            # para definir la variación de pendiente permitida
            if (normal_theta_min <= theta_pt <= normal_theta_max) and (normal_r_min <= r_pt <= normal_r_max):
                local_slope_threshold = slope_variation_threshold  # e.g. 20%
            else:
                # Está en la región extrapolada => 30%
                local_slope_threshold = 0.30
            
            # Tomar subset local (como antes)
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = g['points'][(g['points']['theta'] >= window_min) & (g['points']['theta'] <= window_max)]
            subset = pd.concat([subset, point.to_frame().T], ignore_index=True)
            
            if len(subset) < 2:
                continue
            
            # Ajustar recta al subset
            model.fit(subset[['theta']], subset['r'])
            new_slope = model.coef_[0]
            new_intercept = model.intercept_
            old_slope = g['slope']
            
            # Verificamos variación de pendiente con 'local_slope_threshold'
            if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) > local_slope_threshold:
                continue
            
            # Chequeamos la distancia del punto a esta nueva recta
            dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
            if dist > radius_tolerance:
                continue
            
            # Si pasa todo, es candidato
            if (best_distance is None) or (dist < best_distance):
                best_distance = dist
                best_group = g
        
        if best_group is not None:
            # Asignar el punto
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            
            # Reajustamos la pendiente (con subset local o todo el grupo)
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset_bg = best_group['points'][
                (best_group['points']['theta'] >= window_min) & 
                (best_group['points']['theta'] <= window_max)
            ]
            if len(subset_bg) < 2:
                subset_bg = best_group['points']
            
            model.fit(subset_bg[['theta']], subset_bg['r'])
            best_group['slope'] = model.coef_[0]
            best_group['intercept'] = model.intercept_
            best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
            
            # Fusión
            merged_flag = True
            while merged_flag:
                merged_flag = False
                for other_group in groups:
                    if other_group is best_group or other_group['slope'] is None:
                        continue
                    best_theta_min = best_group['points']['theta'].min()
                    best_theta_max = best_group['points']['theta'].max()
                    best_r_min = best_group['points']['r'].min()
                    best_r_max = best_group['points']['r'].max()

                    other_theta_min = other_group['points']['theta'].min()
                    other_theta_max = other_group['points']['theta'].max()
                    other_r_min = other_group['points']['r'].min()
                    other_r_max = other_group['points']['r'].max()

                    # Criterio de fusión: si los límites en θ y r se solapan lo suficiente
                    if (abs(best_theta_min - other_theta_max) < fusion_theta_threshold and
                        abs(best_theta_max - other_theta_min) < fusion_theta_threshold and
                        abs(best_r_min - other_r_max) < fusion_r_threshold and
                        abs(best_r_max - other_r_min) < fusion_r_threshold):
                        
                        merged_points = pd.concat([best_group['points'], other_group['points']], ignore_index=True)
                        model.fit(merged_points[['theta']], merged_points['r'])
                        best_group['points'] = merged_points
                        best_group['slope'] = model.coef_[0]
                        best_group['intercept'] = model.intercept_
                        best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
                        for idx, grp in enumerate(groups):
                            if grp is other_group:
                                groups.pop(idx)
                                break
                        merged_flag = True
                        break
            
        else:
            # Si no hay grupo candidato, va a temp_groups
            add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window)
        
        iteration += 1
        
        # Revisar temp_groups cada 'review_interval'
        if iteration % review_interval == 0:
            review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range)
        
        # Guardar snapshot
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_groups': copy.deepcopy(temp_groups),
                'remaining': df_local.copy()
            })
    
    # Revisar temp_groups final
    if len(temp_groups) > 0:
        review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range)
    
    # Subdividir si hay gaps
    final_groups = []
    for g in groups:
        segs = subdivide_if_gap(g, max_gap=max_theta_gap)
        final_groups.extend(segs)
    
    # Filtrar grupos con pa fuera de flexible_pa_range
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    
    # Guardar estado final
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'temp_groups': copy.deepcopy(temp_groups),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered


# =========================================================
# ============ OPCIONAL: FUNCIÓN DE PLOTEO ===============
# =========================================================

def plot_intermediate_state_unificado(state, df_all):
    """
    Grafica un estado en el plano (theta, r).
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos totales')
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, label=f"Grupo {idx+1}")
        if g.get('slope') is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', linewidth=1.5)
    
    if 'temp_groups' in state and state['temp_groups']:
        for idx, tg in enumerate(state['temp_groups']):
            pts = tg['points']
            plt.scatter(pts['theta'], pts['r'], s=10, color='black', marker='x', label=f"Temp {idx+1}")
    
    plt.xlabel("theta (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']}")
    plt.grid(True)
    plt.show()


# =========================================================
# ============ EJEMPLO DE USO =============================
# =========================================================
if __name__ == "__main__":
    """
    Ejemplo rápido de cómo llamar a la función unificada con los cambios.
    """

    states, final_groups, df_filtered = process_spiral_arms_unificado(
        id_halo="17",
        quartile_threshold=0.155,
        theta_min=50,
        theta_max=250,
        # Ventanas BFS más pequeñas:
        theta_diff=1.5,
        r_diff=0.3,
        gap_threshold_theta=1.89,
        gap_threshold_r=0.523,
        min_seed_size=50,
        radius_tolerance=0.45,
        # Umbral normal 20%, se usa 30% cuando el punto esté en región extrapolada:
        slope_variation_threshold=0.20,
        flexible_pa_range=(4.0, 30.0),
        max_theta_gap=15.0,
        snapshot_interval=10,
        review_interval=30,
        theta_fit_window=30.0,
        r_fit_window=10.0,
        fusion_theta_threshold=7.0,
        fusion_r_threshold=7.0
    )

    print(f"Total de estados guardados: {len(states)}")
    print(f"Total de grupos finales: {len(final_groups)}")
    
    # En un entorno interactivo:
    # from ipywidgets import interact, IntSlider
    # def view_state(i=0):
    #     plot_intermediate_state_unificado(states[i], df_filtered)
    # interact(view_state, i=IntSlider(min=0, max=len(states)-1, step=1, value=0))


In [0]:
def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state_cartesian(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))

## buena aproximación. 

In [0]:
import copy
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

warnings.filterwarnings('ignore')

# =========================================================
# ============ FUNCIONES AUXILIARES BFS (CÓDIGO 2) ========
# =========================================================

def load_and_filter_data(id_halo, theta_min=50, theta_max=250):
    """
    Carga los datos desde CSV, calcula coordenadas polares y filtra
    según el rango angular [theta_min, theta_max].
    Ajusta la columna 'theta' para que esté en [0, 360).
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    return df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo donde cada punto se conecta a otros
    si las diferencias en θ y r son <= a los umbrales dados.
    Retorna: (graph, n) donde graph[i] es la lista de vecinos de i.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Vecinos cumplen (dtheta <= theta_diff) & (dr <= r_diff), excluyendo i mismo
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    """
    Extrae componentes conexas del grafo usando BFS.
    Cada componente es un DataFrame con los puntos de ese subgrupo.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Subdivide el DataFrame (ordenado por 'theta' o 'r') en subclusters
    si hay saltos (gaps) mayores a gap_threshold.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    
    # Si no hay gap mayor al umbral, devolvemos un solo subcluster
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i + 1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    df_base,
    quartile_threshold=0.55,
    theta_diff=2.0,
    r_diff=0.5,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    """
    Toma un DF, filtra por quantile en 'rho_resta_final_exp', 
    construye grafo y extrae componentes BFS. 
    Luego subdivide cada componente por gaps en θ y r.
    Devuelve una lista de DataFrames (clusters finales) y el DataFrame filtrado.
    """
    threshold = df_base['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df_base[df_base['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        # Subdividir por gaps en θ
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            # Subdividir por gaps en r
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters, df_filtered

# =========================================================
# ============ FUNCIONES AUXILIARES (CÓDIGO 1) ============
# =========================================================

def calculate_distance(slope, intercept, theta, r):
    """
    Calcula la distancia vertical de un punto (theta, r)
    a la recta r_pred = slope * theta + intercept.
    """
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    """
    Calcula el ángulo PA en grados dado slope e intercept.
    Si intercept es 0, se retorna NaN.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan

def add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window):
    """
    Agrega un punto a un temp_group si encaja (según min/max theta/r).
    De lo contrario, crea un nuevo temp_group.
    """
    assigned = False
    for tg in temp_groups:
        tg_theta_min = tg['points']['theta'].min()
        tg_theta_max = tg['points']['theta'].max()
        tg_r_min = tg['points']['r'].min()
        tg_r_max = tg['points']['r'].max()
        if ((point['theta'] >= tg_theta_min - theta_fit_window/2) and 
            (point['theta'] <= tg_theta_max + theta_fit_window/2)) and \
           ((point['r'] >= tg_r_min - r_fit_window/2) and 
            (point['r'] <= tg_r_max + r_fit_window/2)):
            tg['points'] = pd.concat([tg['points'], point.to_frame().T], ignore_index=True)
            assigned = True
            break
    if not assigned:
        new_tg = {'points': point.to_frame().T.copy()}
        temp_groups.append(new_tg)

def review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range):
    """
    Revisa los temp_groups. 
    - Si tienen <3 puntos, no se ajustan todavía.
    - Si >=3, se hace ajuste lineal y se trata de fusionar/promover.
    - Se validan distancias y variación de pendiente con los grupos principales.
    """
    new_temp_groups = []
    for tg in temp_groups:
        # Si muy pocos puntos, los dejamos tal cual
        if len(tg['points']) < 3:
            new_temp_groups.append(tg)
            continue
        
        # Ajuste lineal del temp_group
        X_tg = tg['points'][['theta']]
        y_tg = tg['points']['r']
        model.fit(X_tg, y_tg)
        tg_slope = model.coef_[0]
        tg_intercept = model.intercept_
        
        # Promediamos la theta para ver dónde "cae"
        tg_mean = tg['points']['theta'].mean()
        merged = False
        
        # Intentamos ver si se fusiona con algún grupo principal
        for mg in groups:
            # Si mg no tiene slope, podemos forzar un ajuste rápido
            # con los puntos actuales + mg, si se desea
            if mg['slope'] is not None:
                mg_min = mg['points']['theta'].min()
                mg_max = mg['points']['theta'].max()
                if mg_min <= tg_mean <= mg_max:
                    dist_values = tg['points'].apply(
                        lambda row: calculate_distance(mg['slope'], mg['intercept'], row['theta'], row['r']),
                        axis=1
                    )
                    if dist_values.mean() < radius_tolerance:
                        # Fusionar
                        mg['points'] = pd.concat([mg['points'], tg['points']], ignore_index=True)
                        # Reajustar slope en mg
                        X_mg = mg['points'][['theta']]
                        y_mg = mg['points']['r']
                        model.fit(X_mg, y_mg)
                        new_slope = model.coef_[0]
                        old_slope = mg['slope']
                        if old_slope is not None:
                            if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) < slope_variation_threshold:
                                mg['slope'] = new_slope
                                mg['intercept'] = model.intercept_
                                mg['pa'] = calculate_pa(mg['slope'], mg['intercept'])
                        merged = True
                        break
            else:
                # mg sin slope => Ver si queremos fusionar
                # (Podrías hacer un check de solapamiento en θ/r)
                pass
        
        if not merged:
            # Crear un nuevo grupo principal
            new_group = {
                'cluster_id': None,
                'points': tg['points'],
                'slope': tg_slope,
                'intercept': tg_intercept,
                'pa': calculate_pa(tg_slope, tg_intercept)
            }
            groups.append(new_group)
    
    temp_groups.clear()
    for tg in new_temp_groups:
        temp_groups.append(tg)

def subdivide_if_gap(group, max_gap=15.0):
    """
    Toma un grupo (dict con 'points') y, si hay un gap grande en θ, 
    lo divide en dos subgrupos. Retorna una lista (1 o 2 subgrupos).
    """
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        model = LinearRegression()
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
    if len(pts) < 4:
        return [group]
    
    theta_vals = pts['theta'].values
    dtheta = np.diff(theta_vals)
    gap_idx = np.where(dtheta > max_gap)[0]
    if len(gap_idx) == 0:
        return [group]
    
    # Partimos en el primer gap
    split_idx = gap_idx[0]
    pts1 = pts.iloc[:split_idx+1]
    pts2 = pts.iloc[split_idx+1:]
    
    s1, i1, pa1 = fit_line(pts1)
    s2, i2, pa2 = fit_line(pts2)
    
    seg1 = {
        'cluster_id': group['cluster_id'],
        'points': pts1,
        'slope': s1,
        'intercept': i1,
        'pa': pa1
    }
    seg2 = {
        'cluster_id': group['cluster_id'],
        'points': pts2,
        'slope': s2,
        'intercept': i2,
        'pa': pa2
    }
    return [seg1, seg2]


# =========================================================
# ============ FUNCIÓN PRINCIPAL UNIFICADA ===============
# =========================================================

def process_spiral_arms_unificado(
    id_halo,
    # Parámetros BFS / seeds
    quartile_threshold=0.55,
    theta_min=50, theta_max=250,
    theta_diff=2.0,
    r_diff=0.5,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5,
    min_seed_size=60,

    # Parámetros de la iteración (código 1)
    radius_tolerance=0.45,    
    slope_variation_threshold=0.20,
    flexible_pa_range=(4.0, 30.0),
    max_theta_gap=15.0,
    snapshot_interval=10,
    review_interval=50,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    fusion_theta_threshold=7.0,
    fusion_r_threshold=7.0
):
    """
    Fusión de técnicas (sin ajuste inicial a las semillas):
      1) Se obtienen "semillas" via BFS y subdivisión por gaps.
         *No* se calcula slope/intercept para ellas.
      2) Se descartan (o marcan) los puntos de las semillas y
         se itera sobre el resto.
      3) Si un punto encaja en un grupo que no tiene slope:
         al tener >=2 puntos, se calcula por primera vez la slope.
      4) Si un grupo ya tiene slope, se valida dist. a la recta,
         variación de pendiente, etc.
      5) Fusión de grupos si se superponen.
      6) Al final, subdividir y filtrar por PA.

    Retorna:
      - states: lista de estados
      - final_groups: lista final
      - df_filtered: DataFrame filtrado
    """
    # 1. Cargar datos base
    df_base = load_and_filter_data(id_halo, theta_min, theta_max)
    
    # 2. Semillas BFS + gaps
    seeds_all, df_filtered = process_clusters_rectangular_gaps(
        df_base,
        quartile_threshold=quartile_threshold,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    
    # 3. Quedarnos con seeds grandes
    seed_clusters = [s for s in seeds_all if len(s) >= min_seed_size]
    
    # 4. Marcar en df_filtered cuáles puntos son semilla
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed_df in seed_clusters:
        seed_indices.update(seed_df.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 5. Crear grupos "semilla" sin slope/intercept:
    groups = []
    for idx, seed_pts in enumerate(seed_clusters):
        groups.append({
            'cluster_id': idx,
            'points': seed_pts.copy(),
            'slope': None,
            'intercept': None,
            'pa': None
        })
    
    # 6. Puntos restantes (no en seed)
    df_local = df_filtered[~df_filtered['in_seed']].copy()
    df_local.sort_values(by='theta', inplace=True)
    df_local.reset_index(drop=True, inplace=True)
    
    # 7. Iteración
    model = LinearRegression()
    temp_groups = []
    states = []
    iteration = 0
    
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        
        best_group = None
        best_distance = None
        
        for g in groups:
            g_theta_min = g['points']['theta'].min()
            g_theta_max = g['points']['theta'].max()
            g_r_min = g['points']['r'].min()
            g_r_max = g['points']['r'].max()
            
            # Verificamos si el punto puede entrar en la ventana
            if not (g_theta_min - theta_fit_window <= theta_pt <= g_theta_max + theta_fit_window):
                continue
            if not (g_r_min - r_fit_window <= r_pt <= g_r_max + r_fit_window):
                continue
            
            # pre-subset
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = g['points'][(g['points']['theta'] >= window_min) & (g['points']['theta'] <= window_max)]
            subset = pd.concat([subset, point.to_frame().T], ignore_index=True)
            if len(subset) < 2:
                # Con 1 solo punto no podemos calcular slope -> skip
                continue
            
            # Ajustar recta para ver la slope resultante
            model.fit(subset[['theta']], subset['r'])
            new_slope = model.coef_[0]
            new_intercept = model.intercept_
            
            if g['slope'] is None:
                # Grupo seed sin slope: no comparamos la variación, 
                # pero sí la distancia
                dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
                if dist > radius_tolerance:
                    continue
            else:
                # Si ya tenemos slope, comparamos la variación
                old_slope = g['slope']
                if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) > slope_variation_threshold:
                    continue
                dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
                if dist > radius_tolerance:
                    continue
            
            # Si llega aquí, es candidato
            if best_distance is None or dist < best_distance:
                best_distance = dist
                best_group = g
        
        if best_group is not None:
            # Asignamos el punto
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            
            # Recalculamos slope si es posible (>=2 pts)
            if len(best_group['points']) >= 2:
                # Tomar subset local
                window_min = theta_pt - theta_fit_window/2
                window_max = theta_pt + theta_fit_window/2
                subset_bg = best_group['points'][
                    (best_group['points']['theta'] >= window_min) &
                    (best_group['points']['theta'] <= window_max)
                ]
                if len(subset_bg) < 2:
                    subset_bg = best_group['points']
                
                model.fit(subset_bg[['theta']], subset_bg['r'])
                best_group['slope'] = model.coef_[0]
                best_group['intercept'] = model.intercept_
                best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
            
            # Fusión
            merged_flag = True
            while merged_flag:
                merged_flag = False
                for other_group in groups:
                    if other_group is best_group:
                        continue
                    # Chequeo superposición
                    # Solo fusionar si best_group['slope'] no es None
                    # (Opcional)
                    if best_group['slope'] is None:
                        continue
                    
                    # Calculamos límites
                    bg_th_min = best_group['points']['theta'].min()
                    bg_th_max = best_group['points']['theta'].max()
                    bg_r_min = best_group['points']['r'].min()
                    bg_r_max = best_group['points']['r'].max()

                    og_th_min = other_group['points']['theta'].min()
                    og_th_max = other_group['points']['theta'].max()
                    og_r_min = other_group['points']['r'].min()
                    og_r_max = other_group['points']['r'].max()

                    # Criterio: si se superponen bastante
                    if (abs(bg_th_min - og_th_max) < fusion_theta_threshold and
                        abs(bg_th_max - og_th_min) < fusion_theta_threshold and
                        abs(bg_r_min - og_r_max) < fusion_r_threshold and
                        abs(bg_r_max - og_r_min) < fusion_r_threshold):
                        
                        merged_points = pd.concat([best_group['points'], other_group['points']], ignore_index=True)
                        model.fit(merged_points[['theta']], merged_points['r'])
                        best_group['points'] = merged_points
                        best_group['slope'] = model.coef_[0]
                        best_group['intercept'] = model.intercept_
                        best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
                        
                        # Eliminar other_group
                        for idx, gg in enumerate(groups):
                            if gg is other_group:
                                groups.pop(idx)
                                break
                        
                        merged_flag = True
                        break
        
        else:
            # No se asigna -> temp_groups
            add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window)
        
        iteration += 1
        
        # Revisar temp_groups cada 'review_interval'
        if iteration % review_interval == 0:
            review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range)
        
        # Guardar snapshot
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_groups': copy.deepcopy(temp_groups),
                'remaining': df_local.copy()
            })
    
    # Revisar temp_groups final
    if len(temp_groups) > 0:
        review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range)
    
    # Subdividir si hay gaps
    final_groups = []
    for g in groups:
        segs = subdivide_if_gap(g, max_gap=max_theta_gap)
        final_groups.extend(segs)
    
    # Filtrar grupos con pa fuera de flexible_pa_range
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    
    # Guardar estado final
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'temp_groups': copy.deepcopy(temp_groups),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered


# =========================================================
# ============ FUNCIÓN DE PLOTEO (OPCIONAL) ===============
# =========================================================

def plot_intermediate_state_unificado(state, df_all):
    """
    Grafica un estado en el plano (theta, r).
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos totales')
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, label=f"Grupo {idx+1}")
        if g.get('slope') is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', linewidth=1.5)
    
    if 'temp_groups' in state and state['temp_groups']:
        for idx, tg in enumerate(state['temp_groups']):
            pts = tg['points']
            plt.scatter(pts['theta'], pts['r'], s=10, color='black', marker='x', label=f"Temp {idx+1}")
    
    plt.xlabel("theta (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']}")
    plt.grid(True)
    plt.show()


# =========================================================
# ============ EJEMPLO DE USO (OPCIONAL) ==================
# =========================================================

if __name__ == "__main__":
    """
    Ejemplo rápido de cómo llamar a la función unificada.
    Ajusta parámetros y rutas a tus necesidades.
    """

    states, final_groups, df_filtered = process_spiral_arms_unificado(
        id_halo="17",
        quartile_threshold=0.155,
        theta_min=50,
        theta_max=250,
        theta_diff=2.0,
        r_diff=0.5,
        gap_threshold_theta=2.0,
        gap_threshold_r=2.0,
        min_seed_size=30,
        radius_tolerance=0.45,
        slope_variation_threshold=0.20,
        flexible_pa_range=(4.0, 30.0),
        max_theta_gap=15.0,
        snapshot_interval=10,
        review_interval=30,
        theta_fit_window=30.0,
        r_fit_window=10.0,
        fusion_theta_threshold=7.0,
        fusion_r_threshold=7.0
    )

    print(f"Total de estados guardados: {len(states)}")
    print(f"Total de grupos finales: {len(final_groups)}")
    
    # from ipywidgets import interact, IntSlider
    # def view_state(i=0):
    #     plot_intermediate_state_unificado(states[i], df_filtered)
    # interact(view_state, i=IntSlider(min=0, max=len(states)-1, step=1, value=0))


In [0]:
def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state_cartesian(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))

### nueva aprox. los parametros anteriores dieron buenos resultados.

In [0]:
import copy
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider

warnings.filterwarnings('ignore')


# =========================================================
# ============ FUNCIONES AUXILIARES BFS (CÓDIGO 2) ========
# =========================================================

def load_and_filter_data(id_halo, theta_min=50, theta_max=250):
    """
    Carga los datos desde CSV, calcula coordenadas polares y filtra
    según el rango angular [theta_min, theta_max].
    Ajusta la columna 'theta' para que esté en [0, 360).
    """
    df = pd.read_csv(f"data_rho_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    return df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo donde cada punto se conecta a otros
    si las diferencias en θ y r son <= a los umbrales dados.
    Retorna: (graph, n).
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(theta_arr)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Vecinos cumplen (dtheta <= theta_diff) & (dr <= r_diff), excluyendo i mismo
        neighbors = np.where((dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i))[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    """
    Extrae componentes conexas del grafo usando BFS.
    Cada componente es un DataFrame con los puntos de ese subgrupo.
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy().reset_index(drop=True)
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Subdivide el DataFrame (ordenado por 'theta' o 'r') en subclusters
    si hay saltos (gaps) mayores a gap_threshold.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col).reset_index(drop=True)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i + 1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    df_base,
    quartile_threshold=0.55,
    theta_diff=2.0,
    r_diff=0.5,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    """
    Toma un DF, filtra por quantile en 'rho_resta_final_exp', 
    construye grafo y extrae componentes BFS. 
    Luego subdivide cada componente por gaps en θ y r.
    Devuelve una lista de DataFrames (clusters finales) y el DF filtrado.
    """
    threshold = df_base['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df_base[df_base['rho_resta_final_exp'] > threshold].copy().reset_index(drop=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters, df_filtered

# =========================================================
# ============ FUNCIONES AUXILIARES (CÓDIGO 1) ============
# =========================================================

def calculate_distance(slope, intercept, theta, r):
    """Distancia vertical de un punto (theta, r) a la recta r = slope*theta + intercept."""
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    """Calcula el ángulo PA en grados dado slope e intercept."""
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan

def add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window):
    """
    Agrega un punto a un temp_group si encaja, o crea uno nuevo si no encaja en ninguno.
    """
    assigned = False
    for tg in temp_groups:
        tg_theta_min = tg['points']['theta'].min()
        tg_theta_max = tg['points']['theta'].max()
        tg_r_min = tg['points']['r'].min()
        tg_r_max = tg['points']['r'].max()
        if ((point['theta'] >= tg_theta_min - theta_fit_window/2) and 
            (point['theta'] <= tg_theta_max + theta_fit_window/2)) and \
           ((point['r'] >= tg_r_min - r_fit_window/2) and 
            (point['r'] <= tg_r_max + r_fit_window/2)):
            tg['points'] = pd.concat([tg['points'], point.to_frame().T], ignore_index=True)
            assigned = True
            break
    if not assigned:
        new_tg = {'points': point.to_frame().T.copy()}
        temp_groups.append(new_tg)

def review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range):
    """
    Revisa los temp_groups. 
    - Si tienen <3 puntos: se quedan allí.
    - Si >=3, se ajusta recta y se intenta fusionar/promover a 'groups'.
    """
    new_temp_groups = []
    for tg in temp_groups:
        if len(tg['points']) < 3:
            new_temp_groups.append(tg)
            continue
        
        # Ajuste lineal del temp_group
        X_tg = tg['points'][['theta']]
        y_tg = tg['points']['r']
        model.fit(X_tg, y_tg)
        tg_slope = model.coef_[0]
        tg_intercept = model.intercept_
        tg_mean = tg['points']['theta'].mean()
        
        merged = False
        for mg in groups:
            if mg['slope'] is None:
                # Si mg no tiene slope, se puede fusionar si la distancia es baja
                # y luego mg obtendrá slope
                dist_values = tg['points'].apply(
                    lambda row: calculate_distance(tg_slope, tg_intercept, row['theta'], row['r']),
                    axis=1
                )
                if dist_values.mean() < radius_tolerance:
                    mg['points'] = pd.concat([mg['points'], tg['points']], ignore_index=True)
                    merged = True
                    break
            else:
                mg_min = mg['points']['theta'].min()
                mg_max = mg['points']['theta'].max()
                if mg_min <= tg_mean <= mg_max:
                    dist_values = tg['points'].apply(
                        lambda row: calculate_distance(mg['slope'], mg['intercept'], row['theta'], row['r']),
                        axis=1
                    )
                    if dist_values.mean() < radius_tolerance:
                        mg['points'] = pd.concat([mg['points'], tg['points']], ignore_index=True)
                        # Reajustamos slope
                        model.fit(mg['points'][['theta']], mg['points']['r'])
                        new_slope = model.coef_[0]
                        old_slope = mg['slope']
                        if abs(new_slope - old_slope)/(abs(old_slope)+1e-12) < slope_variation_threshold:
                            mg['slope'] = new_slope
                            mg['intercept'] = model.intercept_
                            mg['pa'] = calculate_pa(mg['slope'], mg['intercept'])
                        merged = True
                        break
        
        if not merged:
            # Crear nuevo grupo real
            new_group = {
                'cluster_id': None,
                'points': tg['points'],
                'slope': tg_slope,
                'intercept': tg_intercept,
                'pa': calculate_pa(tg_slope, tg_intercept)
            }
            groups.append(new_group)
    
    temp_groups.clear()
    for tg in new_temp_groups:
        temp_groups.append(tg)

def subdivide_if_gap(group, max_gap=15.0):
    """
    Si hay un gap grande en θ, divide el grupo en 2.
    """
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        model = LinearRegression()
        model.fit(pts_df[['theta']], pts_df['r'])
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    pts = group['points'].copy().sort_values(by='theta').reset_index(drop=True)
    if len(pts) < 4:
        return [group]
    
    theta_vals = pts['theta'].values
    dtheta = np.diff(theta_vals)
    gap_idx = np.where(dtheta > max_gap)[0]
    if len(gap_idx) == 0:
        return [group]
    
    split_idx = gap_idx[0]
    pts1 = pts.iloc[:split_idx+1]
    pts2 = pts.iloc[split_idx+1:]
    
    s1, i1, pa1 = fit_line(pts1)
    s2, i2, pa2 = fit_line(pts2)
    
    seg1 = {
        'cluster_id': group['cluster_id'],
        'points': pts1,
        'slope': s1,
        'intercept': i1,
        'pa': pa1
    }
    seg2 = {
        'cluster_id': group['cluster_id'],
        'points': pts2,
        'slope': s2,
        'intercept': i2,
        'pa': pa2
    }
    return [seg1, seg2]


# =========================================================
# ============ FUNCIÓN PRINCIPAL UNIFICADA ===============
# =========================================================

def process_spiral_arms_unificado(
    id_halo,
    # Parámetros BFS / seeds
    quartile_threshold=0.55,
    theta_min=50, theta_max=250,
    theta_diff=2.0,
    r_diff=0.5,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5,
    min_seed_size=60,

    # Parámetros para la iteración
    radius_tolerance=0.45,    
    slope_variation_threshold=0.20,
    flexible_pa_range=(4.0, 30.0),
    max_theta_gap=15.0,
    snapshot_interval=10,
    review_interval=50,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    fusion_theta_threshold=7.0,
    fusion_r_threshold=7.0
):
    """
    Flujo unificado y optimizado:
      1) Obtiene “semillas” con BFS. No se ajusta slope en ellas.
      2) Itera sobre los puntos restantes:
         - Verifica radius_tolerance con cada seed/grupo.
         - Si el grupo no tenía slope, lo calculamos en cuanto tenga >=2 puntos.
         - Fusiona “temprano” con grupos contiguos en la lista (si cumplen criterios).
      3) temp_groups para puntos que no encajan.
      4) Al final subdivide y filtra por PA.

    Retorna:
      states, final_groups, df_filtered
    """
    # 1. Cargar y filtrar datos base
    df_base = load_and_filter_data(id_halo, theta_min, theta_max)
    
    # 2. Semillas BFS + subdivisión
    seeds_all, df_filtered = process_clusters_rectangular_gaps(
        df_base,
        quartile_threshold=quartile_threshold,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    
    # 3. Conservar solo semillas grandes
    seed_clusters = [s for s in seeds_all if len(s) >= min_seed_size]
    
    # 4. Marcar en df_filtered quiénes son semilla
    df_filtered['in_seed'] = False
    seed_indices = set()
    for seed_df in seed_clusters:
        seed_indices.update(seed_df.index.tolist())
    df_filtered.loc[list(seed_indices), 'in_seed'] = True
    
    # 5. Crear grupos semilla sin slope
    groups = []
    for idx, seed_pts in enumerate(seed_clusters):
        groups.append({
            'cluster_id': idx,
            'points': seed_pts.copy(),
            'slope': None,
            'intercept': None,
            'pa': None
        })
    
    # 6. Puntos restantes
    df_local = df_filtered[~df_filtered['in_seed']].copy()
    df_local.sort_values(by='theta', inplace=True)
    df_local.reset_index(drop=True, inplace=True)
    
    model = LinearRegression()
    temp_groups = []
    states = []
    iteration = 0
    
    while len(df_local) > 0:
        point = df_local.iloc[0]
        df_local = df_local.iloc[1:]
        theta_pt = point['theta']
        r_pt = point['r']
        
        best_group = None
        best_dist = None
        
        # ========== ASIGNACIÓN RÁPIDA POR radius_tolerance ==========
        # Recorremos todos los grupos
        for g_idx, g in enumerate(groups):
            # Calcular distancia media del point a la "nube" de g
            # Para no recalcular slope, comparamos simplemente la distancia euclídea 
            # en el plano (theta, r) con el centro, O BIEN la "vertical" si ya tiene slope.
            
            if g['slope'] is None:
                # Grupo sin slope => usar distancia (theta,r) al centro
                g_mean_theta = g['points']['theta'].mean()
                g_mean_r = g['points']['r'].mean()
                dist_val = np.sqrt((theta_pt - g_mean_theta)**2 + (r_pt - g_mean_r)**2)
                if dist_val < radius_tolerance:  
                    # Lo consideramos candidato
                    if best_dist is None or dist_val < best_dist:
                        best_dist = dist_val
                        best_group = g
            else:
                # Si ya tiene slope, calculamos la distancia vertical
                dist_val = calculate_distance(g['slope'], g['intercept'], theta_pt, r_pt)
                if dist_val < radius_tolerance:
                    if best_dist is None or dist_val < best_dist:
                        best_dist = dist_val
                        best_group = g
        
        # ========== SI ENCONTRÓ GRUPO, LO ASIGNAMOS DE INMEDIATO ==========
        if best_group is not None:
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            
            # Si no tenía slope y ahora puede calcularse
            if best_group['slope'] is None and len(best_group['points']) >= 2:
                # Ajustar una recta con todos sus puntos
                Xb = best_group['points'][['theta']]
                yb = best_group['points']['r']
                model.fit(Xb, yb)
                best_group['slope'] = model.coef_[0]
                best_group['intercept'] = model.intercept_
                best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
            
            # ========== FUSIÓN TEMPRANA CON GRUPOS CONTIGUOS ==============
            # Revisamos el "vecindario" del best_group en la lista 'groups'.
            # O revisamos toda la lista si no son muchos.
            merged_flag = True
            while merged_flag:
                merged_flag = False
                for other_group in groups:
                    if other_group is best_group:
                        continue
                    if best_group['slope'] is None and other_group['slope'] is None:
                        # Podríamos fusionar 2 grupos seed sin slope 
                        # si su distancia en (theta,r) es pequeña
                        # (a tu gusto)
                        pass
                    elif best_group['slope'] is not None:
                        # Checar superposición en θ y r
                        bg_th_min = best_group['points']['theta'].min()
                        bg_th_max = best_group['points']['theta'].max()
                        bg_r_min = best_group['points']['r'].min()
                        bg_r_max = best_group['points']['r'].max()

                        og_th_min = other_group['points']['theta'].min()
                        og_th_max = other_group['points']['theta'].max()
                        og_r_min = other_group['points']['r'].min()
                        og_r_max = other_group['points']['r'].max()

                        if (abs(bg_th_min - og_th_max) < fusion_theta_threshold and
                            abs(bg_th_max - og_th_min) < fusion_theta_threshold and
                            abs(bg_r_min - og_r_max) < fusion_r_threshold and
                            abs(bg_r_max - og_r_min) < fusion_r_threshold):

                            # Fusionar: unimos puntos y recalculamos slope
                            merged_pts = pd.concat([best_group['points'], other_group['points']], ignore_index=True)
                            model.fit(merged_pts[['theta']], merged_pts['r'])
                            best_group['points'] = merged_pts
                            best_group['slope'] = model.coef_[0]
                            best_group['intercept'] = model.intercept_
                            best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
                            # Quitamos other_group
                            for idxg, grp_ in enumerate(groups):
                                if grp_ is other_group:
                                    groups.pop(idxg)
                                    break
                            merged_flag = True
                            break
        else:
            # No encontró grupo => temp_groups
            add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window)
        
        iteration += 1
        
        # ========== REVIEW TEMP GROUPS CADA review_interval ==========
        if iteration % review_interval == 0:
            review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range)
        
        # ========== SNAPSHOT ==========
        if iteration % snapshot_interval == 0 or len(df_local) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_groups': copy.deepcopy(temp_groups),
                'remaining': df_local.copy()
            })
    
    # ========== FINAL REVIEW DE temp_groups ==========
    if len(temp_groups) > 0:
        review_temp_groups(temp_groups, groups, model, radius_tolerance, slope_variation_threshold, flexible_pa_range)
    
    # ========== SUBDIVIDIR POR GAPS ==========
    final_groups = []
    for g in groups:
        segs = subdivide_if_gap(g, max_gap=max_theta_gap)
        final_groups.extend(segs)
    
    # ========== FILTRAR POR RANGO PA ==========
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    
    # ========== ULTIMO ESTADO ==========
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'temp_groups': copy.deepcopy(temp_groups),
        'remaining': df_local.copy()
    })
    
    return states, valid_groups, df_filtered


# =========================================================
# ============ FUNCIÓN DE PLOTEO (OPCIONAL) ===============
# =========================================================

def plot_intermediate_state_unificado(state, df_all):
    """
    Grafica un estado en el plano (theta, r).
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, color='lightgrey', alpha=0.5, label='Datos totales')
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, label=f"Grupo {idx+1}")
        if g.get('slope') is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', linewidth=1.5)
    
    if 'temp_groups' in state and state['temp_groups']:
        for idx, tg in enumerate(state['temp_groups']):
            pts = tg['points']
            plt.scatter(pts['theta'], pts['r'], s=10, color='black', marker='x', label=f"Temp {idx+1}")
    
    plt.xlabel("theta (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']}")
    plt.grid(True)
    plt.show()


# =========================================================
# ============ EJEMPLO DE USO (OPCIONAL) ==================
# =========================================================
if __name__ == "__main__":
    """
    Ejemplo rápido de cómo llamar a la función con la lógica optimizada.
    Ajusta parámetros y rutas según tus necesidades.
    """
    states, final_groups, df_filtered = process_spiral_arms_unificado(
        id_halo="11",
        quartile_threshold=0.155,
        theta_min=30,
        theta_max=280,
        theta_diff=2.0,
        r_diff=0.5,
        gap_threshold_theta=1.80,
        gap_threshold_r=1.80,
        min_seed_size=50,
        radius_tolerance=0.45,
        slope_variation_threshold=0.25,
        flexible_pa_range=(4.0,30.0),
        max_theta_gap=15.0,
        snapshot_interval=10,
        review_interval=30,
        theta_fit_window=28.0,
        r_fit_window=9.0,
        fusion_theta_threshold=7.0,
        fusion_r_threshold=7.0
    )

    print(f"Total de estados guardados: {len(states)}")
    print(f"Total de grupos finales: {len(final_groups)}")
    
    # Uso interactivo en Jupyter:
    # from ipywidgets import interact, IntSlider
    # def view_state(i=0):
    #     plot_intermediate_state_unificado(states[i], df_filtered)
    # interact(view_state, i=IntSlider(min=0, max=len(states)-1, step=1, value=0))


In [0]:
def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state_cartesian(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))

### ultimo ultimo intento

In [0]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
from collections import deque
from sklearn.linear_model import LinearRegression

# ===================================================
# 1. Funciones auxiliares para generar semillas (BFS)
# ===================================================

def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    """
    Carga datos desde CSV, calcula coordenadas polares (r, theta) 
    y filtra según rango [theta_min, theta_max].
    """
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    # Filtrar en [theta_min, theta_max]
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    # Asegurarse de que las columnas sean float
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un "grafo" (lista de adyacencia) donde cada punto se conecta
    a los vecinos cuya distancia en θ es <= theta_diff y en r <= r_diff.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        # Vecinos: índice j donde dtheta[j] <= theta_diff, dr[j] <= r_diff y j != i
        mask_neighbors = (dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i)
        neighbors = np.where(mask_neighbors)[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    """
    Hace un recorrido BFS sobre el grafo para extraer componentes conexas.
    Cada componente es un subconjunto de filas (por posición entera).
    """
    n = len(graph)
    visited = [False] * n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            # comp_indices son las posiciones de las filas en df_points
            cluster_df = df_points.iloc[comp_indices].copy()
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Subdivide un DataFrame (ordenado por 'theta' o por 'r') si encuentra gaps
    mayores que el umbral en la diferencia sucesiva.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col)  # Mantenemos índices originales
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    
    # Si no hay saltos (gaps) más grandes que el umbral, se retorna tal cual.
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    
    # Caso contrario, se parte en subclusters.
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i + 1
    # Añadimos el último fragmento
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

def process_clusters_rectangular_gaps(
    df,
    quartile_threshold=0.55,
    theta_diff=5.0,
    r_diff=1.0,
    gap_threshold_theta=2.5,
    gap_threshold_r=2.5
):
    """
    Toma el DataFrame ya filtrado (en [theta_min, theta_max]), 
    aplica un corte por percentil en 'rho_resta_final_exp', 
    arma el grafo rectangular y lo subdivide por gaps en θ y r.
    Retorna lista de clusters "en bruto".
    """
    # 1) Filtrar por percentil, SIN reset_index(drop=True)
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    # Importante: Creamos una columna "orig_index" para no perder la referencia
    df_filtered['orig_index'] = df_filtered.index  
    # Ahora sí, reindexamos para BFS
    df_filtered.reset_index(drop=True, inplace=True)
    
    # 2) Construir grafo (trabaja con posiciones 0..N-1)
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    # 3) Extraer componentes con BFS
    clusters = bfs_components(graph, df_filtered)
    
    # 4) Subdividir cada cluster por gaps (theta, r)
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters

# ===================================================
# 2. Funciones de ajuste y validación (de “código 1”)
# ===================================================

def calculate_distance(slope, intercept, theta, r):
    pred_r = slope * theta + intercept
    return abs(pred_r - r)

def calculate_pa(slope, intercept):
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan

def add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window):
    assigned = False
    for tg in temp_groups:
        tg_theta_min = tg['points']['theta'].min()
        tg_theta_max = tg['points']['theta'].max()
        tg_r_min = tg['points']['r'].min()
        tg_r_max = tg['points']['r'].max()
        if ((point['theta'] >= tg_theta_min - theta_fit_window/2) and 
            (point['theta'] <= tg_theta_max + theta_fit_window/2)) and \
           ((point['r'] >= tg_r_min - r_fit_window/2) and 
            (point['r'] <= tg_r_max + r_fit_window/2)):
            tg['points'] = pd.concat([tg['points'], point.to_frame().T], ignore_index=True)
            assigned = True
            break
    if not assigned:
        new_tg = {'points': point.to_frame().T.copy()}
        temp_groups.append(new_tg)

def review_temp_groups(temp_groups, groups, model,
                       radius_tolerance, slope_variation_threshold, flexible_pa_range):
    new_temp_groups = []
    for tg in temp_groups:
        if len(tg['points']) < 3:
            new_temp_groups.append(tg)
            continue
        X_tg = tg['points'][['theta']]
        y_tg = tg['points']['r']
        model.fit(X_tg, y_tg)
        tg_slope = model.coef_[0]
        tg_intercept = model.intercept_
        tg_mean = tg['points']['theta'].mean()
        
        merged = False
        for mg in groups:
            mg_min = mg['points']['theta'].min()
            mg_max = mg['points']['theta'].max()
            if mg_min <= tg_mean <= mg_max:
                dist_values = tg['points'].apply(
                    lambda row: calculate_distance(
                        mg['slope'], mg['intercept'],
                        row['theta'], row['r']
                    ),
                    axis=1
                )
                if dist_values.mean() < radius_tolerance:
                    mg['points'] = pd.concat([mg['points'], tg['points']], ignore_index=True)
                    model.fit(mg['points'][['theta']], mg['points']['r'])
                    new_slope = model.coef_[0]
                    if abs(new_slope - mg['slope'])/(abs(mg['slope'])+1e-12) < slope_variation_threshold:
                        mg['slope'] = new_slope
                        mg['intercept'] = model.intercept_
                        mg['pa'] = calculate_pa(mg['slope'], mg['intercept'])
                    merged = True
                    break
        if not merged:
            new_group = {
                'cluster_id': None,
                'points': tg['points'],
                'slope': tg_slope,
                'intercept': tg_intercept,
                'pa': calculate_pa(tg_slope, tg_intercept)
            }
            groups.append(new_group)
    temp_groups.clear()
    for tg in new_temp_groups:
        temp_groups.append(tg)

# ===================================================
# 3. Función final que INTEGRA BFS + Ajuste Iterativo
# ===================================================

def process_spiral_arms_integrated(
    id_halo,
    # Parámetros BFS (para semillas)
    theta_min=50, 
    theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    # Parámetros de la iteración (tipo código 1)
    radius_tolerance=0.45,
    slope_variation_threshold=0.20,
    flexible_pa_range=(4.0, 30.0), 
    max_theta_gap=15.0,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    fusion_theta_threshold=7.0,
    fusion_r_threshold=7.0,
    snapshot_interval=10,
    do_subdivide=True,
    file_prefix='data_rho'
):
    """
    1) Construye "semillas" con BFS sin pendiente.
    2) Excluye esos puntos semilla de la iteración posterior.
    3) Itera sobre el resto con la mecánica de "código 1":
       - temp_groups, groups
       - fusión y ajustes de pendiente
       - subdivisión final si hay gap
    4) Retorna los estados y los grupos finales, junto con df completo ya filtrado.
    """
    # -------------------------------------------
    # A) Cargar df y filtrar en [theta_min, theta_max]
    # -------------------------------------------
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix=file_prefix)
    # Vamos a mantener este df con su índice original,
    # para poder marcar in_seed=True sin problemas.
    # df.index: array de valores (pueden ser 0.., o no consecutivos si df venía de algo mayor)
    
    # -------------------------------------------
    # B) Hallar clusters BFS "en bruto" + subdivisión
    # -------------------------------------------
    # Ojo: process_clusters_rectangular_gaps() 
    # internamente crea df_filtered con 'orig_index' y reindexa para BFS.
    clusters_bfs = process_clusters_rectangular_gaps(
        df,
        quartile_threshold=quartile_threshold,
        theta_diff=theta_diff,
        r_diff=r_diff,
        gap_threshold_theta=gap_threshold_theta,
        gap_threshold_r=gap_threshold_r
    )
    
    # clusters_bfs es una lista de DataFrames. 
    # Cada DataFrame conserva la columna 'orig_index' para volver a df original.
    
    # Semillas = clusters con al menos 2 puntos
    seed_clusters = [c for c in clusters_bfs if len(c) >= 2]
    
    # -------------------------------------------
    # C) Marcar en df quién está en semilla, usando 'orig_index'
    # -------------------------------------------
    df['in_seed'] = False
    all_seed_indices = set()
    
    for seed_df in seed_clusters:
        # La columna 'orig_index' contiene los índices del df original
        # que formaron parte de ese cluster
        all_seed_indices.update(seed_df['orig_index'].to_list())
    
    # Ahora sí podemos marcar
    df.loc[list(all_seed_indices), 'in_seed'] = True
    
    # -------------------------------------------
    # D) Inicializar "groups" a partir de las semillas
    #    con slope/intercept si >=2 puntos.
    # -------------------------------------------
    model = LinearRegression()
    groups = []
    for idx, seed_df in enumerate(seed_clusters):
        seed_df = seed_df.copy()
        slope_init, intercept_init, pa_init = None, None, None
        if len(seed_df) >= 2:
            X_ = seed_df[['theta']].values
            y_ = seed_df['r'].values
            model.fit(X_, y_)
            slope_init = model.coef_[0]
            intercept_init = model.intercept_
            pa_init = calculate_pa(slope_init, intercept_init)
        groups.append({
            'cluster_id': idx,
            'points': seed_df,  # semilla
            'slope': slope_init,
            'intercept': intercept_init,
            'pa': pa_init
        })
    
    # -------------------------------------------
    # E) Crear DataFrame de puntos "no semilla" para iterar
    # -------------------------------------------
    df_non_seed = df[~df['in_seed']].copy()
    df_non_seed['theta'] = df_non_seed['theta'].astype(float)
    df_non_seed['r'] = df_non_seed['r'].astype(float)
    # Ordenar
    df_non_seed.sort_values(by='theta', inplace=True)
    df_non_seed.reset_index(drop=True, inplace=True)

    # -------------------------------------------
    # F) Iterar con la mecánica "código 1"
    # -------------------------------------------
    states = []
    temp_groups = []
    iteration = 0
    temp_review_interval = snapshot_interval * 15  # Como en el original
    
    while len(df_non_seed) > 0:
        point = df_non_seed.iloc[0]
        df_non_seed = df_non_seed.iloc[1:]
        
        theta_pt = point['theta']
        r_pt = point['r']
        best_group = None
        best_distance = None
        
        # Intentar asignar a un grupo existente
        for g in groups:
            if g['slope'] is None:
                # Si no tiene slope, es un grupo con <2 puntos semilla
                if len(g['points']) < 1:
                    continue
                # Con 1 punto, podríamos asignar por bounding box. Ej:
                g_theta_min = g['points']['theta'].min()
                g_theta_max = g['points']['theta'].max()
                g_r_min = g['points']['r'].min()
                g_r_max = g['points']['r'].max()
                if (theta_pt >= g_theta_min - theta_fit_window/2 and 
                    theta_pt <= g_theta_max + theta_fit_window/2 and
                    r_pt >= g_r_min - r_fit_window/2 and
                    r_pt <= g_r_max + r_fit_window/2):
                    best_group = g
                    best_distance = 0.0
                continue
            
            # Caso normal
            g_theta_min = g['points']['theta'].min()
            g_theta_max = g['points']['theta'].max()
            g_r_min = g['points']['r'].min()
            g_r_max = g['points']['r'].max()
            if not (g_theta_min <= theta_pt <= g_theta_max and 
                    g_r_min <= r_pt <= g_r_max):
                continue
            
            # Ajuste local
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset = g['points'][
                (g['points']['theta'] >= window_min) & 
                (g['points']['theta'] <= window_max)
            ]
            if len(subset) == 0:
                continue
            
            r_min = subset['r'].min()
            r_max = subset['r'].max()
            if not (r_min <= r_pt <= r_max):
                continue
            
            subset_test = pd.concat([subset, point.to_frame().T], ignore_index=True)
            if len(subset_test) < 2:
                continue
            
            model.fit(subset_test[['theta']], subset_test['r'])
            new_slope = model.coef_[0]
            new_intercept = model.intercept_
            old_slope = g['slope']
            
            slope_var = abs(new_slope - old_slope)/(abs(old_slope)+1e-12)
            if slope_var > slope_variation_threshold:
                continue
            
            dist = calculate_distance(new_slope, new_intercept, theta_pt, r_pt)
            if dist > radius_tolerance:
                continue
            
            # Si llegamos aquí, es buen candidato
            if best_distance is None or dist < best_distance:
                best_distance = dist
                best_group = g
        
        if best_group is not None:
            best_group['points'] = pd.concat([best_group['points'], point.to_frame().T], ignore_index=True)
            window_min = theta_pt - theta_fit_window/2
            window_max = theta_pt + theta_fit_window/2
            subset_local = best_group['points'][
                (best_group['points']['theta'] >= window_min) &
                (best_group['points']['theta'] <= window_max)
            ]
            if len(subset_local) < 2:
                subset_local = best_group['points']
            
            model.fit(subset_local[['theta']], subset_local['r'])
            best_group['slope'] = model.coef_[0]
            best_group['intercept'] = model.intercept_
            best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
            
            # Fusión con grupos vecinos
            merged_flag = True
            while merged_flag:
                merged_flag = False
                for other_group in groups:
                    if other_group is best_group:
                        continue
                    if len(other_group['points']) == 0:
                        continue
                    
                    best_theta_min = best_group['points']['theta'].min()
                    best_theta_max = best_group['points']['theta'].max()
                    best_r_min = best_group['points']['r'].min()
                    best_r_max = best_group['points']['r'].max()
                    
                    og_theta_min = other_group['points']['theta'].min()
                    og_theta_max = other_group['points']['theta'].max()
                    og_r_min = other_group['points']['r'].min()
                    og_r_max = other_group['points']['r'].max()
                    
                    if (abs(best_theta_min - og_theta_max) < fusion_theta_threshold and
                        abs(best_theta_max - og_theta_min) < fusion_theta_threshold and
                        abs(best_r_min - og_r_max) < fusion_r_threshold and
                        abs(best_r_max - og_r_min) < fusion_r_threshold):
                        merged_points = pd.concat([best_group['points'], other_group['points']], ignore_index=True)
                        model.fit(merged_points[['theta']], merged_points['r'])
                        best_group['points'] = merged_points
                        best_group['slope'] = model.coef_[0]
                        best_group['intercept'] = model.intercept_
                        best_group['pa'] = calculate_pa(best_group['slope'], best_group['intercept'])
                        
                        idx_to_remove = None
                        for ix, gg in enumerate(groups):
                            if gg is other_group:
                                idx_to_remove = ix
                                break
                        if idx_to_remove is not None:
                            groups.pop(idx_to_remove)
                        merged_flag = True
                        break
        else:
            # No se pudo asignar -> temp_group
            add_to_temp_groups(point, temp_groups, theta_fit_window, r_fit_window)
        
        iteration += 1
        if iteration % temp_review_interval == 0:
            review_temp_groups(temp_groups, groups, model,
                               radius_tolerance, slope_variation_threshold,
                               flexible_pa_range)
        
        if iteration % snapshot_interval == 0 or len(df_non_seed) == 0:
            states.append({
                'iteration': iteration,
                'groups': copy.deepcopy(groups),
                'temp_groups': copy.deepcopy(temp_groups),
                'remaining': df_non_seed.copy()
            })
    
    # Al terminar:
    if len(temp_groups) > 0:
        review_temp_groups(temp_groups, groups, model,
                           radius_tolerance, slope_variation_threshold,
                           flexible_pa_range)
    
    # -------------------------------------------
    # G) Subdividir grupos si hay gap grande en θ (opcional)
    # -------------------------------------------
    def fit_line(pts_df):
        if len(pts_df) < 2:
            return None, None, None
        X_ = pts_df[['theta']].values
        y_ = pts_df['r'].values
        model.fit(X_, y_)
        s_ = model.coef_[0]
        i_ = model.intercept_
        pa_ = calculate_pa(s_, i_)
        return s_, i_, pa_
    
    def subdivide_if_gap(group, max_gap=15.0):
        pts = group['points'].copy().sort_values(by='theta')
        if len(pts) < 4:
            return [group]
        theta_vals = pts['theta'].values
        dtheta = np.diff(theta_vals)
        gap_idx = np.where(dtheta > max_gap)[0]
        if len(gap_idx) == 0:
            return [group]
        split_idx = gap_idx[0]
        pts1 = pts.iloc[:split_idx+1]
        pts2 = pts.iloc[split_idx+1:]
        s1, i1, pa1 = fit_line(pts1)
        s2, i2, pa2 = fit_line(pts2)
        seg1 = {
            'cluster_id': group.get('cluster_id', None),
            'points': pts1,
            'slope': s1,
            'intercept': i1,
            'pa': pa1
        }
        seg2 = {
            'cluster_id': group.get('cluster_id', None),
            'points': pts2,
            'slope': s2,
            'intercept': i2,
            'pa': pa2
        }
        return [seg1, seg2]
    
    final_groups = []
    if do_subdivide:
        for g in groups:
            segs = subdivide_if_gap(g, max_theta_gap)
            final_groups.extend(segs)
    else:
        final_groups = groups
    
    # -------------------------------------------
    # H) Filtro final por rango de PA
    # -------------------------------------------
    valid_groups = []
    for grp in final_groups:
        if grp['slope'] is not None and grp['intercept'] is not None:
            pa_val = grp['pa']
            if pa_val is not None and flexible_pa_range[0] <= abs(pa_val) <= flexible_pa_range[1]:
                valid_groups.append(grp)
    
    # Último snapshot
    states.append({
        'iteration': iteration,
        'groups': copy.deepcopy(valid_groups),
        'temp_groups': copy.deepcopy(temp_groups),
        'remaining': df_non_seed.copy()
    })
    
    return states, valid_groups, df  # df es el dataframe con in_seed=True/False


# ===================================================
# 4. Función de visualización (opcional)
# ===================================================

def plot_intermediate_state_cartesian(state, df_all):
    """
    Similar a plot_intermediate_state de "código 1",
    pero adaptado al pipeline integrado.
    Muestra en (theta, r).
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=1, alpha=0.5, label='Datos completos')
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan']
    for idx, g in enumerate(state['groups']):
        pts = g['points']
        color = colors[idx % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, label=f"Grupo {idx+1}")
        
        # Plot de la recta ajustada
        if g.get('slope') is not None:
            theta_line = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_line = g['slope'] * theta_line + g['intercept']
            plt.plot(theta_line, r_line, '--', linewidth=1.5)
    
    if state.get('temp_groups'):
        for idx_t, tg in enumerate(state['temp_groups']):
            pts = tg['points']
            plt.scatter(pts['theta'], pts['r'], s=10, marker='x', color='black', label=f"Temp {idx_t+1}")
    
    plt.xlabel(r"$\theta$ (grados)")
    plt.ylabel("r")
    plt.title(f"Estado en iteración {state['iteration']} (Plano r, θ)")
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()


In [0]:
states, final_groups, df_all = process_spiral_arms_integrated(
    id_halo="17",     # Cambia si es otro halo
    theta_min=50,
    theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    radius_tolerance=0.45,
    slope_variation_threshold=0.20,
    flexible_pa_range=(4.0, 30.0),
    max_theta_gap=15.0,
    theta_fit_window=30.0,
    r_fit_window=10.0,
    fusion_theta_threshold=7.0,
    fusion_r_threshold=7.0,
    snapshot_interval=10,
    do_subdivide=True,
    file_prefix='data_rho'
)

print("Número de estados:", len(states))
print("Número de grupos finales:", len(final_groups))

# Graficamos el último estado
plot_intermediate_state_cartesian(states[-1], df_all)


In [0]:
def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state_cartesian(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))

In [0]:
states, final_groups, df_all = process_spiral_arms_integrated(
    id_halo="11",     # Cambia si es otro halo
    theta_min=50,
    theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.45,
    gap_threshold_theta=1.80,
    gap_threshold_r=2.0,
    radius_tolerance=0.45,
    slope_variation_threshold=0.20,
    flexible_pa_range=(4.0, 30.0),
    max_theta_gap=9.0,
    theta_fit_window=30.0,
    r_fit_window=6.0,
    fusion_theta_threshold=6.0,
    fusion_r_threshold=6.0,
    snapshot_interval=10,
    do_subdivide=True,
    file_prefix='data_rho'
)

print("Número de estados:", len(states))
print("Número de grupos finales:", len(final_groups))

# Graficamos el último estado
plot_intermediate_state_cartesian(states[-1], df_all)


In [0]:
def view_state(state_index):
    state = states[state_index]
    plot_intermediate_state_cartesian(state, df_filtered)

interact(view_state, state_index=IntSlider(min=0, max=len(states)-1, step=1, value=0))

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

# =========================================
# Funciones Auxiliares: Carga, BFS, Subdivisión
# =========================================

def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    """
    Carga los datos desde CSV, calcula (r, theta) y filtra 
    en el rango de theta [theta_min, theta_max].
    """
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    # Asegurarnos de que sean float
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un "grafo" (lista de adyacencia) donde cada punto se conecta
    con sus vecinos si la diferencia en θ y en r es <= a los umbrales dados.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        mask_neighbors = (dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i)
        neighbors = np.where(mask_neighbors)[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    """
    Recorre el 'graph' con BFS y obtiene componentes conexas. 
    Cada componente devuelta es un DataFrame con esos puntos.
    """
    n = len(graph)
    visited = [False]*n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            clusters.append(cluster_df)
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Subdivide un DataFrame si hay 'gaps' mayores que 'gap_threshold' 
    en la secuencia de theta (o r) ordenada.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

# =========================================
# Función Principal: Genera SOLO las Semillas (BFS) y las Visualiza
# =========================================

def generate_bfs_seeds(
    id_halo,
    theta_min=50,
    theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    """
    1) Carga y filtra datos en [theta_min, theta_max].
    2) Filtra además por 'rho_resta_final_exp' > percentil (quartile_threshold).
    3) Construye un grafo BFS con vecindad rectangular (theta_diff, r_diff).
    4) Extrae componentes conexas.
    5) Subdivide cada cluster por gap en theta y r.
    6) Retorna la lista final de 'semillas' (clusters BFS) y el DataFrame filtrado.

    NOTA: No hay iteración ni asignación adicional de puntos.
    """
    # A) Cargar y filtrar
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix=file_prefix)
    
    # B) Filtrar por percentil
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    # Para BFS necesitaremos indices consecutivos:
    df_filtered.reset_index(drop=True, inplace=True)
    
    # C) Grafo BFS
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    # D) Subdividir por gaps en θ y r
    final_clusters = []
    for clust in clusters:
        # subdividir en theta
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            # subdividir en r
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters, df_filtered

def plot_bfs_seeds_polar(bfs_clusters, df_all):
    """
    Muestra en el plano (theta, r) los puntos totales 'df_all' en gris,
    y colorea cada cluster BFS diferente.
    """
    plt.figure(figsize=(10, 6))
    # Fondo: todos los puntos del df_all
    plt.scatter(df_all['theta'], df_all['r'], s=5, alpha=0.2, label='Datos filtrados')
    
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray']
    for idx, cluster_df in enumerate(bfs_clusters):
        color = colors[idx % len(colors)]
        plt.scatter(cluster_df['theta'], cluster_df['r'], s=10, label=f"Semilla {idx+1}", color=color)
    
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title("Semillas (BFS) antes de iterar")
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()


bfs_clusters, df_filtrado = generate_bfs_seeds(
    id_halo="17",
    theta_min=0,
    theta_max=390,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    file_prefix='data_rho'
)
print(f"Encontré {len(bfs_clusters)} grupos BFS (posibles semillas).")
plot_bfs_seeds_polar(bfs_clusters, df_filtrado)


In [0]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def calculate_pa(slope):
    """Calcula el 'position angle' (en grados) a partir de la pendiente."""
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

def fit_line_to_cluster(df_cluster):
    """
    Ajusta una recta a df_cluster (si >=2 puntos).
    Retorna slope, intercept, pa, bounding box (theta_min, theta_max, r_min, r_max).
    Si <2 puntos, retorna None en slope/intercept.
    """
    if len(df_cluster) < 2:
        return None, None, None, None, None, None, None, None
    
    X = df_cluster[['theta']].values
    y = df_cluster['r'].values
    
    model = LinearRegression()
    model.fit(X, y)
    slope_ = model.coef_[0]
    intercept_ = model.intercept_
    pa_ = calculate_pa(slope_)
    
    theta_min_ = df_cluster['theta'].min()
    theta_max_ = df_cluster['theta'].max()
    r_min_ = df_cluster['r'].min()
    r_max_ = df_cluster['r'].max()
    
    return slope_, intercept_, pa_, theta_min_, theta_max_, r_min_, r_max_

def adjust_and_merge_seeds(
    bfs_clusters,
    slope_variation_threshold=0.40,  # 40% variación de pendiente
    bounding_extrap=0.30            # 30% de extrapolación de bounding box
):
    """
    Toma los clusters BFS (semillas), ajusta una recta a cada uno 
    (si >=2 puntos) y luego realiza fusiones recíprocas mientras:
      - Haya superposición en bounding boxes (extrapoladas 30%)
      - Fusionar no dispare la variación de pendiente más allá del 40%

    Retorna una lista final de grupos (dict) con:
      - 'points': DataFrame del grupo
      - 'slope'
      - 'intercept'
      - 'pa'
      - bounding box final (theta_min, theta_max, r_min, r_max)
    """

    # 1) Crear lista de "grupos" con slope/intercept y bounding box
    groups = []
    for clust_df in bfs_clusters:
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(clust_df)
        grp = {
            'points': clust_df.copy(),
            'slope': slope_,
            'intercept': intercept_,
            'pa': pa_,
            'theta_min': tmin,
            'theta_max': tmax,
            'r_min': rmin,
            'r_max': rmax
        }
        groups.append(grp)
    
    # 2) Función para recalcular slope/intercept y bounding box luego de fusionar
    def recalc_properties(group):
        dfp = group['points']
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(dfp)
        group['slope'] = slope_
        group['intercept'] = intercept_
        group['pa'] = pa_
        group['theta_min'] = tmin
        group['theta_max'] = tmax
        group['r_min'] = rmin
        group['r_max'] = rmax

    # 3) Función para verificar superposición de bounding box con extrapolación
    def boxes_overlap(g1, g2, extrap=0.30):
        """
        Devuelve True si los bounding boxes (theta_min, theta_max, r_min, r_max)
        se superponen cuando se les amplía un 30% (por ejemplo).
        """
        # Extraer bounding box de g1
        tmin1, tmax1 = g1['theta_min'], g1['theta_max']
        rmin1, rmax1 = g1['r_min'], g1['r_max']
        # Extraer bounding box de g2
        tmin2, tmax2 = g2['theta_min'], g2['theta_max']
        rmin2, rmax2 = g2['r_min'], g2['r_max']
        
        if (tmin1 is None or tmin2 is None):
            return False
        
        # Expandir c/ bounding en 30%
        # 1) Calculamos los anchos
        dt1 = (tmax1 - tmin1)
        dr1 = (rmax1 - rmin1)
        dt2 = (tmax2 - tmin2)
        dr2 = (rmax2 - rmin2)
        
        # 2) Sumamos un 0.30 * dt, etc., arriba y abajo
        #    Por ejemplo:
        extr_dt1 = dt1 * extrap
        extr_dr1 = dr1 * extrap
        extr_dt2 = dt2 * extrap
        extr_dr2 = dr2 * extrap
        
        # "caja" g1 extrapolada
        tmin1e = tmin1 - extr_dt1
        tmax1e = tmax1 + extr_dt1
        rmin1e = rmin1 - extr_dr1
        rmax1e = rmax1 + extr_dr1
        
        # "caja" g2 extrapolada
        tmin2e = tmin2 - extr_dt2
        tmax2e = tmax2 + extr_dt2
        rmin2e = rmin2 - extr_dr2
        rmax2e = rmax2 + extr_dr2
        
        # Chequeo de superposición 1D en theta
        overlap_theta = not (tmax1e < tmin2e or tmax2e < tmin1e)
        # Chequeo de superposición 1D en r
        overlap_r = not (rmax1e < rmin2e or rmax2e < rmin1e)
        
        return overlap_theta and overlap_r
    
    # 4) Iterar fusiones
    merged_flag = True
    while merged_flag:
        merged_flag = False
        new_groups = []
        i = 0
        while i < len(groups):
            g1 = groups[i]
            j = i+1
            fused_any = False
            while j < len(groups):
                g2 = groups[j]
                # a) si no tiene slope, no fusionamos nada
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                # b) bounding boxes se superponen (con extrapolación 30%)
                if boxes_overlap(g1, g2, bounding_extrap):
                    # c) Probamos a fusionar: combinamos puntos, recalculamos slope
                    combined_points = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    
                    # Ajuste con lineal
                    slope_, intercept_, _, _, _, _, _ = fit_line_to_cluster(combined_points)
                    if slope_ is not None:
                        # Revisar variación vs la slope de g1
                        slope_var1 = abs(slope_ - g1['slope'])/(abs(g1['slope'])+1e-12)
                        # Revisar también vs la slope de g2
                        slope_var2 = abs(slope_ - g2['slope'])/(abs(g2['slope'])+1e-12)
                        # Si ambas variaciones < 40%, permitimos la fusión
                        if slope_var1 < slope_variation_threshold and slope_var2 < slope_variation_threshold:
                            # Hacemos la fusión
                            g1['points'] = combined_points
                            # Recalcular su slope/bbox
                            recalc_properties(g1)
                            # Eliminamos g2
                            groups.pop(j)
                            fused_any = True
                            merged_flag = True
                            continue
                j += 1
            new_groups.append(g1)
            i += 1
        groups = new_groups
    
    # 5) Limpieza final (por ejemplo, quitar grupos con <2 puntos)
    final_groups = []
    for gg in groups:
        if len(gg['points']) >= 2:
            final_groups.append(gg)
    
    return final_groups


# ===========================
# EJEMPLO DE USO
# ===========================

def example_adjust_and_merge_seeds():
    # Imagina que ya tienes "bfs_clusters" y "df_filtrado" de tu BFS:
    #   bfs_clusters, df_filtrado = generate_bfs_seeds(...)

    # Supongamos que 'bfs_clusters' es una lista de DataFrames con columns 'theta','r', etc.
    # (aquí no pongo el BFS code para no repetir, pero se asume que lo tienes)

    # EJEMPLO: ajustamos y fusionamos
    final = adjust_and_merge_seeds(
        bfs_clusters,                 # tu lista BFS
        slope_variation_threshold=0.40,  # 40%
        bounding_extrap=0.30            # 30%
    )
    print("Número de grupos finales:", len(final))
    for idx, fg in enumerate(final, 1):
        print(f"Grupo {idx}: slope={fg['slope']:.3f}, intercept={fg['intercept']:.3f}, #pts={len(fg['points'])}")
    
    # Si quisieras graficar en (theta,r), algo así:
    # (asumiendo que df_filtrado son todos los puntos BFS previos)
    import matplotlib.pyplot as plt

    plt.figure(figsize=(9,6))
    plt.scatter(df_filtrado['theta'], df_filtrado['r'], s=3, alpha=0.3, label='Datos BFS')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray']
    for i, group in enumerate(final):
        c = colors[i % len(colors)]
        pts = group['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"Grupo {i+1}")
        if group['slope'] is not None:
            # Graficamos la recta
            tmin = pts['theta'].min()
            tmax = pts['theta'].max()
            t_lin = np.linspace(tmin, tmax, 50)
            r_lin = group['slope']*t_lin + group['intercept']
            plt.plot(t_lin, r_lin, '--', color=c)
    plt.xlabel("theta (grados)")
    plt.ylabel("r")
    plt.legend()
    plt.grid(True)
    plt.show()


In [0]:
bfs_clusters, df_filtrado = generate_bfs_seeds(
    id_halo="17",
    theta_min=50,
    theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    file_prefix='data_rho'
)
print(f"Generadas {len(bfs_clusters)} semillas BFS")


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression

# ====================================
# 1) Funciones para Cargar/Filtrar Datos
# ====================================
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    """
    Carga datos desde CSV: data_rho_<id_halo>_filtered.csv
    Calcula r, theta, y filtra en [theta_min, theta_max].
    """
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    # Filtrar en [theta_min, theta_max]
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    # Asegurarnos de que sean float
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

# ====================================
# 2) Construir Grafo (Rectangular BFS)
# ====================================
def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Crea un "grafo" (lista de adyacencia) donde cada punto se conecta
    a los vecinos si |dtheta| <= theta_diff y |dr| <= r_diff.
    """
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        mask_neighbors = (dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i)
        neighbors = np.where(mask_neighbors)[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    """
    Recorre el 'graph' con BFS y produce clusters (componentes conexas).
    Cada cluster se devuelve como un DataFrame con los puntos correspondientes.
    """
    n = len(graph)
    visited = [False]*n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            clusters.append(cluster_df)
    return clusters

# ====================================
# 3) Subdividir por "gaps"
# ====================================
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Ordena df_cluster por 'mode' (theta/r), 
    y si encuentra saltos > gap_threshold, parte en sub-grupos.
    """
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

# ====================================
# 4) Generar las Semillas BFS
# ====================================
def generate_bfs_seeds(
    id_halo,
    theta_min=50,
    theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    """
    1) Carga y filtra datos en [theta_min, theta_max].
    2) Filtra por 'rho_resta_final_exp' > percentile (quartile_threshold).
    3) Construe BFS rectangular (theta_diff, r_diff).
    4) Extrae clusters y los subdivide por gaps en theta y r.
    5) Retorna (bfs_clusters, df_filtrado).
    """
    # A) Carga y filtra
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    # B) Filtrar por percentil
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    # C) Grafo BFS
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    # D) Subdividir por gaps
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters, df_filtered

# ====================================
# 5) Ajustar rectas y Fusionar
# ====================================
def calculate_pa(slope):
    """Calcula el 'position angle' (grados) a partir de la pendiente."""
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

def fit_line_to_cluster(df_cluster):
    """
    Ajusta recta a df_cluster (si >=2 puntos).
    Retorna slope, intercept, pa, bounding box (t_min,t_max,r_min,r_max).
    """
    if len(df_cluster) < 2:
        return None, None, None, None, None, None, None
    
    X = df_cluster[['theta']].values
    y = df_cluster['r'].values
    
    model = LinearRegression()
    model.fit(X, y)
    slope_ = model.coef_[0]
    intercept_ = model.intercept_
    pa_ = calculate_pa(slope_)
    
    t_min = df_cluster['theta'].min()
    t_max = df_cluster['theta'].max()
    r_min = df_cluster['r'].min()
    r_max = df_cluster['r'].max()
    
    return slope_, intercept_, pa_, t_min, t_max, r_min, r_max

def adjust_and_merge_seeds(
    bfs_clusters,
    slope_variation_threshold=0.40,  # 40% máx variación de pendiente
    bounding_extrap=0.30            # 30% extrapolación en bounding box
):
    """
    1) Ajusta recta a cada cluster BFS (>=2 puntos).
    2) Fusiona recíprocamente mientras:
       - Las bounding boxes extrapoladas se solapen
       - La pendiente combinada no difiera > 'slope_variation_threshold' (40%) 
         de la pendiente de cada uno.

    Retorna lista final de grupos con:
      {
        'points': DataFrame,
        'slope',
        'intercept',
        'pa',
        'theta_min',
        'theta_max',
        'r_min',
        'r_max'
      }
    """
    # 1) Crear lista de grupos iniciales
    groups = []
    for clust_df in bfs_clusters:
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(clust_df)
        grp = {
            'points': clust_df.copy(),
            'slope': slope_,
            'intercept': intercept_,
            'pa': pa_,
            'theta_min': tmin,
            'theta_max': tmax,
            'r_min': rmin,
            'r_max': rmax
        }
        groups.append(grp)
    
    # 2) Función para recalcular recta + bounding box
    def recalc_properties(group):
        dfp = group['points']
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(dfp)
        group['slope'] = slope_
        group['intercept'] = intercept_
        group['pa'] = pa_
        group['theta_min'] = tmin
        group['theta_max'] = tmax
        group['r_min'] = rmin
        group['r_max'] = rmax

    # 3) Chequeo de solapamiento en bounding box extrapolado
    def boxes_overlap(g1, g2, extrap=0.30):
        tmin1, tmax1 = g1['theta_min'], g1['theta_max']
        rmin1, rmax1 = g1['r_min'], g1['r_max']
        tmin2, tmax2 = g2['theta_min'], g2['theta_max']
        rmin2, rmax2 = g2['r_min'], g2['r_max']
        
        if tmin1 is None or tmin2 is None:
            return False
        
        dt1 = (tmax1 - tmin1)
        dr1 = (rmax1 - rmin1)
        dt2 = (tmax2 - tmin2)
        dr2 = (rmax2 - rmin2)
        
        extr_dt1 = dt1*extrap
        extr_dr1 = dr1*extrap
        extr_dt2 = dt2*extrap
        extr_dr2 = dr2*extrap
        
        # bounding box expandido g1
        tmin1e = tmin1 - extr_dt1
        tmax1e = tmax1 + extr_dt1
        rmin1e = rmin1 - extr_dr1
        rmax1e = rmax1 + extr_dr1
        
        # bounding box expandido g2
        tmin2e = tmin2 - extr_dt2
        tmax2e = tmax2 + extr_dt2
        rmin2e = rmin2 - extr_dr2
        rmax2e = rmax2 + extr_dr2
        
        overlap_theta = not (tmax1e < tmin2e or tmax2e < tmin1e)
        overlap_r = not (rmax1e < rmin2e or rmax2e < rmin1e)
        
        return overlap_theta and overlap_r
    
    # 4) Iterar fusiones
    merged_flag = True
    while merged_flag:
        merged_flag = False
        new_groups = []
        i = 0
        while i < len(groups):
            g1 = groups[i]
            j = i+1
            fused_any = False
            while j < len(groups):
                g2 = groups[j]
                # Chequea si ambos tienen slope (>=2 puntos)
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                # Verificar bounding box extrapolado
                if boxes_overlap(g1, g2, bounding_extrap):
                    # Combinar puntos
                    combined = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    # Ajustar recta nueva
                    slope_, intercept_, _, _, _, _, _ = fit_line_to_cluster(combined)
                    if slope_ is not None:
                        # Checar variación vs pend. g1 y g2
                        var1 = abs(slope_ - g1['slope'])/(abs(g1['slope'])+1e-12)
                        var2 = abs(slope_ - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if var1 < slope_variation_threshold and var2 < slope_variation_threshold:
                            # Fusionar
                            g1['points'] = combined
                            recalc_properties(g1)
                            groups.pop(j)
                            fused_any = True
                            merged_flag = True
                            continue
                j += 1
            new_groups.append(g1)
            i += 1
        groups = new_groups
    
    # 5) Filtrar grupos con <2 puntos
    final_groups = []
    for gg in groups:
        if len(gg['points']) >= 2:
            final_groups.append(gg)
    
    return final_groups

# ====================================
# 6) Función de ejemplo para ejecutarlo TODO
# ====================================
def main_example():
    # 1) Generar BFS
    bfs_clusters, df_filtrado = generate_bfs_seeds(
        id_halo="17",           # <--- Cambia a tu halo
        theta_min=20,
        theta_max=360,
        quartile_threshold=0.55,
        theta_diff=3.0,
        r_diff=0.5,
        gap_threshold_theta=2.0,
        gap_threshold_r=2.0,
        file_prefix='data_rho'  # Ajusta si tu CSV se llama distinto
    )
    print(f"BFS inicial: {len(bfs_clusters)} grupos")

    # 2) Ajustar & Fusionar
    final_groups = adjust_and_merge_seeds(
        bfs_clusters,
        slope_variation_threshold=0.40,  # 40% máx variación de pendiente
        bounding_extrap=0.30            # 30% de extrapolación bounding box
    )
    print(f"Grupos finales tras fusiones: {len(final_groups)}")
    
    # 3) Graficar en (theta, r)
    plt.figure(figsize=(10,6))
    # Todos los puntos BFS en gris
    plt.scatter(df_filtrado['theta'], df_filtrado['r'], s=3, alpha=0.3, label='Datos BFS')
    
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray']
    for i, group in enumerate(final_groups):
        c = colors[i % len(colors)]
        pts = group['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"Grupo {i+1}")
        if group['slope'] is not None:
            tmin = pts['theta'].min()
            tmax = pts['theta'].max()
            t_lin = np.linspace(tmin, tmax, 50)
            r_lin = group['slope']*t_lin + group['intercept']
            plt.plot(t_lin, r_lin, '--', color=c)
    plt.xlabel("theta (grados)")
    plt.ylabel("r")
    plt.legend(loc='upper right')
    plt.title("BFS + Ajuste y Fusión de Semillas")
    plt.grid(True)
    plt.show()

# ====================================
# Para ejecutar, haz:
# main_example()
# ====================================


In [0]:
main_example()


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression

# ====================================
# 1) Funciones para Cargar/Filtrar Datos
# ====================================
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

# ====================================
# 2) Construir Grafo (Rectangular BFS)
# ====================================
def build_graph_rectangular(df_points, theta_diff, r_diff):
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        mask_neighbors = (dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i)
        neighbors = np.where(mask_neighbors)[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False]*n
    clusters = []
    from collections import deque
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            clusters.append(cluster_df)
    return clusters

# ====================================
# 3) Subdividir por "gaps"
# ====================================
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

# ====================================
# 4) Generar Semillas BFS
# ====================================
def generate_bfs_seeds(
    id_halo,
    theta_min=50,
    theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters, df_filtered

# ====================================
# 5) Ajustar rectas y Fusionar
# ====================================
def calculate_pa(slope):
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return None, None, None, None, None, None, None
    X = df_cluster[['theta']].values
    y = df_cluster['r'].values
    model = LinearRegression()
    model.fit(X, y)
    slope_ = model.coef_[0]
    intercept_ = model.intercept_
    pa_ = calculate_pa(slope_)
    t_min = df_cluster['theta'].min()
    t_max = df_cluster['theta'].max()
    r_min = df_cluster['r'].min()
    r_max = df_cluster['r'].max()
    return slope_, intercept_, pa_, t_min, t_max, r_min, r_max

def adjust_and_merge_seeds(
    bfs_clusters,
    slope_variation_threshold=0.40,
    bounding_extrap=0.30
):
    groups = []
    for clust_df in bfs_clusters:
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(clust_df)
        grp = {
            'points': clust_df.copy(),
            'slope': slope_,
            'intercept': intercept_,
            'pa': pa_,
            'theta_min': tmin,
            'theta_max': tmax,
            'r_min': rmin,
            'r_max': rmax
        }
        groups.append(grp)
    
    def recalc_properties(group):
        dfp = group['points']
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(dfp)
        group['slope'] = slope_
        group['intercept'] = intercept_
        group['pa'] = pa_
        group['theta_min'] = tmin
        group['theta_max'] = tmax
        group['r_min'] = rmin
        group['r_max'] = rmax

    def boxes_overlap(g1, g2, extrap=0.30):
        tmin1, tmax1 = g1['theta_min'], g1['theta_max']
        rmin1, rmax1 = g1['r_min'], g1['r_max']
        tmin2, tmax2 = g2['theta_min'], g2['theta_max']
        rmin2, rmax2 = g2['r_min'], g2['r_max']
        if tmin1 is None or tmin2 is None:
            return False
        dt1 = (tmax1 - tmin1)
        dr1 = (rmax1 - rmin1)
        dt2 = (tmax2 - tmin2)
        dr2 = (rmax2 - rmin2)
        extr_dt1 = dt1*extrap
        extr_dr1 = dr1*extrap
        extr_dt2 = dt2*extrap
        extr_dr2 = dr2*extrap
        
        tmin1e = tmin1 - extr_dt1
        tmax1e = tmax1 + extr_dt1
        rmin1e = rmin1 - extr_dr1
        rmax1e = rmax1 + extr_dr1
        
        tmin2e = tmin2 - extr_dt2
        tmax2e = tmax2 + extr_dt2
        rmin2e = rmin2 - extr_dr2
        rmax2e = rmax2 + extr_dr2
        
        overlap_theta = not (tmax1e < tmin2e or tmax2e < tmin1e)
        overlap_r = not (rmax1e < rmin2e or rmax2e < rmin1e)
        return overlap_theta and overlap_r
    
    merged_flag = True
    while merged_flag:
        merged_flag = False
        new_groups = []
        i = 0
        while i < len(groups):
            g1 = groups[i]
            j = i+1
            fused_any = False
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    combined = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    slope_, intercept_, _, _, _, _, _ = fit_line_to_cluster(combined)
                    if slope_ is not None:
                        var1 = abs(slope_ - g1['slope'])/(abs(g1['slope'])+1e-12)
                        var2 = abs(slope_ - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if var1 < slope_variation_threshold and var2 < slope_variation_threshold:
                            g1['points'] = combined
                            recalc_properties(g1)
                            groups.pop(j)
                            fused_any = True
                            merged_flag = True
                            continue
                j += 1
            new_groups.append(g1)
            i += 1
        groups = new_groups
    
    final_groups = []
    for gg in groups:
        if len(gg['points']) >= 2:
            final_groups.append(gg)
    return final_groups

# ====================================
# 6) [Nuevo] Validar dispersión y re-procesar
# ====================================
def validate_dispersion_and_reprocess(
    groups,
    dispersion_threshold=2.0,   # umbral de desviación std (ejemplo)
    reproc_theta_diff=2.0,      # BFS para reprocesar
    reproc_r_diff=0.5
):
    """
    - Verifica la 'dispersión' (std de los residuos de cada grupo).
    - Si un grupo excede 'dispersion_threshold', se 'rompe':
       * Tomamos sus puntos
       * Reaplicamos BFS (o un mini-proceso) a ese subconjunto
       * Ajustamos rectas otra vez
    - Devuelve la lista final de grupos (los que no se rompieron + 
      los subgrupos reagrupados).
    """
    # 1) Separar grupos "buenos" de grupos "a re-procesar"
    groups_ok = []
    groups_toreprocess = []
    
    for grp in groups:
        slope = grp['slope']
        intercept = grp['intercept']
        pts = grp['points']
        if slope is None or len(pts) < 2:
            # Grupo muy pequeño -> considerarlo "ok" sin dispersión
            groups_ok.append(grp)
            continue
        
        # Calcular residuos
        predicted_r = slope*pts['theta'] + intercept
        residuals = pts['r'] - predicted_r
        std_resid = residuals.std()  # desviación estándar
        # Checar si std_resid > threshold
        if std_resid > dispersion_threshold:
            groups_toreprocess.append(grp)
        else:
            groups_ok.append(grp)
    
    # 2) Re-procesar los que superaron la dispersión
    #    Por ejemplo, usando BFS de nuevo
    new_groups = []
    for grp in groups_toreprocess:
        # Tomar sus puntos:
        subdf = grp['points'].copy()
        subdf.reset_index(drop=True, inplace=True)
        # BFS con reproc_theta_diff, reproc_r_diff
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        clusters_sub = bfs_components(graph, subdf)
        
        # A) Ajustamos recta a cada mini-cluster (sin fusión en este ejemplo)
        #    O podrías re-llamar a adjust_and_merge_seeds con slope_variation_threshold,
        #    si quieres un pipeline más largo.
        
        for csub in clusters_sub:
            slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(csub)
            new_groups.append({
                'points': csub,
                'slope': slope_,
                'intercept': intercept_,
                'pa': pa_,
                'theta_min': tmin,
                'theta_max': tmax,
                'r_min': rmin,
                'r_max': rmax
            })
    
    # 3) Unir "groups_ok" + "new_groups" en la lista final
    final_res = groups_ok + new_groups
    
    return final_res


# ====================================
# 7) Función principal que muestra TODO
# ====================================
def main_example():
    # 1) Generar BFS
    bfs_clusters, df_filtrado = generate_bfs_seeds(
        id_halo="17",
        theta_min=20,
        theta_max=350,
        quartile_threshold=0.155,
        theta_diff=3.0,
        r_diff=0.5,
        gap_threshold_theta=2.0,
        gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs_clusters)} grupos")
    
    # 2) Ajustar y fusionar
    merged_groups = adjust_and_merge_seeds(
        bfs_clusters,
        slope_variation_threshold=0.30,
        bounding_extrap=0.35
    )
    print(f"Grupos tras fusión inicial: {len(merged_groups)}")
    
    # 3) Validar dispersión y re-procesar grupos con alta dispersión
    #    Por ejemplo, dispersion_threshold=2.0
    validated_groups = validate_dispersion_and_reprocess(
        merged_groups,
        dispersion_threshold=1.80, # si la std del residuo >2 -> re-proceso BFS
        reproc_theta_diff=1.25,
        reproc_r_diff=0.4
    )
    print(f"Grupos tras validación dispersión: {len(validated_groups)}")
    
    # 4) Graficar
    plt.figure(figsize=(10,6))
    plt.scatter(df_filtrado['theta'], df_filtrado['r'], s=3, alpha=0.3, label='Datos BFS')
    
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray']
    for i, group in enumerate(validated_groups):
        c = colors[i % len(colors)]
        pts = group['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"G{i+1}")
        if group['slope'] is not None:
            tmin = pts['theta'].min()
            tmax = pts['theta'].max()
            t_lin = np.linspace(tmin, tmax, 50)
            r_lin = group['slope']*t_lin + group['intercept']
            plt.plot(t_lin, r_lin, '--', color=c)
    plt.xlabel("theta (grados)")
    plt.ylabel("r")
    plt.legend(loc='upper right')
    plt.title("BFS + Fusión + Validación de Dispersión (reprocesado)")
    plt.grid(True)
    plt.show()


# ====================================
# Ejecución:
# main_example()
# ====================================


In [0]:
main_example()

## Halo 17, procesamiendo y agregacion

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression

# ====================================
# 1) Funciones para Cargar/Filtrar Datos
# ====================================
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

# ====================================
# 2) Construir Grafo (Rectangular BFS)
# ====================================
def build_graph_rectangular(df_points, theta_diff, r_diff):
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        mask_neighbors = (dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i)
        neighbors = np.where(mask_neighbors)[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False]*n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            clusters.append(cluster_df)
    return clusters

# ====================================
# 3) Subdividir por "gaps"
# ====================================
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

# ====================================
# 4) Generar Semillas BFS
# ====================================
def generate_bfs_seeds(
    id_halo,
    theta_min=50,
    theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters, df_filtered

# ====================================
# 5) Ajustar rectas y Fusionar
# ====================================
def calculate_pa(slope):
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return None, None, None, None, None, None, None
    X = df_cluster[['theta']].values
    y = df_cluster['r'].values
    model = LinearRegression()
    model.fit(X, y)
    slope_ = model.coef_[0]
    intercept_ = model.intercept_
    pa_ = calculate_pa(slope_)
    t_min = df_cluster['theta'].min()
    t_max = df_cluster['theta'].max()
    r_min = df_cluster['r'].min()
    r_max = df_cluster['r'].max()
    return slope_, intercept_, pa_, t_min, t_max, r_min, r_max

def adjust_and_merge_seeds(
    bfs_clusters,
    slope_variation_threshold=0.40,
    bounding_extrap=0.30
):
    groups = []
    for clust_df in bfs_clusters:
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(clust_df)
        grp = {
            'points': clust_df.copy(),
            'slope': slope_,
            'intercept': intercept_,
            'pa': pa_,
            'theta_min': tmin,
            'theta_max': tmax,
            'r_min': rmin,
            'r_max': rmax
        }
        groups.append(grp)
    
    def recalc_properties(group):
        dfp = group['points']
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(dfp)
        group['slope'] = slope_
        group['intercept'] = intercept_
        group['pa'] = pa_
        group['theta_min'] = tmin
        group['theta_max'] = tmax
        group['r_min'] = rmin
        group['r_max'] = rmax

    def boxes_overlap(g1, g2, extrap=0.30):
        tmin1, tmax1 = g1['theta_min'], g1['theta_max']
        rmin1, rmax1 = g1['r_min'], g1['r_max']
        tmin2, tmax2 = g2['theta_min'], g2['theta_max']
        rmin2, rmax2 = g2['r_min'], g2['r_max']
        if tmin1 is None or tmin2 is None:
            return False
        dt1 = (tmax1 - tmin1)
        dr1 = (rmax1 - rmin1)
        dt2 = (tmax2 - tmin2)
        dr2 = (rmax2 - rmin2)
        extr_dt1 = dt1*extrap
        extr_dr1 = dr1*extrap
        extr_dt2 = dt2*extrap
        extr_dr2 = dr2*extrap
        
        tmin1e = tmin1 - extr_dt1
        tmax1e = tmax1 + extr_dt1
        rmin1e = rmin1 - extr_dr1
        rmax1e = rmax1 + extr_dr1
        
        tmin2e = tmin2 - extr_dt2
        tmax2e = tmax2 + extr_dt2
        rmin2e = rmin2 - extr_dr2
        rmax2e = rmax2 + extr_dr2
        
        overlap_theta = not (tmax1e < tmin2e or tmax2e < tmin1e)
        overlap_r = not (rmax1e < rmin2e or rmax2e < rmin1e)
        return overlap_theta and overlap_r
    
    merged_flag = True
    while merged_flag:
        merged_flag = False
        new_groups = []
        i = 0
        while i < len(groups):
            g1 = groups[i]
            j = i+1
            fused_any = False
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    combined = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    slope_, intercept_, _, _, _, _, _ = fit_line_to_cluster(combined)
                    if slope_ is not None:
                        var1 = abs(slope_ - g1['slope'])/(abs(g1['slope'])+1e-12)
                        var2 = abs(slope_ - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if var1 < slope_variation_threshold and var2 < slope_variation_threshold:
                            g1['points'] = combined
                            recalc_properties(g1)
                            groups.pop(j)
                            fused_any = True
                            merged_flag = True
                            continue
                j += 1
            new_groups.append(g1)
            i += 1
        groups = new_groups
    
    final_groups = []
    for gg in groups:
        if len(gg['points']) >= 2:
            final_groups.append(gg)
    return final_groups

# ====================================
# 6) Validar dispersión y re-procesar (opcional)
# ====================================
def validate_dispersion_and_reprocess(
    groups,
    dispersion_threshold=2.0,   # umbral de desviación std (ejemplo)
    reproc_theta_diff=2.0,      # BFS para reprocesar
    reproc_r_diff=0.5
):
    groups_ok = []
    groups_toreprocess = []
    
    for grp in groups:
        slope = grp['slope']
        intercept = grp['intercept']
        pts = grp['points']
        if slope is None or len(pts) < 2:
            groups_ok.append(grp)
            continue
        
        # Calcular std de residuo
        predicted_r = slope*pts['theta'] + intercept
        residuals = pts['r'] - predicted_r
        std_resid = residuals.std()
        if std_resid > dispersion_threshold:
            groups_toreprocess.append(grp)
        else:
            groups_ok.append(grp)
    
    # Reprocesar los grupos con alta dispersión
    new_groups = []
    for grp in groups_toreprocess:
        subdf = grp['points'].copy()
        subdf.reset_index(drop=True, inplace=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        clusters_sub = bfs_components(graph, subdf)
        
        for csub in clusters_sub:
            slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(csub)
            new_groups.append({
                'points': csub,
                'slope': slope_,
                'intercept': intercept_,
                'pa': pa_,
                'theta_min': tmin,
                'theta_max': tmax,
                'r_min': rmin,
                'r_max': rmax
            })
    
    final_res = groups_ok + new_groups
    return final_res

# =========================================
# 7) Nueva fase: Elegir grupos con >= 60 puntos y graficar con bounding box extrapolado
# =========================================
def plot_final_segments_over_60(groups, df_all, bounding_extrap=0.30):
    """
    - Toma los grupos finales
    - Filtra los que tienen >= 60 puntos
    - Grafica en un nuevo plot:
        * Los puntos de cada grupo
        * La recta
        * Un rectángulo que muestre el bounding box extrapolado un 30%
    """
    # 1. Filtrar
    large_groups = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos con >= 60 puntos: {len(large_groups)}")
    
    # 2. Crear figura
    plt.figure(figsize=(10, 6))
    
    # Fondo con df_all en gris
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')
    
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    
    for i, grp in enumerate(large_groups):
        c = colors[i % len(colors)]
        pts = grp['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"Grupo_{i+1} ({len(pts)} pts)")
        
        slope_ = grp['slope']
        intercept_ = grp['intercept']
        if slope_ is not None:
            tmin = pts['theta'].min()
            tmax = pts['theta'].max()
            t_lin = np.linspace(tmin, tmax, 100)
            r_lin = slope_*t_lin + intercept_
            plt.plot(t_lin, r_lin, '--', color=c)
        
        # 3. Dibujar bounding box extrapolado 30%
        # bounding box actual
        tmin_bb = grp['theta_min']
        tmax_bb = grp['theta_max']
        rmin_bb = grp['r_min']
        rmax_bb = grp['r_max']
        
        # Calcular ancho/alto
        dt = (tmax_bb - tmin_bb)
        dr = (rmax_bb - rmin_bb)
        dt_ext = dt * bounding_extrap
        dr_ext = dr * bounding_extrap
        
        # bounding box expandido
        tmin_e = tmin_bb - dt_ext
        tmax_e = tmax_bb + dt_ext
        rmin_e = rmin_bb - dr_ext
        rmax_e = rmax_bb + dr_ext
        
        # Dibujar rectángulo con plt.plot (4 lados)
        # forma => [(x1, y1), (x1, y2), (x2, y2), (x2, y1), (x1, y1)]
        # donde x=theta, y=r
        box_x = [tmin_e, tmin_e, tmax_e, tmax_e, tmin_e]
        box_y = [rmin_e, rmax_e, rmax_e, rmin_e, rmin_e]
        plt.plot(box_x, box_y, color=c, linewidth=1.2)
    
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title("Grupos con >= 60 puntos y su bounding box extrapolado")
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

# =========================================
# 8) Función principal: ejemplo
# =========================================
def main_example():
    # Paso 1: generar BFS
    bfs_clusters, df_filtrado = generate_bfs_seeds(
        id_halo="17",
        theta_min=20,
        theta_max=350,
        quartile_threshold=0.155,
        theta_diff=3.0,
        r_diff=0.5,
        gap_threshold_theta=2.0,
        gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs_clusters)} grupos")
    
    # Paso 2: ajustar/fusionar
    merged_groups = adjust_and_merge_seeds(
        bfs_clusters,
        slope_variation_threshold=0.30,
        bounding_extrap=0.35
    )
    print(f"Grupos tras fusión inicial: {len(merged_groups)}")
    
    # Paso 3: validación de dispersión (opcional)
    validated_groups = validate_dispersion_and_reprocess(
        merged_groups,
        dispersion_threshold=1.80,
        reproc_theta_diff=1.25,
        reproc_r_diff=0.4
    )
    print(f"Grupos tras validación dispersión: {len(validated_groups)}")
    
    # Plot de esos grupos en general
    plt.figure(figsize=(8,6))
    plt.scatter(df_filtrado['theta'], df_filtrado['r'], s=3, alpha=0.3, label='Datos BFS')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray']
    for i, group in enumerate(validated_groups):
        c = colors[i % len(colors)]
        pts = group['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"G{i+1} ({len(pts)}pts)")
        if group['slope'] is not None:
            tmin = pts['theta'].min()
            tmax = pts['theta'].max()
            t_lin = np.linspace(tmin, tmax, 100)
            r_lin = group['slope']*t_lin + group['intercept']
            plt.plot(t_lin, r_lin, '--', color=c)
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    #plt.legend(loc='best')
    plt.title("Grupos finales (validados)")
    plt.grid(True)
    plt.show()
    
    # Paso 4: ahora filtrar los grupos con >=60 puntos y graficarlos aparte
    plot_final_segments_over_60(validated_groups, df_filtrado, bounding_extrap=0.10)


main_example()



In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression

# ====================================
# 1) Funciones para Cargar/Filtrar Datos
# ====================================
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

# ====================================
# 2) Construir Grafo (Rectangular BFS)
# ====================================
def build_graph_rectangular(df_points, theta_diff, r_diff):
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        mask_neighbors = (dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i)
        neighbors = np.where(mask_neighbors)[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False]*n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            clusters.append(cluster_df)
    return clusters

# ====================================
# 3) Subdividir por "gaps"
# ====================================
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

# ====================================
# 4) Generar Semillas BFS
# ====================================
def generate_bfs_seeds(
    id_halo,
    theta_min=50,
    theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters, df_filtered

# ====================================
# 5) Ajustar rectas y Fusionar
# ====================================
def calculate_pa(slope):
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return None, None, None, None, None, None, None
    X = df_cluster[['theta']].values
    y = df_cluster['r'].values
    model = LinearRegression()
    model.fit(X, y)
    slope_ = model.coef_[0]
    intercept_ = model.intercept_
    pa_ = calculate_pa(slope_)
    t_min = df_cluster['theta'].min()
    t_max = df_cluster['theta'].max()
    r_min = df_cluster['r'].min()
    r_max = df_cluster['r'].max()
    return slope_, intercept_, pa_, t_min, t_max, r_min, r_max

def adjust_and_merge_seeds(
    bfs_clusters,
    slope_variation_threshold=0.40,
    bounding_extrap=0.30
):
    groups = []
    for clust_df in bfs_clusters:
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(clust_df)
        grp = {
            'points': clust_df.copy(),
            'slope': slope_,
            'intercept': intercept_,
            'pa': pa_,
            'theta_min': tmin,
            'theta_max': tmax,
            'r_min': rmin,
            'r_max': rmax
        }
        groups.append(grp)
    
    def recalc_properties(group):
        dfp = group['points']
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(dfp)
        group['slope'] = slope_
        group['intercept'] = intercept_
        group['pa'] = pa_
        group['theta_min'] = tmin
        group['theta_max'] = tmax
        group['r_min'] = rmin
        group['r_max'] = rmax

    def boxes_overlap(g1, g2, extrap=0.30):
        tmin1, tmax1 = g1['theta_min'], g1['theta_max']
        rmin1, rmax1 = g1['r_min'], g1['r_max']
        tmin2, tmax2 = g2['theta_min'], g2['theta_max']
        rmin2, rmax2 = g2['r_min'], g2['r_max']
        if tmin1 is None or tmin2 is None:
            return False
        dt1 = (tmax1 - tmin1)
        dr1 = (rmax1 - rmin1)
        dt2 = (tmax2 - tmin2)
        dr2 = (rmax2 - rmin2)
        extr_dt1 = dt1*extrap
        extr_dr1 = dr1*extrap
        extr_dt2 = dt2*extrap
        extr_dr2 = dr2*extrap
        
        tmin1e = tmin1 - extr_dt1
        tmax1e = tmax1 + extr_dt1
        rmin1e = rmin1 - extr_dr1
        rmax1e = rmax1 + extr_dr1
        
        tmin2e = tmin2 - extr_dt2
        tmax2e = tmax2 + extr_dt2
        rmin2e = rmin2 - extr_dr2
        rmax2e = rmax2 + extr_dr2
        
        overlap_theta = not (tmax1e < tmin2e or tmax2e < tmin1e)
        overlap_r = not (rmax1e < rmin2e or rmax2e < rmin1e)
        return overlap_theta and overlap_r
    
    merged_flag = True
    while merged_flag:
        merged_flag = False
        new_groups = []
        i = 0
        while i < len(groups):
            g1 = groups[i]
            j = i+1
            fused_any = False
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    combined = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    slope_, intercept_, _, _, _, _, _ = fit_line_to_cluster(combined)
                    if slope_ is not None:
                        var1 = abs(slope_ - g1['slope'])/(abs(g1['slope'])+1e-12)
                        var2 = abs(slope_ - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if var1 < slope_variation_threshold and var2 < slope_variation_threshold:
                            g1['points'] = combined
                            recalc_properties(g1)
                            groups.pop(j)
                            fused_any = True
                            merged_flag = True
                            continue
                j += 1
            new_groups.append(g1)
            i += 1
        groups = new_groups
    
    final_groups = []
    for gg in groups:
        if len(gg['points']) >= 2:
            final_groups.append(gg)
    return final_groups

# ====================================
# 6) Validar dispersión y re-procesar (opcional)
# ====================================
def validate_dispersion_and_reprocess(
    groups,
    dispersion_threshold=2.0,   # umbral de desviación std (ejemplo)
    reproc_theta_diff=2.0,      # BFS para reprocesar
    reproc_r_diff=0.5
):
    groups_ok = []
    groups_toreprocess = []
    
    for grp in groups:
        slope = grp['slope']
        intercept = grp['intercept']
        pts = grp['points']
        if slope is None or len(pts) < 2:
            groups_ok.append(grp)
            continue
        
        predicted_r = slope*pts['theta'] + intercept
        residuals = pts['r'] - predicted_r
        std_resid = residuals.std()
        if std_resid > dispersion_threshold:
            groups_toreprocess.append(grp)
        else:
            groups_ok.append(grp)
    
    new_groups = []
    for grp in groups_toreprocess:
        subdf = grp['points'].copy()
        subdf.reset_index(drop=True, inplace=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        clusters_sub = bfs_components(graph, subdf)
        
        for csub in clusters_sub:
            slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(csub)
            new_groups.append({
                'points': csub,
                'slope': slope_,
                'intercept': intercept_,
                'pa': pa_,
                'theta_min': tmin,
                'theta_max': tmax,
                'r_min': rmin,
                'r_max': rmax
            })
    
    final_res = groups_ok + new_groups
    return final_res

# =========================================
# 7) Nueva función: Graficar grupos >= 60 puntos con la recta extrapolada un 30%
# =========================================
def plot_final_segments_over_60_with_line_extrapolation(groups, df_all, line_extrap=0.3):
    """
    - Filtra grupos con >= 60 puntos
    - Grafica cada uno en un nuevo plot (theta vs r)
    - En lugar de dibujar bounding box, extiende la recta un 'line_extrap'% 
      por debajo y por encima del rango de theta.
    """
    # 1) Filtrar los grupos grandes
    large_groups = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos con >= 60 puntos: {len(large_groups)}")

    # 2) Crear figura
    plt.figure(figsize=(10, 6))

    # Fondo con df_all en gris
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')

    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']

    for i, grp in enumerate(large_groups):
        c = colors[i % len(colors)]
        pts = grp['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c,
                    label=f"Grupo_{i+1} ({len(pts)} pts)")

        slope_ = grp['slope']
        intercept_ = grp['intercept']
        if slope_ is not None and grp['theta_min'] is not None and grp['theta_max'] is not None:
            # rango de theta del grupo
            theta_min = grp['theta_min']
            theta_max = grp['theta_max']
            dtheta = (theta_max - theta_min)
            # Extender un 30%
            theta_min_ext = theta_min - line_extrap*dtheta
            theta_max_ext = theta_max + line_extrap*dtheta

            # Graficar la línea en el rango extendido
            t_lin = np.linspace(theta_min_ext, theta_max_ext, 200)
            r_lin = slope_*t_lin + intercept_
            plt.plot(t_lin, r_lin, '--', color=c)
            plt.plot(t_lin, r_lin, '-.', color='k', alpha=0.3)

    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(f"Grupos >= 60 puntos con recta extrapolada {int(line_extrap*100)}%")
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

# =========================================
# 8) Función principal: ejemplo
# =========================================
def main_example():
    # A) Generar BFS
    bfs_clusters, df_filtrado = generate_bfs_seeds(
        id_halo="17",
        theta_min=0,
        theta_max=360,
        quartile_threshold=0.155,
        theta_diff=3.0,
        r_diff=0.5,
        gap_threshold_theta=2.0,
        gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs_clusters)} grupos")

    # B) Ajustar y fusionar
    merged_groups = adjust_and_merge_seeds(
        bfs_clusters,
        slope_variation_threshold=0.30,
        bounding_extrap=0.35
    )
    print(f"Grupos tras fusión inicial: {len(merged_groups)}")

    # C) Validar dispersión y re-procesar
    validated_groups = validate_dispersion_and_reprocess(
        merged_groups,
        dispersion_threshold=1.80,
        reproc_theta_diff=1.25,
        reproc_r_diff=0.4
    )
    print(f"Grupos tras validación dispersión: {len(validated_groups)}")



    # D) Grafico global (todos los grupos) en (theta, r)
    plt.figure(figsize=(10,6))
    plt.scatter(df_filtrado['theta'], df_filtrado['r'], s=3, alpha=0.3, label='Datos BFS')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray']
    for i, group in enumerate(validated_groups):
        c = colors[i % len(colors)]
        pts = group['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"G{i+1} ({len(pts)}pts)")
        if group['slope'] is not None:
            tmin = pts['theta'].min()
            tmax = pts['theta'].max()
            t_lin = np.linspace(tmin, tmax, 100)
            r_lin = group['slope']*t_lin + group['intercept']
            plt.plot(t_lin, r_lin, '--', color=c)

    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    #plt.legend(loc='best')
    plt.title("Grupos finales (validados)")
    plt.grid(True)
    plt.show()

    # E) Ahora, filtrar los grupos con >=60 puntos y graficar su recta extrapolada
    plot_final_segments_over_60_with_line_extrapolation(
        validated_groups,
        df_filtrado,
        line_extrap=0.15  # 30% de extrapolación
    )


    def plot_final_segments_over_60_polar(groups, df_all, line_extrap=0.15):
        """
        Grafica en coordenadas polares (r, theta):
        - Filtra los grupos con >= 60 puntos.
        - Para cada grupo, crea una curva r = slope*theta_deg + intercept,
        donde theta_deg se extrapola un 'line_extrap'% a cada lado
        y se convierte a radianes para dibujar.

        Param:
        groups: lista de dicts con campos:
                'points' (DataFrame con 'theta','r'),
                'slope','intercept','theta_min','theta_max', ...
        df_all: DataFrame con las columnas 'theta','r' (theta en grados).
        line_extrap: factor de extrapolación (e.g. 0.3 = 30%)
        """
        # 1) Filtrar grupos con >=60 puntos
        large_groups = [g for g in groups if len(g['points']) >= 60]
        print(f"Grupos con >= 60 puntos: {len(large_groups)}")

        # 2) Crear figura con proyección polar
        fig = plt.figure(figsize=(9, 8))
        ax = plt.subplot(111, projection='polar')

        # 3) Fondo con df_all en gris (convirtiendo theta a radianes)
        theta_all_rad = np.radians(df_all['theta'].values)
        r_all = df_all['r'].values
        ax.scatter(theta_all_rad, r_all, s=3, alpha=0.3, color='gray', label='Datos base')

        # 4) Graficar cada grupo
        colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
        for i, grp in enumerate(large_groups):
            c = colors[i % len(colors)]
            pts = grp['points']

            # a) scatter de puntos del grupo (convertir theta->rad)
            theta_pts_rad = np.radians(pts['theta'].values)
            r_pts = pts['r'].values
            label_txt = f"Grupo_{i+1} ({len(pts)} pts)"
            ax.scatter(theta_pts_rad, r_pts, s=10, color=c, label=label_txt)

            # b) línea extrapolada
            slope_ = grp['slope']
            intercept_ = grp['intercept']
            tmin_deg = grp.get('theta_min', None)
            tmax_deg = grp.get('theta_max', None)
            if slope_ is not None and tmin_deg is not None and tmax_deg is not None:
                dt = tmax_deg - tmin_deg
                tmin_ext = tmin_deg - line_extrap*dt
                tmax_ext = tmax_deg + line_extrap*dt

                # Generamos 200 puntos en grados
                theta_deg_line = np.linspace(tmin_ext, tmax_ext, 200)
                # Calculamos r en base a slope*(theta_deg) + intercept
                r_line = slope_*theta_deg_line + intercept_
                # Convertimos a radianes para plot
                theta_rad_line = np.radians(theta_deg_line)

                ax.plot(theta_rad_line, r_line, '--', color=c)

        # 5) Ajustes finales en polar
        ax.set_theta_zero_location("E")   # opcional: 0° a la derecha (eje x)
        ax.set_theta_direction(-1)        # opcional: ángulos crecen en sentido horario
        ax.set_rlabel_position(0)         # etiquetas de r en 0 grados
        ax.set_title("Grupos >= 60 puntos (coordenadas polares, 30% extrapolación)", y=1.05)
        ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0))
        plt.show()

    plot_final_segments_over_60_polar(
    validated_groups,
    df_filtrado,
    line_extrap=0.15  # 30% de extrapolación
)

main_example()



In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression

# ====================================
# 1) Funciones para Cargar/Filtrar Datos
# ====================================
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

# ====================================
# 2) Construir Grafo (Rectangular BFS)
# ====================================
def build_graph_rectangular(df_points, theta_diff, r_diff):
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        mask_neighbors = (dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i)
        neighbors = np.where(mask_neighbors)[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False]*n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            clusters.append(cluster_df)
    return clusters

# ====================================
# 3) Subdividir por "gaps"
# ====================================
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

# ====================================
# 4) Generar Semillas BFS
# ====================================
def generate_bfs_seeds(
    id_halo,
    theta_min=0,
    theta_max=450,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters, df_filtered

# ====================================
# 5) Ajustar rectas y Fusionar
# ====================================
def calculate_pa(slope):
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return None, None, None, None, None, None, None
    X = df_cluster[['theta']].values
    y = df_cluster['r'].values
    model = LinearRegression()
    model.fit(X, y)
    slope_ = model.coef_[0]
    intercept_ = model.intercept_
    pa_ = calculate_pa(slope_)
    t_min = df_cluster['theta'].min()
    t_max = df_cluster['theta'].max()
    r_min = df_cluster['r'].min()
    r_max = df_cluster['r'].max()
    return slope_, intercept_, pa_, t_min, t_max, r_min, r_max

def adjust_and_merge_seeds(
    bfs_clusters,
    slope_variation_threshold=0.40,
    bounding_extrap=0.30
):
    groups = []
    for clust_df in bfs_clusters:
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(clust_df)
        grp = {
            'points': clust_df.copy(),
            'slope': slope_,
            'intercept': intercept_,
            'pa': pa_,
            'theta_min': tmin,
            'theta_max': tmax,
            'r_min': rmin,
            'r_max': rmax
        }
        groups.append(grp)
    
    def recalc_properties(group):
        dfp = group['points']
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(dfp)
        group['slope'] = slope_
        group['intercept'] = intercept_
        group['pa'] = pa_
        group['theta_min'] = tmin
        group['theta_max'] = tmax
        group['r_min'] = rmin
        group['r_max'] = rmax

    def boxes_overlap(g1, g2, extrap=0.30):
        tmin1, tmax1 = g1['theta_min'], g1['theta_max']
        rmin1, rmax1 = g1['r_min'], g1['r_max']
        tmin2, tmax2 = g2['theta_min'], g2['theta_max']
        rmin2, rmax2 = g2['r_min'], g2['r_max']
        if tmin1 is None or tmin2 is None:
            return False
        dt1 = (tmax1 - tmin1)
        dr1 = (rmax1 - rmin1)
        dt2 = (tmax2 - tmin2)
        dr2 = (rmax2 - rmin2)
        extr_dt1 = dt1*extrap
        extr_dr1 = dr1*extrap
        extr_dt2 = dt2*extrap
        extr_dr2 = dr2*extrap
        
        tmin1e = tmin1 - extr_dt1
        tmax1e = tmax1 + extr_dt1
        rmin1e = rmin1 - extr_dr1
        rmax1e = rmax1 + extr_dr1
        
        tmin2e = tmin2 - extr_dt2
        tmax2e = tmax2 + extr_dt2
        rmin2e = rmin2 - extr_dr2
        rmax2e = rmax2 + extr_dr2
        
        overlap_theta = not (tmax1e < tmin2e or tmax2e < tmin1e)
        overlap_r = not (rmax1e < rmin2e or rmax2e < rmin1e)
        return overlap_theta and overlap_r
    
    merged_flag = True
    while merged_flag:
        merged_flag = False
        new_groups = []
        i = 0
        while i < len(groups):
            g1 = groups[i]
            j = i+1
            fused_any = False
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    combined = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    slope_, intercept_, _, _, _, _, _ = fit_line_to_cluster(combined)
                    if slope_ is not None:
                        var1 = abs(slope_ - g1['slope'])/(abs(g1['slope'])+1e-12)
                        var2 = abs(slope_ - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if var1 < slope_variation_threshold and var2 < slope_variation_threshold:
                            g1['points'] = combined
                            recalc_properties(g1)
                            groups.pop(j)
                            fused_any = True
                            merged_flag = True
                            continue
                j += 1
            new_groups.append(g1)
            i += 1
        groups = new_groups
    
    final_groups = []
    for gg in groups:
        if len(gg['points']) >= 2:
            final_groups.append(gg)
    return final_groups

# ====================================
# 6) Validar dispersión y re-procesar (opcional)
# ====================================
def validate_dispersion_and_reprocess(
    groups,
    dispersion_threshold=2.0,   # umbral de desviación std (ejemplo)
    reproc_theta_diff=2.0,      # BFS para reprocesar
    reproc_r_diff=0.5
):
    groups_ok = []
    groups_toreprocess = []
    
    for grp in groups:
        slope = grp['slope']
        intercept = grp['intercept']
        pts = grp['points']
        if slope is None or len(pts) < 2:
            groups_ok.append(grp)
            continue
        
        predicted_r = slope*pts['theta'] + intercept
        residuals = pts['r'] - predicted_r
        std_resid = residuals.std()
        if std_resid > dispersion_threshold:
            groups_toreprocess.append(grp)
        else:
            groups_ok.append(grp)
    
    new_groups = []
    for grp in groups_toreprocess:
        subdf = grp['points'].copy()
        subdf.reset_index(drop=True, inplace=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        clusters_sub = bfs_components(graph, subdf)
        
        for csub in clusters_sub:
            slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(csub)
            new_groups.append({
                'points': csub,
                'slope': slope_,
                'intercept': intercept_,
                'pa': pa_,
                'theta_min': tmin,
                'theta_max': tmax,
                'r_min': rmin,
                'r_max': rmax
            })
    
    final_res = groups_ok + new_groups
    return final_res

# =========================================
# 7) Nueva función: Graficar grupos >= 60 puntos con la recta extrapolada un 30%
# =========================================
def plot_final_segments_over_60_with_line_extrapolation(groups, df_all, line_extrap=0.3):
    """
    - Filtra grupos con >= 60 puntos
    - Grafica cada uno en un nuevo plot (theta vs r)
    - En lugar de dibujar bounding box, extiende la recta un 'line_extrap'% 
      por debajo y por encima del rango de theta.
    """
    # 1) Filtrar los grupos grandes
    large_groups = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos con >= 60 puntos: {len(large_groups)}")

    # 2) Crear figura
    plt.figure(figsize=(10, 6))

    # Fondo con df_all en gris
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')

    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']

    for i, grp in enumerate(large_groups):
        c = colors[i % len(colors)]
        pts = grp['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c,
                    label=f"Grupo_{i+1} ({len(pts)} pts)")

        slope_ = grp['slope']
        intercept_ = grp['intercept']
        if slope_ is not None and grp['theta_min'] is not None and grp['theta_max'] is not None:
            # rango de theta del grupo
            theta_min = grp['theta_min']
            theta_max = grp['theta_max']
            dtheta = (theta_max - theta_min)
            # Extender un 30%
            theta_min_ext = theta_min - line_extrap*dtheta
            theta_max_ext = theta_max + line_extrap*dtheta

            # Graficar la línea en el rango extendido
            t_lin = np.linspace(theta_min_ext, theta_max_ext, 200)
            r_lin = slope_*t_lin + intercept_
            plt.plot(t_lin, r_lin, '--', color=c)
            plt.plot(t_lin, r_lin, '-.', color='k', alpha=0.3)

    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title(f"Grupos >= 60 puntos con recta extrapolada {int(line_extrap*100)}%")
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

# =========================================
# 8) Función principal: ejemplo
# =========================================
def main_example():
    # A) Generar BFS
    bfs_clusters, df_filtrado = generate_bfs_seeds(
        id_halo="17",
        theta_min=0,
        theta_max=460,
        quartile_threshold=0.155,
        theta_diff=3.0,
        r_diff=0.5,
        gap_threshold_theta=2.0,
        gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs_clusters)} grupos")

    # B) Ajustar y fusionar
    merged_groups = adjust_and_merge_seeds(
        bfs_clusters,
        slope_variation_threshold=0.20,
        bounding_extrap=0.35
    )
    print(f"Grupos tras fusión inicial: {len(merged_groups)}")

    # C) Validar dispersión y re-procesar
    validated_groups = validate_dispersion_and_reprocess(
        merged_groups,
        dispersion_threshold=1.30,
        reproc_theta_diff=1.05,
        reproc_r_diff=0.4
    )
    print(f"Grupos tras validación dispersión: {len(validated_groups)}")



    # D) Grafico global (todos los grupos) en (theta, r)
    plt.figure(figsize=(10,6))
    plt.scatter(df_filtrado['theta'], df_filtrado['r'], s=3, alpha=0.3, label='Datos BFS')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray']
    for i, group in enumerate(validated_groups):
        c = colors[i % len(colors)]
        pts = group['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"G{i+1} ({len(pts)}pts)")
        if group['slope'] is not None:
            tmin = pts['theta'].min()
            tmax = pts['theta'].max()
            t_lin = np.linspace(tmin, tmax, 100)
            r_lin = group['slope']*t_lin + group['intercept']
            plt.plot(t_lin, r_lin, '--', color=c)

    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    #plt.legend(loc='best')
    plt.title("Grupos finales (validados)")
    plt.grid(True)
    plt.show()

    # E) Ahora, filtrar los grupos con >=60 puntos y graficar su recta extrapolada
    plot_final_segments_over_60_with_line_extrapolation(
        validated_groups,
        df_filtrado,
        line_extrap=0.15  # 30% de extrapolación
    )


    def plot_final_segments_over_60_polar(groups, df_all, line_extrap=0.15):
        """
        Grafica en coordenadas polares (r, theta):
        - Filtra los grupos con >= 60 puntos.
        - Para cada grupo, crea una curva r = slope*theta_deg + intercept,
        donde theta_deg se extrapola un 'line_extrap'% a cada lado
        y se convierte a radianes para dibujar.

        Param:
        groups: lista de dicts con campos:
                'points' (DataFrame con 'theta','r'),
                'slope','intercept','theta_min','theta_max', ...
        df_all: DataFrame con las columnas 'theta','r' (theta en grados).
        line_extrap: factor de extrapolación (e.g. 0.3 = 30%)
        """
        # 1) Filtrar grupos con >=60 puntos
        large_groups = [g for g in groups if len(g['points']) >= 60]
        print(f"Grupos con >= 60 puntos: {len(large_groups)}")

        # 2) Crear figura con proyección polar
        fig = plt.figure(figsize=(9, 8))
        ax = plt.subplot(111, projection='polar')

        # 3) Fondo con df_all en gris (convirtiendo theta a radianes)
        theta_all_rad = np.radians(df_all['theta'].values)
        r_all = df_all['r'].values
        ax.scatter(theta_all_rad, r_all, s=3, alpha=0.3, color='gray', label='Datos base')

        # 4) Graficar cada grupo
        colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
        for i, grp in enumerate(large_groups):
            c = colors[i % len(colors)]
            pts = grp['points']

            # a) scatter de puntos del grupo (convertir theta->rad)
            theta_pts_rad = np.radians(pts['theta'].values)
            r_pts = pts['r'].values
            label_txt = f"Grupo_{i+1} ({len(pts)} pts)"
            ax.scatter(theta_pts_rad, r_pts, s=10, color=c, label=label_txt)

            # b) línea extrapolada
            slope_ = grp['slope']
            intercept_ = grp['intercept']
            tmin_deg = grp.get('theta_min', None)
            tmax_deg = grp.get('theta_max', None)
            if slope_ is not None and tmin_deg is not None and tmax_deg is not None:
                dt = tmax_deg - tmin_deg
                tmin_ext = tmin_deg - line_extrap*dt
                tmax_ext = tmax_deg + line_extrap*dt

                # Generamos 200 puntos en grados
                theta_deg_line = np.linspace(tmin_ext, tmax_ext, 200)
                # Calculamos r en base a slope*(theta_deg) + intercept
                r_line = slope_*theta_deg_line + intercept_
                # Convertimos a radianes para plot
                theta_rad_line = np.radians(theta_deg_line)

                ax.plot(theta_rad_line, r_line, '--', color=c)

        # 5) Ajustes finales en polar
        ax.set_theta_zero_location("E")   # opcional: 0° a la derecha (eje x)
        ax.set_theta_direction(-1)        # opcional: ángulos crecen en sentido horario
        ax.set_rlabel_position(0)         # etiquetas de r en 0 grados
        ax.set_title("Grupos >= 60 puntos (coordenadas polares, 30% extrapolación)", y=1.05)
        ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0))
        plt.show()

    plot_final_segments_over_60_polar(
    validated_groups,
    df_filtrado,
    line_extrap=0.15  # 30% de extrapolación
)

main_example()



In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Algoritmo mejorado para agrupar partículas (θ, r) y ajustar segmentos
lineales.  Incluye validación adaptativa cuando el dataset > 2000 puntos.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression


# ═══════════════════════════════════════════════════════════════════════
# 1) Funciones para Cargar/Filtrar Datos
# ═══════════════════════════════════════════════════════════════════════
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r']   = df['r'].astype(float)
    return df


# ═══════════════════════════════════════════════════════════════════════
# 2) Construir Grafo (Rectangular BFS)
# ═══════════════════════════════════════════════════════════════════════
def build_graph_rectangular(df_points, theta_diff, r_diff):
    theta_arr = df_points['theta'].values
    r_arr     = df_points['r'].values
    n         = len(df_points)

    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr     = np.abs(r_arr[i]     - r_arr)
        mask_neighbors = (dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i)
        neighbors      = np.where(mask_neighbors)[0]
        graph[i]       = list(neighbors)
    return graph, n


def bfs_components(graph, df_points):
    n        = len(graph)
    visited  = [False]*n
    clusters = []

    for start in range(n):
        if not visited[start]:
            queue        = deque([start])
            visited[start] = True
            comp_indices = [start]

            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)

            clusters.append(df_points.iloc[comp_indices].copy())

    return clusters


# ═══════════════════════════════════════════════════════════════════════
# 3) Subdividir por "gaps"
# ═══════════════════════════════════════════════════════════════════════
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col)
    arr   = df_cluster[order_col].values
    diffs = np.diff(arr)

    if np.all(diffs <= gap_threshold):
        return [df_cluster]

    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i + 1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters


# ═══════════════════════════════════════════════════════════════════════
# 4) Generar Semillas BFS  (con validación adaptativa > 2000 pts)
# ═══════════════════════════════════════════════════════════════════════
def _adaptive_factor(n_pts, reference=2000):
    """
    Devuelve √(n_pts / reference).  Si n_pts <= reference, devuelve 1.
    """
    return np.sqrt(max(n_pts / reference, 1.0))


def generate_bfs_seeds(
    id_halo,
    theta_min=50,
    theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    # 4.1) Cargar y filtrar por rango angular
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)

    # 4.2) Filtrar por densidad (rho_resta_final_exp > percentil)
    threshold    = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered  = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)

    # ─── NEW: Validación adaptativa para datasets grandes ───────────────
    n_pts = len(df_filtered)
    adapt = _adaptive_factor(n_pts)   # = 1 si n_pts ≤ 2000

    theta_diff_eff        = theta_diff        / adapt
    r_diff_eff            = r_diff            / adapt
    gap_threshold_theta_e = gap_threshold_theta / adapt
    gap_threshold_r_e     = gap_threshold_r     / adapt

    # 4.3) Construir grafo y obtener componentes
    graph, _   = build_graph_rectangular(df_filtered, theta_diff_eff, r_diff_eff)
    clusters   = bfs_components(graph, df_filtered)

    # 4.4) Subdividir cada componente por "gaps"
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta_e, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r_e, mode='r')
            final_clusters.extend(sub_r)

    return final_clusters, df_filtered


# ═══════════════════════════════════════════════════════════════════════
# 5) Ajustar rectas y Fusionar
# ═══════════════════════════════════════════════════════════════════════
def calculate_pa(slope):
    return None if slope is None else np.degrees(np.arctan(slope))


def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return (None,)*7

    X = df_cluster[['theta']].values
    y = df_cluster['r'].values
    model = LinearRegression().fit(X, y)

    slope_     = model.coef_[0]
    intercept_ = model.intercept_
    pa_        = calculate_pa(slope_)
    t_min      = df_cluster['theta'].min()
    t_max      = df_cluster['theta'].max()
    r_min      = df_cluster['r'].min()
    r_max      = df_cluster['r'].max()
    return slope_, intercept_, pa_, t_min, t_max, r_min, r_max


def adjust_and_merge_seeds(
    bfs_clusters,
    slope_variation_threshold=0.40,
    bounding_extrap=0.30
):
    # 5.1) Convertir cada cluster en dict con propiedades
    groups = []
    for clust_df in bfs_clusters:
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(clust_df)
        groups.append({
            'points'     : clust_df.copy(),
            'slope'      : slope_,
            'intercept'  : intercept_,
            'pa'         : pa_,
            'theta_min'  : tmin,
            'theta_max'  : tmax,
            'r_min'      : rmin,
            'r_max'      : rmax
        })

    # 5.2) Funciones auxiliares
    def recalc(group):
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(group['points'])
        group.update({
            'slope'     : slope_,
            'intercept' : intercept_,
            'pa'        : pa_,
            'theta_min' : tmin,
            'theta_max' : tmax,
            'r_min'     : rmin,
            'r_max'     : rmax
        })

    def boxes_overlap(g1, g2, extrap=0.30):
        # Expandir cajas en ambas dimensiones
        def expand(tmin, tmax, rmin, rmax):
            dt = (tmax - tmin) * extrap
            dr = (rmax - rmin) * extrap
            return tmin - dt, tmax + dt, rmin - dr, rmax + dr

        t1min, t1max, r1min, r1max = expand(g1['theta_min'], g1['theta_max'], g1['r_min'], g1['r_max'])
        t2min, t2max, r2min, r2max = expand(g2['theta_min'], g2['theta_max'], g2['r_min'], g2['r_max'])

        overlap_theta = not (t1max < t2min or t2max < t1min)
        overlap_r     = not (r1max < r2min or r2max < r1min)
        return overlap_theta and overlap_r

    # 5.3) Fusionar recursivamente
    merged = True
    while merged:
        merged   = False
        new_list = []
        i = 0
        while i < len(groups):
            g1 = groups[i]
            j  = i + 1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue

                if boxes_overlap(g1, g2, bounding_extrap):
                    comb_pts = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    slope_c, *_ = fit_line_to_cluster(comb_pts)

                    if slope_c is not None:
                        var1 = abs(slope_c - g1['slope'])/(abs(g1['slope']) + 1e-12)
                        var2 = abs(slope_c - g2['slope'])/(abs(g2['slope']) + 1e-12)
                        if var1 < slope_variation_threshold and var2 < slope_variation_threshold:
                            g1['points'] = comb_pts
                            recalc(g1)
                            groups.pop(j)
                            merged = True
                            continue
                j += 1
            new_list.append(g1)
            i += 1
        groups = new_list

    # 5.4) Filtrar grupos muy pequeños
    return [g for g in groups if len(g['points']) >= 2]


# ═══════════════════════════════════════════════════════════════════════
# 6) Validar dispersión y re-procesar (opcional)
# ═══════════════════════════════════════════════════════════════════════
def validate_dispersion_and_reprocess(
    groups,
    dispersion_threshold=2.0,
    reproc_theta_diff=2.0,
    reproc_r_diff=0.5
):
    ok_groups, to_reproc = [], []

    for grp in groups:
        if grp['slope'] is None or len(grp['points']) < 2:
            ok_groups.append(grp)
            continue

        residuals = grp['points']['r'] - (grp['slope']*grp['points']['theta'] + grp['intercept'])
        if residuals.std() > dispersion_threshold:
            to_reproc.append(grp)
        else:
            ok_groups.append(grp)

    # Reprocesar los grupos con alta dispersión
    new_groups = []
    for grp in to_reproc:
        subdf = grp['points'].copy().reset_index(drop=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for csub in bfs_components(graph, subdf):
            slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(csub)
            new_groups.append({
                'points'     : csub,
                'slope'      : slope_,
                'intercept'  : intercept_,
                'pa'         : pa_,
                'theta_min'  : tmin,
                'theta_max'  : tmax,
                'r_min'      : rmin,
                'r_max'      : rmax
            })

    return ok_groups + new_groups


# ═══════════════════════════════════════════════════════════════════════
# 7) Graficar grupos ≥ 60 puntos con recta extrapolada
# ═══════════════════════════════════════════════════════════════════════
def plot_final_segments_over_60_with_line_extrapolation(groups, df_all, line_extrap=0.3):
    large_groups = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos con ≥60 puntos: {len(large_groups)}")

    plt.figure(figsize=(10, 6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')

    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, grp in enumerate(large_groups):
        c   = colors[i % len(colors)]
        pts = grp['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"G{i+1} ({len(pts)})")

        if grp['slope'] is not None:
            dtheta = grp['theta_max'] - grp['theta_min']
            t_min  = grp['theta_min'] - line_extrap*dtheta
            t_max  = grp['theta_max'] + line_extrap*dtheta
            t_lin  = np.linspace(t_min, t_max, 200)
            r_lin  = grp['slope']*t_lin + grp['intercept']
            plt.plot(t_lin, r_lin, '--', color=c)

    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.title(f"Grupos ≥60 puntos – recta extrapolada {int(line_extrap*100)}%")
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()


# ═══════════════════════════════════════════════════════════════════════
# 8) Ejemplo principal
# ═══════════════════════════════════════════════════════════════════════
def main_example():
    # A) Generar BFS + subdivisión (ahora adaptativo)
    bfs_clusters, df_filtrado = generate_bfs_seeds(
        id_halo="17",
        theta_min=0,
        theta_max=360,
        quartile_threshold=0.155,
        theta_diff=3.0,
        r_diff=0.5,
        gap_threshold_theta=2.0,
        gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs_clusters)} clusters")

    # B) Fusionar
    merged_groups = adjust_and_merge_seeds(
        bfs_clusters,
        slope_variation_threshold=0.30,
        bounding_extrap=0.35
    )
    print(f"Tras fusión: {len(merged_groups)} grupos")

    # C) Validar dispersión
    validated_groups = validate_dispersion_and_reprocess(
        merged_groups,
        dispersion_threshold=1.80,
        reproc_theta_diff=1.25,
        reproc_r_diff=0.4
    )
    print(f"Tras validación: {len(validated_groups)} grupos")

    # D) Gráfico global
    plt.figure(figsize=(10, 6))
    plt.scatter(df_filtrado['theta'], df_filtrado['r'], s=3, alpha=0.3, label='Datos filtrados')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray']
    for i, g in enumerate(validated_groups):
        c   = colors[i % len(colors)]
        pts = g['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"G{i+1} ({len(pts)})")
        if g['slope'] is not None:
            t_lin = np.linspace(pts['theta'].min(), pts['theta'].max(), 100)
            r_lin = g['slope']*t_lin + g['intercept']
            plt.plot(t_lin, r_lin, '--', color=c)

    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.title("Grupos finales (validados)")
    plt.grid(True)
    plt.show()

    # E) Grupos grandes con recta extrapolada
    plot_final_segments_over_60_with_line_extrapolation(validated_groups, df_filtrado, line_extrap=0.15)


# ═══════════════════════════════════════════════════════════════════════
# 9) Llamada a main
# ═══════════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    main_example()


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
###############################################################################
###############################################################################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression


# ═══════════════════════════════════════════════════════════════════════
# 0)  PITCH‑ANGLE 
# ═══════════════════════════════════════════════════════════════════════
def calculate_pa(slope, intercept):
    """
    PA = arctan[(slope·180/π) / intercept]  en grados, si intercept ≠ 0.
    Devuelve NaN si intercept == 0.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan


# ═══════════════════════════════════════════════════════════════════════
# 1) Funciones para Cargar/Filtrar Datos
# ═══════════════════════════════════════════════════════════════════════
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r']     = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r']     = df['r'].astype(float)
    return df


# ═══════════════════════════════════════════════════════════════════════
# 2) Construir Grafo (Rectangular BFS)
# ═══════════════════════════════════════════════════════════════════════
def build_graph_rectangular(df_points, theta_diff, r_diff):
    th, rr = df_points['theta'].values, df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        mask = (np.abs(th[i] - th) <= theta_diff) & (np.abs(rr[i] - rr) <= r_diff) & (np.arange(n) != i)
        graph[i] = list(np.where(mask)[0])
    return graph, n


def bfs_components(graph, df_points):
    visited, clusters = [False]*len(graph), []
    for s in range(len(graph)):
        if not visited[s]:
            q, comp = deque([s]), [s]
            visited[s] = True
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters


# ═══════════════════════════════════════════════════════════════════════
# 3) Subdividir por "gaps"
# ═══════════════════════════════════════════════════════════════════════
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    col = 'theta' if mode == 'theta' else 'r'
    df  = df_cluster.sort_values(col)
    dif = np.diff(df[col].values)
    if np.all(dif <= gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(dif):
        if d > gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i + 1
    subs.append(df.iloc[start:].copy())
    return subs


# ═══════════════════════════════════════════════════════════════════════
# 4) Generar Semillas BFS  (validación adaptativa)
# ═══════════════════════════════════════════════════════════════════════
def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts / ref, 1.0))


def generate_bfs_seeds(
    id_halo,
    theta_min=50, theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0, r_diff=0.5,
    gap_threshold_theta=2.0, gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp'] > thr].copy().reset_index(drop=True)

    f = _adaptive_factor(len(df_f))
    th_diff, r_d = theta_diff/f, r_diff/f
    gap_th, gap_r = gap_threshold_theta/f, gap_threshold_r/f

    graph, _ = build_graph_rectangular(df_f, th_diff, r_d)
    clusters = bfs_components(graph, df_f)

    final_cl = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_th, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r, 'r'))
    return final_cl, df_f


# ═══════════════════════════════════════════════════════════════════════
# 5) Ajustar rectas y Fusionar
# ═══════════════════════════════════════════════════════════════════════
def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return (None,)*7
    mdl = LinearRegression().fit(df_cluster[['theta']], df_cluster['r'])
    s, b = mdl.coef_[0], mdl.intercept_
    pa   = calculate_pa(s, b)
    return s, b, pa, df_cluster['theta'].min(), df_cluster['theta'].max(), df_cluster['r'].min(), df_cluster['r'].max()


def adjust_and_merge_seeds(bfs_clusters, slope_variation_threshold=0.40, bounding_extrap=0.30):
    groups = []
    for cl in bfs_clusters:
        groups.append(dict(zip(
            ['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
            (*fit_line_to_cluster(cl), cl)
        )))

    def recalc(g):
        g['slope'], g['intercept'], g['pa'], g['theta_min'], g['theta_max'], g['r_min'], g['r_max'] = \
            fit_line_to_cluster(g['points'])

    def boxes_overlap(g1, g2, e):
        def exp(t0,t1,r0,r1):
            dt, dr = (t1-t0)*e, (r1-r0)*e
            return t0-dt, t1+dt, r0-dr, r1+dr
        a = exp(g1['theta_min'],g1['theta_max'],g1['r_min'],g1['r_max'])
        b = exp(g2['theta_min'],g2['theta_max'],g2['r_min'],g2['r_max'])
        return not (a[1]<b[0] or b[1]<a[0]) and not (a[3]<b[2] or b[3]<a[2])

    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i+1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1; continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, b_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb-g1['slope'])/(abs(g1['slope'])+1e-12)
                        v2 = abs(s_comb-g2['slope'])/(abs(g2['slope'])+1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb; recalc(g1)
                            groups.pop(j); merged = True; continue
                j += 1
            new.append(g1); i += 1
        groups = new
    return [g for g in groups if len(g['points']) >= 2]


# ═══════════════════════════════════════════════════════════════════════
# 6) Validar dispersión y re‑procesar
# ═══════════════════════════════════════════════════════════════════════
def validate_dispersion_and_reprocess(groups, dispersion_threshold=1.80, reproc_theta_diff=1.25, reproc_r_diff=0.4):
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points']) < 2:
            ok.append(g); continue
        res = g['points']['r'] - (g['slope']*g['points']['theta'] + g['intercept'])
        (todo if res.std() > dispersion_threshold else ok).append(g)

    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph,_ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            new.append(dict(zip(
                ['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
                (*fit_line_to_cluster(c), c)
            )))
    return ok + new


# ═══════════════════════════════════════════════════════════════════════
# 7‑A) Plot cartesiano grupos grandes (PA visible)
# ═══════════════════════════════════════════════════════════════════════
def plot_groups_cartesian(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i%len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        plt.scatter(pts['theta'], pts['r'], s=10, color=c,
                    label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            plt.plot(t, g['slope']*t + g['intercept'], '--', color=c)
    plt.xlabel("θ (°)"); plt.ylabel("r"); plt.grid(True)
    plt.title("Subhalo 17 – grupos ≥60 pts (cartesiano, PA visible)")
    plt.legend(); plt.show()


# ═══════════════════════════════════════════════════════════════════════
# 7‑B) Plot POLAR grupos grandes (PA visible)
# ═══════════════════════════════════════════════════════════════════════
def plot_groups_polar(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos (polar): {len(big)}")

    fig = plt.figure(figsize=(9,8))
    ax  = plt.subplot(111, projection='polar')
    ax.scatter(np.radians(df_all['theta']), df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')

    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i%len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        ax.scatter(np.radians(pts['theta']), pts['r'], s=10, color=c,
                   label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")

        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t_deg = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            r_line = g['slope']*t_deg + g['intercept']
            ax.plot(np.radians(t_deg), r_line, '.-', color="gray", alpha=0.20)
            ax.plot(np.radians(t_deg), r_line, '--', color=c)

    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title("Subhalo 17 – proyección polar (PA visible)", y=1.05)
    ax.legend(loc='upper right', bbox_to_anchor=(1.20,1.0))
    plt.show()


# ═══════════════════════════════════════════════════════════════════════
# 8) MAIN – pipeline completo subhalo 17
# ═══════════════════════════════════════════════════════════════════════
def main_example():
    bfs, df_f = generate_bfs_seeds(
        id_halo="17",
        theta_min=0, theta_max=360,
        quartile_threshold=0.155,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=2.0, gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")

    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.30, bounding_extrap=0.35)
    print(f"Tras fusión: {len(merged)} grupos")

    validated = validate_dispersion_and_reprocess(merged)
    print(f"Tras validación: {len(validated)} grupos")

    # ─── Visualizaciones ────────────────────────────────────────────────
    plot_groups_cartesian(validated, df_f, line_extrap=0.15)
    plot_groups_polar(validated, df_f, line_extrap=0.15)


# ═══════════════════════════════════════════════════════════════════════
# 9) Run
# ═══════════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    main_example()


## serpentiando entre puntos ...

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
###############################################################################
###############################################################################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression

# ═══════════════════════════════════════════════════════════════════════
# 0)  PITCH‑ANGLE 
# ═══════════════════════════════════════════════════════════════════════
def calculate_pa(slope, intercept):
    """
    PA = arctan[(slope·180/π) / intercept]  en grados, si intercept ≠ 0.
    Devuelve NaN si intercept == 0.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan


# ═══════════════════════════════════════════════════════════════════════
# 1) Funciones para Cargar/Filtrar Datos
# ═══════════════════════════════════════════════════════════════════════
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    # Agregar columna de id para preservar el índice original
    df['id'] = df.index
    df['r']     = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r']     = df['r'].astype(float)
    return df


# ═══════════════════════════════════════════════════════════════════════
# 2) Construir Grafo (Rectangular BFS)
# ═══════════════════════════════════════════════════════════════════════
def build_graph_rectangular(df_points, theta_diff, r_diff):
    th, rr = df_points['theta'].values, df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        mask = (np.abs(th[i] - th) <= theta_diff) & (np.abs(rr[i] - rr) <= r_diff) & (np.arange(n) != i)
        graph[i] = list(np.where(mask)[0])
    return graph, n


def bfs_components(graph, df_points):
    visited, clusters = [False]*len(graph), []
    for s in range(len(graph)):
        if not visited[s]:
            q, comp = deque([s]), [s]
            visited[s] = True
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters


# ═══════════════════════════════════════════════════════════════════════
# 3) Subdividir por "gaps"
# ═══════════════════════════════════════════════════════════════════════
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    col = 'theta' if mode == 'theta' else 'r'
    df  = df_cluster.sort_values(col)
    dif = np.diff(df[col].values)
    if np.all(dif <= gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(dif):
        if d > gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i + 1
    subs.append(df.iloc[start:].copy())
    return subs


# ═══════════════════════════════════════════════════════════════════════
# 4) Generar Semillas BFS  (validación adaptativa)
# ═══════════════════════════════════════════════════════════════════════
def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts / ref, 1.0))


def generate_bfs_seeds(
    id_halo,
    theta_min=50, theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0, r_diff=0.5,
    gap_threshold_theta=2.0, gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp'] > thr].copy().reset_index(drop=True)

    f = _adaptive_factor(len(df_f))
    th_diff, r_d = theta_diff/f, r_diff/f
    gap_th, gap_r = gap_threshold_theta/f, gap_threshold_r/f

    graph, _ = build_graph_rectangular(df_f, th_diff, r_d)
    clusters = bfs_components(graph, df_f)

    final_cl = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_th, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r, 'r'))
    return final_cl, df_f


# ═══════════════════════════════════════════════════════════════════════
# 5) Ajustar rectas y Fusionar
# ═══════════════════════════════════════════════════════════════════════
def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return (None,)*7
    mdl = LinearRegression().fit(df_cluster[['theta']], df_cluster['r'])
    s, b = mdl.coef_[0], mdl.intercept_
    pa   = calculate_pa(s, b)
    return s, b, pa, df_cluster['theta'].min(), df_cluster['theta'].max(), df_cluster['r'].min(), df_cluster['r'].max()


def adjust_and_merge_seeds(bfs_clusters, slope_variation_threshold=0.40, bounding_extrap=0.30):
    groups = []
    for cl in bfs_clusters:
        groups.append(dict(zip(
            ['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
            (*fit_line_to_cluster(cl), cl)
        )))

    def recalc(g):
        g['slope'], g['intercept'], g['pa'], g['theta_min'], g['theta_max'], g['r_min'], g['r_max'] = \
            fit_line_to_cluster(g['points'])

    def boxes_overlap(g1, g2, e):
        def exp(t0,t1,r0,r1):
            dt, dr = (t1-t0)*e, (r1-r0)*e
            return t0-dt, t1+dt, r0-dr, r1+dr
        a = exp(g1['theta_min'],g1['theta_max'],g1['r_min'],g1['r_max'])
        b = exp(g2['theta_min'],g2['theta_max'],g2['r_min'],g2['r_max'])
        return not (a[1]<b[0] or b[1]<a[0]) and not (a[3]<b[2] or b[3]<a[2])

    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i+1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, b_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb - g1['slope'])/(abs(g1['slope'])+1e-12)
                        v2 = abs(s_comb - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb
                            recalc(g1)
                            groups.pop(j)
                            merged = True
                            continue
                j += 1
            new.append(g1)
            i += 1
        groups = new
    return [g for g in groups if len(g['points']) >= 2]


# ═══════════════════════════════════════════════════════════════════════
# 6) Validar dispersión y re‑procesar
# ═══════════════════════════════════════════════════════════════════════
def validate_dispersion_and_reprocess(groups, dispersion_threshold=1.80, reproc_theta_diff=1.25, reproc_r_diff=0.4):
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points']) < 2:
            ok.append(g)
            continue
        res = g['points']['r'] - (g['slope'] * g['points']['theta'] + g['intercept'])
        (todo if res.std() > dispersion_threshold else ok).append(g)

    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            new.append(dict(zip(
                ['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
                (*fit_line_to_cluster(c), c)
            )))
    return ok + new


# ═══════════════════════════════════════════════════════════════════════
# 7‑A) Plot cartesiano grupos grandes (PA visible)
# ═══════════════════════════════════════════════════════════════════════
def plot_groups_cartesian(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        plt.scatter(pts['theta'], pts['r'], s=10, color=c,
                    label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            plt.plot(t, g['slope'] * t + g['intercept'], '--', color=c)
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.grid(True)
    plt.title("Subhalo 17 – grupos ≥60 pts (cartesiano, PA visible)")
    plt.legend()
    plt.show()


# ═══════════════════════════════════════════════════════════════════════
# 7‑B) Plot POLAR grupos grandes (PA visible)
# ═══════════════════════════════════════════════════════════════════════
def plot_groups_polar(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos (polar): {len(big)}")

    fig = plt.figure(figsize=(9,8))
    ax  = plt.subplot(111, projection='polar')
    ax.scatter(np.radians(df_all['theta']), df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')

    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        ax.scatter(np.radians(pts['theta']), pts['r'], s=10, color=c,
                   label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")

        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t_deg = np.linspace(g['theta_min'] - line_extrap * dt, g['theta_max'] + line_extrap * dt, 200)
            r_line = g['slope'] * t_deg + g['intercept']
            ax.plot(np.radians(t_deg), r_line, '.-', color="gray", alpha=0.20)
            ax.plot(np.radians(t_deg), r_line, '--', color=c)

    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title("Subhalo 17 – proyección polar (PA visible)", y=1.05)
    ax.legend(loc='upper right', bbox_to_anchor=(1.20, 1.0))
    plt.show()


# ═══════════════════════════════════════════════════════════════════════
# 8) MAIN – pipeline completo subhalo 17 (modificado para almacenar grupos y background)
# ═══════════════════════════════════════════════════════════════════════
def main_example_modificado():
    # Genera las semillas BFS y carga los datos filtrados
    bfs, df_f = generate_bfs_seeds(
        id_halo="17",
        theta_min=0, theta_max=360,
        quartile_threshold=0.155,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=2.0, gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")

    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.30, bounding_extrap=0.35)
    print(f"Tras fusión: {len(merged)} grupos")

    validated = validate_dispersion_and_reprocess(merged)
    print(f"Tras validación: {len(validated)} grupos")

    # Visualizaciones (si se desea mantener)
    plot_groups_cartesian(validated, df_f, line_extrap=0.15)
    plot_groups_polar(validated, df_f, line_extrap=0.15)

    # Almacenar grupos finales: solo aquellos con 60 o más puntos
    grupos_finales = [g for g in validated if len(g['points']) >= 60]

    # Identificar los IDs de los puntos que pertenecen a los grupos finales.
    puntos_grupos = set()
    for g in grupos_finales:
        # Se espera que la columna "id" esté en los datos originales
        puntos_grupos.update(g['points']['id'].tolist())

    # Los puntos "background" son aquellos de df_f que no pertenecen a ningún grupo final.
    background = df_f[~df_f['id'].isin(puntos_grupos)].copy()
    print(f"Puntos background: {len(background)}")

    # Se puede devolver un diccionario con los dos conjuntos de información:
    return {'grupos_finales': grupos_finales, 'background': background}


# ═══════════════════════════════════════════════════════════════════════
# 9) Run
# ═══════════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    datos_finales = main_example_modificado()
    # Ahora, datos_finales['grupos_finales'] contiene los grupos con ≥ 60 pts, 
    # y datos_finales['background'] contiene la información original de los puntos
    # que no fueron asignados a ningún grupo final.


In [0]:
grupos_finales = datos_finales['grupos_finales']
grupos_finales[2]

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segundo paso: Trazado de un camino continuo (de derecha a izquierda) en el background,
utilizando la información de las fronteras de los grupos finales.

La idea es:
    1. Iniciar en el punto con mayor x (más a la derecha) del background.
    2. De entre los puntos restantes, sólo considerar aquellos cuyo valor de x es menor que el actual (para asegurar
       el recorrido de derecha a izquierda).
    3. Seleccionar el candidato más cercano (mínima distancia Euclidiana) entre los que cumplan esa restricción.
    4. Evaluar si el candidato está “muy cerca” de la frontera de algún grupo final (distancia < umbral, basada en la
       distancia promedio entre puntos vecinos del background). Si es así, se entiende que se ha tocado la frontera y
       se finaliza el brazo.
    5. En caso contrario, se añade el candidato al camino y se actualiza el punto actual para continuar el recorrido.
    6. Se repite hasta que no haya candidatos (o la distancia es mayor al umbral) y se obtiene un brazo espiral.

El resultado se grafica en el espacio (θ, r).
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# Función para convertir de coordenadas polares a cartesianas
def polar_to_cartesian(df):
    df = df.copy()
    rad = np.radians(df['theta'].values)
    df['x'] = df['r'] * np.cos(rad)
    df['y'] = df['r'] * np.sin(rad)
    return df

# Función para calcular la distancia promedio al vecino más cercano de un conjunto de puntos
def compute_avg_nn_distance(points):
    if len(points) < 2:
        return np.inf
    dists = cdist(points, points)
    np.fill_diagonal(dists, np.inf)
    return np.mean(np.min(dists, axis=1))

# Función para trazar el brazo espiral a partir del background y tomando en cuenta las fronteras de los grupos finales
def trace_spiral_arm(background, grupos_finales, tol_factor=1.1):
    """
    background: DataFrame de puntos en coordenadas polares que no pertenecen a ningún grupo final.
    grupos_finales: lista de diccionarios (con clave 'points') que contienen los grupos finales ya calculados.
                  Se asume que la "frontera" se obtiene tomando (por simplicidad) todos los puntos de dichos grupos.
    tol_factor: factor de tolerancia para definir el umbral de conexión.
    
    Retorna un DataFrame con el camino (ordenado) que representa el brazo espiral.
    """
    # Convertir los puntos del background a coordenadas cartesianas.
    bg_cart = polar_to_cartesian(background)
    pts_bg = bg_cart[['x', 'y']].values

    # Preparar los puntos frontera a partir de los grupos finales.
    # Aquí se toma como frontera de cada grupo todos sus puntos, pues se entiende que
    # el interior ya está definido y no se requiere recorrerlo nuevamente.
    boundary_list = []
    for grupo in grupos_finales:
        pts = grupo['points']
        pts_cart = polar_to_cartesian(pts)
        boundary_list.append(pts_cart[['x', 'y']].values)
    if boundary_list:
        boundary_all = np.vstack(boundary_list)
    else:
        boundary_all = np.empty((0, 2))
    
    # Calcular un umbral base usando la distancia promedio entre vecinos en el background.
    avg_nn_bg = compute_avg_nn_distance(pts_bg)
    threshold = tol_factor * avg_nn_bg

    # Preparar el recorrido:
    # Partir del punto con el mayor x (más a la derecha).
    remaining_indices = list(range(len(pts_bg)))
    start_idx = max(remaining_indices, key=lambda i: pts_bg[i, 0])
    spiral_indices = [start_idx]
    current_idx = start_idx
    remaining_indices.remove(start_idx)
    
    # Búsqueda greedy:
    while remaining_indices:
        current_pt = pts_bg[current_idx].reshape(1,2)
        # Solo se consideran candidatos con x menor que la del punto actual para mantener la dirección (derecha a izquierda)
        valid_candidates = [i for i in remaining_indices if pts_bg[i,0] < pts_bg[current_idx,0]]
        if not valid_candidates:
            break
        candidates = pts_bg[valid_candidates]
        dists = np.linalg.norm(candidates - current_pt, axis=1)
        min_dist = np.min(dists)
        candidate_local_idx = np.argmin(dists)
        candidate_idx = valid_candidates[candidate_local_idx]

        # Verificar conexión con la frontera de algún grupo final:
        if boundary_all.shape[0] > 0:
            dist_to_boundary = np.min(np.linalg.norm(boundary_all - pts_bg[candidate_idx], axis=1))
        else:
            dist_to_boundary = np.inf

        # Si la distancia al candidato o la distancia a la frontera es menor al umbral,
        # se asume que se toca la frontera y se finaliza el recorrido (se añade el candidato y se cierra el brazo).
        if min_dist <= threshold or dist_to_boundary <= threshold:
            spiral_indices.append(candidate_idx)
            # Una vez que se alcanza la frontera, se finaliza el brazo espiral.
            break
        else:
            # Se añade el candidato al camino y se actualiza el punto actual.
            spiral_indices.append(candidate_idx)
            current_idx = candidate_idx
            remaining_indices.remove(candidate_idx)

    # Extraer el conjunto de puntos (en orden) que conforman el brazo espiral.
    spiral_arm = bg_cart.iloc[spiral_indices].copy()
    return spiral_arm

# Función para graficar en (θ, r) el brazo espiral obtenido.
def plot_spiral_arm(spiral_arm, title="Brazo Espiral (θ, r)"):
    plt.figure(figsize=(8,6))
    plt.scatter(spiral_arm['theta'], spiral_arm['r'], color='blue', s=20, label="Puntos del brazo")
    plt.plot(spiral_arm['theta'], spiral_arm['r'], '-o', color='red')
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.title(title)
    plt.grid(True)
    plt.legend()
    plt.show()


# Ejemplo de uso en la pipeline:
if __name__ == "__main__":

    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']
    

    if not background.empty and len(grupos_finales) > 0:
        brazo_espiral = trace_spiral_arm(background, grupos_finales, tol_factor=1.1)
        print(f"Brazo espiral trazado con {len(brazo_espiral)} puntos.")
        plot_spiral_arm(brazo_espiral)
    else:
        print("No hay datos suficientes en el background o en los grupos finales para trazar el brazo espiral.")



In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Prototipo para el segundo paso: trazado de un brazo espiral a partir del background.
La idea es recorrer de derecha a izquierda (según coordenada x) de manera greedy; 
en cada iteración se evalúa si el candidato toca la frontera de algún grupo final. 
Si ocurre, se añade ese candidato y se lanza una búsqueda complementaria sobre el extremo
opuesto para verificar la conexión con otro grupo.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# --- Funciones auxiliares ---

def polar_to_cartesian(df):
    """Convierte de coordenadas polares (theta, r) a cartesianas (x, y)."""
    df = df.copy()
    rad = np.radians(df['theta'].values)
    df['x'] = df['r'] * np.cos(rad)
    df['y'] = df['r'] * np.sin(rad)
    return df

def compute_avg_nn_distance(points):
    """
    Calcula la distancia promedio al vecino más cercano.
    points: arreglo de forma (n,2)
    """
    if len(points) < 2:
        return np.inf
    dists = cdist(points, points)
    np.fill_diagonal(dists, np.inf)
    return np.mean(np.min(dists, axis=1))

def get_boundary_points(grupos_finales):
    """
    Dado que ya se tienen los grupos finales, se extrae el conjunto de puntos que
    se usan para representar la frontera. En este ejemplo se toman todos los puntos.
    """
    boundary_list = []
    for grupo in grupos_finales:
        pts_cart = polar_to_cartesian(grupo['points'])
        boundary_list.append(pts_cart[['x','y']].values)
    if boundary_list:
        return np.vstack(boundary_list)
    return np.empty((0,2))

# --- Funciones para el trazado del brazo espiral ---

def trace_spiral_arm(background, grupos_finales, tol_factor=1.1):
    """
    Recorre los puntos del background de derecha a izquierda en forma greedy.
    Para cada candidato se comprueba si toca la frontera de algún grupo final,
    usando como umbral tol_factor * (distancia promedio entre vecinos del background).
    
    Si se detecta conexión, se añade el punto y se evalúa el extremo opuesto.
    
    Retorna un diccionario con:
      - 'brazo': DataFrame con el camino trazado (ordenado)
      - 'conexiones': información sobre las conexiones con grupos finales (extremos)
    """
    # Convertir background a coordenadas cartesianas.
    bg_cart = polar_to_cartesian(background)
    pts_bg = bg_cart[['x','y']].values

    # Obtener el conjunto de puntos frontera de los grupos finales.
    boundary_all = get_boundary_points(grupos_finales)
    
    # Definir el umbral base usando la distancia promedio de vecinos en el background.
    avg_nn = compute_avg_nn_distance(pts_bg)
    threshold = tol_factor * avg_nn

    # Inicializar el recorrido: partir del punto con máximo x (más a la derecha).
    remaining_indices = list(range(len(pts_bg)))
    start_idx = max(remaining_indices, key=lambda i: pts_bg[i,0])
    brazo_indices = [start_idx]
    current_idx = start_idx
    remaining_indices.remove(start_idx)

    # Búsqueda greedy: recorrer background en dirección de menor x.
    while remaining_indices:
        current_pt = pts_bg[current_idx].reshape(1,2)
        # Filtrar candidatos que tengan x menor al del punto actual.
        valid_candidates = [i for i in remaining_indices if pts_bg[i,0] < pts_bg[current_idx,0]]
        if not valid_candidates:
            break
        candidates = pts_bg[valid_candidates]
        dists = np.linalg.norm(candidates - current_pt, axis=1)
        min_dist = np.min(dists)
        candidate_local_idx = np.argmin(dists)
        candidate_idx = valid_candidates[candidate_local_idx]
        
        # Evaluar la distancia del candidato a la frontera de los grupos finales.
        if boundary_all.shape[0] > 0:
            dist_to_boundary = np.min(np.linalg.norm(boundary_all - pts_bg[candidate_idx], axis=1))
        else:
            dist_to_boundary = np.inf
        
        # Si el candidato cumple la condición (candidato muy cerca de la frontera)...
        if min_dist <= threshold or dist_to_boundary <= threshold:
            brazo_indices.append(candidate_idx)
            # Se anota la conexión.
            conexion = {
                'punto_conexion': bg_cart.iloc[candidate_idx],
                'distancia_a_frontera': dist_to_boundary
            }
            # Se evalúa el extremo opuesto: a partir de este candidato, se lanza un método 
            # complementario para ver con qué otro grupo se conecta o si se sigue el background.
            brazo_opuesto = trace_opposite_extremity(candidate_idx, pts_bg, remaining_indices, boundary_all, threshold)
            if brazo_opuesto is not None:
                brazo_indices.extend(brazo_opuesto['indices'])
                conexion['extremo_opuesto'] = brazo_opuesto['punto_final']
            else:
                conexion['extremo_opuesto'] = None
            # Se finaliza el recorrido.
            break
        else:
            # Si no se conecta, se añade el candidato y se continúa.
            brazo_indices.append(candidate_idx)
            current_idx = candidate_idx
            remaining_indices.remove(candidate_idx)
    
    # Obtener el DataFrame final del brazo en el orden de recorrido.
    brazo = bg_cart.iloc[brazo_indices].copy()
    conexiones = None  # Aquí se podría incluir información adicional sobre las conexiones.
    return {'brazo': brazo, 'conexiones': conexion if 'conexion' in locals() else None}

def trace_opposite_extremity(current_idx, pts_bg, remaining_indices, boundary_all, threshold):
    """
    A partir del punto actual (que acaba de tocar una frontera), se evalúa el "extremo opuesto"
    buscando entre los puntos del background (de los que aún quedan) si alguno conecta con 
    la frontera de otro grupo final.
    
    Retorna:
      - un diccionario con 'indices': la secuencia de índices del brazo opuesto,
                     'punto_final': la última conexión encontrada.
      - None si no se encontró conexión.
    """
    brazo_indices = []
    cur_idx = current_idx
    # Se hará una búsqueda similar pero ahora se busca extender el brazo (por ejemplo, en la dirección que se aleja)
    # De forma simplificada, se puede intentar usar la dirección contraria: candidatos cuyo x sean mayores.
    valid_candidates = [i for i in remaining_indices if pts_bg[i,0] > pts_bg[cur_idx,0]]
    if not valid_candidates:
        return None
    # Se selecciona el más cercano en esa dirección.
    candidates = pts_bg[valid_candidates]
    dists = np.linalg.norm(candidates - pts_bg[cur_idx].reshape(1,2), axis=1)
    min_dist = np.min(dists)
    candidate_local_idx = np.argmin(dists)
    candidate_idx = valid_candidates[candidate_local_idx]
    
    # Verificar la conexión con la frontera.
    if boundary_all.shape[0] > 0:
        dist_to_boundary = np.min(np.linalg.norm(boundary_all - pts_bg[candidate_idx], axis=1))
    else:
        dist_to_boundary = np.inf
    
    if min_dist <= threshold or dist_to_boundary <= threshold:
        brazo_indices.append(candidate_idx)
        return {'indices': brazo_indices, 'punto_final': candidate_idx}
    return None

# --- Función para graficar el brazo espiral en (θ, r) ---
def plot_spiral_arm(arm_dict, title="Brazo Espiral (θ, r)"):
    brazo = arm_dict['brazo']
    plt.figure(figsize=(8,6))
    plt.scatter(brazo['theta'], brazo['r'], color='blue', s=20, label="Puntos del brazo")
    plt.plot(brazo['theta'], brazo['r'], '-o', color='red', label="Camino trazado")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.title(title)
    plt.grid(True)
    plt.legend()
    plt.show()

# --- Ejemplo de uso ---
if __name__ == "__main__":

    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']

    
    if not background.empty and len(grupos_finales) > 0:
        arm_result = trace_spiral_arm(background, grupos_finales, tol_factor=1.1)
        print(f"Brazo espiral trazado con {len(arm_result['brazo'])} puntos.")
        if arm_result['conexiones']:
            print("Se estableció conexión con la frontera de un grupo final:")
            print(arm_result['conexiones'])
        plot_spiral_arm(arm_result)
    else:
        print("No hay datos suficientes en el background o en los grupos finales para trazar el brazo espiral.")


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Código completo para:
  1. Procesar y agrupar los datos (primer paso).
  2. Del background obtener un camino continuo (brazo espiral) mediante recorrido greedy,
     evaluando la conexión con la frontera de los grupos finales.
  3. Visualizar en el espacio (θ, r) el brazo espiral, superponiendo todos los puntos leídos y los
     grupos finales con baja opacidad (alpha=0.1).
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import cdist

# =============================================================================
# 0) Funciones básicas de la etapa de agrupación
# =============================================================================

def calculate_pa(slope, intercept):
    """
    Calcula el pitch-angle (PA) en grados: PA = arctan[(slope*180/π) / intercept] si intercept≠0.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan

def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    """
    Carga el archivo CSV, calcula 'r' y 'theta', y filtra según el rango de theta.
    Se añade una columna 'id' para preservar el índice original.
    """
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['id'] = df.index
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo de adyacencia (BFS) en el espacio (theta, r) considerando una tolerancia.
    """
    th, rr = df_points['theta'].values, df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        mask = (np.abs(th[i] - th) <= theta_diff) & (np.abs(rr[i] - rr) <= r_diff) & (np.arange(n) != i)
        graph[i] = list(np.where(mask)[0])
    return graph, n

def bfs_components(graph, df_points):
    """
    Extrae componentes conexas usando BFS.
    """
    visited, clusters = [False] * len(graph), []
    for s in range(len(graph)):
        if not visited[s]:
            q, comp = deque([s]), [s]
            visited[s] = True
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Subdivide un clúster en subgrupos si existen "gaps" (saltos en theta o r).
    """
    col = 'theta' if mode == 'theta' else 'r'
    df = df_cluster.sort_values(col)
    dif = np.diff(df[col].values)
    if np.all(dif <= gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(dif):
        if d > gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i + 1
    subs.append(df.iloc[start:].copy())
    return subs

def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts / ref, 1.0))

def generate_bfs_seeds(id_halo, theta_min, theta_max,
                         quartile_threshold=0.55,
                         theta_diff=3.0, r_diff=0.5,
                         gap_threshold_theta=2.0, gap_threshold_r=2.0,
                         file_prefix='data_rho'):
    """
    Genera las semillas BFS a partir de los datos filtrados.
    """
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp'] > thr].copy().reset_index(drop=True)

    f = _adaptive_factor(len(df_f))
    th_diff, r_d = theta_diff / f, r_diff / f
    gap_th, gap_r = gap_threshold_theta / f, gap_threshold_r / f

    graph, _ = build_graph_rectangular(df_f, th_diff, r_d)
    clusters = bfs_components(graph, df_f)

    final_cl = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_th, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r, 'r'))
    return final_cl, df_f

def fit_line_to_cluster(df_cluster):
    """
    Ajusta una línea (regresión lineal) a los puntos del clúster.
    """
    if len(df_cluster) < 2:
        return (None,) * 7
    mdl = LinearRegression().fit(df_cluster[['theta']], df_cluster['r'])
    s, b = mdl.coef_[0], mdl.intercept_
    pa = calculate_pa(s, b)
    return s, b, pa, df_cluster['theta'].min(), df_cluster['theta'].max(), df_cluster['r'].min(), df_cluster['r'].max()

def adjust_and_merge_seeds(bfs_clusters, slope_variation_threshold=0.40, bounding_extrap=0.30):
    """
    Fusiona los clústeres que se solapan en la caja definida.
    """
    groups = []
    for cl in bfs_clusters:
        groups.append(dict(zip(
            ['slope', 'intercept', 'pa', 'theta_min', 'theta_max', 'r_min', 'r_max', 'points'],
            (*fit_line_to_cluster(cl), cl)
        )))

    def recalc(g):
        g['slope'], g['intercept'], g['pa'], g['theta_min'], g['theta_max'], g['r_min'], g['r_max'] = \
            fit_line_to_cluster(g['points'])

    def boxes_overlap(g1, g2, e):
        def exp(t0, t1, r0, r1):
            dt, dr = (t1 - t0) * e, (r1 - r0) * e
            return t0 - dt, t1 + dt, r0 - dr, r1 + dr
        a = exp(g1['theta_min'], g1['theta_max'], g1['r_min'], g1['r_max'])
        b = exp(g2['theta_min'], g2['theta_max'], g2['r_min'], g2['r_max'])
        return not (a[1] < b[0] or b[1] < a[0]) and not (a[3] < b[2] or b[3] < a[2])

    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i + 1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, b_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb - g1['slope']) / (abs(g1['slope']) + 1e-12)
                        v2 = abs(s_comb - g2['slope']) / (abs(g2['slope']) + 1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb
                            recalc(g1)
                            groups.pop(j)
                            merged = True
                            continue
                j += 1
            new.append(g1)
            i += 1
        groups = new
    return [g for g in groups if len(g['points']) >= 2]

def validate_dispersion_and_reprocess(groups, dispersion_threshold=1.80, reproc_theta_diff=1.25, reproc_r_diff=0.4):
    """
    Valida la dispersión y re-procesa aquellos grupos que no cumplen el criterio.
    """
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points']) < 2:
            ok.append(g)
            continue
        res = g['points']['r'] - (g['slope'] * g['points']['theta'] + g['intercept'])
        (todo if res.std() > dispersion_threshold else ok).append(g)
    
    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            new.append(dict(zip(
                ['slope', 'intercept', 'pa', 'theta_min', 'theta_max', 'r_min', 'r_max', 'points'],
                (*fit_line_to_cluster(c), c)
            )))
    return ok + new

def plot_groups_cartesian(groups, df_all, line_extrap=0.15):
    """
    Visualiza en un gráfico cartesiano los grupos finales con ≥ 60 puntos (PA visible).
    """
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        plt.scatter(pts['theta'], pts['r'], s=10, color=c,
                    label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t = np.linspace(g['theta_min'] - line_extrap * dt, g['theta_max'] + line_extrap * dt, 200)
            plt.plot(t, g['slope'] * t + g['intercept'], '--', color=c)
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.grid(True)
    plt.title("Subhalo 17 – grupos ≥60 pts (cartesiano, PA visible)")
    plt.legend()
    plt.show()

def plot_groups_polar(groups, df_all, line_extrap=0.15):
    """
    Visualización en proyección polar de los grupos finales con ≥ 60 puntos.
    """
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos (polar): {len(big)}")
    fig = plt.figure(figsize=(9,8))
    ax = plt.subplot(111, projection='polar')
    ax.scatter(np.radians(df_all['theta']), df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        ax.scatter(np.radians(pts['theta']), pts['r'], s=10, color=c,
                   label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t_deg = np.linspace(g['theta_min'] - line_extrap * dt, g['theta_max'] + line_extrap * dt, 200)
            r_line = g['slope'] * t_deg + g['intercept']
            ax.plot(np.radians(t_deg), r_line, '.-', color="gray", alpha=0.20)
            ax.plot(np.radians(t_deg), r_line, '--', color=c)
    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title("Subhalo 17 – proyección polar (PA visible)", y=1.05)
    ax.legend(loc='upper right', bbox_to_anchor=(1.20, 1.0))
    plt.show()

def main_example_modificado():
    """
    Ejecuta el pipeline completo para subhalo 17.
    Retorna un diccionario con:
       - 'grupos_finales': lista de grupos (con ≥ 60 pts)
       - 'background': DataFrame con los puntos que no están en ningún grupo final.
    """
    bfs, df_f = generate_bfs_seeds(
        id_halo="17",
        theta_min=0, theta_max=360,
        quartile_threshold=0.155,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=2.0, gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")
    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.30, bounding_extrap=0.35)
    print(f"Tras fusión: {len(merged)} grupos")
    validated = validate_dispersion_and_reprocess(merged)
    print(f"Tras validación: {len(validated)} grupos")
    plot_groups_cartesian(validated, df_f, line_extrap=0.15)
    plot_groups_polar(validated, df_f, line_extrap=0.15)
    
    # Almacenar grupos finales: únicamente aquellos con 60 o más puntos.
    grupos_finales = [g for g in validated if len(g['points']) >= 60]
    
    # Determinar los puntos (IDs) que aparecen en los grupos finales.
    puntos_grupos = set()
    for g in grupos_finales:
        puntos_grupos.update(g['points']['id'].tolist())
    
    # Los puntos "background" son los que en df_f no están en ningún grupo final.
    background = df_f[~df_f['id'].isin(puntos_grupos)].copy()
    print(f"Puntos background: {len(background)}")
    
    return {'grupos_finales': grupos_finales, 'background': background}

# =============================================================================
# 1) Funciones para el trazado del brazo espiral a partir del background
# =============================================================================

def polar_to_cartesian_simple(df):
    """
    Versión simplificada para convertir a cartesianas (usa 'theta' y 'r').
    """
    df = df.copy()
    rad = np.radians(df['theta'].values)
    df['x'] = df['r'] * np.cos(rad)
    df['y'] = df['r'] * np.sin(rad)
    return df

def compute_avg_nn_distance(points):
    if len(points) < 2:
        return np.inf
    dists = cdist(points, points)
    np.fill_diagonal(dists, np.inf)
    return np.mean(np.min(dists, axis=1))

def get_boundary_points(grupos_finales):
    """
    Extrae, de los grupos finales, el conjunto de puntos a considerar como frontera.
    En este ejemplo se toman todos los puntos de cada grupo.
    """
    boundary_list = []
    for grupo in grupos_finales:
        pts_cart = polar_to_cartesian_simple(grupo['points'])
        boundary_list.append(pts_cart[['x', 'y']].values)
    if boundary_list:
        return np.vstack(boundary_list)
    return np.empty((0,2))

def trace_spiral_arm(background, grupos_finales, tol_factor=1.1):
    """
    Recorre los puntos del background de derecha a izquierda (por x) de forma greedy.
    Para cada candidato, se evalúa si toca la frontera de algún grupo final usando
    un umbral definido como tol_factor * (distancia promedio entre vecinos del background).
    
    Además, si se detecta conexión, se lanza una función complementaria para evaluar el extremo opuesto.
    
    Retorna un diccionario con:
      - 'brazo': DataFrame con el camino (en orden) del brazo espiral.
      - 'conexiones': información sobre la(s) conexión(es) detectada(s).
    """
    bg_cart = polar_to_cartesian_simple(background)
    pts_bg = bg_cart[['x', 'y']].values

    boundary_all = get_boundary_points(grupos_finales)
    avg_nn = compute_avg_nn_distance(pts_bg)
    threshold = tol_factor * avg_nn

    remaining_indices = list(range(len(pts_bg)))
    start_idx = max(remaining_indices, key=lambda i: pts_bg[i, 0])
    brazo_indices = [start_idx]
    current_idx = start_idx
    remaining_indices.remove(start_idx)

    while remaining_indices:
        current_pt = pts_bg[current_idx].reshape(1,2)
        valid_candidates = [i for i in remaining_indices if pts_bg[i, 0] < pts_bg[current_idx, 0]]
        if not valid_candidates:
            break
        candidates = pts_bg[valid_candidates]
        dists = np.linalg.norm(candidates - current_pt, axis=1)
        min_dist = np.min(dists)
        candidate_local_idx = np.argmin(dists)
        candidate_idx = valid_candidates[candidate_local_idx]
        
        if boundary_all.shape[0] > 0:
            dist_to_boundary = np.min(np.linalg.norm(boundary_all - pts_bg[candidate_idx], axis=1))
        else:
            dist_to_boundary = np.inf
        
        # Si el candidato cumple la condición de cercanía o se toca la frontera, se anota y se evalúa el extremo opuesto.
        if min_dist <= threshold or dist_to_boundary <= threshold:
            brazo_indices.append(candidate_idx)
            conexion = {
                'punto_conexion': bg_cart.iloc[candidate_idx],
                'distancia_a_frontera': dist_to_boundary
            }
            brazo_opuesto = trace_opposite_extremity(candidate_idx, pts_bg, remaining_indices, boundary_all, threshold)
            if brazo_opuesto is not None:
                brazo_indices.extend(brazo_opuesto['indices'])
                conexion['extremo_opuesto'] = brazo_opuesto['punto_final']
            else:
                conexion['extremo_opuesto'] = None
            break
        else:
            brazo_indices.append(candidate_idx)
            current_idx = candidate_idx
            remaining_indices.remove(candidate_idx)
    
    brazo = bg_cart.iloc[brazo_indices].copy()
    conexiones = conexion if 'conexion' in locals() else None
    return {'brazo': brazo, 'conexiones': conexiones}

def trace_opposite_extremity(current_idx, pts_bg, remaining_indices, boundary_all, threshold):
    """
    A partir de current_idx (punto que tocó la frontera), se busca extender el brazo en sentido opuesto
    (por ejemplo, candidatos con x mayores) para evaluar conexión con otro grupo final.
    Retorna un diccionario con:
       - 'indices': la secuencia de índices extendida.
       - 'punto_final': el último índice agregado.
    O None si no se encuentra conexión.
    """
    brazo_indices = []
    cur_idx = current_idx
    valid_candidates = [i for i in remaining_indices if pts_bg[i,0] > pts_bg[cur_idx,0]]
    if not valid_candidates:
        return None
    candidates = pts_bg[valid_candidates]
    dists = np.linalg.norm(candidates - pts_bg[cur_idx].reshape(1,2), axis=1)
    min_dist = np.min(dists)
    candidate_local_idx = np.argmin(dists)
    candidate_idx = valid_candidates[candidate_local_idx]
    
    if boundary_all.shape[0] > 0:
        dist_to_boundary = np.min(np.linalg.norm(boundary_all - pts_bg[candidate_idx], axis=1))
    else:
        dist_to_boundary = np.inf
    
    if min_dist <= threshold or dist_to_boundary <= threshold:
        brazo_indices.append(candidate_idx)
        return {'indices': brazo_indices, 'punto_final': candidate_idx}
    return None

# =============================================================================
# 2) Función de graficado final: brazo espiral con background y grupos
# =============================================================================

def plot_spiral_arm_with_background(arm_dict, background, grupos_finales, title="Brazo Espiral (θ, r) con Background"):
    """
    Grafica el brazo espiral obtenido (arm_dict['brazo']) en (θ, r) y superpone:
      - Todos los puntos del background (alpha=0.1).
      - Los puntos de cada grupo final (alpha=0.1).
    """
    plt.figure(figsize=(10,7))
    
    # Graficar background en gris (alpha=0.1)
    plt.scatter(background['theta'], background['r'], c='gray', s=3, alpha=0.1, label="Background")
    
    # Graficar cada grupo final con baja opacidad
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','lime']
    for i, grupo in enumerate(grupos_finales):
        pts = grupo['points']
        c = colors[i % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, alpha=0.1, label=f"Grupo {i+1}")
    
    # Graficar el brazo espiral resaltado
    brazo = arm_dict['brazo']
    plt.scatter(brazo['theta'], brazo['r'], color='blue', s=20, label="Brazo Espiral")
    plt.plot(brazo['theta'], brazo['r'], '-o', color='red', label="Camino del Brazo")
    
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.title(title)
    plt.grid(True)
    plt.legend(loc="best")
    plt.show()

# =============================================================================
# MAIN: Ejecución de toda la pipeline
# =============================================================================

if __name__ == "__main__":
    # Paso 1: Generar los grupos finales y extraer el background
    datos_finales = main_example_modificado()
    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']
    
    # Paso 2: Trazar el brazo espiral usando el background y la información de los grupos finales
    if not background.empty and len(grupos_finales) > 0:
        arm_result = trace_spiral_arm(background, grupos_finales, tol_factor=1.1)
        print(f"Brazo espiral trazado con {len(arm_result['brazo'])} puntos.")
        if arm_result['conexiones']:
            print("Conexión establecida con la frontera de algún grupo final:")
            print(arm_result['conexiones'])
        
        # Graficar el brazo espiral superpuesto a background y grupos finales
        plot_spiral_arm_with_background(arm_result, background, grupos_finales)
    else:
        print("No hay datos suficientes en el background o en los grupos finales para trazar el brazo espiral.")


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Código completo para:
  1. Procesar y agrupar los datos (etapa de agrupación) a partir de los CSV.
  2. Del conjunto background (puntos que no pertenecen a ningún grupo final) se trazan
     varios caminos (“raíces”) de conexión, recorriendo en θ descendente. Cada camino se
     forma comparando uno a muchos: se evalúa la cercanía entre un candidato y la unión de
     (background + fronteras de grupos finales). Los puntos internos de un grupo ya se 
     ignoran para ahorrar cómputo.
  3. Se grafican los caminos sobre el background y los grupos finales (con transparencia α = 0.1).
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import cdist

# =============================================================================
# 0) Funciones para la etapa de agrupación (primer paso)
# =============================================================================

def calculate_pa(slope, intercept):
    """
    Calcula el pitch-angle (PA) en grados:
        PA = arctan[(slope * 180/π) / intercept], si intercept ≠ 0.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan

def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    """
    Carga el archivo CSV, calcula 'r' y 'theta', filtra según el rango de θ y añade
    una columna 'id' para preservar el índice original.
    """
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['id'] = df.index
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo de adyacencia (BFS) en el espacio (θ, r) usando una tolerancia.
    """
    th, rr = df_points['theta'].values, df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        mask = (np.abs(th[i] - th) <= theta_diff) & (np.abs(rr[i] - rr) <= r_diff) & (np.arange(n) != i)
        graph[i] = list(np.where(mask)[0])
    return graph, n

def bfs_components(graph, df_points):
    """
    Extrae componentes conexas usando BFS.
    """
    visited, clusters = [False]*len(graph), []
    for s in range(len(graph)):
        if not visited[s]:
            q, comp = deque([s]), [s]
            visited[s] = True
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Subdivide un clúster en subgrupos si existen "gaps" (saltos en θ o r).
    """
    col = 'theta' if mode == 'theta' else 'r'
    df = df_cluster.sort_values(col)
    dif = np.diff(df[col].values)
    if np.all(dif <= gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(dif):
        if d > gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i + 1
    subs.append(df.iloc[start:].copy())
    return subs

def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts / ref, 1.0))

def generate_bfs_seeds(id_halo, theta_min, theta_max,
                         quartile_threshold=0.55,
                         theta_diff=3.0, r_diff=0.5,
                         gap_threshold_theta=2.0, gap_threshold_r=2.0,
                         file_prefix='data_rho'):
    """
    Genera las semillas BFS a partir de los datos filtrados y retorna:
       - final_cl: lista de clústeres resultantes.
       - df_f: DataFrame filtrado (datos base usados para agrupación).
    """
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp'] > thr].copy().reset_index(drop=True)

    f = _adaptive_factor(len(df_f))
    th_diff, r_d = theta_diff/f, r_diff/f
    gap_th, gap_r = gap_threshold_theta/f, gap_threshold_r/f

    graph, _ = build_graph_rectangular(df_f, th_diff, r_d)
    clusters = bfs_components(graph, df_f)

    final_cl = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_th, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r, 'r'))
    return final_cl, df_f

def fit_line_to_cluster(df_cluster):
    """
    Ajusta una línea (regresión lineal) a los puntos del clúster.
    """
    if len(df_cluster) < 2:
        return (None,)*7
    mdl = LinearRegression().fit(df_cluster[['theta']], df_cluster['r'])
    s, b = mdl.coef_[0], mdl.intercept_
    pa = calculate_pa(s, b)
    return s, b, pa, df_cluster['theta'].min(), df_cluster['theta'].max(), df_cluster['r'].min(), df_cluster['r'].max()

def adjust_and_merge_seeds(bfs_clusters, slope_variation_threshold=0.40, bounding_extrap=0.30):
    """
    Fusiona los clústeres que se solapan en la caja definida.
    """
    groups = []
    for cl in bfs_clusters:
        groups.append(dict(zip(
            ['slope', 'intercept', 'pa', 'theta_min', 'theta_max', 'r_min', 'r_max', 'points'],
            (*fit_line_to_cluster(cl), cl)
        )))

    def recalc(g):
        g['slope'], g['intercept'], g['pa'], g['theta_min'], g['theta_max'], g['r_min'], g['r_max'] = \
            fit_line_to_cluster(g['points'])

    def boxes_overlap(g1, g2, e):
        def exp(t0, t1, r0, r1):
            dt, dr = (t1-t0)*e, (r1-r0)*e
            return t0-dt, t1+dt, r0-dr, r1+dr
        a = exp(g1['theta_min'], g1['theta_max'], g1['r_min'], g1['r_max'])
        b = exp(g2['theta_min'], g2['theta_max'], g2['r_min'], g2['r_max'])
        return not (a[1] < b[0] or b[1] < a[0]) and not (a[3] < b[2] or b[3] < a[2])

    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i+1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, b_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb - g1['slope'])/(abs(g1['slope'])+1e-12)
                        v2 = abs(s_comb - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb
                            recalc(g1)
                            groups.pop(j)
                            merged = True
                            continue
                j += 1
            new.append(g1)
            i += 1
        groups = new
    return [g for g in groups if len(g['points']) >= 2]

def validate_dispersion_and_reprocess(groups, dispersion_threshold=1.80, reproc_theta_diff=1.25, reproc_r_diff=0.4):
    """
    Valida la dispersión de cada grupo y re-procesa aquellos que superen el umbral.
    """
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points']) < 2:
            ok.append(g)
            continue
        res = g['points']['r'] - (g['slope'] * g['points']['theta'] + g['intercept'])
        (todo if res.std() > dispersion_threshold else ok).append(g)
    
    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            new.append(dict(zip(
                ['slope', 'intercept', 'pa', 'theta_min', 'theta_max', 'r_min', 'r_max', 'points'],
                (*fit_line_to_cluster(c), c)
            )))
    return ok + new

def plot_groups_cartesian(groups, df_all, line_extrap=0.15):
    """
    Visualiza en un gráfico cartesiano los grupos finales (con ≥ 60 puntos) en (θ, r).
    """
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        plt.scatter(pts['theta'], pts['r'], s=10, color=c,
                    label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t = np.linspace(g['theta_min'] - line_extrap * dt, g['theta_max'] + line_extrap * dt, 200)
            plt.plot(t, g['slope'] * t + g['intercept'], '--', color=c)
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.grid(True)
    plt.title("Subhalo 17 – grupos ≥60 pts (cartesiano, PA visible)")
    plt.legend()
    plt.show()

def plot_groups_polar(groups, df_all, line_extrap=0.15):
    """
    Visualización en proyección polar de los grupos finales (con ≥ 60 puntos).
    """
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos (polar): {len(big)}")
    fig = plt.figure(figsize=(9,8))
    ax = plt.subplot(111, projection='polar')
    ax.scatter(np.radians(df_all['theta']), df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        ax.scatter(np.radians(pts['theta']), pts['r'], s=10, color=c,
                   label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t_deg = np.linspace(g['theta_min'] - line_extrap * dt, g['theta_max'] + line_extrap * dt, 200)
            r_line = g['slope'] * t_deg + g['intercept']
            ax.plot(np.radians(t_deg), r_line, '.-', color="gray", alpha=0.20)
            ax.plot(np.radians(t_deg), r_line, '--', color=c)
    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title("Subhalo 17 – proyección polar (PA visible)", y=1.05)
    ax.legend(loc='upper right', bbox_to_anchor=(1.20, 1.0))
    plt.show()

def main_example_modificado():
    """
    Ejecuta el pipeline completo para subhalo 17.
    Retorna un diccionario con:
       - 'grupos_finales': lista de grupos (con ≥ 60 puntos)
       - 'background': DataFrame con los puntos que no están en ningún grupo final.
    """
    bfs, df_f = generate_bfs_seeds(
        id_halo="17",
        theta_min=0, theta_max=360,
        quartile_threshold=0.155,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=2.0, gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")
    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.30, bounding_extrap=0.35)
    print(f"Tras fusión: {len(merged)} grupos")
    validated = validate_dispersion_and_reprocess(merged)
    print(f"Tras validación: {len(validated)} grupos")
    plot_groups_cartesian(validated, df_f, line_extrap=0.15)
    plot_groups_polar(validated, df_f, line_extrap=0.15)
    
    # Almacenar grupos finales: únicamente aquellos con 60 o más puntos.
    grupos_finales = [g for g in validated if len(g['points']) >= 60]
    
    # Determinar los puntos (IDs) que aparecen en los grupos finales.
    puntos_grupos = set()
    for g in grupos_finales:
        puntos_grupos.update(g['points']['id'].tolist())
    
    # Los puntos "background" son aquellos de df_f que no están en ningún grupo final.
    background = df_f[~df_f['id'].isin(puntos_grupos)].copy()
    print(f"Puntos background: {len(background)}")
    
    return {'grupos_finales': grupos_finales, 'background': background}

# =============================================================================
# 1) Funciones para el trazado de varios caminos ("raíces") a partir del background,
#    recorriendo en orden descendente de θ
# =============================================================================

def trace_spiral_arms(background, grupos_finales, tol_factor=1.1):
    """
    Recorre el conjunto de puntos del background (en coordenadas polares) para generar
    *varios* caminos independientes (raíces) de conexión. Se parte de la semilla con mayor
    valor de θ y se sigue de forma greedy (solo se consideran candidatos con θ menor que el
    punto actual). Para cada candidato se evalúa su distancia (en coordenadas cartesianas)
    tanto respecto al punto actual como respecto a la unión de las fronteras de los grupos
    finales. Se selecciona el candidato si alguna de esas distancias es menor que threshold.
    
    Los caminos no se unirán entre sí: una vez iniciado un camino se remueven los puntos de
    background empleados, permitiendo la creación de ramas paralelas.
    
    Retorna una lista de DataFrames, cada uno representando un camino (brazo espiral).
    """
    # Ordenar background por θ descendente.
    bg = background.copy().sort_values("theta", ascending=False).reset_index(drop=True)
    
    # Función para convertir un punto (fila) a coordenadas cartesianas.
    def to_cartesian(row):
        rad = np.radians(row['theta'])
        return np.array([row['r'] * np.cos(rad), row['r'] * np.sin(rad)])
    
    # Obtener la unión de las fronteras de los grupos finales:
    boundary_list = []
    for grupo in grupos_finales:
        pts_grp = grupo['points']
        for _, row in pts_grp.iterrows():
            boundary_list.append(to_cartesian(row))
    if boundary_list:
        boundary_all = np.vstack(boundary_list)
    else:
        boundary_all = np.empty((0,2))
    
    # Para definir threshold, calcular la distancia promedio entre vecinos en el background.
    pts_bg_cart = np.array([to_cartesian(row) for _, row in bg.iterrows()])
    if len(pts_bg_cart) < 2:
        avg_nn = np.inf
    else:
        dists = cdist(pts_bg_cart, pts_bg_cart)
        np.fill_diagonal(dists, np.inf)
        avg_nn = np.mean(np.min(dists, axis=1))
    threshold = tol_factor * avg_nn
    
    arms = []  # Lista para almacenar los caminos resultantes.
    
    # Mientras hayan puntos en el background:
    while not bg.empty:
        # Semilla: punto con mayor θ.
        seed_idx = bg['theta'].idxmax()
        seed = bg.loc[seed_idx]
        path = [seed]
        # Remover la semilla del background.
        bg = bg.drop(seed_idx).reset_index(drop=True)
        
        current = seed
        while True:
            # Seleccionar candidatos: aquellos con θ menor que el punto actual.
            candidates = bg[bg['theta'] < current['theta']]
            if candidates.empty:
                break
            # Calcular la distancia desde el punto actual a cada candidato en coordenadas cartesianas.
            current_cart = to_cartesian(current)
            candidate_coords = np.array([to_cartesian(row) for _, row in candidates.iterrows()])
            dists_current = np.linalg.norm(candidate_coords - current_cart, axis=1)
            # Calcular la distancia de cada candidato a la unión de fronteras.
            if boundary_all.shape[0] > 0:
                dists_boundary = np.array([np.min(np.linalg.norm(candidate - boundary_all, axis=1))
                                           for candidate in candidate_coords])
            else:
                dists_boundary = np.full(len(candidates), np.inf)
            # Se define la distancia efectiva como el mínimo entre la distancia al actual y a la frontera.
            effective_dists = np.minimum(dists_current, dists_boundary)
            min_eff = np.min(effective_dists)
            idx_min = np.argmin(effective_dists)
            # Verificar condición.
            if min_eff <= threshold:
                # Agregar el candidato a la ruta.
                candidate = candidates.iloc[idx_min]
                path.append(candidate)
                current = candidate
                # Remover candidato del background.
                bg = bg.drop(candidate.name).reset_index(drop=True)
            else:
                # No hay candidato cercano, se termina el camino.
                break
        
        arms.append(pd.DataFrame(path))
    
    return arms

# =============================================================================
# 2) Función de graficado final: muestra los caminos (raíces) junto con background y grupos
# =============================================================================

def plot_arms_with_background(arms, background, grupos_finales, title="Caminos (raíces) en (θ, r) con Background"):
    """
    Grafica los caminos trazados (lista de DataFrames) sobre el background y los puntos de
    los grupos finales. Se emplea α = 0.1 para background y grupos, para resaltarlos de forma tenue.
    """
    plt.figure(figsize=(12,8))
    
    # Background con baja opacidad.
    plt.scatter(background['theta'], background['r'], c='gray', s=3, alpha=0.1, label="Background")
    
    # Grupos finales con baja opacidad.
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','lime']
    for i, grupo in enumerate(grupos_finales):
        pts = grupo['points']
        c = colors[i % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, alpha=0.1, label=f"Grupo {i+1}")
    
    # Graficar cada camino (brazo) con un color distinto.
    for j, arm in enumerate(arms):
        plt.plot(arm['theta'], arm['r'], '-o', label=f"Camino {j+1}", linewidth=2)
    
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.title(title)
    plt.grid(True)
    plt.legend(loc="best")
    plt.show()

# =============================================================================
# MAIN: Ejecución de toda la pipeline
# =============================================================================

if __name__ == "__main__":
    # Paso 1: Procesar y agrupar los datos
    datos_finales = main_example_modificado()
    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']
    
    # Paso 2: Trazar varios caminos ("raíces") a partir del background, recorriendo en θ descendente
    if not background.empty and len(grupos_finales) > 0:
        arms = trace_spiral_arms(background, grupos_finales, tol_factor=1.1)
        print(f"Se trazaron {len(arms)} caminos (raíces).")
        # Paso 3: Graficar los caminos junto con background y grupos finales
        plot_arms_with_background(arms, background, grupos_finales)
    else:
        print("No hay datos suficientes en el background o en los grupos finales para trazar los caminos.")


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Código completo para:
  1. Procesar y agrupar los datos (etapa de agrupación) a partir de los CSV.
  2. Del conjunto background (puntos que no pertenecen a ningún grupo final) se trazan
     varios caminos ("raíces") de conexión, recorriendo en θ descendente. Cada camino se
     forma comparando uno a muchos: se evalúa la cercanía entre un candidato y la unión de
     (background + fronteras de grupos finales). Se ignoran los puntos internos de un grupo
     para ahorrar cómputo. Además, se fuerza que los caminos no estén separados de otros grupos
     por una distancia mayor fija de 0.65 (en r y θ), salvo cuando se considere una frontera.
  3. Se grafican los caminos sobre el background y los grupos finales (con transparencia α = 0.1).
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import cdist

# =============================================================================
# 0) Funciones para la etapa de agrupación (primer paso)
# =============================================================================

def calculate_pa(slope, intercept):
    """
    Calcula el pitch-angle (PA) en grados:
        PA = arctan[(slope * 180/π) / intercept], si intercept ≠ 0.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan

def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    """
    Carga el archivo CSV, calcula 'r' y 'theta', filtra según el rango de θ y añade
    una columna 'id' para preservar el índice original.
    """
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['id'] = df.index
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo de adyacencia (BFS) en el espacio (θ, r) usando una tolerancia.
    """
    th, rr = df_points['theta'].values, df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        mask = (np.abs(th[i] - th) <= theta_diff) & (np.abs(rr[i] - rr) <= r_diff) & (np.arange(n) != i)
        graph[i] = list(np.where(mask)[0])
    return graph, n

def bfs_components(graph, df_points):
    """
    Extrae componentes conexas usando BFS.
    """
    visited, clusters = [False]*len(graph), []
    for s in range(len(graph)):
        if not visited[s]:
            q, comp = deque([s]), [s]
            visited[s] = True
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Subdivide un clúster en subgrupos si existen "gaps" (saltos en θ o r).
    """
    col = 'theta' if mode == 'theta' else 'r'
    df = df_cluster.sort_values(col)
    dif = np.diff(df[col].values)
    if np.all(dif <= gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(dif):
        if d > gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i + 1
    subs.append(df.iloc[start:].copy())
    return subs

def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts / ref, 1.0))

def generate_bfs_seeds(id_halo, theta_min, theta_max,
                         quartile_threshold=0.55,
                         theta_diff=3.0, r_diff=0.5,
                         gap_threshold_theta=2.0, gap_threshold_r=2.0,
                         file_prefix='data_rho'):
    """
    Genera las semillas BFS a partir de los datos filtrados y retorna:
       - final_cl: lista de clústeres resultantes.
       - df_f: DataFrame filtrado (datos base usados para agrupación).
    """
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp'] > thr].copy().reset_index(drop=True)

    f = _adaptive_factor(len(df_f))
    th_diff, r_d = theta_diff/f, r_diff/f
    gap_th, gap_r = gap_threshold_theta/f, gap_threshold_r/f

    graph, _ = build_graph_rectangular(df_f, th_diff, r_d)
    clusters = bfs_components(graph, df_f)

    final_cl = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_th, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r, 'r'))
    return final_cl, df_f

def fit_line_to_cluster(df_cluster):
    """
    Ajusta una línea (regresión lineal) a los puntos del clúster.
    """
    if len(df_cluster) < 2:
        return (None,)*7
    mdl = LinearRegression().fit(df_cluster[['theta']], df_cluster['r'])
    s, b = mdl.coef_[0], mdl.intercept_
    pa = calculate_pa(s, b)
    return s, b, pa, df_cluster['theta'].min(), df_cluster['theta'].max(), df_cluster['r'].min(), df_cluster['r'].max()

def adjust_and_merge_seeds(bfs_clusters, slope_variation_threshold=0.40, bounding_extrap=0.30):
    """
    Fusiona los clústeres que se solapan en la caja definida.
    """
    groups = []
    for cl in bfs_clusters:
        groups.append(dict(zip(
            ['slope', 'intercept', 'pa', 'theta_min', 'theta_max', 'r_min', 'r_max', 'points'],
            (*fit_line_to_cluster(cl), cl)
        )))

    def recalc(g):
        g['slope'], g['intercept'], g['pa'], g['theta_min'], g['theta_max'], g['r_min'], g['r_max'] = \
            fit_line_to_cluster(g['points'])

    def boxes_overlap(g1, g2, e):
        def exp(t0, t1, r0, r1):
            dt, dr = (t1-t0)*e, (r1-r0)*e
            return t0-dt, t1+dt, r0-dr, r1+dr
        a = exp(g1['theta_min'], g1['theta_max'], g1['r_min'], g1['r_max'])
        b = exp(g2['theta_min'], g2['theta_max'], g2['r_min'], g2['r_max'])
        return not (a[1] < b[0] or b[1] < a[0]) and not (a[3] < b[2] or b[3] < a[2])

    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i+1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, b_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb - g1['slope'])/(abs(g1['slope'])+1e-12)
                        v2 = abs(s_comb - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb
                            recalc(g1)
                            groups.pop(j)
                            merged = True
                            continue
                j += 1
            new.append(g1)
            i += 1
        groups = new
    return [g for g in groups if len(g['points']) >= 2]

def validate_dispersion_and_reprocess(groups, dispersion_threshold=1.80, reproc_theta_diff=1.25, reproc_r_diff=0.4):
    """
    Valida la dispersión de cada grupo y re-procesa aquellos que superen el umbral.
    """
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points']) < 2:
            ok.append(g)
            continue
        res = g['points']['r'] - (g['slope'] * g['points']['theta'] + g['intercept'])
        (todo if res.std() > dispersion_threshold else ok).append(g)
    
    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            new.append(dict(zip(
                ['slope', 'intercept', 'pa', 'theta_min', 'theta_max', 'r_min', 'r_max', 'points'],
                (*fit_line_to_cluster(c), c)
            )))
    return ok + new

def plot_groups_cartesian(groups, df_all, line_extrap=0.15):
    """
    Visualiza en un gráfico cartesiano los grupos finales (con ≥ 60 puntos) en (θ, r).
    """
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        plt.scatter(pts['theta'], pts['r'], s=10, color=c,
                    label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t = np.linspace(g['theta_min'] - line_extrap * dt, g['theta_max'] + line_extrap * dt, 200)
            plt.plot(t, g['slope'] * t + g['intercept'], '--', color=c)
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.grid(True)
    plt.title("Subhalo 17 – grupos ≥60 pts (cartesiano, PA visible)")
    plt.legend()
    plt.show()

def plot_groups_polar(groups, df_all, line_extrap=0.15):
    """
    Visualización en proyección polar de los grupos finales (con ≥ 60 puntos).
    """
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos (polar): {len(big)}")
    fig = plt.figure(figsize=(9,8))
    ax = plt.subplot(111, projection='polar')
    ax.scatter(np.radians(df_all['theta']), df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        ax.scatter(np.radians(pts['theta']), pts['r'], s=10, color=c,
                   label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t_deg = np.linspace(g['theta_min'] - line_extrap * dt, g['theta_max'] + line_extrap * dt, 200)
            r_line = g['slope'] * t_deg + g['intercept']
            ax.plot(np.radians(t_deg), r_line, '.-', color="gray", alpha=0.20)
            ax.plot(np.radians(t_deg), r_line, '--', color=c)
    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title("Subhalo 17 – proyección polar (PA visible)", y=1.05)
    ax.legend(loc='upper right', bbox_to_anchor=(1.20, 1.0))
    plt.show()

def main_example_modificado():
    """
    Ejecuta el pipeline completo para subhalo 17.
    Retorna un diccionario con:
       - 'grupos_finales': lista de grupos (con ≥ 60 pts)
       - 'background': DataFrame con los puntos que no están en ningún grupo final.
    """
    bfs, df_f = generate_bfs_seeds(
        id_halo="17",
        theta_min=0, theta_max=360,
        quartile_threshold=0.155,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=2.0, gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")
    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.30, bounding_extrap=0.35)
    print(f"Tras fusión: {len(merged)} grupos")
    validated = validate_dispersion_and_reprocess(merged)
    print(f"Tras validación: {len(validated)} grupos")
    plot_groups_cartesian(validated, df_f, line_extrap=0.15)
    plot_groups_polar(validated, df_f, line_extrap=0.15)
    
    # Almacenar grupos finales: únicamente aquellos con 60 o más puntos.
    grupos_finales = [g for g in validated if len(g['points']) >= 60]
    
    # Determinar los puntos (IDs) que aparecen en los grupos finales.
    puntos_grupos = set()
    for g in grupos_finales:
        puntos_grupos.update(g['points']['id'].tolist())
    
    # Los puntos "background" son aquellos de df_f que no están en ningún grupo final.
    background = df_f[~df_f['id'].isin(puntos_grupos)].copy()
    print(f"Puntos background: {len(background)}")
    
    return {'grupos_finales': grupos_finales, 'background': background}

# =============================================================================
# 1) Funciones para el trazado de varios caminos ("raíces") a partir del background,
#    recorriendo en orden descendente de θ
# =============================================================================

def trace_spiral_arms(background, grupos_finales, tol_factor=1.1):
    """
    Recorre el conjunto de puntos del background (en coordenadas polares) para generar
    *varios* caminos independientes ("raíces") de conexión. Se parte de la semilla con mayor
    valor de θ y se sigue de forma greedy (solo se consideran candidatos con θ menor que el
    punto actual). Para cada candidato se evalúa su distancia (en coordenadas cartesianas)
    tanto respecto al punto actual como respecto a la unión de las fronteras de los grupos
    finales. Se selecciona el candidato si alguna de esas distancias es menor que el threshold.
    
    Además, se fuerza que el threshold no supere un valor fijo de 0.65 (en unidades compatibles),
    de forma que los caminos no se separen de otros grupos por más de esa distancia.
    
    Los caminos se generan de forma paralela (una vez empleado un punto se remueve del background)
    para formar ramas separadas.
    
    Retorna una lista de DataFrames, cada uno representando un camino (brazo espiral).
    """
    # Ordenar background por θ descendente.
    bg = background.copy().sort_values("theta", ascending=False).reset_index(drop=True)
    
    # Función para convertir un punto (fila) a coordenadas cartesianas.
    def to_cartesian(row):
        rad = np.radians(row['theta'])
        return np.array([row['r'] * np.cos(rad), row['r'] * np.sin(rad)])
    
    # Obtener la unión de las fronteras de los grupos finales.
    boundary_list = []
    for grupo in grupos_finales:
        pts_grp = grupo['points']
        for _, row in pts_grp.iterrows():
            boundary_list.append(to_cartesian(row))
    if boundary_list:
        boundary_all = np.vstack(boundary_list)
    else:
        boundary_all = np.empty((0,2))
    
    # Para definir threshold: calcular la distancia promedio entre vecinos en el background.
    pts_bg_cart = np.array([to_cartesian(row) for _, row in bg.iterrows()])
    if len(pts_bg_cart) < 2:
        avg_nn = np.inf
    else:
        dists = cdist(pts_bg_cart, pts_bg_cart)
        np.fill_diagonal(dists, np.inf)
        avg_nn = np.mean(np.min(dists, axis=1))
    # Se fuerza que el threshold no supere 0.65 (para r y θ en unidades compatibles)
    threshold = min(tol_factor * avg_nn, 0.65)
    
    arms = []  # Lista para almacenar los caminos resultantes.
    
    # Mientras queden puntos en el background:
    while not bg.empty:
        # Semilla: el punto con mayor θ.
        seed_idx = bg['theta'].idxmax()
        seed = bg.loc[seed_idx]
        path = [seed]
        # Remover la semilla del background.
        bg = bg.drop(seed_idx).reset_index(drop=True)
        
        current = seed
        while True:
            # Seleccionar candidatos con θ menor que la del punto actual.
            candidates = bg[bg['theta'] < current['theta']]
            if candidates.empty:
                break
            # Convertir candidatos a coordenadas cartesianas.
            candidate_coords = np.array([to_cartesian(row) for _, row in candidates.iterrows()])
            current_cart = to_cartesian(current)
            dists_current = np.linalg.norm(candidate_coords - current_cart, axis=1)
            # Calcular la distancia de cada candidato a la unión de fronteras.
            if boundary_all.shape[0] > 0:
                dists_boundary = np.array([np.min(np.linalg.norm(candidate - boundary_all, axis=1))
                                           for candidate in candidate_coords])
            else:
                dists_boundary = np.full(len(candidates), np.inf)
            # La distancia efectiva será el mínimo entre la distancia al actual y a la frontera.
            effective_dists = np.minimum(dists_current, dists_boundary)
            min_eff = np.min(effective_dists)
            idx_min = np.argmin(effective_dists)
            
            # Si el mínimo efectivo es menor o igual al threshold, se añade el candidato.
            if min_eff <= threshold:
                candidate = candidates.iloc[idx_min]
                path.append(candidate)
                current = candidate
                # Remover el candidato del background.
                bg = bg.drop(candidate.name).reset_index(drop=True)
            else:
                # Si no se encuentra candidato cercano, se termina el camino.
                break
        
        arms.append(pd.DataFrame(path))
    
    return arms

# =============================================================================
# 2) Función de graficado final: muestra los caminos ("raíces") junto con background y grupos
# =============================================================================

def plot_arms_with_background(arms, background, grupos_finales, title="Caminos (raíces) en (θ, r) con Background"):
    """
    Grafica los caminos trazados (lista de DataFrames) sobre el background y los puntos de
    los grupos finales. Se emplea α = 0.1 para background y grupos, para resaltarlos de forma tenue.
    """
    plt.figure(figsize=(12,8))
    
    # Background con baja opacidad.
    plt.scatter(background['theta'], background['r'], c='gray', s=3, alpha=0.1, label="Background")
    
    # Grupos finales con baja opacidad.
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','lime']
    for i, grupo in enumerate(grupos_finales):
        pts = grupo['points']
        c = colors[i % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, alpha=0.1, label=f"Grupo {i+1}")
    
    # Graficar cada camino (brazo) con un color distinto.
    for j, arm in enumerate(arms):
        plt.plot(arm['theta'], arm['r'], '-o', label=f"Camino {j+1}", linewidth=2)
    
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.title(title)
    plt.grid(True)
    plt.legend(loc="best")
    plt.show()

# =============================================================================
# MAIN: Ejecución de toda la pipeline
# =============================================================================

if __name__ == "__main__":
    # Paso 1: Procesar y agrupar los datos
    datos_finales = main_example_modificado()
    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']
    
    # Paso 2: Trazar varios caminos ("raíces") a partir del background, recorriendo en θ descendente
    if not background.empty and len(grupos_finales) > 0:
        arms = trace_spiral_arms(background, grupos_finales, tol_factor=1.1)
        print(f"Se trazaron {len(arms)} caminos (raíces).")
        # Paso 3: Graficar los caminos junto con background y grupos finales
        plot_arms_with_background(arms, background, grupos_finales)
    else:
        print("No hay datos suficientes en el background o en los grupos finales para trazar los caminos.")


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Código completo para:
  1. Procesar y agrupar los datos (etapa de agrupación) a partir de los CSV.
  2. Del conjunto background (puntos que no pertenecen a ningún grupo final) se trazan
     varios caminos ("raíces") de conexión, recorriendo en orden descendente de θ.
     Aquí se exige que el camino sea físicamente continuo, es decir, que los puntos conectados
     tengan una distancia estricta (máx. 0.65 o tol_factor * promedio) y que la dirección de
     cada segmento no varíe abruptamente (diferencia angular menor a 10°).
  3. Se grafican los caminos sobre el background y los grupos finales (con α = 0.1).
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import cdist

# =============================================================================
# 0) Funciones para la etapa de agrupación (primer paso)
# =============================================================================

def calculate_pa(slope, intercept):
    """
    Calcula el pitch-angle (PA) en grados:
        PA = arctan[(slope * 180/π) / intercept], si intercept ≠ 0.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan

def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    """
    Carga el archivo CSV, calcula 'r' y 'theta', filtra según el rango de θ y añade
    una columna 'id' para preservar el índice original.
    """
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['id'] = df.index
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo de adyacencia (BFS) en el espacio (θ, r) usando una tolerancia.
    """
    th, rr = df_points['theta'].values, df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        mask = (np.abs(th[i] - th) <= theta_diff) & (np.abs(rr[i] - rr) <= r_diff) & (np.arange(n) != i)
        graph[i] = list(np.where(mask)[0])
    return graph, n

def bfs_components(graph, df_points):
    """
    Extrae componentes conexas usando BFS.
    """
    visited, clusters = [False]*len(graph), []
    for s in range(len(graph)):
        if not visited[s]:
            q, comp = deque([s]), [s]
            visited[s] = True
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Subdivide un clúster en subgrupos si existen "gaps" (saltos en θ o r).
    """
    col = 'theta' if mode == 'theta' else 'r'
    df = df_cluster.sort_values(col)
    dif = np.diff(df[col].values)
    if np.all(dif <= gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(dif):
        if d > gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i + 1
    subs.append(df.iloc[start:].copy())
    return subs

def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts / ref, 1.0))

def generate_bfs_seeds(id_halo, theta_min, theta_max,
                         quartile_threshold=0.55,
                         theta_diff=3.0, r_diff=0.5,
                         gap_threshold_theta=2.0, gap_threshold_r=2.0,
                         file_prefix='data_rho'):
    """
    Genera las semillas BFS a partir de los datos filtrados y retorna:
       - final_cl: lista de clústeres resultantes.
       - df_f: DataFrame filtrado (datos base usados para agrupación).
    """
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp'] > thr].copy().reset_index(drop=True)

    f = _adaptive_factor(len(df_f))
    th_diff, r_d = theta_diff/f, r_diff/f
    gap_th, gap_r = gap_threshold_theta/f, gap_threshold_r/f

    graph, _ = build_graph_rectangular(df_f, th_diff, r_d)
    clusters = bfs_components(graph, df_f)

    final_cl = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_th, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r, 'r'))
    return final_cl, df_f

def fit_line_to_cluster(df_cluster):
    """
    Ajusta una línea (regresión lineal) a los puntos del clúster.
    """
    if len(df_cluster) < 2:
        return (None,)*7
    mdl = LinearRegression().fit(df_cluster[['theta']], df_cluster['r'])
    s, b = mdl.coef_[0], mdl.intercept_
    pa = calculate_pa(s, b)
    return s, b, pa, df_cluster['theta'].min(), df_cluster['theta'].max(), df_cluster['r'].min(), df_cluster['r'].max()

def adjust_and_merge_seeds(bfs_clusters, slope_variation_threshold=0.40, bounding_extrap=0.30):
    """
    Fusiona los clústeres que se solapan en la caja definida.
    """
    groups = []
    for cl in bfs_clusters:
        groups.append(dict(zip(
            ['slope', 'intercept', 'pa', 'theta_min', 'theta_max', 'r_min', 'r_max', 'points'],
            (*fit_line_to_cluster(cl), cl)
        )))

    def recalc(g):
        g['slope'], g['intercept'], g['pa'], g['theta_min'], g['theta_max'], g['r_min'], g['r_max'] = \
            fit_line_to_cluster(g['points'])

    def boxes_overlap(g1, g2, e):
        def exp(t0, t1, r0, r1):
            dt, dr = (t1-t0)*e, (r1-r0)*e
            return t0-dt, t1+dt, r0-dr, r1+dr
        a = exp(g1['theta_min'], g1['theta_max'], g1['r_min'], g1['r_max'])
        b = exp(g2['theta_min'], g2['theta_max'], g2['r_min'], g2['r_max'])
        return not (a[1] < b[0] or b[1] < a[0]) and not (a[3] < b[2] or b[3] < a[2])

    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i+1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, b_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb - g1['slope'])/(abs(g1['slope'])+1e-12)
                        v2 = abs(s_comb - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb
                            recalc(g1)
                            groups.pop(j)
                            merged = True
                            continue
                j += 1
            new.append(g1)
            i += 1
        groups = new
    return [g for g in groups if len(g['points']) >= 2]

def validate_dispersion_and_reprocess(groups, dispersion_threshold=1.80, reproc_theta_diff=1.25, reproc_r_diff=0.4):
    """
    Valida la dispersión de cada grupo y re-procesa aquellos que superen el umbral.
    """
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points']) < 2:
            ok.append(g)
            continue
        res = g['points']['r'] - (g['slope'] * g['points']['theta'] + g['intercept'])
        (todo if res.std() > dispersion_threshold else ok).append(g)
    
    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            new.append(dict(zip(
                ['slope', 'intercept', 'pa', 'theta_min', 'theta_max', 'r_min', 'r_max', 'points'],
                (*fit_line_to_cluster(c), c)
            )))
    return ok + new

def plot_groups_cartesian(groups, df_all, line_extrap=0.15):
    """
    Visualiza en un gráfico cartesiano los grupos finales (con ≥ 60 puntos) en (θ, r).
    """
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        plt.scatter(pts['theta'], pts['r'], s=10, color=c,
                    label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t = np.linspace(g['theta_min'] - line_extrap * dt, g['theta_max'] + line_extrap * dt, 200)
            plt.plot(t, g['slope'] * t + g['intercept'], '--', color=c)
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.grid(True)
    plt.title("Subhalo 17 – grupos ≥60 pts (cartesiano, PA visible)")
    plt.legend()
    plt.show()

def plot_groups_polar(groups, df_all, line_extrap=0.15):
    """
    Visualización en proyección polar de los grupos finales (con ≥ 60 puntos).
    """
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos (polar): {len(big)}")
    fig = plt.figure(figsize=(9,8))
    ax = plt.subplot(111, projection='polar')
    ax.scatter(np.radians(df_all['theta']), df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        ax.scatter(np.radians(pts['theta']), pts['r'], s=10, color=c,
                   label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t_deg = np.linspace(g['theta_min'] - line_extrap * dt, g['theta_max'] + line_extrap * dt, 200)
            r_line = g['slope'] * t_deg + g['intercept']
            ax.plot(np.radians(t_deg), r_line, '.-', color="gray", alpha=0.20)
            ax.plot(np.radians(t_deg), r_line, '--', color=c)
    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title("Subhalo 17 – proyección polar (PA visible)", y=1.05)
    ax.legend(loc='upper right', bbox_to_anchor=(1.20, 1.0))
    plt.show()

def main_example_modificado():
    """
    Ejecuta el pipeline completo para subhalo 17.
    Retorna un diccionario con:
       - 'grupos_finales': lista de grupos (con ≥ 60 pts)
       - 'background': DataFrame con los puntos que no están en ningún grupo final.
    """
    bfs, df_f = generate_bfs_seeds(
        id_halo="17",
        theta_min=0, theta_max=360,
        quartile_threshold=0.155,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=2.0, gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")
    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.30, bounding_extrap=0.35)
    print(f"Tras fusión: {len(merged)} grupos")
    validated = validate_dispersion_and_reprocess(merged)
    print(f"Tras validación: {len(validated)} grupos")
    plot_groups_cartesian(validated, df_f, line_extrap=0.15)
    plot_groups_polar(validated, df_f, line_extrap=0.15)
    
    # Almacenar grupos finales: únicamente aquellos con 60 o más puntos.
    grupos_finales = [g for g in validated if len(g['points']) >= 60]
    
    # Determinar los puntos (IDs) que aparecen en los grupos finales.
    puntos_grupos = set()
    for g in grupos_finales:
        puntos_grupos.update(g['points']['id'].tolist())
    
    # Los puntos "background" son aquellos de df_f que no están en ningún grupo final.
    background = df_f[~df_f['id'].isin(puntos_grupos)].copy()
    print(f"Puntos background: {len(background)}")
    
    return {'grupos_finales': grupos_finales, 'background': background}

# =============================================================================
# 1) Funciones para el trazado de varios caminos ("raíces") a partir del background,
#    recorriendo en orden descendente de θ, con conexión "física" y estricta
# =============================================================================

def trace_spiral_arms(background, grupos_finales, tol_factor=1.1, max_angle_diff=10):
    """
    Genera varios caminos ("raíces") a partir del background (en coordenadas polares),
    recorriendo en orden descendente de θ. Se parte del punto semilla con mayor θ y para cada
    candidato se evalúan dos condiciones:
      1. La distancia en coordenadas cartesianas desde el punto actual debe ser menor a un threshold,
         definido como min(tol_factor * (promedio distancia entre vecinos), 0.65).
      2. La diferencia angular entre el segmento actual (la dirección del camino) y la dirección
         hacia el candidato debe ser menor a max_angle_diff (en grados).
    Estos criterios deben cumplirse para formar un camino continuo y físico.
    
    Para evaluar la condición 2, si existe un segmento previo se calcula el ángulo del segmento;
    de lo contrario se acepta el primer salto. Los puntos empleados se remueven del background
    para que los caminos se formen de forma paralela y no se unan.
    
    Retorna una lista de DataFrames, cada uno representando un camino.
    """
    # Ordenar background por θ descendente.
    bg = background.copy().sort_values("theta", ascending=False).reset_index(drop=True)
    
    # Función para convertir un punto a coordenadas cartesianas.
    def to_cartesian(row):
        rad = np.radians(row['theta'])
        return np.array([row['r'] * np.cos(rad), row['r'] * np.sin(rad)])
    
    # Obtener la unión de las fronteras de los grupos finales.
    boundary_list = []
    for grupo in grupos_finales:
        pts_grp = grupo['points']
        for _, row in pts_grp.iterrows():
            boundary_list.append(to_cartesian(row))
    if boundary_list:
        boundary_all = np.vstack(boundary_list)
    else:
        boundary_all = np.empty((0,2))
    
    # Calcular la distancia promedio entre vecinos en el background (en coordenadas cartesianas)
    pts_bg_cart = np.array([to_cartesian(row) for _, row in bg.iterrows()])
    if len(pts_bg_cart) < 2:
        avg_nn = np.inf
    else:
        dists = cdist(pts_bg_cart, pts_bg_cart)
        np.fill_diagonal(dists, np.inf)
        avg_nn = np.mean(np.min(dists, axis=1))
    threshold = min(tol_factor * avg_nn, 0.65)
    
    arms = []  # Lista de caminos resultantes.
    
    # Mientras queden puntos en el background:
    while not bg.empty:
        # Semilla: punto con mayor θ.
        seed_idx = bg['theta'].idxmax()
        seed = bg.loc[seed_idx]
        path = [seed]
        bg = bg.drop(seed_idx).reset_index(drop=True)
        current = seed
        prev_direction = None  # No hay segmento anterior para la primera conexión.
        
        while True:
            # Seleccionar candidatos con θ menor que el punto actual.
            candidates = bg[bg['theta'] < current['theta']]
            if candidates.empty:
                break
            # Convertir candidatos a coordenadas cartesianas.
            candidate_coords = np.array([to_cartesian(row) for _, row in candidates.iterrows()])
            current_cart = to_cartesian(current)
            dists_current = np.linalg.norm(candidate_coords - current_cart, axis=1)
            # También, calcular distancia a la frontera de grupos finales.
            if boundary_all.shape[0] > 0:
                dists_boundary = np.array([np.min(np.linalg.norm(candidate - boundary_all, axis=1))
                                           for candidate in candidate_coords])
            else:
                dists_boundary = np.full(len(candidates), np.inf)
            effective_dists = np.minimum(dists_current, dists_boundary)
            
            # Para cada candidato, calcule su dirección desde el actual.
            directions = []
            for coord in candidate_coords:
                vec = coord - current_cart
                angle = np.degrees(np.arctan2(vec[1], vec[0])) % 360
                directions.append(angle)
            directions = np.array(directions)
            
            # Si ya existe un segmento previo, la dirección actual es prev_direction.
            # Se exige que la diferencia angular entre prev_direction y la dirección del candidato sea < max_angle_diff.
            if prev_direction is not None:
                angle_diffs = np.abs((directions - prev_direction + 180) % 360 - 180)
                valid_idx = np.where(angle_diffs < max_angle_diff)[0]
                if len(valid_idx) == 0:
                    break  # No hay candidato que mantenga la continuidad del camino.
                effective_dists = effective_dists[valid_idx]
                candidate_subset = candidates.iloc[valid_idx]
                directions = directions[valid_idx]
            else:
                candidate_subset = candidates
            
            # Seleccionar el candidato con la mínima effective_dist.
            if len(effective_dists) == 0:
                break
            min_eff = np.min(effective_dists)
            idx_min = np.argmin(effective_dists)
            if min_eff > threshold:
                break  # Ningún candidato cumple la condición física.
            
            candidate = candidate_subset.iloc[idx_min]
            # Actualizar el segmento: dirección actual
            current_coord = to_cartesian(current)
            candidate_coord = to_cartesian(candidate)
            new_direction = np.degrees(np.arctan2(candidate_coord[1]-current_coord[1],
                                                   candidate_coord[0]-current_coord[0])) % 360
            
            # Agregar el candidato al camino.
            path.append(candidate)
            prev_direction = new_direction
            current = candidate
            bg = bg.drop(candidate.name).reset_index(drop=True)
        
        arms.append(pd.DataFrame(path))
    
    return arms

# =============================================================================
# 2) Función de graficado final: muestra los caminos ("raíces") junto con background y grupos
# =============================================================================

def plot_arms_with_background(arms, background, grupos_finales, title="Caminos (raíces) en (θ, r) con Background"):
    """
    Grafica los caminos trazados (lista de DataFrames) sobre el background y los puntos de
    los grupos finales. Se emplea α = 0.1 para background y grupos.
    """
    plt.figure(figsize=(12,8))
    
    # Graficar background con baja opacidad.
    plt.scatter(background['theta'], background['r'], c='gray', s=3, alpha=0.1, label="Background")
    
    # Graficar cada grupo final con baja opacidad.
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','lime']
    for i, grupo in enumerate(grupos_finales):
        pts = grupo['points']
        c = colors[i % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, alpha=0.1, label=f"Grupo {i+1}")
    
    # Graficar cada camino con un color diferente.
    for j, arm in enumerate(arms):
        plt.plot(arm['theta'], arm['r'], '-o', label=f"Camino {j+1}", linewidth=2)
    
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.title(title)
    plt.grid(True)
    plt.legend(loc="best")
    plt.show()

# =============================================================================
# MAIN: Ejecución de toda la pipeline
# =============================================================================

if __name__ == "__main__":
    # Paso 1: Procesar y agrupar los datos
    datos_finales = main_example_modificado()
    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']
    
    # Paso 2: Trazar varios caminos ("raíces") a partir del background, recorriendo en θ descendente
    if not background.empty and len(grupos_finales) > 0:
        arms = trace_spiral_arms(background, grupos_finales, tol_factor=1.1, max_angle_diff=10)
        print(f"Se trazaron {len(arms)} caminos (raíces).")
        # Paso 3: Graficar los caminos junto con background y grupos finales
        plot_arms_with_background(arms, background, grupos_finales)
    else:
        print("No hay datos suficientes en el background o en los grupos finales para trazar los caminos.")


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Código completo para:
  1. Procesar y agrupar los datos (etapa de agrupación) a partir de CSV.
  2. Del conjunto background (puntos sin asignar a grupos finales) se trazan
     varios caminos ("raíces") de conexión, recorriendo en orden descendente de θ.
     Se exige que:
       - Cada segmento tenga una distancia menor que: min(tol_factor × promedio, 0.65)
       - La diferencia angular entre segmentos consecutivos sea menor a 10°.
       - El candidato a conexión esté en una región de densidad local ≥ densidad media del background.
  3. Se grafican solo los caminos con las distancias (longitudes) más largas (por encima de la mediana)
     superpuestos al background y grupos finales (con α = 0.1).
     
Puedes ajustar los parámetros (tol_factor, max_angle_diff, density_radius) según tus necesidades.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import cdist

# =============================================================================
# 0) Funciones para la etapa de agrupación (primer paso)
# =============================================================================

def calculate_pa(slope, intercept):
    """
    Calcula el pitch-angle (PA) en grados:
      PA = arctan[(slope * 180/π) / intercept], si intercept ≠ 0.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180/np.pi)) / intercept))
    return np.nan

def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    """
    Carga el CSV, calcula 'r' y 'theta', filtra según el rango de θ y añade una
    columna 'id' para preservar el índice original.
    """
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['id'] = df.index
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

def build_graph_rectangular(df_points, theta_diff, r_diff):
    """
    Construye un grafo de adyacencia (BFS) en el espacio (θ, r) usando una tolerancia.
    """
    th, rr = df_points['theta'].values, df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        mask = (np.abs(th[i] - th) <= theta_diff) & (np.abs(rr[i] - rr) <= r_diff) & (np.arange(n) != i)
        graph[i] = list(np.where(mask)[0])
    return graph, n

def bfs_components(graph, df_points):
    """
    Extrae componentes conexas usando BFS.
    """
    visited, clusters = [False]*len(graph), []
    for s in range(len(graph)):
        if not visited[s]:
            q, comp = deque([s]), [s]
            visited[s] = True
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    """
    Subdivide un clúster en subgrupos si existen "gaps" (saltos en θ o r).
    """
    col = 'theta' if mode=='theta' else 'r'
    df = df_cluster.sort_values(col)
    dif = np.diff(df[col].values)
    if np.all(dif <= gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(dif):
        if d > gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i + 1
    subs.append(df.iloc[start:].copy())
    return subs

def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts/ref, 1.0))

def generate_bfs_seeds(id_halo, theta_min, theta_max,
                         quartile_threshold=0.55,
                         theta_diff=3.0, r_diff=0.5,
                         gap_threshold_theta=2.0, gap_threshold_r=2.0,
                         file_prefix='data_rho'):
    """
    Genera las semillas BFS a partir de los datos filtrados y retorna:
      - final_cl: lista de clústeres resultantes.
      - df_f: DataFrame filtrado (datos base).
    """
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp'] > thr].copy().reset_index(drop=True)
    
    f = _adaptive_factor(len(df_f))
    th_diff, r_d = theta_diff/f, r_diff/f
    gap_th, gap_r = gap_threshold_theta/f, gap_threshold_r/f
    
    graph, _ = build_graph_rectangular(df_f, th_diff, r_d)
    clusters = bfs_components(graph, df_f)
    
    final_cl = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_th, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r, 'r'))
    return final_cl, df_f

def fit_line_to_cluster(df_cluster):
    """
    Ajusta una línea (regresión lineal) a los puntos del clúster.
    """
    if len(df_cluster) < 2:
        return (None,)*7
    mdl = LinearRegression().fit(df_cluster[['theta']], df_cluster['r'])
    s, b = mdl.coef_[0], mdl.intercept_
    pa = calculate_pa(s, b)
    return s, b, pa, df_cluster['theta'].min(), df_cluster['theta'].max(), df_cluster['r'].min(), df_cluster['r'].max()

def adjust_and_merge_seeds(bfs_clusters, slope_variation_threshold=0.40, bounding_extrap=0.30):
    """
    Fusiona los clústeres que se solapan.
    """
    groups = []
    for cl in bfs_clusters:
        groups.append(dict(zip(
            ['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
            (*fit_line_to_cluster(cl), cl)
        )))
    
    def recalc(g):
        g['slope'], g['intercept'], g['pa'], g['theta_min'], g['theta_max'], g['r_min'], g['r_max'] = \
            fit_line_to_cluster(g['points'])
    
    def boxes_overlap(g1, g2, e):
        def exp(t0, t1, r0, r1):
            dt, dr = (t1-t0)*e, (r1-r0)*e
            return t0-dt, t1+dt, r0-dr, r1+dr
        a = exp(g1['theta_min'], g1['theta_max'], g1['r_min'], g1['r_max'])
        b = exp(g2['theta_min'], g2['theta_max'], g2['r_min'], g2['r_max'])
        return not (a[1] < b[0] or b[1] < a[0]) and not (a[3] < b[2] or b[3] < a[2])
    
    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i+1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, b_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb - g1['slope'])/(abs(g1['slope'])+1e-12)
                        v2 = abs(s_comb - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb
                            recalc(g1)
                            groups.pop(j)
                            merged = True
                            continue
                j += 1
            new.append(g1)
            i += 1
        groups = new
    return [g for g in groups if len(g['points']) >= 2]

def validate_dispersion_and_reprocess(groups, dispersion_threshold=1.80, reproc_theta_diff=1.25, reproc_r_diff=0.4):
    """
    Valida la dispersión de cada grupo y re-procesa aquellos que superen el umbral.
    """
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points']) < 2:
            ok.append(g)
            continue
        res = g['points']['r'] - (g['slope'] * g['points']['theta'] + g['intercept'])
        (todo if res.std() > dispersion_threshold else ok).append(g)
    
    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            new.append(dict(zip(
                ['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
                (*fit_line_to_cluster(c), c)
            )))
    return ok + new

def plot_groups_cartesian(groups, df_all, line_extrap=0.15):
    """
    Visualiza en un gráfico cartesiano los grupos finales (con ≥ 60 puntos) en (θ, r).
    """
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, alpha=0.3, label=f"Grupo {i+1}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            plt.plot(t, g['slope']*t+g['intercept'], '--', color=c)
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.grid(True)
    plt.title("Subhalo 17 – Grupos (≥ 60 pts) - Cartesiano")
    plt.legend()
    plt.show()

def plot_groups_polar(groups, df_all, line_extrap=0.15):
    """
    Visualización en proyección polar de los grupos finales (con ≥ 60 puntos).
    """
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos (polar): {len(big)}")
    fig = plt.figure(figsize=(9,8))
    ax = plt.subplot(111, projection='polar')
    ax.scatter(np.radians(df_all['theta']), df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        ax.scatter(np.radians(pts['theta']), pts['r'], s=10, color=c, alpha=0.3,
                   label=f"Grupo {i+1}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t_deg = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            r_line = g['slope']*t_deg+g['intercept']
            ax.plot(np.radians(t_deg), r_line, '--', color=c)
    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title("Subhalo 17 – Grupos (≥ 60 pts) - Polar", y=1.05)
    ax.legend(loc='upper right', bbox_to_anchor=(1.20,1.0))
    plt.show()

def main_example_modificado():
    """
    Ejecuta el pipeline completo para subhalo 17.
    Retorna un diccionario con:
      - 'grupos_finales': lista de grupos (con ≥ 60 pts).
      - 'background': DataFrame con los puntos que no están en ningún grupo final.
    """
    bfs, df_f = generate_bfs_seeds(
        id_halo="17",
        theta_min=0, theta_max=360,
        quartile_threshold=0.155,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=2.0, gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")
    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.30, bounding_extrap=0.35)
    print(f"Tras fusión: {len(merged)} grupos")
    validated = validate_dispersion_and_reprocess(merged)
    print(f"Tras validación: {len(validated)} grupos")
    plot_groups_cartesian(validated, df_f, line_extrap=0.15)
    plot_groups_polar(validated, df_f, line_extrap=0.15)
    
    # Almacenar grupos finales: aquellos con 60 o más puntos.
    grupos_finales = [g for g in validated if len(g['points']) >= 60]
    
    # Puntos de grupos.
    puntos_grupos = set()
    for g in grupos_finales:
        puntos_grupos.update(g['points']['id'].tolist())
    
    # Background: puntos que no están en ningún grupo final.
    background = df_f[~df_f['id'].isin(puntos_grupos)].copy()
    print(f"Puntos background: {len(background)}")
    
    return {'grupos_finales': grupos_finales, 'background': background}

# =============================================================================
# 1) Funciones para trazado de caminos ("raíces") conectados físicamente
# =============================================================================

def local_density(point, points, radius=0.5):
    """
    Calcula la cantidad de puntos en 'points' que se encuentran a menos de 'radius'
    distancia de 'point'. Se usa como criterio local de densidad.
    """
    d = np.linalg.norm(points - point, axis=1)
    return np.sum(d < radius)

def trace_spiral_arms(background, grupos_finales, tol_factor=1.1, max_angle_diff=10, density_radius=0.5):
    """
    Genera varios caminos ("raíces") a partir del background (en coordenadas polares),
    recorriendo en orden descendente de θ. Para cada candidato se exige:
      1. Distancia en coordenadas cartesianas < min(tol_factor*(promedio), 0.65)
      2. Diferencia angular entre el segmento actual y la dirección hacia el candidato < max_angle_diff
      3. El candidato debe encontrarse en una región de densidad local (contando vecinos en un radio dado)
         mayor o igual que la densidad media del background.
    Los puntos usados se remueven del background para que los caminos sean independientes.
    
    Retorna una lista de DataFrames, cada uno representando un camino.
    """
    # Ordenar background por θ descendente.
    bg = background.copy().sort_values("theta", ascending=False).reset_index(drop=True)
    
    def to_cartesian(row):
        rad = np.radians(row['theta'])
        return np.array([row['r']*np.cos(rad), row['r']*np.sin(rad)])
    
    # Calcular coordenadas del background.
    pts_bg_cart = np.array([to_cartesian(row) for _, row in bg.iterrows()])
    
    # Calcular densidad local para cada punto del background.
    densities = np.array([local_density(pt, pts_bg_cart, radius=density_radius) for pt in pts_bg_cart])
    global_mean_density = np.mean(densities) if len(densities) > 0 else 0

    # Obtener la unión de las fronteras de los grupos finales.
    boundary_list = []
    for grupo in grupos_finales:
        pts_grp = grupo['points']
        for _, row in pts_grp.iterrows():
            boundary_list.append(to_cartesian(row))
    if boundary_list:
        boundary_all = np.vstack(boundary_list)
    else:
        boundary_all = np.empty((0,2))
    
    # Calcular promedio de distancia entre puntos (en coordenadas cartesianas)
    if len(pts_bg_cart) < 2:
        avg_nn = np.inf
    else:
        dists = cdist(pts_bg_cart, pts_bg_cart)
        np.fill_diagonal(dists, np.inf)
        avg_nn = np.mean(np.min(dists, axis=1))
    threshold = min(tol_factor * avg_nn, 0.65)
    
    arms = []
    
    while not bg.empty:
        # Semilla: punto con mayor θ.
        seed_idx = bg['theta'].idxmax()
        seed = bg.loc[seed_idx]
        path = [seed]
        bg = bg.drop(seed_idx).reset_index(drop=True)
        current = seed
        prev_direction = None
        
        while True:
            candidates = bg[bg['theta'] < current['theta']]
            if candidates.empty:
                break
            candidate_coords = np.array([to_cartesian(row) for _, row in candidates.iterrows()])
            current_cart = to_cartesian(current)
            dists_current = np.linalg.norm(candidate_coords - current_cart, axis=1)
            if boundary_all.shape[0] > 0:
                dists_boundary = np.array([np.min(np.linalg.norm(candidate - boundary_all, axis=1))
                                           for candidate in candidate_coords])
            else:
                dists_boundary = np.full(len(candidates), np.inf)
            effective_dists = np.minimum(dists_current, dists_boundary)
            
            # Calcular dirección hacia cada candidato.
            directions = []
            for coord in candidate_coords:
                vec = coord - current_cart
                angle = np.degrees(np.arctan2(vec[1], vec[0])) % 360
                directions.append(angle)
            directions = np.array(directions)
            
            # Filtrar candidatos que mantengan continuidad direccional.
            if prev_direction is not None:
                angle_diffs = np.abs((directions - prev_direction + 180) % 360 - 180)
                valid_idx = np.where(angle_diffs < max_angle_diff)[0]
                if len(valid_idx) == 0:
                    break
                effective_dists = effective_dists[valid_idx]
                candidate_subset = candidates.iloc[valid_idx]
                directions = directions[valid_idx]
            else:
                candidate_subset = candidates
            
            if len(effective_dists) == 0:
                break
            min_eff = np.min(effective_dists)
            idx_min = np.argmin(effective_dists)
            # Criterio de distancia.
            if min_eff > threshold:
                break
            
            # Criterio de densidad:
            candidate = candidate_subset.iloc[idx_min]
            candidate_cart = to_cartesian(candidate)
            cand_density = local_density(candidate_cart, pts_bg_cart, radius=density_radius)
            if cand_density < global_mean_density:
                # Si la densidad local es menor que la media, se descarta este candidato.
                # Se remueve el candidato y se continúa.
                bg = bg.drop(candidate.name).reset_index(drop=True)
                continue
            
            current_coord = to_cartesian(current)
            candidate_coord = to_cartesian(candidate)
            new_direction = np.degrees(np.arctan2(candidate_coord[1]-current_coord[1],
                                                   candidate_coord[0]-current_coord[0])) % 360
            
            path.append(candidate)
            prev_direction = new_direction
            current = candidate
            bg = bg.drop(candidate.name).reset_index(drop=True)
        
        arms.append(pd.DataFrame(path))
    
    # Calcular longitud de cada camino.
    def path_length(df_path):
        pts = np.array([to_cartesian(row) for _, row in df_path.iterrows()])
        if len(pts) < 2:
            return 0
        return np.sum(np.linalg.norm(pts[1:] - pts[:-1], axis=1))
    
    lengths = np.array([path_length(arm) for arm in arms])
    if len(lengths) == 0:
        return arms
    median_length = np.median(lengths)
    # Se filtran y retornan únicamente los caminos cuya longitud es mayor o igual a la mediana.
    filtered_arms = [arm for arm, L in zip(arms, lengths) if L >= median_length]
    return filtered_arms

# =============================================================================
# 2) Función de graficado final: mostrar caminos con background y grupos finales
# =============================================================================

def plot_arms_with_background(arms, background, grupos_finales, title="Caminos (raíces) con Background"):
    plt.figure(figsize=(12,8))
    plt.scatter(background['theta'], background['r'], c='gray', s=3, alpha=0.1, label="Background")
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','lime']
    for i, grupo in enumerate(grupos_finales):
        pts = grupo['points']
        c = colors[i % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, alpha=0.1, label=f"Grupo {i+1}")
    for j, arm in enumerate(arms):
        plt.plot(arm['theta'], arm['r'], '-o', label=f"Camino {j+1}", linewidth=2)
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.title(title)
    plt.grid(True)
    plt.legend(loc="best")
    plt.show()

# =============================================================================
# MAIN: Ejecución de la pipeline completa
# =============================================================================

if __name__ == "__main__":
    # Paso 1: Procesar y agrupar los datos.
    datos_finales = main_example_modificado()
    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']
    
    # Paso 2: Trazar caminos ("raíces") con criterio de densidad y dirección.
    if not background.empty and len(grupos_finales) > 0:
        arms = trace_spiral_arms(background, grupos_finales, tol_factor=1.1, max_angle_diff=10, density_radius=0.5)
        print(f"Se trazaron {len(arms)} caminos (raíces) que cumplen los criterios.")
        # Paso 3: Graficar los caminos (solo los de mayor longitud) junto con background y grupos finales.
        plot_arms_with_background(arms, background, grupos_finales, title="Caminos largos (raíces) con Background")
    else:
        print("No hay datos suficientes en el background o en los grupos finales para trazar caminos.")


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Ejemplo de conexión de grupos mediante una malla de densidad SPH.
Se crea una grilla sobre el dominio, se interpola la densidad usando un kernel SPH,
se extraen isocontornos (los pozos densos) y se “conectan” sus fronteras
buscando en la malla siempre el vecino más cercano.
Donde la densidad cae a cero se interpreta como un corte natural (aislamiento).
Se grafican los caminos más largos encontrados.
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.ndimage import gaussian_filter

# --- Definir kernel SPH (cúbico spline en 2D) ---
def cubic_spline_kernel(r, h):
    q = r/h
    sigma = 10.0/(7.0*np.pi*h**2)  # normalización en 2D
    if q >= 0 and q <= 1:
        return sigma * (1 - 1.5*q**2 + 0.75*q**3)
    elif q > 1 and q <= 2:
        return sigma * (0.25 * (2 - q)**3)
    else:
        return 0.0

# --- Interpolar la densidad en una malla ---
def compute_density_grid(points_cart, grid_x, grid_y, h=0.5):
    """
    points_cart: array (N,2) con coordenadas cartesianas de todos los puntos.
    grid_x, grid_y: matrices resultantes de np.meshgrid.
    h: longitud de suavizado.
    Retorna la densidad evaluada en cada nodo de la malla.
    """
    density = np.zeros(grid_x.shape)
    for pt in points_cart:
        # Calcular la distancia desde cada nodo al punto pt
        d = np.sqrt((grid_x - pt[0])**2 + (grid_y - pt[1])**2)
        # Sumar la contribución del kernel
        density += np.vectorize(cubic_spline_kernel)(d, h)
    return density

# --- Extracción y conexión de fronteras ---
def extract_and_connect_paths(density, grid_x, grid_y, density_threshold=0.1):
    """
    Dado un campo de densidad evaluado en la malla (grid_x, grid_y),
    extrae los contornos donde la densidad es igual a density_threshold
    y simula la conexión entre los contornos (usando el camino de distancia mínima).
    
    Esta función es ilustrativa; en una implementación real se podría usar
    watershed o algoritmos de grafos sobre la malla.
    
    Retorna una lista de caminos, cada uno como array de coordenadas (x,y).
    """
    # Utilizamos matplotlib para extraer contornos:
    cs = plt.contour(grid_x, grid_y, density, levels=[density_threshold])
    paths = []
    # Extraer coordenadas de cada contorno
    for collection in cs.collections:
        for path in collection.get_paths():
            v = path.vertices
            paths.append(v)
    plt.clf()  # limpiar la figura de contornos
    # Aquí se puede aplicar un criterio para conectar contornos cercanos
    # Por ejemplo, para cada contorno, buscar el contorno vecino cuyo punto
    # esté a menos de una distancia crítica (p.ej. 0.65) y unirlos.
    # En este ejemplo se devuelven directamente los contornos.
    return paths

# --- Ejemplo de uso: Crear malla, calcular densidad y extraer caminos ---
def main_sph_path():
    # Simulamos algunos puntos (por ejemplo, del background y fronteras de grupos)
    # Supongamos que points_cart es un array (N,2) con coordenadas cartesianas.
    # En un caso real usarías la información combinada de los grupos y background.
    np.random.seed(0)
    # Crear puntos distribuidos en dos pozos densos:
    n1, n2 = 200, 150
    points1 = np.random.normal(loc=[2,2], scale=0.2, size=(n1,2))
    points2 = np.random.normal(loc=[4,4], scale=0.3, size=(n2,2))
    points_cart = np.vstack([points1, points2])
    
    # Definir la malla que cubra el dominio
    xmin, xmax = points_cart[:,0].min()-1, points_cart[:,0].max()+1
    ymin, ymax = points_cart[:,1].min()-1, points_cart[:,1].max()+1
    n_grid = 200
    x = np.linspace(xmin, xmax, n_grid)
    y = np.linspace(ymin, ymax, n_grid)
    grid_x, grid_y = np.meshgrid(x, y)
    
    # Calcular la densidad en la malla usando el kernel SPH.
    h = 0.5
    density = compute_density_grid(points_cart, grid_x, grid_y, h=h)
    
    # Opcional: Suavizar la densidad (para obtener contornos más continuos)
    density_smooth = gaussian_filter(density, sigma=1)
    
    # Extraer contornos (fronteras densas) donde la densidad es mayor a un umbral.
    density_threshold = np.percentile(density_smooth, 90)  # por ejemplo, el 90% superior
    paths = extract_and_connect_paths(density_smooth, grid_x, grid_y, density_threshold=density_threshold)
    
    # Para este ejemplo, consideramos como caminos los contornos extraídos y
    # seleccionamos los de mayor longitud.
    path_lengths = [np.sum(np.linalg.norm(path[1:]-path[:-1], axis=1)) for path in paths]
    if len(path_lengths)==0:
        print("No se han encontrado caminos.")
        return
    median_length = np.median(path_lengths)
    selected_paths = [path for path, L in zip(paths, path_lengths) if L >= median_length]
    
    # Graficar: mostrar la malla de densidad y los caminos
    plt.figure(figsize=(10,8))
    plt.imshow(density_smooth, extent=[xmin, xmax, ymin, ymax], origin='lower', cmap='inferno')
    plt.colorbar(label='Densidad')
    for path in selected_paths:
        plt.plot(path[:,0], path[:,1], '-o', linewidth=2)
    plt.scatter(points_cart[:,0], points_cart[:,1], c='cyan', s=10, label='Puntos')
    plt.title("Malla de densidad SPH y caminos densos conectados")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.show()
    
    return selected_paths

if __name__ == "__main__":
    paths = main_sph_path()


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Pipeline completo usando datos reales para:
  1. Procesar y agrupar los datos (a partir de CSV, por ejemplo, data_rho_17_filtered.csv).
  2. Del conjunto background (puntos no asignados a ningún grupo final) se crea una malla
     y se interpola la densidad mediante un kernel SPH. Se extraen isocontornos
     (los pozos densos) y se usan para rastrear caminos que conecten las fronteras de los grupos.
  3. Se grafican únicamente los caminos (los de mayor longitud) superpuestos al mapa de densidad,
     con los puntos originales y los grupos finales mostrados con baja opacidad (α = 0.1).
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import cdist
from scipy.ndimage import gaussian_filter

# =============================================================================
# 0) Funciones del pipeline de agrupación (usa tus datos reales)
# =============================================================================

def calculate_pa(slope, intercept):
    if intercept != 0:
        return np.degrees(np.arctan((slope*(180/np.pi))/intercept))
    return np.nan

def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['id'] = df.index
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

def build_graph_rectangular(df_points, theta_diff, r_diff):
    th, rr = df_points['theta'].values, df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        mask = (np.abs(th[i]-th)<=theta_diff) & (np.abs(rr[i]-rr)<=r_diff) & (np.arange(n)!=i)
        graph[i] = list(np.where(mask)[0])
    return graph, n

def bfs_components(graph, df_points):
    visited, clusters = [False]*len(graph), []
    for s in range(len(graph)):
        if not visited[s]:
            q, comp = deque([s]), [s]
            visited[s] = True
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    col = 'theta' if mode=='theta' else 'r'
    df = df_cluster.sort_values(col)
    dif = np.diff(df[col].values)
    if np.all(dif<=gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(dif):
        if d>gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i+1
    subs.append(df.iloc[start:].copy())
    return subs

def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts/ref,1.0))

def generate_bfs_seeds(id_halo, theta_min, theta_max,
                         quartile_threshold=0.55,
                         theta_diff=3.0, r_diff=0.5,
                         gap_threshold_theta=2.0, gap_threshold_r=2.0,
                         file_prefix='data_rho'):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp']>thr].copy().reset_index(drop=True)
    
    f = _adaptive_factor(len(df_f))
    th_diff, r_d = theta_diff/f, r_diff/f
    gap_th, gap_r = gap_threshold_theta/f, gap_threshold_r/f
    
    graph, _ = build_graph_rectangular(df_f, th_diff, r_d)
    clusters = bfs_components(graph, df_f)
    
    final_cl = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_th, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r, 'r'))
    return final_cl, df_f

def fit_line_to_cluster(df_cluster):
    if len(df_cluster)<2:
        return (None,)*7
    mdl = LinearRegression().fit(df_cluster[['theta']], df_cluster['r'])
    s, b = mdl.coef_[0], mdl.intercept_
    pa = calculate_pa(s,b)
    return s, b, pa, df_cluster['theta'].min(), df_cluster['theta'].max(), df_cluster['r'].min(), df_cluster['r'].max()

def adjust_and_merge_seeds(bfs_clusters, slope_variation_threshold=0.40, bounding_extrap=0.30):
    groups = []
    for cl in bfs_clusters:
        groups.append(dict(zip(['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
                                 (*fit_line_to_cluster(cl), cl))))
    def recalc(g):
        g['slope'],g['intercept'],g['pa'],g['theta_min'],g['theta_max'],g['r_min'],g['r_max'] = fit_line_to_cluster(g['points'])
    def boxes_overlap(g1, g2, e):
        def exp(t0,t1,r0,r1):
            dt, dr = (t1-t0)*e, (r1-r0)*e
            return t0-dt, t1+dt, r0-dr, r1+dr
        a = exp(g1['theta_min'],g1['theta_max'],g1['r_min'],g1['r_max'])
        b = exp(g2['theta_min'],g2['theta_max'],g2['r_min'],g2['r_max'])
        return not (a[1]<b[0] or b[1]<a[0]) and not (a[3]<b[2] or b[3]<a[2])
    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i+1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, b_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb-g1['slope'])/(abs(g1['slope'])+1e-12)
                        v2 = abs(s_comb-g2['slope'])/(abs(g2['slope'])+1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb
                            recalc(g1)
                            groups.pop(j)
                            merged = True
                            continue
                j += 1
            new.append(g1)
            i += 1
        groups = new
    return [g for g in groups if len(g['points']) >= 2]

def validate_dispersion_and_reprocess(groups, dispersion_threshold=1.80, reproc_theta_diff=1.25, reproc_r_diff=0.4):
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points']) < 2:
            ok.append(g)
            continue
        res = g['points']['r'] - (g['slope']*g['points']['theta']+g['intercept'])
        (todo if res.std()>dispersion_threshold else ok).append(g)
    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            new.append(dict(zip(['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
                                  (*fit_line_to_cluster(c), c))))
    return ok + new

def plot_groups_cartesian(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, alpha=0.3, label=f"Grupo {i+1}")
        if g['slope'] is not None:
            dt = g['theta_max']-g['theta_min']
            t = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            plt.plot(t, g['slope']*t+g['intercept'], '--', color=c)
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.grid(True)
    plt.title("Subhalo 17 – Grupos (≥ 60 pts) - Cartesiano")
    plt.legend()
    plt.show()

def plot_groups_polar(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos (polar): {len(big)}")
    fig = plt.figure(figsize=(9,8))
    ax = plt.subplot(111, projection='polar')
    ax.scatter(np.radians(df_all['theta']), df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        ax.scatter(np.radians(pts['theta']), pts['r'], s=10, color=c, alpha=0.3, label=f"Grupo {i+1}")
        if g['slope'] is not None:
            dt = g['theta_max']-g['theta_min']
            t_deg = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            r_line = g['slope']*t_deg+g['intercept']
            ax.plot(np.radians(t_deg), r_line, '--', color=c)
    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title("Subhalo 17 – Grupos (≥ 60 pts) - Polar", y=1.05)
    ax.legend(loc='upper right', bbox_to_anchor=(1.20,1.0))
    plt.show()

def main_example_modificado():
    bfs, df_f = generate_bfs_seeds(
        id_halo="17",
        theta_min=0, theta_max=360,
        quartile_threshold=0.155,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=2.0, gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")
    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.30, bounding_extrap=0.35)
    print(f"Tras fusión: {len(merged)} grupos")
    validated = validate_dispersion_and_reprocess(merged)
    print(f"Tras validación: {len(validated)} grupos")
    plot_groups_cartesian(validated, df_f, line_extrap=0.15)
    plot_groups_polar(validated, df_f, line_extrap=0.15)
    
    grupos_finales = [g for g in validated if len(g['points']) >= 60]
    
    puntos_grupos = set()
    for g in grupos_finales:
        puntos_grupos.update(g['points']['id'].tolist())
    
    background = df_f[~df_f['id'].isin(puntos_grupos)].copy()
    print(f"Puntos background: {len(background)}")
    
    return {'grupos_finales': grupos_finales, 'background': background}

# =============================================================================
# 2) Funciones para trazado de caminos ("raíces") conectados basados en densidad SPH
# =============================================================================

def cubic_spline_kernel(r, h):
    q = r/h
    sigma = 10.0/(7.0*np.pi*h**2)
    if 0 <= q <= 1:
        return sigma*(1 - 1.5*q**2 + 0.75*q**3)
    elif 1 < q <= 2:
        return sigma*0.25*(2 - q)**3
    else:
        return 0.0

def compute_density_grid(points_cart, grid_x, grid_y, h=0.5):
    density = np.zeros(grid_x.shape)
    for pt in points_cart:
        d = np.sqrt((grid_x - pt[0])**2 + (grid_y - pt[1])**2)
        density += np.vectorize(cubic_spline_kernel)(d, h)
    return density

def trace_spiral_arms(background, grupos_finales, tol_factor=1.1, max_angle_diff=10, h=0.5, density_radius=0.5):
    """
    Genera caminos ("raíces") usando un criterio que combine la conexión física (distancia y ángulo)
    y la condición de que el camino transcurra en zonas de alta densidad (según SPH).
    
    Se calcula la densidad en el background mediante un kernel SPH,
    y se descartan candidatos cuya densidad sea menor que la media global.
    Al final se filtran y solo se retornan los caminos (raíz) de longitud mayor o igual a la mediana.
    """
    bg = background.copy().sort_values("theta", ascending=False).reset_index(drop=True)
    
    def to_cartesian(row):
        rad = np.radians(row['theta'])
        return np.array([row['r']*np.cos(rad), row['r']*np.sin(rad)])
    
    pts_bg_cart = np.array([to_cartesian(row) for _, row in bg.iterrows()])
    densities = np.array([0.0]*len(pts_bg_cart))
    for i in range(len(pts_bg_cart)):
        d = cdist(pts_bg_cart[i].reshape(1,-1), pts_bg_cart)
        # Se suma el kernel para cada punto; masa = 1.
        densities[i] = np.sum(np.vectorize(cubic_spline_kernel)(d, h))
    
    global_mean_density = np.mean(densities)
    bg = bg.assign(density=densities)
    
    if len(pts_bg_cart) < 2:
        avg_nn = np.inf
    else:
        dists = cdist(pts_bg_cart, pts_bg_cart)
        np.fill_diagonal(dists, np.inf)
        avg_nn = np.mean(np.min(dists, axis=1))
    connection_threshold = min(tol_factor * avg_nn, 0.65)
    
    # Obtener la unión de las fronteras de los grupos finales.
    boundary_list = []
    for grupo in grupos_finales:
        pts_grp = grupo['points']
        for _, row in pts_grp.iterrows():
            boundary_list.append(to_cartesian(row))
    if boundary_list:
        boundary_all = np.vstack(boundary_list)
    else:
        boundary_all = np.empty((0,2))
    
    arms = []
    
    while not bg.empty:
        seed_idx = bg['theta'].idxmax()
        seed = bg.loc[seed_idx]
        # Usamos solo semillas con densidad ≥ global_mean_density.
        if seed['density'] < global_mean_density:
            bg = bg.drop(seed_idx).reset_index(drop=True)
            continue
        path = [seed]
        bg = bg.drop(seed_idx).reset_index(drop=True)
        current = seed
        prev_direction = None
        
        while True:
            candidates = bg[bg['theta'] < current['theta']]
            if candidates.empty:
                break
            candidate_coords = np.array([to_cartesian(row) for _, row in candidates.iterrows()])
            current_cart = to_cartesian(current)
            dists_current = np.linalg.norm(candidate_coords - current_cart, axis=1)
            if boundary_all.shape[0] > 0:
                dists_boundary = np.array([np.min(np.linalg.norm(candidate - boundary_all, axis=1))
                                           for candidate in candidate_coords])
            else:
                dists_boundary = np.full(len(candidates), np.inf)
            effective_dists = np.minimum(dists_current, dists_boundary)
            directions = []
            for coord in candidate_coords:
                vec = coord - current_cart
                angle = np.degrees(np.arctan2(vec[1], vec[0])) % 360
                directions.append(angle)
            directions = np.array(directions)
            if prev_direction is not None:
                angle_diffs = np.abs((directions - prev_direction + 180) % 360 - 180)
                valid_idx = np.where(angle_diffs < max_angle_diff)[0]
                if len(valid_idx) == 0:
                    break
                effective_dists = effective_dists[valid_idx]
                candidate_subset = candidates.iloc[valid_idx]
                directions = directions[valid_idx]
            else:
                candidate_subset = candidates
            
            if len(effective_dists)==0:
                break
            min_eff = np.min(effective_dists)
            idx_min = np.argmin(effective_dists)
            if min_eff > connection_threshold:
                break
            candidate = candidate_subset.iloc[idx_min]
            # Rechazar candidatos con densidad menor a la media.
            if candidate['density'] < global_mean_density:
                bg = bg.drop(candidate.name).reset_index(drop=True)
                continue
            current_coord = to_cartesian(current)
            candidate_coord = to_cartesian(candidate)
            new_direction = np.degrees(np.arctan2(candidate_coord[1]-current_coord[1],
                                                   candidate_coord[0]-current_coord[0])) % 360
            path.append(candidate)
            prev_direction = new_direction
            current = candidate
            bg = bg.drop(candidate.name).reset_index(drop=True)
        
        arms.append(pd.DataFrame(path))
    
    # Filtrar para quedarse solo con los caminos largos.
    def path_length(df_path):
        pts = np.array([to_cartesian(row) for _, row in df_path.iterrows()])
        if len(pts) < 2:
            return 0
        return np.sum(np.linalg.norm(pts[1:]-pts[:-1], axis=1))
    
    lengths = np.array([path_length(arm) for arm in arms])
    if len(lengths)==0:
        return arms
    median_length = np.median(lengths)
    filtered_arms = [arm for arm, L in zip(arms, lengths) if L >= median_length]
    return filtered_arms

# =============================================================================
# Función de graficado final
# =============================================================================

def plot_arms_with_background(arms, background, grupos_finales, title="Caminos densos conectados"):
    plt.figure(figsize=(12,8))
    plt.scatter(background['theta'], background['r'], c='gray', s=3, alpha=0.1, label="Background")
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','lime']
    for i, grupo in enumerate(grupos_finales):
        pts = grupo['points']
        c = colors[i % len(colors)]
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, alpha=0.1, label=f"Grupo {i+1}")
    for j, arm in enumerate(arms):
        plt.plot(arm['theta'], arm['r'], '-o', label=f"Camino {j+1}", linewidth=2)
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.title(title)
    plt.grid(True)
    plt.legend(loc="best")
    plt.show()

# =============================================================================
# MAIN: Ejecución de la pipeline completa usando datos reales.
# =============================================================================

if __name__ == "__main__":
    # Primero se ejecuta el pipeline de agrupación real.
    datos_finales = main_example_modificado()
    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']
    
    if not background.empty and len(grupos_finales) > 0:
        arms = trace_spiral_arms(background, grupos_finales, tol_factor=1.1, max_angle_diff=10, h=0.5, density_radius=0.5)
        print(f"Se trazaron {len(arms)} caminos densos conectados.")
        plot_arms_with_background(arms, background, grupos_finales, title="Caminos densos conectados con datos reales")
    else:
        print("No hay datos suficientes en el background o en los grupos finales para trazar caminos.")


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Ejemplo de conexión de grupos mediante una malla de densidad SPH, usando datos reales.
Se crea una grilla sobre el dominio de los datos (background y grupos finales), se interpola
la densidad usando un kernel SPH, se extraen isocontornos (los pozos densos) y se muestran
los caminos (contornos) de mayor longitud.
Se usa como fuente de datos:
    datos_finales = main_example_modificado()
    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.ndimage import gaussian_filter
import pandas as pd

# --- Función SPH: Kernel cúbico spline en 2D ---
def cubic_spline_kernel(r, h):
    q = r/h
    sigma = 10.0/(7.0*np.pi*h**2)  # normalización en 2D
    if q >= 0 and q <= 1:
        return sigma * (1 - 1.5*q**2 + 0.75*q**3)
    elif q > 1 and q <= 2:
        return sigma * (0.25 * (2 - q)**3)
    else:
        return 0.0

# --- Función para interpolar la densidad en una malla ---
def compute_density_grid(points_cart, grid_x, grid_y, h=0.5):
    """
    points_cart: array (N,2) con coordenadas cartesianas.
    grid_x, grid_y: matrices resultantes de np.meshgrid.
    h: radio de suavizado.
    Retorna la densidad evaluada en cada nodo de la malla.
    """
    density = np.zeros(grid_x.shape)
    # Vectorizamos el kernel para mayor eficiencia:
    kernel_func = np.vectorize(cubic_spline_kernel)
    for pt in points_cart:
        d = np.sqrt((grid_x - pt[0])**2 + (grid_y - pt[1])**2)
        density += kernel_func(d, h)
    return density

# --- Función para extraer isocontornos (caminos densos) ---
def extract_and_connect_paths(density, grid_x, grid_y, density_threshold):
    """
    Dado un campo de densidad sobre la malla (grid_x, grid_y), extrae los isocontornos
    donde la densidad es igual a density_threshold.
    
    Retorna una lista de caminos, cada uno como un array de coordenadas (x, y).
    """
    cs = plt.contour(grid_x, grid_y, density, levels=[density_threshold])
    paths = []
    for collection in cs.collections:
        for path in collection.get_paths():
            v = path.vertices
            paths.append(v)
    plt.clf()  # Limpiar la figura
    return paths

# --- Función principal que usa datos reales (del pipeline) ---
def main_sph_path_reales():
    # Cargar datos reales: se usa el pipeline anterior.
    # Asegúrate de que la función main_example_modificado() ya se encuentre definida.
    datos_finales = main_example_modificado()
    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']
    
    # Para definir la malla, usamos la unión de background y fronteras de los grupos.
    # Extraer puntos del background:
    bg_points = background[['x','y']] if 'x' in background.columns and 'y' in background.columns else None
    if bg_points is None:
        # Si no existen "x" e "y", convertir a cartesianas a partir de "theta" y "r":
        def pol2cart(df):
            rad = np.radians(df['theta'])
            df = df.copy()
            df['x'] = df['r'] * np.cos(rad)
            df['y'] = df['r'] * np.sin(rad)
            return df[['x','y']]
        bg_points = pol2cart(background)
    
    # Extraer puntos de los grupos finales (se supone que estos DataFrames ya tienen x,y o se convierten)
    group_points = []
    for grupo in grupos_finales:
        pts = grupo['points']
        if not {'x','y'}.issubset(set(pts.columns)):
            # Convertir a cartesianas
            pts = pts.copy()
            rad = np.radians(pts['theta'])
            pts['x'] = pts['r']*np.cos(rad)
            pts['y'] = pts['r']*np.sin(rad)
        group_points.append(pts[['x','y']].values)
    if group_points:
        group_points = np.vstack(group_points)
    else:
        group_points = np.empty((0,2))
    
    # Combinar puntos: background y fronteras de grupos.
    all_points = np.vstack([bg_points.values, group_points])
    
    # Definir la malla:
    xmin, xmax = all_points[:,0].min()-1, all_points[:,0].max()+1
    ymin, ymax = all_points[:,1].min()-1, all_points[:,1].max()+1
    n_grid = 300  # resolución de la grilla
    x = np.linspace(xmin, xmax, n_grid)
    y = np.linspace(ymin, ymax, n_grid)
    grid_x, grid_y = np.meshgrid(x, y)
    
    # Calcular la densidad en la malla con SPH:
    h = 0.5  # radio de suavizado
    density = compute_density_grid(all_points, grid_x, grid_y, h=h)
    density_smooth = gaussian_filter(density, sigma=1)
    
    # Elegir un umbral de densidad: por ejemplo, el percentil 90 de la densidad suavizada.
    density_threshold = np.percentile(density_smooth, 90)
    
    # Extraer contornos (caminos) donde la densidad = density_threshold.
    paths = extract_and_connect_paths(density_smooth, grid_x, grid_y, density_threshold)
    
    # Seleccionar los caminos más largos (por ejemplo, aquellos cuya longitud sea ≥ mediana de longitudes)
    def path_length(path):
        return np.sum(np.linalg.norm(path[1:]-path[:-1], axis=1))
    lengths = np.array([path_length(p) for p in paths])
    if len(lengths)==0:
        print("No se encontraron caminos.")
        return
    median_length = np.median(lengths)
    selected_paths = [p for p, L in zip(paths, lengths) if L >= median_length]
    
    # Graficar la malla de densidad y los caminos
    plt.figure(figsize=(12,10))
    plt.imshow(density_smooth, extent=[xmin, xmax, ymin, ymax], origin='lower', cmap='inferno')
    plt.colorbar(label='Densidad')
    for p in selected_paths:
        plt.plot(p[:,0], p[:,1], '-o', linewidth=2)
    plt.scatter(all_points[:,0], all_points[:,1], c='cyan', s=0.2, label='Puntos', alpha=0.3)
    plt.title("Malla de densidad SPH y caminos densos conectados (datos reales)")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.show()
    
    return selected_paths

if __name__ == "__main__":
    paths = main_sph_path_reales()


### ultimo intento

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Código completo para:
  1. Procesar y agrupar los datos reales (desde CSV) mediante el pipeline de agrupación.
  2. Crear una malla en el espacio (θ, r) y calcular la densidad SPH con pesos diferenciados:
     se asigna mayor masa a los puntos que pertenecen a los grupos finales (pozos densos) y masa 1
     al background.
  3. Extraer isocontornos (caminos) en la malla en función de la densidad, filtrando sólo aquellos
     caminos de mayor longitud (por encima de la mediana).
  4. Graficar los caminos sobre la malla en el espacio (θ, r) sin barra de color.
  
Asegúrate de que el archivo de datos (por ejemplo, data_rho_17_filtered.csv) y todo el entorno
se encuentren en el mismo directorio.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import cdist
from scipy.ndimage import gaussian_filter

# =============================================================================
# 1) Pipeline de Agrupación (ya existente)
# =============================================================================

def calculate_pa(slope, intercept):
    if intercept != 0:
        return np.degrees(np.arctan((slope*(180/np.pi))/intercept))
    return np.nan

def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['id'] = df.index
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

def build_graph_rectangular(df_points, theta_diff, r_diff):
    th, rr = df_points['theta'].values, df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        mask = (np.abs(th[i]-th)<=theta_diff) & (np.abs(rr[i]-rr)<=r_diff) & (np.arange(n)!=i)
        graph[i] = list(np.where(mask)[0])
    return graph, n

def bfs_components(graph, df_points):
    visited, clusters = [False]*len(graph), []
    for s in range(len(graph)):
        if not visited[s]:
            q, comp = deque([s]), [s]
            visited[s] = True
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters

def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    col = 'theta' if mode=='theta' else 'r'
    df = df_cluster.sort_values(col)
    dif = np.diff(df[col].values)
    if np.all(dif<=gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(dif):
        if d>gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i+1
    subs.append(df.iloc[start:].copy())
    return subs

def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts/ref,1.0))

def generate_bfs_seeds(id_halo, theta_min, theta_max,
                         quartile_threshold=0.155,
                         theta_diff=3.0, r_diff=0.5,
                         gap_threshold_theta=2.0, gap_threshold_r=2.0,
                         file_prefix='data_rho'):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp']>thr].copy().reset_index(drop=True)
    
    f = _adaptive_factor(len(df_f))
    th_diff, r_d = theta_diff/f, r_diff/f
    gap_th, gap_r = gap_threshold_theta/f, gap_threshold_r/f
    
    graph, _ = build_graph_rectangular(df_f, th_diff, r_d)
    clusters = bfs_components(graph, df_f)
    
    final_cl = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_th, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r, 'r'))
    return final_cl, df_f

def fit_line_to_cluster(df_cluster):
    if len(df_cluster)<2:
        return (None,)*7
    mdl = LinearRegression().fit(df_cluster[['theta']], df_cluster['r'])
    s, b = mdl.coef_[0], mdl.intercept_
    pa = calculate_pa(s,b)
    return s, b, pa, df_cluster['theta'].min(), df_cluster['theta'].max(), df_cluster['r'].min(), df_cluster['r'].max()

def adjust_and_merge_seeds(bfs_clusters, slope_variation_threshold=0.30, bounding_extrap=0.35):
    groups = []
    for cl in bfs_clusters:
        groups.append(dict(zip(['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
                                 (*fit_line_to_cluster(cl), cl))))
    def recalc(g):
        g['slope'],g['intercept'],g['pa'],g['theta_min'],g['theta_max'],g['r_min'],g['r_max'] = fit_line_to_cluster(g['points'])
    def boxes_overlap(g1,g2,e):
        def exp(t0,t1,r0,r1):
            dt, dr = (t1-t0)*e, (r1-r0)*e
            return t0-dt, t1+dt, r0-dr, r1+dr
        a = exp(g1['theta_min'],g1['theta_max'],g1['r_min'],g1['r_max'])
        b = exp(g2['theta_min'],g2['theta_max'],g2['r_min'],g2['r_max'])
        return not (a[1]<b[0] or b[1]<a[0]) and not (a[3]<b[2] or b[3]<a[2])
    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i+1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1,g2,bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, b_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb - g1['slope'])/(abs(g1['slope'])+1e-12)
                        v2 = abs(s_comb - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb
                            recalc(g1)
                            groups.pop(j)
                            merged = True
                            continue
                j += 1
            new.append(g1)
            i += 1
        groups = new
    return [g for g in groups if len(g['points'])>=2]

def validate_dispersion_and_reprocess(groups, dispersion_threshold=1.80, reproc_theta_diff=1.25, reproc_r_diff=0.4):
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points'])<2:
            ok.append(g)
            continue
        res = g['points']['r'] - (g['slope']*g['points']['theta']+g['intercept'])
        (todo if res.std()>dispersion_threshold else ok).append(g)
    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            new.append(dict(zip(['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
                                  (*fit_line_to_cluster(c), c))))
    return ok + new

def plot_groups_cartesian(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points'])>=60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i%len(colors)], g['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, alpha=0.3, label=f"Grupo {i+1}")
        if g['slope'] is not None:
            dt = g['theta_max']-g['theta_min']
            t = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            plt.plot(t, g['slope']*t+g['intercept'], '--', color=c)
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.grid(True)
    plt.title("Subhalo 17 – Grupos (≥ 60 pts) - Cartesiano")
    plt.legend()
    plt.show()

def plot_groups_polar(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points'])>=60]
    print(f"Grupos ≥60 puntos (polar): {len(big)}")
    fig = plt.figure(figsize=(9,8))
    ax = plt.subplot(111, projection='polar')
    ax.scatter(np.radians(df_all['theta']), df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        ax.scatter(np.radians(pts['theta']), pts['r'], s=10, color=c, alpha=0.3, label=f"Grupo {i+1}")
        if g['slope'] is not None:
            dt = g['theta_max']-g['theta_min']
            t_deg = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            r_line = g['slope']*t_deg+g['intercept']
            ax.plot(np.radians(t_deg), r_line, '--', color=c)
    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title("Subhalo 17 – Grupos (≥ 60 pts) - Polar", y=1.05)
    ax.legend(loc='upper right', bbox_to_anchor=(1.20,1.0))
    plt.show()

def main_example_modificado():
    bfs, df_f = generate_bfs_seeds(
        id_halo="17",
        theta_min=0, theta_max=360,
        quartile_threshold=0.155,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=2.0, gap_threshold_r=2.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")
    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.30, bounding_extrap=0.35)
    print(f"Tras fusión: {len(merged)} grupos")
    validated = validate_dispersion_and_reprocess(merged)
    print(f"Tras validación: {len(validated)} grupos")
    plot_groups_cartesian(validated, df_f, line_extrap=0.15)
    plot_groups_polar(validated, df_f, line_extrap=0.15)
    
    grupos_finales = [g for g in validated if len(g['points'])>=60]
    
    puntos_grupos = set()
    for g in grupos_finales:
        puntos_grupos.update(g['points']['id'].tolist())
    
    background = df_f[~df_f['id'].isin(puntos_grupos)].copy()
    print(f"Puntos background: {len(background)}")
    
    return {'grupos_finales': grupos_finales, 'background': background}

# =============================================================================
# 2) Funciones SPH en el espacio (θ, r) y trazado de caminos densos
# =============================================================================

def cubic_spline_kernel(r, h):
    q = r / h
    sigma = 10.0/(7.0*np.pi*h**2)
    if 0 <= q <= 1:
        return sigma*(1 - 1.5*q**2 + 0.75*q**3)
    elif 1 < q <= 2:
        return sigma*0.25*(2 - q)**3
    else:
        return 0.0

def compute_density_grid(points_pr, grid_theta, grid_r, masses, h=5.0):
    """
    points_pr: array (N,2) con [θ, r] (θ en grados, r en sus unidades).
    masses: array (N,) con la masa (peso) de cada punto.
    grid_theta, grid_r: malla en el espacio (θ, r).
    h: radio de suavizado.
    Retorna la densidad evaluada en cada nodo.
    """
    density = np.zeros(grid_theta.shape)
    kernel_func = np.vectorize(cubic_spline_kernel)
    for i, pt in enumerate(points_pr):
        d = np.sqrt((grid_theta - pt[0])**2 + (grid_r - pt[1])**2)
        density += masses[i] * kernel_func(d, h)
    return density

def extract_paths(density, grid_theta, grid_r, density_threshold):
    cs = plt.contour(grid_theta, grid_r, density, levels=[density_threshold])
    paths = []
    for collection in cs.collections:
        for path in collection.get_paths():
            v = path.vertices
            paths.append(v)
    plt.clf()
    return paths

def main_sph_path_reales():
    """
    Usa los datos reales del pipeline para construir la malla en (θ, r), calcular la densidad
    con masa diferenciada (grupos finales tienen mayor masa) y extraer los caminos (isocontornos)
    de mayor longitud.
    """
    datos_finales = main_example_modificado()
    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']
    
    # Extraer puntos en (θ, r)
    bg_points = background[['theta','r']].values
    bg_masses = np.ones(len(bg_points))  # masa 1 para background
    group_points_list = []
    group_masses_list = []
    for grupo in grupos_finales:
        pts = grupo['points'][['theta','r']].values
        group_points_list.append(pts)
        # Asignar mayor masa, por ejemplo 5, a los puntos de grupos finales
        group_masses_list.append(5 * np.ones(len(pts)))
    if group_points_list:
        group_points = np.vstack(group_points_list)
        group_masses = np.hstack(group_masses_list)
    else:
        group_points = np.empty((0,2))
        group_masses = np.empty((0,))
    
    all_points = np.vstack([bg_points, group_points])
    all_masses = np.hstack([bg_masses, group_masses])
    
    # Definir la malla en (θ, r)
    theta_min, theta_max = all_points[:,0].min()-5, all_points[:,0].max()+5
    r_min, r_max = all_points[:,1].min()-1, all_points[:,1].max()+1
    n_theta, n_r = 300, 300
    theta_vals = np.linspace(theta_min, theta_max, n_theta)
    r_vals = np.linspace(r_min, r_max, n_r)
    grid_theta, grid_r = np.meshgrid(theta_vals, r_vals)
    
    # Calcular la densidad en la malla con SPH
    h = 5.0
    density = compute_density_grid(all_points, grid_theta, grid_r, all_masses, h=h)
    density_smooth = gaussian_filter(density, sigma=1)
    
    # Elegir umbral de densidad, por ejemplo el percentil 90 de la densidad suave.
    density_threshold = np.percentile(density_smooth, 90)
    paths = extract_paths(density_smooth, grid_theta, grid_r, density_threshold)
    
    # Filtrar caminos por longitud (calcular la longitud en (θ, r))
    def path_length(path):
        return np.sum(np.linalg.norm(path[1:]-path[:-1], axis=1))
    lengths = np.array([path_length(p) for p in paths])
    if len(lengths)==0:
        print("No se encontraron caminos.")
        return
    median_length = np.median(lengths)
    selected_paths = [p for p, L in zip(paths, lengths) if L >= median_length]
    
    # Graficar la malla de densidad y los caminos en (θ, r)
    plt.figure(figsize=(16,6))
    plt.imshow(density_smooth, extent=[theta_min, theta_max, r_min, r_max],
               origin='lower', cmap='inferno', aspect='auto')
    # No mostramos colorbar para mayor claridad
    for p in selected_paths:
        plt.plot(p[:,0], p[:,1], '-o', linewidth=2)
    plt.scatter(all_points[:,0], all_points[:,1], c='cyan', s=0.4, label='Puntos', alpha=0.3)
    plt.title("Malla de densidad SPH y caminos densos (espacio (θ, r))")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend()
    plt.show()
    
    return selected_paths

if __name__ == "__main__":
    paths = main_sph_path_reales()


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Ejemplo de conexión de grupos mediante una malla de densidad SPH en el espacio (θ, r).

Se crea una grilla en el espacio de parámetros (θ, r) de los datos reales (background y grupos finales),
se interpola la densidad usando un kernel SPH (calculado en (θ, r)) y se extraen los isocontornos
(de mayor longitud, filtrados por la mediana) para representar los caminos o “fronteras densas”.

Se usan como datos fuente:
    datos_finales = main_example_modificado()
    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']

La visualización se realiza en un tamaño amplio y sin colorbar.
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import pandas as pd

# --- Función SPH: Kernel cúbico spline en 2D, aplicado en (θ, r) ---
def cubic_spline_kernel(r, h):
    q = r/h
    sigma = 10.0/(7.0*np.pi*h**2)  # normalización en 2D (puedes ajustar este valor según la escala de θ y r)
    if 0 <= q <= 1:
        return sigma * (1 - 1.5*q**2 + 0.75*q**3)
    elif 1 < q <= 2:
        return sigma * (0.25 * (2 - q)**3)
    else:
        return 0.0

# --- Función para interpolar la densidad en una malla en (θ, r) ---
def compute_density_grid(points_pr, grid_theta, grid_r, h=5.0):
    """
    points_pr: array (N,2) con columnas [θ, r] (θ en grados, r en sus unidades).
    grid_theta, grid_r: matrices resultantes de np.meshgrid en (θ, r).
    h: radio de suavizado (en unidades de θ y r).
    Retorna la densidad evaluada en cada nodo de la malla.
    """
    density = np.zeros(grid_theta.shape)
    kernel_func = np.vectorize(cubic_spline_kernel)
    for pt in points_pr:
        # La distancia se calcula de forma Euclidiana en (θ, r):
        d = np.sqrt((grid_theta - pt[0])**2 + (grid_r - pt[1])**2)
        density += kernel_func(d, h)
    return density

# --- Función para extraer isocontornos (caminos densos) en (θ, r) ---
def extract_and_connect_paths(density, grid_theta, grid_r, density_threshold):
    """
    A partir del campo de densidad evaluado en la malla (grid_theta, grid_r),
    extrae los contornos (isocontornos) donde la densidad es igual a density_threshold.
    Retorna una lista de caminos, cada uno como un array de coordenadas [θ, r].
    """
    cs = plt.contour(grid_theta, grid_r, density, levels=[density_threshold])
    paths = []
    for collection in cs.collections:
        for path in collection.get_paths():
            v = path.vertices
            paths.append(v)
    plt.clf()  # Limpiar la figura para no sobreponer gráficos previos
    return paths

# --- Función principal utilizando datos reales en (θ, r) ---
def main_sph_path_reales():
    # Se obtiene la información real del pipeline
    datos_finales = main_example_modificado()
    grupos_finales = datos_finales['grupos_finales']
    background = datos_finales['background']
    
    # En este enfoque se trabaja directamente en (θ, r).
    # Extraer puntos del background y de los grupos finales (usamos únicamente las columnas 'theta' y 'r')
    bg_points = background[['theta','r']].values
    # Para enfatizar los grupos finales (pozos densos), asignamos mayor "peso" a sus puntos.
    group_points_list = []
    for grupo in grupos_finales:
        pts = grupo['points'][['theta','r']].values
        group_points_list.append(pts)
    if group_points_list:
        group_points = np.vstack(group_points_list)
    else:
        group_points = np.empty((0,2))
    
    # Combinar puntos (trabajamos en (θ, r)):
    all_points = np.vstack([bg_points, group_points])
    
    # Definir la malla en (θ, r).
    theta_min, theta_max = all_points[:,0].min()-5, all_points[:,0].max()+5
    r_min, r_max = all_points[:,1].min()-1, all_points[:,1].max()+1
    n_theta, n_r = 300, 300
    theta_vals = np.linspace(theta_min, theta_max, n_theta)
    r_vals = np.linspace(r_min, r_max, n_r)
    grid_theta, grid_r = np.meshgrid(theta_vals, r_vals)
    
    # Asignar masas: masa 1 para background y masa 5 para puntos de grupos finales.
    bg_masses = np.ones(len(bg_points))
    if group_points.shape[0] > 0:
        group_masses = 5 * np.ones(len(group_points))
        all_masses = np.hstack([bg_masses, group_masses])
    else:
        all_masses = bg_masses
    
    # Calcular la densidad en la malla usando SPH (trabajando en (θ, r)).
    h = 5.0  # Ajusta según la escala de θ (grados) y r
    density = compute_density_grid(all_points, grid_theta, grid_r, h=h)
    density_smooth = gaussian_filter(density, sigma=1)
    
    # Definir el umbral de densidad; por ejemplo, el percentil 90
    density_threshold = np.percentile(density_smooth, 60)
    
    # Extraer los caminos (isocontornos) en (θ, r) donde la densidad = density_threshold
    paths = extract_and_connect_paths(density_smooth, grid_theta, grid_r, density_threshold)
    
    # Filtrar los caminos: se seleccionan aquellos que tengan una longitud mayor o igual a la mediana.
    def path_length(path):
        return np.sum(np.linalg.norm(path[1:] - path[:-1], axis=1))
    lengths = np.array([path_length(p) for p in paths])
    if len(lengths)==0:
        print("No se encontraron caminos.")
        return
    median_length = np.median(lengths)
    selected_paths = [p for p, L in zip(paths, lengths) if L >= median_length]
    
    # Graficar la malla de densidad y los caminos en (θ, r) sin colorbar, en un tamaño amplio.
    plt.figure(figsize=(16,6))
    plt.imshow(density_smooth, extent=[theta_min, theta_max, r_min, r_max],
               origin='lower', cmap='inferno', aspect='auto')
    for p in selected_paths:
        plt.plot(p[:,0], p[:,1], '-o', linewidth=2)
    plt.scatter(all_points[:,0], all_points[:,1], c='cyan', s=10, alpha=0.3, label='Puntos')
    plt.title("Malla de densidad SPH y caminos densos (espacio (θ, r))")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend()
    plt.show()
    
    return selected_paths

if __name__ == "__main__":
    paths = main_sph_path_reales()


In [0]:
%pip install scikit-image --quiet


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Script SPH + morfología para conectar islas de puntos en espacio (θ, r).
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
from skimage import measure
import pandas as pd

# --- Función SPH: kernel cúbico spline en 2D (θ, r) ---
def cubic_spline_kernel(r, h):
    q = r / h
    sigma = 10.0 / (7.0 * np.pi * h**2)
    if 0 <= q <= 1:
        return sigma * (1 - 1.5*q**2 + 0.75*q**3)
    elif 1 < q <= 2:
        return sigma * (0.25*(2 - q)**3)
    else:
        return 0.0

# --- Interpola densidad en la malla (θ, r) ---
def compute_density_grid(points, grid_theta, grid_r, h=5.0):
    density = np.zeros(grid_theta.shape)
    kernel = np.vectorize(cubic_spline_kernel)
    for pt in points:
        d = np.sqrt((grid_theta - pt[0])**2 + (grid_r - pt[1])**2)
        density += kernel(d, h)
    return density

# --- Función principal ---
def main_sph_connected():
    # 1) Obtén tus datos reales:
    datos = main_example_modificado()
    grupos = datos['grupos_finales']
    background = datos['background']

    # 2) Prepara array de puntos (θ, r):
    bg_pts = background[['theta','r']].values
    grp_pts = np.vstack([g['points'][['theta','r']].values for g in grupos]) \
             if grupos else np.empty((0,2))
    all_pts = np.vstack([bg_pts, grp_pts])

    # 3) Configura malla:
    tmin, tmax = all_pts[:,0].min()-5, all_pts[:,0].max()+5
    rmin, rmax = all_pts[:,1].min()-1, all_pts[:,1].max()+1
    nθ, nr = 300, 300
    thetas = np.linspace(tmin, tmax, nθ)
    rs    = np.linspace(rmin, rmax, nr)
    gridθ, gridr = np.meshgrid(thetas, rs)

    # 4) Calcula y suaviza densidad SPH:
    h = 5.0
    dens = compute_density_grid(all_pts, gridθ, gridr, h=h)
    dens_s = gaussian_filter(dens, sigma=1)

    # 5) Binariza y cierra huecos pequeños:
    pct = 70
    thresh = np.percentile(dens_s, pct)
    mask = dens_s > thresh
    closing_size = 5
    mask_c = binary_closing(mask, structure=np.ones((closing_size, closing_size)))

    # 6) Etiqueta componentes conectadas y filtra por área:
    lbls, n_lbls = label(mask_c)
    areas = np.bincount(lbls.ravel())[1:]  # omite etiqueta 0
    min_area_pixels = 500  # ajusta según tu malla
    good = np.where(areas >= min_area_pixels)[0] + 1

    # 7) Extrae contornos de cada región válida:
    final_paths = []
    for lab in good:
        region = (lbls == lab).astype(float)
        contours = measure.find_contours(region, 0.5)
        for c in contours:
            # convierte índices de píxel a valores reales (θ, r)
            θs = thetas[c[:,1].astype(int)]
            rs_ = rs[c[:,0].astype(int)]
            final_paths.append(np.vstack([θs, rs_]).T)

    # 8) Grafica resultado:
    plt.figure(figsize=(16,6))
    plt.imshow(dens_s, extent=[tmin, tmax, rmin, rmax],
               origin='lower', cmap='inferno', aspect='auto')
    for path in final_paths:
        plt.plot(path[:,0], path[:,1], '-k', linewidth=2)
    plt.scatter(all_pts[:,0], all_pts[:,1], c='cyan', s=5, alpha=0.3, label='Puntos')
    plt.title("SPH + morfología: islas conectadas en (θ, r)")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend()
    plt.show()

    return final_paths

if __name__ == "__main__":
    paths = main_sph_connected()


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación alternativa con MeanShift en espacio (θ, r):
encuentra islas densas sin necesidad de mallas ni grafos,
y permite variar fácilmente el parámetro de bandwidth.
"""

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift, estimate_bandwidth
from scipy.spatial import ConvexHull
import pandas as pd

def main_meanshift_clusters(bandwidth=None, min_cluster_size=15):
    """
    bandwidth: radio de ventana para MeanShift. Si es None, se estima automáticamente.
    min_cluster_size: mínimo de puntos para conservar un cluster.
    """
    # 1) Carga tus datos reales
    datos = main_example_modificado()      # {'grupos_finales', 'background'}
    grupos = datos['grupos_finales']
    background = datos['background']

    # 2) Prepara el array de puntos (θ, r)
    bg_pts = background[['theta','r']].values
    grp_pts = (np.vstack([g['points'][['theta','r']].values for g in grupos])
               if grupos else np.empty((0,2)))
    all_pts = np.vstack([bg_pts, grp_pts])

    # 3) Estima bandwidth si no se proporcionó
    if bandwidth is None:
        # toma una muestra para acelerar la estimación si tienes muchos puntos
        sample = all_pts if all_pts.shape[0] < 5000 else all_pts[np.random.choice(all_pts.shape[0], 5000, replace=False)]
        bandwidth = estimate_bandwidth(sample, quantile=0.2, n_samples=500)
        print(f"Bandwidth estimado: {bandwidth:.2f}")

    # 4) Aplica MeanShift
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    labels = ms.fit_predict(all_pts)
    unique_labels = np.unique(labels)

    # 5) Recopila clusters válidos
    clusters = []
    for lab in unique_labels:
        pts = all_pts[labels == lab]
        if pts.shape[0] < min_cluster_size:
            continue
        clusters.append((lab, pts))

    # 6) Gráfica resultados
    plt.figure(figsize=(16,6))
    # fondo: todos los puntos en gris claro
    plt.scatter(all_pts[:,0], all_pts[:,1],
                c='lightgray', s=5, alpha=0.5, label='Background/Ruido')

    for lab, pts in clusters:
        # puntos del cluster
        plt.scatter(pts[:,0], pts[:,1],
                    s=10, alpha=0.7, label=f'Cluster {lab} ({pts.shape[0]} pts)')
        # envolvente convexa para contorno
        if pts.shape[0] >= 3:
            hull = ConvexHull(pts)
            hull_pts = pts[hull.vertices]
            hull_closed = np.vstack([hull_pts, hull_pts[0]])
            plt.plot(hull_closed[:,0], hull_closed[:,1], '-', linewidth=2)

    plt.title(f"Segmentación con MeanShift (bandwidth={bandwidth:.2f})")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend(markerscale=2, fontsize='small')
    plt.tight_layout()
    plt.show()

    # 7) Devuelve solo los arrays de puntos para cada cluster
    return [pts for _, pts in clusters]

if __name__ == "__main__":
    # Ejemplo de llamada variando parámetros:
    #  - bandwidth=None (se estima)
    #  - min_cluster_size=20
    clusters = main_meanshift_clusters(bandwidth=None, min_cluster_size=20)


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación ligera por histograma 2D + cierre morfológico + asignación de puntos.
Evita cálculos de distancia O(N²) usando una malla de baja resolución.
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import pandas as pd

def main_histogram_clusters(
    bins_theta=200,
    bins_r=100,
    smooth_sigma=1,
    density_percentile=75,
    closing_size=3,
    min_cluster_size=50
):
    """
    Parámetros:
      bins_theta        → número de bins en θ
      bins_r            → número de bins en r
      smooth_sigma      → sigma para suavizado gaussiano del histograma
      density_percentile→ percentil para umbralizar densidad
      closing_size      → tamaño del structuring element para cerrar gaps
      min_cluster_size  → mínimo de puntos para conservar un cluster
    """

    # 1) Carga datos reales
    datos      = main_example_modificado()       # debe devolver {'grupos_finales','background'}
    grupos     = datos['grupos_finales']
    background = datos['background']

    # 2) Construye array (θ, r)
    bg_pts  = background[['theta','r']].values
    grp_pts = np.vstack([g['points'][['theta','r']].values for g in grupos]) \
              if grupos else np.empty((0,2))
    pts = np.vstack([bg_pts, grp_pts])

    # 3) Histograma 2D
    hist, theta_edges, r_edges = np.histogram2d(
        pts[:,0], pts[:,1],
        bins=[bins_theta, bins_r]
    )

    # 4) Suaviza el histograma y umbraliza
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)
    thr = np.percentile(hist_s[hist_s>0], density_percentile)
    mask = hist_s > thr

    # 5) Cierra pequeños huecos en la máscara
    mask_c = binary_closing(mask, structure=np.ones((closing_size, closing_size)))

    # 6) Etiqueta regiones conectadas
    labels, n_labels = label(mask_c)

    # 7) Asigna cada punto a la etiqueta de su bin
    #    digitize da índice de bin [1..bins], restamos 1 para [0..bins-1]
    t_idx = np.digitize(pts[:,0], theta_edges) - 1
    r_idx = np.digitize(pts[:,1], r_edges) - 1
    t_idx = np.clip(t_idx, 0, bins_theta-1)
    r_idx = np.clip(r_idx, 0, bins_r-1)
    pt_labels = labels[t_idx, r_idx]

    # 8) Recolecta clusters válidos
    clusters = []
    for lab in range(1, n_labels+1):
        mask_pts = (pt_labels == lab)
        if mask_pts.sum() >= min_cluster_size:
            clusters.append(pts[mask_pts])

    # 9) Visualiza
    plt.figure(figsize=(16, 6))
    # mapa de calor de densidad
    extent = [theta_edges[0], theta_edges[-1], r_edges[0], r_edges[-1]]
    plt.imshow(hist_s.T, origin='lower', extent=extent, aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad (histogram2d suavizado)')
    # superpone puntos de cada cluster
    for i, c in enumerate(clusters):
        plt.scatter(c[:,0], c[:,1], s=10, alpha=0.7, label=f'Cluster {i} ({len(c)} pts)')
    plt.title("Segmentación ligera: histograma2D + morfología")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend(markerscale=2, fontsize='small')
    plt.tight_layout()
    plt.show()

    return clusters

if __name__ == "__main__":
    clusters = main_histogram_clusters(
        bins_theta=150,
        bins_r=80,
        smooth_sigma=1,
        density_percentile=70,
        closing_size=4,
        min_cluster_size=40
    )


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación ligera + BFS ponderado sobre histograma 2D:
conecta islas separadas por huecos cuyo ancho real sea ≤ max_gap_dist.
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import pandas as pd
import heapq
import math
from collections import defaultdict

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra

def main_histogram_weighted_bfs_clusters(
    bins_theta=150,
    bins_r=80,
    smooth_sigma=1,
    density_percentile=70,
    closing_size=4,
    min_cluster_size=40,
    max_gap_dist=2.0  # distancia máxima (en unidades reales) para cruzar gaps
):
    # 1) Carga datos
    datos      = main_example_modificado()
    grupos     = datos['grupos_finales']
    background = datos['background']

    # 2) Construye array de puntos (θ, r)
    bg_pts  = background[['theta','r']].values
    grp_pts = np.vstack([g['points'][['theta','r']].values for g in grupos]) \
              if grupos else np.empty((0,2))
    pts = np.vstack([bg_pts, grp_pts])

    # 3) Histograma 2D y suavizado
    hist, theta_edges, r_edges = np.histogram2d(
        pts[:,0], pts[:,1], bins=[bins_theta, bins_r]
    )
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)

    # 4) Máscara y cierre morfológico
    thr    = np.percentile(hist_s[hist_s>0], density_percentile)
    mask   = hist_s > thr
    mask_c = binary_closing(mask, structure=np.ones((closing_size,closing_size)))

    # 5) Etiqueta componentes en la máscara
    labels_grid, n_labels = label(mask_c)

    # 6) Prepara Dijkstra multinodal
    #    Calcula los “pasos” reales en θ y r
    dtheta = theta_edges[1] - theta_edges[0]
    dr     = r_edges[1]     - r_edges[0]
    #    Vecinos 8‑conectividad
    neigh = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]

    #    UnionFind sobre etiquetas [1..n_labels]
    uf = UnionFind(n_labels+1)

    #    Matriz de mejor distancia alcanzada (solo para background)
    h, w = labels_grid.shape
    best_dist = np.full((h,w), np.inf, dtype=float)
    label_map = np.zeros((h,w), dtype=int)

    #    Cola de prioridad: (dist, i, j, lab)
    pq = []

    # 7) Semillas = celdas de frontera (mask→fondo)
    for i in range(h):
        for j in range(w):
            lab = labels_grid[i,j]
            if lab==0: 
                continue
            # si algún vecino es fondo, es frontera
            for di,dj in neigh:
                ni,nj = i+di, j+dj
                if 0<=ni<h and 0<=nj<w and labels_grid[ni,nj]==0:
                    # inicializa ese fondo vecino como semilla
                    dist0 = math.hypot(di*dtheta, dj*dr)
                    if dist0 <= max_gap_dist:
                        # seed position is the fondo cell
                        if dist0 < best_dist[ni,nj]:
                            best_dist[ni,nj], label_map[ni,nj] = dist0, lab
                            heapq.heappush(pq, (dist0, ni, nj, lab))
            # no buscamos en más vecinos: pasamos al siguiente

    # 8) Dijkstra multinodal sobre fondo
    while pq:
        dist, i, j, lab = heapq.heappop(pq)
        if dist > best_dist[i,j] or dist > max_gap_dist:
            continue
        # mira fronteras de mask que toquen este fondo: si encuentra otra etiqueta, une
        for di,dj in neigh:
            ni,nj = i+di, j+dj
            if not (0<=ni<h and 0<=nj<w):
                continue
            lab2 = labels_grid[ni,nj]
            if lab2>0 and lab2!=lab:
                uf.union(lab, lab2)
        # avanza en fondo
        for di,dj in neigh:
            ni,nj = i+di, j+dj
            if not (0<=ni<h and 0<=nj<w): 
                continue
            if labels_grid[ni,nj]!=0:
                continue  # no queremos pasar dentro de máscara
            step = math.hypot(di*dtheta, dj*dr)
            nd = dist + step
            if nd <= max_gap_dist and nd < best_dist[ni,nj]:
                best_dist[ni,nj] = nd
                label_map[ni,nj] = lab
                heapq.heappush(pq, (nd, ni, nj, lab))

    # 9) Reconstruye clusters finales uniendo etiquetas según UnionFind
    merged = defaultdict(list)
    # asigna a cada punto la etiqueta raíz de su bin
    t_idx = np.clip(np.digitize(pts[:,0], theta_edges)-1, 0, bins_theta-1)
    r_idx = np.clip(np.digitize(pts[:,1], r_edges)-1, 0, bins_r-1)
    for pi, (ti, ri) in enumerate(zip(t_idx, r_idx)):
        old_lab = labels_grid[ti, ri]
        if old_lab>0:
            root = uf.find(old_lab)
            merged[root].append(pts[pi])

    # filtra tamaño mínimo
    final_clusters = [
        np.vstack(ps) for ps in merged.values() if len(ps) >= min_cluster_size
    ]

    # 10) Visualiza
    plt.figure(figsize=(16,6))
    plt.imshow(hist_s.T, origin='lower',
               extent=[theta_edges[0],theta_edges[-1],
                       r_edges[0],  r_edges[-1]],
               aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for i, c in enumerate(final_clusters):
        plt.scatter(c[:,0], c[:,1],
                    s=10, alpha=0.7, label=f'Isla {i} ({len(c)} pts)')
    plt.title(f"Gap‑sensible BFS (max_gap={max_gap_dist})")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend(markerscale=2, fontsize='small')
    plt.tight_layout()
    plt.show()

    return final_clusters

if __name__ == "__main__":
    clusters = main_histogram_weighted_bfs_clusters(
        bins_theta=100,
        bins_r=60,
        smooth_sigma=1,
        density_percentile=80,
        closing_size=2,
        min_cluster_size=40,
        max_gap_dist=0.5
    )


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import pandas as pd
import heapq
import math
from collections import defaultdict
from itertools import combinations

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra

def main_histogram_weighted_bfs_clusters(
    bins_theta=100,
    bins_r=60,
    smooth_sigma=1,
    density_percentile=80,
    closing_size=2,
    min_cluster_size=40,
    max_gap_dist=0.5,
    theta_bin_size=10
):
    datos      = main_example_modificado()
    grupos     = datos['grupos_finales']
    background = datos['background']

    bg_pts  = background[['theta','r']].values
    grp_pts = np.vstack([g['points'][['theta','r']].values for g in grupos]) if grupos else np.empty((0,2))
    pts = np.vstack([bg_pts, grp_pts])

    hist, theta_edges, r_edges = np.histogram2d(
        pts[:,0], pts[:,1], bins=[bins_theta, bins_r]
    )
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)

    thr    = np.percentile(hist_s[hist_s>0], density_percentile)
    mask   = hist_s > thr
    mask_c = binary_closing(mask, structure=np.ones((closing_size,closing_size)))
    labels_grid, n_labels = label(mask_c)

    dtheta = theta_edges[1] - theta_edges[0]
    dr     = r_edges[1]     - r_edges[0]
    neigh = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf = UnionFind(n_labels+1)
    h, w = labels_grid.shape
    best_dist = np.full((h,w), np.inf, dtype=float)
    label_map = np.zeros((h,w), dtype=int)
    pq = []

    for i in range(h):
        for j in range(w):
            lab = labels_grid[i,j]
            if lab==0: continue
            for di,dj in neigh:
                ni,nj = i+di, j+dj
                if 0<=ni<h and 0<=nj<w and labels_grid[ni,nj]==0:
                    dist0 = math.hypot(di*dtheta, dj*dr)
                    if dist0 <= max_gap_dist:
                        if dist0 < best_dist[ni,nj]:
                            best_dist[ni,nj], label_map[ni,nj] = dist0, lab
                            heapq.heappush(pq, (dist0, ni, nj, lab))

    while pq:
        dist, i, j, lab = heapq.heappop(pq)
        if dist > best_dist[i,j] or dist > max_gap_dist:
            continue
        for di,dj in neigh:
            ni,nj = i+di, j+dj
            if not (0<=ni<h and 0<=nj<w): continue
            lab2 = labels_grid[ni,nj]
            if lab2>0 and lab2!=lab:
                uf.union(lab, lab2)
        for di,dj in neigh:
            ni,nj = i+di, j+dj
            if not (0<=ni<h and 0<=nj<w): continue
            if labels_grid[ni,nj]!=0: continue
            step = math.hypot(di*dtheta, dj*dr)
            nd = dist + step
            if nd <= max_gap_dist and nd < best_dist[ni,nj]:
                best_dist[ni,nj] = nd
                label_map[ni,nj] = lab
                heapq.heappush(pq, (nd, ni, nj, lab))

    merged = defaultdict(list)
    t_idx = np.clip(np.digitize(pts[:,0], theta_edges)-1, 0, bins_theta-1)
    r_idx = np.clip(np.digitize(pts[:,1], r_edges)-1, 0, bins_r-1)
    for pi, (ti, ri) in enumerate(zip(t_idx, r_idx)):
        old_lab = labels_grid[ti, ri]
        if old_lab>0:
            root = uf.find(old_lab)
            merged[root].append(pts[pi])

    final_clusters = [
        np.vstack(ps) for ps in merged.values() if len(ps) >= min_cluster_size
    ]

    # CALCULAR CENTROIDES POR BLOQUE DE THETA
    island_skeletons = []
    for c in final_clusters:
        theta = c[:, 0]
        r = c[:, 1]
        theta_bins = np.arange(np.min(theta), np.max(theta) + theta_bin_size, theta_bin_size)
        skeleton = []
        for i in range(len(theta_bins) - 1):
            mask = (theta >= theta_bins[i]) & (theta < theta_bins[i+1])
            if np.any(mask):
                r_mean = np.mean(r[mask])
                theta_mean = np.mean(theta[mask])
                skeleton.append((theta_mean, r_mean))
        island_skeletons.append(skeleton)

    # VISUALIZACIÓN
    plt.figure(figsize=(16,6))
    plt.imshow(hist_s.T, origin='lower',
               extent=[theta_edges[0],theta_edges[-1],
                       r_edges[0],  r_edges[-1]],
               aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')

    # Dibujar clusters
    for i, c in enumerate(final_clusters):
        plt.scatter(c[:,0], c[:,1],
                    s=10, alpha=0.7, label=f'Isla {i} ({len(c)} pts)')

    # Dibujar puntos blancos = centroides cada 10°
    for skeleton in island_skeletons:
        skeleton = np.array(skeleton)
        plt.plot(skeleton[:, 0], skeleton[:, 1], 'wo', markersize=4, label='_nolegend_')

    # Conexión entre centroides extremos si Δr > 0, Δθ < 0, y fondo denso
    background_thr = np.percentile(hist_s[hist_s > 0], density_percentile)

    for i, skel_i in enumerate(island_skeletons):
        for j, skel_j in enumerate(island_skeletons):
            if i == j or not skel_i or not skel_j:
                continue
            p_i = skel_i[-1]  # punto más a la derecha (θ alto)
            p_j = skel_j[0]   # punto más a la izquierda (θ bajo)

            delta_r = p_j[1] - p_i[1]
            delta_theta = p_j[0] - p_i[0]
            if delta_r > 0 and delta_theta < 0:
                # Obtener índices en histograma
                t_idx = np.clip(np.digitize([p_i[0], p_j[0]], theta_edges)-1, 0, bins_theta-1)
                r_idx = np.clip(np.digitize([p_i[1], p_j[1]], r_edges)-1, 0, bins_r-1)
                slice_hist = hist_s[min(t_idx):max(t_idx)+1, min(r_idx):max(r_idx)+1]
                if np.mean(slice_hist) >= background_thr:
                    plt.plot([p_i[0], p_j[0]], [p_i[1], p_j[1]], 'c--', lw=1.5, alpha=0.8)

    plt.title(f"Conexiones entre centroides Δr>0, Δθ<0 y fondo denso")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend(markerscale=2, fontsize='small')
    plt.tight_layout()
    plt.show()

    return final_clusters

if __name__ == "__main__":
    clusters = main_histogram_weighted_bfs_clusters()


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación ligera + BFS ponderado sobre histograma 2D:
– conecta islas separadas por huecos ≤ max_gap_dist
– calcula centroides cada theta_bin_size grados
– conecta cada primer y último centroide de cada isla con su vecino
  más cercano que cumpla Δr>0 y Δθ<0 (distancia mínima)
  para un enfoque alternativo más simple
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq
import math
from collections import defaultdict

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra

def main_histogram_weighted_bfs_clusters(
    bins_theta=100,
    bins_r=60,
    smooth_sigma=1.0,
    density_percentile=80,
    closing_size=2,
    min_cluster_size=40,
    max_gap_dist=0.5,
    theta_bin_size=10
):
    # 1) Carga datos
    datos      = main_example_modificado()
    grupos     = datos['grupos_finales']
    background = datos['background']

    # 2) Apilar puntos (θ, r)
    bg_pts  = background[['theta','r']].values
    grp_pts = np.vstack([g['points'][['theta','r']].values for g in grupos]) \
              if grupos else np.empty((0,2))
    pts = np.vstack([bg_pts, grp_pts])

    # 3) Histograma 2D y suavizado
    hist, theta_edges, r_edges = np.histogram2d(
        pts[:,0], pts[:,1], bins=[bins_theta, bins_r]
    )
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)

    # 4) Máscara y cierre morfológico
    thr = np.percentile(hist_s[hist_s>0], density_percentile)
    mask = hist_s > thr
    mask_c = binary_closing(mask, structure=np.ones((closing_size,closing_size)))
    labels_grid, n_labels = label(mask_c)

    # 5) BFS ponderado para unir componentes
    dtheta = theta_edges[1] - theta_edges[0]
    dr     = r_edges[1]     - r_edges[0]
    neigh  = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf     = UnionFind(n_labels+1)
    h, w   = labels_grid.shape
    best_dist = np.full((h,w), np.inf, dtype=float)
    pq = []

    # semillas en la frontera
    for i in range(h):
        for j in range(w):
            lab = labels_grid[i,j]
            if lab == 0: continue
            for di, dj in neigh:
                ni, nj = i+di, j+dj
                if 0 <= ni < h and 0 <= nj < w and labels_grid[ni,nj] == 0:
                    d0 = math.hypot(di*dtheta, dj*dr)
                    if d0 <= max_gap_dist and d0 < best_dist[ni,nj]:
                        best_dist[ni,nj] = d0
                        heapq.heappush(pq, (d0, ni, nj, lab))

    # expansión sobre el fondo
    while pq:
        dist, i, j, lab = heapq.heappop(pq)
        if dist > best_dist[i,j] or dist > max_gap_dist:
            continue
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if not (0 <= ni < h and 0 <= nj < w): continue
            lab2 = labels_grid[ni,nj]
            if lab2 > 0 and lab2 != lab:
                uf.union(lab, lab2)
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if not (0 <= ni < h and 0 <= nj < w): continue
            if labels_grid[ni,nj] != 0: continue
            step = math.hypot(di*dtheta, dj*dr)
            nd = dist + step
            if nd <= max_gap_dist and nd < best_dist[ni,nj]:
                best_dist[ni,nj] = nd
                heapq.heappush(pq, (nd, ni, nj, lab))

    # 6) Reconstruir clusters finales
    merged = defaultdict(list)
    t_idx = np.clip(np.digitize(pts[:,0], theta_edges)-1, 0, bins_theta-1)
    r_idx = np.clip(np.digitize(pts[:,1], r_edges)-1,     0, bins_r-1)
    for pi, (ti, ri) in enumerate(zip(t_idx, r_idx)):
        lab = labels_grid[ti, ri]
        if lab > 0:
            root = uf.find(lab)
            merged[root].append(pts[pi])
    final_clusters = [np.vstack(v) for v in merged.values() if len(v) >= min_cluster_size]

    # 7) Calcular centroides por bloque de θ
    island_skeletons = []
    for c in final_clusters:
        thetas = c[:,0]; rs = c[:,1]
        bins = np.arange(thetas.min(), thetas.max() + theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins)-1):
            mask_block = (thetas >= bins[k]) & (thetas < bins[k+1])
            if mask_block.any():
                ske.append((thetas[mask_block].mean(), rs[mask_block].mean()))
        island_skeletons.append(ske)

    # 8) Visualización
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower',
               extent=[theta_edges[0],theta_edges[-1],r_edges[0],r_edges[-1]],
               aspect='auto', cmap='inferno', zorder=0)
    plt.colorbar(label='Densidad suavizada')

    # dibujar clustering
    for i, c in enumerate(final_clusters):
        plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {i}', zorder=1)

    # dibujar centroides
    for ske in island_skeletons:
        arr = np.array(ske)
        if arr.size:
            plt.scatter(arr[:,0], arr[:,1], c='w', s=40,
                        edgecolors='k', linewidths=0.5, zorder=2)

    # 9) Conexiones por distancia mínima en Δr>0, Δθ<0
    connections = []
    for idx_i, ske_i in enumerate(island_skeletons):
        if not ske_i:
            continue
        for pi in (ske_i[0], ske_i[-1]):
            best_dist_seg = np.inf
            best_pj = None
            for idx_j, ske_j in enumerate(island_skeletons):
                if idx_i == idx_j or not ske_j:
                    continue
                for pj in (ske_j[0], ske_j[-1]):
                    dr_delta = pj[1] - pi[1]
                    dtheta_delta = pj[0] - pi[0]
                    if dr_delta > 0 and dtheta_delta < 0:
                        dist_seg = math.hypot(dtheta_delta, dr_delta)
                        if dist_seg < best_dist_seg:
                            best_dist_seg = dist_seg
                            best_pj = pj
            if best_pj is not None:
                connections.append((pi, best_pj))

    # dibujar conexiones en blanco
    for pi, pj in connections:
        plt.plot([pi[0], pj[0]], [pi[1], pj[1]],
                 linestyle='--', linewidth=2.5,
                 color='white', alpha=0.9, zorder=3)

    plt.title("Conexiones por distancia mínima (Δr>0, Δθ<0)")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend(markerscale=2, fontsize='small', loc='upper right')
    plt.tight_layout()
    plt.show()

    return final_clusters

if __name__ == "__main__":
    clusters = main_histogram_weighted_bfs_clusters()




In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación ligera + BFS ponderado sobre histograma 2D + conexiones de centroides:
- Conecta islas separadas por huecos ≤ max_gap_dist
- Calcula centroides cada theta_bin_size grados
- Conecta internamente cada isla (skeleton) entre centroides adyacentes
- Conecta externamente solo desde los extremos de cada isla (primer y último centroid),
  máximo dos conexiones por par de islas (Δr>0, Δθ<0)
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq
import math
from collections import defaultdict

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100,
    bins_r=60,
    smooth_sigma=1.0,
    density_percentile=80,
    closing_size=2,
    min_cluster_size=40,
    max_gap_dist=0.5,
    theta_bin_size=10
):
    datos      = main_example_modificado()
    grupos     = datos['grupos_finales']
    background = datos['background']

    # Apilar puntos (θ, r)
    bg_pts  = background[['theta','r']].values
    grp_pts = np.vstack([g['points'][['theta','r']].values for g in grupos]) if grupos else np.empty((0,2))
    pts = np.vstack([bg_pts, grp_pts])

    # Histograma 2D + suavizado
    hist, theta_edges, r_edges = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)

    # Segmentación y etiquetado
    thr = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size,closing_size)))
    labels_grid, n_labels = label(mask_c)

    # BFS ponderado para unir componentes
    dtheta = theta_edges[1] - theta_edges[0]
    dr     = r_edges[1]     - r_edges[0]
    neigh  = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf     = UnionFind(n_labels+1)
    h, w   = labels_grid.shape
    best_dist = np.full((h,w), np.inf)
    pq = []

    # semillas en la frontera
    for i in range(h):
        for j in range(w):
            lab = labels_grid[i,j]
            if lab == 0: continue
            for di, dj in neigh:
                ni, nj = i+di, j+dj
                if 0 <= ni < h and 0 <= nj < w and labels_grid[ni,nj] == 0:
                    step = math.hypot(di*dtheta, dj*dr)
                    if step <= max_gap_dist and step < best_dist[ni,nj]:
                        best_dist[ni,nj] = step
                        heapq.heappush(pq, (step, ni, nj, lab))

    # expansión del BFS
    while pq:
        dist, i, j, lab = heapq.heappop(pq)
        if dist > best_dist[i,j] or dist > max_gap_dist:
            continue
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if 0 <= ni < h and 0 <= nj < w:
                lab2 = labels_grid[ni,nj]
                if lab2 > 0 and lab2 != lab:
                    uf.union(lab, lab2)
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if not (0 <= ni < h and 0 <= nj < w) or labels_grid[ni,nj] != 0:
                continue
            nd = dist + math.hypot(di*dtheta, dj*dr)
            if nd <= max_gap_dist and nd < best_dist[ni,nj]:
                best_dist[ni,nj] = nd
                heapq.heappush(pq, (nd, ni, nj, lab))

    # Recolectar clusters finales
    merged = defaultdict(list)
    t_idx = np.clip(np.digitize(pts[:,0],theta_edges)-1,0,bins_theta-1)
    r_idx = np.clip(np.digitize(pts[:,1],r_edges)-1,0,bins_r-1)
    for pi, (ti, ri) in enumerate(zip(t_idx, r_idx)):
        lab = labels_grid[ti, ri]
        if lab > 0:
            merged[uf.find(lab)].append(pts[pi])
    clusters = [np.vstack(v) for v in merged.values() if len(v) >= min_cluster_size]

    # Calcular centroides (skeletons) y ordenar θ descendente
    skeletons = []
    for c in clusters:
        thetas, rs = c[:,0], c[:,1]
        bins = np.arange(thetas.min(), thetas.max() + theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins)-1):
            mask_block = (thetas >= bins[k]) & (thetas < bins[k+1])
            if mask_block.any():
                ske.append((thetas[mask_block].mean(), rs[mask_block].mean()))
        skeletons.append(sorted(ske, key=lambda x: -x[0]))

    return clusters, skeletons, hist_s, theta_edges, r_edges


def connect_skeletons(skeletons):
    # Conexiones internas: centroides adyacentes en cada isla
    internal = []
    for ske in skeletons:
        for a, b in zip(ske, ske[1:]):
            internal.append((a, b))

    # Conexiones externas: solo desde extremos de cada isla
    external = []
    count_pair = defaultdict(int)
    for i, ske_i in enumerate(skeletons):
        extremes = []
        if len(ske_i) >= 1:
            extremes.append(ske_i[0])                   # extremo máximo θ
        if len(ske_i) > 1:
            extremes.append(ske_i[-1])                  # extremo mínimo θ
        for p in extremes:
            best = None
            best_d = math.inf
            best_j = None
            for j, ske_j in enumerate(skeletons):
                if i == j or not ske_j:
                    continue
                for q in ske_j:
                    dr_delta = q[1] - p[1]
                    dtheta_delta = q[0] - p[0]
                    if dr_delta > 0 and dtheta_delta < 0:
                        dist = math.hypot(dtheta_delta, dr_delta)
                        if dist < best_d:
                            best_d = dist
                            best = (p, q)
                            best_j = j
            if best and count_pair[(i, best_j)] < 2:
                external.append(best)
                count_pair[(i, best_j)] += 1
    return internal, external


def plot_clusters_and_connections(clusters, skeletons, hist_s, theta_edges, r_edges, internal, external):
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower', extent=[theta_edges[0], theta_edges[-1], r_edges[0], r_edges[-1]], aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters):
        plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size:
            plt.scatter(arr[:,0], arr[:,1], c='w', s=40, edgecolors='k', linewidths=0.5)
    # dibujar conexiones internas
    for a, b in internal:
        plt.plot([a[0], b[0]], [a[1], b[1]], '-', linewidth=2)
    # dibujar conexiones externas
    for p, q in external:
        plt.plot([p[0], q[0]], [p[1], q[1]], '--', linewidth=2.5, color='white', alpha=0.9)
    plt.title("Islas y conexiones (internas vs externas)")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend(markerscale=2, fontsize='small', loc='upper right')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    clusters, skeletons, hist_s, theta_edges, r_edges = main_histogram_weighted_bfs_clusters()
    internal, external = connect_skeletons(skeletons)
    plot_clusters_and_connections(clusters, skeletons, hist_s, theta_edges, r_edges, internal, external)



In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación ligera + BFS ponderado sobre histograma 2D + conexiones de centroides:
- Conecta islas separadas por huecos ≤ max_gap_dist
- Calcula centroides cada theta_bin_size grados
- Conecta internamente cada isla (skeleton) entre centroides adyacentes
- Conecta externamente siempre el centroide de menor θ de cada isla con el centroide de mayor θ de otra isla,
  priorizando la cercanía (distancia Euclidiana) y filtrando dr>0, dθ<0 y r>0,
  dejando una única conexión por isla
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq
import math
from collections import defaultdict

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100,
    bins_r=60,
    smooth_sigma=1.0,
    density_percentile=80,
    closing_size=2,
    min_cluster_size=40,
    max_gap_dist=0.5,
    theta_bin_size=10
):
    datos      = main_example_modificado()
    grupos     = datos['grupos_finales']
    background = datos['background']

    # Apilar puntos (θ, r)
    bg_pts  = background[['theta','r']].values
    grp_pts = np.vstack([g['points'][['theta','r']].values for g in grupos]) if grupos else np.empty((0,2))
    pts = np.vstack([bg_pts, grp_pts])

    # Histograma 2D + suavizado
    hist, theta_edges, r_edges = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)

    # Segmentación y etiquetado
    thr = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size,closing_size)))
    labels_grid, n_labels = label(mask_c)

    # BFS ponderado para unir componentes
    dtheta = theta_edges[1] - theta_edges[0]
    dr     = r_edges[1]     - r_edges[0]
    neigh  = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf     = UnionFind(n_labels+1)
    h, w   = labels_grid.shape
    best_dist_grid = np.full((h,w), np.inf)
    pq = []

    # semillas en la frontera
    for i in range(h):
        for j in range(w):
            lab = labels_grid[i,j]
            if lab == 0: continue
            for di, dj in neigh:
                ni, nj = i+di, j+dj
                if 0 <= ni < h and 0 <= nj < w and labels_grid[ni,nj] == 0:
                    step = math.hypot(di*dtheta, dj*dr)
                    if step <= max_gap_dist and step < best_dist_grid[ni,nj]:
                        best_dist_grid[ni,nj] = step
                        heapq.heappush(pq, (step, ni, nj, lab))

    # expansión del BFS
    while pq:
        dist, i, j, lab = heapq.heappop(pq)
        if dist > best_dist_grid[i,j] or dist > max_gap_dist:
            continue
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if 0 <= ni < h and 0 <= nj < w:
                lab2 = labels_grid[ni,nj]
                if lab2 > 0 and lab2 != lab:
                    uf.union(lab, lab2)
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if not (0 <= ni < h and 0 <= nj < w) or labels_grid[ni,nj] != 0:
                continue
            nd = dist + math.hypot(di*dtheta, dj*dr)
            if nd <= max_gap_dist and nd < best_dist_grid[ni,nj]:
                best_dist_grid[ni,nj] = nd
                heapq.heappush(pq, (nd, ni, nj, lab))

    # Recolectar clusters finales
    merged = defaultdict(list)
    t_idx = np.clip(np.digitize(pts[:,0],theta_edges)-1,0,bins_theta-1)
    r_idx = np.clip(np.digitize(pts[:,1],r_edges)-1,0,bins_r-1)
    for pi, (ti, ri) in enumerate(zip(t_idx, r_idx)):
        lab = labels_grid[ti, ri]
        if lab > 0:
            merged[uf.find(lab)].append(pts[pi])
    clusters = [np.vstack(v) for v in merged.values() if len(v) >= min_cluster_size]

    # Calcular centroides (skeletons) y ordenar θ descendente
    skeletons = []
    for c in clusters:
        thetas, rs = c[:,0], c[:,1]
        bins = np.arange(thetas.min(), thetas.max() + theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins)-1):
            mask_block = (thetas >= bins[k]) & (thetas < bins[k+1])
            if mask_block.any():
                ske.append((thetas[mask_block].mean(), rs[mask_block].mean()))
        skeletons.append(sorted(ske, key=lambda x: -x[0]))

    return clusters, skeletons, hist_s, theta_edges, r_edges


def connect_skeletons(skeletons, hist_s, theta_edges, r_edges):
    # Conexiones internas: centroides adyacentes en cada isla
    internal = []
    for ske in skeletons:
        for a, b in zip(ske, ske[1:]):
            internal.append((a, b))

    # Conexiones externas: conecta el mínimo θ de cada isla con el máximo θ de la isla más cercana
    external = []
    for i, ske_i in enumerate(skeletons):
        if not ske_i:
            continue
        p = ske_i[-1]  # centroide con menor θ en la isla i
        best = None
        best_dist = math.inf
        for j, ske_j in enumerate(skeletons):
            if i == j or not ske_j:
                continue
            q = ske_j[0]  # centroide con mayor θ en la isla j
            dr_delta = q[1] - p[1]
            dtheta_delta = q[0] - p[0]
            if dr_delta > 0 and dtheta_delta < 0 and q[1] > 0:
                dist = math.hypot(dtheta_delta, dr_delta)
                if dist < best_dist:
                    best_dist = dist
                    best = (p, q)
        if best:
            external.append(best)
    return internal, external


def plot_clusters_and_connections(clusters, skeletons, hist_s, theta_edges, r_edges, internal, external):
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower', extent=[theta_edges[0], theta_edges[-1], r_edges[0], r_edges[-1]], aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters):
        plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size:
            plt.scatter(arr[:,0], arr[:,1], c='w', s=40, edgecolors='k', linewidths=0.5)
    # dibujar conexiones internas
    for a, b in internal:
        plt.plot([a[0], b[0]], [a[1], b[1]], '-', linewidth=2)
    # dibujar conexiones externas
    for p, q in external:
        plt.plot([p[0], q[0]], [p[1], q[1]], '--', linewidth=2.5, color='white', alpha=0.9)
    plt.title("Islas y conexiones (internas vs externas)")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend(markerscale=2, fontsize='small', loc='upper right')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    clusters, skeletons, hist_s, theta_edges, r_edges = main_histogram_weighted_bfs_clusters()
    internal, external = connect_skeletons(skeletons, hist_s, theta_edges, r_edges)
    plot_clusters_and_connections(clusters, skeletons, hist_s, theta_edges, r_edges, internal, external)


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación ligera + BFS ponderado sobre histograma 2D + conexiones de centroides:
- Conecta islas separadas por huecos ≤ max_gap_dist
- Calcula centroides cada theta_bin_size grados
- Conecta internamente cada isla (skeleton) entre centroides adyacentes
- Conecta externamente siempre el centroide de menor θ de cada isla con el centroide de mayor θ de otra isla,
  priorizando la cercanía (distancia Euclidiana) y filtrando dr>0, dθ<0 y r>0,
  dejando una única conexión por isla
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq
import math
from collections import defaultdict

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100,
    bins_r=60,
    smooth_sigma=1.0,
    density_percentile=80,
    closing_size=2,
    min_cluster_size=40,
    max_gap_dist=0.5,
    theta_bin_size=10
):
    datos      = main_example_modificado()
    grupos     = datos['grupos_finales']
    background = datos['background']

    # Apilar puntos (θ, r)
    bg_pts  = background[['theta','r']].values
    grp_pts = np.vstack([g['points'][['theta','r']].values for g in grupos]) if grupos else np.empty((0,2))
    pts = np.vstack([bg_pts, grp_pts])

    # Histograma 2D + suavizado
    hist, theta_edges, r_edges = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)

    # Segmentación y etiquetado
    thr = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size,closing_size)))
    labels_grid, n_labels = label(mask_c)

    # BFS ponderado para unir componentes
    dtheta = theta_edges[1] - theta_edges[0]
    dr     = r_edges[1]     - r_edges[0]
    neigh  = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf     = UnionFind(n_labels+1)
    h, w   = labels_grid.shape
    best_dist_grid = np.full((h,w), np.inf)
    pq = []

    # semillas en la frontera
    for i in range(h):
        for j in range(w):
            lab = labels_grid[i,j]
            if lab == 0: continue
            for di, dj in neigh:
                ni, nj = i+di, j+dj
                if 0 <= ni < h and 0 <= nj < w and labels_grid[ni,nj] == 0:
                    step = math.hypot(di*dtheta, dj*dr)
                    if step <= max_gap_dist and step < best_dist_grid[ni,nj]:
                        best_dist_grid[ni,nj] = step
                        heapq.heappush(pq, (step, ni, nj, lab))

    # expansión del BFS
    while pq:
        dist, i, j, lab = heapq.heappop(pq)
        if dist > best_dist_grid[i,j] or dist > max_gap_dist:
            continue
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if 0 <= ni < h and 0 <= nj < w:
                lab2 = labels_grid[ni,nj]
                if lab2 > 0 and lab2 != lab:
                    uf.union(lab, lab2)
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if not (0 <= ni < h and 0 <= nj < w) or labels_grid[ni,nj] != 0:
                continue
            nd = dist + math.hypot(di*dtheta, dj*dr)
            if nd <= max_gap_dist and nd < best_dist_grid[ni,nj]:
                best_dist_grid[ni,nj] = nd
                heapq.heappush(pq, (nd, ni, nj, lab))

    # Recolectar clusters finales
    merged = defaultdict(list)
    t_idx = np.clip(np.digitize(pts[:,0],theta_edges)-1,0,bins_theta-1)
    r_idx = np.clip(np.digitize(pts[:,1],r_edges)-1,0,bins_r-1)
    for pi, (ti, ri) in enumerate(zip(t_idx, r_idx)):
        lab = labels_grid[ti, ri]
        if lab > 0:
            merged[uf.find(lab)].append(pts[pi])
    clusters = [np.vstack(v) for v in merged.values() if len(v) >= min_cluster_size]

    # Calcular centroides (skeletons) y ordenar θ descendente
    skeletons = []
    for c in clusters:
        thetas, rs = c[:,0], c[:,1]
        bins = np.arange(thetas.min(), thetas.max() + theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins)-1):
            mask_block = (thetas >= bins[k]) & (thetas < bins[k+1])
            if mask_block.any():
                ske.append((thetas[mask_block].mean(), rs[mask_block].mean()))
        skeletons.append(sorted(ske, key=lambda x: -x[0]))

    return clusters, skeletons, hist_s, theta_edges, r_edges


def connect_skeletons(skeletons, hist_s, theta_edges, r_edges):
    # Conexiones internas: centroides adyacentes en cada isla
    internal = []
    for ske in skeletons:
        for a, b in zip(ske, ske[1:]):
            internal.append((a, b))

    # Conexiones externas: conecta el mínimo θ de cada isla con el máximo θ de la isla más cercana
    external = []
    for i, ske_i in enumerate(skeletons):
        if not ske_i:
            continue
        p = ske_i[-1]  # centroide con menor θ en la isla i
        best = None
        best_dist = math.inf
        for j, ske_j in enumerate(skeletons):
            if i == j or not ske_j:
                continue
            q = ske_j[0]  # centroide con mayor θ en la isla j
            dr_delta = q[1] - p[1]
            dtheta_delta = q[0] - p[0]
            if dr_delta > 0 and dtheta_delta < 0 and q[1] > 0:
                dist = math.hypot(dtheta_delta, dr_delta)
                if dist < best_dist:
                    best_dist = dist
                    best = (p, q)
        if best:
            external.append(best)
    return internal, external


def plot_clusters_and_connections(clusters, skeletons, hist_s, theta_edges, r_edges, internal, external):
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower', extent=[theta_edges[0], theta_edges[-1], r_edges[0], r_edges[-1]], aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters):
        plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size:
            plt.scatter(arr[:,0], arr[:,1], c='w', s=40, edgecolors='k', linewidths=0.5)
    # dibujar conexiones internas
    for a, b in internal:
        plt.plot([a[0], b[0]], [a[1], b[1]], '-', color='white', linewidth=2)
    # dibujar conexiones externas
    for p, q in external:
        plt.plot([p[0], q[0]], [p[1], q[1]], '--', linewidth=2.5, color='white', alpha=0.9)
    plt.title("Islas y conexiones (internas vs externas)")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend(markerscale=2, fontsize='small', loc='upper right')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    clusters, skeletons, hist_s, theta_edges, r_edges = main_histogram_weighted_bfs_clusters()
    internal, external = connect_skeletons(skeletons, hist_s, theta_edges, r_edges)
    plot_clusters_and_connections(clusters, skeletons, hist_s, theta_edges, r_edges, internal, external)


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación ligera + BFS ponderado sobre histograma 2D + conexiones de centroides con centroides de densidad:
- Conecta islas separadas por huecos ≤ max_gap_dist
- Calcula centroides cada theta_bin_size grados
- Conecta internamente cada isla (skeleton) entre centroides adyacentes
- Conecta externamente siempre el centroide de menor θ de cada isla con el centroide de mayor θ de otra isla,
  priorizando cercanía y filtrando dr>0, dθ<0 y r>0 (una conexión por isla)
- Para cada conexión externa, rastrea densidad suavizada alrededor del segmento dentro de un radio dado
  y construye un centroide de densidad, siguiendo los máximos en θ de la densidad suavizada
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq
import math
from collections import defaultdict

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100,
    bins_r=60,
    smooth_sigma=1.0,
    density_percentile=80,
    closing_size=2,
    min_cluster_size=40,
    max_gap_dist=0.5,
    theta_bin_size=10
):
    datos      = main_example_modificado()
    grupos     = datos['grupos_finales']
    background = datos['background']

    # Apilar puntos (θ, r)
    bg_pts  = background[['theta','r']].values
    grp_pts = np.vstack([g['points'][['theta','r']].values for g in grupos]) if grupos else np.empty((0,2))
    pts = np.vstack([bg_pts, grp_pts])

    # Histograma 2D + suavizado
    hist, theta_edges, r_edges = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)

    # Segmentación y etiquetado
    thr = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size,closing_size)))
    labels_grid, n_labels = label(mask_c)

    # BFS ponderado para unir componentes
    dtheta = theta_edges[1] - theta_edges[0]
    dr     = r_edges[1]     - r_edges[0]
    neigh  = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf     = UnionFind(n_labels+1)
    h, w   = labels_grid.shape
    best_dist_grid = np.full((h,w), np.inf)
    pq = []

    # semillas en la frontera
    for i in range(h):
        for j in range(w):
            lab = labels_grid[i,j]
            if lab == 0: continue
            for di, dj in neigh:
                ni, nj = i+di, j+dj
                if 0 <= ni < h and 0 <= nj < w and labels_grid[ni,nj] == 0:
                    step = math.hypot(di*dtheta, dj*dr)
                    if step <= max_gap_dist and step < best_dist_grid[ni,nj]:
                        best_dist_grid[ni,nj] = step
                        heapq.heappush(pq, (step, ni, nj, lab))

    # expansión del BFS
    while pq:
        dist, i, j, lab = heapq.heappop(pq)
        if dist > best_dist_grid[i,j] or dist > max_gap_dist:
            continue
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if 0 <= ni < h and 0 <= nj < w:
                lab2 = labels_grid[ni,nj]
                if lab2 > 0 and lab2 != lab:
                    uf.union(lab, lab2)
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if not (0 <= ni < h and 0 <= nj < w) or labels_grid[ni,nj] != 0:
                continue
            nd = dist + math.hypot(di*dtheta, dj*dr)
            if nd <= max_gap_dist and nd < best_dist_grid[ni,nj]:
                best_dist_grid[ni,nj] = nd
                heapq.heappush(pq, (nd, ni, nj, lab))

    # Recolectar clusters finales
    merged = defaultdict(list)
    t_idx = np.clip(np.digitize(pts[:,0],theta_edges)-1,0,bins_theta-1)
    r_idx = np.clip(np.digitize(pts[:,1],r_edges)-1,0,bins_r-1)
    for pi, (ti, ri) in enumerate(zip(t_idx, r_idx)):
        lab = labels_grid[ti, ri]
        if lab > 0:
            merged[uf.find(lab)].append(pts[pi])
    clusters = [np.vstack(v) for v in merged.values() if len(v) >= min_cluster_size]

    # Calcular centroides (skeletons) y ordenar θ descendente
    skeletons = []
    for c in clusters:
        thetas, rs = c[:,0], c[:,1]
        bins = np.arange(thetas.min(), thetas.max() + theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins)-1):
            mask_block = (thetas >= bins[k]) & (thetas < bins[k+1])
            if mask_block.any():
                ske.append((thetas[mask_block].mean(), rs[mask_block].mean()))
        skeletons.append(sorted(ske, key=lambda x: -x[0]))

    return clusters, skeletons, hist_s, theta_edges, r_edges


def compute_density_centroid(p, q, hist_s, theta_edges, r_edges, radius, samples=100):
    """
    Para un segmento p->q, muestrea puntos a lo largo, recoge vecinos
    dentro de "radius" y calcula el centroide de densidad ponderado.
    """
    # parametrización del segmento
    ts = np.linspace(p[0], q[0], samples)
    rs = np.linspace(p[1], q[1], samples)
    weights = []
    points = []
    # para cada muestra, buscar puntos en hist_s dentro del radio
    for theta_s, r_s in zip(ts, rs):
        # convertir a indices
        ti = np.clip(np.digitize(theta_s, theta_edges)-1, 0, hist_s.shape[0]-1)
        ri = np.clip(np.digitize(r_s,     r_edges)-1,     0, hist_s.shape[1]-1)
        # rango de vecinos
        rad_t = int(np.ceil(radius / (theta_edges[1]-theta_edges[0])))
        rad_r = int(np.ceil(radius / (r_edges[1]-r_edges[0])))
        for di in range(-rad_t, rad_t+1):
            for dj in range(-rad_r, rad_r+1):
                nti, nrj = ti+di, ri+dj
                if 0 <= nti < hist_s.shape[0] and 0 <= nrj < hist_s.shape[1]:
                    weights.append(hist_s[nti, nrj])
                    # coordenadas reales
                    theta_coord = theta_edges[nti] + (theta_edges[1]-theta_edges[0])/2
                    r_coord     = r_edges[nrj]     + (r_edges[1]-r_edges[0])/2
                    points.append((theta_coord, r_coord))
    if not weights:
        return None
    weights = np.array(weights)
    pts    = np.array(points)
    # centroide ponderado
    idx = weights > 0
    if not np.any(idx):
        return None
    centroid = np.average(pts[idx], axis=0, weights=weights[idx])
    return tuple(centroid)


def connect_skeletons(skeletons, hist_s, theta_edges, r_edges,
                      density_radius=0.2):
    # Conexiones internas
    internal = [(a,b) for ske in skeletons for a,b in zip(ske, ske[1:])]
    # Conexiones externas
    external = []
    density_centroids = []
    for i, ske_i in enumerate(skeletons):
        if not ske_i:
            continue
        p = ske_i[-1]
        best = None
        best_dist = math.inf
        for j, ske_j in enumerate(skeletons):
            if i == j or not ske_j:
                continue
            q = ske_j[0]
            dr_delta = q[1] - p[1]
            dtheta_delta = q[0] - p[0]
            if dr_delta>0 and dtheta_delta<0:
                dist = math.hypot(dtheta_delta, dr_delta)
                if dist < best_dist:
                    best_dist = dist; best = (p,q)
        if best:
            external.append(best)
            # calcular centroide de densidad alrededor del segmento
            dcent = compute_density_centroid(best[0], best[1], hist_s,
                                             theta_edges, r_edges,
                                             density_radius)
            if dcent:
                density_centroids.append(dcent)
    return internal, external, density_centroids


def plot_clusters_and_connections(clusters, skeletons, hist_s,
                                  theta_edges, r_edges,
                                  internal, external,
                                  density_centroids=None):
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower', extent=[theta_edges[0], theta_edges[-1],
                                                 r_edges[0], r_edges[-1]],
               aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters):
        plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size:
            plt.scatter(arr[:,0], arr[:,1], c='w', s=40,
                        edgecolors='k', linewidths=0.5)
    # conexiones internas en blanco
    for a, b in internal:
        plt.plot([a[0], b[0]], [a[1], b[1]], '-', color='white', linewidth=2)
    # conexiones externas
    for p, q in external:
        plt.plot([p[0], q[0]], [p[1], q[1]], '--', linewidth=2.5,
                 color='white', alpha=0.9)
    # centroides de densidad como puntos rojos
    if density_centroids:
        dc = np.array(density_centroids)
        plt.scatter(dc[:,0], dc[:,1], marker='o', s=50,
                    facecolors='none', edgecolors='red', linewidths=2,
                    label='Centroides de densidad')
    plt.title("Islas, conexiones y centroides de densidad")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend(markerscale=2, fontsize='small', loc='upper right')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    clusters, skeletons, hist_s, theta_edges, r_edges = main_histogram_weighted_bfs_clusters()
    internal, external, dens_centroids = connect_skeletons(skeletons,
                                                          hist_s,
                                                          theta_edges,
                                                          r_edges,
                                                          density_radius=0.2)
    plot_clusters_and_connections(clusters, skeletons, hist_s,
                                  theta_edges, r_edges,
                                  internal, external,
                                  dens_centroids)


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación ligera + BFS ponderado sobre histograma 2D + conexiones de centroides con centroides de densidad:
- Conecta islas separadas por huecos ≤ max_gap_dist
- Calcula centroides cada theta_bin_size grados
- Conecta internamente cada isla (skeleton) entre centroides adyacentes
- Conecta externamente siempre el centroide de menor θ de cada isla con el centroide de mayor θ de otra isla,
  priorizando cercanía (dist Euclidiana) y filtrando dr>0, dθ<0 y r>0 (una conexión por isla)
- Para cada conexión externa, rastrea densidad suavizada alrededor del segmento dentro de un radio = 1/4
  de la longitud del segmento y construye un centroide de densidad
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq
import math
from collections import defaultdict

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100,
    bins_r=60,
    smooth_sigma=1.0,
    density_percentile=80,
    closing_size=2,
    min_cluster_size=40,
    max_gap_dist=0.5,
    theta_bin_size=10
):
    datos      = main_example_modificado()
    grupos     = datos['grupos_finales']
    background = datos['background']

    bg_pts  = background[['theta','r']].values
    grp_pts = np.vstack([g['points'][['theta','r']].values for g in grupos]) if grupos else np.empty((0,2))
    pts = np.vstack([bg_pts, grp_pts])

    hist, theta_edges, r_edges = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)

    thr = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size,closing_size)))
    labels_grid, n_labels = label(mask_c)

    dtheta = theta_edges[1] - theta_edges[0]
    dr     = r_edges[1]     - r_edges[0]
    neigh  = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf     = UnionFind(n_labels+1)
    h, w   = labels_grid.shape
    best_dist_grid = np.full((h,w), np.inf)
    pq = []

    for i in range(h):
        for j in range(w):
            lab = labels_grid[i,j]
            if lab == 0: continue
            for di, dj in neigh:
                ni, nj = i+di, j+dj
                if 0 <= ni < h and 0 <= nj < w and labels_grid[ni,nj] == 0:
                    step = math.hypot(di*dtheta, dj*dr)
                    if step <= max_gap_dist and step < best_dist_grid[ni,nj]:
                        best_dist_grid[ni,nj] = step
                        heapq.heappush(pq, (step, ni, nj, lab))
    while pq:
        dist, i, j, lab = heapq.heappop(pq)
        if dist > best_dist_grid[i,j] or dist > max_gap_dist:
            continue
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if 0 <= ni < h and 0 <= nj < w:
                lab2 = labels_grid[ni,nj]
                if lab2 > 0 and lab2 != lab:
                    uf.union(lab, lab2)
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if not (0 <= ni < h and 0 <= nj < w) or labels_grid[ni,nj] != 0:
                continue
            nd = dist + math.hypot(di*dtheta, dj*dr)
            if nd <= max_gap_dist and nd < best_dist_grid[ni,nj]:
                best_dist_grid[ni,nj] = nd
                heapq.heappush(pq, (nd, ni, nj, lab))

    merged = defaultdict(list)
    t_idx = np.clip(np.digitize(pts[:,0],theta_edges)-1,0,bins_theta-1)
    r_idx = np.clip(np.digitize(pts[:,1],r_edges)-1,0,bins_r-1)
    for pi, (ti, ri) in enumerate(zip(t_idx, r_idx)):
        lab = labels_grid[ti, ri]
        if lab > 0:
            merged[uf.find(lab)].append(pts[pi])
    clusters = [np.vstack(v) for v in merged.values() if len(v) >= min_cluster_size]

    skeletons = []
    for c in clusters:
        thetas, rs = c[:,0], c[:,1]
        bins = np.arange(thetas.min(), thetas.max() + theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins)-1):
            mask_block = (thetas >= bins[k]) & (thetas < bins[k+1])
            if mask_block.any():
                ske.append((thetas[mask_block].mean(), rs[mask_block].mean()))
        skeletons.append(sorted(ske, key=lambda x: -x[0]))

    return clusters, skeletons, hist_s, theta_edges, r_edges


def compute_density_centroid(p, q, hist_s, theta_edges, r_edges, radius, samples=100):
    ts = np.linspace(p[0], q[0], samples)
    rs = np.linspace(p[1], q[1], samples)
    weights, points = [], []
    for theta_s, r_s in zip(ts, rs):
        ti = np.clip(np.digitize(theta_s, theta_edges)-1, 0, hist_s.shape[0]-1)
        ri = np.clip(np.digitize(r_s,     r_edges)-1,     0, hist_s.shape[1]-1)
        rad_t = int(np.ceil(radius / (theta_edges[1]-theta_edges[0])))
        rad_r = int(np.ceil(radius / (r_edges[1]-r_edges[0])))
        for di in range(-rad_t, rad_t+1):
            for dj in range(-rad_r, rad_r+1):
                nti, nrj = ti+di, ri+dj
                if 0 <= nti < hist_s.shape[0] and 0 <= nrj < hist_s.shape[1]:
                    weights.append(hist_s[nti, nrj])
                    theta_coord = theta_edges[nti] + (theta_edges[1]-theta_edges[0])/2
                    r_coord     = r_edges[nrj]     + (r_edges[1]-r_edges[0])/2
                    points.append((theta_coord, r_coord))
    if not weights:
        return None
    weights = np.array(weights)
    pts    = np.array(points)
    idx = weights > 0
    if not np.any(idx):
        return None
    return tuple(np.average(pts[idx], axis=0, weights=weights[idx]))


def connect_skeletons(skeletons, hist_s, theta_edges, r_edges, density_ratio=0.25):
    internal = [(a,b) for ske in skeletons for a,b in zip(ske, ske[1:])]
    external, density_centroids = [], []
    for i, ske_i in enumerate(skeletons):
        if not ske_i:
            continue
        p = ske_i[-1]
        best, best_dist = None, math.inf
        for j, ske_j in enumerate(skeletons):
            if i==j or not ske_j:
                continue
            q = ske_j[0]
            dr_delta = q[1]-p[1]; dtheta_delta = q[0]-p[0]
            if dr_delta>0 and dtheta_delta<0:
                dist = math.hypot(dtheta_delta, dr_delta)
                if dist<best_dist:
                    best_dist, best = dist, (p,q)
        if best:
            external.append(best)
            radius = best_dist * density_ratio
            dcent = compute_density_centroid(best[0], best[1], hist_s,
                                             theta_edges, r_edges,
                                             radius)
            if dcent:
                density_centroids.append(dcent)
    return internal, external, density_centroids


def plot_clusters_and_connections(clusters, skeletons, hist_s, theta_edges, r_edges, internal, external, density_centroids=None):
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower', extent=[theta_edges[0],theta_edges[-1],r_edges[0],r_edges[-1]], aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters): plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size: plt.scatter(arr[:,0], arr[:,1], c='w', s=40, edgecolors='k', linewidths=0.5)
    for a,b in internal: plt.plot([a[0],b[0]],[a[1],b[1]],'-',color='white',linewidth=2)
    for p,q in external: plt.plot([p[0],q[0]],[p[1],q[1]],'--',linewidth=2.5,color='white',alpha=0.9)
    if density_centroids:
        dc = np.array(density_centroids)
        plt.scatter(dc[:,0],dc[:,1],marker='o',s=50,facecolors='none',edgecolors='red',linewidths=2,label='Centroides de densidad')
    plt.title("Islas, conexiones y centroides de densidad")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend(markerscale=2,fontsize='small',loc='upper right')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    clusters, skeletons, hist_s, theta_edges, r_edges = main_histogram_weighted_bfs_clusters()
    internal, external, dens_centroids = connect_skeletons(skeletons, hist_s, theta_edges, r_edges)
    plot_clusters_and_connections(clusters, skeletons, hist_s, theta_edges, r_edges, internal, external, dens_centroids)

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación ligera + BFS ponderado sobre histograma 2D + conexiones de centroides con centroides de densidad:
- Conecta islas separadas por huecos ≤ max_gap_dist
- Calcula centroides cada theta_bin_size grados
- Conecta internamente cada isla (skeleton) entre centroides adyacentes
- Conecta externamente siempre el centroide de menor θ de cada isla con el centroide de mayor θ de otra isla,
  priorizando cercanía (dist Euclidiana) y filtrando dr>0, dθ<0 y r>0 (una conexión por isla)
- Para cada conexión externa, rastrea densidad suavizada alrededor del segmento en incrementos de 2°
  y construye un centroide de densidad, siguiendo los máximos en θ de la densidad suavizada
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq
import math
from collections import defaultdict

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100,
    bins_r=60,
    smooth_sigma=1.0,
    density_percentile=80,
    closing_size=2,
    min_cluster_size=40,
    max_gap_dist=0.5,
    theta_bin_size=2  # ahora 2° en 2°
):
    datos      = main_example_modificado()
    grupos     = datos['grupos_finales']
    background = datos['background']

    # Apilar puntos (θ, r)
    bg_pts  = background[['theta','r']].values
    grp_pts = np.vstack([g['points'][['theta','r']].values for g in grupos]) if grupos else np.empty((0,2))
    pts = np.vstack([bg_pts, grp_pts])

    # Histograma 2D + suavizado
    hist, theta_edges, r_edges = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)

    # Segmentación y etiquetado
    thr = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size,closing_size)))
    labels_grid, n_labels = label(mask_c)

    # BFS ponderado para unir componentes
    dtheta = theta_edges[1] - theta_edges[0]
    dr     = r_edges[1]     - r_edges[0]
    neigh  = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf     = UnionFind(n_labels+1)
    h, w   = labels_grid.shape
    best_dist_grid = np.full((h,w), np.inf)
    pq = []

    for i in range(h):
        for j in range(w):
            lab = labels_grid[i,j]
            if lab == 0: continue
            for di, dj in neigh:
                ni, nj = i+di, j+dj
                if 0 <= ni < h and 0 <= nj < w and labels_grid[ni,nj] == 0:
                    step = math.hypot(di*dtheta, dj*dr)
                    if step <= max_gap_dist and step < best_dist_grid[ni,nj]:
                        best_dist_grid[ni,nj] = step
                        heapq.heappush(pq, (step, ni, nj, lab))
    while pq:
        dist, i, j, lab = heapq.heappop(pq)
        if dist > best_dist_grid[i,j] or dist > max_gap_dist:
            continue
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if 0 <= ni < h and 0 <= nj < w:
                lab2 = labels_grid[ni,nj]
                if lab2 > 0 and lab2 != lab:
                    uf.union(lab, lab2)
        for di, dj in neigh:
            ni, nj = i+di, j+dj
            if not (0 <= ni < h and 0 <= nj < w) or labels_grid[ni,nj] != 0:
                continue
            nd = dist + math.hypot(di*dtheta, dj*dr)
            if nd <= max_gap_dist and nd < best_dist_grid[ni,nj]:
                best_dist_grid[ni,nj] = nd
                heapq.heappush(pq, (nd, ni, nj, lab))

    # Recolectar clusters finales
    merged = defaultdict(list)
    t_idx = np.clip(np.digitize(pts[:,0],theta_edges)-1,0,bins_theta-1)
    r_idx = np.clip(np.digitize(pts[:,1],r_edges)-1,0,bins_r-1)
    for pi, (ti, ri) in enumerate(zip(t_idx, r_idx)):
        lab = labels_grid[ti, ri]
        if lab > 0:
            merged[uf.find(lab)].append(pts[pi])
    clusters = [np.vstack(v) for v in merged.values() if len(v) >= min_cluster_size]

    # Calcular centroides (skeletons) y ordenar θ descendente
    skeletons = []
    for c in clusters:
        thetas, rs = c[:,0], c[:,1]
        bins = np.arange(thetas.min(), thetas.max() + theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins)-1):
            mask_block = (thetas >= bins[k]) & (thetas < bins[k+1])
            if mask_block.any():
                ske.append((thetas[mask_block].mean(), rs[mask_block].mean()))
        skeletons.append(sorted(ske, key=lambda x: -x[0]))

    return clusters, skeletons, hist_s, theta_edges, r_edges


def compute_density_centroid(p, q, hist_s, theta_edges, r_edges, radius):
    """
    Para un segmento p->q, muestrea cada 2° en θ, recoge vecinos
    dentro de "radius" y calcula el centroide de densidad ponderado.
    """
    theta_start, theta_end = p[0], q[0]
    step = 2 if theta_end > theta_start else -2
    ts = np.arange(theta_start, theta_end + step, step)
    # modelo lineal r(θ)
    if q[0] != p[0]:
        m = (q[1] - p[1]) / (q[0] - p[0])
        rs = p[1] + m * (ts - p[0])
    else:
        rs = np.linspace(p[1], q[1], len(ts))
    weights, points = [], []
    for theta_s, r_s in zip(ts, rs):
        ti = np.clip(np.digitize(theta_s, theta_edges)-1, 0, hist_s.shape[0]-1)
        ri = np.clip(np.digitize(r_s,     r_edges)-1,     0, hist_s.shape[1]-1)
        rad_t = int(np.ceil(radius / (theta_edges[1]-theta_edges[0])))
        rad_r = int(np.ceil(radius / (r_edges[1]-r_edges[0])))
        for di in range(-rad_t, rad_t+1):
            for dj in range(-rad_r, rad_r+1):
                nti, nrj = ti+di, ri+dj
                if 0 <= nti < hist_s.shape[0] and 0 <= nrj < hist_s.shape[1]:
                    w = hist_s[nti, nrj]
                    if w > 0:
                        weights.append(w)
                        theta_coord = theta_edges[nti] + (theta_edges[1]-theta_edges[0])/2
                        r_coord     = r_edges[nrj]     + (r_edges[1]-r_edges[0])/2
                        points.append((theta_coord, r_coord))
    if not weights:
        return None
    weights = np.array(weights)
    pts    = np.array(points)
    return tuple(np.average(pts, axis=0, weights=weights))


def connect_skeletons(skeletons, hist_s, theta_edges, r_edges, density_ratio=0.25):
    internal = [(a,b) for ske in skeletons for a,b in zip(ske, ske[1:])]
    external, density_centroids = [], []
    for i, ske_i in enumerate(skeletons):
        if not ske_i:
            continue
        p = ske_i[-1]
        best, best_dist = None, math.inf
        for j, ske_j in enumerate(skeletons):
            if i==j or not ske_j:
                continue
            q = ske_j[0]
            dr_delta = q[1]-p[1]; dtheta_delta = q[0]-p[0]
            if dr_delta>0 and dtheta_delta<0:
                dist = math.hypot(dtheta_delta, dr_delta)
                if dist<best_dist:
                    best_dist, best = dist, (p,q)
        if best:
            external.append(best)
            radius = best_dist * density_ratio
            dcent = compute_density_centroid(best[0], best[1], hist_s,
                                             theta_edges, r_edges,
                                             radius)
            if dcent:
                density_centroids.append(dcent)
    return internal, external, density_centroids


def plot_clusters_and_connections(clusters, skeletons, hist_s, theta_edges, r_edges, internal, external, density_centroids=None):
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower', extent=[theta_edges[0],theta_edges[-1],r_edges[0],r_edges[-1]], aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters): plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size: plt.scatter(arr[:,0], arr[:,1], c='w', s=40, edgecolors='k', linewidths=0.5)
    for a,b in internal: plt.plot([a[0],b[0]],[a[1],b[1]],'-',color='white',linewidth=2)
    for p,q in external: plt.plot([p[0],q[0]],[p[1],q[1]],'--',linewidth=2.5,color='white',alpha=0.9)
    if density_centroids:
        dc = np.array(density_centroids)
        plt.scatter(dc[:,0],dc[:,1],marker='o',s=50,facecolors='none',edgecolors='red',linewidths=2,label='Centroides de densidad')
    plt.title("Islas, conexiones y centroides de densidad")
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.legend(markerscale=2,fontsize='small',loc='upper right')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    clusters, skeletons, hist_s, theta_edges, r_edges = main_histogram_weighted_bfs_clusters()
    internal, external, dens_centroids = connect_skeletons(skeletons, hist_s, theta_edges, r_edges)
    plot_clusters_and_connections(clusters, skeletons, hist_s, theta_edges, r_edges, internal, external, dens_centroids)


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación ligera + BFS ponderado sobre histograma 2D + conexiones dinámicas de centroides basadas en densidad:
- Conecta islas separadas por huecos ≤ max_gap_dist
- Calcula skeletons de centroides cada theta_bin_size grados
- Conecta internamente centroides adyacentes en cada isla
- Para cada isla, conecta el extremo mínimo θ con el extremo máximo θ de la isla más cercana (dr>0, dθ<0)
- Además, genera un camino de centroides adicionales siguiendo los máximos de densidad en background
  recorriendo el segmento a pasos de theta_step grados y buscando el punto de máxima densidad radial dentro de radius
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq, math
from collections import defaultdict

class UnionFind:
    def __init__(self, n): self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100, bins_r=60, smooth_sigma=1.0,
    density_percentile=80, closing_size=2,
    min_cluster_size=40, max_gap_dist=0.5,
    theta_bin_size=10
):
    datos = main_example_modificado()
    bg = datos['background'][['theta','r']].values
    grp = np.vstack([g['points'][['theta','r']].values for g in datos['grupos_finales']]) if datos['grupos_finales'] else np.empty((0,2))
    pts = np.vstack([bg, grp])
    hist, te, re = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)
    thr = np.percentile(hist_s[hist_s > 0], density_percentile)
    mask = hist_s > thr
    mask_c = binary_closing(mask, structure=np.ones((closing_size, closing_size)))
    labels, n_lab = label(mask_c)
    dθ = te[1] - te[0]
    dr = re[1] - re[0]
    neigh = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf = UnionFind(n_lab + 1)
    h, w = labels.shape
    dist_grid = np.full((h, w), np.inf)
    pq = []
    # BFS frontera
    for i in range(h):
        for j in range(w):
            if labels[i,j] > 0:
                for di, dj in neigh:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w and labels[ni,nj] == 0:
                        d0 = math.hypot(di * dθ, dj * dr)
                        if d0 <= max_gap_dist and d0 < dist_grid[ni,nj]:
                            dist_grid[ni,nj] = d0
                            heapq.heappush(pq, (d0, ni, nj, labels[i,j]))
    while pq:
        dist, i, j, lab = heapq.heappop(pq)
        if dist > dist_grid[i,j] or dist > max_gap_dist:
            continue
        for di, dj in neigh:
            ni, nj = i + di, j + dj
            if 0 <= ni < h and 0 <= nj < w:
                lab2 = labels[ni,nj]
                if lab2 > 0 and lab2 != lab:
                    uf.union(lab, lab2)
        for di, dj in neigh:
            ni, nj = i + di, j + dj
            if 0 <= ni < h and 0 <= nj < w and labels[ni,nj] == 0:
                nd = dist + math.hypot(di * dθ, dj * dr)
                if nd <= max_gap_dist and nd < dist_grid[ni,nj]:
                    dist_grid[ni,nj] = nd
                    heapq.heappush(pq, (nd, ni, nj, lab))
    # Formar clusters
    merged = defaultdict(list)
    tidx = np.clip(np.digitize(pts[:,0], te) - 1, 0, bins_theta - 1)
    ridx = np.clip(np.digitize(pts[:,1], re) - 1, 0, bins_r - 1)
    for p, (ti, ri) in enumerate(zip(tidx, ridx)):
        lab = labels[ti, ri]
        if lab > 0:
            merged[uf.find(lab)].append(pts[p])
    clusters = [np.vstack(v) for v in merged.values() if len(v) >= min_cluster_size]
    # Skeletons
    ske_list = []
    for c in clusters:
        thetas, rs = c[:,0], c[:,1]
        bins_arr = np.arange(thetas.min(), thetas.max() + theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins_arr) - 1):
            mask_block = (thetas >= bins_arr[k]) & (thetas < bins_arr[k+1])
            if mask_block.any():
                ske.append((thetas[mask_block].mean(), rs[mask_block].mean()))
        ske_list.append(sorted(ske, key=lambda x: -x[0]))
    return clusters, ske_list, hist_s, te, re


def compute_density_path(p, q, hist_s, te, re, radius, theta_step=2):
    """
    Para cada ángulo a lo largo del segmento p->q, busca en radial máximo.
    """
    # Resoluciones
    dθ = te[1] - te[0]
    dr = re[1] - re[0]
    # División en pasos según theta_step
    total_dθ = q[0] - p[0]
    steps = max(int(abs(total_dθ) / theta_step) + 1, 2)
    thetas = np.linspace(p[0], q[0], steps)
    rs = np.linspace(p[1], q[1], steps)
    path = []
    for θ_i, r_i in zip(thetas, rs):
        ti = int(np.clip(np.digitize(θ_i, te) - 1, 0, hist_s.shape[0] - 1))
        ri = int(np.clip(np.digitize(r_i, re) - 1, 0, hist_s.shape[1] - 1))
        # Barrido radial para maxizar densidad
        best_pt = None
        best_val = -np.inf
        max_r = int(np.ceil(radius / dr))
        for dr_off in range(-max_r, max_r + 1):
            rj = ri + dr_off
            if 0 <= rj < hist_s.shape[1]:
                val = hist_s[ti, rj]
                if val > best_val:
                    best_val = val
                    # Coordenadas reales centradas
                    θc = te[ti] + dθ / 2
                    rc = re[rj] + dr / 2
                    best_pt = (θc, rc)
        if best_pt:
            path.append(best_pt)
    return path


def connect_skeletons(skeletons, hist_s, te, re, density_ratio=0.25, theta_step=2):
    internal = [(a, b) for ske in skeletons for a, b in zip(ske, ske[1:])]
    external = []
    density_paths = []
    for i, ske in enumerate(skeletons):
        if not ske:
            continue
        p = ske[-1]
        best = None
        bd = math.inf
        for j, ske2 in enumerate(skeletons):
            if i == j or not ske2:
                continue
            q = ske2[0]
            drd = q[1] - p[1]
            dtd = q[0] - p[0]
            if drd > 0 and dtd < 0:
                dist = math.hypot(dtd, drd)
                if dist < bd:
                    bd = dist
                    best = (p, q)
        if best:
            external.append(best)
            radius = bd * density_ratio
            path = compute_density_path(best[0], best[1], hist_s, te, re, radius, theta_step)
            density_paths.extend(path)
    return internal, external, density_paths


def plot_clusters_and_connections(clusters, skeletons, hist_s, te, re, internal, external, density_paths=None):
    plt.figure(figsize=(14, 6))
    plt.imshow(hist_s.T, origin='lower', extent=[te[0], te[-1], re[0], re[-1]], aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters):
        plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size:
            plt.scatter(arr[:,0], arr[:,1], c='w', s=40, edgecolors='k')
    # Conexiones internas en blanco
    for a, b in internal:
        plt.plot([a[0], b[0]], [a[1], b[1]], '-', color='white', linewidth=2)
    # Conexiones externas
    for p, q in external:
        plt.plot([p[0], q[0]], [p[1], q[1]], '--', color='white', linewidth=2.5)
    # Camino de densidad en cian
    if density_paths:
        dp = np.array(density_paths)
        plt.scatter(dp[:,0], dp[:,1], c='cyan', s=30, marker='x', label='Camino densidad')
    plt.title('Islas, conexiones y camino de densidad')
    plt.xlabel('θ (°)')
    plt.ylabel('r')
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    clusters, skeletons, hist_s, te, re = main_histogram_weighted_bfs_clusters()
    internal, external, path = connect_skeletons(skeletons, hist_s, te, re)
    plot_clusters_and_connections(clusters, skeletons, hist_s, te, re, internal, external, path)


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación ligera + BFS ponderado sobre histograma 2D + conexiones dinámicas de centroides basadas en densidad:
- Conecta islas separadas por huecos ≤ max_gap_dist
- Calcula skeletons de centroides cada theta_bin_size grados
- Conecta internamente centroides adyacentes en cada isla
- Para cada isla, conecta el extremo mínimo θ con el extremo máximo θ de la isla más cercana (dr>0, dθ<0)
- Además, genera un camino de centroides adicionales siguiendo los máximos de densidad en background
  recorriendo el segmento a pasos de theta_step grados y buscando el punto de máxima densidad radial dentro de radius
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq, math
from collections import defaultdict

class UnionFind:
    def __init__(self, n): self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb: self.parent[rb] = ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100, bins_r=60, smooth_sigma=1.0,
    density_percentile=80, closing_size=2,
    min_cluster_size=40, max_gap_dist=0.5,
    theta_bin_size=10
):
    datos = main_example_modificado()
    bg = datos['background'][['theta','r']].values
    grp = np.vstack([g['points'][['theta','r']].values for g in datos['grupos_finales']]) if datos['grupos_finales'] else np.empty((0,2))
    pts = np.vstack([bg, grp])
    hist, te, re = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)
    thr = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size, closing_size)))
    labels, n_lab = label(mask_c)

    theta_res = te[1] - te[0]
    r_res     = re[1] - re[0]
    neigh     = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf        = UnionFind(n_lab+1)
    h, w      = labels.shape
    dist_grid = np.full((h, w), np.inf)
    pq        = []

    # BFS frontera
    for i in range(h):
        for j in range(w):
            if labels[i,j] > 0:
                for di, dj in neigh:
                    ni, nj = i+di, j+dj
                    if 0<=ni<h and 0<=nj<w and labels[ni,nj]==0:
                        d0 = math.hypot(di*theta_res, dj*r_res)
                        if d0<=max_gap_dist and d0<dist_grid[ni,nj]:
                            dist_grid[ni,nj] = d0
                            heapq.heappush(pq, (d0, ni, nj, labels[i,j]))
    while pq:
        dist,i,j,lab = heapq.heappop(pq)
        if dist>dist_grid[i,j] or dist>max_gap_dist: continue
        for di,dj in neigh:
            ni,nj = i+di, j+dj
            if 0<=ni<h and 0<=nj<w:
                lab2 = labels[ni,nj]
                if lab2>0 and lab2!=lab: uf.union(lab, lab2)
        for di,dj in neigh:
            ni,nj = i+di, j+dj
            if 0<=ni<h and 0<=nj<w and labels[ni,nj]==0:
                nd = dist + math.hypot(di*theta_res, dj*r_res)
                if nd<=max_gap_dist and nd<dist_grid[ni,nj]:
                    dist_grid[ni,nj] = nd
                    heapq.heappush(pq, (nd, ni, nj, lab))

    # Formar clusters
    merged = defaultdict(list)
    tidx = np.clip(np.digitize(pts[:,0], te)-1, 0, bins_theta-1)
    ridx = np.clip(np.digitize(pts[:,1], re)-1, 0, bins_r-1)
    for p, (ti, ri) in enumerate(zip(tidx, ridx)):
        lab = labels[ti, ri]
        if lab>0: merged[uf.find(lab)].append(pts[p])
    clusters = [np.vstack(v) for v in merged.values() if len(v)>=min_cluster_size]

    # Skeletons
    ske_list = []
    for c in clusters:
        thetas, rs = c[:,0], c[:,1]
        bins_arr = np.arange(thetas.min(), thetas.max()+theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins_arr)-1):
            mask_block = (thetas>=bins_arr[k]) & (thetas<bins_arr[k+1])
            if mask_block.any():
                ske.append((thetas[mask_block].mean(), rs[mask_block].mean()))
        ske_list.append(sorted(ske, key=lambda x:-x[0]))
    return clusters, ske_list, hist_s, te, re


def compute_density_path(p, q, hist_s, te, re, radius, theta_step=2):
    """
    Para cada paso en θ a lo largo de p->q, busca radialmente dentro de radius
    el punto de máxima densidad en hist_s.
    """
    theta_res = te[1] - te[0]
    r_res     = re[1] - re[0]
    total_dθ  = q[0] - p[0]
    steps = max(int(abs(total_dθ)/theta_step)+1, 2)
    thetas = np.linspace(p[0], q[0], steps)
    rs_vals = np.linspace(p[1], q[1], steps)
    path = []
    for θ_i, r_i in zip(thetas, rs_vals):
        ti = int(np.clip(np.digitize(θ_i, te)-1, 0, hist_s.shape[0]-1))
        ri = int(np.clip(np.digitize(r_i, re)-1, 0, hist_s.shape[1]-1))
        best_pt = None
        best_val = -np.inf
        max_off = int(np.ceil(radius / r_res))
        for off in range(-max_off, max_off+1):
            idx_r = ri + off
            if 0<=idx_r<hist_s.shape[1]:
                val = hist_s[ti, idx_r]
                if val > best_val:
                    best_val = val
                    θc = te[ti] + theta_res/2
                    rc = re[idx_r] + r_res/2
                    best_pt = (θc, rc)
        if best_pt:
            path.append(best_pt)
    return path


def connect_skeletons(skeletons, hist_s, te, re, density_ratio=0.25, theta_step=2):
    internal = [(a, b) for ske in skeletons for a, b in zip(ske, ske[1:])]
    external = []
    density_paths = []
    for i, ske in enumerate(skeletons):
        if not ske: continue
        p = ske[-1]
        best, bd = None, math.inf
        for j, ske2 in enumerate(skeletons):
            if i==j or not ske2: continue
            q = ske2[0]
            drd = q[1] - p[1]
            dtd = q[0] - p[0]
            if drd>0 and dtd<0:
                dist = math.hypot(dtd, drd)
                if dist<bd: bd, best = dist, (p,q)
        if best:
            external.append(best)
            radius = bd * density_ratio
            density_paths.extend(compute_density_path(best[0], best[1], hist_s, te, re, radius, theta_step))
    return internal, external, density_paths


def plot_clusters_and_connections(clusters, skeletons, hist_s, te, re, internal, external, density_paths=None):
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower', extent=[te[0], te[-1], re[0], re[-1]], aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters):
        plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size:
            plt.scatter(arr[:,0], arr[:,1], c='w', s=40, edgecolors='k')
    for a, b in internal:
        plt.plot([a[0], b[0]], [a[1], b[1]], '-', color='white', linewidth=2)
    for p, q in external:
        plt.plot([p[0], q[0]], [p[1], q[1]], '--', color='white', linewidth=2.5)
    if density_paths:
        dp = np.array(density_paths)
        plt.scatter(dp[:,0], dp[:,1], c='cyan', s=30, marker='x', label='Camino densidad')
    plt.title('Islas, conexiones y camino de densidad')
    plt.xlabel('θ (°)'); plt.ylabel('r (units)')
    plt.legend(loc='upper right'); plt.tight_layout(); plt.show()

if __name__=='__main__':
    clusters, skeletons, hist_s, te, re = main_histogram_weighted_bfs_clusters()
    internal, external, path = connect_skeletons(skeletons, hist_s, te, re)
    plot_clusters_and_connections(clusters, skeletons, hist_s, te, re, internal, external, path)


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación ligera + BFS ponderado sobre histograma 2D + conexiones dinámicas de centroides basadas en densidad:
- Conecta islas separadas por huecos ≤ max_gap_dist
- Calcula skeletons de centroides cada theta_bin_size grados
- Conecta internamente centroides adyacentes en cada isla
- Para cada isla, conecta el extremo mínimo θ con el extremo máximo θ de la isla más cercana (dr>0, dθ<0)
- Además, genera un camino de centroides adicionales siguiendo los máximos de densidad en background
  recorriendo el segmento a pasos de theta_step grados y buscando el punto de máxima densidad radial dentro de radius
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq, math
from collections import defaultdict

class UnionFind:
    def __init__(self, n): self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb: self.parent[rb] = ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100, bins_r=60, smooth_sigma=1.0,
    density_percentile=80, closing_size=2,
    min_cluster_size=40, max_gap_dist=0.5,
    theta_bin_size=8
):
    datos = main_example_modificado()
    bg = datos['background'][['theta','r']].values
    grp = np.vstack([g['points'][['theta','r']].values for g in datos['grupos_finales']]) if datos['grupos_finales'] else np.empty((0,2))
    pts = np.vstack([bg, grp])
    hist, te, re = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s = gaussian_filter(hist, sigma=smooth_sigma)
    thr = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size, closing_size)))
    labels, n_lab = label(mask_c)

    theta_res = te[1] - te[0]
    r_res     = re[1] - re[0]
    neigh     = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf        = UnionFind(n_lab+1)
    h, w      = labels.shape
    dist_grid = np.full((h, w), np.inf)
    pq        = []

    # BFS frontera
    for i in range(h):
        for j in range(w):
            if labels[i,j] > 0:
                for di, dj in neigh:
                    ni, nj = i+di, j+dj
                    if 0<=ni<h and 0<=nj<w and labels[ni,nj]==0:
                        d0 = math.hypot(di*theta_res, dj*r_res)
                        if d0<=max_gap_dist and d0<dist_grid[ni,nj]:
                            dist_grid[ni,nj] = d0
                            heapq.heappush(pq, (d0, ni, nj, labels[i,j]))
    while pq:
        dist,i,j,lab = heapq.heappop(pq)
        if dist>dist_grid[i,j] or dist>max_gap_dist: continue
        for di,dj in neigh:
            ni,nj = i+di, j+dj
            if 0<=ni<h and 0<=nj<w:
                lab2 = labels[ni,nj]
                if lab2>0 and lab2!=lab: uf.union(lab, lab2)
        for di,dj in neigh:
            ni,nj = i+di, j+dj
            if 0<=ni<h and 0<=nj<w and labels[ni,nj]==0:
                nd = dist + math.hypot(di*theta_res, dj*r_res)
                if nd<=max_gap_dist and nd<dist_grid[ni,nj]:
                    dist_grid[ni,nj] = nd
                    heapq.heappush(pq, (nd, ni, nj, lab))

    # Formar clusters
    merged = defaultdict(list)
    tidx = np.clip(np.digitize(pts[:,0], te)-1, 0, bins_theta-1)
    ridx = np.clip(np.digitize(pts[:,1], re)-1, 0, bins_r-1)
    for p, (ti, ri) in enumerate(zip(tidx, ridx)):
        lab = labels[ti, ri]
        if lab>0: merged[uf.find(lab)].append(pts[p])
    clusters = [np.vstack(v) for v in merged.values() if len(v)>=min_cluster_size]

    # Skeletons
    ske_list = []
    for c in clusters:
        thetas, rs = c[:,0], c[:,1]
        bins_arr = np.arange(thetas.min(), thetas.max()+theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins_arr)-1):
            mask_block = (thetas>=bins_arr[k]) & (thetas<bins_arr[k+1])
            if mask_block.any():
                ske.append((thetas[mask_block].mean(), rs[mask_block].mean()))
        ske_list.append(sorted(ske, key=lambda x:-x[0]))
    return clusters, ske_list, hist_s, te, re


def compute_density_path(p, q, hist_s, te, re, radius, theta_step=2):
    """
    Para cada paso en θ a lo largo de p->q:
    - samplea radios alrededor del punto en [−radius,+radius]
    - identifica todos los máximos locales en el perfil radial (val > vecinos inmediatos)
      con densidad > 0 como centroides débiles
    - construye coordenadas reales de cada máximo detectado
    """
    theta_res = te[1] - te[0]
    r_res     = re[1] - re[0]
    total_dθ  = q[0] - p[0]
    steps     = max(int(abs(total_dθ)/theta_step) + 1, 2)
    thetas    = np.linspace(p[0], q[0], steps)
    rs_vals   = np.linspace(p[1], q[1], steps)
    path = []
    for θ_i, r_i in zip(thetas, rs_vals):
        # índice central
        ti = int(np.clip(np.digitize(θ_i, te) - 1, 0, hist_s.shape[0] - 1))
        ri0 = int(np.clip(np.digitize(r_i, re) - 1, 0, hist_s.shape[1] - 1))
        max_off = int(np.ceil(radius / r_res))
        # perfil radial
        vals = []
        offs = []
        for off in range(-max_off, max_off + 1):
            idx_r = ri0 + off
            if 0 <= idx_r < hist_s.shape[1]:
                vals.append(hist_s[ti, idx_r])
                offs.append(off)
        # detectar máximos locales sobre perfil vals
        for k in range(1, len(vals)-1):
            if vals[k] > vals[k-1] and vals[k] > vals[k+1] and vals[k] > 0:
                off = offs[k]
                θc = te[ti] + theta_res / 2
                rc = (re[ri0 + off] + r_res/2) if 0 <= ri0+off < len(re) else r_i
                path.append((θc, rc))
    return path


def connect_skeletons(skeletons, hist_s, te, re, density_ratio=0.25, theta_step=5):
    internal = [(a, b) for ske in skeletons for a, b in zip(ske, ske[1:])]
    external = []
    density_paths = []
    for i, ske in enumerate(skeletons):
        if not ske: continue
        p = ske[-1]
        best, bd = None, math.inf
        for j, ske2 in enumerate(skeletons):
            if i==j or not ske2: continue
            q = ske2[0]
            drd = q[1] - p[1]
            dtd = q[0] - p[0]
            if drd>0 and dtd<0:
                dist = math.hypot(dtd, drd)
                if dist<bd: bd, best = dist, (p,q)
        if best:
            external.append(best)
            radius = bd * density_ratio
            density_paths.extend(compute_density_path(best[0], best[1], hist_s, te, re, radius, theta_step))
    return internal, external, density_paths


def plot_clusters_and_connections(clusters, skeletons, hist_s, te, re, internal, external, density_paths=None):
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower', extent=[te[0], te[-1], re[0], re[-1]], aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters):
        plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size:
            plt.scatter(arr[:,0], arr[:,1], c='w', s=20, edgecolors='k')
    for a, b in internal:
        plt.plot([a[0], b[0]], [a[1], b[1]], '-', color='white', linewidth=2)
    for p, q in external:
        plt.plot([p[0], q[0]], [p[1], q[1]], '--', color='white', linewidth=2.5)
    if density_paths:
        dp = np.array(density_paths)
        plt.scatter(dp[:,0], dp[:,1], c='red', s=10, marker='o', edgecolors='k' ,label='Camino densidad')
    plt.title('Islas, conexiones y camino de densidad')
    plt.xlabel('θ (°)'); plt.ylabel('r (units)')
    plt.legend(loc='upper right'); plt.tight_layout(); plt.show()

if __name__=='__main__':
    clusters, skeletons, hist_s, te, re = main_histogram_weighted_bfs_clusters()
    internal, external, path = connect_skeletons(skeletons, hist_s, te, re)
    plot_clusters_and_connections(clusters, skeletons, hist_s, te, re, internal, external, path)


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación + BFS + conexiones dinámicas + contornos filtrados:
- Detecta islas en histograma 2D suavizado
- Calcula skeletons de centroides cada theta_bin_size grados
- Conecta internamente centroides adyacentes
- Para cada isla, conecta el extremo mínimo θ con el extremo máximo θ de la isla más cercana (dr>0, dθ<0)
- Genera puntos adicionales por máximos locales de densidad en background a lo largo del segmento
- Solo retiene esos puntos si |r_c - r_segmento| ≤ r_threshold (0.9)
- Construye contornos cerrados con esos centroides y los dibuja en un segundo plot
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq, math
from collections import defaultdict

class UnionFind:
    def __init__(self,n): self.parent=list(range(n))
    def find(self,x):
        while self.parent[x]!=x:
            self.parent[x]=self.parent[self.parent[x]]
            x=self.parent[x]
        return x
    def union(self,a,b):
        ra,rb=self.find(a),self.find(b)
        if ra!=rb: self.parent[rb]=ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100, bins_r=60, smooth_sigma=1.0,
    density_percentile=80, closing_size=2,
    min_cluster_size=40, max_gap_dist=0.5,
    theta_bin_size=10
):
    datos = main_example_modificado()
    bg    = datos['background'][['theta','r']].values
    grp   = np.vstack([g['points'][['theta','r']].values for g in datos['grupos_finales']])
    pts   = np.vstack([bg, grp])

    # Histograma 2D + suavizado
    hist, te, re = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s       = gaussian_filter(hist, sigma=smooth_sigma)

    # Segmentación y etiquetado
    thr    = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size,closing_size)))
    labels, n_lab = label(mask_c)

    theta_res = te[1] - te[0]
    r_res     = re[1] - re[0]
    neigh     = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf        = UnionFind(n_lab+1)
    h, w      = labels.shape
    dist_grid = np.full((h,w), np.inf)
    pq        = []

    # BFS ponderado en frontera
    for i in range(h):
        for j in range(w):
            if labels[i,j] > 0:
                for di,dj in neigh:
                    ni,nj = i+di, j+dj
                    if 0 <= ni < h and 0 <= nj < w and labels[ni,nj] == 0:
                        d0 = math.hypot(di*theta_res, dj*r_res)
                        if d0 <= max_gap_dist and d0 < dist_grid[ni,nj]:
                            dist_grid[ni,nj] = d0
                            heapq.heappush(pq, (d0, ni, nj, labels[i,j]))
    while pq:
        dist,i,j,lab = heapq.heappop(pq)
        if dist > dist_grid[i,j] or dist > max_gap_dist:
            continue
        for di,dj in neigh:
            ni,nj = i+di, j+dj
            if 0 <= ni < h and 0 <= nj < w:
                lab2 = labels[ni,nj]
                if lab2 > 0 and lab2 != lab:
                    uf.union(lab, lab2)
        for di,dj in neigh:
            ni,nj = i+di, j+dj
            if 0 <= ni < h and 0 <= nj < w and labels[ni,nj] == 0:
                nd = dist + math.hypot(di*theta_res, dj*r_res)
                if nd <= max_gap_dist and nd < dist_grid[ni,nj]:
                    dist_grid[ni,nj] = nd
                    heapq.heappush(pq, (nd, ni, nj, lab))

    # Recolectar clusters
    merged = defaultdict(list)
    tidx   = np.clip(np.digitize(pts[:,0], te) - 1, 0, bins_theta-1)
    ridx   = np.clip(np.digitize(pts[:,1], re) - 1, 0, bins_r-1)
    for p, (ti, ri) in enumerate(zip(tidx, ridx)):
        lab = labels[ti, ri]
        if lab > 0:
            merged[uf.find(lab)].append(pts[p])
    clusters = [np.vstack(v) for v in merged.values() if len(v) >= min_cluster_size]

    # Skeletons
    skeletons = []
    for c in clusters:
        thetas, rs = c[:,0], c[:,1]
        bins_arr    = np.arange(thetas.min(), thetas.max() + theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins_arr)-1):
            m = (thetas >= bins_arr[k]) & (thetas < bins_arr[k+1])
            if m.any():
                ske.append((thetas[m].mean(), rs[m].mean()))
        skeletons.append(sorted(ske, key=lambda x: -x[0]))

    return clusters, skeletons, hist_s, te, re


def compute_density_path(p, q, hist_s, te, re, radius,
                         theta_step=2, r_threshold=0.9):
    """
    Muestrea el segmento p->q en θ cada theta_step,
    para cada muestra explora radialmente dentro de ±radius,
    detecta máximos locales y sólo retiene aquellos con
    |r_c - r_segmento| ≤ r_threshold.
    """
    theta_res = te[1] - te[0]
    r_res     = re[1] - re[0]
    total_dθ  = q[0] - p[0]
    steps     = max(int(abs(total_dθ)/theta_step) + 1, 2)
    thetas    = np.linspace(p[0], q[0], steps)
    rs_vals   = np.linspace(p[1], q[1], steps)
    path = []
    for θ_i, r_i in zip(thetas, rs_vals):
        ti  = int(np.clip(np.digitize(θ_i, te) - 1, 0, hist_s.shape[0] - 1))
        ri0 = int(np.clip(np.digitize(r_i, re) - 1, 0, hist_s.shape[1] - 1))
        max_off = int(np.ceil(radius / r_res))
        vals, offs = [], []
        for off in range(-max_off, max_off + 1):
            idx_r = ri0 + off
            if 0 <= idx_r < hist_s.shape[1]:
                vals.append(hist_s[ti, idx_r])
                offs.append(off)
        for k in range(1, len(vals)-1):
            if vals[k] > vals[k-1] and vals[k] > vals[k+1] and vals[k] > 0:
                off = offs[k]
                θc  = te[ti] + theta_res/2
                rc  = (re[ri0+off] + r_res/2) if 0 <= ri0+off < len(re) else r_i
                if abs(rc - r_i) <= r_threshold:
                    path.append((θc, rc))
    return path


def connect_skeletons(skeletons, hist_s, te, re,
                      density_ratio=0.25, theta_step=2, r_threshold=0.9):
    internal = []
    contours = []
    # internas
    for ske in skeletons:
        internal += list(zip(ske, ske[1:]))
    # contornos cerrados
    for i, ske in enumerate(skeletons):
        if not ske:
            continue
        p = ske[-1]
        best, bd = None, math.inf
        for j, ske2 in enumerate(skeletons):
            if i == j or not ske2:
                continue
            q = ske2[0]
            drd, dtd = q[1] - p[1], q[0] - p[0]
            if drd > 0 and dtd < 0:
                d = math.hypot(dtd, drd)
                if d < bd:
                    bd, best = d, (p, q)
        if best:
            radius = bd * density_ratio
            path   = compute_density_path(best[0], best[1], hist_s, te, re,
                                           radius, theta_step, r_threshold)
            contour = [best[0]] + path + [best[1]]
            contours.append(contour)
    return internal, contours


def plot_clusters_and_connections(
    clusters, skeletons, hist_s, te, re,
    internal, contours
):
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower', extent=[te[0],te[-1],re[0],re[-1]], aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters):
        plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size:
            plt.scatter(arr[:,0], arr[:,1], c='w', s=40, edgecolors='k')
    for a, b in internal:
        plt.plot([a[0], b[0]], [a[1], b[1]], '-', color='white', linewidth=2)
    for cnt in contours:
        xs, ys = zip(*cnt)
        plt.plot(xs, ys, '-o', color='red', markersize=4, linewidth=2, alpha=0.8)
    plt.title('Islas, conexiones y contornos filtrados')
    plt.xlabel('θ (°)'); plt.ylabel('r (units)')
    plt.legend(loc='upper right'); plt.tight_layout(); plt.show()

if __name__ == '__main__':
    clusters, skeletons, hist_s, te, re = main_histogram_weighted_bfs_clusters()
    internal, contours = connect_skeletons(skeletons, hist_s, te, re)
    plot_clusters_and_connections(clusters, skeletons, hist_s, te, re, internal, contours)
    bg    = datos['background'][['theta','r']].values
    grp   = np.vstack([g['points'][['theta','r']].values for g in datos['grupos_finales']])
    pts   = np.vstack([bg, grp])

    hist, te, re = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s       = gaussian_filter(hist, sigma=smooth_sigma)

    thr    = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size,closing_size)))
    labels, n_lab = label(mask_c)

    theta_res = te[1]-te[0]
    r_res     = re[1]-re[0]
    neigh     = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf        = UnionFind(n_lab+1)
    h, w      = labels.shape
    dist_grid = np.full((h,w), np.inf)
    pq        = []

    # BFS ponderado en frontera
    for i in range(h):
        for j in range(w):
            if labels[i,j]>0:
                for di,dj in neigh:
                    ni,nj = i+di, j+dj
                    if 0<=ni<h and 0<=nj<w and labels[ni,nj]==0:
                        d0 = math.hypot(di*theta_res, dj*r_res)
                        if d0<=max_gap_dist and d0<dist_grid[ni,nj]:
                            dist_grid[ni,nj]=d0
                            heapq.heappush(pq,(d0,ni,nj,labels[i,j]))
    while pq:
        dist,i,j,lab=heapq.heappop(pq)
        if dist>dist_grid[i,j] or dist>max_gap_dist: continue
        for di,dj in neigh:
            ni,nj=i+di, j+dj
            if 0<=ni<h and 0<=nj<w:
                lab2 = labels[ni,nj]
                if lab2>0 and lab2!=lab: uf.union(lab, lab2)
        for di,dj in neigh:
            ni,nj=i+di, j+dj
            if 0<=ni<h and 0<=nj<w and labels[ni,nj]==0:
                nd = dist + math.hypot(di*theta_res, dj*r_res)
                if nd<=max_gap_dist and nd<dist_grid[ni,nj]:
                    dist_grid[ni,nj]=nd
                    heapq.heappush(pq,(nd,ni,nj,lab))

    # Recolectar clusters
    merged=defaultdict(list)
    tidx=np.clip(np.digitize(pts[:,0],te)-1,0,bins_theta-1)
    ridx=np.clip(np.digitize(pts[:,1],re)-1,0,bins_r-1)
    for p,(ti,ri) in enumerate(zip(tidx,ridx)):
        lab=labels[ti,ri]
        if lab>0: merged[uf.find(lab)].append(pts[p])
    clusters=[np.vstack(v) for v in merged.values() if len(v)>=min_cluster_size]

    # Skeletons
    skeletons=[]
    for c in clusters:
        thetas,rs=c[:,0],c[:,1]
        bins_arr=np.arange(thetas.min(), thetas.max()+theta_bin_size, theta_bin_size)
        ske=[]
        for k in range(len(bins_arr)-1):
            m=(thetas>=bins_arr[k])&(thetas<bins_arr[k+1])
            if m.any(): ske.append((thetas[m].mean(), rs[m].mean()))
        skeletons.append(sorted(ske, key=lambda x:-x[0]))

        return clusters, skeletons, hist_s, te, re


def compute_density_path(p, q, hist_s, te, re, radius,
                         theta_step=2, r_threshold=0.9):
    """
    Muestrea el segmento p->q en θ cada theta_step,
    para cada muestra explora radialmente dentro de ±radius,
    detecta máximos locales y sólo retiene aquellos con
    |r_c - r_segmento| ≤ r_threshold.
    """
    theta_res=te[1]-te[0]
    r_res    =re[1]-re[0]
    total_dθ =q[0]-p[0]
    steps    =max(int(abs(total_dθ)/theta_step)+1,2)
    thetas   =np.linspace(p[0], q[0], steps)
    rs_vals  =np.linspace(p[1], q[1], steps)
    path=[]
    for θ_i, r_i in zip(thetas, rs_vals):
        ti  =int(np.clip(np.digitize(θ_i,te)-1,0,hist_s.shape[0]-1))
        ri0 =int(np.clip(np.digitize(r_i,re)-1,0,hist_s.shape[1]-1))
        max_off=int(np.ceil(radius/r_res))
        vals, offs = [], []
        for off in range(-max_off, max_off+1):
            idx_r=ri0+off
            if 0<=idx_r<hist_s.shape[1]:
                vals.append(hist_s[ti, idx_r]); offs.append(off)
        for k in range(1,len(vals)-1):
            if vals[k]>vals[k-1] and vals[k]>vals[k+1] and vals[k]>0:
                off=offs[k]
                θc = te[ti] + theta_res/2
                rc = (re[ri0+off] + r_res/2) if 0<=ri0+off<len(re) else r_i
                if abs(rc - r_i) <= r_threshold:
                    path.append((θc, rc))
    return path


def connect_skeletons(skeletons, hist_s, te, re,
                      density_ratio=0.25, theta_step=2, r_threshold=0.9):
    internal=[]; contours=[]
    # internas
    for ske in skeletons:
        internal += list(zip(ske, ske[1:]))
    # contornos cerrados
    for i, ske in enumerate(skeletons):
        if not ske: continue
        p=ske[-1]; best, bd=None, math.inf
        for j, ske2 in enumerate(skeletons):
            if i==j or not ske2: continue
            q=ske2[0]
            drd, dtd = q[1]-p[1], q[0]-p[0]
            if drd>0 and dtd<0:
                d=math.hypot(dtd,drd)
                if d<bd: bd, best=d, (p,q)
        if best:
            radius=bd*density_ratio
            path=compute_density_path(best[0],best[1],hist_s,te,re,
                                      radius, theta_step, r_threshold)
            contour=[best[0]] + path + [best[1]]
            contours.append(contour)
    return internal, contours


def plot_clusters_and_connections(
    clusters, skeletons, hist_s, te, re,
    internal, contours
):
    # gráfico principal
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower',
               extent=[te[0],te[-1],re[0],re[-1]],
               aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters):
        plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6,
                    label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size:
            plt.scatter(arr[:,0], arr[:,1], c='w', s=40,
                        edgecolors='k')
    for a, b in internal:
        plt.plot([a[0],b[0]],[a[1],b[1]], '-',
                 color='white', linewidth=2)
    for cnt in contours:
        xs, ys = zip(*cnt)
        plt.plot(xs, ys, '-o', color='red',
                 markersize=4, linewidth=2, alpha=0.8)
    plt.title('Islas, conexiones y contornos filtrados')
    plt.xlabel('θ (°)'); plt.ylabel('r');
    plt.legend(loc='upper right'); plt.tight_layout();
    plt.show()

if __name__=='__main__':
    clusters, skeletons, hist_s, te, re = \
        main_histogram_weighted_bfs_clusters()
    internal, contours = \
        connect_skeletons(skeletons, hist_s, te, re)
    plot_clusters_and_connections(
        clusters, skeletons,
        hist_s, te, re,
        internal, contours
    )



### version caminos ok 

In [0]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación + BFS + conexiones dinámicas + contornos filtrados:
- Detecta islas en histograma 2D suavizado
- Calcula skeletons de centroides cada theta_bin_size grados
- Conecta internamente centroides adyacentes
- Para cada isla, conecta el extremo mínimo θ con el extremo máximo θ de la isla más cercana (dr>0, dθ<0)
- Genera puntos adicionales por máximos locales de densidad en background a lo largo del segmento
- Solo retiene esos puntos si |r_c - r_segmento| ≤ r_threshold (0.9)
- Construye contornos cerrados con esos centroides y los dibuja en un segundo plot
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq, math
from collections import defaultdict

class UnionFind:
    def __init__(self,n): self.parent=list(range(n))
    def find(self,x):
        while self.parent[x]!=x:
            self.parent[x]=self.parent[self.parent[x]]
            x=self.parent[x]
        return x
    def union(self,a,b):
        ra,rb=self.find(a),self.find(b)
        if ra!=rb: self.parent[rb]=ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100, bins_r=60, smooth_sigma=1.0,
    density_percentile=80, closing_size=2,
    min_cluster_size=40, max_gap_dist=0.5,
    theta_bin_size=10
):
    datos = main_example_modificado()
    bg    = datos['background'][['theta','r']].values
    grp   = np.vstack([g['points'][['theta','r']].values for g in datos['grupos_finales']])
    pts   = np.vstack([bg, grp])

    hist, te, re = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s       = gaussian_filter(hist, sigma=smooth_sigma)

    thr    = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size,closing_size)))
    labels, n_lab = label(mask_c)

    theta_res = te[1]-te[0]
    r_res     = re[1]-re[0]
    neigh     = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf        = UnionFind(n_lab+1)
    h, w      = labels.shape
    dist_grid = np.full((h,w), np.inf)
    pq        = []

    # BFS ponderado en frontera
    for i in range(h):
        for j in range(w):
            if labels[i,j]>0:
                for di,dj in neigh:
                    ni,nj = i+di, j+dj
                    if 0<=ni<h and 0<=nj<w and labels[ni,nj]==0:
                        d0 = math.hypot(di*theta_res, dj*r_res)
                        if d0<=max_gap_dist and d0<dist_grid[ni,nj]:
                            dist_grid[ni,nj]=d0
                            heapq.heappush(pq,(d0,ni,nj,labels[i,j]))
    while pq:
        dist,i,j,lab=heapq.heappop(pq)
        if dist>dist_grid[i,j] or dist>max_gap_dist: continue
        for di,dj in neigh:
            ni,nj=i+di, j+dj
            if 0<=ni<h and 0<=nj<w:
                lab2 = labels[ni,nj]
                if lab2>0 and lab2!=lab: uf.union(lab, lab2)
        for di,dj in neigh:
            ni,nj=i+di, j+dj
            if 0<=ni<h and 0<=nj<w and labels[ni,nj]==0:
                nd = dist + math.hypot(di*theta_res, dj*r_res)
                if nd<=max_gap_dist and nd<dist_grid[ni,nj]:
                    dist_grid[ni,nj]=nd
                    heapq.heappush(pq,(nd,ni,nj,lab))

    # Recolectar clusters
    merged=defaultdict(list)
    tidx=np.clip(np.digitize(pts[:,0],te)-1,0,bins_theta-1)
    ridx=np.clip(np.digitize(pts[:,1],re)-1,0,bins_r-1)
    for p,(ti,ri) in enumerate(zip(tidx,ridx)):
        lab=labels[ti,ri]
        if lab>0: merged[uf.find(lab)].append(pts[p])
    clusters=[np.vstack(v) for v in merged.values() if len(v)>=min_cluster_size]

    # Skeletons
    skeletons=[]
    for c in clusters:
        thetas,rs=c[:,0],c[:,1]
        bins_arr=np.arange(thetas.min(), thetas.max()+theta_bin_size, theta_bin_size)
        ske=[]
        for k in range(len(bins_arr)-1):
            m=(thetas>=bins_arr[k])&(thetas<bins_arr[k+1])
            if m.any(): ske.append((thetas[m].mean(), rs[m].mean()))
        skeletons.append(sorted(ske, key=lambda x:-x[0]))

    return clusters, skeletons, hist_s, te, re


def compute_density_path(p, q, hist_s, te, re, radius,
                         theta_step=2, r_threshold=0.9):
    """
    Muestrea el segmento p->q en θ cada theta_step,
    para cada muestra explora radialmente dentro de ±radius,
    detecta máximos locales y sólo retiene aquellos con
    |r_c - r_segmento| ≤ r_threshold.
    """
    theta_res=te[1]-te[0]
    r_res    =re[1]-re[0]
    total_dθ =q[0]-p[0]
    steps    =max(int(abs(total_dθ)/theta_step)+1,2)
    thetas   =np.linspace(p[0], q[0], steps)
    rs_vals  =np.linspace(p[1], q[1], steps)
    path=[]
    for θ_i, r_i in zip(thetas, rs_vals):
        ti  =int(np.clip(np.digitize(θ_i,te)-1,0,hist_s.shape[0]-1))
        ri0 =int(np.clip(np.digitize(r_i,re)-1,0,hist_s.shape[1]-1))
        max_off=int(np.ceil(radius/r_res))
        vals, offs = [], []
        for off in range(-max_off, max_off+1):
            idx_r=ri0+off
            if 0<=idx_r<hist_s.shape[1]:
                vals.append(hist_s[ti, idx_r]); offs.append(off)
        for k in range(1,len(vals)-1):
            if vals[k]>vals[k-1] and vals[k]>vals[k+1] and vals[k]>0:
                off=offs[k]
                θc = te[ti] + theta_res/2
                rc = (re[ri0+off] + r_res/2) if 0<=ri0+off<len(re) else r_i
                if abs(rc - r_i) <= r_threshold:
                    path.append((θc, rc))
    return path


def connect_skeletons(skeletons, hist_s, te, re,
                      density_ratio=0.25, theta_step=2, r_threshold=0.9):
    internal=[]; contours=[]
    # internas
    for ske in skeletons:
        internal += list(zip(ske, ske[1:]))
    # contornos cerrados
    for i, ske in enumerate(skeletons):
        if not ske: continue
        p=ske[-1]; best, bd=None, math.inf
        for j, ske2 in enumerate(skeletons):
            if i==j or not ske2: continue
            q=ske2[0]
            drd, dtd = q[1]-p[1], q[0]-p[0]
            if drd>0 and dtd<0:
                d=math.hypot(dtd,drd)
                if d<bd: bd, best=d, (p,q)
        if best:
            radius=bd*density_ratio
            path=compute_density_path(best[0],best[1],hist_s,te,re,
                                      radius, theta_step, r_threshold)
            contour=[best[0]] + path + [best[1]]
            contours.append(contour)
    return internal, contours


def plot_clusters_and_connections(
    clusters, skeletons, hist_s, te, re,
    internal, contours
):
    # gráfico principal
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower',
               extent=[te[0],te[-1],re[0],re[-1]],
               aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters):
        plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6,
                    label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size:
            plt.scatter(arr[:,0], arr[:,1], c='w', s=40,
                        edgecolors='k')
    for a, b in internal:
        plt.plot([a[0],b[0]],[a[1],b[1]], '-',
                 color='white', linewidth=2)
    for cnt in contours:
        xs, ys = zip(*cnt)
        plt.plot(xs, ys, '-o', color='red',
                 markersize=4, linewidth=2, alpha=0.8)
    plt.title('Islas, conexiones y contornos filtrados')
    plt.xlabel('θ (°)'); plt.ylabel('r');
    plt.legend(loc='upper right'); plt.tight_layout();
    plt.show()

if __name__=='__main__':
    clusters, skeletons, hist_s, te, re = \
        main_histogram_weighted_bfs_clusters()
    internal, contours = \
        connect_skeletons(skeletons, hist_s, te, re)
    plot_clusters_and_connections(
        clusters, skeletons,
        hist_s, te, re,
        internal, contours
    )

## version cuasi_definitiva:

    def __init__(self, bins_theta=100, bins_r=80, smooth_sigma=1.0,
                 density_percentile=80, closing_size=2, min_cluster_size=40,
                 max_gap_dist=0.45, theta_bin_size=8, density_ratio=0.25,
                 theta_step=3, r_threshold=0.9, dr_multiplier=3.1):

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Histogram Segmenter
===================
Módulo autocontenido y reutilizable para:
1. Detección de "islas" en un histograma 2D (θ, r).
2. Cálculo de esqueletos por bins angulares (θ).
3. Generación **siempre** de centroides adicionales sobre *todas* las rectas
   candidatas (pares de esqueletos contiguos e inter‑isla), muestreando la
   densidad de fondo.
4. Segmentación de contornos: las conexiones se parten si Δr > *dr_multiplier*·μ,
   donde μ es el promedio de Δr de los segmentos generados.
5. Clasificación de objetos resultantes:
   • ``isla‑conexion‑isla``  (contornos válidos)
   • ``isla‑conexiones‑adicionales``  (segmentos descartados > umbral)
6. Salida estructurada + visualización opcional.

El diseño busca **estabilidad** ante pequeños cambios en parámetros para poder
aplicarlo a múltiples datasets sin retocar el código.
"""
from __future__ import annotations

import math
import heapq
from dataclasses import dataclass
from typing import List, Tuple, Dict, Any

import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter, binary_closing, label

# ---------------------------------------------------------------------------
# Dataclasses de apoyo
# ---------------------------------------------------------------------------
@dataclass
class Connection:
    a: Tuple[float, float]
    b: Tuple[float, float]
    delta_r: float
    euclidean: float

@dataclass
class IslandObject:
    type: str  # "isla-conexion-isla" | "isla-conexiones-adicionales"
    boundary: List[Tuple[float, float]]

# ---------------------------------------------------------------------------
# Algoritmo principal
# ---------------------------------------------------------------------------
class HistogramSegmenter:
    """Segmentador configurable.

    Parameters
    ----------
    *bins_theta*, *bins_r* : resolución del histograma (θ, r).
    *smooth_sigma*        : σ del filtro gaussiano.
    *density_percentile*  : umbral de densidad para binarización.
    *closing_size*        : tamaño del cierre morfológico.
    *min_cluster_size*    : puntos mínimos por isla.
    *max_gap_dist*        : distancia máx. para unir islas en BFS.
    *theta_bin_size*      : paso angular para esqueletos.
    *density_ratio*       : radio *relative* para muestreo de densidad.
    *theta_step*          : Δθ entre muestras sobre la recta.
    *r_threshold*         : tolerancia radial al validar centroides.
    *dr_multiplier*       : factor de corte (p.e. 1.3 o 3.1).
    """

    def __init__(self,
                 bins_theta: int = 100,
                 bins_r: int = 80,
                 smooth_sigma: float = 1.0,
                 density_percentile: float = 80,
                 closing_size: int = 2,
                 min_cluster_size: int = 40,
                 max_gap_dist: float = 0.45,
                 theta_bin_size: float = 8,
                 density_ratio: float = 0.25,
                 theta_step: float = 3,
                 r_threshold: float = 0.9,
                 dr_multiplier: float = 3.0):
        self.bins_theta = bins_theta
        self.bins_r = bins_r
        self.smooth_sigma = smooth_sigma
        self.density_percentile = density_percentile
        self.closing_size = closing_size
        self.min_cluster_size = min_cluster_size
        self.max_gap_dist = max_gap_dist
        self.theta_bin_size = theta_bin_size
        self.density_ratio = density_ratio
        self.theta_step = theta_step
        self.r_threshold = r_threshold
        self.dr_multiplier = dr_multiplier

    # ------------------------------------------------------------------
    # Paso 1 – Histograma + suavizado
    # ------------------------------------------------------------------
    def _histogram(self, pts: np.ndarray):
        hist, te, re = np.histogram2d(
            pts[:, 0], pts[:, 1],
            bins=[self.bins_theta, self.bins_r]
        )
        hist_s = gaussian_filter(hist, sigma=self.smooth_sigma)
        return hist_s, te, re

    # ------------------------------------------------------------------
    # Paso 2 – BFS ponderado para unir islas próximas
    # ------------------------------------------------------------------
    def _weighted_bfs(self, hist_s, te, re):
        thr = np.percentile(hist_s[hist_s > 0], self.density_percentile)
        mask = binary_closing(hist_s > thr,
                              structure=np.ones((self.closing_size,
                                                 self.closing_size)))
        labels, n_lab = label(mask)

        theta_res, r_res = te[1] - te[0], re[1] - re[0]
        neigh = [(-1, 0), (1, 0), (0, -1), (0, 1),
                 (-1, -1), (-1, 1), (1, -1), (1, 1)]

        parent = list(range(n_lab + 1))
        def find(x):
            while parent[x] != x:
                parent[x] = parent[parent[x]]
                x = parent[x]
            return x
        def union(a, b):
            ra, rb = find(a), find(b)
            if ra != rb:
                parent[rb] = ra

        h, w = labels.shape
        dist_grid = np.full((h, w), np.inf)
        pq: List[Tuple[float, int, int, int]] = []

        for i in range(h):
            for j in range(w):
                lab = labels[i, j]
                if lab:
                    for di, dj in neigh:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and labels[ni, nj] == 0:
                            d0 = math.hypot(di * theta_res, dj * r_res)
                            if d0 <= self.max_gap_dist and d0 < dist_grid[ni, nj]:
                                dist_grid[ni, nj] = d0
                                heapq.heappush(pq, (d0, ni, nj, lab))

        while pq:
            dist, i, j, lab = heapq.heappop(pq)
            if dist > dist_grid[i, j] or dist > self.max_gap_dist:
                continue
            for di, dj in neigh:
                ni, nj = i + di, j + dj
                if 0 <= ni < h and 0 <= nj < w:
                    lab2 = labels[ni, nj]
                    if lab2 and lab2 != lab:
                        union(lab, lab2)
            for di, dj in neigh:
                ni, nj = i + di, j + dj
                if 0 <= ni < h and 0 <= nj < w and labels[ni, nj] == 0:
                    nd = dist + math.hypot(di * theta_res, dj * r_res)
                    if nd <= self.max_gap_dist and nd < dist_grid[ni, nj]:
                        dist_grid[ni, nj] = nd
                        heapq.heappush(pq, (nd, ni, nj, lab))
        return find, labels

    # ------------------------------------------------------------------
    # Paso 3 – Clusters y esqueletos
    # ------------------------------------------------------------------
    def _clusters_and_skeletons(self, pts, find, labels, te, re):
        tidx = np.clip(np.digitize(pts[:, 0], te) - 1, 0, self.bins_theta - 1)
        ridx = np.clip(np.digitize(pts[:, 1], re) - 1, 0, self.bins_r - 1)
        merged: Dict[int, List[np.ndarray]] = {}
        for p, (ti, ri) in enumerate(zip(tidx, ridx)):
            lab = labels[ti, ri]
            if lab:
                merged.setdefault(find(lab), []).append(pts[p])
        clusters = [np.vstack(v) for v in merged.values()
                    if len(v) >= self.min_cluster_size]

        skeletons: List[List[Tuple[float, float]]] = []
        for cl in clusters:
            thetas, rs = cl[:, 0], cl[:, 1]
            bins_arr = np.arange(thetas.min(),
                                 thetas.max() + self.theta_bin_size,
                                 self.theta_bin_size)
            ske = []
            for k in range(len(bins_arr) - 1):
                m = (thetas >= bins_arr[k]) & (thetas < bins_arr[k + 1])
                if m.any():
                    ske.append((thetas[m].mean(), rs[m].mean()))
            skeletons.append(sorted(ske, key=lambda x: -x[0]))
        return clusters, skeletons

    # ------------------------------------------------------------------
    # Paso 4 – Centroides adicionales sobre *todas* las rectas candidatas
    # ------------------------------------------------------------------
    def _density_path(self, a, b, hist_s, te, re):
        """Genera centroides extra a lo largo del segmento a‑b."""
        theta_res, r_res = te[1] - te[0], re[1] - re[0]
        steps = max(int(abs(b[0] - a[0]) / self.theta_step) + 1, 2)
        thetas = np.linspace(a[0], b[0], steps)
        rs_seg = np.linspace(a[1], b[1], steps)
        radius = math.hypot(b[0] - a[0], b[1] - a[1]) * self.density_ratio

        out: List[Tuple[float, float]] = []
        for θ_i, r_i in zip(thetas, rs_seg):
            ti = int(np.clip(np.digitize(θ_i, te) - 1, 0, hist_s.shape[0] - 1))
            ri0 = int(np.clip(np.digitize(r_i, re) - 1, 0, hist_s.shape[1] - 1))
            max_off = int(math.ceil(radius / r_res))
            vals, offs = [], []
            for off in range(-max_off, max_off + 1):
                idx = ri0 + off
                if 0 <= idx < hist_s.shape[1]:
                    vals.append(hist_s[ti, idx]); offs.append(off)
            for k in range(1, len(vals) - 1):
                if vals[k] > vals[k - 1] and vals[k] > vals[k + 1] and vals[k] > 0:
                    rc = re[ri0 + offs[k]] + r_res / 2.0
                    if abs(rc - r_i) <= self.r_threshold:
                        out.append((θ_i, rc))
        return out

    def _candidate_segments(self, skeletons):
        segs = []
        # 4a) internos (pares contiguos)
        for ske in skeletons:
            segs.extend(list(zip(ske, ske[1:])))
        # 4b) mejores conexiones inter‑isla (criterio dr>0, dθ<0, distancia mínima)
        for i, ske in enumerate(skeletons):
            if not ske:
                continue
            a = ske[-1]
            best, bd = None, math.inf
            for j, ske2 in enumerate(skeletons):
                if i == j or not ske2:
                    continue
                b = ske2[0]
                dr, dtheta = b[1] - a[1], b[0] - a[0]
                if dr > 0 and dtheta < 0:
                    d = math.hypot(dr, dtheta)
                    if d < bd:
                        bd, best = d, (a, b)
            if best:
                segs.append(best)
        return segs

    # ------------------------------------------------------------------
    # Paso 5 – Construir rutas (contours) y filtrar por Δr
    # ------------------------------------------------------------------
    def _build_contours(self, segs, hist_s, te, re):
        contours: List[List[Tuple[float, float]]] = []
        connections: List[Connection] = []
        for a, b in segs:
            path = self._density_path(a, b, hist_s, te, re)
            route = [a] + path + [b]
            contours.append(route)
            # registrar Δr entre a y b (para estadística global)
            connections.append(Connection(a, b, abs(b[1] - a[1]),
                                           math.hypot(b[0] - a[0], b[1] - a[1])))
        return contours, connections

    @staticmethod
    def _split_by_threshold(contours, thr):
        """Parte las rutas cuando Δr > thr."""
        out = []
        for cnt in contours:
            tmp = [cnt[0]]
            for a, b in zip(cnt, cnt[1:]):
                if abs(b[1] - a[1]) <= thr:
                    tmp.append(b)
                else:
                    if len(tmp) > 1:
                        out.append(tmp)
                    tmp = [b]
            if len(tmp) > 1:
                out.append(tmp)
        return out

    # ------------------------------------------------------------------
    # Método público -----------------------------------------------------------------
    # ------------------------------------------------------------------
    def run(self, datos: Dict[str, Any]):
        # Entrada esperada
        bg = datos['background'][['theta', 'r']].values
        grp = np.vstack([g['points'][['theta', 'r']].values
                          for g in datos['grupos_finales']])
        pts = np.vstack([bg, grp])

        # 1) Histograma
        hist_s, te, re = self._histogram(pts)

        # 2) BFS + 3) clusters + esqueletos
        find, labels = self._weighted_bfs(hist_s, te, re)
        clusters, skeletons = self._clusters_and_skeletons(pts, find, labels, te, re)

        # 4) segmentos candidatos + rutas con centroides extra
        segs = self._candidate_segments(skeletons)
        raw_contours, conns = self._build_contours(segs, hist_s, te, re)

        # 5) filtrado por Δr
        mean_dr = float(np.mean([c.delta_r for c in conns])) if conns else 0.0
        thr = mean_dr * self.dr_multiplier
        final_contours = self._split_by_threshold(raw_contours, thr)

        # Objetos
        objects = [IslandObject('isla-conexion-isla', cnt) for cnt in final_contours]
        for c in conns:
            if c.delta_r > thr:
                objects.append(IslandObject('isla-conexiones-adicionales', [c.a, c.b]))

        return {
            'hist_s': hist_s, 'te': te, 're': re,
            'clusters': clusters, 'skeletons': skeletons,
            'contours': final_contours, 'connections': conns,
            'mean_dr': mean_dr, 'threshold': thr, 'objects': objects
        }

    # ------------------------------------------------------------------
    # Utilidad rápida para visualizar
    # ------------------------------------------------------------------
    def quick_plot(self, result, figsize=(14, 6)):
        plt.figure(figsize=figsize)
        plt.imshow(result['hist_s'].T, origin='lower',
                   extent=[result['te'][0], result['te'][-1],
                           result['re'][0], result['re'][-1]],
                   aspect='auto', cmap='inferno')
        plt.colorbar(label='Densidad suavizada')
        for idx, cl in enumerate(result['clusters']):
            plt.scatter(cl[:, 0], cl[:, 1], s=8, alpha=0.6,
                        label=f'Isla {idx}')
        for ske in result['skeletons']:
            arr = np.array(ske)
            if arr.size:
                plt.scatter(arr[:, 0], arr[:, 1], c='w', s=40,
                            edgecolors='k')
        for cnt in result['contours']:
            xs, ys = zip(*cnt)
            plt.plot(xs, ys, '-o', color='cyan', markersize=5)


# ------------------------------------------------------------
# 1.  Importa el módulo y los datos de ejemplo
# ------------------------------------------------------------
#from histogram_segmenter import HistogramSegmenter   #  <-- el archivo del canvas
#from your_data_module import main_example_modificado #  <-- tu función de carga

# ------------------------------------------------------------
# 2.  Carga datos y crea la instancia con los parámetros deseados
# ------------------------------------------------------------
datos = main_example_modificado()          # debe devolver un dict con background y grupos_finales
segmenter = HistogramSegmenter(            # puedes ajustar cualquier parámetro aquí
    bins_theta=120,
    bins_r=80,
    smooth_sigma=1.0,
    theta_bin_size=8,
    dr_multiplier=3.0                      # 3.0 × μ para partir conexiones
)

# ------------------------------------------------------------
# 3.  Ejecuta el algoritmo
# ------------------------------------------------------------
result = segmenter.run(datos)

# ------------------------------------------------------------
# 4.  Consulta la salida estructurada
# ------------------------------------------------------------
print("Islas encontradas:", len(result['clusters']))
print("Promedio Δr:", result['mean_dr'])
print("Umbral usado:", result['threshold'])
print("Objetos clasificados:", len(result['objects']))

# ------------------------------------------------------------
# 5.  Visualiza (opcional)
# ------------------------------------------------------------
segmenter.quick_plot(result)


In [0]:
import matplotlib.patheffects as PathEffects
def plot_islands_and_paths_polar(result, datos, id_halo="11", line_extrap=0.5):
    """
    Función para crear un gráfico polar destacando las islas encontradas, los caminos de los centroides,
    los contornos y añadiendo los puntos de fondo con un color gris muy tenue.
    """

    # Filtrar islas grandes
    big_islands = [cl for cl in result['clusters'] if len(cl) >= 60]
    print(f"Islas ≥60 puntos (polar): {len(big_islands)}")

    fig = plt.figure(figsize=(9, 8))
    ax = fig.add_subplot(111, projection='polar')

    # Añadir puntos de fondo con gris muy tenue
    bg = datos['background'][['theta', 'r']].values
    ax.scatter(np.radians(bg[:, 0]), bg[:, 1], s=3, color='gray', edgecolor='black' ,alpha=0.21, label='Puntos de fondo')

    # Colores para las islas
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan', 'gold', 'gray', 'lime']

    # Resaltar islas y caminos
    for idx, cl in enumerate(big_islands):
        c = colors[idx % len(colors)]
        theta = np.radians(cl[:, 0])  # Convertir theta de grados a radianes
        r = cl[:, 1]
        ax.scatter(theta, r, s=10, color=c, alpha=0.34, label=f'Isla {idx+1} ({len(cl)})')

        # Resaltar el camino de los centroides (esqueletos) con borde
        for ske in result['skeletons']:
            arr = np.array(ske)
            if arr.size:
                theta_ske = np.radians(arr[:, 0])  # Convertir theta a radianes
                r_ske = arr[:, 1]
                line = ax.plot(theta_ske, r_ske, color='yellow', alpha=0.95, linewidth=2, label='Camino centroides')

                # Añadir borde a la línea
                for l in line:
                    l.set_path_effects([PathEffects.withStroke(linewidth=3, alpha=0.51,foreground='black')])

        #Resaltar las conexiones entre los centroides
        for cnt in result['contours']:
            theta_cnt, r_cnt = zip(*cnt)
            ax.plot(np.radians(theta_cnt), r_cnt, 'o', color='cyan',markersize=2.5, alpha=0.585, label='Contorno')
            ax.plot(np.radians(theta_cnt), r_cnt, '-o', color='cyan',markersize=1.0, alpha=0.465, label='Contorno')

        # Extrapolación de la línea (si es necesario)
        if 'slope' in cl and cl['slope'] is not None:  # Usamos 'cl' aquí en lugar de 'g'
            dt = cl['theta_max'] - cl['theta_min']
            t_deg = np.linspace(cl['theta_min'] - line_extrap * dt, cl['theta_max'] + line_extrap * dt, 200)
            r_line = cl['slope'] * t_deg + cl['intercept']
            ax.plot(np.radians(t_deg), r_line, '.-', color="gray", alpha=0.20)
            ax.plot(np.radians(t_deg), r_line, '--', color=c)

    # Configuración de los ejes polares
    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title(f"Halo{id_halo} - Caminos de Centroides", y=1.05)
    ax.grid(True, alpha=0.1, color='gray')
    #ax.legend(loc='upper right', bbox_to_anchor=(1.20, 1.0))

    # Mostrar gráfico
    plt.show()

# ------------------------------------------------------------
# Llamada a la nueva celda después de ejecutar el algoritmo
# ------------------------------------------------------------
plot_islands_and_paths_polar(result, datos, id_halo="17")

### termina verison definitiva

## intento de mejora

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación + BFS + conexiones dinámicas + contornos filtrados,
almacenamiento de centroides adicionales y conexiones con distancias Euclidiana y radial,
resaltando segmentos con Δr > 1 en el gráfico.
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq, math
from collections import defaultdict

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100, bins_r=60, smooth_sigma=1.0,
    density_percentile=80, closing_size=2,
    min_cluster_size=40, max_gap_dist=0.5,
    theta_bin_size=10
):
    datos = main_example_modificado()
    bg    = datos['background'][['theta','r']].values
    grp   = np.vstack([g['points'][['theta','r']].values for g in datos['grupos_finales']])
    pts   = np.vstack([bg, grp])

    hist, te, re = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s       = gaussian_filter(hist, sigma=smooth_sigma)

    thr    = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size, closing_size)))
    labels, n_lab = label(mask_c)

    theta_res = te[1] - te[0]
    r_res     = re[1] - re[0]
    neigh     = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf        = UnionFind(n_lab + 1)
    h, w      = labels.shape
    dist_grid = np.full((h, w), np.inf)
    pq        = []

    # BFS ponderado en frontera
    for i in range(h):
        for j in range(w):
            if labels[i,j] > 0:
                for di, dj in neigh:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w and labels[ni,nj] == 0:
                        d0 = math.hypot(di*theta_res, dj*r_res)
                        if d0 <= max_gap_dist and d0 < dist_grid[ni,nj]:
                            dist_grid[ni,nj] = d0
                            heapq.heappush(pq, (d0, ni, nj, labels[i,j]))
    while pq:
        dist, i, j, lab = heapq.heappop(pq)
        if dist > dist_grid[i,j] or dist > max_gap_dist:
            continue
        for di, dj in neigh:
            ni, nj = i + di, j + dj
            if 0 <= ni < h and 0 <= nj < w:
                lab2 = labels[ni,nj]
                if lab2 > 0 and lab2 != lab:
                    uf.union(lab, lab2)
        for di, dj in neigh:
            ni, nj = i + di, j + dj
            if 0 <= ni < h and 0 <= nj < w and labels[ni,nj] == 0:
                nd = dist + math.hypot(di*theta_res, dj*r_res)
                if nd <= max_gap_dist and nd < dist_grid[ni,nj]:
                    dist_grid[ni,nj] = nd
                    heapq.heappush(pq, (nd, ni, nj, lab))

    # Recolectar clusters
    merged = defaultdict(list)
    tidx = np.clip(np.digitize(pts[:,0], te) - 1, 0, bins_theta - 1)
    ridx = np.clip(np.digitize(pts[:,1], re) - 1, 0, bins_r - 1)
    for p, (ti, ri) in enumerate(zip(tidx, ridx)):
        lab = labels[ti, ri]
        if lab > 0:
            merged[uf.find(lab)].append(pts[p])
    clusters = [np.vstack(v) for v in merged.values() if len(v) >= min_cluster_size]

    # Skeletons
    skeletons = []
    for c in clusters:
        thetas, rs = c[:,0], c[:,1]
        bins_arr = np.arange(thetas.min(), thetas.max() + theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins_arr) - 1):
            mask = (thetas >= bins_arr[k]) & (thetas < bins_arr[k+1])
            if mask.any():
                ske.append((thetas[mask].mean(), rs[mask].mean()))
        skeletons.append(sorted(ske, key=lambda x: -x[0]))

    return clusters, skeletons, hist_s, te, re


def compute_density_path(p, q, hist_s, te, re, radius,
                         theta_step=2, r_threshold=0.9):
    theta_res = te[1] - te[0]
    r_res     = re[1] - re[0]
    total_dθ  = q[0] - p[0]
    steps     = max(int(abs(total_dθ) / theta_step) + 1, 2)
    thetas    = np.linspace(p[0], q[0], steps)
    rs_vals   = np.linspace(p[1], q[1], steps)
    path = []
    for θ_i, r_i in zip(thetas, rs_vals):
        ti  = int(np.clip(np.digitize(θ_i, te) - 1, 0, hist_s.shape[0] - 1))
        ri0 = int(np.clip(np.digitize(r_i, re) - 1, 0, hist_s.shape[1] - 1))
        max_off = int(np.ceil(radius / r_res))
        vals, offs = [], []
        for off in range(-max_off, max_off + 1):
            idx_r = ri0 + off
            if 0 <= idx_r < hist_s.shape[1]:
                vals.append(hist_s[ti, idx_r])
                offs.append(off)
        for k in range(1, len(vals)-1):
            if vals[k] > vals[k-1] and vals[k] > vals[k+1] and vals[k] > 0:
                off = offs[k]
                θc = te[ti] + theta_res/2
                rc = (re[ri0+off] + r_res/2) if 0 <= ri0+off < len(re) else r_i
                if abs(rc - r_i) <= r_threshold:
                    path.append((θc, rc))
    return path


def connect_skeletons(skeletons, hist_s, te, re,
                      density_ratio=0.25, theta_step=2, r_threshold=0.9):
    internal = []
    contours = []
    merged_conns = []

    for ske in skeletons:
        internal += list(zip(ske, ske[1:]))

    for i, ske in enumerate(skeletons):
        if not ske:
            continue
        p = ske[-1]
        best_pair = None
        best_dist = math.inf
        for j, ske2 in enumerate(skeletons):
            if i == j or not ske2:
                continue
            q = ske2[0]
            drd = q[1] - p[1]
            dtd = q[0] - p[0]
            if drd > 0 and dtd < 0:
                d = math.hypot(dtd, drd)
                if d < best_dist:
                    best_dist = d
                    best_pair = (p, q)
        if best_pair:
            p0, q0 = best_pair
            radius = best_dist * density_ratio
            path = compute_density_path(p0, q0, hist_s, te, re, radius,
                                        theta_step, r_threshold)
            contour = [p0] + path + [q0]
            contours.append(contour)

    additional_centroids = []
    connections = []

    for contour in contours:
        additional_centroids.extend(contour[1:-1])
        for a, b in zip(contour, contour[1:]):
            euclidean = math.hypot(b[0]-a[0], b[1]-a[1])
            delta_r = abs(b[1]-a[1])
            connections.append({'points':(a,b), 'delta_r':delta_r, 'euclidean':euclidean})
    for a, b in internal:
        euclidean = math.hypot(b[0]-a[0], b[1]-a[1])
        delta_r = abs(b[1]-a[1])
        connections.append({'points':(a,b), 'delta_r':delta_r, 'euclidean':euclidean})

    additional_centroids = list(dict.fromkeys(additional_centroids))
    return internal, contours, additional_centroids, connections


def plot_clusters_and_connections(
    clusters, skeletons, hist_s, te, re,
    internal, contours, connections
):
    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower', extent=[te[0],te[-1],re[0],re[-1]], aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters): plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size: plt.scatter(arr[:,0], arr[:,1], c='w', s=40, edgecolors='k')
    # conexiones normales
    for a, b in internal:
        plt.plot([a[0],b[0]],[a[1],b[1]], '-', linewidth=2)
    for cnt in contours:
        xs, ys = zip(*cnt)
        plt.plot(xs, ys, '-o', markersize=4, linewidth=2, alpha=0.8)
    # resaltar segmentos con delta_r > 1
    for conn in connections:
        a, b = conn['points']
        if conn['delta_r'] > 1:
            # línea blanca de fondo gruesa
            plt.plot([a[0],b[0]],[a[1],b[1]], '-', color='white', linewidth=6, zorder=5)
            # línea roja encima
            plt.plot([a[0],b[0]],[a[1],b[1]], '-', color='red', linewidth=4, zorder=6)
    plt.title('Islas, conexiones y contornos filtrados')
    plt.xlabel('θ (°)'); plt.ylabel('r'); plt.legend(loc='upper right'); plt.tight_layout(); plt.show()

if __name__ == '__main__':
    clusters, skeletons, hist_s, te, re = main_histogram_weighted_bfs_clusters()
    internal, contours, additional_pts, connections = connect_skeletons(skeletons, hist_s, te, re)
    info = {'additional_centroids': additional_pts, 'connections': connections}
    print(f"Centroides adicionales: {len(additional_pts)}")
    print(f"Conexiones totales: {len(connections)}")
    plot_clusters_and_connections(clusters, skeletons, hist_s, te, re, internal, contours, connections)


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Segmentación + BFS + conexiones dinámicas + contornos filtrados,
almacenamiento de centroides adicionales y conexiones con distancias Euclidiana y radial,
resaltando segmentos con Δr > 1 y considerando aislados los centroides adicionales
con Δr > 1.3 * promedio.
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, binary_closing, label
import heapq, math
from collections import defaultdict

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra


def main_histogram_weighted_bfs_clusters(
    bins_theta=100, bins_r=60, smooth_sigma=1.0,
    density_percentile=80, closing_size=2,
    min_cluster_size=40, max_gap_dist=0.5,
    theta_bin_size=10
):
    datos = main_example_modificado()
    bg    = datos['background'][['theta','r']].values
    grp   = np.vstack([g['points'][['theta','r']].values for g in datos['grupos_finales']])
    pts   = np.vstack([bg, grp])

    hist, te, re = np.histogram2d(pts[:,0], pts[:,1], bins=[bins_theta, bins_r])
    hist_s       = gaussian_filter(hist, sigma=smooth_sigma)

    thr    = np.percentile(hist_s[hist_s>0], density_percentile)
    mask_c = binary_closing(hist_s>thr, structure=np.ones((closing_size, closing_size)))
    labels, n_lab = label(mask_c)

    theta_res = te[1] - te[0]
    r_res     = re[1] - re[0]
    neigh     = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    uf        = UnionFind(n_lab + 1)
    h, w      = labels.shape
    dist_grid = np.full((h, w), np.inf)
    pq        = []

    # BFS ponderado en frontera
    for i in range(h):
        for j in range(w):
            if labels[i,j] > 0:
                for di, dj in neigh:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w and labels[ni,nj] == 0:
                        d0 = math.hypot(di*theta_res, dj*r_res)
                        if d0 <= max_gap_dist and d0 < dist_grid[ni,nj]:
                            dist_grid[ni,nj] = d0
                            heapq.heappush(pq, (d0, ni, nj, labels[i,j]))
    while pq:
        dist, i, j, lab = heapq.heappop(pq)
        if dist > dist_grid[i,j] or dist > max_gap_dist:
            continue
        for di, dj in neigh:
            ni, nj = i + di, j + dj
            if 0 <= ni < h and 0 <= nj < w:
                lab2 = labels[ni,nj]
                if lab2 > 0 and lab2 != lab:
                    uf.union(lab, lab2)
        for di, dj in neigh:
            ni, nj = i + di, j + dj
            if 0 <= ni < h and 0 <= nj < w and labels[ni,nj] == 0:
                nd = dist + math.hypot(di*theta_res, dj*r_res)
                if nd <= max_gap_dist and nd < dist_grid[ni,nj]:
                    dist_grid[ni,nj] = nd
                    heapq.heappush(pq, (nd, ni, nj, lab))

    # Recolectar clusters
    merged = defaultdict(list)
    tidx = np.clip(np.digitize(pts[:,0], te) - 1, 0, bins_theta - 1)
    ridx = np.clip(np.digitize(pts[:,1], re) - 1, 0, bins_r - 1)
    for p, (ti, ri) in enumerate(zip(tidx, ridx)):
        lab = labels[ti, ri]
        if lab > 0:
            merged[uf.find(lab)].append(pts[p])
    clusters = [np.vstack(v) for v in merged.values() if len(v) >= min_cluster_size]

    # Skeletons
    skeletons = []
    for c in clusters:
        thetas, rs = c[:,0], c[:,1]
        bins_arr = np.arange(thetas.min(), thetas.max() + theta_bin_size, theta_bin_size)
        ske = []
        for k in range(len(bins_arr) - 1):
            mask = (thetas >= bins_arr[k]) & (thetas < bins_arr[k+1])
            if mask.any():
                ske.append((thetas[mask].mean(), rs[mask].mean()))
        skeletons.append(sorted(ske, key=lambda x: -x[0]))

    return clusters, skeletons, hist_s, te, re


def compute_density_path(p, q, hist_s, te, re, radius,
                         theta_step=2, r_threshold=0.9):
    theta_res = te[1] - te[0]
    r_res     = re[1] - re[0]
    total_dθ  = q[0] - p[0]
    steps     = max(int(abs(total_dθ) / theta_step) + 1, 2)
    thetas    = np.linspace(p[0], q[0], steps)
    rs_vals   = np.linspace(p[1], q[1], steps)
    path = []
    for θ_i, r_i in zip(thetas, rs_vals):
        ti  = int(np.clip(np.digitize(θ_i, te) - 1, 0, hist_s.shape[0] - 1))
        ri0 = int(np.clip(np.digitize(r_i, re) - 1, 0, hist_s.shape[1] - 1))
        max_off = int(np.ceil(radius / r_res))
        vals, offs = [], []
        for off in range(-max_off, max_off + 1):
            idx_r = ri0 + off
            if 0 <= idx_r < hist_s.shape[1]:
                vals.append(hist_s[ti, idx_r])
                offs.append(off)
        for k in range(1, len(vals)-1):
            if vals[k] > vals[k-1] and vals[k] > vals[k+1] and vals[k] > 0:
                off = offs[k]
                θc = te[ti] + theta_res/2
                rc = (re[ri0+off] + r_res/2) if 0 <= ri0+off < len(re) else r_i
                if abs(rc - r_i) <= r_threshold:
                    path.append((θc, rc))
    return path


def connect_skeletons(skeletons, hist_s, te, re,
                      density_ratio=0.25, theta_step=2, r_threshold=0.9):
    internal = []
    contours = []
    connections = []

    # conexiones internas
    for ske in skeletons:
        internal += list(zip(ske, ske[1:]))
    # contornos
    for i, ske in enumerate(skeletons):
        if not ske: continue
        p = ske[-1]; best_pair, best_dist = None, math.inf
        for j, ske2 in enumerate(skeletons):
            if i==j or not ske2: continue
            q = ske2[0]
            drd, dtd = q[1]-p[1], q[0]-p[0]
            if drd>0 and dtd<0:
                d = math.hypot(dtd, drd)
                if d<best_dist: best_dist, best_pair = d, (p,q)
        if best_pair:
            p0,q0 = best_pair
            radius = best_dist * density_ratio
            path = compute_density_path(p0,q0,hist_s,te,re,radius,theta_step,r_threshold)
            contour = [p0] + path + [q0]
            contours.append(contour)
    # extraer conexiones con distancias
    for a,b in internal + [pt for contour in contours for pt in zip(contour, contour[1:])]:
        euclid = math.hypot(b[0]-a[0], b[1]-a[1])
        delta_r = abs(b[1]-a[1])
        connections.append({'points':(a,b), 'delta_r':delta_r, 'euclidean':euclid})

    return internal, contours, connections


def plot_and_analyze(
    clusters, skeletons, hist_s, te, re,
    internal, contours, connections
):
    # Estadísticos para conexiones de centroides adicionales
    # Filtrar solo conexiones con centroides generados (contours)
    ad_conns = [c for c in connections if c['points'][0] in {pt for cnt in contours for pt in cnt[1:-1]} or
                                           c['points'][1] in {pt for cnt in contours for pt in cnt[1:-1]}]
    dr_vals = [c['delta_r'] for c in ad_conns]
    mean_dr = np.mean(dr_vals)
    median_dr = np.median(dr_vals)
    threshold = 3.1 * mean_dr
    isolated = {pt for c in ad_conns if c['delta_r']>threshold for pt in c['points']}

    plt.figure(figsize=(14,6))
    plt.imshow(hist_s.T, origin='lower', extent=[te[0],te[-1],re[0],re[-1]], aspect='auto', cmap='inferno')
    plt.colorbar(label='Densidad suavizada')
    for idx, c in enumerate(clusters): plt.scatter(c[:,0], c[:,1], s=8, alpha=0.6, label=f'Isla {idx}')
    for ske in skeletons:
        arr = np.array(ske)
        if arr.size: plt.scatter(arr[:,0], arr[:,1], c='w', s=40, edgecolors='k')
    # conexiones normales y contornos
    for a,b in internal:
        plt.plot([a[0],b[0]],[a[1],b[1]], '-', linewidth=2)
    for cnt in contours:
        xs, ys = zip(*cnt)
        plt.plot(xs, ys, '-o', markersize=4, linewidth=2, alpha=0.8)
    # resaltar Δr >1 y aislados
    for c in connections:
        a,b = c['points']
        if c['delta_r']>1:
            plt.plot([a[0],b[0]],[a[1],b[1]], '-', color='white', linewidth=6, zorder=5)
            plt.plot([a[0],b[0]],[a[1],b[1]], '-', color='red', linewidth=4, zorder=6)
    # marcar aislados
    for pt in isolated:
        plt.scatter(pt[0], pt[1], marker='X', s=100, edgecolors='yellow', linewidth=2, facecolors='none', zorder=7)
    plt.title(f'Umbral Δr>1.3∗μ={threshold:.2f}; mediana={median_dr:.2f}')
    plt.xlabel('θ (°)'); plt.ylabel('r'); plt.legend(loc='upper right'); plt.tight_layout(); plt.show()

if __name__ == '__main__':
    clusters, skeletons, hist_s, te, re = main_histogram_weighted_bfs_clusters()
    internal, contours, connections = connect_skeletons(skeletons, hist_s, te, re)
    print(f'Promedio Δr adicionales: {np.mean([c["delta_r"] for c in connections]):.3f}')
    print(f'Mediana Δr adicionales: {np.median([c["delta_r"] for c in connections]):.3f}')
    plot_and_analyze(clusters, skeletons, hist_s, te, re, internal, contours, connections)



## halo 11

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression

# ====================================
# 1) Funciones para Cargar/Filtrar Datos
# ====================================
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

# ====================================
# 2) Construir Grafo (Rectangular BFS)
# ====================================
def build_graph_rectangular(df_points, theta_diff, r_diff):
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        mask_neighbors = (dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i)
        neighbors = np.where(mask_neighbors)[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False]*n
    clusters = []
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            clusters.append(cluster_df)
    return clusters

# ====================================
# 3) Subdividir por "gaps"
# ====================================
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

# ====================================
# 4) Generar Semillas BFS
# ====================================
def generate_bfs_seeds(
    id_halo,
    theta_min=50,
    theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters, df_filtered

# ====================================
# 5) Ajustar rectas y Fusionar
# ====================================
def calculate_pa(slope):
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return None, None, None, None, None, None, None
    X = df_cluster[['theta']].values
    y = df_cluster['r'].values
    model = LinearRegression()
    model.fit(X, y)
    slope_ = model.coef_[0]
    intercept_ = model.intercept_
    pa_ = calculate_pa(slope_)
    t_min = df_cluster['theta'].min()
    t_max = df_cluster['theta'].max()
    r_min = df_cluster['r'].min()
    r_max = df_cluster['r'].max()
    return slope_, intercept_, pa_, t_min, t_max, r_min, r_max

def adjust_and_merge_seeds(
    bfs_clusters,
    slope_variation_threshold=0.40,
    bounding_extrap=0.30
):
    groups = []
    for clust_df in bfs_clusters:
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(clust_df)
        grp = {
            'points': clust_df.copy(),
            'slope': slope_,
            'intercept': intercept_,
            'pa': pa_,
            'theta_min': tmin,
            'theta_max': tmax,
            'r_min': rmin,
            'r_max': rmax
        }
        groups.append(grp)
    
    def recalc_properties(group):
        dfp = group['points']
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(dfp)
        group['slope'] = slope_
        group['intercept'] = intercept_
        group['pa'] = pa_
        group['theta_min'] = tmin
        group['theta_max'] = tmax
        group['r_min'] = rmin
        group['r_max'] = rmax

    def boxes_overlap(g1, g2, extrap=0.30):
        tmin1, tmax1 = g1['theta_min'], g1['theta_max']
        rmin1, rmax1 = g1['r_min'], g1['r_max']
        tmin2, tmax2 = g2['theta_min'], g2['theta_max']
        rmin2, rmax2 = g2['r_min'], g2['r_max']
        if tmin1 is None or tmin2 is None:
            return False
        dt1 = (tmax1 - tmin1)
        dr1 = (rmax1 - rmin1)
        dt2 = (tmax2 - tmin2)
        dr2 = (rmax2 - rmin2)
        extr_dt1 = dt1*extrap
        extr_dr1 = dr1*extrap
        extr_dt2 = dt2*extrap
        extr_dr2 = dr2*extrap
        
        tmin1e = tmin1 - extr_dt1
        tmax1e = tmax1 + extr_dt1
        rmin1e = rmin1 - extr_dr1
        rmax1e = rmax1 + extr_dr1
        
        tmin2e = tmin2 - extr_dt2
        tmax2e = tmax2 + extr_dt2
        rmin2e = rmin2 - extr_dr2
        rmax2e = rmax2 + extr_dr2
        
        overlap_theta = not (tmax1e < tmin2e or tmax2e < tmin1e)
        overlap_r = not (rmax1e < rmin2e or rmax2e < rmin1e)
        return overlap_theta and overlap_r
    
    merged_flag = True
    while merged_flag:
        merged_flag = False
        new_groups = []
        i = 0
        while i < len(groups):
            g1 = groups[i]
            j = i+1
            fused_any = False
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    combined = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    slope_, intercept_, _, _, _, _, _ = fit_line_to_cluster(combined)
                    if slope_ is not None:
                        var1 = abs(slope_ - g1['slope'])/(abs(g1['slope'])+1e-12)
                        var2 = abs(slope_ - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if var1 < slope_variation_threshold and var2 < slope_variation_threshold:
                            g1['points'] = combined
                            recalc_properties(g1)
                            groups.pop(j)
                            fused_any = True
                            merged_flag = True
                            continue
                j += 1
            new_groups.append(g1)
            i += 1
        groups = new_groups
    
    final_groups = []
    for gg in groups:
        if len(gg['points']) >= 2:
            final_groups.append(gg)
    return final_groups

# ====================================
# 6) Validar dispersión y re-procesar (opcional)
# ====================================
def validate_dispersion_and_reprocess(
    groups,
    dispersion_threshold=2.0,   # umbral de desviación std (ejemplo)
    reproc_theta_diff=2.0,      # BFS para reprocesar
    reproc_r_diff=0.5
):
    groups_ok = []
    groups_toreprocess = []
    
    for grp in groups:
        slope = grp['slope']
        intercept = grp['intercept']
        pts = grp['points']
        if slope is None or len(pts) < 2:
            groups_ok.append(grp)
            continue
        
        # Calcular std de residuo
        predicted_r = slope*pts['theta'] + intercept
        residuals = pts['r'] - predicted_r
        std_resid = residuals.std()
        if std_resid > dispersion_threshold:
            groups_toreprocess.append(grp)
        else:
            groups_ok.append(grp)
    
    # Reprocesar los grupos con alta dispersión
    new_groups = []
    for grp in groups_toreprocess:
        subdf = grp['points'].copy()
        subdf.reset_index(drop=True, inplace=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        clusters_sub = bfs_components(graph, subdf)
        
        for csub in clusters_sub:
            slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(csub)
            new_groups.append({
                'points': csub,
                'slope': slope_,
                'intercept': intercept_,
                'pa': pa_,
                'theta_min': tmin,
                'theta_max': tmax,
                'r_min': rmin,
                'r_max': rmax
            })
    
    final_res = groups_ok + new_groups
    return final_res

# =========================================
# 7) Nueva fase: Elegir grupos con >= 60 puntos y graficar con bounding box extrapolado
# =========================================
def plot_final_segments_over_60(groups, df_all, bounding_extrap=0.30):
    """
    - Toma los grupos finales
    - Filtra los que tienen >= 60 puntos
    - Grafica en un nuevo plot:
        * Los puntos de cada grupo
        * La recta
        * Un rectángulo que muestre el bounding box extrapolado un 30%
    """
    # 1. Filtrar
    large_groups = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos con >= 60 puntos: {len(large_groups)}")
    
    # 2. Crear figura
    plt.figure(figsize=(10, 6))
    
    # Fondo con df_all en gris
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')
    
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    
    for i, grp in enumerate(large_groups):
        c = colors[i % len(colors)]
        pts = grp['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"Grupo_{i+1} ({len(pts)} pts)")
        
        slope_ = grp['slope']
        intercept_ = grp['intercept']
        if slope_ is not None:
            tmin = pts['theta'].min()
            tmax = pts['theta'].max()
            t_lin = np.linspace(tmin, tmax, 100)
            r_lin = slope_*t_lin + intercept_
            plt.plot(t_lin, r_lin, '--', color=c)
        
        # 3. Dibujar bounding box extrapolado 30%
        # bounding box actual
        tmin_bb = grp['theta_min']
        tmax_bb = grp['theta_max']
        rmin_bb = grp['r_min']
        rmax_bb = grp['r_max']
        
        # Calcular ancho/alto
        dt = (tmax_bb - tmin_bb)
        dr = (rmax_bb - rmin_bb)
        dt_ext = dt * bounding_extrap
        dr_ext = dr * bounding_extrap
        
        # bounding box expandido
        tmin_e = tmin_bb - dt_ext
        tmax_e = tmax_bb + dt_ext
        rmin_e = rmin_bb - dr_ext
        rmax_e = rmax_bb + dr_ext
        
        # Dibujar rectángulo con plt.plot (4 lados)
        # forma => [(x1, y1), (x1, y2), (x2, y2), (x2, y1), (x1, y1)]
        # donde x=theta, y=r
        box_x = [tmin_e, tmin_e, tmax_e, tmax_e, tmin_e]
        box_y = [rmin_e, rmax_e, rmax_e, rmin_e, rmin_e]
        plt.plot(box_x, box_y, color=c, linewidth=1.2)
    
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    plt.title("Grupos con >= 60 puntos y su bounding box extrapolado")
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

# =========================================
# 8) Función principal: ejemplo
# =========================================
def main_example():
    # Paso 1: generar BFS
    bfs_clusters, df_filtrado = generate_bfs_seeds(
        id_halo="11",
        theta_min=20,
        theta_max=350,
        quartile_threshold=0.155,
        theta_diff=3.0,
        r_diff=0.5,
        gap_threshold_theta=1.80,
        gap_threshold_r=1.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs_clusters)} grupos")
    
    # Paso 2: ajustar/fusionar
    merged_groups = adjust_and_merge_seeds(
        bfs_clusters,
        slope_variation_threshold=0.30,
        bounding_extrap=0.35
    )
    print(f"Grupos tras fusión inicial: {len(merged_groups)}")
    
    # Paso 3: validación de dispersión (opcional)
    validated_groups = validate_dispersion_and_reprocess(
        merged_groups,
        dispersion_threshold=0.80,
        reproc_theta_diff=1.025,
        reproc_r_diff=0.34
    )
    print(f"Grupos tras validación dispersión: {len(validated_groups)}")
    
    # Plot de esos grupos en general
    plt.figure(figsize=(8,6))
    plt.scatter(df_filtrado['theta'], df_filtrado['r'], s=3, alpha=0.3, label='Datos BFS')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray']
    for i, group in enumerate(validated_groups):
        c = colors[i % len(colors)]
        pts = group['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"G{i+1} ({len(pts)}pts)")
        if group['slope'] is not None:
            tmin = pts['theta'].min()
            tmax = pts['theta'].max()
            t_lin = np.linspace(tmin, tmax, 100)
            r_lin = group['slope']*t_lin + group['intercept']
            plt.plot(t_lin, r_lin, '--', color=c)
    plt.xlabel("θ (grados)")
    plt.ylabel("r")
    #plt.legend(loc='best')
    plt.title("Grupos finales (validados)")
    plt.grid(True)
    plt.show()
    
    # Paso 4: ahora filtrar los grupos con >=60 puntos y graficarlos aparte
    plot_final_segments_over_60(validated_groups, df_filtrado, bounding_extrap=0.10)


main_example()



In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression

# ====================================
# 1) Funciones para Cargar/Filtrar Datos
# ====================================
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df['theta'] = np.where(df['theta'] < 0, df['theta'] + 360, df['theta'])
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r'] = df['r'].astype(float)
    return df

# ====================================
# 2) Construir Grafo (Rectangular BFS)
# ====================================
def build_graph_rectangular(df_points, theta_diff, r_diff):
    theta_arr = df_points['theta'].values
    r_arr = df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dtheta = np.abs(theta_arr[i] - theta_arr)
        dr = np.abs(r_arr[i] - r_arr)
        mask_neighbors = (dtheta <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i)
        neighbors = np.where(mask_neighbors)[0]
        graph[i] = list(neighbors)
    return graph, n

def bfs_components(graph, df_points):
    n = len(graph)
    visited = [False]*n
    clusters = []
    from collections import deque
    for start in range(n):
        if not visited[start]:
            queue = deque([start])
            visited[start] = True
            comp_indices = [start]
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        queue.append(v)
                        comp_indices.append(v)
            cluster_df = df_points.iloc[comp_indices].copy()
            clusters.append(cluster_df)
    return clusters

# ====================================
# 3) Subdividir por "gaps"
# ====================================
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    order_col = 'theta' if mode == 'theta' else 'r'
    df_cluster = df_cluster.sort_values(by=order_col)
    arr = df_cluster[order_col].values
    diffs = np.diff(arr)
    if np.all(diffs <= gap_threshold):
        return [df_cluster]
    subclusters = []
    start = 0
    for i, diff in enumerate(diffs):
        if diff > gap_threshold:
            subclusters.append(df_cluster.iloc[start:i+1].copy())
            start = i+1
    subclusters.append(df_cluster.iloc[start:].copy())
    return subclusters

# ====================================
# 4) Generar Semillas BFS
# ====================================
def generate_bfs_seeds(
    id_halo,
    theta_min=50,
    theta_max=250,
    quartile_threshold=0.155,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    threshold = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_filtered = df[df['rho_resta_final_exp'] > threshold].copy()
    df_filtered.reset_index(drop=True, inplace=True)
    
    graph, _ = build_graph_rectangular(df_filtered, theta_diff, r_diff)
    clusters = bfs_components(graph, df_filtered)
    
    final_clusters = []
    for clust in clusters:
        sub_theta = subdivide_by_gap(clust, gap_threshold=gap_threshold_theta, mode='theta')
        for st in sub_theta:
            sub_r = subdivide_by_gap(st, gap_threshold=gap_threshold_r, mode='r')
            final_clusters.extend(sub_r)
    
    return final_clusters, df_filtered

# ====================================
# 5) Ajustar rectas y Fusionar
# ====================================
def calculate_pa(slope):
    if slope is None:
        return None
    return np.degrees(np.arctan(slope))

def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return None, None, None, None, None, None, None
    X = df_cluster[['theta']].values
    y = df_cluster['r'].values
    model = LinearRegression()
    model.fit(X, y)
    slope_ = model.coef_[0]
    intercept_ = model.intercept_
    pa_ = calculate_pa(slope_)
    t_min = df_cluster['theta'].min()
    t_max = df_cluster['theta'].max()
    r_min = df_cluster['r'].min()
    r_max = df_cluster['r'].max()
    return slope_, intercept_, pa_, t_min, t_max, r_min, r_max

def adjust_and_merge_seeds(
    bfs_clusters,
    slope_variation_threshold=0.20,
    bounding_extrap=0.20
):
    groups = []
    for clust_df in bfs_clusters:
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(clust_df)
        grp = {
            'points': clust_df.copy(),
            'slope': slope_,
            'intercept': intercept_,
            'pa': pa_,
            'theta_min': tmin,
            'theta_max': tmax,
            'r_min': rmin,
            'r_max': rmax
        }
        groups.append(grp)
    
    def recalc_properties(group):
        dfp = group['points']
        slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(dfp)
        group['slope'] = slope_
        group['intercept'] = intercept_
        group['pa'] = pa_
        group['theta_min'] = tmin
        group['theta_max'] = tmax
        group['r_min'] = rmin
        group['r_max'] = rmax

    def boxes_overlap(g1, g2, extrap=0.30):
        tmin1, tmax1 = g1['theta_min'], g1['theta_max']
        rmin1, rmax1 = g1['r_min'], g1['r_max']
        tmin2, tmax2 = g2['theta_min'], g2['theta_max']
        rmin2, rmax2 = g2['r_min'], g2['r_max']
        if tmin1 is None or tmin2 is None:
            return False
        dt1 = (tmax1 - tmin1)
        dr1 = (rmax1 - rmin1)
        dt2 = (tmax2 - tmin2)
        dr2 = (rmax2 - rmin2)
        extr_dt1 = dt1*extrap
        extr_dr1 = dr1*extrap
        extr_dt2 = dt2*extrap
        extr_dr2 = dr2*extrap
        
        tmin1e = tmin1 - extr_dt1
        tmax1e = tmax1 + extr_dt1
        rmin1e = rmin1 - extr_dr1
        rmax1e = rmax1 + extr_dr1
        
        tmin2e = tmin2 - extr_dt2
        tmax2e = tmax2 + extr_dt2
        rmin2e = rmin2 - extr_dr2
        rmax2e = rmax2 + extr_dr2
        
        overlap_theta = not (tmax1e < tmin2e or tmax2e < tmin1e)
        overlap_r = not (rmax1e < rmin2e or rmax2e < rmin1e)
        return overlap_theta and overlap_r
    
    merged_flag = True
    while merged_flag:
        merged_flag = False
        new_groups = []
        i = 0
        while i < len(groups):
            g1 = groups[i]
            j = i+1
            fused_any = False
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    combined = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    slope_, intercept_, _, _, _, _, _ = fit_line_to_cluster(combined)
                    if slope_ is not None:
                        var1 = abs(slope_ - g1['slope'])/(abs(g1['slope'])+1e-12)
                        var2 = abs(slope_ - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if var1 < slope_variation_threshold and var2 < slope_variation_threshold:
                            g1['points'] = combined
                            recalc_properties(g1)
                            groups.pop(j)
                            fused_any = True
                            merged_flag = True
                            continue
                j += 1
            new_groups.append(g1)
            i += 1
        groups = new_groups
    
    final_groups = []
    for gg in groups:
        if len(gg['points']) >= 2:
            final_groups.append(gg)
    return final_groups

# ====================================
# 6) [Nuevo] Validar dispersión y re-procesar
# ====================================
def validate_dispersion_and_reprocess(
    groups,
    dispersion_threshold=2.0,   # umbral de desviación std (ejemplo)
    reproc_theta_diff=2.0,      # BFS para reprocesar
    reproc_r_diff=0.5
):
    """
    - Verifica la 'dispersión' (std de los residuos de cada grupo).
    - Si un grupo excede 'dispersion_threshold', se 'rompe':
       * Tomamos sus puntos
       * Reaplicamos BFS (o un mini-proceso) a ese subconjunto
       * Ajustamos rectas otra vez
    - Devuelve la lista final de grupos (los que no se rompieron + 
      los subgrupos reagrupados).
    """
    # 1) Separar grupos "buenos" de grupos "a re-procesar"
    groups_ok = []
    groups_toreprocess = []
    
    for grp in groups:
        slope = grp['slope']
        intercept = grp['intercept']
        pts = grp['points']
        if slope is None or len(pts) < 2:
            # Grupo muy pequeño -> considerarlo "ok" sin dispersión
            groups_ok.append(grp)
            continue
        
        # Calcular residuos
        predicted_r = slope*pts['theta'] + intercept
        residuals = pts['r'] - predicted_r
        std_resid = residuals.std()  # desviación estándar
        # Checar si std_resid > threshold
        if std_resid > dispersion_threshold:
            groups_toreprocess.append(grp)
        else:
            groups_ok.append(grp)
    
    # 2) Re-procesar los que superaron la dispersión
    #    Por ejemplo, usando BFS de nuevo
    new_groups = []
    for grp in groups_toreprocess:
        # Tomar sus puntos:
        subdf = grp['points'].copy()
        subdf.reset_index(drop=True, inplace=True)
        # BFS con reproc_theta_diff, reproc_r_diff
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        clusters_sub = bfs_components(graph, subdf)
        
        # A) Ajustamos recta a cada mini-cluster (sin fusión en este ejemplo)
        #    O podrías re-llamar a adjust_and_merge_seeds con slope_variation_threshold,
        #    si quieres un pipeline más largo.
        
        for csub in clusters_sub:
            slope_, intercept_, pa_, tmin, tmax, rmin, rmax = fit_line_to_cluster(csub)
            new_groups.append({
                'points': csub,
                'slope': slope_,
                'intercept': intercept_,
                'pa': pa_,
                'theta_min': tmin,
                'theta_max': tmax,
                'r_min': rmin,
                'r_max': rmax
            })
    
    # 3) Unir "groups_ok" + "new_groups" en la lista final
    final_res = groups_ok + new_groups
    
    return final_res


# ====================================
# 7) Función principal que muestra TODO
# ====================================
def main_example():
    # 1) Generar BFS
    bfs_clusters, df_filtrado = generate_bfs_seeds(
        id_halo="11",
        theta_min=20,
        theta_max=350,
        quartile_threshold=0.55,
        theta_diff=3.0,
        r_diff=0.5,
        gap_threshold_theta=1.80,
        gap_threshold_r=1.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs_clusters)} grupos")
    
    # 2) Ajustar y fusionar
    merged_groups = adjust_and_merge_seeds(
        bfs_clusters,
        slope_variation_threshold=0.40,
        bounding_extrap=0.30
    )
    print(f"Grupos tras fusión inicial: {len(merged_groups)}")
    
    # 3) Validar dispersión y re-procesar grupos con alta dispersión
    #    Por ejemplo, dispersion_threshold=2.0
    validated_groups = validate_dispersion_and_reprocess(
        merged_groups,
        dispersion_threshold=0.6920, # si la std del residuo >2 -> re-proceso BFS
        reproc_theta_diff=1.05,
        reproc_r_diff=0.4
    )
    print(f"Grupos tras validación dispersión: {len(validated_groups)}")
    
    # 4) Graficar
    plt.figure(figsize=(10,6))
    plt.scatter(df_filtrado['theta'], df_filtrado['r'], s=3, alpha=0.3, label='Datos BFS')
    
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray']
    for i, group in enumerate(validated_groups):
        c = colors[i % len(colors)]
        pts = group['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"G{i+1}_{len(pts)}")
        if group['slope'] is not None:
            tmin = pts['theta'].min()
            tmax = pts['theta'].max()
            t_lin = np.linspace(tmin, tmax, 50)
            r_lin = group['slope']*t_lin + group['intercept']
            plt.plot(t_lin, r_lin, '--', color=c)
    plt.xlabel("theta (grados)")
    plt.ylabel("r")
    plt.legend(loc='upper right')
    plt.title("BFS + Fusión + Validación de Dispersión (reprocesado)")
    plt.grid(True)
    plt.show()


# ====================================
# Ejecución:
# main_example()
# ====================================


In [0]:
main_example()

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Algoritmo mejorado (versión subhalo 11) con validación adaptativa
para datasets > 2000 puntos y visualizaciones completas.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression


# ═══════════════════════════════════════════════════════════════════════
# 1) Cargar y filtrar datos
# ═══════════════════════════════════════════════════════════════════════
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r']     = df['r'].astype(float)
    return df


# ═══════════════════════════════════════════════════════════════════════
# 2) Grafo rectangular (BFS)
# ═══════════════════════════════════════════════════════════════════════
def build_graph_rectangular(df_points, theta_diff, r_diff):
    th = df_points['theta'].values
    rr = df_points['r'].values
    n  = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        dth = np.abs(th[i] - th)
        dr  = np.abs(rr[i] - rr)
        mask = (dth <= theta_diff) & (dr <= r_diff) & (np.arange(n) != i)
        graph[i] = list(np.where(mask)[0])
    return graph, n


def bfs_components(graph, df_points):
    n, visited, clusters = len(graph), [False]*len(graph), []
    for s in range(n):
        if not visited[s]:
            q = deque([s]); visited[s] = True; comp = [s]
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters


# ═══════════════════════════════════════════════════════════════════════
# 3) Subdividir por “gaps”
# ═══════════════════════════════════════════════════════════════════════
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    col   = 'theta' if mode == 'theta' else 'r'
    df    = df_cluster.sort_values(col)
    diffs = np.diff(df[col].values)
    if np.all(diffs <= gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(diffs):
        if d > gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i + 1
    subs.append(df.iloc[start:].copy())
    return subs


# ═══════════════════════════════════════════════════════════════════════
# 4) Generar semillas BFS  (validación adaptativa)
# ═══════════════════════════════════════════════════════════════════════
def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts / ref, 1.0))


def generate_bfs_seeds(
    id_halo,
    theta_min=50,
    theta_max=250,
    quartile_threshold=0.155,
    theta_diff=3.0,
    r_diff=0.5,
    gap_threshold_theta=2.0,
    gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp'] > thr].copy().reset_index(drop=True)

    # ─── ajuste adaptativo ──────────────────────────────────────────────
    fct                 = _adaptive_factor(len(df_f))
    theta_diff_eff      = theta_diff / fct
    r_diff_eff          = r_diff     / fct
    gap_theta_eff       = gap_threshold_theta / fct
    gap_r_eff           = gap_threshold_r     / fct

    graph, _  = build_graph_rectangular(df_f, theta_diff_eff, r_diff_eff)
    clusters  = bfs_components(graph, df_f)

    final_cl  = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_theta_eff, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r_eff, 'r'))

    return final_cl, df_f


# ═══════════════════════════════════════════════════════════════════════
# 5) Ajustar rectas y fusionar
# ═══════════════════════════════════════════════════════════════════════
def calculate_pa(slope):
    return None if slope is None else np.degrees(np.arctan(slope))


def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return (None,)*7
    X = df_cluster[['theta']].values
    y = df_cluster['r'].values
    mdl = LinearRegression().fit(X, y)
    s, b = mdl.coef_[0], mdl.intercept_
    tmin, tmax = df_cluster['theta'].min(), df_cluster['theta'].max()
    rmin, rmax = df_cluster['r'].min(),    df_cluster['r'].max()
    return s, b, calculate_pa(s), tmin, tmax, rmin, rmax


def adjust_and_merge_seeds(
    bfs_clusters,
    slope_variation_threshold=0.20,
    bounding_extrap=0.20
):
    groups = []
    for cl in bfs_clusters:
        s, b, pa, tmin, tmax, rmin, rmax = fit_line_to_cluster(cl)
        groups.append({'points': cl, 'slope': s, 'intercept': b, 'pa': pa,
                       'theta_min': tmin, 'theta_max': tmax, 'r_min': rmin, 'r_max': rmax})

    def recalc(g):
        s, b, pa, tmin, tmax, rmin, rmax = fit_line_to_cluster(g['points'])
        g.update({'slope': s, 'intercept': b, 'pa': pa,
                  'theta_min': tmin, 'theta_max': tmax,
                  'r_min': rmin, 'r_max': rmax})

    def boxes_overlap(g1, g2, extr):
        def exp(tmin, tmax, rmin, rmax):
            dt, dr = (tmax-tmin)*extr, (rmax-rmin)*extr
            return tmin-dt, tmax+dt, rmin-dr, rmax+dr
        t1l, t1u, r1l, r1u = exp(g1['theta_min'], g1['theta_max'], g1['r_min'], g1['r_max'])
        t2l, t2u, r2l, r2u = exp(g2['theta_min'], g2['theta_max'], g2['r_min'], g2['r_max'])
        return not (t1u < t2l or t2u < t1l) and not (r1u < r2l or r2u < r1l)

    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i+1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1; continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb - g1['slope'])/(abs(g1['slope'])+1e-12)
                        v2 = abs(s_comb - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb; recalc(g1)
                            groups.pop(j); merged = True; continue
                j += 1
            new.append(g1); i += 1
        groups = new
    return [g for g in groups if len(g['points']) >= 2]


# ═══════════════════════════════════════════════════════════════════════
# 6) Validar dispersión y re‑procesar
# ═══════════════════════════════════════════════════════════════════════
def validate_dispersion_and_reprocess(
    groups,
    dispersion_threshold=0.6920,
    reproc_theta_diff=1.05,
    reproc_r_diff=0.4
):
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points']) < 2:
            ok.append(g); continue
        res = g['points']['r'] - (g['slope']*g['points']['theta'] + g['intercept'])
        (todo if res.std() > dispersion_threshold else ok).append(g)

    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            s, b, pa, tmin, tmax, rmin, rmax = fit_line_to_cluster(c)
            new.append({'points': c, 'slope': s, 'intercept': b, 'pa': pa,
                        'theta_min': tmin, 'theta_max': tmax, 'r_min': rmin, 'r_max': rmax})
    return ok + new


# ═══════════════════════════════════════════════════════════════════════
# 7) Plot de grupos grandes con recta extrapolada
# ═══════════════════════════════════════════════════════════════════════
def plot_final_segments_over_60_with_line_extrapolation(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i%len(colors)], g['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"G{i+1} ({len(pts)})")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t1 = g['theta_min'] - line_extrap*dt
            t2 = g['theta_max'] + line_extrap*dt
            t = np.linspace(t1, t2, 200)
            plt.plot(t, g['slope']*t + g['intercept'], '--', color=c)
    plt.xlabel("θ (°)"); plt.ylabel("r")
    plt.title("Grupos ≥60 puntos – extrapolación 15 %")
    plt.legend(); plt.grid(True); plt.show()


# ═══════════════════════════════════════════════════════════════════════
# 8) Ejecución de ejemplo completo (subhalo 11)
# ═══════════════════════════════════════════════════════════════════════
def main_example():
    # A) BFS + subdivisión adaptativa
    bfs, df_f = generate_bfs_seeds(
        id_halo="11",
        theta_min=20,
        theta_max=350,
        quartile_threshold=0.55,
        theta_diff=3.0,
        r_diff=0.5,
        gap_threshold_theta=1.80,
        gap_threshold_r=1.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")

    # B) Fusionar
    merged = adjust_and_merge_seeds(
        bfs,
        slope_variation_threshold=0.40,   # llamada con 0.40
        bounding_extrap=0.30
    )
    print(f"Tras fusión: {len(merged)} grupos")

    # C) Validar dispersión
    validated = validate_dispersion_and_reprocess(
        merged,
        dispersion_threshold=0.6920,
        reproc_theta_diff=1.05,
        reproc_r_diff=0.4
    )
    print(f"Tras validación: {len(validated)} grupos")

    # D) Gráfico global
    plt.figure(figsize=(10,6))
    plt.scatter(df_f['theta'], df_f['r'], s=3, alpha=0.3, label='Datos filtrados')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray']
    for i, g in enumerate(validated):
        c, pts = colors[i%len(colors)], g['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"G{i+1} ({len(pts)})")
        if g['slope'] is not None:
            t = np.linspace(pts['theta'].min(), pts['theta'].max(), 80)
            plt.plot(t, g['slope']*t + g['intercept'], '--', color=c)
    plt.xlabel("θ (°)"); plt.ylabel("r")
    plt.title("Subhalo 11 – grupos finales"); plt.legend(loc='upper right')
    plt.grid(True); plt.show()

    # E) Grupos grandes con extrapolación
    plot_final_segments_over_60_with_line_extrapolation(validated, df_f, line_extrap=0.15)


# ═══════════════════════════════════════════════════════════════════════
# 9) Llamada a main
# ═══════════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    main_example()


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
###############################################################################
#                           🌌  SUBHALO 11 – VERSIÓN FINAL  🌌
#  ░█▀▀░█░█░█▀▀░█▀▀░█░█░█▀█░█░█   INCLUYE:
#  ░█░░░░█░░█▀▀░█░░░█▀▄░█▀█░█▀█   1) VALIDACIÓN ADAPTATIVA > 2000 PUNTOS
#  ░▀▀▀░░▀░░▀▀▀░▀▀▀░▀░▀░▀░▀░▀░▀   2) PLOTS CARTESIANO Y **¡¡NUEVO!!** POLAR
###############################################################################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression


# ═══════════════════════════════════════════════════════════════════════
# 1) Cargar y filtrar datos
# ═══════════════════════════════════════════════════════════════════════
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r']     = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r']     = df['r'].astype(float)
    return df


# ═══════════════════════════════════════════════════════════════════════
# 2) Grafo rectangular (BFS)
# ═══════════════════════════════════════════════════════════════════════
def build_graph_rectangular(df_points, theta_diff, r_diff):
    th, rr = df_points['theta'].values, df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        mask = (np.abs(th[i] - th) <= theta_diff) & (np.abs(rr[i] - rr) <= r_diff) & (np.arange(n) != i)
        graph[i] = list(np.where(mask)[0])
    return graph, n


def bfs_components(graph, df_points):
    visited, clusters = [False]*len(graph), []
    for s in range(len(graph)):
        if not visited[s]:
            q, comp = deque([s]), [s]
            visited[s] = True
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters


# ═══════════════════════════════════════════════════════════════════════
# 3) Subdividir por “gaps”
# ═══════════════════════════════════════════════════════════════════════
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    col = 'theta' if mode == 'theta' else 'r'
    df  = df_cluster.sort_values(col)
    dif = np.diff(df[col].values)
    if np.all(dif <= gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(dif):
        if d > gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i + 1
    subs.append(df.iloc[start:].copy())
    return subs


# ═══════════════════════════════════════════════════════════════════════
# 4) Generar semillas BFS  (💥 VALIDACIÓN ADAPTATIVA 💥)
# ═══════════════════════════════════════════════════════════════════════
def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts / ref, 1.0))


def generate_bfs_seeds(
    id_halo,
    theta_min=50, theta_max=250,
    quartile_threshold=0.155,
    theta_diff=3.0, r_diff=0.5,
    gap_threshold_theta=2.0, gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp'] > thr].copy().reset_index(drop=True)

    f = _adaptive_factor(len(df_f))                    # ←—— FACTOR ADAPTATIVO
    th_diff, r_d = theta_diff/f, r_diff/f
    gap_th, gap_r = gap_threshold_theta/f, gap_threshold_r/f

    graph, _ = build_graph_rectangular(df_f, th_diff, r_d)
    clusters = bfs_components(graph, df_f)

    final_cl = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_th, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r, 'r'))
    return final_cl, df_f


# ═══════════════════════════════════════════════════════════════════════
# 5) Ajustar rectas y fusionar
# ═══════════════════════════════════════════════════════════════════════
def calculate_pa(slope):
    return None if slope is None else np.degrees(np.arctan(slope))


def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return (None,)*7
    mdl = LinearRegression().fit(df_cluster[['theta']], df_cluster['r'])
    s, b = mdl.coef_[0], mdl.intercept_
    return s, b, calculate_pa(s), df_cluster['theta'].min(), df_cluster['theta'].max(), df_cluster['r'].min(), df_cluster['r'].max()


def adjust_and_merge_seeds(bfs_clusters, slope_variation_threshold=0.20, bounding_extrap=0.20):
    groups = []
    for cl in bfs_clusters:
        groups.append(dict(zip(
            ['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
            (*fit_line_to_cluster(cl), cl)
        )))

    def recalc(g):
        g['slope'], g['intercept'], g['pa'], g['theta_min'], g['theta_max'], g['r_min'], g['r_max'] = \
            fit_line_to_cluster(g['points'])

    def boxes_overlap(g1, g2, e):
        def exp(t0,t1,r0,r1):
            dt, dr = (t1-t0)*e, (r1-r0)*e
            return t0-dt, t1+dt, r0-dr, r1+dr
        a = exp(g1['theta_min'],g1['theta_max'],g1['r_min'],g1['r_max'])
        b = exp(g2['theta_min'],g2['theta_max'],g2['r_min'],g2['r_max'])
        return not (a[1]<b[0] or b[1]<a[0]) and not (a[3]<b[2] or b[3]<a[2])

    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i+1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1; continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb-g1['slope'])/(abs(g1['slope'])+1e-12)
                        v2 = abs(s_comb-g2['slope'])/(abs(g2['slope'])+1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb; recalc(g1)
                            groups.pop(j); merged = True; continue
                j += 1
            new.append(g1); i += 1
        groups = new
    return [g for g in groups if len(g['points']) >= 2]


# ═══════════════════════════════════════════════════════════════════════
# 6) Validar dispersión y re‑procesar
# ═══════════════════════════════════════════════════════════════════════
def validate_dispersion_and_reprocess(groups, dispersion_threshold=0.6920, reproc_theta_diff=1.05, reproc_r_diff=0.4):
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points']) < 2:
            ok.append(g); continue
        res = g['points']['r'] - (g['slope']*g['points']['theta'] + g['intercept'])
        (todo if res.std() > dispersion_threshold else ok).append(g)

    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph,_ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            new.append(dict(zip(
                ['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
                (*fit_line_to_cluster(c), c)
            )))
    return ok + new


# ═══════════════════════════════════════════════════════════════════════
# 7‑A) Plot cartesiano grupos grandes
# ═══════════════════════════════════════════════════════════════════════
def plot_final_segments_over_60(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i%len(colors)], g['points']
        plt.scatter(pts['theta'], pts['r'], s=10, color=c, label=f"G{i+1} ({len(pts)})")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            plt.plot(t, g['slope']*t + g['intercept'], '--', color=c)
    plt.xlabel("θ (°)"); plt.ylabel("r"); plt.grid(True)
    plt.title("Grupos ≥60 puntos – extrapolación 15 %")
    plt.legend(); plt.show()


# ═══════════════════════════════════════════════════════════════════════
# 7‑B) ⭐ NUEVO ⭐  PLOT POLAR grupos grandes
# ═══════════════════════════════════════════════════════════════════════
def plot_final_segments_over_60_polar(groups, df_all, line_extrap=0.15):
    """
    🔭 Representa los grupos (≥60 pts) en coordenadas polares.
    """
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos (polar): {len(big)}")

    fig = plt.figure(figsize=(9,8))
    ax  = plt.subplot(111, projection='polar')

    ax.scatter(np.radians(df_all['theta']), df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')

    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i%len(colors)], g['points']
        ax.scatter(np.radians(pts['theta']), pts['r'], s=10, color=c, label=f"G{i+1} ({len(pts)})")

        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t_deg = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            r_line = g['slope']*t_deg + g['intercept']
            ax.plot(np.radians(t_deg), r_line, '--', color=c)

    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title("Grupos ≥60 puntos – proyección polar (15 % extrap.)", y=1.05)
    ax.legend(loc='upper right', bbox_to_anchor=(1.15,1.0))
    plt.show()


# ═══════════════════════════════════════════════════════════════════════
# 8) MAIN – pipeline completo subhalo 11
# ═══════════════════════════════════════════════════════════════════════
def main_example():
    bfs, df_f = generate_bfs_seeds(
        id_halo="11",
        theta_min=20, theta_max=350,
        quartile_threshold=0.55,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=1.80, gap_threshold_r=1.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")

    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.40, bounding_extrap=0.30)
    print(f"Tras fusión: {len(merged)} grupos")

    validated = validate_dispersion_and_reprocess(merged, dispersion_threshold=0.6920,
                                                  reproc_theta_diff=1.05, reproc_r_diff=0.4)
    print(f"Tras validación: {len(validated)} grupos")

    # ─── Visualizaciones ────────────────────────────────────────────────
    plot_final_segments_over_60(validated, df_f, line_extrap=0.15)   # cartesiano
    plot_final_segments_over_60_polar(validated, df_f, line_extrap=0.15)  # ⭐ NUEVO ⭐


# ═══════════════════════════════════════════════════════════════════════
# 9) Run
# ═══════════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    main_example()


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
###############################################################################
###############################################################################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression


# ═══════════════════════════════════════════════════════════════════════
# 0)  PITCH‑ANGLE  (versión solicitada)  🔥
# ═══════════════════════════════════════════════════════════════════════
def calculate_pa(slope, intercept):
    """
    Pitch‑angle en grados según la fórmula proporcionada por el usuario:
        PA = arctan[(slope·180/π) / intercept]   (si intercept ≠ 0)
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan


# ═══════════════════════════════════════════════════════════════════════
# 1) Cargar y filtrar datos
# ═══════════════════════════════════════════════════════════════════════
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    df['r']     = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r']     = df['r'].astype(float)
    return df


# ═══════════════════════════════════════════════════════════════════════
# 2) Grafo rectangular (BFS)
# ═══════════════════════════════════════════════════════════════════════
def build_graph_rectangular(df_points, theta_diff, r_diff):
    th, rr = df_points['theta'].values, df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        mask = (np.abs(th[i] - th) <= theta_diff) & (np.abs(rr[i] - rr) <= r_diff) & (np.arange(n) != i)
        graph[i] = list(np.where(mask)[0])
    return graph, n


def bfs_components(graph, df_points):
    visited, clusters = [False]*len(graph), []
    for s in range(len(graph)):
        if not visited[s]:
            q, comp = deque([s]), [s]
            visited[s] = True
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters


# ═══════════════════════════════════════════════════════════════════════
# 3) Subdividir por “gaps”
# ═══════════════════════════════════════════════════════════════════════
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    col = 'theta' if mode == 'theta' else 'r'
    df  = df_cluster.sort_values(col)
    dif = np.diff(df[col].values)
    if np.all(dif <= gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(dif):
        if d > gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i + 1
    subs.append(df.iloc[start:].copy())
    return subs


# ═══════════════════════════════════════════════════════════════════════
# 4) Generar semillas BFS  (💥 VALIDACIÓN ADAPTATIVA 💥)
# ═══════════════════════════════════════════════════════════════════════
def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts / ref, 1.0))


def generate_bfs_seeds(
    id_halo,
    theta_min=50, theta_max=250,
    quartile_threshold=0.155,
    theta_diff=3.0, r_diff=0.5,
    gap_threshold_theta=2.0, gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp'] > thr].copy().reset_index(drop=True)

    f = _adaptive_factor(len(df_f))
    th_diff, r_d = theta_diff/f, r_diff/f
    gap_th, gap_r = gap_threshold_theta/f, gap_threshold_r/f

    graph, _ = build_graph_rectangular(df_f, th_diff, r_d)
    clusters = bfs_components(graph, df_f)

    final_cl = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_th, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r, 'r'))
    return final_cl, df_f


# ═══════════════════════════════════════════════════════════════════════
# 5) Ajustar rectas y fusionar
# ═══════════════════════════════════════════════════════════════════════
def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return (None,)*7
    mdl = LinearRegression().fit(df_cluster[['theta']], df_cluster['r'])
    s, b = mdl.coef_[0], mdl.intercept_
    pa   = calculate_pa(s, b)
    return s, b, pa, df_cluster['theta'].min(), df_cluster['theta'].max(), df_cluster['r'].min(), df_cluster['r'].max()


def adjust_and_merge_seeds(bfs_clusters, slope_variation_threshold=0.20, bounding_extrap=0.20):
    groups = []
    for cl in bfs_clusters:
        groups.append(dict(zip(
            ['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
            (*fit_line_to_cluster(cl), cl)
        )))

    def recalc(g):
        g['slope'], g['intercept'], g['pa'], g['theta_min'], g['theta_max'], g['r_min'], g['r_max'] = \
            fit_line_to_cluster(g['points'])

    def boxes_overlap(g1, g2, e):
        def exp(t0,t1,r0,r1):
            dt, dr = (t1-t0)*e, (r1-r0)*e
            return t0-dt, t1+dt, r0-dr, r1+dr
        a = exp(g1['theta_min'],g1['theta_max'],g1['r_min'],g1['r_max'])
        b = exp(g2['theta_min'],g2['theta_max'],g2['r_min'],g2['r_max'])
        return not (a[1]<b[0] or b[1]<a[0]) and not (a[3]<b[2] or b[3]<a[2])

    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i+1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1; continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, b_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb-g1['slope'])/(abs(g1['slope'])+1e-12)
                        v2 = abs(s_comb-g2['slope'])/(abs(g2['slope'])+1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb; recalc(g1)
                            groups.pop(j); merged = True; continue
                j += 1
            new.append(g1); i += 1
        groups = new
    return [g for g in groups if len(g['points']) >= 2]


# ═══════════════════════════════════════════════════════════════════════
# 6) Validar dispersión y re‑procesar
# ═══════════════════════════════════════════════════════════════════════
def validate_dispersion_and_reprocess(groups, dispersion_threshold=0.6920, reproc_theta_diff=1.05, reproc_r_diff=0.4):
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points']) < 2:
            ok.append(g); continue
        res = g['points']['r'] - (g['slope']*g['points']['theta'] + g['intercept'])
        (todo if res.std() > dispersion_threshold else ok).append(g)

    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph,_ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            new.append(dict(zip(
                ['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
                (*fit_line_to_cluster(c), c)
            )))
    return ok + new


# ═══════════════════════════════════════════════════════════════════════
# 7‑A) Plot cartesiano grupos grandes (PA visible)
# ═══════════════════════════════════════════════════════════════════════
def plot_final_segments_over_60(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i%len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        plt.scatter(pts['theta'], pts['r'], s=10, color=c,
                    label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            plt.plot(t, g['slope']*t + g['intercept'], '--', color=c)
    plt.xlabel("θ (°)"); plt.ylabel("r"); plt.grid(True)
    plt.title("Grupos ≥60 puntos – extrapolación 15 % (con PA)")
    plt.legend(); plt.show()


# ═══════════════════════════════════════════════════════════════════════
# 7‑B) Plot POLAR grupos grandes (PA visible)
# ═══════════════════════════════════════════════════════════════════════
def plot_final_segments_over_60_polar(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos (polar): {len(big)}")

    fig = plt.figure(figsize=(9,8))
    ax  = plt.subplot(111, projection='polar')

    ax.scatter(np.radians(df_all['theta']), df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')

    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i%len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        ax.scatter(np.radians(pts['theta']), pts['r'], s=10, color=c,
                   label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")

        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t_deg = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            r_line = g['slope']*t_deg + g['intercept']
            ax.plot(np.radians(t_deg), r_line, '.-', color="gray", alpha=0.20)
            ax.plot(np.radians(t_deg), r_line, '--', color=c)

    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title("Grupos ≥60 puntos – proyección polar (15 % extrap.)", y=1.05)
    ax.legend(loc='upper right', bbox_to_anchor=(1.20,1.0))
    plt.show()


# ═══════════════════════════════════════════════════════════════════════
# 8) MAIN – pipeline completo subhalo 11
# ═══════════════════════════════════════════════════════════════════════
def main_example():
    bfs, df_f = generate_bfs_seeds(
        id_halo="11",
        theta_min=20, theta_max=350,
        quartile_threshold=0.55,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=1.80, gap_threshold_r=1.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")

    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.40, bounding_extrap=0.30)
    print(f"Tras fusión: {len(merged)} grupos")

    validated = validate_dispersion_and_reprocess(merged, dispersion_threshold=0.6920,
                                                  reproc_theta_diff=1.05, reproc_r_diff=0.4)
    print(f"Tras validación: {len(validated)} grupos")

    # ─── Visualizaciones ────────────────────────────────────────────────
    plot_final_segments_over_60(validated, df_f, line_extrap=0.15)        # cartesiano
    plot_final_segments_over_60_polar(validated, df_f, line_extrap=0.15)  # polar


# ═══════════════════════════════════════════════════════════════════════
# 9) Run
# ═══════════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    main_example()



### halo 11 version camino

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
###############################################################################
###############################################################################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import LinearRegression

# ═══════════════════════════════════════════════════════════════════════
# 0)  PITCH‑ANGLE 
# ═══════════════════════════════════════════════════════════════════════
def calculate_pa(slope, intercept):
    """
    PA = arctan[(slope·180/π) / intercept]  en grados, si intercept ≠ 0.
    Devuelve NaN si intercept == 0.
    """
    if intercept != 0:
        return np.degrees(np.arctan((slope * (180 / np.pi)) / intercept))
    return np.nan


# ═══════════════════════════════════════════════════════════════════════
# 1) Funciones para Cargar/Filtrar Datos
# ═══════════════════════════════════════════════════════════════════════
def load_and_filter_data(id_halo, theta_min, theta_max, file_prefix='data_rho'):
    df = pd.read_csv(f"{file_prefix}_{id_halo}_filtered.csv")
    # Agregar columna de id para preservar el índice original
    df['id'] = df.index
    df['r']     = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))
    df.loc[df['theta'] < 0, 'theta'] += 360
    df = df[(df['theta'] >= theta_min) & (df['theta'] <= theta_max)].copy()
    df['theta'] = df['theta'].astype(float)
    df['r']     = df['r'].astype(float)
    return df


# ═══════════════════════════════════════════════════════════════════════
# 2) Construir Grafo (Rectangular BFS)
# ═══════════════════════════════════════════════════════════════════════
def build_graph_rectangular(df_points, theta_diff, r_diff):
    th, rr = df_points['theta'].values, df_points['r'].values
    n = len(df_points)
    graph = [[] for _ in range(n)]
    for i in range(n):
        mask = (np.abs(th[i] - th) <= theta_diff) & (np.abs(rr[i] - rr) <= r_diff) & (np.arange(n) != i)
        graph[i] = list(np.where(mask)[0])
    return graph, n


def bfs_components(graph, df_points):
    visited, clusters = [False]*len(graph), []
    for s in range(len(graph)):
        if not visited[s]:
            q, comp = deque([s]), [s]
            visited[s] = True
            while q:
                u = q.popleft()
                for v in graph[u]:
                    if not visited[v]:
                        visited[v] = True
                        q.append(v)
                        comp.append(v)
            clusters.append(df_points.iloc[comp].copy())
    return clusters


# ═══════════════════════════════════════════════════════════════════════
# 3) Subdividir por "gaps"
# ═══════════════════════════════════════════════════════════════════════
def subdivide_by_gap(df_cluster, gap_threshold=2.5, mode='theta'):
    col = 'theta' if mode == 'theta' else 'r'
    df  = df_cluster.sort_values(col)
    dif = np.diff(df[col].values)
    if np.all(dif <= gap_threshold):
        return [df]
    subs, start = [], 0
    for i, d in enumerate(dif):
        if d > gap_threshold:
            subs.append(df.iloc[start:i+1].copy())
            start = i + 1
    subs.append(df.iloc[start:].copy())
    return subs


# ═══════════════════════════════════════════════════════════════════════
# 4) Generar Semillas BFS  (validación adaptativa)
# ═══════════════════════════════════════════════════════════════════════
def _adaptive_factor(n_pts, ref=2000):
    return np.sqrt(max(n_pts / ref, 1.0))


def generate_bfs_seeds(
    id_halo,
    theta_min=50, theta_max=250,
    quartile_threshold=0.55,
    theta_diff=3.0, r_diff=0.5,
    gap_threshold_theta=2.0, gap_threshold_r=2.0,
    file_prefix='data_rho'
):
    df = load_and_filter_data(id_halo, theta_min, theta_max, file_prefix)
    thr = df['rho_resta_final_exp'].quantile(quartile_threshold)
    df_f = df[df['rho_resta_final_exp'] > thr].copy().reset_index(drop=True)

    f = _adaptive_factor(len(df_f))
    th_diff, r_d = theta_diff/f, r_diff/f
    gap_th, gap_r = gap_threshold_theta/f, gap_threshold_r/f

    graph, _ = build_graph_rectangular(df_f, th_diff, r_d)
    clusters = bfs_components(graph, df_f)

    final_cl = []
    for cl in clusters:
        for st in subdivide_by_gap(cl, gap_th, 'theta'):
            final_cl.extend(subdivide_by_gap(st, gap_r, 'r'))
    return final_cl, df_f


# ═══════════════════════════════════════════════════════════════════════
# 5) Ajustar rectas y Fusionar
# ═══════════════════════════════════════════════════════════════════════
def fit_line_to_cluster(df_cluster):
    if len(df_cluster) < 2:
        return (None,)*7
    mdl = LinearRegression().fit(df_cluster[['theta']], df_cluster['r'])
    s, b = mdl.coef_[0], mdl.intercept_
    pa   = calculate_pa(s, b)
    return s, b, pa, df_cluster['theta'].min(), df_cluster['theta'].max(), df_cluster['r'].min(), df_cluster['r'].max()


def adjust_and_merge_seeds(bfs_clusters, slope_variation_threshold=0.40, bounding_extrap=0.30):
    groups = []
    for cl in bfs_clusters:
        groups.append(dict(zip(
            ['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
            (*fit_line_to_cluster(cl), cl)
        )))

    def recalc(g):
        g['slope'], g['intercept'], g['pa'], g['theta_min'], g['theta_max'], g['r_min'], g['r_max'] = \
            fit_line_to_cluster(g['points'])

    def boxes_overlap(g1, g2, e):
        def exp(t0,t1,r0,r1):
            dt, dr = (t1-t0)*e, (r1-r0)*e
            return t0-dt, t1+dt, r0-dr, r1+dr
        a = exp(g1['theta_min'],g1['theta_max'],g1['r_min'],g1['r_max'])
        b = exp(g2['theta_min'],g2['theta_max'],g2['r_min'],g2['r_max'])
        return not (a[1]<b[0] or b[1]<a[0]) and not (a[3]<b[2] or b[3]<a[2])

    merged = True
    while merged:
        merged, new = False, []
        i = 0
        while i < len(groups):
            g1, j = groups[i], i+1
            while j < len(groups):
                g2 = groups[j]
                if g1['slope'] is None or g2['slope'] is None:
                    j += 1
                    continue
                if boxes_overlap(g1, g2, bounding_extrap):
                    comb = pd.concat([g1['points'], g2['points']], ignore_index=True)
                    s_comb, b_comb, *_ = fit_line_to_cluster(comb)
                    if s_comb is not None:
                        v1 = abs(s_comb - g1['slope'])/(abs(g1['slope'])+1e-12)
                        v2 = abs(s_comb - g2['slope'])/(abs(g2['slope'])+1e-12)
                        if v1 < slope_variation_threshold and v2 < slope_variation_threshold:
                            g1['points'] = comb
                            recalc(g1)
                            groups.pop(j)
                            merged = True
                            continue
                j += 1
            new.append(g1)
            i += 1
        groups = new
    return [g for g in groups if len(g['points']) >= 2]


# ═══════════════════════════════════════════════════════════════════════
# 6) Validar dispersión y re‑procesar
# ═══════════════════════════════════════════════════════════════════════
def validate_dispersion_and_reprocess(groups, dispersion_threshold=1.80, reproc_theta_diff=1.25, reproc_r_diff=0.4):
    ok, todo = [], []
    for g in groups:
        if g['slope'] is None or len(g['points']) < 2:
            ok.append(g)
            continue
        res = g['points']['r'] - (g['slope'] * g['points']['theta'] + g['intercept'])
        (todo if res.std() > dispersion_threshold else ok).append(g)

    new = []
    for g in todo:
        subdf = g['points'].reset_index(drop=True)
        graph, _ = build_graph_rectangular(subdf, reproc_theta_diff, reproc_r_diff)
        for c in bfs_components(graph, subdf):
            new.append(dict(zip(
                ['slope','intercept','pa','theta_min','theta_max','r_min','r_max','points'],
                (*fit_line_to_cluster(c), c)
            )))
    return ok + new


# ═══════════════════════════════════════════════════════════════════════
# 7‑A) Plot cartesiano grupos grandes (PA visible)
# ═══════════════════════════════════════════════════════════════════════
def plot_groups_cartesian(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos: {len(big)}")
    plt.figure(figsize=(10,6))
    plt.scatter(df_all['theta'], df_all['r'], s=3, alpha=0.3, color='gray')
    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        plt.scatter(pts['theta'], pts['r'], s=10, color=c,
                    label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")
        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t = np.linspace(g['theta_min']-line_extrap*dt, g['theta_max']+line_extrap*dt, 200)
            plt.plot(t, g['slope'] * t + g['intercept'], '--', color=c)
    plt.xlabel("θ (°)")
    plt.ylabel("r")
    plt.grid(True)
    plt.title("Subhalo 17 – grupos ≥60 pts (cartesiano, PA visible)")
    plt.legend()
    plt.show()


# ═══════════════════════════════════════════════════════════════════════
# 7‑B) Plot POLAR grupos grandes (PA visible)
# ═══════════════════════════════════════════════════════════════════════
def plot_groups_polar(groups, df_all, line_extrap=0.15):
    big = [g for g in groups if len(g['points']) >= 60]
    print(f"Grupos ≥60 puntos (polar): {len(big)}")

    fig = plt.figure(figsize=(9,8))
    ax  = plt.subplot(111, projection='polar')
    ax.scatter(np.radians(df_all['theta']), df_all['r'], s=3, alpha=0.3, color='gray', label='Datos base')

    colors = ['red','blue','green','purple','orange','brown','magenta','cyan','gold','gray','lime']
    for i, g in enumerate(big):
        c, pts = colors[i % len(colors)], g['points']
        pa_txt = f"{g['pa']:.2f}°" if g['pa'] is not None else "—"
        ax.scatter(np.radians(pts['theta']), pts['r'], s=10, color=c,
                   label=f"G{i+1} ({len(pts)}) | PA={pa_txt}")

        if g['slope'] is not None:
            dt = g['theta_max'] - g['theta_min']
            t_deg = np.linspace(g['theta_min'] - line_extrap * dt, g['theta_max'] + line_extrap * dt, 200)
            r_line = g['slope'] * t_deg + g['intercept']
            ax.plot(np.radians(t_deg), r_line, '.-', color="gray", alpha=0.20)
            ax.plot(np.radians(t_deg), r_line, '--', color=c)

    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title("Subhalo 17 – proyección polar (PA visible)", y=1.05)
    ax.legend(loc='upper right', bbox_to_anchor=(1.20, 1.0))
    plt.show()


# ═══════════════════════════════════════════════════════════════════════
# 8) MAIN – pipeline completo subhalo 17 (modificado para almacenar grupos y background)
# ═══════════════════════════════════════════════════════════════════════
def main_example_modificado():
    # Genera las semillas BFS y carga los datos filtrados
    bfs, df_f = generate_bfs_seeds(
        id_halo="11",
        theta_min=0, theta_max=360,
        quartile_threshold=0.425,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=1.80, gap_threshold_r=1.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")

    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.30, bounding_extrap=0.35)
    print(f"Tras fusión: {len(merged)} grupos")

    validated = validate_dispersion_and_reprocess(merged)
    print(f"Tras validación: {len(validated)} grupos")

    # Visualizaciones (si se desea mantener)
    plot_groups_cartesian(validated, df_f, line_extrap=0.15)
    plot_groups_polar(validated, df_f, line_extrap=0.15)

    # Almacenar grupos finales: solo aquellos con 60 o más puntos
    grupos_finales = [g for g in validated if len(g['points']) >= 60]

    # Identificar los IDs de los puntos que pertenecen a los grupos finales.
    puntos_grupos = set()
    for g in grupos_finales:
        # Se espera que la columna "id" esté en los datos originales
        puntos_grupos.update(g['points']['id'].tolist())

    # Los puntos "background" son aquellos de df_f que no pertenecen a ningún grupo final.
    background = df_f[~df_f['id'].isin(puntos_grupos)].copy()
    print(f"Puntos background: {len(background)}")

    # Se puede devolver un diccionario con los dos conjuntos de información:
    return {'grupos_finales': grupos_finales, 'background': background}


# ═══════════════════════════════════════════════════════════════════════
# 9) Run
# ═══════════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    datos_finales = main_example_modificado()
    # Ahora, datos_finales['grupos_finales'] contiene los grupos con ≥ 60 pts, 
    # y datos_finales['background'] contiene la información original de los puntos
    # que no fueron asignados a ningún grupo final.

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Histogram Segmenter
===================
Módulo autocontenido y reutilizable para:
1. Detección de "islas" en un histograma 2D (θ, r).
2. Cálculo de esqueletos por bins angulares (θ).
3. Generación **siempre** de centroides adicionales sobre *todas* las rectas
   candidatas (pares de esqueletos contiguos e inter‑isla), muestreando la
   densidad de fondo.
4. Segmentación de contornos: las conexiones se parten si Δr > *dr_multiplier*·μ,
   donde μ es el promedio de Δr de los segmentos generados.
5. Clasificación de objetos resultantes:
   • ``isla‑conexion‑isla``  (contornos válidos)
   • ``isla‑conexiones‑adicionales``  (segmentos descartados > umbral)
6. Salida estructurada + visualización opcional.

El diseño busca **estabilidad** ante pequeños cambios en parámetros para poder
aplicarlo a múltiples datasets sin retocar el código.
"""
from __future__ import annotations

import math
import heapq
from dataclasses import dataclass
from typing import List, Tuple, Dict, Any

import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter, binary_closing, label

# ---------------------------------------------------------------------------
# Dataclasses de apoyo
# ---------------------------------------------------------------------------
@dataclass
class Connection:
    a: Tuple[float, float]
    b: Tuple[float, float]
    delta_r: float
    euclidean: float

@dataclass
class IslandObject:
    type: str  # "isla-conexion-isla" | "isla-conexiones-adicionales"
    boundary: List[Tuple[float, float]]

# ---------------------------------------------------------------------------
# Algoritmo principal
# ---------------------------------------------------------------------------
class HistogramSegmenter:
    """Segmentador configurable.

    Parameters
    ----------
    *bins_theta*, *bins_r* : resolución del histograma (θ, r).
    *smooth_sigma*        : σ del filtro gaussiano.
    *density_percentile*  : umbral de densidad para binarización.
    *closing_size*        : tamaño del cierre morfológico.
    *min_cluster_size*    : puntos mínimos por isla.
    *max_gap_dist*        : distancia máx. para unir islas en BFS.
    *theta_bin_size*      : paso angular para esqueletos.
    *density_ratio*       : radio *relative* para muestreo de densidad.
    *theta_step*          : Δθ entre muestras sobre la recta.
    *r_threshold*         : tolerancia radial al validar centroides.
    *dr_multiplier*       : factor de corte (p.e. 1.3 o 3.1).
    """

    def __init__(self,
                 bins_theta: int = 100,
                 bins_r: int = 80,
                 smooth_sigma: float = 1.0,
                 density_percentile: float = 80,
                 closing_size: int = 2,
                 min_cluster_size: int = 40,
                 max_gap_dist: float = 0.45,
                 theta_bin_size: float = 8,
                 density_ratio: float = 0.25,
                 theta_step: float = 3,
                 r_threshold: float = 0.9,
                 dr_multiplier: float = 3.0):
        self.bins_theta = bins_theta
        self.bins_r = bins_r
        self.smooth_sigma = smooth_sigma
        self.density_percentile = density_percentile
        self.closing_size = closing_size
        self.min_cluster_size = min_cluster_size
        self.max_gap_dist = max_gap_dist
        self.theta_bin_size = theta_bin_size
        self.density_ratio = density_ratio
        self.theta_step = theta_step
        self.r_threshold = r_threshold
        self.dr_multiplier = dr_multiplier

    # ------------------------------------------------------------------
    # Paso 1 – Histograma + suavizado
    # ------------------------------------------------------------------
    def _histogram(self, pts: np.ndarray):
        hist, te, re = np.histogram2d(
            pts[:, 0], pts[:, 1],
            bins=[self.bins_theta, self.bins_r]
        )
        hist_s = gaussian_filter(hist, sigma=self.smooth_sigma)
        return hist_s, te, re

    # ------------------------------------------------------------------
    # Paso 2 – BFS ponderado para unir islas próximas
    # ------------------------------------------------------------------
    def _weighted_bfs(self, hist_s, te, re):
        thr = np.percentile(hist_s[hist_s > 0], self.density_percentile)
        mask = binary_closing(hist_s > thr,
                              structure=np.ones((self.closing_size,
                                                 self.closing_size)))
        labels, n_lab = label(mask)

        theta_res, r_res = te[1] - te[0], re[1] - re[0]
        neigh = [(-1, 0), (1, 0), (0, -1), (0, 1),
                 (-1, -1), (-1, 1), (1, -1), (1, 1)]

        parent = list(range(n_lab + 1))
        def find(x):
            while parent[x] != x:
                parent[x] = parent[parent[x]]
                x = parent[x]
            return x
        def union(a, b):
            ra, rb = find(a), find(b)
            if ra != rb:
                parent[rb] = ra

        h, w = labels.shape
        dist_grid = np.full((h, w), np.inf)
        pq: List[Tuple[float, int, int, int]] = []

        for i in range(h):
            for j in range(w):
                lab = labels[i, j]
                if lab:
                    for di, dj in neigh:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and labels[ni, nj] == 0:
                            d0 = math.hypot(di * theta_res, dj * r_res)
                            if d0 <= self.max_gap_dist and d0 < dist_grid[ni, nj]:
                                dist_grid[ni, nj] = d0
                                heapq.heappush(pq, (d0, ni, nj, lab))

        while pq:
            dist, i, j, lab = heapq.heappop(pq)
            if dist > dist_grid[i, j] or dist > self.max_gap_dist:
                continue
            for di, dj in neigh:
                ni, nj = i + di, j + dj
                if 0 <= ni < h and 0 <= nj < w:
                    lab2 = labels[ni, nj]
                    if lab2 and lab2 != lab:
                        union(lab, lab2)
            for di, dj in neigh:
                ni, nj = i + di, j + dj
                if 0 <= ni < h and 0 <= nj < w and labels[ni, nj] == 0:
                    nd = dist + math.hypot(di * theta_res, dj * r_res)
                    if nd <= self.max_gap_dist and nd < dist_grid[ni, nj]:
                        dist_grid[ni, nj] = nd
                        heapq.heappush(pq, (nd, ni, nj, lab))
        return find, labels

    # ------------------------------------------------------------------
    # Paso 3 – Clusters y esqueletos
    # ------------------------------------------------------------------
    def _clusters_and_skeletons(self, pts, find, labels, te, re):
        tidx = np.clip(np.digitize(pts[:, 0], te) - 1, 0, self.bins_theta - 1)
        ridx = np.clip(np.digitize(pts[:, 1], re) - 1, 0, self.bins_r - 1)
        merged: Dict[int, List[np.ndarray]] = {}
        for p, (ti, ri) in enumerate(zip(tidx, ridx)):
            lab = labels[ti, ri]
            if lab:
                merged.setdefault(find(lab), []).append(pts[p])
        clusters = [np.vstack(v) for v in merged.values()
                    if len(v) >= self.min_cluster_size]

        skeletons: List[List[Tuple[float, float]]] = []
        for cl in clusters:
            thetas, rs = cl[:, 0], cl[:, 1]
            bins_arr = np.arange(thetas.min(),
                                 thetas.max() + self.theta_bin_size,
                                 self.theta_bin_size)
            ske = []
            for k in range(len(bins_arr) - 1):
                m = (thetas >= bins_arr[k]) & (thetas < bins_arr[k + 1])
                if m.any():
                    ske.append((thetas[m].mean(), rs[m].mean()))
            skeletons.append(sorted(ske, key=lambda x: -x[0]))
        return clusters, skeletons

    # ------------------------------------------------------------------
    # Paso 4 – Centroides adicionales sobre *todas* las rectas candidatas
    # ------------------------------------------------------------------
    def _density_path(self, a, b, hist_s, te, re):
        """Genera centroides extra a lo largo del segmento a‑b."""
        theta_res, r_res = te[1] - te[0], re[1] - re[0]
        steps = max(int(abs(b[0] - a[0]) / self.theta_step) + 1, 2)
        thetas = np.linspace(a[0], b[0], steps)
        rs_seg = np.linspace(a[1], b[1], steps)
        radius = math.hypot(b[0] - a[0], b[1] - a[1]) * self.density_ratio

        out: List[Tuple[float, float]] = []
        for θ_i, r_i in zip(thetas, rs_seg):
            ti = int(np.clip(np.digitize(θ_i, te) - 1, 0, hist_s.shape[0] - 1))
            ri0 = int(np.clip(np.digitize(r_i, re) - 1, 0, hist_s.shape[1] - 1))
            max_off = int(math.ceil(radius / r_res))
            vals, offs = [], []
            for off in range(-max_off, max_off + 1):
                idx = ri0 + off
                if 0 <= idx < hist_s.shape[1]:
                    vals.append(hist_s[ti, idx]); offs.append(off)
            for k in range(1, len(vals) - 1):
                if vals[k] > vals[k - 1] and vals[k] > vals[k + 1] and vals[k] > 0:
                    rc = re[ri0 + offs[k]] + r_res / 2.0
                    if abs(rc - r_i) <= self.r_threshold:
                        out.append((θ_i, rc))
        return out

    def _candidate_segments(self, skeletons):
        segs = []
        # 4a) internos (pares contiguos)
        for ske in skeletons:
            segs.extend(list(zip(ske, ske[1:])))
        # 4b) mejores conexiones inter‑isla (criterio dr>0, dθ<0, distancia mínima)
        for i, ske in enumerate(skeletons):
            if not ske:
                continue
            a = ske[-1]
            best, bd = None, math.inf
            for j, ske2 in enumerate(skeletons):
                if i == j or not ske2:
                    continue
                b = ske2[0]
                dr, dtheta = b[1] - a[1], b[0] - a[0]
                if dr > 0 and dtheta < 0:
                    d = math.hypot(dr, dtheta)
                    if d < bd:
                        bd, best = d, (a, b)
            if best:
                segs.append(best)
        return segs

    # ------------------------------------------------------------------
    # Paso 5 – Construir rutas (contours) y filtrar por Δr
    # ------------------------------------------------------------------
    def _build_contours(self, segs, hist_s, te, re):
        contours: List[List[Tuple[float, float]]] = []
        connections: List[Connection] = []
        for a, b in segs:
            path = self._density_path(a, b, hist_s, te, re)
            route = [a] + path + [b]
            contours.append(route)
            # registrar Δr entre a y b (para estadística global)
            connections.append(Connection(a, b, abs(b[1] - a[1]),
                                           math.hypot(b[0] - a[0], b[1] - a[1])))
        return contours, connections

    @staticmethod
    def _split_by_threshold(contours, thr):
        """Parte las rutas cuando Δr > thr."""
        out = []
        for cnt in contours:
            tmp = [cnt[0]]
            for a, b in zip(cnt, cnt[1:]):
                if abs(b[1] - a[1]) <= thr:
                    tmp.append(b)
                else:
                    if len(tmp) > 1:
                        out.append(tmp)
                    tmp = [b]
            if len(tmp) > 1:
                out.append(tmp)
        return out

    # ------------------------------------------------------------------
    # Método público -----------------------------------------------------------------
    # ------------------------------------------------------------------
    def run(self, datos: Dict[str, Any]):
        # Entrada esperada
        bg = datos['background'][['theta', 'r']].values
        grp = np.vstack([g['points'][['theta', 'r']].values
                          for g in datos['grupos_finales']])
        pts = np.vstack([bg, grp])

        # 1) Histograma
        hist_s, te, re = self._histogram(pts)

        # 2) BFS + 3) clusters + esqueletos
        find, labels = self._weighted_bfs(hist_s, te, re)
        clusters, skeletons = self._clusters_and_skeletons(pts, find, labels, te, re)

        # 4) segmentos candidatos + rutas con centroides extra
        segs = self._candidate_segments(skeletons)
        raw_contours, conns = self._build_contours(segs, hist_s, te, re)

        # 5) filtrado por Δr
        mean_dr = float(np.mean([c.delta_r for c in conns])) if conns else 0.0
        thr = mean_dr * self.dr_multiplier
        final_contours = self._split_by_threshold(raw_contours, thr)

        # Objetos
        objects = [IslandObject('isla-conexion-isla', cnt) for cnt in final_contours]
        for c in conns:
            if c.delta_r > thr:
                objects.append(IslandObject('isla-conexiones-adicionales', [c.a, c.b]))

        return {
            'hist_s': hist_s, 'te': te, 're': re,
            'clusters': clusters, 'skeletons': skeletons,
            'contours': final_contours, 'connections': conns,
            'mean_dr': mean_dr, 'threshold': thr, 'objects': objects
        }

    # ------------------------------------------------------------------
    # Utilidad rápida para visualizar
    # ------------------------------------------------------------------
    def quick_plot(self, result, figsize=(14, 6)):
        plt.figure(figsize=figsize)
        plt.imshow(result['hist_s'].T, origin='lower',
                   extent=[result['te'][0], result['te'][-1],
                           result['re'][0], result['re'][-1]],
                   aspect='auto', cmap='inferno')
        plt.colorbar(label='Densidad suavizada')
        for idx, cl in enumerate(result['clusters']):
            plt.scatter(cl[:, 0], cl[:, 1], s=8, alpha=0.6,
                        label=f'Isla {idx}')
        for ske in result['skeletons']:
            arr = np.array(ske)
            if arr.size:
                plt.scatter(arr[:, 0], arr[:, 1], c='w', s=40,
                            edgecolors='k')
        for cnt in result['contours']:
            xs, ys = zip(*cnt)
            plt.plot(xs, ys, '-o', color='cyan', markersize=5)


# ------------------------------------------------------------
# 1.  Importa el módulo y los datos de ejemplo
# ------------------------------------------------------------
#from histogram_segmenter import HistogramSegmenter   #  <-- el archivo del canvas
#from your_data_module import main_example_modificado #  <-- tu función de carga

# ------------------------------------------------------------
# 2.  Carga datos y crea la instancia con los parámetros deseados
# ------------------------------------------------------------
datos = main_example_modificado()          # debe devolver un dict con background y grupos_finales
segmenter = HistogramSegmenter(            # puedes ajustar cualquier parámetro aquí
    bins_theta=120,
    bins_r=80,
    smooth_sigma=1.0,
    theta_bin_size=8,
    dr_multiplier=2.5                      # 3.0 × μ para partir conexiones
)

# ------------------------------------------------------------
# 3.  Ejecuta el algoritmo
# ------------------------------------------------------------
result = segmenter.run(datos)

# ------------------------------------------------------------
# 4.  Consulta la salida estructurada
# ------------------------------------------------------------
print("Islas encontradas:", len(result['clusters']))
print("Promedio Δr:", result['mean_dr'])
print("Umbral usado:", result['threshold'])
print("Objetos clasificados:", len(result['objects']))

# ------------------------------------------------------------
# 5.  Visualiza (opcional)
# ------------------------------------------------------------
segmenter.quick_plot(result)


In [0]:
import matplotlib.patheffects as PathEffects
def plot_islands_and_paths_polar(result, datos, id_halo="11", line_extrap=0.5):
    """
    Función para crear un gráfico polar destacando las islas encontradas, los caminos de los centroides,
    los contornos y añadiendo los puntos de fondo con un color gris muy tenue.
    """

    # Filtrar islas grandes
    big_islands = [cl for cl in result['clusters'] if len(cl) >= 60]
    print(f"Islas ≥60 puntos (polar): {len(big_islands)}")

    fig = plt.figure(figsize=(9, 8))
    ax = fig.add_subplot(111, projection='polar')

    # Añadir puntos de fondo con gris muy tenue
    bg = datos['background'][['theta', 'r']].values
    ax.scatter(np.radians(bg[:, 0]), bg[:, 1], s=3, color='gray', edgecolor='black' ,alpha=0.21, label='Puntos de fondo')

    # Colores para las islas
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'magenta', 'cyan', 'gold', 'gray', 'lime']

    # Resaltar islas y caminos
    for idx, cl in enumerate(big_islands):
        c = colors[idx % len(colors)]
        theta = np.radians(cl[:, 0])  # Convertir theta de grados a radianes
        r = cl[:, 1]
        ax.scatter(theta, r, s=10, color=c, alpha=0.34, label=f'Isla {idx+1} ({len(cl)})')

        # Resaltar el camino de los centroides (esqueletos) con borde
        for ske in result['skeletons']:
            arr = np.array(ske)
            if arr.size:
                theta_ske = np.radians(arr[:, 0])  # Convertir theta a radianes
                r_ske = arr[:, 1]
                line = ax.plot(theta_ske, r_ske, color='yellow', alpha=0.95, linewidth=2, label='Camino centroides')

                # Añadir borde a la línea
                for l in line:
                    l.set_path_effects([PathEffects.withStroke(linewidth=3, alpha=0.51,foreground='black')])

        #Resaltar las conexiones entre los centroides
        for cnt in result['contours']:
            theta_cnt, r_cnt = zip(*cnt)
            ax.plot(np.radians(theta_cnt), r_cnt, 'o', color='cyan',markersize=2.5, alpha=0.85, label='Contorno')
            ax.plot(np.radians(theta_cnt), r_cnt, '-o', color='cyan',markersize=1.0, alpha=0.65, label='Contorno')

        # Extrapolación de la línea (si es necesario)
        if 'slope' in cl and cl['slope'] is not None:  # Usamos 'cl' aquí en lugar de 'g'
            dt = cl['theta_max'] - cl['theta_min']
            t_deg = np.linspace(cl['theta_min'] - line_extrap * dt, cl['theta_max'] + line_extrap * dt, 200)
            r_line = cl['slope'] * t_deg + cl['intercept']
            ax.plot(np.radians(t_deg), r_line, '.-', color="gray", alpha=0.20)
            ax.plot(np.radians(t_deg), r_line, '--', color=c)

    # Configuración de los ejes polares
    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_title(f"Halo{id_halo} - Caminos de Centroides", y=1.05)
    ax.grid(True, alpha=0.1, color='gray')
    #ax.legend(loc='upper right', bbox_to_anchor=(1.20, 1.0))

    # Mostrar gráfico
    plt.show()

# ------------------------------------------------------------
# Llamada a la nueva celda después de ejecutar el algoritmo
# ------------------------------------------------------------
plot_islands_and_paths_polar(result, datos, id_halo="11")


# halo 418336....


In [0]:

# ═══════════════════════════════════════════════════════════════════════
# 8) MAIN – pipeline completo subhalo 17 (modificado para almacenar grupos y background)
# ═══════════════════════════════════════════════════════════════════════
def main_example_modificado():
    # Genera las semillas BFS y carga los datos filtrados
    bfs, df_f = generate_bfs_seeds(
        id_halo="418336",
        theta_min=0, theta_max=360,
        quartile_threshold=0.525,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=1.80, gap_threshold_r=1.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")

    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.30, bounding_extrap=0.35)
    print(f"Tras fusión: {len(merged)} grupos")

    validated = validate_dispersion_and_reprocess(merged)
    print(f"Tras validación: {len(validated)} grupos")

    # Visualizaciones (si se desea mantener)
    plot_groups_cartesian(validated, df_f, line_extrap=0.15)
    plot_groups_polar(validated, df_f, line_extrap=0.15)

    # Almacenar grupos finales: solo aquellos con 60 o más puntos
    grupos_finales = [g for g in validated if len(g['points']) >= 60]

    # Identificar los IDs de los puntos que pertenecen a los grupos finales.
    puntos_grupos = set()
    for g in grupos_finales:
        # Se espera que la columna "id" esté en los datos originales
        puntos_grupos.update(g['points']['id'].tolist())

    # Los puntos "background" son aquellos de df_f que no pertenecen a ningún grupo final.
    background = df_f[~df_f['id'].isin(puntos_grupos)].copy()
    print(f"Puntos background: {len(background)}")

    # Se puede devolver un diccionario con los dos conjuntos de información:
    return {'grupos_finales': grupos_finales, 'background': background}


# ═══════════════════════════════════════════════════════════════════════
# 9) Run
# ═══════════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    datos_finales = main_example_modificado()
    # Ahora, datos_finales['grupos_finales'] contiene los grupos con ≥ 60 pts, 
    # y datos_finales['background'] contiene la información original de los puntos
    # que no fueron asignados a ningún grupo final.

In [0]:

# ------------------------------------------------------------
# 2.  Carga datos y crea la instancia con los parámetros deseados
# ------------------------------------------------------------
datos = main_example_modificado()          # debe devolver un dict con background y grupos_finales
segmenter = HistogramSegmenter(            # puedes ajustar cualquier parámetro aquí
    bins_theta=120,
    bins_r=80,
    smooth_sigma=1.0,
    theta_bin_size=8,
    dr_multiplier=2.5                      # 3.0 × μ para partir conexiones
)

# ------------------------------------------------------------
# 3.  Ejecuta el algoritmo
# ------------------------------------------------------------
result = segmenter.run(datos)

# ------------------------------------------------------------
# 4.  Consulta la salida estructurada
# ------------------------------------------------------------
print("Islas encontradas:", len(result['clusters']))
print("Promedio Δr:", result['mean_dr'])
print("Umbral usado:", result['threshold'])
print("Objetos clasificados:", len(result['objects']))

# ------------------------------------------------------------
# 5.  Visualiza (opcional)
# ------------------------------------------------------------
segmenter.quick_plot(result)

In [0]:

# ------------------------------------------------------------
# Llamada a la nueva celda después de ejecutar el algoritmo
# ------------------------------------------------------------
plot_islands_and_paths_polar(result, datos, id_halo="418336")

### halo 198185


In [0]:

# ═══════════════════════════════════════════════════════════════════════
# 8) MAIN – pipeline completo subhalo 17 (modificado para almacenar grupos y background)
# ═══════════════════════════════════════════════════════════════════════
def main_example_modificado():
    # Genera las semillas BFS y carga los datos filtrados
    bfs, df_f = generate_bfs_seeds(
        id_halo="198185",
        theta_min=0, theta_max=360,
        quartile_threshold=0.625,
        theta_diff=3.0, r_diff=0.5,
        gap_threshold_theta=1.80, gap_threshold_r=1.0,
        file_prefix='data_rho'
    )
    print(f"BFS inicial: {len(bfs)} clusters")

    merged = adjust_and_merge_seeds(bfs, slope_variation_threshold=0.30, bounding_extrap=0.35)
    print(f"Tras fusión: {len(merged)} grupos")

    validated = validate_dispersion_and_reprocess(merged)
    print(f"Tras validación: {len(validated)} grupos")

    # Visualizaciones (si se desea mantener)
    plot_groups_cartesian(validated, df_f, line_extrap=0.15)
    plot_groups_polar(validated, df_f, line_extrap=0.15)

    # Almacenar grupos finales: solo aquellos con 60 o más puntos
    grupos_finales = [g for g in validated if len(g['points']) >= 60]

    # Identificar los IDs de los puntos que pertenecen a los grupos finales.
    puntos_grupos = set()
    for g in grupos_finales:
        # Se espera que la columna "id" esté en los datos originales
        puntos_grupos.update(g['points']['id'].tolist())

    # Los puntos "background" son aquellos de df_f que no pertenecen a ningún grupo final.
    background = df_f[~df_f['id'].isin(puntos_grupos)].copy()
    print(f"Puntos background: {len(background)}")

    # Se puede devolver un diccionario con los dos conjuntos de información:
    return {'grupos_finales': grupos_finales, 'background': background}


# ═══════════════════════════════════════════════════════════════════════
# 9) Run
# ═══════════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    datos_finales = main_example_modificado()
    # Ahora, datos_finales['grupos_finales'] contiene los grupos con ≥ 60 pts, 
    # y datos_finales['background'] contiene la información original de los puntos
    # que no fueron asignados a ningún grupo final.

In [0]:

# ------------------------------------------------------------
# 2.  Carga datos y crea la instancia con los parámetros deseados
# ------------------------------------------------------------
datos = main_example_modificado()          # debe devolver un dict con background y grupos_finales
segmenter = HistogramSegmenter(            # puedes ajustar cualquier parámetro aquí
    bins_theta=120,
    bins_r=80,
    smooth_sigma=1.0,
    theta_bin_size=8,
    dr_multiplier=2.5                      # 3.0 × μ para partir conexiones
)

# ------------------------------------------------------------
# 3.  Ejecuta el algoritmo
# ------------------------------------------------------------
result = segmenter.run(datos)

# ------------------------------------------------------------
# 4.  Consulta la salida estructurada
# ------------------------------------------------------------
print("Islas encontradas:", len(result['clusters']))
print("Promedio Δr:", result['mean_dr'])
print("Umbral usado:", result['threshold'])
print("Objetos clasificados:", len(result['objects']))

# ------------------------------------------------------------
# 5.  Visualiza (opcional)
# ------------------------------------------------------------
segmenter.quick_plot(result)

In [0]:

# ------------------------------------------------------------
# Llamada a la nueva celda después de ejecutar el algoritmo
# ------------------------------------------------------------
plot_islands_and_paths_polar(result, datos, id_halo="198185")